# export this to other file formats

These files are symbolically linked such that their source is the
version-controlled directory, but they can be exported from the local directory
where the analysis happens

Due to this difference in directory structure, anyone else using these scripts
will have to make sure the locations of their data are specified correctly in
the set working directories section.

## convert to jupyter notebook

### using pandoc

this looks nice but it's just a big markdown block
#+BEGIN_SRC bash :session idk
pandoc 6.6.5_analysis.org -o myoactivity_analysis_pandoc.ipynb

### using orgparse

this is the only way that makes runnable python cells, though lots of formatting
doesn't work

In [ ]:
import orgparse
import nbformat as nbf
import re
import base64

def parse_results(results_block):
    """
    Parse the #+RESULTS: block to extract outputs.
    """
    outputs = []
    lines = results_block.strip().split("\n")

    for line in lines:
        if line.strip().startswith(": "):
            outputs.append(nbf.v4.new_output(output_type="stream", name="stdout", text=line[2:].strip() + "\n"))
        elif re.match(r"\[\[.*\.(png|jpg|jpeg|gif)\]\]", line.strip()):
            image_path = re.search(r"\[\[(.*\.(png|jpg|jpeg|gif))\]\]", line.strip()).group(1)
            try:
                with open(image_path, "rb") as f:
                    image_data = base64.b64encode(f.read()).decode("utf-8")
                outputs.append(nbf.v4.new_output(
                    output_type="display_data",
                    data={"image/png": image_data},
                    metadata={}
                ))
            except FileNotFoundError:
                print(f"Warning: Image file not found: {image_path}")
    return outputs

def convert_org_links_to_markdown(text):
    """
    Convert Org mode links to Markdown links.
    """
    text = re.sub(r"\[\[([^\]]+)\]\[([^\]]+)\]\]", r"[\2](\1)", text)
    text = re.sub(r"\[\[([^\]]+)\]\]", r"[\1](\1)", text)
    return text

def org_to_ipynb(org_file, ipynb_file):
    org = orgparse.load(org_file)
    nb = nbf.v4.new_notebook()
    cells = []

    for node in org[1:]:
        if ":noexport:" in node.tags or (node.heading and ":noexport:" in node.heading.lower()):
            continue

        if node.heading:
            heading_level = "#" * node.level
            cells.append(nbf.v4.new_markdown_cell(f"{heading_level} {node.heading}"))

        in_code_block = False
        in_results_block = False
        code_lines = []
        markdown_lines = []
        results_block = ""

        for line in node.body.split("\n"):
            if line.strip().startswith("#+BEGIN_SRC python"):
                in_code_block = True
                if markdown_lines:
                    markdown_text = convert_org_links_to_markdown("\n".join(markdown_lines)).strip()
                    if markdown_text:
                        cells.append(nbf.v4.new_markdown_cell(markdown_text))
                    markdown_lines = []
                continue
            elif line.strip().startswith("#+END_SRC"):
                in_code_block = False
                if code_lines:
                    code_cell = nbf.v4.new_code_cell("\n".join(code_lines))
                    cells.append(code_cell)
                    code_lines = []
                continue
            elif line.strip().startswith("#+RESULTS:"):
                in_results_block = True
                continue
            elif in_results_block and line.strip() == ":end:":
                in_results_block = False
                if results_block.strip():
                    outputs = parse_results(results_block)
                    if outputs and cells:
                        cells[-1].outputs = outputs
                results_block = ""
            elif in_code_block:
                code_lines.append(line)
            elif in_results_block:
                results_block += line + "\n"
            else:
                markdown_lines.append(convert_org_links_to_markdown(line))

        markdown_text = convert_org_links_to_markdown("\n".join(markdown_lines)).strip()
        if markdown_text:
            cells.append(nbf.v4.new_markdown_cell(markdown_text))

    nb.cells = cells
    with open(ipynb_file, "w") as f:
        nbf.write(nb, f)

# Convert your Org file
org_to_ipynb("6.6.5_analysis.org",
             "myoactivity_analysis_orgparse.ipynb")

5d01e125d7015a0a0eabb72883c6337b


#+BEGIN_SRC emacs-lisp :tangle no :session idk
(org-babel-tangle)

## export to org document

#+begin_src emacs-lisp :tangle no
(org-org-export-to-org)
#+end_src

## export to markdown document

#+begin_src emacs-lisp :tangle no
(org-md-export-to-markdown)
#+end_src

# figure generation index

| figure | panel | link                                 |
|--------+-------+--------------------------------------|
| S2     | B     | myosin step size                     |
| S2     | C     | myosin attachment duration vs. force |

# configuration

## set global variables

In [ ]:
#timestep = 5e-5
report = 'yes'
replace_movies = 'no'
save_figures = 'yes'
save_dataframes = 'yes'

## load and configure libraries

In [ ]:
import os
import sys
from tabulate import tabulate
import math
import numpy as np
import pandas as pd
import shutil
import subprocess
from subprocess import Popen
import datetime
import matplotlib.pyplot as plt  # plotting
import seaborn as sns  # nicer plotting
from decimal import Decimal
import matplotlib.colors as mplcolors
import matplotlib
from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm
from matplotlib.cm import ScalarMappable
from scipy.stats import binned_statistic_2d
from scipy import stats
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
import textwrap
from decimal import Decimal
from scipy.stats import binned_statistic_2d
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt  # plotting
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
from matplotlib.colors import SymLogNorm
plt.style.use('seaborn-v0_8-colorblind') # set plot style
plt.cool()                          # heatmap color scheme
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=mcolors.TABLEAU_COLORS)
%matplotlib inline

import seaborn as sns  # nicer plotting
sns.set_style('whitegrid')  # set plot style

SMALL_SIZE = 20
MEDIUM_SIZE = 24
BIGGER_SIZE = 30

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=[6,5]) # default figure width, height

now = datetime.datetime.now()
date = now.strftime('%Y%m%d')
pref = date

/home/maxferrin/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).


from pandas.core import (


<Figure size 640x480 with 0 Axes>


## set working directories

In [ ]:
#machine = 'peeks'
machine = 'ltpbukem'

if machine == 'ltpbukem':
    drive_dir = '/home/maxferrin/google_drive/'

if machine == 'bizon':
    drive_dir = '/media/bizon/DATA/MFerrin/insync/'

if machine == 'peeks':
    drive_dir = '/scratch/ferrin/unison_peeks/'

if machine == 'drumroom':
    drive_dir = '/Users/max/google_drive/'

if machine == 'mbp':
    drive_dir = '/Users/maxferrin/google_drive/'

if machine == 'sobarky':
    drive_dir = '/Users/dblab/google_drive/'


working_dir = os.path.join(drive_dir, 'grad_school/db_lab/code/analysis/20230508_6.6.5_myominimal/')
cytosim_dir = os.path.join(drive_dir, 'grad_school/db_lab/code/cytosim_dblab/', machine)

if machine == 'peeks':
    working_dir = '/run/media/ferrin/Volume/max/analysis/20230508_6.6.5_myominimal/'
    cytosim_dir = '/home/ferrin/cytosim/'

dataframes_dir = os.path.join(working_dir,'dataframes/')

os.chdir(working_dir)

if os.path.isdir('figures') == False:
    os.mkdir('figures')
if os.path.isdir('dataframes') == False:
    os.mkdir('dataframes')

## load custom functions

In [ ]:
# add parent folder to path
sys.path.insert(1, '../')
from cytosim_analysis import cytosim_analysis_functions as caf

# reload custom library
from importlib import reload
reload(sys.modules['cytosim_analysis'])

<module 'cytosim_analysis' from '/home/maxferrin/SynologyDrive/google_drive/grad_school/db_lab/code/analysis/20230508_6.6.5_myominimal/../cytosim_analysis/__init__.py'>


## for running in emacs org-mode

In [ ]:
import IPython
from tabulate import tabulate

class OrgFormatter(IPython.core.formatters.BaseFormatter):
    def __call__(self, obj):
        try:
            return tabulate(obj, headers='keys',
                            tablefmt='orgtbl', showindex='always')
        except:
            return None

ip = get_ipython()
ip.display_formatter.formatters['text/org'] = OrgFormatter()

# find directories that have outputs or config files

In [ ]:
   output_dirs = []
   config_dirs = []

   for d in next(os.walk('simulations/'))[1]:
       for f in os.listdir('simulations/'+d):
           if f.endswith('.cym'):
               config_dirs.append(d)
               break
       for sd in next(os.walk('simulations/'+d))[1]:
           if sd.endswith('output') or sd.startswith('run'):
               output_dirs.append(d)
               break


   print(output_dirs, config_dirs)

['20230504_6.6.5_myominimal_output'] ['20230504_6.6.5_myominimal']


# OLD report simulations

In [ ]:
 os.chdir(working_dir)

 solid_allruns_allparams = dict()
 properties_allruns_allparams = dict()
 configs_allruns_allparams = dict()
 single_hip1r_allruns_allparams = dict()
 single_membrane_myosin_allruns_allparams = dict()
 fiber_forces_allruns_allparams = dict()
 fiber_ends_allruns_allparams = dict()
 rundirs_allparams = dict()

 total_runs = 0

 # config = open(config_sweep, 'r')
 # config_lines = config.readlines()
 # config.close()

 for output_dir in output_dirs:
 #for output_dir in [output_dirs[0]]:
     config_dir = output_dir.replace('_output', '')
     if config_dir in config_dirs:
         config_exists = 'yes'
     else:
         config_exists = 'no'
     configs_allruns = dict()
     solid_allruns = dict()
     single_hip1r_allruns = dict()
     single_membrane_myosin_allruns = dict()
     # fiber_cluster_allruns = dict()
     properties_allruns = dict()
     fiber_forces_allruns = dict()
     fiber_ends_allruns = dict()
     rundirs = []
     subprocess.call(['bash', os.path.join(cytosim_dir,'../modProperties.sh'), os.path.join('simulations/',output_dir)])
     for rundir in os.listdir('simulations/' + output_dir):
     #for rundir in [os.listdir('simulations/' + output_dir)[0]]:
     #for rundir in ['output01_0000']:
         reportdir = 'simulations/'+output_dir+'/'+rundir
         run_num = rundir.split('_')[0][-4:]
         conf_file = 'simulations/'+config_dir+'/'+config_dir+run_num+'.cym'
         if rundir.startswith('output') or rundir.startswith('run') and 'properties.cmo' in os.listdir(reportdir):
             if config_exists == 'yes':
                 conf = open(conf_file, 'r')
                 configs_allruns[rundir] = conf.readlines()
                 conf.close()
             os.chdir(reportdir)
             if report == 'yes':
                 subprocess.call([os.path.join(cytosim_dir, 'bin/report'),
                                 'solid', 'output=solid.txt'])
                 subprocess.call([os.path.join(cytosim_dir, 'bin/report'),
                                 'single:hip1r', 'output=single_hip1r.txt'])
                 subprocess.call([os.path.join(cytosim_dir, 'bin/report'),
                             'single:membrane_myosin', 'output=single_membrane_myosin.txt'])
                 subprocess.call([os.path.join(cytosim_dir, 'bin/report'),
                                 'fiber:forces', 'output=fiber_forces.txt'])
                 subprocess.call([os.path.join(cytosim_dir, 'bin/report'),
                             'fiber:ends', 'output=fiber_ends.txt'])
             solid = open('solid.txt', 'r')
             solid_allruns[rundir] = solid.readlines()
             solid.close()
             single_hip1r = open('single_hip1r.txt', 'r')
             single_hip1r_allruns[rundir] = single_hip1r.readlines()
             single_hip1r.close()
             single_membrane_myosin = open('single_membrane_myosin.txt', 'r')
             single_membrane_myosin_allruns[rundir] = single_membrane_myosin.readlines()
             single_membrane_myosin.close()
             # subprocess.call([cytosim_dir+'bin/report',
             #                 'fiber:cluster', 'output=fiber_cluster.txt'])
             # fiber_cluster = open('fiber_cluster.txt', 'r')
             # fiber_cluster_allruns[rundir] = fiber_cluster.readlines()
             # fiber_cluster.close()
             fiber_forces = open('fiber_forces.txt', 'r')
             fiber_forces_allruns[rundir] = fiber_forces.readlines()
             fiber_forces.close()
             fiber_ends = open('fiber_ends.txt', 'r')
             fiber_ends_allruns[rundir] = fiber_ends.readlines()
             fiber_ends.close()
             properties = open('properties.cmo', 'r')
             properties_allruns[rundir] = properties.readlines()
             properties.close()
             rundirs.append(rundir)
             total_runs += 1
             print ('finished reporting ' + output_dir + ' ' + rundir)
             if replace_movies == 'yes':
                 if 'movie.mp4' in os.listdir('.'):
                     os.remove('movie.mp4')
                 #%run -i cytosim_dir+'make_movie.py' cytosim_dir+'bin/play'
                 subprocess.call(['python', os.path.join(cytosim_dir,'../make_movie.py'),
                                  os.path.join(cytosim_dir,'bin/play')])
                 print ('finished making video of ' + output_dir + ' ' + rundir)
             os.chdir(working_dir)
     rundirs_allparams[output_dir] = rundirs
     solid_allruns_allparams[output_dir] = solid_allruns
     single_hip1r_allruns_allparams[output_dir] = single_hip1r_allruns
     single_membrane_myosin_allruns_allparams[output_dir] = single_membrane_myosin_allruns
     fiber_forces_allruns_allparams[output_dir] = fiber_forces_allruns
     fiber_ends_allruns_allparams[output_dir] = fiber_ends_allruns
     properties_allruns_allparams[output_dir] = properties_allruns
     configs_allruns_allparams[output_dir] = configs_allruns


finished reporting 20230504_6.6.5_myominimal_output run0000_0000


modifying display params


run0000_0000/


replacing display text


# report simulations

In [ ]:
caf.report_sims(working_dir, output_dirs, cytosim_dir, report)

# read in reports

- not enough memory to do all at once!
- just fiber forces alone requires too much memory

In [ ]:
reports = [
    'solid',
    # 'single_hip1r',
    'single_membrane_myosin'#,
    # 'fiber_cluster',
    # 'fiber_forces',
    # 'fiber_tensions',
    # 'fiber_ends'
    ]

#reports = ['solid']
#reports = ['fiber_ends']

solid_allruns_allparams, properties_allruns_allparams, \
configs_allruns_allparams, single_hip1r_allruns_allparams, \
single_membrane_myosin_allruns_allparams, fiber_forces_allruns_allparams, \
fiber_clusters_allruns_allparams, fiber_tensions_allruns_allparams, \
fiber_ends_allruns_allparams, rundirs_allparams, total_runs = \
caf.open_reports(reports, working_dir, output_dirs, config_dirs,
cytosim_dir, replace_movies)

if save_dataframes == 'yes':
    pd.DataFrame.from_dict(rundirs_allparams, orient = 'index').to_pickle(dataframes_dir+'rundirs_allparams.pkl')

finished reporting 20230504_6.6.5_myominimal_output


# read simulation properties

## put all properties and configs into dataframes

In [ ]:
 properties_list = []
 config_list = []

 for output_dir in output_dirs:
     properties_dict_allruns = dict()
     config_dict_allruns = dict()
     # for line in config_lines:
     #     if 'x =' in line:
     #         membrane_myosins = line.strip().strip('[[ x = [').strip('] ]]').split(', ')
     #     if 'y =' in line:
     #         myosins_radius = line.strip().strip('[[ y = [').strip('] ]]').split(', ')

     for rundir in rundirs_allparams[output_dir]:
         properties = properties_allruns_allparams[output_dir][rundir]
         conf = configs_allruns_allparams[output_dir][rundir]
         properties_dict = dict()
         config_dict = dict()
         # added this because not all properites have these parameters set, which
         # messes up the dataframe later
         properties_dict['internalize.cym_viscosity'] = 1
         properties_dict['blobneck_viscosity'] = 1
         properties_dict['bud_viscosity'] = 1
         config_dict['bud_viscosity'] = 1
         config_dict['blobneck_viscosity'] = 1
         for line in properties:
             if 'set' in line:
                 obj = line.strip().split(' ')[-1]
             if '=' in line:
                 line = line.strip().split(' = ')
                 param = line[0].strip()
                 val = line[-1].strip(';')
                 if val.isnumeric() == True:
                     val = float(val)
                 properties_dict[obj+'_'+param] = val

         for line in conf:
             if line.strip().startswith('%'):
                 continue
             if '%' in line:
                 line = line.split('%')[0]
             if 'set' in line:
                 obj = line.strip().split(' ')[-1]
             if '=' in line:
                 line = line.strip().split(' = ')
                 param = line[0].strip()
                 val = line[-1].strip(';')
                 if val.isnumeric() == True:
                     val = float(val)
                 config_dict[obj+'_'+param] = val
             if 'new' in line:
                 line = line.strip().split(' ')
                 obj = line[-1]
                 val = line[1]
                 # if type(val) != int:
                 #     val = 1
                 config_dict[obj+'_number'] = val

         # properties_dict['unbinding_rate'] = properties_dict['unbinding'].split(' ')[0].strip(',')
         # properties_dict['unbinding_force'] = properties_dict['unbinding'].split(' ')[-1]
         config_dict['actin_catastrophe_rate'] = float(config_dict['actin_catastrophe_rate'].split(',')[0])
         properties_dict_allruns[rundir] = properties_dict
         config_dict_allruns[rundir] = config_dict
         # print 'finished reading ' + output_dir + ' ' + rundir + ' properties'

     properties_df = pd.DataFrame.from_dict(properties_dict_allruns, orient = 'index')
     properties_list.append(properties_df)
     config_df = pd.DataFrame.from_dict(config_dict_allruns, orient = 'index')
     config_list.append(config_df)

 properties_allparams = pd.concat(properties_list, keys = output_dirs,
                             names = ['param_sweep', 'run'])

 config_allparams = pd.concat(config_list, keys = output_dirs,
                              names = ['param_sweep', 'run'])


#+RESULTS[68e0af332b21a585c2fbff13634cc149ff3e54fd]:
 :results:
 :end:

## filter for properties that vary among simulations

In [ ]:
   cols = list(properties_allparams)
   nunique = properties_allparams.apply(pd.Series.nunique)
   cols_to_drop = nunique[nunique == 1].index
   properties_unique = properties_allparams.drop(cols_to_drop, axis=1)
   properties_unique = properties_unique.drop(labels='internalize.cym_random_seed',axis=1)
   properties_unique.head()

#+RESULTS[15c9038d48ede74074eac256ede50861d5ef9b40]:
 :results:
 # [goto error]
 #+begin_example
 [0;31m---------------------------------------------------------------------------[0m
 [0;31mKeyError[0m                                  Traceback (most recent call last)
 Cell [0;32mIn[889], line 5[0m
 [1;32m      3[0m cols_to_drop [38;5;241m=[39m nunique[nunique [38;5;241m==[39m [38;5;241m1[39m][38;5;241m.[39mindex
 [1;32m      4[0m properties_unique [38;5;241m=[39m properties_allparams[38;5;241m.[39mdrop(cols_to_drop, axis[38;5;241m=[39m[38;5;241m1[39m)
 [0;32m----> 5[0m properties_unique [38;5;241m=[39m [43mproperties_unique[49m[38;5;241;43m.[39;49m[43mdrop[49m[43m([49m[43mlabels[49m[38;5;241;43m=[39;49m[38;5;124;43m'[39;49m[38;5;124;43minternalize.cym_random_seed[39;49m[38;5;124;43m'[39;49m[43m,[49m[43maxis[49m[38;5;241;43m=[39;49m[38;5;241;43m1[39;49m[43m)[49m
 [1;32m      6[0m properties_unique[38;5;241m.[39mhead()

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/util/_decorators.py:331[0m, in [0;36mdeprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper[0;34m(*args, **kwargs)[0m
 [1;32m    325[0m [38;5;28;01mif[39;00m [38;5;28mlen[39m(args) [38;5;241m>[39m num_allow_args:
 [1;32m    326[0m     warnings[38;5;241m.[39mwarn(
 [1;32m    327[0m         msg[38;5;241m.[39mformat(arguments[38;5;241m=[39m_format_argument_list(allow_args)),
 [1;32m    328[0m         [38;5;167;01mFutureWarning[39;00m,
 [1;32m    329[0m         stacklevel[38;5;241m=[39mfind_stack_level(),
 [1;32m    330[0m     )
 [0;32m--> 331[0m [38;5;28;01mreturn[39;00m [43mfunc[49m[43m([49m[38;5;241;43m*[39;49m[43margs[49m[43m,[49m[43m [49m[38;5;241;43m*[39;49m[38;5;241;43m*[39;49m[43mkwargs[49m[43m)[49m

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/core/frame.py:5399[0m, in [0;36mDataFrame.drop[0;34m(self, labels, axis, index, columns, level, inplace, errors)[0m
 [1;32m   5251[0m [38;5;129m@deprecate_nonkeyword_arguments[39m(version[38;5;241m=[39m[38;5;28;01mNone[39;00m, allowed_args[38;5;241m=[39m[[38;5;124m"[39m[38;5;124mself[39m[38;5;124m"[39m, [38;5;124m"[39m[38;5;124mlabels[39m[38;5;124m"[39m])
 [1;32m   5252[0m [38;5;28;01mdef[39;00m [38;5;21mdrop[39m(  [38;5;66;03m# type: ignore[override][39;00m
 [1;32m   5253[0m     [38;5;28mself[39m,
 [0;32m   (...)[0m
 [1;32m   5260[0m     errors: IgnoreRaise [38;5;241m=[39m [38;5;124m"[39m[38;5;124mraise[39m[38;5;124m"[39m,
 [1;32m   5261[0m ) [38;5;241m-[39m[38;5;241m>[39m DataFrame [38;5;241m|[39m [38;5;28;01mNone[39;00m:
 [1;32m   5262[0m [38;5;250m    [39m[38;5;124;03m"""[39;00m
 [1;32m   5263[0m [38;5;124;03m    Drop specified labels from rows or columns.[39;00m
 [1;32m   5264[0m
 [0;32m   (...)[0m
 [1;32m   5397[0m [38;5;124;03m            weight  1.0     0.8[39;00m
 [1;32m   5398[0m [38;5;124;03m    """[39;00m
 [0;32m-> 5399[0m     [38;5;28;01mreturn[39;00m [38;5;28;43msuper[39;49m[43m([49m[43m)[49m[38;5;241;43m.[39;49m[43mdrop[49m[43m([49m
 [1;32m   5400[0m [43m        [49m[43mlabels[49m[38;5;241;43m=[39;49m[43mlabels[49m[43m,[49m
 [1;32m   5401[0m [43m        [49m[43maxis[49m[38;5;241;43m=[39;49m[43maxis[49m[43m,[49m
 [1;32m   5402[0m [43m        [49m[43mindex[49m[38;5;241;43m=[39;49m[43mindex[49m[43m,[49m
 [1;32m   5403[0m [43m        [49m[43mcolumns[49m[38;5;241;43m=[39;49m[43mcolumns[49m[43m,[49m
 [1;32m   5404[0m [43m        [49m[43mlevel[49m[38;5;241;43m=[39;49m[43mlevel[49m[43m,[49m
 [1;32m   5405[0m [43m        [49m[43minplace[49m[38;5;241;43m=[39;49m[43minplace[49m[43m,[49m
 [1;32m   5406[0m [43m        [49m[43merrors[49m[38;5;241;43m=[39;49m[43merrors[49m[43m,[49m
 [1;32m   5407[0m [43m    [49m[43m)[49m

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/util/_decorators.py:331[0m, in [0;36mdeprecate_nonkeyword_arguments.<locals>.decorate.<locals>.wrapper[0;34m(*args, **kwargs)[0m
 [1;32m    325[0m [38;5;28;01mif[39;00m [38;5;28mlen[39m(args) [38;5;241m>[39m num_allow_args:
 [1;32m    326[0m     warnings[38;5;241m.[39mwarn(
 [1;32m    327[0m         msg[38;5;241m.[39mformat(arguments[38;5;241m=[39m_format_argument_list(allow_args)),
 [1;32m    328[0m         [38;5;167;01mFutureWarning[39;00m,
 [1;32m    329[0m         stacklevel[38;5;241m=[39mfind_stack_level(),
 [1;32m    330[0m     )
 [0;32m--> 331[0m [38;5;28;01mreturn[39;00m [43mfunc[49m[43m([49m[38;5;241;43m*[39;49m[43margs[49m[43m,[49m[43m [49m[38;5;241;43m*[39;49m[38;5;241;43m*[39;49m[43mkwargs[49m[43m)[49m

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/core/generic.py:4505[0m, in [0;36mNDFrame.drop[0;34m(self, labels, axis, index, columns, level, inplace, errors)[0m
 [1;32m   4503[0m [38;5;28;01mfor[39;00m axis, labels [38;5;129;01min[39;00m axes[38;5;241m.[39mitems():
 [1;32m   4504[0m     [38;5;28;01mif[39;00m labels [38;5;129;01mis[39;00m [38;5;129;01mnot[39;00m [38;5;28;01mNone[39;00m:
 [0;32m-> 4505[0m         obj [38;5;241m=[39m [43mobj[49m[38;5;241;43m.[39;49m[43m_drop_axis[49m[43m([49m[43mlabels[49m[43m,[49m[43m [49m[43maxis[49m[43m,[49m[43m [49m[43mlevel[49m[38;5;241;43m=[39;49m[43mlevel[49m[43m,[49m[43m [49m[43merrors[49m[38;5;241;43m=[39;49m[43merrors[49m[43m)[49m
 [1;32m   4507[0m [38;5;28;01mif[39;00m inplace:
 [1;32m   4508[0m     [38;5;28mself[39m[38;5;241m.[39m_update_inplace(obj)

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/core/generic.py:4546[0m, in [0;36mNDFrame._drop_axis[0;34m(self, labels, axis, level, errors, only_slice)[0m
 [1;32m   4544[0m         new_axis [38;5;241m=[39m axis[38;5;241m.[39mdrop(labels, level[38;5;241m=[39mlevel, errors[38;5;241m=[39merrors)
 [1;32m   4545[0m     [38;5;28;01melse[39;00m:
 [0;32m-> 4546[0m         new_axis [38;5;241m=[39m [43maxis[49m[38;5;241;43m.[39;49m[43mdrop[49m[43m([49m[43mlabels[49m[43m,[49m[43m [49m[43merrors[49m[38;5;241;43m=[39;49m[43merrors[49m[43m)[49m
 [1;32m   4547[0m     indexer [38;5;241m=[39m axis[38;5;241m.[39mget_indexer(new_axis)
 [1;32m   4549[0m [38;5;66;03m# Case for non-unique axis[39;00m
 [1;32m   4550[0m [38;5;28;01melse[39;00m:

 File [0;32m~/anaconda3/envs/max_cytosim_analysis/lib/python3.11/site-packages/pandas/core/indexes/base.py:6934[0m, in [0;36mIndex.drop[0;34m(self, labels, errors)[0m
 [1;32m   6932[0m [38;5;28;01mif[39;00m mask[38;5;241m.[39many():
 [1;32m   6933[0m     [38;5;28;01mif[39;00m errors [38;5;241m!=[39m [38;5;124m"[39m[38;5;124mignore[39m[38;5;124m"[39m:
 [0;32m-> 6934[0m         [38;5;28;01mraise[39;00m [38;5;167;01mKeyError[39;00m([38;5;124mf[39m[38;5;124m"[39m[38;5;132;01m{[39;00m[38;5;28mlist[39m(labels[mask])[38;5;132;01m}[39;00m[38;5;124m not found in axis[39m[38;5;124m"[39m)
 [1;32m   6935[0m     indexer [38;5;241m=[39m indexer[[38;5;241m~[39mmask]
 [1;32m   6936[0m [38;5;28;01mreturn[39;00m [38;5;28mself[39m[38;5;241m.[39mdelete(indexer)

 [0;31mKeyError[0m: "['internalize.cym_random_seed'] not found in axis"
 #+end_example
 :end:

In [ ]:
 cols = list(config_allparams)
 nunique = config_allparams.apply(pd.Series.nunique)
 cols_to_drop = nunique[nunique == 1].index
 config_unique = config_allparams.drop(cols_to_drop, axis=1)
 config_unique = config_unique.astype('float')
 config_unique.head()

#+RESULTS[1b422a68534578bf15fed4ee46a5f69db45e3025]:
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000') |
 :results:
 :end:

# load in previously parsed dataframes

#+begin_src python
#solid_allparams = pd.read_pickle(dataframes_dir+'solid_allparams.pkl')
membrane_myosin_allparams = pd.read_pickle(dataframes_dir+'membrane_myosin_allparams.pkl')
# means = pd.read_pickle(dataframes_dir+'means.pkl')
# stds = pd.read_pickle(dataframes_dir+'stds.pkl')
# sweep_means = pd.read_pickle(dataframes_dir+'sweep_means.pkl')
# sweep_stds = pd.read_pickle(dataframes_dir+'sweep_stds.pkl')
rundirs_allparams_df = pd.read_pickle(dataframes_dir+'rundirs_allparams.pkl')
rundirs_allparams_df.fillna(value='empty', inplace=True)
properties_allparams = pd.read_pickle(dataframes_dir+'properties_allparams.pkl')
properties_unique = pd.read_pickle(dataframes_dir+'properties_unique.pkl')
property_groups = list(properties_unique)
#+end_src


#+begin_src python
total_runs = len(solid_allparams.groupby(['param_sweep','run']).count())
solid_allparams.head()
#+end_src

# OLD parse results into dataframe

## all myosin

In [ ]:
 membrane_myosin_list = []

 for output_dir in output_dirs:
     rundirs = rundirs_allparams[output_dir]
     single_membrane_myosin_outputs_allruns = []
     for rundir in rundirs:
         single_all_lines = single_membrane_myosin_allruns_allparams[output_dir][rundir]
         timepoints = []
         outputs = []
         for line in single_all_lines:
             line = line.strip()
             if line.startswith('%'):
                 if line.startswith('% time'):
                     time = float(line.split(' ')[-1])
                     timepoints.append(time)
                     singles = {}
                 elif line.startswith('% end'):
                     df = pd.DataFrame.from_dict(singles, orient = 'index')
                     outputs.append(df)
                     # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
             elif len(line.split()) > 0:
                 [single_class, single_id, state, xpos, ypos, zpos, xforce, yforce, zforce,
                  fiber_id, xdirection, ydirection, zdirection, abscissa] = line.split()[:14]
                 singles[int(single_id)] = {'state' : int(state), 'fiber_id' : int(fiber_id),
                                         'xpos': float(xpos), 'single_id' : int(single_id),
                                         'ypos' : float(ypos), 'zpos': float(zpos),
                                         'xforce' : float(xforce), 'yforce' : float(yforce),
                                         'zforce': float(zforce), 'abscissa' : float(abscissa),
                                            'xdirection' : float(xdirection), 'ydirection' : float(ydirection),
                                            'zdirection' : float(zdirection)}

         all_outputs = pd.concat(outputs, keys = timepoints,
                             names = ['time', 'id'])
         # all_outputs = all_outputs.swaplevel('time','id',axis=0).sort_index()
         all_outputs['scalar_force'] = np.sqrt(np.square(all_outputs['xforce']) +
                                             np.square(all_outputs['yforce']) +
                                             np.square(all_outputs['zforce']))
         all_outputs['rpos'] = np.sqrt(np.square(all_outputs['xpos']) +
                                     np.square(all_outputs['ypos']))

         single_membrane_myosin_outputs_allruns.append(all_outputs)

         print('finished parsing ' + rundir)

     all_single_membrane_myosin = pd.concat(single_membrane_myosin_outputs_allruns, keys = rundirs,
                                 names = ['run', 'time', 'id'])

     # bound_membrane_myosin = all_single_membrane_myosin.loc[all_single_membrane_myosin['state'] == 1]
     membrane_myosin_list.append(all_single_membrane_myosin)

 membrane_myosin_allparams = pd.concat(membrane_myosin_list, keys = output_dirs,
                             names = ['param_sweep', 'run', 'time', 'id'])

 membrane_myosin_allparams.head()

#+RESULTS[4063cc63182ef6956ce44d57471fabd48ac08a61]:
 |                                                                 |   state |   fiber_id |        xpos |   single_id |       ypos |   zpos |   xforce |   yforce |   zforce |   abscissa |   xdirection |   ydirection |   zdirection |   scalar_force |       rpos |
 |-----------------------------------------------------------------+---------+------------+-------------+-------------+------------+--------+----------+----------+----------+------------+--------------+--------------+--------------+----------------+------------|
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 5e-05, 20) |       0 |          0 | -0.0661983  |          20 |  0.0768269 | -0.386 |        0 |        0 |        0 |          0 |            0 |            0 |            0 |              0 | 0.101413   |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 5e-05, 21) |       0 |          0 |  0.0491387  |          21 |  0.206657  | -0.386 |        0 |        0 |        0 |          0 |            0 |            0 |            0 |              0 | 0.212419   |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 5e-05, 22) |       0 |          0 | -0.0837687  |          22 | -0.222402  | -0.386 |        0 |        0 |        0 |          0 |            0 |            0 |            0 |              0 | 0.237655   |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 5e-05, 23) |       0 |          0 |  0.00646242 |          23 |  0.0019717 | -0.386 |        0 |        0 |        0 |          0 |            0 |            0 |            0 |              0 | 0.00675651 |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 5e-05, 24) |       0 |          0 | -0.0667466  |          24 | -0.254067  | -0.386 |        0 |        0 |        0 |          0 |            0 |            0 |            0 |              0 | 0.262688   |
 :results:
 : finished parsing run0000_0000
 :end:

## fiber forces

In [ ]:

   forces_list = []

   for output_dir in output_dirs:
       rundirs = rundirs_allparams[output_dir]
       fiber_forces_outputs_allruns = []
       for rundir in rundirs:
           single_all_lines = fiber_forces_allruns_allparams[output_dir][rundir]
           timepoints = []
           outputs = []
           for line in single_all_lines:
               line = line.strip()
               if line.startswith('%'):
                   if line.startswith('% time'):
                       time = float(line.split(' ')[-1])
                       timepoints.append(time)
                       singles = {}
                   elif line.startswith('% end'):
                       df = pd.DataFrame.from_dict(singles, orient = 'index')
                       outputs.append(df)
                       # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
               elif len(line.split()) > 0:
                   [fiber_id, fiber_point, xpos, ypos, zpos,
                   xforce, yforce, zforce, idk1] = line.split()
                   singles[str(fiber_id)+'_'+str(fiber_point)] = {'fiber_id' : int(fiber_id),
                                                               'fiber_point' : int(fiber_point),
                                           'xpos': float(xpos), 'ypos' : float(ypos), 'zpos': float(zpos),
                                           'xforce' : float(xforce), 'yforce' : float(yforce),
                                           'zforce': float(zforce)}

           all_outputs = pd.concat(outputs, keys = timepoints,
                               names = ['time', 'id'])
           # all_outputs = all_outputs.swaplevel('time','id',axis=0).sort_index()
           all_outputs['scalar_force'] = np.sqrt(np.square(all_outputs['xforce']) +
                                               np.square(all_outputs['yforce']) +
                                               np.square(all_outputs['zforce']))
           all_outputs['rpos'] = np.sqrt(np.square(all_outputs['xpos']) +
                                       np.square(all_outputs['ypos']))

           fiber_forces_outputs_allruns.append(all_outputs)

           print('finished parsing ' + rundir)

       all_fiber_forces = pd.concat(fiber_forces_outputs_allruns, keys = rundirs,
                                   names = ['run', 'time', 'id'])

       forces_list.append(all_fiber_forces)

   forces_allparams = pd.concat(forces_list, keys = output_dirs,
                                names = ['param_sweep', 'run', 'time', 'id'])
   forces_allparams.head()

#+RESULTS[65bfba7fc87791e1579c9f75a8ae935cae28d127]:
 :results:
 # [goto error]
 #+begin_example
 [0;31m---------------------------------------------------------------------------[0m
 [0;31mKeyError[0m                                  Traceback (most recent call last)
 Cell [0;32mIn[15], line 4[0m
 [1;32m      1[0m forces_list [38;5;241m=[39m []
 [1;32m      3[0m [38;5;28;01mfor[39;00m output_dir [38;5;129;01min[39;00m output_dirs:
 [0;32m----> 4[0m     rundirs [38;5;241m=[39m [43mrundirs_allparams[49m[43m[[49m[43moutput_dir[49m[43m][49m
 [1;32m      5[0m     fiber_forces_outputs_allruns [38;5;241m=[39m []
 [1;32m      6[0m     [38;5;28;01mfor[39;00m rundir [38;5;129;01min[39;00m rundirs:

 File [0;32m~/miniconda3/lib/python3.10/site-packages/pandas/core/frame.py:4102[0m, in [0;36mDataFrame.__getitem__[0;34m(self, key)[0m
 [1;32m   4100[0m [38;5;28;01mif[39;00m [38;5;28mself[39m[38;5;241m.[39mcolumns[38;5;241m.[39mnlevels [38;5;241m>[39m [38;5;241m1[39m:
 [1;32m   4101[0m     [38;5;28;01mreturn[39;00m [38;5;28mself[39m[38;5;241m.[39m_getitem_multilevel(key)
 [0;32m-> 4102[0m indexer [38;5;241m=[39m [38;5;28;43mself[39;49m[38;5;241;43m.[39;49m[43mcolumns[49m[38;5;241;43m.[39;49m[43mget_loc[49m[43m([49m[43mkey[49m[43m)[49m
 [1;32m   4103[0m [38;5;28;01mif[39;00m is_integer(indexer):
 [1;32m   4104[0m     indexer [38;5;241m=[39m [indexer]

 File [0;32m~/miniconda3/lib/python3.10/site-packages/pandas/core/indexes/range.py:417[0m, in [0;36mRangeIndex.get_loc[0;34m(self, key)[0m
 [1;32m    415[0m         [38;5;28;01mraise[39;00m [38;5;167;01mKeyError[39;00m(key) [38;5;28;01mfrom[39;00m [38;5;21;01merr[39;00m
 [1;32m    416[0m [38;5;28;01mif[39;00m [38;5;28misinstance[39m(key, Hashable):
 [0;32m--> 417[0m     [38;5;28;01mraise[39;00m [38;5;167;01mKeyError[39;00m(key)
 [1;32m    418[0m [38;5;28mself[39m[38;5;241m.[39m_check_indexing_error(key)
 [1;32m    419[0m [38;5;28;01mraise[39;00m [38;5;167;01mKeyError[39;00m(key)

 [0;31mKeyError[0m: '20230504_6.6.5_myominimal_output'
 #+end_example
 :end:
 :results:
 : finished parsing run0000_0000
 :end:

## fiber ends

In [ ]:
   ends_list = []
   for output_dir in output_dirs:
       rundirs = rundirs_allparams[output_dir]
       fiber_ends_outputs_allruns = []
       for rundir in rundirs:
           single_all_lines = fiber_ends_allruns_allparams[output_dir][rundir]
           timepoints = []
           outputs = []
           for line in single_all_lines:
               line = line.strip()
               if line.startswith('%'):
                   if line.startswith('% time'):
                       time = float(line.split(' ')[-1])
                       timepoints.append(time)
                       singles = {}
                   elif line.startswith('% end'):
                       df = pd.DataFrame.from_dict(singles, orient = 'index')
                       outputs.append(df)
                       print('finished parsing ' + rundir + ' timepoint ' + str(time))
               elif len(line.split()) > 0:
                   [fiber_class, fiber_id, length, minus_state, minus_xpos, minus_ypos, minus_zpos,
                   minus_xdir, minus_ydir, minus_zdir, plus_state, plus_xpos, plus_ypos,
                   plus_zpos, plus_xdir, plus_ydir, plus_zir] = line.split()
                   singles[int(fiber_id)] = {'fiber_id' : int(fiber_id), 'length':float(length),
                                           'minus_state':int(minus_state), 'minus_xpos':float(minus_xpos),
                                           'minus_ypos':float(minus_ypos), 'minus_zpos':float(minus_zpos),
                                           'minus_xdir':float(minus_xdir), 'minus_ydir':float(minus_ydir),
                                           'minus_zdir':float(minus_zdir), 'plus_state':int(plus_state),
                                           'plus_xpos':float(plus_xpos), 'plus_ypos':float(plus_ypos),
                                           'plus_zpos':float(plus_zpos), 'plus_xdir':float(plus_xdir),
                                           'plus_ydir':float(plus_ydir), 'plus_zir':float(plus_zir)}

           all_outputs = pd.concat(outputs, keys = timepoints,
                               names = ['time', 'id'])
           if len(all_outputs) > 0:
               all_outputs['plus_rpos'] = np.sqrt(np.square(all_outputs['plus_xpos']) +
                                           np.square(all_outputs['plus_ypos']))

           fiber_ends_outputs_allruns.append(all_outputs)

           print('finished parsing ' + rundir)

       all_fiber_ends = pd.concat(fiber_ends_outputs_allruns, keys = rundirs,
                                   names = ['run', 'time', 'id'])
       ends_list.append(all_fiber_ends)
   ends_allparams = pd.concat(ends_list, keys = output_dirs,
         names = ['param_sweep', 'run', 'time', 'id'])
   ends_allparams.head()



|                                                                  |   fiber_id |   length |   minus_state |   minus_xpos |   minus_ypos |   minus_zpos |   minus_xdir |   minus_ydir |   minus_zdir |   plus_state |   plus_xpos |   plus_ypos |   plus_zpos |   plus_xdir |   plus_ydir |   plus_zir |   plus_rpos |
 |------------------------------------------------------------------+------------+----------+---------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-------------+-------------+-------------+-------------+-------------+------------+-------------|
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 0.0001, 1)  |          1 |  0.005   |             0 |     -0.02887 |     -0.01636 |     -0.39931 |     -0.96281 |     -0.13293 |      0.2352  |            1 |    -0.03369 |    -0.01702 |    -0.39813 |    -0.96281 |    -0.13293 |    0.2352  |   0.0377452 |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 0.00015, 1) |          1 |  0.00502 |             0 |     -0.02849 |     -0.01733 |     -0.39829 |     -0.97647 |     -0.21563 |     -0.00304 |            1 |    -0.03339 |    -0.01841 |    -0.39831 |    -0.97647 |    -0.21563 |   -0.00304 |   0.038129  |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 0.0002, 1)  |          1 |  0.00505 |             0 |     -0.029   |     -0.01736 |     -0.39785 |     -0.92453 |     -0.23015 |      0.30377 |            1 |    -0.03366 |    -0.01852 |    -0.39631 |    -0.92453 |    -0.23015 |    0.30377 |   0.0384186 |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 0.00025, 1) |          1 |  0.00508 |             0 |     -0.03147 |     -0.01732 |     -0.3985  |     -0.88756 |     -0.38991 |      0.24538 |            1 |    -0.03597 |    -0.0193  |    -0.39726 |    -0.88756 |    -0.38991 |    0.24538 |   0.0408207 |
 | ('20230504_6.6.5_myominimal_output', 'run0000_0000', 0.0003, 1)  |          1 |  0.0051  |             0 |     -0.03336 |     -0.01948 |     -0.40023 |     -0.79765 |      0.07911 |      0.5979  |            1 |    -0.03743 |    -0.01908 |    -0.39718 |    -0.79765 |     0.07911 |    0.5979  |   0.0420125 |
 #+RESULTS[78acfe33023b62622d7d567b5433cbc87b645c3d]:
 : finished parsing run0000_0000
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9986
 finished parsing run0000_0000 timepoint 0.99865
 finished parsing run0000_0000 timepoint 0.9987
 finished parsing run0000_0000 timepoint 0.99875
 finished parsing run0000_0000 timepoint 0.9988
 finished parsing run0000_0000 timepoint 0.99885
 finished parsing run0000_0000 timepoint 0.9989
 finished parsing run0000_0000 timepoint 0.99895
 finished parsing run0000_0000 timepoint 0.999
 finished parsing run0000_0000 timepoint 0.99905
 finished parsing run0000_0000 timepoint 0.9991
 finished parsing run0000_0000 timepoint 0.99915
 finished parsing run0000_0000 timepoint 0.9992
 finished parsing run0000_0000 timepoint 0.99925
 finished parsing run0000_0000 timepoint 0.9993
 finished parsing run0000_0000 timepoint 0.99935
 finished parsing run0000_0000 timepoint 0.9994
 finished parsing run0000_0000 timepoint 0.99945
 finished parsing run0000_0000 timepoint 0.9995
 finished parsing run0000_0000 timepoint 0.99955
 finished parsing run0000_0000 timepoint 0.9996
 finished parsing run0000_0000 timepoint 0.99965
 finished parsing run0000_0000 timepoint 0.9997
 finished parsing run0000_0000 timepoint 0.99975
 finished parsing run0000_0000 timepoint 0.9998
 finished parsing run0000_0000 timepoint 0.99985
 finished parsing run0000_0000 timepoint 0.9999
 finished parsing run0000_0000 timepoint 0.99995
 finished parsing run0000_0000 timepoint 1.0
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.99195
 finished parsing run0000_0000 timepoint 0.992
 finished parsing run0000_0000 timepoint 0.99205
 finished parsing run0000_0000 timepoint 0.9921
 finished parsing run0000_0000 timepoint 0.99215
 finished parsing run0000_0000 timepoint 0.9922
 finished parsing run0000_0000 timepoint 0.99225
 finished parsing run0000_0000 timepoint 0.9923
 finished parsing run0000_0000 timepoint 0.99235
 finished parsing run0000_0000 timepoint 0.9924
 finished parsing run0000_0000 timepoint 0.99245
 finished parsing run0000_0000 timepoint 0.9925
 finished parsing run0000_0000 timepoint 0.99255
 finished parsing run0000_0000 timepoint 0.9926
 finished parsing run0000_0000 timepoint 0.99265
 finished parsing run0000_0000 timepoint 0.9927
 finished parsing run0000_0000 timepoint 0.99275
 finished parsing run0000_0000 timepoint 0.9928
 finished parsing run0000_0000 timepoint 0.99285
 finished parsing run0000_0000 timepoint 0.9929
 finished parsing run0000_0000 timepoint 0.99295
 finished parsing run0000_0000 timepoint 0.993
 finished parsing run0000_0000 timepoint 0.99305
 finished parsing run0000_0000 timepoint 0.9931
 finished parsing run0000_0000 timepoint 0.99315
 finished parsing run0000_0000 timepoint 0.9932
 finished parsing run0000_0000 timepoint 0.99325
 finished parsing run0000_0000 timepoint 0.9933
 finished parsing run0000_0000 timepoint 0.99335
 finished parsing run0000_0000 timepoint 0.9934
 finished parsing run0000_0000 timepoint 0.99345
 finished parsing run0000_0000 timepoint 0.9935
 finished parsing run0000_0000 timepoint 0.99355
 finished parsing run0000_0000 timepoint 0.9936
 finished parsing run0000_0000 timepoint 0.99365
 finished parsing run0000_0000 timepoint 0.9937
 finished parsing run0000_0000 timepoint 0.99375
 finished parsing run0000_0000 timepoint 0.9938
 finished parsing run0000_0000 timepoint 0.99385
 finished parsing run0000_0000 timepoint 0.9939
 finished parsing run0000_0000 timepoint 0.99395
 finished parsing run0000_0000 timepoint 0.994
 finished parsing run0000_0000 timepoint 0.99405
 finished parsing run0000_0000 timepoint 0.9941
 finished parsing run0000_0000 timepoint 0.99415
 finished parsing run0000_0000 timepoint 0.9942
 finished parsing run0000_0000 timepoint 0.99425
 finished parsing run0000_0000 timepoint 0.9943
 finished parsing run0000_0000 timepoint 0.99435
 finished parsing run0000_0000 timepoint 0.9944
 finished parsing run0000_0000 timepoint 0.99445
 finished parsing run0000_0000 timepoint 0.9945
 finished parsing run0000_0000 timepoint 0.99455
 finished parsing run0000_0000 timepoint 0.9946
 finished parsing run0000_0000 timepoint 0.99465
 finished parsing run0000_0000 timepoint 0.9947
 finished parsing run0000_0000 timepoint 0.99475
 finished parsing run0000_0000 timepoint 0.9948
 finished parsing run0000_0000 timepoint 0.99485
 finished parsing run0000_0000 timepoint 0.9949
 finished parsing run0000_0000 timepoint 0.99495
 finished parsing run0000_0000 timepoint 0.995
 finished parsing run0000_0000 timepoint 0.99505
 finished parsing run0000_0000 timepoint 0.9951
 finished parsing run0000_0000 timepoint 0.99515
 finished parsing run0000_0000 timepoint 0.9952
 finished parsing run0000_0000 timepoint 0.99525
 finished parsing run0000_0000 timepoint 0.9953
 finished parsing run0000_0000 timepoint 0.99535
 finished parsing run0000_0000 timepoint 0.9954
 finished parsing run0000_0000 timepoint 0.99545
 finished parsing run0000_0000 timepoint 0.9955
 finished parsing run0000_0000 timepoint 0.99555
 finished parsing run0000_0000 timepoint 0.9956
 finished parsing run0000_0000 timepoint 0.99565
 finished parsing run0000_0000 timepoint 0.9957
 finished parsing run0000_0000 timepoint 0.99575
 finished parsing run0000_0000 timepoint 0.9958
 finished parsing run0000_0000 timepoint 0.99585
 finished parsing run0000_0000 timepoint 0.9959
 finished parsing run0000_0000 timepoint 0.99595
 finished parsing run0000_0000 timepoint 0.996
 finished parsing run0000_0000 timepoint 0.99605
 finished parsing run0000_0000 timepoint 0.9961
 finished parsing run0000_0000 timepoint 0.99615
 finished parsing run0000_0000 timepoint 0.9962
 finished parsing run0000_0000 timepoint 0.99625
 finished parsing run0000_0000 timepoint 0.9963
 finished parsing run0000_0000 timepoint 0.99635
 finished parsing run0000_0000 timepoint 0.9964
 finished parsing run0000_0000 timepoint 0.99645
 finished parsing run0000_0000 timepoint 0.9965
 finished parsing run0000_0000 timepoint 0.99655
 finished parsing run0000_0000 timepoint 0.9966
 finished parsing run0000_0000 timepoint 0.99665
 finished parsing run0000_0000 timepoint 0.9967
 finished parsing run0000_0000 timepoint 0.99675
 finished parsing run0000_0000 timepoint 0.9968
 finished parsing run0000_0000 timepoint 0.99685
 finished parsing run0000_0000 timepoint 0.9969
 finished parsing run0000_0000 timepoint 0.99695
 finished parsing run0000_0000 timepoint 0.997
 finished parsing run0000_0000 timepoint 0.99705
 finished parsing run0000_0000 timepoint 0.9971
 finished parsing run0000_0000 timepoint 0.99715
 finished parsing run0000_0000 timepoint 0.9972
 finished parsing run0000_0000 timepoint 0.99725
 finished parsing run0000_0000 timepoint 0.9973
 finished parsing run0000_0000 timepoint 0.99735
 finished parsing run0000_0000 timepoint 0.9974
 finished parsing run0000_0000 timepoint 0.99745
 finished parsing run0000_0000 timepoint 0.9975
 finished parsing run0000_0000 timepoint 0.99755
 finished parsing run0000_0000 timepoint 0.9976
 finished parsing run0000_0000 timepoint 0.99765
 finished parsing run0000_0000 timepoint 0.9977
 finished parsing run0000_0000 timepoint 0.99775
 finished parsing run0000_0000 timepoint 0.9978
 finished parsing run0000_0000 timepoint 0.99785
 finished parsing run0000_0000 timepoint 0.9979
 finished parsing run0000_0000 timepoint 0.99795
 finished parsing run0000_0000 timepoint 0.998
 finished parsing run0000_0000 timepoint 0.99805
 finished parsing run0000_0000 timepoint 0.9981
 finished parsing run0000_0000 timepoint 0.99815
 finished parsing run0000_0000 timepoint 0.9982
 finished parsing run0000_0000 timepoint 0.99825
 finished parsing run0000_0000 timepoint 0.9983
 finished parsing run0000_0000 timepoint 0.99835
 finished parsing run0000_0000 timepoint 0.9984
 finished parsing run0000_0000 timepoint 0.99845
 finished parsing run0000_0000 timepoint 0.9985
 finished parsing run0000_0000 timepoint 0.99855
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.98545
 finished parsing run0000_0000 timepoint 0.9855
 finished parsing run0000_0000 timepoint 0.98555
 finished parsing run0000_0000 timepoint 0.9856
 finished parsing run0000_0000 timepoint 0.98565
 finished parsing run0000_0000 timepoint 0.9857
 finished parsing run0000_0000 timepoint 0.98575
 finished parsing run0000_0000 timepoint 0.9858
 finished parsing run0000_0000 timepoint 0.98585
 finished parsing run0000_0000 timepoint 0.9859
 finished parsing run0000_0000 timepoint 0.98595
 finished parsing run0000_0000 timepoint 0.986
 finished parsing run0000_0000 timepoint 0.98605
 finished parsing run0000_0000 timepoint 0.9861
 finished parsing run0000_0000 timepoint 0.98615
 finished parsing run0000_0000 timepoint 0.9862
 finished parsing run0000_0000 timepoint 0.98625
 finished parsing run0000_0000 timepoint 0.9863
 finished parsing run0000_0000 timepoint 0.98635
 finished parsing run0000_0000 timepoint 0.9864
 finished parsing run0000_0000 timepoint 0.98645
 finished parsing run0000_0000 timepoint 0.9865
 finished parsing run0000_0000 timepoint 0.98655
 finished parsing run0000_0000 timepoint 0.9866
 finished parsing run0000_0000 timepoint 0.98665
 finished parsing run0000_0000 timepoint 0.9867
 finished parsing run0000_0000 timepoint 0.98675
 finished parsing run0000_0000 timepoint 0.9868
 finished parsing run0000_0000 timepoint 0.98685
 finished parsing run0000_0000 timepoint 0.9869
 finished parsing run0000_0000 timepoint 0.98695
 finished parsing run0000_0000 timepoint 0.987
 finished parsing run0000_0000 timepoint 0.98705
 finished parsing run0000_0000 timepoint 0.9871
 finished parsing run0000_0000 timepoint 0.98715
 finished parsing run0000_0000 timepoint 0.9872
 finished parsing run0000_0000 timepoint 0.98725
 finished parsing run0000_0000 timepoint 0.9873
 finished parsing run0000_0000 timepoint 0.98735
 finished parsing run0000_0000 timepoint 0.9874
 finished parsing run0000_0000 timepoint 0.98745
 finished parsing run0000_0000 timepoint 0.9875
 finished parsing run0000_0000 timepoint 0.98755
 finished parsing run0000_0000 timepoint 0.9876
 finished parsing run0000_0000 timepoint 0.98765
 finished parsing run0000_0000 timepoint 0.9877
 finished parsing run0000_0000 timepoint 0.98775
 finished parsing run0000_0000 timepoint 0.9878
 finished parsing run0000_0000 timepoint 0.98785
 finished parsing run0000_0000 timepoint 0.9879
 finished parsing run0000_0000 timepoint 0.98795
 finished parsing run0000_0000 timepoint 0.988
 finished parsing run0000_0000 timepoint 0.98805
 finished parsing run0000_0000 timepoint 0.9881
 finished parsing run0000_0000 timepoint 0.98815
 finished parsing run0000_0000 timepoint 0.9882
 finished parsing run0000_0000 timepoint 0.98825
 finished parsing run0000_0000 timepoint 0.9883
 finished parsing run0000_0000 timepoint 0.98835
 finished parsing run0000_0000 timepoint 0.9884
 finished parsing run0000_0000 timepoint 0.98845
 finished parsing run0000_0000 timepoint 0.9885
 finished parsing run0000_0000 timepoint 0.98855
 finished parsing run0000_0000 timepoint 0.9886
 finished parsing run0000_0000 timepoint 0.98865
 finished parsing run0000_0000 timepoint 0.9887
 finished parsing run0000_0000 timepoint 0.98875
 finished parsing run0000_0000 timepoint 0.9888
 finished parsing run0000_0000 timepoint 0.98885
 finished parsing run0000_0000 timepoint 0.9889
 finished parsing run0000_0000 timepoint 0.98895
 finished parsing run0000_0000 timepoint 0.989
 finished parsing run0000_0000 timepoint 0.98905
 finished parsing run0000_0000 timepoint 0.9891
 finished parsing run0000_0000 timepoint 0.98915
 finished parsing run0000_0000 timepoint 0.9892
 finished parsing run0000_0000 timepoint 0.98925
 finished parsing run0000_0000 timepoint 0.9893
 finished parsing run0000_0000 timepoint 0.98935
 finished parsing run0000_0000 timepoint 0.9894
 finished parsing run0000_0000 timepoint 0.98945
 finished parsing run0000_0000 timepoint 0.9895
 finished parsing run0000_0000 timepoint 0.98955
 finished parsing run0000_0000 timepoint 0.9896
 finished parsing run0000_0000 timepoint 0.98965
 finished parsing run0000_0000 timepoint 0.9897
 finished parsing run0000_0000 timepoint 0.98975
 finished parsing run0000_0000 timepoint 0.9898
 finished parsing run0000_0000 timepoint 0.98985
 finished parsing run0000_0000 timepoint 0.9899
 finished parsing run0000_0000 timepoint 0.98995
 finished parsing run0000_0000 timepoint 0.99
 finished parsing run0000_0000 timepoint 0.99005
 finished parsing run0000_0000 timepoint 0.9901
 finished parsing run0000_0000 timepoint 0.99015
 finished parsing run0000_0000 timepoint 0.9902
 finished parsing run0000_0000 timepoint 0.99025
 finished parsing run0000_0000 timepoint 0.9903
 finished parsing run0000_0000 timepoint 0.99035
 finished parsing run0000_0000 timepoint 0.9904
 finished parsing run0000_0000 timepoint 0.99045
 finished parsing run0000_0000 timepoint 0.9905
 finished parsing run0000_0000 timepoint 0.99055
 finished parsing run0000_0000 timepoint 0.9906
 finished parsing run0000_0000 timepoint 0.99065
 finished parsing run0000_0000 timepoint 0.9907
 finished parsing run0000_0000 timepoint 0.99075
 finished parsing run0000_0000 timepoint 0.9908
 finished parsing run0000_0000 timepoint 0.99085
 finished parsing run0000_0000 timepoint 0.9909
 finished parsing run0000_0000 timepoint 0.99095
 finished parsing run0000_0000 timepoint 0.991
 finished parsing run0000_0000 timepoint 0.99105
 finished parsing run0000_0000 timepoint 0.9911
 finished parsing run0000_0000 timepoint 0.99115
 finished parsing run0000_0000 timepoint 0.9912
 finished parsing run0000_0000 timepoint 0.99125
 finished parsing run0000_0000 timepoint 0.9913
 finished parsing run0000_0000 timepoint 0.99135
 finished parsing run0000_0000 timepoint 0.9914
 finished parsing run0000_0000 timepoint 0.99145
 finished parsing run0000_0000 timepoint 0.9915
 finished parsing run0000_0000 timepoint 0.99155
 finished parsing run0000_0000 timepoint 0.9916
 finished parsing run0000_0000 timepoint 0.99165
 finished parsing run0000_0000 timepoint 0.9917
 finished parsing run0000_0000 timepoint 0.99175
 finished parsing run0000_0000 timepoint 0.9918
 finished parsing run0000_0000 timepoint 0.99185
 finished parsing run0000_0000 timepoint 0.9919
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.979
 finished parsing run0000_0000 timepoint 0.97905
 finished parsing run0000_0000 timepoint 0.9791
 finished parsing run0000_0000 timepoint 0.97915
 finished parsing run0000_0000 timepoint 0.9792
 finished parsing run0000_0000 timepoint 0.97925
 finished parsing run0000_0000 timepoint 0.9793
 finished parsing run0000_0000 timepoint 0.97935
 finished parsing run0000_0000 timepoint 0.9794
 finished parsing run0000_0000 timepoint 0.97945
 finished parsing run0000_0000 timepoint 0.9795
 finished parsing run0000_0000 timepoint 0.97955
 finished parsing run0000_0000 timepoint 0.9796
 finished parsing run0000_0000 timepoint 0.97965
 finished parsing run0000_0000 timepoint 0.9797
 finished parsing run0000_0000 timepoint 0.97975
 finished parsing run0000_0000 timepoint 0.9798
 finished parsing run0000_0000 timepoint 0.97985
 finished parsing run0000_0000 timepoint 0.9799
 finished parsing run0000_0000 timepoint 0.97995
 finished parsing run0000_0000 timepoint 0.98
 finished parsing run0000_0000 timepoint 0.98005
 finished parsing run0000_0000 timepoint 0.9801
 finished parsing run0000_0000 timepoint 0.98015
 finished parsing run0000_0000 timepoint 0.9802
 finished parsing run0000_0000 timepoint 0.98025
 finished parsing run0000_0000 timepoint 0.9803
 finished parsing run0000_0000 timepoint 0.98035
 finished parsing run0000_0000 timepoint 0.9804
 finished parsing run0000_0000 timepoint 0.98045
 finished parsing run0000_0000 timepoint 0.9805
 finished parsing run0000_0000 timepoint 0.98055
 finished parsing run0000_0000 timepoint 0.9806
 finished parsing run0000_0000 timepoint 0.98065
 finished parsing run0000_0000 timepoint 0.9807
 finished parsing run0000_0000 timepoint 0.98075
 finished parsing run0000_0000 timepoint 0.9808
 finished parsing run0000_0000 timepoint 0.98085
 finished parsing run0000_0000 timepoint 0.9809
 finished parsing run0000_0000 timepoint 0.98095
 finished parsing run0000_0000 timepoint 0.981
 finished parsing run0000_0000 timepoint 0.98105
 finished parsing run0000_0000 timepoint 0.9811
 finished parsing run0000_0000 timepoint 0.98115
 finished parsing run0000_0000 timepoint 0.9812
 finished parsing run0000_0000 timepoint 0.98125
 finished parsing run0000_0000 timepoint 0.9813
 finished parsing run0000_0000 timepoint 0.98135
 finished parsing run0000_0000 timepoint 0.9814
 finished parsing run0000_0000 timepoint 0.98145
 finished parsing run0000_0000 timepoint 0.9815
 finished parsing run0000_0000 timepoint 0.98155
 finished parsing run0000_0000 timepoint 0.9816
 finished parsing run0000_0000 timepoint 0.98165
 finished parsing run0000_0000 timepoint 0.9817
 finished parsing run0000_0000 timepoint 0.98175
 finished parsing run0000_0000 timepoint 0.9818
 finished parsing run0000_0000 timepoint 0.98185
 finished parsing run0000_0000 timepoint 0.9819
 finished parsing run0000_0000 timepoint 0.98195
 finished parsing run0000_0000 timepoint 0.982
 finished parsing run0000_0000 timepoint 0.98205
 finished parsing run0000_0000 timepoint 0.9821
 finished parsing run0000_0000 timepoint 0.98215
 finished parsing run0000_0000 timepoint 0.9822
 finished parsing run0000_0000 timepoint 0.98225
 finished parsing run0000_0000 timepoint 0.9823
 finished parsing run0000_0000 timepoint 0.98235
 finished parsing run0000_0000 timepoint 0.9824
 finished parsing run0000_0000 timepoint 0.98245
 finished parsing run0000_0000 timepoint 0.9825
 finished parsing run0000_0000 timepoint 0.98255
 finished parsing run0000_0000 timepoint 0.9826
 finished parsing run0000_0000 timepoint 0.98265
 finished parsing run0000_0000 timepoint 0.9827
 finished parsing run0000_0000 timepoint 0.98275
 finished parsing run0000_0000 timepoint 0.9828
 finished parsing run0000_0000 timepoint 0.98285
 finished parsing run0000_0000 timepoint 0.9829
 finished parsing run0000_0000 timepoint 0.98295
 finished parsing run0000_0000 timepoint 0.983
 finished parsing run0000_0000 timepoint 0.98305
 finished parsing run0000_0000 timepoint 0.9831
 finished parsing run0000_0000 timepoint 0.98315
 finished parsing run0000_0000 timepoint 0.9832
 finished parsing run0000_0000 timepoint 0.98325
 finished parsing run0000_0000 timepoint 0.9833
 finished parsing run0000_0000 timepoint 0.98335
 finished parsing run0000_0000 timepoint 0.9834
 finished parsing run0000_0000 timepoint 0.98345
 finished parsing run0000_0000 timepoint 0.9835
 finished parsing run0000_0000 timepoint 0.98355
 finished parsing run0000_0000 timepoint 0.9836
 finished parsing run0000_0000 timepoint 0.98365
 finished parsing run0000_0000 timepoint 0.9837
 finished parsing run0000_0000 timepoint 0.98375
 finished parsing run0000_0000 timepoint 0.9838
 finished parsing run0000_0000 timepoint 0.98385
 finished parsing run0000_0000 timepoint 0.9839
 finished parsing run0000_0000 timepoint 0.98395
 finished parsing run0000_0000 timepoint 0.984
 finished parsing run0000_0000 timepoint 0.98405
 finished parsing run0000_0000 timepoint 0.9841
 finished parsing run0000_0000 timepoint 0.98415
 finished parsing run0000_0000 timepoint 0.9842
 finished parsing run0000_0000 timepoint 0.98425
 finished parsing run0000_0000 timepoint 0.9843
 finished parsing run0000_0000 timepoint 0.98435
 finished parsing run0000_0000 timepoint 0.9844
 finished parsing run0000_0000 timepoint 0.98445
 finished parsing run0000_0000 timepoint 0.9845
 finished parsing run0000_0000 timepoint 0.98455
 finished parsing run0000_0000 timepoint 0.9846
 finished parsing run0000_0000 timepoint 0.98465
 finished parsing run0000_0000 timepoint 0.9847
 finished parsing run0000_0000 timepoint 0.98475
 finished parsing run0000_0000 timepoint 0.9848
 finished parsing run0000_0000 timepoint 0.98485
 finished parsing run0000_0000 timepoint 0.9849
 finished parsing run0000_0000 timepoint 0.98495
 finished parsing run0000_0000 timepoint 0.985
 finished parsing run0000_0000 timepoint 0.98505
 finished parsing run0000_0000 timepoint 0.9851
 finished parsing run0000_0000 timepoint 0.98515
 finished parsing run0000_0000 timepoint 0.9852
 finished parsing run0000_0000 timepoint 0.98525
 finished parsing run0000_0000 timepoint 0.9853
 finished parsing run0000_0000 timepoint 0.98535
 finished parsing run0000_0000 timepoint 0.9854
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9725
 finished parsing run0000_0000 timepoint 0.97255
 finished parsing run0000_0000 timepoint 0.9726
 finished parsing run0000_0000 timepoint 0.97265
 finished parsing run0000_0000 timepoint 0.9727
 finished parsing run0000_0000 timepoint 0.97275
 finished parsing run0000_0000 timepoint 0.9728
 finished parsing run0000_0000 timepoint 0.97285
 finished parsing run0000_0000 timepoint 0.9729
 finished parsing run0000_0000 timepoint 0.97295
 finished parsing run0000_0000 timepoint 0.973
 finished parsing run0000_0000 timepoint 0.97305
 finished parsing run0000_0000 timepoint 0.9731
 finished parsing run0000_0000 timepoint 0.97315
 finished parsing run0000_0000 timepoint 0.9732
 finished parsing run0000_0000 timepoint 0.97325
 finished parsing run0000_0000 timepoint 0.9733
 finished parsing run0000_0000 timepoint 0.97335
 finished parsing run0000_0000 timepoint 0.9734
 finished parsing run0000_0000 timepoint 0.97345
 finished parsing run0000_0000 timepoint 0.9735
 finished parsing run0000_0000 timepoint 0.97355
 finished parsing run0000_0000 timepoint 0.9736
 finished parsing run0000_0000 timepoint 0.97365
 finished parsing run0000_0000 timepoint 0.9737
 finished parsing run0000_0000 timepoint 0.97375
 finished parsing run0000_0000 timepoint 0.9738
 finished parsing run0000_0000 timepoint 0.97385
 finished parsing run0000_0000 timepoint 0.9739
 finished parsing run0000_0000 timepoint 0.97395
 finished parsing run0000_0000 timepoint 0.974
 finished parsing run0000_0000 timepoint 0.97405
 finished parsing run0000_0000 timepoint 0.9741
 finished parsing run0000_0000 timepoint 0.97415
 finished parsing run0000_0000 timepoint 0.9742
 finished parsing run0000_0000 timepoint 0.97425
 finished parsing run0000_0000 timepoint 0.9743
 finished parsing run0000_0000 timepoint 0.97435
 finished parsing run0000_0000 timepoint 0.9744
 finished parsing run0000_0000 timepoint 0.97445
 finished parsing run0000_0000 timepoint 0.9745
 finished parsing run0000_0000 timepoint 0.97455
 finished parsing run0000_0000 timepoint 0.9746
 finished parsing run0000_0000 timepoint 0.97465
 finished parsing run0000_0000 timepoint 0.9747
 finished parsing run0000_0000 timepoint 0.97475
 finished parsing run0000_0000 timepoint 0.9748
 finished parsing run0000_0000 timepoint 0.97485
 finished parsing run0000_0000 timepoint 0.9749
 finished parsing run0000_0000 timepoint 0.97495
 finished parsing run0000_0000 timepoint 0.975
 finished parsing run0000_0000 timepoint 0.97505
 finished parsing run0000_0000 timepoint 0.9751
 finished parsing run0000_0000 timepoint 0.97515
 finished parsing run0000_0000 timepoint 0.9752
 finished parsing run0000_0000 timepoint 0.97525
 finished parsing run0000_0000 timepoint 0.9753
 finished parsing run0000_0000 timepoint 0.97535
 finished parsing run0000_0000 timepoint 0.9754
 finished parsing run0000_0000 timepoint 0.97545
 finished parsing run0000_0000 timepoint 0.9755
 finished parsing run0000_0000 timepoint 0.97555
 finished parsing run0000_0000 timepoint 0.9756
 finished parsing run0000_0000 timepoint 0.97565
 finished parsing run0000_0000 timepoint 0.9757
 finished parsing run0000_0000 timepoint 0.97575
 finished parsing run0000_0000 timepoint 0.9758
 finished parsing run0000_0000 timepoint 0.97585
 finished parsing run0000_0000 timepoint 0.9759
 finished parsing run0000_0000 timepoint 0.97595
 finished parsing run0000_0000 timepoint 0.976
 finished parsing run0000_0000 timepoint 0.97605
 finished parsing run0000_0000 timepoint 0.9761
 finished parsing run0000_0000 timepoint 0.97615
 finished parsing run0000_0000 timepoint 0.9762
 finished parsing run0000_0000 timepoint 0.97625
 finished parsing run0000_0000 timepoint 0.9763
 finished parsing run0000_0000 timepoint 0.97635
 finished parsing run0000_0000 timepoint 0.9764
 finished parsing run0000_0000 timepoint 0.97645
 finished parsing run0000_0000 timepoint 0.9765
 finished parsing run0000_0000 timepoint 0.97655
 finished parsing run0000_0000 timepoint 0.9766
 finished parsing run0000_0000 timepoint 0.97665
 finished parsing run0000_0000 timepoint 0.9767
 finished parsing run0000_0000 timepoint 0.97675
 finished parsing run0000_0000 timepoint 0.9768
 finished parsing run0000_0000 timepoint 0.97685
 finished parsing run0000_0000 timepoint 0.9769
 finished parsing run0000_0000 timepoint 0.97695
 finished parsing run0000_0000 timepoint 0.977
 finished parsing run0000_0000 timepoint 0.97705
 finished parsing run0000_0000 timepoint 0.9771
 finished parsing run0000_0000 timepoint 0.97715
 finished parsing run0000_0000 timepoint 0.9772
 finished parsing run0000_0000 timepoint 0.97725
 finished parsing run0000_0000 timepoint 0.9773
 finished parsing run0000_0000 timepoint 0.97735
 finished parsing run0000_0000 timepoint 0.9774
 finished parsing run0000_0000 timepoint 0.97745
 finished parsing run0000_0000 timepoint 0.9775
 finished parsing run0000_0000 timepoint 0.97755
 finished parsing run0000_0000 timepoint 0.9776
 finished parsing run0000_0000 timepoint 0.97765
 finished parsing run0000_0000 timepoint 0.9777
 finished parsing run0000_0000 timepoint 0.97775
 finished parsing run0000_0000 timepoint 0.9778
 finished parsing run0000_0000 timepoint 0.97785
 finished parsing run0000_0000 timepoint 0.9779
 finished parsing run0000_0000 timepoint 0.97795
 finished parsing run0000_0000 timepoint 0.978
 finished parsing run0000_0000 timepoint 0.97805
 finished parsing run0000_0000 timepoint 0.9781
 finished parsing run0000_0000 timepoint 0.97815
 finished parsing run0000_0000 timepoint 0.9782
 finished parsing run0000_0000 timepoint 0.97825
 finished parsing run0000_0000 timepoint 0.9783
 finished parsing run0000_0000 timepoint 0.97835
 finished parsing run0000_0000 timepoint 0.9784
 finished parsing run0000_0000 timepoint 0.97845
 finished parsing run0000_0000 timepoint 0.9785
 finished parsing run0000_0000 timepoint 0.97855
 finished parsing run0000_0000 timepoint 0.9786
 finished parsing run0000_0000 timepoint 0.97865
 finished parsing run0000_0000 timepoint 0.9787
 finished parsing run0000_0000 timepoint 0.97875
 finished parsing run0000_0000 timepoint 0.9788
 finished parsing run0000_0000 timepoint 0.97885
 finished parsing run0000_0000 timepoint 0.9789
 finished parsing run0000_0000 timepoint 0.97895
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.96605
 finished parsing run0000_0000 timepoint 0.9661
 finished parsing run0000_0000 timepoint 0.96615
 finished parsing run0000_0000 timepoint 0.9662
 finished parsing run0000_0000 timepoint 0.96625
 finished parsing run0000_0000 timepoint 0.9663
 finished parsing run0000_0000 timepoint 0.96635
 finished parsing run0000_0000 timepoint 0.9664
 finished parsing run0000_0000 timepoint 0.96645
 finished parsing run0000_0000 timepoint 0.9665
 finished parsing run0000_0000 timepoint 0.96655
 finished parsing run0000_0000 timepoint 0.9666
 finished parsing run0000_0000 timepoint 0.96665
 finished parsing run0000_0000 timepoint 0.9667
 finished parsing run0000_0000 timepoint 0.96675
 finished parsing run0000_0000 timepoint 0.9668
 finished parsing run0000_0000 timepoint 0.96685
 finished parsing run0000_0000 timepoint 0.9669
 finished parsing run0000_0000 timepoint 0.96695
 finished parsing run0000_0000 timepoint 0.967
 finished parsing run0000_0000 timepoint 0.96705
 finished parsing run0000_0000 timepoint 0.9671
 finished parsing run0000_0000 timepoint 0.96715
 finished parsing run0000_0000 timepoint 0.9672
 finished parsing run0000_0000 timepoint 0.96725
 finished parsing run0000_0000 timepoint 0.9673
 finished parsing run0000_0000 timepoint 0.96735
 finished parsing run0000_0000 timepoint 0.9674
 finished parsing run0000_0000 timepoint 0.96745
 finished parsing run0000_0000 timepoint 0.9675
 finished parsing run0000_0000 timepoint 0.96755
 finished parsing run0000_0000 timepoint 0.9676
 finished parsing run0000_0000 timepoint 0.96765
 finished parsing run0000_0000 timepoint 0.9677
 finished parsing run0000_0000 timepoint 0.96775
 finished parsing run0000_0000 timepoint 0.9678
 finished parsing run0000_0000 timepoint 0.96785
 finished parsing run0000_0000 timepoint 0.9679
 finished parsing run0000_0000 timepoint 0.96795
 finished parsing run0000_0000 timepoint 0.968
 finished parsing run0000_0000 timepoint 0.96805
 finished parsing run0000_0000 timepoint 0.9681
 finished parsing run0000_0000 timepoint 0.96815
 finished parsing run0000_0000 timepoint 0.9682
 finished parsing run0000_0000 timepoint 0.96825
 finished parsing run0000_0000 timepoint 0.9683
 finished parsing run0000_0000 timepoint 0.96835
 finished parsing run0000_0000 timepoint 0.9684
 finished parsing run0000_0000 timepoint 0.96845
 finished parsing run0000_0000 timepoint 0.9685
 finished parsing run0000_0000 timepoint 0.96855
 finished parsing run0000_0000 timepoint 0.9686
 finished parsing run0000_0000 timepoint 0.96865
 finished parsing run0000_0000 timepoint 0.9687
 finished parsing run0000_0000 timepoint 0.96875
 finished parsing run0000_0000 timepoint 0.9688
 finished parsing run0000_0000 timepoint 0.96885
 finished parsing run0000_0000 timepoint 0.9689
 finished parsing run0000_0000 timepoint 0.96895
 finished parsing run0000_0000 timepoint 0.969
 finished parsing run0000_0000 timepoint 0.96905
 finished parsing run0000_0000 timepoint 0.9691
 finished parsing run0000_0000 timepoint 0.96915
 finished parsing run0000_0000 timepoint 0.9692
 finished parsing run0000_0000 timepoint 0.96925
 finished parsing run0000_0000 timepoint 0.9693
 finished parsing run0000_0000 timepoint 0.96935
 finished parsing run0000_0000 timepoint 0.9694
 finished parsing run0000_0000 timepoint 0.96945
 finished parsing run0000_0000 timepoint 0.9695
 finished parsing run0000_0000 timepoint 0.96955
 finished parsing run0000_0000 timepoint 0.9696
 finished parsing run0000_0000 timepoint 0.96965
 finished parsing run0000_0000 timepoint 0.9697
 finished parsing run0000_0000 timepoint 0.96975
 finished parsing run0000_0000 timepoint 0.9698
 finished parsing run0000_0000 timepoint 0.96985
 finished parsing run0000_0000 timepoint 0.9699
 finished parsing run0000_0000 timepoint 0.96995
 finished parsing run0000_0000 timepoint 0.97
 finished parsing run0000_0000 timepoint 0.97005
 finished parsing run0000_0000 timepoint 0.9701
 finished parsing run0000_0000 timepoint 0.97015
 finished parsing run0000_0000 timepoint 0.9702
 finished parsing run0000_0000 timepoint 0.97025
 finished parsing run0000_0000 timepoint 0.9703
 finished parsing run0000_0000 timepoint 0.97035
 finished parsing run0000_0000 timepoint 0.9704
 finished parsing run0000_0000 timepoint 0.97045
 finished parsing run0000_0000 timepoint 0.9705
 finished parsing run0000_0000 timepoint 0.97055
 finished parsing run0000_0000 timepoint 0.9706
 finished parsing run0000_0000 timepoint 0.97065
 finished parsing run0000_0000 timepoint 0.9707
 finished parsing run0000_0000 timepoint 0.97075
 finished parsing run0000_0000 timepoint 0.9708
 finished parsing run0000_0000 timepoint 0.97085
 finished parsing run0000_0000 timepoint 0.9709
 finished parsing run0000_0000 timepoint 0.97095
 finished parsing run0000_0000 timepoint 0.971
 finished parsing run0000_0000 timepoint 0.97105
 finished parsing run0000_0000 timepoint 0.9711
 finished parsing run0000_0000 timepoint 0.97115
 finished parsing run0000_0000 timepoint 0.9712
 finished parsing run0000_0000 timepoint 0.97125
 finished parsing run0000_0000 timepoint 0.9713
 finished parsing run0000_0000 timepoint 0.97135
 finished parsing run0000_0000 timepoint 0.9714
 finished parsing run0000_0000 timepoint 0.97145
 finished parsing run0000_0000 timepoint 0.9715
 finished parsing run0000_0000 timepoint 0.97155
 finished parsing run0000_0000 timepoint 0.9716
 finished parsing run0000_0000 timepoint 0.97165
 finished parsing run0000_0000 timepoint 0.9717
 finished parsing run0000_0000 timepoint 0.97175
 finished parsing run0000_0000 timepoint 0.9718
 finished parsing run0000_0000 timepoint 0.97185
 finished parsing run0000_0000 timepoint 0.9719
 finished parsing run0000_0000 timepoint 0.97195
 finished parsing run0000_0000 timepoint 0.972
 finished parsing run0000_0000 timepoint 0.97205
 finished parsing run0000_0000 timepoint 0.9721
 finished parsing run0000_0000 timepoint 0.97215
 finished parsing run0000_0000 timepoint 0.9722
 finished parsing run0000_0000 timepoint 0.97225
 finished parsing run0000_0000 timepoint 0.9723
 finished parsing run0000_0000 timepoint 0.97235
 finished parsing run0000_0000 timepoint 0.9724
 finished parsing run0000_0000 timepoint 0.97245
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.95975
 finished parsing run0000_0000 timepoint 0.9598
 finished parsing run0000_0000 timepoint 0.95985
 finished parsing run0000_0000 timepoint 0.9599
 finished parsing run0000_0000 timepoint 0.95995
 finished parsing run0000_0000 timepoint 0.96
 finished parsing run0000_0000 timepoint 0.96005
 finished parsing run0000_0000 timepoint 0.9601
 finished parsing run0000_0000 timepoint 0.96015
 finished parsing run0000_0000 timepoint 0.9602
 finished parsing run0000_0000 timepoint 0.96025
 finished parsing run0000_0000 timepoint 0.9603
 finished parsing run0000_0000 timepoint 0.96035
 finished parsing run0000_0000 timepoint 0.9604
 finished parsing run0000_0000 timepoint 0.96045
 finished parsing run0000_0000 timepoint 0.9605
 finished parsing run0000_0000 timepoint 0.96055
 finished parsing run0000_0000 timepoint 0.9606
 finished parsing run0000_0000 timepoint 0.96065
 finished parsing run0000_0000 timepoint 0.9607
 finished parsing run0000_0000 timepoint 0.96075
 finished parsing run0000_0000 timepoint 0.9608
 finished parsing run0000_0000 timepoint 0.96085
 finished parsing run0000_0000 timepoint 0.9609
 finished parsing run0000_0000 timepoint 0.96095
 finished parsing run0000_0000 timepoint 0.961
 finished parsing run0000_0000 timepoint 0.96105
 finished parsing run0000_0000 timepoint 0.9611
 finished parsing run0000_0000 timepoint 0.96115
 finished parsing run0000_0000 timepoint 0.9612
 finished parsing run0000_0000 timepoint 0.96125
 finished parsing run0000_0000 timepoint 0.9613
 finished parsing run0000_0000 timepoint 0.96135
 finished parsing run0000_0000 timepoint 0.9614
 finished parsing run0000_0000 timepoint 0.96145
 finished parsing run0000_0000 timepoint 0.9615
 finished parsing run0000_0000 timepoint 0.96155
 finished parsing run0000_0000 timepoint 0.9616
 finished parsing run0000_0000 timepoint 0.96165
 finished parsing run0000_0000 timepoint 0.9617
 finished parsing run0000_0000 timepoint 0.96175
 finished parsing run0000_0000 timepoint 0.9618
 finished parsing run0000_0000 timepoint 0.96185
 finished parsing run0000_0000 timepoint 0.9619
 finished parsing run0000_0000 timepoint 0.96195
 finished parsing run0000_0000 timepoint 0.962
 finished parsing run0000_0000 timepoint 0.96205
 finished parsing run0000_0000 timepoint 0.9621
 finished parsing run0000_0000 timepoint 0.96215
 finished parsing run0000_0000 timepoint 0.9622
 finished parsing run0000_0000 timepoint 0.96225
 finished parsing run0000_0000 timepoint 0.9623
 finished parsing run0000_0000 timepoint 0.96235
 finished parsing run0000_0000 timepoint 0.9624
 finished parsing run0000_0000 timepoint 0.96245
 finished parsing run0000_0000 timepoint 0.9625
 finished parsing run0000_0000 timepoint 0.96255
 finished parsing run0000_0000 timepoint 0.9626
 finished parsing run0000_0000 timepoint 0.96265
 finished parsing run0000_0000 timepoint 0.9627
 finished parsing run0000_0000 timepoint 0.96275
 finished parsing run0000_0000 timepoint 0.9628
 finished parsing run0000_0000 timepoint 0.96285
 finished parsing run0000_0000 timepoint 0.9629
 finished parsing run0000_0000 timepoint 0.96295
 finished parsing run0000_0000 timepoint 0.963
 finished parsing run0000_0000 timepoint 0.96305
 finished parsing run0000_0000 timepoint 0.9631
 finished parsing run0000_0000 timepoint 0.96315
 finished parsing run0000_0000 timepoint 0.9632
 finished parsing run0000_0000 timepoint 0.96325
 finished parsing run0000_0000 timepoint 0.9633
 finished parsing run0000_0000 timepoint 0.96335
 finished parsing run0000_0000 timepoint 0.9634
 finished parsing run0000_0000 timepoint 0.96345
 finished parsing run0000_0000 timepoint 0.9635
 finished parsing run0000_0000 timepoint 0.96355
 finished parsing run0000_0000 timepoint 0.9636
 finished parsing run0000_0000 timepoint 0.96365
 finished parsing run0000_0000 timepoint 0.9637
 finished parsing run0000_0000 timepoint 0.96375
 finished parsing run0000_0000 timepoint 0.9638
 finished parsing run0000_0000 timepoint 0.96385
 finished parsing run0000_0000 timepoint 0.9639
 finished parsing run0000_0000 timepoint 0.96395
 finished parsing run0000_0000 timepoint 0.964
 finished parsing run0000_0000 timepoint 0.96405
 finished parsing run0000_0000 timepoint 0.9641
 finished parsing run0000_0000 timepoint 0.96415
 finished parsing run0000_0000 timepoint 0.9642
 finished parsing run0000_0000 timepoint 0.96425
 finished parsing run0000_0000 timepoint 0.9643
 finished parsing run0000_0000 timepoint 0.96435
 finished parsing run0000_0000 timepoint 0.9644
 finished parsing run0000_0000 timepoint 0.96445
 finished parsing run0000_0000 timepoint 0.9645
 finished parsing run0000_0000 timepoint 0.96455
 finished parsing run0000_0000 timepoint 0.9646
 finished parsing run0000_0000 timepoint 0.96465
 finished parsing run0000_0000 timepoint 0.9647
 finished parsing run0000_0000 timepoint 0.96475
 finished parsing run0000_0000 timepoint 0.9648
 finished parsing run0000_0000 timepoint 0.96485
 finished parsing run0000_0000 timepoint 0.9649
 finished parsing run0000_0000 timepoint 0.96495
 finished parsing run0000_0000 timepoint 0.965
 finished parsing run0000_0000 timepoint 0.96505
 finished parsing run0000_0000 timepoint 0.9651
 finished parsing run0000_0000 timepoint 0.96515
 finished parsing run0000_0000 timepoint 0.9652
 finished parsing run0000_0000 timepoint 0.96525
 finished parsing run0000_0000 timepoint 0.9653
 finished parsing run0000_0000 timepoint 0.96535
 finished parsing run0000_0000 timepoint 0.9654
 finished parsing run0000_0000 timepoint 0.96545
 finished parsing run0000_0000 timepoint 0.9655
 finished parsing run0000_0000 timepoint 0.96555
 finished parsing run0000_0000 timepoint 0.9656
 finished parsing run0000_0000 timepoint 0.96565
 finished parsing run0000_0000 timepoint 0.9657
 finished parsing run0000_0000 timepoint 0.96575
 finished parsing run0000_0000 timepoint 0.9658
 finished parsing run0000_0000 timepoint 0.96585
 finished parsing run0000_0000 timepoint 0.9659
 finished parsing run0000_0000 timepoint 0.96595
 finished parsing run0000_0000 timepoint 0.966
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.95265
 finished parsing run0000_0000 timepoint 0.9527
 finished parsing run0000_0000 timepoint 0.95275
 finished parsing run0000_0000 timepoint 0.9528
 finished parsing run0000_0000 timepoint 0.95285
 finished parsing run0000_0000 timepoint 0.9529
 finished parsing run0000_0000 timepoint 0.95295
 finished parsing run0000_0000 timepoint 0.953
 finished parsing run0000_0000 timepoint 0.95305
 finished parsing run0000_0000 timepoint 0.9531
 finished parsing run0000_0000 timepoint 0.95315
 finished parsing run0000_0000 timepoint 0.9532
 finished parsing run0000_0000 timepoint 0.95325
 finished parsing run0000_0000 timepoint 0.9533
 finished parsing run0000_0000 timepoint 0.95335
 finished parsing run0000_0000 timepoint 0.9534
 finished parsing run0000_0000 timepoint 0.95345
 finished parsing run0000_0000 timepoint 0.9535
 finished parsing run0000_0000 timepoint 0.95355
 finished parsing run0000_0000 timepoint 0.9536
 finished parsing run0000_0000 timepoint 0.95365
 finished parsing run0000_0000 timepoint 0.9537
 finished parsing run0000_0000 timepoint 0.95375
 finished parsing run0000_0000 timepoint 0.9538
 finished parsing run0000_0000 timepoint 0.95385
 finished parsing run0000_0000 timepoint 0.9539
 finished parsing run0000_0000 timepoint 0.95395
 finished parsing run0000_0000 timepoint 0.954
 finished parsing run0000_0000 timepoint 0.95405
 finished parsing run0000_0000 timepoint 0.9541
 finished parsing run0000_0000 timepoint 0.95415
 finished parsing run0000_0000 timepoint 0.9542
 finished parsing run0000_0000 timepoint 0.95425
 finished parsing run0000_0000 timepoint 0.9543
 finished parsing run0000_0000 timepoint 0.95435
 finished parsing run0000_0000 timepoint 0.9544
 finished parsing run0000_0000 timepoint 0.95445
 finished parsing run0000_0000 timepoint 0.9545
 finished parsing run0000_0000 timepoint 0.95455
 finished parsing run0000_0000 timepoint 0.9546
 finished parsing run0000_0000 timepoint 0.95465
 finished parsing run0000_0000 timepoint 0.9547
 finished parsing run0000_0000 timepoint 0.95475
 finished parsing run0000_0000 timepoint 0.9548
 finished parsing run0000_0000 timepoint 0.95485
 finished parsing run0000_0000 timepoint 0.9549
 finished parsing run0000_0000 timepoint 0.95495
 finished parsing run0000_0000 timepoint 0.955
 finished parsing run0000_0000 timepoint 0.95505
 finished parsing run0000_0000 timepoint 0.9551
 finished parsing run0000_0000 timepoint 0.95515
 finished parsing run0000_0000 timepoint 0.9552
 finished parsing run0000_0000 timepoint 0.95525
 finished parsing run0000_0000 timepoint 0.9553
 finished parsing run0000_0000 timepoint 0.95535
 finished parsing run0000_0000 timepoint 0.9554
 finished parsing run0000_0000 timepoint 0.95545
 finished parsing run0000_0000 timepoint 0.9555
 finished parsing run0000_0000 timepoint 0.95555
 finished parsing run0000_0000 timepoint 0.9556
 finished parsing run0000_0000 timepoint 0.95565
 finished parsing run0000_0000 timepoint 0.9557
 finished parsing run0000_0000 timepoint 0.95575
 finished parsing run0000_0000 timepoint 0.9558
 finished parsing run0000_0000 timepoint 0.95585
 finished parsing run0000_0000 timepoint 0.9559
 finished parsing run0000_0000 timepoint 0.95595
 finished parsing run0000_0000 timepoint 0.956
 finished parsing run0000_0000 timepoint 0.95605
 finished parsing run0000_0000 timepoint 0.9561
 finished parsing run0000_0000 timepoint 0.95615
 finished parsing run0000_0000 timepoint 0.9562
 finished parsing run0000_0000 timepoint 0.95625
 finished parsing run0000_0000 timepoint 0.9563
 finished parsing run0000_0000 timepoint 0.95635
 finished parsing run0000_0000 timepoint 0.9564
 finished parsing run0000_0000 timepoint 0.95645
 finished parsing run0000_0000 timepoint 0.9565
 finished parsing run0000_0000 timepoint 0.95655
 finished parsing run0000_0000 timepoint 0.9566
 finished parsing run0000_0000 timepoint 0.95665
 finished parsing run0000_0000 timepoint 0.9567
 finished parsing run0000_0000 timepoint 0.95675
 finished parsing run0000_0000 timepoint 0.9568
 finished parsing run0000_0000 timepoint 0.95685
 finished parsing run0000_0000 timepoint 0.9569
 finished parsing run0000_0000 timepoint 0.95695
 finished parsing run0000_0000 timepoint 0.957
 finished parsing run0000_0000 timepoint 0.95705
 finished parsing run0000_0000 timepoint 0.9571
 finished parsing run0000_0000 timepoint 0.95715
 finished parsing run0000_0000 timepoint 0.9572
 finished parsing run0000_0000 timepoint 0.95725
 finished parsing run0000_0000 timepoint 0.9573
 finished parsing run0000_0000 timepoint 0.95735
 finished parsing run0000_0000 timepoint 0.9574
 finished parsing run0000_0000 timepoint 0.95745
 finished parsing run0000_0000 timepoint 0.9575
 finished parsing run0000_0000 timepoint 0.95755
 finished parsing run0000_0000 timepoint 0.9576
 finished parsing run0000_0000 timepoint 0.95765
 finished parsing run0000_0000 timepoint 0.9577
 finished parsing run0000_0000 timepoint 0.95775
 finished parsing run0000_0000 timepoint 0.9578
 finished parsing run0000_0000 timepoint 0.95785
 finished parsing run0000_0000 timepoint 0.9579
 finished parsing run0000_0000 timepoint 0.95795
 finished parsing run0000_0000 timepoint 0.958
 finished parsing run0000_0000 timepoint 0.95805
 finished parsing run0000_0000 timepoint 0.9581
 finished parsing run0000_0000 timepoint 0.95815
 finished parsing run0000_0000 timepoint 0.9582
 finished parsing run0000_0000 timepoint 0.95825
 finished parsing run0000_0000 timepoint 0.9583
 finished parsing run0000_0000 timepoint 0.95835
 finished parsing run0000_0000 timepoint 0.9584
 finished parsing run0000_0000 timepoint 0.95845
 finished parsing run0000_0000 timepoint 0.9585
 finished parsing run0000_0000 timepoint 0.95855
 finished parsing run0000_0000 timepoint 0.9586
 finished parsing run0000_0000 timepoint 0.95865
 finished parsing run0000_0000 timepoint 0.9587
 finished parsing run0000_0000 timepoint 0.95875
 finished parsing run0000_0000 timepoint 0.9588
 finished parsing run0000_0000 timepoint 0.95885
 finished parsing run0000_0000 timepoint 0.9589
 finished parsing run0000_0000 timepoint 0.95895
 finished parsing run0000_0000 timepoint 0.959
 finished parsing run0000_0000 timepoint 0.95905
 finished parsing run0000_0000 timepoint 0.9591
 finished parsing run0000_0000 timepoint 0.95915
 finished parsing run0000_0000 timepoint 0.9592
 finished parsing run0000_0000 timepoint 0.95925
 finished parsing run0000_0000 timepoint 0.9593
 finished parsing run0000_0000 timepoint 0.95935
 finished parsing run0000_0000 timepoint 0.9594
 finished parsing run0000_0000 timepoint 0.95945
 finished parsing run0000_0000 timepoint 0.9595
 finished parsing run0000_0000 timepoint 0.95955
 finished parsing run0000_0000 timepoint 0.9596
 finished parsing run0000_0000 timepoint 0.95965
 finished parsing run0000_0000 timepoint 0.9597
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.94605
 finished parsing run0000_0000 timepoint 0.9461
 finished parsing run0000_0000 timepoint 0.94615
 finished parsing run0000_0000 timepoint 0.9462
 finished parsing run0000_0000 timepoint 0.94625
 finished parsing run0000_0000 timepoint 0.9463
 finished parsing run0000_0000 timepoint 0.94635
 finished parsing run0000_0000 timepoint 0.9464
 finished parsing run0000_0000 timepoint 0.94645
 finished parsing run0000_0000 timepoint 0.9465
 finished parsing run0000_0000 timepoint 0.94655
 finished parsing run0000_0000 timepoint 0.9466
 finished parsing run0000_0000 timepoint 0.94665
 finished parsing run0000_0000 timepoint 0.9467
 finished parsing run0000_0000 timepoint 0.94675
 finished parsing run0000_0000 timepoint 0.9468
 finished parsing run0000_0000 timepoint 0.94685
 finished parsing run0000_0000 timepoint 0.9469
 finished parsing run0000_0000 timepoint 0.94695
 finished parsing run0000_0000 timepoint 0.947
 finished parsing run0000_0000 timepoint 0.94705
 finished parsing run0000_0000 timepoint 0.9471
 finished parsing run0000_0000 timepoint 0.94715
 finished parsing run0000_0000 timepoint 0.9472
 finished parsing run0000_0000 timepoint 0.94725
 finished parsing run0000_0000 timepoint 0.9473
 finished parsing run0000_0000 timepoint 0.94735
 finished parsing run0000_0000 timepoint 0.9474
 finished parsing run0000_0000 timepoint 0.94745
 finished parsing run0000_0000 timepoint 0.9475
 finished parsing run0000_0000 timepoint 0.94755
 finished parsing run0000_0000 timepoint 0.9476
 finished parsing run0000_0000 timepoint 0.94765
 finished parsing run0000_0000 timepoint 0.9477
 finished parsing run0000_0000 timepoint 0.94775
 finished parsing run0000_0000 timepoint 0.9478
 finished parsing run0000_0000 timepoint 0.94785
 finished parsing run0000_0000 timepoint 0.9479
 finished parsing run0000_0000 timepoint 0.94795
 finished parsing run0000_0000 timepoint 0.948
 finished parsing run0000_0000 timepoint 0.94805
 finished parsing run0000_0000 timepoint 0.9481
 finished parsing run0000_0000 timepoint 0.94815
 finished parsing run0000_0000 timepoint 0.9482
 finished parsing run0000_0000 timepoint 0.94825
 finished parsing run0000_0000 timepoint 0.9483
 finished parsing run0000_0000 timepoint 0.94835
 finished parsing run0000_0000 timepoint 0.9484
 finished parsing run0000_0000 timepoint 0.94845
 finished parsing run0000_0000 timepoint 0.9485
 finished parsing run0000_0000 timepoint 0.94855
 finished parsing run0000_0000 timepoint 0.9486
 finished parsing run0000_0000 timepoint 0.94865
 finished parsing run0000_0000 timepoint 0.9487
 finished parsing run0000_0000 timepoint 0.94875
 finished parsing run0000_0000 timepoint 0.9488
 finished parsing run0000_0000 timepoint 0.94885
 finished parsing run0000_0000 timepoint 0.9489
 finished parsing run0000_0000 timepoint 0.94895
 finished parsing run0000_0000 timepoint 0.949
 finished parsing run0000_0000 timepoint 0.94905
 finished parsing run0000_0000 timepoint 0.9491
 finished parsing run0000_0000 timepoint 0.94915
 finished parsing run0000_0000 timepoint 0.9492
 finished parsing run0000_0000 timepoint 0.94925
 finished parsing run0000_0000 timepoint 0.9493
 finished parsing run0000_0000 timepoint 0.94935
 finished parsing run0000_0000 timepoint 0.9494
 finished parsing run0000_0000 timepoint 0.94945
 finished parsing run0000_0000 timepoint 0.9495
 finished parsing run0000_0000 timepoint 0.94955
 finished parsing run0000_0000 timepoint 0.9496
 finished parsing run0000_0000 timepoint 0.94965
 finished parsing run0000_0000 timepoint 0.9497
 finished parsing run0000_0000 timepoint 0.94975
 finished parsing run0000_0000 timepoint 0.9498
 finished parsing run0000_0000 timepoint 0.94985
 finished parsing run0000_0000 timepoint 0.9499
 finished parsing run0000_0000 timepoint 0.94995
 finished parsing run0000_0000 timepoint 0.95
 finished parsing run0000_0000 timepoint 0.95005
 finished parsing run0000_0000 timepoint 0.9501
 finished parsing run0000_0000 timepoint 0.95015
 finished parsing run0000_0000 timepoint 0.9502
 finished parsing run0000_0000 timepoint 0.95025
 finished parsing run0000_0000 timepoint 0.9503
 finished parsing run0000_0000 timepoint 0.95035
 finished parsing run0000_0000 timepoint 0.9504
 finished parsing run0000_0000 timepoint 0.95045
 finished parsing run0000_0000 timepoint 0.9505
 finished parsing run0000_0000 timepoint 0.95055
 finished parsing run0000_0000 timepoint 0.9506
 finished parsing run0000_0000 timepoint 0.95065
 finished parsing run0000_0000 timepoint 0.9507
 finished parsing run0000_0000 timepoint 0.95075
 finished parsing run0000_0000 timepoint 0.9508
 finished parsing run0000_0000 timepoint 0.95085
 finished parsing run0000_0000 timepoint 0.9509
 finished parsing run0000_0000 timepoint 0.95095
 finished parsing run0000_0000 timepoint 0.951
 finished parsing run0000_0000 timepoint 0.95105
 finished parsing run0000_0000 timepoint 0.9511
 finished parsing run0000_0000 timepoint 0.95115
 finished parsing run0000_0000 timepoint 0.9512
 finished parsing run0000_0000 timepoint 0.95125
 finished parsing run0000_0000 timepoint 0.9513
 finished parsing run0000_0000 timepoint 0.95135
 finished parsing run0000_0000 timepoint 0.9514
 finished parsing run0000_0000 timepoint 0.95145
 finished parsing run0000_0000 timepoint 0.9515
 finished parsing run0000_0000 timepoint 0.95155
 finished parsing run0000_0000 timepoint 0.9516
 finished parsing run0000_0000 timepoint 0.95165
 finished parsing run0000_0000 timepoint 0.9517
 finished parsing run0000_0000 timepoint 0.95175
 finished parsing run0000_0000 timepoint 0.9518
 finished parsing run0000_0000 timepoint 0.95185
 finished parsing run0000_0000 timepoint 0.9519
 finished parsing run0000_0000 timepoint 0.95195
 finished parsing run0000_0000 timepoint 0.952
 finished parsing run0000_0000 timepoint 0.95205
 finished parsing run0000_0000 timepoint 0.9521
 finished parsing run0000_0000 timepoint 0.95215
 finished parsing run0000_0000 timepoint 0.9522
 finished parsing run0000_0000 timepoint 0.95225
 finished parsing run0000_0000 timepoint 0.9523
 finished parsing run0000_0000 timepoint 0.95235
 finished parsing run0000_0000 timepoint 0.9524
 finished parsing run0000_0000 timepoint 0.95245
 finished parsing run0000_0000 timepoint 0.9525
 finished parsing run0000_0000 timepoint 0.95255
 finished parsing run0000_0000 timepoint 0.9526
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9393
 finished parsing run0000_0000 timepoint 0.93935
 finished parsing run0000_0000 timepoint 0.9394
 finished parsing run0000_0000 timepoint 0.93945
 finished parsing run0000_0000 timepoint 0.9395
 finished parsing run0000_0000 timepoint 0.93955
 finished parsing run0000_0000 timepoint 0.9396
 finished parsing run0000_0000 timepoint 0.93965
 finished parsing run0000_0000 timepoint 0.9397
 finished parsing run0000_0000 timepoint 0.93975
 finished parsing run0000_0000 timepoint 0.9398
 finished parsing run0000_0000 timepoint 0.93985
 finished parsing run0000_0000 timepoint 0.9399
 finished parsing run0000_0000 timepoint 0.93995
 finished parsing run0000_0000 timepoint 0.94
 finished parsing run0000_0000 timepoint 0.94005
 finished parsing run0000_0000 timepoint 0.9401
 finished parsing run0000_0000 timepoint 0.94015
 finished parsing run0000_0000 timepoint 0.9402
 finished parsing run0000_0000 timepoint 0.94025
 finished parsing run0000_0000 timepoint 0.9403
 finished parsing run0000_0000 timepoint 0.94035
 finished parsing run0000_0000 timepoint 0.9404
 finished parsing run0000_0000 timepoint 0.94045
 finished parsing run0000_0000 timepoint 0.9405
 finished parsing run0000_0000 timepoint 0.94055
 finished parsing run0000_0000 timepoint 0.9406
 finished parsing run0000_0000 timepoint 0.94065
 finished parsing run0000_0000 timepoint 0.9407
 finished parsing run0000_0000 timepoint 0.94075
 finished parsing run0000_0000 timepoint 0.9408
 finished parsing run0000_0000 timepoint 0.94085
 finished parsing run0000_0000 timepoint 0.9409
 finished parsing run0000_0000 timepoint 0.94095
 finished parsing run0000_0000 timepoint 0.941
 finished parsing run0000_0000 timepoint 0.94105
 finished parsing run0000_0000 timepoint 0.9411
 finished parsing run0000_0000 timepoint 0.94115
 finished parsing run0000_0000 timepoint 0.9412
 finished parsing run0000_0000 timepoint 0.94125
 finished parsing run0000_0000 timepoint 0.9413
 finished parsing run0000_0000 timepoint 0.94135
 finished parsing run0000_0000 timepoint 0.9414
 finished parsing run0000_0000 timepoint 0.94145
 finished parsing run0000_0000 timepoint 0.9415
 finished parsing run0000_0000 timepoint 0.94155
 finished parsing run0000_0000 timepoint 0.9416
 finished parsing run0000_0000 timepoint 0.94165
 finished parsing run0000_0000 timepoint 0.9417
 finished parsing run0000_0000 timepoint 0.94175
 finished parsing run0000_0000 timepoint 0.9418
 finished parsing run0000_0000 timepoint 0.94185
 finished parsing run0000_0000 timepoint 0.9419
 finished parsing run0000_0000 timepoint 0.94195
 finished parsing run0000_0000 timepoint 0.942
 finished parsing run0000_0000 timepoint 0.94205
 finished parsing run0000_0000 timepoint 0.9421
 finished parsing run0000_0000 timepoint 0.94215
 finished parsing run0000_0000 timepoint 0.9422
 finished parsing run0000_0000 timepoint 0.94225
 finished parsing run0000_0000 timepoint 0.9423
 finished parsing run0000_0000 timepoint 0.94235
 finished parsing run0000_0000 timepoint 0.9424
 finished parsing run0000_0000 timepoint 0.94245
 finished parsing run0000_0000 timepoint 0.9425
 finished parsing run0000_0000 timepoint 0.94255
 finished parsing run0000_0000 timepoint 0.9426
 finished parsing run0000_0000 timepoint 0.94265
 finished parsing run0000_0000 timepoint 0.9427
 finished parsing run0000_0000 timepoint 0.94275
 finished parsing run0000_0000 timepoint 0.9428
 finished parsing run0000_0000 timepoint 0.94285
 finished parsing run0000_0000 timepoint 0.9429
 finished parsing run0000_0000 timepoint 0.94295
 finished parsing run0000_0000 timepoint 0.943
 finished parsing run0000_0000 timepoint 0.94305
 finished parsing run0000_0000 timepoint 0.9431
 finished parsing run0000_0000 timepoint 0.94315
 finished parsing run0000_0000 timepoint 0.9432
 finished parsing run0000_0000 timepoint 0.94325
 finished parsing run0000_0000 timepoint 0.9433
 finished parsing run0000_0000 timepoint 0.94335
 finished parsing run0000_0000 timepoint 0.9434
 finished parsing run0000_0000 timepoint 0.94345
 finished parsing run0000_0000 timepoint 0.9435
 finished parsing run0000_0000 timepoint 0.94355
 finished parsing run0000_0000 timepoint 0.9436
 finished parsing run0000_0000 timepoint 0.94365
 finished parsing run0000_0000 timepoint 0.9437
 finished parsing run0000_0000 timepoint 0.94375
 finished parsing run0000_0000 timepoint 0.9438
 finished parsing run0000_0000 timepoint 0.94385
 finished parsing run0000_0000 timepoint 0.9439
 finished parsing run0000_0000 timepoint 0.94395
 finished parsing run0000_0000 timepoint 0.944
 finished parsing run0000_0000 timepoint 0.94405
 finished parsing run0000_0000 timepoint 0.9441
 finished parsing run0000_0000 timepoint 0.94415
 finished parsing run0000_0000 timepoint 0.9442
 finished parsing run0000_0000 timepoint 0.94425
 finished parsing run0000_0000 timepoint 0.9443
 finished parsing run0000_0000 timepoint 0.94435
 finished parsing run0000_0000 timepoint 0.9444
 finished parsing run0000_0000 timepoint 0.94445
 finished parsing run0000_0000 timepoint 0.9445
 finished parsing run0000_0000 timepoint 0.94455
 finished parsing run0000_0000 timepoint 0.9446
 finished parsing run0000_0000 timepoint 0.94465
 finished parsing run0000_0000 timepoint 0.9447
 finished parsing run0000_0000 timepoint 0.94475
 finished parsing run0000_0000 timepoint 0.9448
 finished parsing run0000_0000 timepoint 0.94485
 finished parsing run0000_0000 timepoint 0.9449
 finished parsing run0000_0000 timepoint 0.94495
 finished parsing run0000_0000 timepoint 0.945
 finished parsing run0000_0000 timepoint 0.94505
 finished parsing run0000_0000 timepoint 0.9451
 finished parsing run0000_0000 timepoint 0.94515
 finished parsing run0000_0000 timepoint 0.9452
 finished parsing run0000_0000 timepoint 0.94525
 finished parsing run0000_0000 timepoint 0.9453
 finished parsing run0000_0000 timepoint 0.94535
 finished parsing run0000_0000 timepoint 0.9454
 finished parsing run0000_0000 timepoint 0.94545
 finished parsing run0000_0000 timepoint 0.9455
 finished parsing run0000_0000 timepoint 0.94555
 finished parsing run0000_0000 timepoint 0.9456
 finished parsing run0000_0000 timepoint 0.94565
 finished parsing run0000_0000 timepoint 0.9457
 finished parsing run0000_0000 timepoint 0.94575
 finished parsing run0000_0000 timepoint 0.9458
 finished parsing run0000_0000 timepoint 0.94585
 finished parsing run0000_0000 timepoint 0.9459
 finished parsing run0000_0000 timepoint 0.94595
 finished parsing run0000_0000 timepoint 0.946
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9328
 finished parsing run0000_0000 timepoint 0.93285
 finished parsing run0000_0000 timepoint 0.9329
 finished parsing run0000_0000 timepoint 0.93295
 finished parsing run0000_0000 timepoint 0.933
 finished parsing run0000_0000 timepoint 0.93305
 finished parsing run0000_0000 timepoint 0.9331
 finished parsing run0000_0000 timepoint 0.93315
 finished parsing run0000_0000 timepoint 0.9332
 finished parsing run0000_0000 timepoint 0.93325
 finished parsing run0000_0000 timepoint 0.9333
 finished parsing run0000_0000 timepoint 0.93335
 finished parsing run0000_0000 timepoint 0.9334
 finished parsing run0000_0000 timepoint 0.93345
 finished parsing run0000_0000 timepoint 0.9335
 finished parsing run0000_0000 timepoint 0.93355
 finished parsing run0000_0000 timepoint 0.9336
 finished parsing run0000_0000 timepoint 0.93365
 finished parsing run0000_0000 timepoint 0.9337
 finished parsing run0000_0000 timepoint 0.93375
 finished parsing run0000_0000 timepoint 0.9338
 finished parsing run0000_0000 timepoint 0.93385
 finished parsing run0000_0000 timepoint 0.9339
 finished parsing run0000_0000 timepoint 0.93395
 finished parsing run0000_0000 timepoint 0.934
 finished parsing run0000_0000 timepoint 0.93405
 finished parsing run0000_0000 timepoint 0.9341
 finished parsing run0000_0000 timepoint 0.93415
 finished parsing run0000_0000 timepoint 0.9342
 finished parsing run0000_0000 timepoint 0.93425
 finished parsing run0000_0000 timepoint 0.9343
 finished parsing run0000_0000 timepoint 0.93435
 finished parsing run0000_0000 timepoint 0.9344
 finished parsing run0000_0000 timepoint 0.93445
 finished parsing run0000_0000 timepoint 0.9345
 finished parsing run0000_0000 timepoint 0.93455
 finished parsing run0000_0000 timepoint 0.9346
 finished parsing run0000_0000 timepoint 0.93465
 finished parsing run0000_0000 timepoint 0.9347
 finished parsing run0000_0000 timepoint 0.93475
 finished parsing run0000_0000 timepoint 0.9348
 finished parsing run0000_0000 timepoint 0.93485
 finished parsing run0000_0000 timepoint 0.9349
 finished parsing run0000_0000 timepoint 0.93495
 finished parsing run0000_0000 timepoint 0.935
 finished parsing run0000_0000 timepoint 0.93505
 finished parsing run0000_0000 timepoint 0.9351
 finished parsing run0000_0000 timepoint 0.93515
 finished parsing run0000_0000 timepoint 0.9352
 finished parsing run0000_0000 timepoint 0.93525
 finished parsing run0000_0000 timepoint 0.9353
 finished parsing run0000_0000 timepoint 0.93535
 finished parsing run0000_0000 timepoint 0.9354
 finished parsing run0000_0000 timepoint 0.93545
 finished parsing run0000_0000 timepoint 0.9355
 finished parsing run0000_0000 timepoint 0.93555
 finished parsing run0000_0000 timepoint 0.9356
 finished parsing run0000_0000 timepoint 0.93565
 finished parsing run0000_0000 timepoint 0.9357
 finished parsing run0000_0000 timepoint 0.93575
 finished parsing run0000_0000 timepoint 0.9358
 finished parsing run0000_0000 timepoint 0.93585
 finished parsing run0000_0000 timepoint 0.9359
 finished parsing run0000_0000 timepoint 0.93595
 finished parsing run0000_0000 timepoint 0.936
 finished parsing run0000_0000 timepoint 0.93605
 finished parsing run0000_0000 timepoint 0.9361
 finished parsing run0000_0000 timepoint 0.93615
 finished parsing run0000_0000 timepoint 0.9362
 finished parsing run0000_0000 timepoint 0.93625
 finished parsing run0000_0000 timepoint 0.9363
 finished parsing run0000_0000 timepoint 0.93635
 finished parsing run0000_0000 timepoint 0.9364
 finished parsing run0000_0000 timepoint 0.93645
 finished parsing run0000_0000 timepoint 0.9365
 finished parsing run0000_0000 timepoint 0.93655
 finished parsing run0000_0000 timepoint 0.9366
 finished parsing run0000_0000 timepoint 0.93665
 finished parsing run0000_0000 timepoint 0.9367
 finished parsing run0000_0000 timepoint 0.93675
 finished parsing run0000_0000 timepoint 0.9368
 finished parsing run0000_0000 timepoint 0.93685
 finished parsing run0000_0000 timepoint 0.9369
 finished parsing run0000_0000 timepoint 0.93695
 finished parsing run0000_0000 timepoint 0.937
 finished parsing run0000_0000 timepoint 0.93705
 finished parsing run0000_0000 timepoint 0.9371
 finished parsing run0000_0000 timepoint 0.93715
 finished parsing run0000_0000 timepoint 0.9372
 finished parsing run0000_0000 timepoint 0.93725
 finished parsing run0000_0000 timepoint 0.9373
 finished parsing run0000_0000 timepoint 0.93735
 finished parsing run0000_0000 timepoint 0.9374
 finished parsing run0000_0000 timepoint 0.93745
 finished parsing run0000_0000 timepoint 0.9375
 finished parsing run0000_0000 timepoint 0.93755
 finished parsing run0000_0000 timepoint 0.9376
 finished parsing run0000_0000 timepoint 0.93765
 finished parsing run0000_0000 timepoint 0.9377
 finished parsing run0000_0000 timepoint 0.93775
 finished parsing run0000_0000 timepoint 0.9378
 finished parsing run0000_0000 timepoint 0.93785
 finished parsing run0000_0000 timepoint 0.9379
 finished parsing run0000_0000 timepoint 0.93795
 finished parsing run0000_0000 timepoint 0.938
 finished parsing run0000_0000 timepoint 0.93805
 finished parsing run0000_0000 timepoint 0.9381
 finished parsing run0000_0000 timepoint 0.93815
 finished parsing run0000_0000 timepoint 0.9382
 finished parsing run0000_0000 timepoint 0.93825
 finished parsing run0000_0000 timepoint 0.9383
 finished parsing run0000_0000 timepoint 0.93835
 finished parsing run0000_0000 timepoint 0.9384
 finished parsing run0000_0000 timepoint 0.93845
 finished parsing run0000_0000 timepoint 0.9385
 finished parsing run0000_0000 timepoint 0.93855
 finished parsing run0000_0000 timepoint 0.9386
 finished parsing run0000_0000 timepoint 0.93865
 finished parsing run0000_0000 timepoint 0.9387
 finished parsing run0000_0000 timepoint 0.93875
 finished parsing run0000_0000 timepoint 0.9388
 finished parsing run0000_0000 timepoint 0.93885
 finished parsing run0000_0000 timepoint 0.9389
 finished parsing run0000_0000 timepoint 0.93895
 finished parsing run0000_0000 timepoint 0.939
 finished parsing run0000_0000 timepoint 0.93905
 finished parsing run0000_0000 timepoint 0.9391
 finished parsing run0000_0000 timepoint 0.93915
 finished parsing run0000_0000 timepoint 0.9392
 finished parsing run0000_0000 timepoint 0.93925
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9264
 finished parsing run0000_0000 timepoint 0.92645
 finished parsing run0000_0000 timepoint 0.9265
 finished parsing run0000_0000 timepoint 0.92655
 finished parsing run0000_0000 timepoint 0.9266
 finished parsing run0000_0000 timepoint 0.92665
 finished parsing run0000_0000 timepoint 0.9267
 finished parsing run0000_0000 timepoint 0.92675
 finished parsing run0000_0000 timepoint 0.9268
 finished parsing run0000_0000 timepoint 0.92685
 finished parsing run0000_0000 timepoint 0.9269
 finished parsing run0000_0000 timepoint 0.92695
 finished parsing run0000_0000 timepoint 0.927
 finished parsing run0000_0000 timepoint 0.92705
 finished parsing run0000_0000 timepoint 0.9271
 finished parsing run0000_0000 timepoint 0.92715
 finished parsing run0000_0000 timepoint 0.9272
 finished parsing run0000_0000 timepoint 0.92725
 finished parsing run0000_0000 timepoint 0.9273
 finished parsing run0000_0000 timepoint 0.92735
 finished parsing run0000_0000 timepoint 0.9274
 finished parsing run0000_0000 timepoint 0.92745
 finished parsing run0000_0000 timepoint 0.9275
 finished parsing run0000_0000 timepoint 0.92755
 finished parsing run0000_0000 timepoint 0.9276
 finished parsing run0000_0000 timepoint 0.92765
 finished parsing run0000_0000 timepoint 0.9277
 finished parsing run0000_0000 timepoint 0.92775
 finished parsing run0000_0000 timepoint 0.9278
 finished parsing run0000_0000 timepoint 0.92785
 finished parsing run0000_0000 timepoint 0.9279
 finished parsing run0000_0000 timepoint 0.92795
 finished parsing run0000_0000 timepoint 0.928
 finished parsing run0000_0000 timepoint 0.92805
 finished parsing run0000_0000 timepoint 0.9281
 finished parsing run0000_0000 timepoint 0.92815
 finished parsing run0000_0000 timepoint 0.9282
 finished parsing run0000_0000 timepoint 0.92825
 finished parsing run0000_0000 timepoint 0.9283
 finished parsing run0000_0000 timepoint 0.92835
 finished parsing run0000_0000 timepoint 0.9284
 finished parsing run0000_0000 timepoint 0.92845
 finished parsing run0000_0000 timepoint 0.9285
 finished parsing run0000_0000 timepoint 0.92855
 finished parsing run0000_0000 timepoint 0.9286
 finished parsing run0000_0000 timepoint 0.92865
 finished parsing run0000_0000 timepoint 0.9287
 finished parsing run0000_0000 timepoint 0.92875
 finished parsing run0000_0000 timepoint 0.9288
 finished parsing run0000_0000 timepoint 0.92885
 finished parsing run0000_0000 timepoint 0.9289
 finished parsing run0000_0000 timepoint 0.92895
 finished parsing run0000_0000 timepoint 0.929
 finished parsing run0000_0000 timepoint 0.92905
 finished parsing run0000_0000 timepoint 0.9291
 finished parsing run0000_0000 timepoint 0.92915
 finished parsing run0000_0000 timepoint 0.9292
 finished parsing run0000_0000 timepoint 0.92925
 finished parsing run0000_0000 timepoint 0.9293
 finished parsing run0000_0000 timepoint 0.92935
 finished parsing run0000_0000 timepoint 0.9294
 finished parsing run0000_0000 timepoint 0.92945
 finished parsing run0000_0000 timepoint 0.9295
 finished parsing run0000_0000 timepoint 0.92955
 finished parsing run0000_0000 timepoint 0.9296
 finished parsing run0000_0000 timepoint 0.92965
 finished parsing run0000_0000 timepoint 0.9297
 finished parsing run0000_0000 timepoint 0.92975
 finished parsing run0000_0000 timepoint 0.9298
 finished parsing run0000_0000 timepoint 0.92985
 finished parsing run0000_0000 timepoint 0.9299
 finished parsing run0000_0000 timepoint 0.92995
 finished parsing run0000_0000 timepoint 0.93
 finished parsing run0000_0000 timepoint 0.93005
 finished parsing run0000_0000 timepoint 0.9301
 finished parsing run0000_0000 timepoint 0.93015
 finished parsing run0000_0000 timepoint 0.9302
 finished parsing run0000_0000 timepoint 0.93025
 finished parsing run0000_0000 timepoint 0.9303
 finished parsing run0000_0000 timepoint 0.93035
 finished parsing run0000_0000 timepoint 0.9304
 finished parsing run0000_0000 timepoint 0.93045
 finished parsing run0000_0000 timepoint 0.9305
 finished parsing run0000_0000 timepoint 0.93055
 finished parsing run0000_0000 timepoint 0.9306
 finished parsing run0000_0000 timepoint 0.93065
 finished parsing run0000_0000 timepoint 0.9307
 finished parsing run0000_0000 timepoint 0.93075
 finished parsing run0000_0000 timepoint 0.9308
 finished parsing run0000_0000 timepoint 0.93085
 finished parsing run0000_0000 timepoint 0.9309
 finished parsing run0000_0000 timepoint 0.93095
 finished parsing run0000_0000 timepoint 0.931
 finished parsing run0000_0000 timepoint 0.93105
 finished parsing run0000_0000 timepoint 0.9311
 finished parsing run0000_0000 timepoint 0.93115
 finished parsing run0000_0000 timepoint 0.9312
 finished parsing run0000_0000 timepoint 0.93125
 finished parsing run0000_0000 timepoint 0.9313
 finished parsing run0000_0000 timepoint 0.93135
 finished parsing run0000_0000 timepoint 0.9314
 finished parsing run0000_0000 timepoint 0.93145
 finished parsing run0000_0000 timepoint 0.9315
 finished parsing run0000_0000 timepoint 0.93155
 finished parsing run0000_0000 timepoint 0.9316
 finished parsing run0000_0000 timepoint 0.93165
 finished parsing run0000_0000 timepoint 0.9317
 finished parsing run0000_0000 timepoint 0.93175
 finished parsing run0000_0000 timepoint 0.9318
 finished parsing run0000_0000 timepoint 0.93185
 finished parsing run0000_0000 timepoint 0.9319
 finished parsing run0000_0000 timepoint 0.93195
 finished parsing run0000_0000 timepoint 0.932
 finished parsing run0000_0000 timepoint 0.93205
 finished parsing run0000_0000 timepoint 0.9321
 finished parsing run0000_0000 timepoint 0.93215
 finished parsing run0000_0000 timepoint 0.9322
 finished parsing run0000_0000 timepoint 0.93225
 finished parsing run0000_0000 timepoint 0.9323
 finished parsing run0000_0000 timepoint 0.93235
 finished parsing run0000_0000 timepoint 0.9324
 finished parsing run0000_0000 timepoint 0.93245
 finished parsing run0000_0000 timepoint 0.9325
 finished parsing run0000_0000 timepoint 0.93255
 finished parsing run0000_0000 timepoint 0.9326
 finished parsing run0000_0000 timepoint 0.93265
 finished parsing run0000_0000 timepoint 0.9327
 finished parsing run0000_0000 timepoint 0.93275
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.9198
 finished parsing run0000_0000 timepoint 0.91985
 finished parsing run0000_0000 timepoint 0.9199
 finished parsing run0000_0000 timepoint 0.91995
 finished parsing run0000_0000 timepoint 0.92
 finished parsing run0000_0000 timepoint 0.92005
 finished parsing run0000_0000 timepoint 0.9201
 finished parsing run0000_0000 timepoint 0.92015
 finished parsing run0000_0000 timepoint 0.9202
 finished parsing run0000_0000 timepoint 0.92025
 finished parsing run0000_0000 timepoint 0.9203
 finished parsing run0000_0000 timepoint 0.92035
 finished parsing run0000_0000 timepoint 0.9204
 finished parsing run0000_0000 timepoint 0.92045
 finished parsing run0000_0000 timepoint 0.9205
 finished parsing run0000_0000 timepoint 0.92055
 finished parsing run0000_0000 timepoint 0.9206
 finished parsing run0000_0000 timepoint 0.92065
 finished parsing run0000_0000 timepoint 0.9207
 finished parsing run0000_0000 timepoint 0.92075
 finished parsing run0000_0000 timepoint 0.9208
 finished parsing run0000_0000 timepoint 0.92085
 finished parsing run0000_0000 timepoint 0.9209
 finished parsing run0000_0000 timepoint 0.92095
 finished parsing run0000_0000 timepoint 0.921
 finished parsing run0000_0000 timepoint 0.92105
 finished parsing run0000_0000 timepoint 0.9211
 finished parsing run0000_0000 timepoint 0.92115
 finished parsing run0000_0000 timepoint 0.9212
 finished parsing run0000_0000 timepoint 0.92125
 finished parsing run0000_0000 timepoint 0.9213
 finished parsing run0000_0000 timepoint 0.92135
 finished parsing run0000_0000 timepoint 0.9214
 finished parsing run0000_0000 timepoint 0.92145
 finished parsing run0000_0000 timepoint 0.9215
 finished parsing run0000_0000 timepoint 0.92155
 finished parsing run0000_0000 timepoint 0.9216
 finished parsing run0000_0000 timepoint 0.92165
 finished parsing run0000_0000 timepoint 0.9217
 finished parsing run0000_0000 timepoint 0.92175
 finished parsing run0000_0000 timepoint 0.9218
 finished parsing run0000_0000 timepoint 0.92185
 finished parsing run0000_0000 timepoint 0.9219
 finished parsing run0000_0000 timepoint 0.92195
 finished parsing run0000_0000 timepoint 0.922
 finished parsing run0000_0000 timepoint 0.92205
 finished parsing run0000_0000 timepoint 0.9221
 finished parsing run0000_0000 timepoint 0.92215
 finished parsing run0000_0000 timepoint 0.9222
 finished parsing run0000_0000 timepoint 0.92225
 finished parsing run0000_0000 timepoint 0.9223
 finished parsing run0000_0000 timepoint 0.92235
 finished parsing run0000_0000 timepoint 0.9224
 finished parsing run0000_0000 timepoint 0.92245
 finished parsing run0000_0000 timepoint 0.9225
 finished parsing run0000_0000 timepoint 0.92255
 finished parsing run0000_0000 timepoint 0.9226
 finished parsing run0000_0000 timepoint 0.92265
 finished parsing run0000_0000 timepoint 0.9227
 finished parsing run0000_0000 timepoint 0.92275
 finished parsing run0000_0000 timepoint 0.9228
 finished parsing run0000_0000 timepoint 0.92285
 finished parsing run0000_0000 timepoint 0.9229
 finished parsing run0000_0000 timepoint 0.92295
 finished parsing run0000_0000 timepoint 0.923
 finished parsing run0000_0000 timepoint 0.92305
 finished parsing run0000_0000 timepoint 0.9231
 finished parsing run0000_0000 timepoint 0.92315
 finished parsing run0000_0000 timepoint 0.9232
 finished parsing run0000_0000 timepoint 0.92325
 finished parsing run0000_0000 timepoint 0.9233
 finished parsing run0000_0000 timepoint 0.92335
 finished parsing run0000_0000 timepoint 0.9234
 finished parsing run0000_0000 timepoint 0.92345
 finished parsing run0000_0000 timepoint 0.9235
 finished parsing run0000_0000 timepoint 0.92355
 finished parsing run0000_0000 timepoint 0.9236
 finished parsing run0000_0000 timepoint 0.92365
 finished parsing run0000_0000 timepoint 0.9237
 finished parsing run0000_0000 timepoint 0.92375
 finished parsing run0000_0000 timepoint 0.9238
 finished parsing run0000_0000 timepoint 0.92385
 finished parsing run0000_0000 timepoint 0.9239
 finished parsing run0000_0000 timepoint 0.92395
 finished parsing run0000_0000 timepoint 0.924
 finished parsing run0000_0000 timepoint 0.92405
 finished parsing run0000_0000 timepoint 0.9241
 finished parsing run0000_0000 timepoint 0.92415
 finished parsing run0000_0000 timepoint 0.9242
 finished parsing run0000_0000 timepoint 0.92425
 finished parsing run0000_0000 timepoint 0.9243
 finished parsing run0000_0000 timepoint 0.92435
 finished parsing run0000_0000 timepoint 0.9244
 finished parsing run0000_0000 timepoint 0.92445
 finished parsing run0000_0000 timepoint 0.9245
 finished parsing run0000_0000 timepoint 0.92455
 finished parsing run0000_0000 timepoint 0.9246
 finished parsing run0000_0000 timepoint 0.92465
 finished parsing run0000_0000 timepoint 0.9247
 finished parsing run0000_0000 timepoint 0.92475
 finished parsing run0000_0000 timepoint 0.9248
 finished parsing run0000_0000 timepoint 0.92485
 finished parsing run0000_0000 timepoint 0.9249
 finished parsing run0000_0000 timepoint 0.92495
 finished parsing run0000_0000 timepoint 0.925
 finished parsing run0000_0000 timepoint 0.92505
 finished parsing run0000_0000 timepoint 0.9251
 finished parsing run0000_0000 timepoint 0.92515
 finished parsing run0000_0000 timepoint 0.9252
 finished parsing run0000_0000 timepoint 0.92525
 finished parsing run0000_0000 timepoint 0.9253
 finished parsing run0000_0000 timepoint 0.92535
 finished parsing run0000_0000 timepoint 0.9254
 finished parsing run0000_0000 timepoint 0.92545
 finished parsing run0000_0000 timepoint 0.9255
 finished parsing run0000_0000 timepoint 0.92555
 finished parsing run0000_0000 timepoint 0.9256
 finished parsing run0000_0000 timepoint 0.92565
 finished parsing run0000_0000 timepoint 0.9257
 finished parsing run0000_0000 timepoint 0.92575
 finished parsing run0000_0000 timepoint 0.9258
 finished parsing run0000_0000 timepoint 0.92585
 finished parsing run0000_0000 timepoint 0.9259
 finished parsing run0000_0000 timepoint 0.92595
 finished parsing run0000_0000 timepoint 0.926
 finished parsing run0000_0000 timepoint 0.92605
 finished parsing run0000_0000 timepoint 0.9261
 finished parsing run0000_0000 timepoint 0.92615
 finished parsing run0000_0000 timepoint 0.9262
 finished parsing run0000_0000 timepoint 0.92625
 finished parsing run0000_0000 timepoint 0.9263
 finished parsing run0000_0000 timepoint 0.92635
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.91325
 finished parsing run0000_0000 timepoint 0.9133
 finished parsing run0000_0000 timepoint 0.91335
 finished parsing run0000_0000 timepoint 0.9134
 finished parsing run0000_0000 timepoint 0.91345
 finished parsing run0000_0000 timepoint 0.9135
 finished parsing run0000_0000 timepoint 0.91355
 finished parsing run0000_0000 timepoint 0.9136
 finished parsing run0000_0000 timepoint 0.91365
 finished parsing run0000_0000 timepoint 0.9137
 finished parsing run0000_0000 timepoint 0.91375
 finished parsing run0000_0000 timepoint 0.9138
 finished parsing run0000_0000 timepoint 0.91385
 finished parsing run0000_0000 timepoint 0.9139
 finished parsing run0000_0000 timepoint 0.91395
 finished parsing run0000_0000 timepoint 0.914
 finished parsing run0000_0000 timepoint 0.91405
 finished parsing run0000_0000 timepoint 0.9141
 finished parsing run0000_0000 timepoint 0.91415
 finished parsing run0000_0000 timepoint 0.9142
 finished parsing run0000_0000 timepoint 0.91425
 finished parsing run0000_0000 timepoint 0.9143
 finished parsing run0000_0000 timepoint 0.91435
 finished parsing run0000_0000 timepoint 0.9144
 finished parsing run0000_0000 timepoint 0.91445
 finished parsing run0000_0000 timepoint 0.9145
 finished parsing run0000_0000 timepoint 0.91455
 finished parsing run0000_0000 timepoint 0.9146
 finished parsing run0000_0000 timepoint 0.91465
 finished parsing run0000_0000 timepoint 0.9147
 finished parsing run0000_0000 timepoint 0.91475
 finished parsing run0000_0000 timepoint 0.9148
 finished parsing run0000_0000 timepoint 0.91485
 finished parsing run0000_0000 timepoint 0.9149
 finished parsing run0000_0000 timepoint 0.91495
 finished parsing run0000_0000 timepoint 0.915
 finished parsing run0000_0000 timepoint 0.91505
 finished parsing run0000_0000 timepoint 0.9151
 finished parsing run0000_0000 timepoint 0.91515
 finished parsing run0000_0000 timepoint 0.9152
 finished parsing run0000_0000 timepoint 0.91525
 finished parsing run0000_0000 timepoint 0.9153
 finished parsing run0000_0000 timepoint 0.91535
 finished parsing run0000_0000 timepoint 0.9154
 finished parsing run0000_0000 timepoint 0.91545
 finished parsing run0000_0000 timepoint 0.9155
 finished parsing run0000_0000 timepoint 0.91555
 finished parsing run0000_0000 timepoint 0.9156
 finished parsing run0000_0000 timepoint 0.91565
 finished parsing run0000_0000 timepoint 0.9157
 finished parsing run0000_0000 timepoint 0.91575
 finished parsing run0000_0000 timepoint 0.9158
 finished parsing run0000_0000 timepoint 0.91585
 finished parsing run0000_0000 timepoint 0.9159
 finished parsing run0000_0000 timepoint 0.91595
 finished parsing run0000_0000 timepoint 0.916
 finished parsing run0000_0000 timepoint 0.91605
 finished parsing run0000_0000 timepoint 0.9161
 finished parsing run0000_0000 timepoint 0.91615
 finished parsing run0000_0000 timepoint 0.9162
 finished parsing run0000_0000 timepoint 0.91625
 finished parsing run0000_0000 timepoint 0.9163
 finished parsing run0000_0000 timepoint 0.91635
 finished parsing run0000_0000 timepoint 0.9164
 finished parsing run0000_0000 timepoint 0.91645
 finished parsing run0000_0000 timepoint 0.9165
 finished parsing run0000_0000 timepoint 0.91655
 finished parsing run0000_0000 timepoint 0.9166
 finished parsing run0000_0000 timepoint 0.91665
 finished parsing run0000_0000 timepoint 0.9167
 finished parsing run0000_0000 timepoint 0.91675
 finished parsing run0000_0000 timepoint 0.9168
 finished parsing run0000_0000 timepoint 0.91685
 finished parsing run0000_0000 timepoint 0.9169
 finished parsing run0000_0000 timepoint 0.91695
 finished parsing run0000_0000 timepoint 0.917
 finished parsing run0000_0000 timepoint 0.91705
 finished parsing run0000_0000 timepoint 0.9171
 finished parsing run0000_0000 timepoint 0.91715
 finished parsing run0000_0000 timepoint 0.9172
 finished parsing run0000_0000 timepoint 0.91725
 finished parsing run0000_0000 timepoint 0.9173
 finished parsing run0000_0000 timepoint 0.91735
 finished parsing run0000_0000 timepoint 0.9174
 finished parsing run0000_0000 timepoint 0.91745
 finished parsing run0000_0000 timepoint 0.9175
 finished parsing run0000_0000 timepoint 0.91755
 finished parsing run0000_0000 timepoint 0.9176
 finished parsing run0000_0000 timepoint 0.91765
 finished parsing run0000_0000 timepoint 0.9177
 finished parsing run0000_0000 timepoint 0.91775
 finished parsing run0000_0000 timepoint 0.9178
 finished parsing run0000_0000 timepoint 0.91785
 finished parsing run0000_0000 timepoint 0.9179
 finished parsing run0000_0000 timepoint 0.91795
 finished parsing run0000_0000 timepoint 0.918
 finished parsing run0000_0000 timepoint 0.91805
 finished parsing run0000_0000 timepoint 0.9181
 finished parsing run0000_0000 timepoint 0.91815
 finished parsing run0000_0000 timepoint 0.9182
 finished parsing run0000_0000 timepoint 0.91825
 finished parsing run0000_0000 timepoint 0.9183
 finished parsing run0000_0000 timepoint 0.91835
 finished parsing run0000_0000 timepoint 0.9184
 finished parsing run0000_0000 timepoint 0.91845
 finished parsing run0000_0000 timepoint 0.9185
 finished parsing run0000_0000 timepoint 0.91855
 finished parsing run0000_0000 timepoint 0.9186
 finished parsing run0000_0000 timepoint 0.91865
 finished parsing run0000_0000 timepoint 0.9187
 finished parsing run0000_0000 timepoint 0.91875
 finished parsing run0000_0000 timepoint 0.9188
 finished parsing run0000_0000 timepoint 0.91885
 finished parsing run0000_0000 timepoint 0.9189
 finished parsing run0000_0000 timepoint 0.91895
 finished parsing run0000_0000 timepoint 0.919
 finished parsing run0000_0000 timepoint 0.91905
 finished parsing run0000_0000 timepoint 0.9191
 finished parsing run0000_0000 timepoint 0.91915
 finished parsing run0000_0000 timepoint 0.9192
 finished parsing run0000_0000 timepoint 0.91925
 finished parsing run0000_0000 timepoint 0.9193
 finished parsing run0000_0000 timepoint 0.91935
 finished parsing run0000_0000 timepoint 0.9194
 finished parsing run0000_0000 timepoint 0.91945
 finished parsing run0000_0000 timepoint 0.9195
 finished parsing run0000_0000 timepoint 0.91955
 finished parsing run0000_0000 timepoint 0.9196
 finished parsing run0000_0000 timepoint 0.91965
 finished parsing run0000_0000 timepoint 0.9197
 finished parsing run0000_0000 timepoint 0.91975
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.90665
 finished parsing run0000_0000 timepoint 0.9067
 finished parsing run0000_0000 timepoint 0.90675
 finished parsing run0000_0000 timepoint 0.9068
 finished parsing run0000_0000 timepoint 0.90685
 finished parsing run0000_0000 timepoint 0.9069
 finished parsing run0000_0000 timepoint 0.90695
 finished parsing run0000_0000 timepoint 0.907
 finished parsing run0000_0000 timepoint 0.90705
 finished parsing run0000_0000 timepoint 0.9071
 finished parsing run0000_0000 timepoint 0.90715
 finished parsing run0000_0000 timepoint 0.9072
 finished parsing run0000_0000 timepoint 0.90725
 finished parsing run0000_0000 timepoint 0.9073
 finished parsing run0000_0000 timepoint 0.90735
 finished parsing run0000_0000 timepoint 0.9074
 finished parsing run0000_0000 timepoint 0.90745
 finished parsing run0000_0000 timepoint 0.9075
 finished parsing run0000_0000 timepoint 0.90755
 finished parsing run0000_0000 timepoint 0.9076
 finished parsing run0000_0000 timepoint 0.90765
 finished parsing run0000_0000 timepoint 0.9077
 finished parsing run0000_0000 timepoint 0.90775
 finished parsing run0000_0000 timepoint 0.9078
 finished parsing run0000_0000 timepoint 0.90785
 finished parsing run0000_0000 timepoint 0.9079
 finished parsing run0000_0000 timepoint 0.90795
 finished parsing run0000_0000 timepoint 0.908
 finished parsing run0000_0000 timepoint 0.90805
 finished parsing run0000_0000 timepoint 0.9081
 finished parsing run0000_0000 timepoint 0.90815
 finished parsing run0000_0000 timepoint 0.9082
 finished parsing run0000_0000 timepoint 0.90825
 finished parsing run0000_0000 timepoint 0.9083
 finished parsing run0000_0000 timepoint 0.90835
 finished parsing run0000_0000 timepoint 0.9084
 finished parsing run0000_0000 timepoint 0.90845
 finished parsing run0000_0000 timepoint 0.9085
 finished parsing run0000_0000 timepoint 0.90855
 finished parsing run0000_0000 timepoint 0.9086
 finished parsing run0000_0000 timepoint 0.90865
 finished parsing run0000_0000 timepoint 0.9087
 finished parsing run0000_0000 timepoint 0.90875
 finished parsing run0000_0000 timepoint 0.9088
 finished parsing run0000_0000 timepoint 0.90885
 finished parsing run0000_0000 timepoint 0.9089
 finished parsing run0000_0000 timepoint 0.90895
 finished parsing run0000_0000 timepoint 0.909
 finished parsing run0000_0000 timepoint 0.90905
 finished parsing run0000_0000 timepoint 0.9091
 finished parsing run0000_0000 timepoint 0.90915
 finished parsing run0000_0000 timepoint 0.9092
 finished parsing run0000_0000 timepoint 0.90925
 finished parsing run0000_0000 timepoint 0.9093
 finished parsing run0000_0000 timepoint 0.90935
 finished parsing run0000_0000 timepoint 0.9094
 finished parsing run0000_0000 timepoint 0.90945
 finished parsing run0000_0000 timepoint 0.9095
 finished parsing run0000_0000 timepoint 0.90955
 finished parsing run0000_0000 timepoint 0.9096
 finished parsing run0000_0000 timepoint 0.90965
 finished parsing run0000_0000 timepoint 0.9097
 finished parsing run0000_0000 timepoint 0.90975
 finished parsing run0000_0000 timepoint 0.9098
 finished parsing run0000_0000 timepoint 0.90985
 finished parsing run0000_0000 timepoint 0.9099
 finished parsing run0000_0000 timepoint 0.90995
 finished parsing run0000_0000 timepoint 0.91
 finished parsing run0000_0000 timepoint 0.91005
 finished parsing run0000_0000 timepoint 0.9101
 finished parsing run0000_0000 timepoint 0.91015
 finished parsing run0000_0000 timepoint 0.9102
 finished parsing run0000_0000 timepoint 0.91025
 finished parsing run0000_0000 timepoint 0.9103
 finished parsing run0000_0000 timepoint 0.91035
 finished parsing run0000_0000 timepoint 0.9104
 finished parsing run0000_0000 timepoint 0.91045
 finished parsing run0000_0000 timepoint 0.9105
 finished parsing run0000_0000 timepoint 0.91055
 finished parsing run0000_0000 timepoint 0.9106
 finished parsing run0000_0000 timepoint 0.91065
 finished parsing run0000_0000 timepoint 0.9107
 finished parsing run0000_0000 timepoint 0.91075
 finished parsing run0000_0000 timepoint 0.9108
 finished parsing run0000_0000 timepoint 0.91085
 finished parsing run0000_0000 timepoint 0.9109
 finished parsing run0000_0000 timepoint 0.91095
 finished parsing run0000_0000 timepoint 0.911
 finished parsing run0000_0000 timepoint 0.91105
 finished parsing run0000_0000 timepoint 0.9111
 finished parsing run0000_0000 timepoint 0.91115
 finished parsing run0000_0000 timepoint 0.9112
 finished parsing run0000_0000 timepoint 0.91125
 finished parsing run0000_0000 timepoint 0.9113
 finished parsing run0000_0000 timepoint 0.91135
 finished parsing run0000_0000 timepoint 0.9114
 finished parsing run0000_0000 timepoint 0.91145
 finished parsing run0000_0000 timepoint 0.9115
 finished parsing run0000_0000 timepoint 0.91155
 finished parsing run0000_0000 timepoint 0.9116
 finished parsing run0000_0000 timepoint 0.91165
 finished parsing run0000_0000 timepoint 0.9117
 finished parsing run0000_0000 timepoint 0.91175
 finished parsing run0000_0000 timepoint 0.9118
 finished parsing run0000_0000 timepoint 0.91185
 finished parsing run0000_0000 timepoint 0.9119
 finished parsing run0000_0000 timepoint 0.91195
 finished parsing run0000_0000 timepoint 0.912
 finished parsing run0000_0000 timepoint 0.91205
 finished parsing run0000_0000 timepoint 0.9121
 finished parsing run0000_0000 timepoint 0.91215
 finished parsing run0000_0000 timepoint 0.9122
 finished parsing run0000_0000 timepoint 0.91225
 finished parsing run0000_0000 timepoint 0.9123
 finished parsing run0000_0000 timepoint 0.91235
 finished parsing run0000_0000 timepoint 0.9124
 finished parsing run0000_0000 timepoint 0.91245
 finished parsing run0000_0000 timepoint 0.9125
 finished parsing run0000_0000 timepoint 0.91255
 finished parsing run0000_0000 timepoint 0.9126
 finished parsing run0000_0000 timepoint 0.91265
 finished parsing run0000_0000 timepoint 0.9127
 finished parsing run0000_0000 timepoint 0.91275
 finished parsing run0000_0000 timepoint 0.9128
 finished parsing run0000_0000 timepoint 0.91285
 finished parsing run0000_0000 timepoint 0.9129
 finished parsing run0000_0000 timepoint 0.91295
 finished parsing run0000_0000 timepoint 0.913
 finished parsing run0000_0000 timepoint 0.91305
 finished parsing run0000_0000 timepoint 0.9131
 finished parsing run0000_0000 timepoint 0.91315
 finished parsing run0000_0000 timepoint 0.9132
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.90015
 finished parsing run0000_0000 timepoint 0.9002
 finished parsing run0000_0000 timepoint 0.90025
 finished parsing run0000_0000 timepoint 0.9003
 finished parsing run0000_0000 timepoint 0.90035
 finished parsing run0000_0000 timepoint 0.9004
 finished parsing run0000_0000 timepoint 0.90045
 finished parsing run0000_0000 timepoint 0.9005
 finished parsing run0000_0000 timepoint 0.90055
 finished parsing run0000_0000 timepoint 0.9006
 finished parsing run0000_0000 timepoint 0.90065
 finished parsing run0000_0000 timepoint 0.9007
 finished parsing run0000_0000 timepoint 0.90075
 finished parsing run0000_0000 timepoint 0.9008
 finished parsing run0000_0000 timepoint 0.90085
 finished parsing run0000_0000 timepoint 0.9009
 finished parsing run0000_0000 timepoint 0.90095
 finished parsing run0000_0000 timepoint 0.901
 finished parsing run0000_0000 timepoint 0.90105
 finished parsing run0000_0000 timepoint 0.9011
 finished parsing run0000_0000 timepoint 0.90115
 finished parsing run0000_0000 timepoint 0.9012
 finished parsing run0000_0000 timepoint 0.90125
 finished parsing run0000_0000 timepoint 0.9013
 finished parsing run0000_0000 timepoint 0.90135
 finished parsing run0000_0000 timepoint 0.9014
 finished parsing run0000_0000 timepoint 0.90145
 finished parsing run0000_0000 timepoint 0.9015
 finished parsing run0000_0000 timepoint 0.90155
 finished parsing run0000_0000 timepoint 0.9016
 finished parsing run0000_0000 timepoint 0.90165
 finished parsing run0000_0000 timepoint 0.9017
 finished parsing run0000_0000 timepoint 0.90175
 finished parsing run0000_0000 timepoint 0.9018
 finished parsing run0000_0000 timepoint 0.90185
 finished parsing run0000_0000 timepoint 0.9019
 finished parsing run0000_0000 timepoint 0.90195
 finished parsing run0000_0000 timepoint 0.902
 finished parsing run0000_0000 timepoint 0.90205
 finished parsing run0000_0000 timepoint 0.9021
 finished parsing run0000_0000 timepoint 0.90215
 finished parsing run0000_0000 timepoint 0.9022
 finished parsing run0000_0000 timepoint 0.90225
 finished parsing run0000_0000 timepoint 0.9023
 finished parsing run0000_0000 timepoint 0.90235
 finished parsing run0000_0000 timepoint 0.9024
 finished parsing run0000_0000 timepoint 0.90245
 finished parsing run0000_0000 timepoint 0.9025
 finished parsing run0000_0000 timepoint 0.90255
 finished parsing run0000_0000 timepoint 0.9026
 finished parsing run0000_0000 timepoint 0.90265
 finished parsing run0000_0000 timepoint 0.9027
 finished parsing run0000_0000 timepoint 0.90275
 finished parsing run0000_0000 timepoint 0.9028
 finished parsing run0000_0000 timepoint 0.90285
 finished parsing run0000_0000 timepoint 0.9029
 finished parsing run0000_0000 timepoint 0.90295
 finished parsing run0000_0000 timepoint 0.903
 finished parsing run0000_0000 timepoint 0.90305
 finished parsing run0000_0000 timepoint 0.9031
 finished parsing run0000_0000 timepoint 0.90315
 finished parsing run0000_0000 timepoint 0.9032
 finished parsing run0000_0000 timepoint 0.90325
 finished parsing run0000_0000 timepoint 0.9033
 finished parsing run0000_0000 timepoint 0.90335
 finished parsing run0000_0000 timepoint 0.9034
 finished parsing run0000_0000 timepoint 0.90345
 finished parsing run0000_0000 timepoint 0.9035
 finished parsing run0000_0000 timepoint 0.90355
 finished parsing run0000_0000 timepoint 0.9036
 finished parsing run0000_0000 timepoint 0.90365
 finished parsing run0000_0000 timepoint 0.9037
 finished parsing run0000_0000 timepoint 0.90375
 finished parsing run0000_0000 timepoint 0.9038
 finished parsing run0000_0000 timepoint 0.90385
 finished parsing run0000_0000 timepoint 0.9039
 finished parsing run0000_0000 timepoint 0.90395
 finished parsing run0000_0000 timepoint 0.904
 finished parsing run0000_0000 timepoint 0.90405
 finished parsing run0000_0000 timepoint 0.9041
 finished parsing run0000_0000 timepoint 0.90415
 finished parsing run0000_0000 timepoint 0.9042
 finished parsing run0000_0000 timepoint 0.90425
 finished parsing run0000_0000 timepoint 0.9043
 finished parsing run0000_0000 timepoint 0.90435
 finished parsing run0000_0000 timepoint 0.9044
 finished parsing run0000_0000 timepoint 0.90445
 finished parsing run0000_0000 timepoint 0.9045
 finished parsing run0000_0000 timepoint 0.90455
 finished parsing run0000_0000 timepoint 0.9046
 finished parsing run0000_0000 timepoint 0.90465
 finished parsing run0000_0000 timepoint 0.9047
 finished parsing run0000_0000 timepoint 0.90475
 finished parsing run0000_0000 timepoint 0.9048
 finished parsing run0000_0000 timepoint 0.90485
 finished parsing run0000_0000 timepoint 0.9049
 finished parsing run0000_0000 timepoint 0.90495
 finished parsing run0000_0000 timepoint 0.905
 finished parsing run0000_0000 timepoint 0.90505
 finished parsing run0000_0000 timepoint 0.9051
 finished parsing run0000_0000 timepoint 0.90515
 finished parsing run0000_0000 timepoint 0.9052
 finished parsing run0000_0000 timepoint 0.90525
 finished parsing run0000_0000 timepoint 0.9053
 finished parsing run0000_0000 timepoint 0.90535
 finished parsing run0000_0000 timepoint 0.9054
 finished parsing run0000_0000 timepoint 0.90545
 finished parsing run0000_0000 timepoint 0.9055
 finished parsing run0000_0000 timepoint 0.90555
 finished parsing run0000_0000 timepoint 0.9056
 finished parsing run0000_0000 timepoint 0.90565
 finished parsing run0000_0000 timepoint 0.9057
 finished parsing run0000_0000 timepoint 0.90575
 finished parsing run0000_0000 timepoint 0.9058
 finished parsing run0000_0000 timepoint 0.90585
 finished parsing run0000_0000 timepoint 0.9059
 finished parsing run0000_0000 timepoint 0.90595
 finished parsing run0000_0000 timepoint 0.906
 finished parsing run0000_0000 timepoint 0.90605
 finished parsing run0000_0000 timepoint 0.9061
 finished parsing run0000_0000 timepoint 0.90615
 finished parsing run0000_0000 timepoint 0.9062
 finished parsing run0000_0000 timepoint 0.90625
 finished parsing run0000_0000 timepoint 0.9063
 finished parsing run0000_0000 timepoint 0.90635
 finished parsing run0000_0000 timepoint 0.9064
 finished parsing run0000_0000 timepoint 0.90645
 finished parsing run0000_0000 timepoint 0.9065
 finished parsing run0000_0000 timepoint 0.90655
 finished parsing run0000_0000 timepoint 0.9066
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8937
 finished parsing run0000_0000 timepoint 0.89375
 finished parsing run0000_0000 timepoint 0.8938
 finished parsing run0000_0000 timepoint 0.89385
 finished parsing run0000_0000 timepoint 0.8939
 finished parsing run0000_0000 timepoint 0.89395
 finished parsing run0000_0000 timepoint 0.894
 finished parsing run0000_0000 timepoint 0.89405
 finished parsing run0000_0000 timepoint 0.8941
 finished parsing run0000_0000 timepoint 0.89415
 finished parsing run0000_0000 timepoint 0.8942
 finished parsing run0000_0000 timepoint 0.89425
 finished parsing run0000_0000 timepoint 0.8943
 finished parsing run0000_0000 timepoint 0.89435
 finished parsing run0000_0000 timepoint 0.8944
 finished parsing run0000_0000 timepoint 0.89445
 finished parsing run0000_0000 timepoint 0.8945
 finished parsing run0000_0000 timepoint 0.89455
 finished parsing run0000_0000 timepoint 0.8946
 finished parsing run0000_0000 timepoint 0.89465
 finished parsing run0000_0000 timepoint 0.8947
 finished parsing run0000_0000 timepoint 0.89475
 finished parsing run0000_0000 timepoint 0.8948
 finished parsing run0000_0000 timepoint 0.89485
 finished parsing run0000_0000 timepoint 0.8949
 finished parsing run0000_0000 timepoint 0.89495
 finished parsing run0000_0000 timepoint 0.895
 finished parsing run0000_0000 timepoint 0.89505
 finished parsing run0000_0000 timepoint 0.8951
 finished parsing run0000_0000 timepoint 0.89515
 finished parsing run0000_0000 timepoint 0.8952
 finished parsing run0000_0000 timepoint 0.89525
 finished parsing run0000_0000 timepoint 0.8953
 finished parsing run0000_0000 timepoint 0.89535
 finished parsing run0000_0000 timepoint 0.8954
 finished parsing run0000_0000 timepoint 0.89545
 finished parsing run0000_0000 timepoint 0.8955
 finished parsing run0000_0000 timepoint 0.89555
 finished parsing run0000_0000 timepoint 0.8956
 finished parsing run0000_0000 timepoint 0.89565
 finished parsing run0000_0000 timepoint 0.8957
 finished parsing run0000_0000 timepoint 0.89575
 finished parsing run0000_0000 timepoint 0.8958
 finished parsing run0000_0000 timepoint 0.89585
 finished parsing run0000_0000 timepoint 0.8959
 finished parsing run0000_0000 timepoint 0.89595
 finished parsing run0000_0000 timepoint 0.896
 finished parsing run0000_0000 timepoint 0.89605
 finished parsing run0000_0000 timepoint 0.8961
 finished parsing run0000_0000 timepoint 0.89615
 finished parsing run0000_0000 timepoint 0.8962
 finished parsing run0000_0000 timepoint 0.89625
 finished parsing run0000_0000 timepoint 0.8963
 finished parsing run0000_0000 timepoint 0.89635
 finished parsing run0000_0000 timepoint 0.8964
 finished parsing run0000_0000 timepoint 0.89645
 finished parsing run0000_0000 timepoint 0.8965
 finished parsing run0000_0000 timepoint 0.89655
 finished parsing run0000_0000 timepoint 0.8966
 finished parsing run0000_0000 timepoint 0.89665
 finished parsing run0000_0000 timepoint 0.8967
 finished parsing run0000_0000 timepoint 0.89675
 finished parsing run0000_0000 timepoint 0.8968
 finished parsing run0000_0000 timepoint 0.89685
 finished parsing run0000_0000 timepoint 0.8969
 finished parsing run0000_0000 timepoint 0.89695
 finished parsing run0000_0000 timepoint 0.897
 finished parsing run0000_0000 timepoint 0.89705
 finished parsing run0000_0000 timepoint 0.8971
 finished parsing run0000_0000 timepoint 0.89715
 finished parsing run0000_0000 timepoint 0.8972
 finished parsing run0000_0000 timepoint 0.89725
 finished parsing run0000_0000 timepoint 0.8973
 finished parsing run0000_0000 timepoint 0.89735
 finished parsing run0000_0000 timepoint 0.8974
 finished parsing run0000_0000 timepoint 0.89745
 finished parsing run0000_0000 timepoint 0.8975
 finished parsing run0000_0000 timepoint 0.89755
 finished parsing run0000_0000 timepoint 0.8976
 finished parsing run0000_0000 timepoint 0.89765
 finished parsing run0000_0000 timepoint 0.8977
 finished parsing run0000_0000 timepoint 0.89775
 finished parsing run0000_0000 timepoint 0.8978
 finished parsing run0000_0000 timepoint 0.89785
 finished parsing run0000_0000 timepoint 0.8979
 finished parsing run0000_0000 timepoint 0.89795
 finished parsing run0000_0000 timepoint 0.898
 finished parsing run0000_0000 timepoint 0.89805
 finished parsing run0000_0000 timepoint 0.8981
 finished parsing run0000_0000 timepoint 0.89815
 finished parsing run0000_0000 timepoint 0.8982
 finished parsing run0000_0000 timepoint 0.89825
 finished parsing run0000_0000 timepoint 0.8983
 finished parsing run0000_0000 timepoint 0.89835
 finished parsing run0000_0000 timepoint 0.8984
 finished parsing run0000_0000 timepoint 0.89845
 finished parsing run0000_0000 timepoint 0.8985
 finished parsing run0000_0000 timepoint 0.89855
 finished parsing run0000_0000 timepoint 0.8986
 finished parsing run0000_0000 timepoint 0.89865
 finished parsing run0000_0000 timepoint 0.8987
 finished parsing run0000_0000 timepoint 0.89875
 finished parsing run0000_0000 timepoint 0.8988
 finished parsing run0000_0000 timepoint 0.89885
 finished parsing run0000_0000 timepoint 0.8989
 finished parsing run0000_0000 timepoint 0.89895
 finished parsing run0000_0000 timepoint 0.899
 finished parsing run0000_0000 timepoint 0.89905
 finished parsing run0000_0000 timepoint 0.8991
 finished parsing run0000_0000 timepoint 0.89915
 finished parsing run0000_0000 timepoint 0.8992
 finished parsing run0000_0000 timepoint 0.89925
 finished parsing run0000_0000 timepoint 0.8993
 finished parsing run0000_0000 timepoint 0.89935
 finished parsing run0000_0000 timepoint 0.8994
 finished parsing run0000_0000 timepoint 0.89945
 finished parsing run0000_0000 timepoint 0.8995
 finished parsing run0000_0000 timepoint 0.89955
 finished parsing run0000_0000 timepoint 0.8996
 finished parsing run0000_0000 timepoint 0.89965
 finished parsing run0000_0000 timepoint 0.8997
 finished parsing run0000_0000 timepoint 0.89975
 finished parsing run0000_0000 timepoint 0.8998
 finished parsing run0000_0000 timepoint 0.89985
 finished parsing run0000_0000 timepoint 0.8999
 finished parsing run0000_0000 timepoint 0.89995
 finished parsing run0000_0000 timepoint 0.9
 finished parsing run0000_0000 timepoint 0.90005
 finished parsing run0000_0000 timepoint 0.9001
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8871
 finished parsing run0000_0000 timepoint 0.88715
 finished parsing run0000_0000 timepoint 0.8872
 finished parsing run0000_0000 timepoint 0.88725
 finished parsing run0000_0000 timepoint 0.8873
 finished parsing run0000_0000 timepoint 0.88735
 finished parsing run0000_0000 timepoint 0.8874
 finished parsing run0000_0000 timepoint 0.88745
 finished parsing run0000_0000 timepoint 0.8875
 finished parsing run0000_0000 timepoint 0.88755
 finished parsing run0000_0000 timepoint 0.8876
 finished parsing run0000_0000 timepoint 0.88765
 finished parsing run0000_0000 timepoint 0.8877
 finished parsing run0000_0000 timepoint 0.88775
 finished parsing run0000_0000 timepoint 0.8878
 finished parsing run0000_0000 timepoint 0.88785
 finished parsing run0000_0000 timepoint 0.8879
 finished parsing run0000_0000 timepoint 0.88795
 finished parsing run0000_0000 timepoint 0.888
 finished parsing run0000_0000 timepoint 0.88805
 finished parsing run0000_0000 timepoint 0.8881
 finished parsing run0000_0000 timepoint 0.88815
 finished parsing run0000_0000 timepoint 0.8882
 finished parsing run0000_0000 timepoint 0.88825
 finished parsing run0000_0000 timepoint 0.8883
 finished parsing run0000_0000 timepoint 0.88835
 finished parsing run0000_0000 timepoint 0.8884
 finished parsing run0000_0000 timepoint 0.88845
 finished parsing run0000_0000 timepoint 0.8885
 finished parsing run0000_0000 timepoint 0.88855
 finished parsing run0000_0000 timepoint 0.8886
 finished parsing run0000_0000 timepoint 0.88865
 finished parsing run0000_0000 timepoint 0.8887
 finished parsing run0000_0000 timepoint 0.88875
 finished parsing run0000_0000 timepoint 0.8888
 finished parsing run0000_0000 timepoint 0.88885
 finished parsing run0000_0000 timepoint 0.8889
 finished parsing run0000_0000 timepoint 0.88895
 finished parsing run0000_0000 timepoint 0.889
 finished parsing run0000_0000 timepoint 0.88905
 finished parsing run0000_0000 timepoint 0.8891
 finished parsing run0000_0000 timepoint 0.88915
 finished parsing run0000_0000 timepoint 0.8892
 finished parsing run0000_0000 timepoint 0.88925
 finished parsing run0000_0000 timepoint 0.8893
 finished parsing run0000_0000 timepoint 0.88935
 finished parsing run0000_0000 timepoint 0.8894
 finished parsing run0000_0000 timepoint 0.88945
 finished parsing run0000_0000 timepoint 0.8895
 finished parsing run0000_0000 timepoint 0.88955
 finished parsing run0000_0000 timepoint 0.8896
 finished parsing run0000_0000 timepoint 0.88965
 finished parsing run0000_0000 timepoint 0.8897
 finished parsing run0000_0000 timepoint 0.88975
 finished parsing run0000_0000 timepoint 0.8898
 finished parsing run0000_0000 timepoint 0.88985
 finished parsing run0000_0000 timepoint 0.8899
 finished parsing run0000_0000 timepoint 0.88995
 finished parsing run0000_0000 timepoint 0.89
 finished parsing run0000_0000 timepoint 0.89005
 finished parsing run0000_0000 timepoint 0.8901
 finished parsing run0000_0000 timepoint 0.89015
 finished parsing run0000_0000 timepoint 0.8902
 finished parsing run0000_0000 timepoint 0.89025
 finished parsing run0000_0000 timepoint 0.8903
 finished parsing run0000_0000 timepoint 0.89035
 finished parsing run0000_0000 timepoint 0.8904
 finished parsing run0000_0000 timepoint 0.89045
 finished parsing run0000_0000 timepoint 0.8905
 finished parsing run0000_0000 timepoint 0.89055
 finished parsing run0000_0000 timepoint 0.8906
 finished parsing run0000_0000 timepoint 0.89065
 finished parsing run0000_0000 timepoint 0.8907
 finished parsing run0000_0000 timepoint 0.89075
 finished parsing run0000_0000 timepoint 0.8908
 finished parsing run0000_0000 timepoint 0.89085
 finished parsing run0000_0000 timepoint 0.8909
 finished parsing run0000_0000 timepoint 0.89095
 finished parsing run0000_0000 timepoint 0.891
 finished parsing run0000_0000 timepoint 0.89105
 finished parsing run0000_0000 timepoint 0.8911
 finished parsing run0000_0000 timepoint 0.89115
 finished parsing run0000_0000 timepoint 0.8912
 finished parsing run0000_0000 timepoint 0.89125
 finished parsing run0000_0000 timepoint 0.8913
 finished parsing run0000_0000 timepoint 0.89135
 finished parsing run0000_0000 timepoint 0.8914
 finished parsing run0000_0000 timepoint 0.89145
 finished parsing run0000_0000 timepoint 0.8915
 finished parsing run0000_0000 timepoint 0.89155
 finished parsing run0000_0000 timepoint 0.8916
 finished parsing run0000_0000 timepoint 0.89165
 finished parsing run0000_0000 timepoint 0.8917
 finished parsing run0000_0000 timepoint 0.89175
 finished parsing run0000_0000 timepoint 0.8918
 finished parsing run0000_0000 timepoint 0.89185
 finished parsing run0000_0000 timepoint 0.8919
 finished parsing run0000_0000 timepoint 0.89195
 finished parsing run0000_0000 timepoint 0.892
 finished parsing run0000_0000 timepoint 0.89205
 finished parsing run0000_0000 timepoint 0.8921
 finished parsing run0000_0000 timepoint 0.89215
 finished parsing run0000_0000 timepoint 0.8922
 finished parsing run0000_0000 timepoint 0.89225
 finished parsing run0000_0000 timepoint 0.8923
 finished parsing run0000_0000 timepoint 0.89235
 finished parsing run0000_0000 timepoint 0.8924
 finished parsing run0000_0000 timepoint 0.89245
 finished parsing run0000_0000 timepoint 0.8925
 finished parsing run0000_0000 timepoint 0.89255
 finished parsing run0000_0000 timepoint 0.8926
 finished parsing run0000_0000 timepoint 0.89265
 finished parsing run0000_0000 timepoint 0.8927
 finished parsing run0000_0000 timepoint 0.89275
 finished parsing run0000_0000 timepoint 0.8928
 finished parsing run0000_0000 timepoint 0.89285
 finished parsing run0000_0000 timepoint 0.8929
 finished parsing run0000_0000 timepoint 0.89295
 finished parsing run0000_0000 timepoint 0.893
 finished parsing run0000_0000 timepoint 0.89305
 finished parsing run0000_0000 timepoint 0.8931
 finished parsing run0000_0000 timepoint 0.89315
 finished parsing run0000_0000 timepoint 0.8932
 finished parsing run0000_0000 timepoint 0.89325
 finished parsing run0000_0000 timepoint 0.8933
 finished parsing run0000_0000 timepoint 0.89335
 finished parsing run0000_0000 timepoint 0.8934
 finished parsing run0000_0000 timepoint 0.89345
 finished parsing run0000_0000 timepoint 0.8935
 finished parsing run0000_0000 timepoint 0.89355
 finished parsing run0000_0000 timepoint 0.8936
 finished parsing run0000_0000 timepoint 0.89365
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8809
 finished parsing run0000_0000 timepoint 0.88095
 finished parsing run0000_0000 timepoint 0.881
 finished parsing run0000_0000 timepoint 0.88105
 finished parsing run0000_0000 timepoint 0.8811
 finished parsing run0000_0000 timepoint 0.88115
 finished parsing run0000_0000 timepoint 0.8812
 finished parsing run0000_0000 timepoint 0.88125
 finished parsing run0000_0000 timepoint 0.8813
 finished parsing run0000_0000 timepoint 0.88135
 finished parsing run0000_0000 timepoint 0.8814
 finished parsing run0000_0000 timepoint 0.88145
 finished parsing run0000_0000 timepoint 0.8815
 finished parsing run0000_0000 timepoint 0.88155
 finished parsing run0000_0000 timepoint 0.8816
 finished parsing run0000_0000 timepoint 0.88165
 finished parsing run0000_0000 timepoint 0.8817
 finished parsing run0000_0000 timepoint 0.88175
 finished parsing run0000_0000 timepoint 0.8818
 finished parsing run0000_0000 timepoint 0.88185
 finished parsing run0000_0000 timepoint 0.8819
 finished parsing run0000_0000 timepoint 0.88195
 finished parsing run0000_0000 timepoint 0.882
 finished parsing run0000_0000 timepoint 0.88205
 finished parsing run0000_0000 timepoint 0.8821
 finished parsing run0000_0000 timepoint 0.88215
 finished parsing run0000_0000 timepoint 0.8822
 finished parsing run0000_0000 timepoint 0.88225
 finished parsing run0000_0000 timepoint 0.8823
 finished parsing run0000_0000 timepoint 0.88235
 finished parsing run0000_0000 timepoint 0.8824
 finished parsing run0000_0000 timepoint 0.88245
 finished parsing run0000_0000 timepoint 0.8825
 finished parsing run0000_0000 timepoint 0.88255
 finished parsing run0000_0000 timepoint 0.8826
 finished parsing run0000_0000 timepoint 0.88265
 finished parsing run0000_0000 timepoint 0.8827
 finished parsing run0000_0000 timepoint 0.88275
 finished parsing run0000_0000 timepoint 0.8828
 finished parsing run0000_0000 timepoint 0.88285
 finished parsing run0000_0000 timepoint 0.8829
 finished parsing run0000_0000 timepoint 0.88295
 finished parsing run0000_0000 timepoint 0.883
 finished parsing run0000_0000 timepoint 0.88305
 finished parsing run0000_0000 timepoint 0.8831
 finished parsing run0000_0000 timepoint 0.88315
 finished parsing run0000_0000 timepoint 0.8832
 finished parsing run0000_0000 timepoint 0.88325
 finished parsing run0000_0000 timepoint 0.8833
 finished parsing run0000_0000 timepoint 0.88335
 finished parsing run0000_0000 timepoint 0.8834
 finished parsing run0000_0000 timepoint 0.88345
 finished parsing run0000_0000 timepoint 0.8835
 finished parsing run0000_0000 timepoint 0.88355
 finished parsing run0000_0000 timepoint 0.8836
 finished parsing run0000_0000 timepoint 0.88365
 finished parsing run0000_0000 timepoint 0.8837
 finished parsing run0000_0000 timepoint 0.88375
 finished parsing run0000_0000 timepoint 0.8838
 finished parsing run0000_0000 timepoint 0.88385
 finished parsing run0000_0000 timepoint 0.8839
 finished parsing run0000_0000 timepoint 0.88395
 finished parsing run0000_0000 timepoint 0.884
 finished parsing run0000_0000 timepoint 0.88405
 finished parsing run0000_0000 timepoint 0.8841
 finished parsing run0000_0000 timepoint 0.88415
 finished parsing run0000_0000 timepoint 0.8842
 finished parsing run0000_0000 timepoint 0.88425
 finished parsing run0000_0000 timepoint 0.8843
 finished parsing run0000_0000 timepoint 0.88435
 finished parsing run0000_0000 timepoint 0.8844
 finished parsing run0000_0000 timepoint 0.88445
 finished parsing run0000_0000 timepoint 0.8845
 finished parsing run0000_0000 timepoint 0.88455
 finished parsing run0000_0000 timepoint 0.8846
 finished parsing run0000_0000 timepoint 0.88465
 finished parsing run0000_0000 timepoint 0.8847
 finished parsing run0000_0000 timepoint 0.88475
 finished parsing run0000_0000 timepoint 0.8848
 finished parsing run0000_0000 timepoint 0.88485
 finished parsing run0000_0000 timepoint 0.8849
 finished parsing run0000_0000 timepoint 0.88495
 finished parsing run0000_0000 timepoint 0.885
 finished parsing run0000_0000 timepoint 0.88505
 finished parsing run0000_0000 timepoint 0.8851
 finished parsing run0000_0000 timepoint 0.88515
 finished parsing run0000_0000 timepoint 0.8852
 finished parsing run0000_0000 timepoint 0.88525
 finished parsing run0000_0000 timepoint 0.8853
 finished parsing run0000_0000 timepoint 0.88535
 finished parsing run0000_0000 timepoint 0.8854
 finished parsing run0000_0000 timepoint 0.88545
 finished parsing run0000_0000 timepoint 0.8855
 finished parsing run0000_0000 timepoint 0.88555
 finished parsing run0000_0000 timepoint 0.8856
 finished parsing run0000_0000 timepoint 0.88565
 finished parsing run0000_0000 timepoint 0.8857
 finished parsing run0000_0000 timepoint 0.88575
 finished parsing run0000_0000 timepoint 0.8858
 finished parsing run0000_0000 timepoint 0.88585
 finished parsing run0000_0000 timepoint 0.8859
 finished parsing run0000_0000 timepoint 0.88595
 finished parsing run0000_0000 timepoint 0.886
 finished parsing run0000_0000 timepoint 0.88605
 finished parsing run0000_0000 timepoint 0.8861
 finished parsing run0000_0000 timepoint 0.88615
 finished parsing run0000_0000 timepoint 0.8862
 finished parsing run0000_0000 timepoint 0.88625
 finished parsing run0000_0000 timepoint 0.8863
 finished parsing run0000_0000 timepoint 0.88635
 finished parsing run0000_0000 timepoint 0.8864
 finished parsing run0000_0000 timepoint 0.88645
 finished parsing run0000_0000 timepoint 0.8865
 finished parsing run0000_0000 timepoint 0.88655
 finished parsing run0000_0000 timepoint 0.8866
 finished parsing run0000_0000 timepoint 0.88665
 finished parsing run0000_0000 timepoint 0.8867
 finished parsing run0000_0000 timepoint 0.88675
 finished parsing run0000_0000 timepoint 0.8868
 finished parsing run0000_0000 timepoint 0.88685
 finished parsing run0000_0000 timepoint 0.8869
 finished parsing run0000_0000 timepoint 0.88695
 finished parsing run0000_0000 timepoint 0.887
 finished parsing run0000_0000 timepoint 0.88705
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.87445
 finished parsing run0000_0000 timepoint 0.8745
 finished parsing run0000_0000 timepoint 0.87455
 finished parsing run0000_0000 timepoint 0.8746
 finished parsing run0000_0000 timepoint 0.87465
 finished parsing run0000_0000 timepoint 0.8747
 finished parsing run0000_0000 timepoint 0.87475
 finished parsing run0000_0000 timepoint 0.8748
 finished parsing run0000_0000 timepoint 0.87485
 finished parsing run0000_0000 timepoint 0.8749
 finished parsing run0000_0000 timepoint 0.87495
 finished parsing run0000_0000 timepoint 0.875
 finished parsing run0000_0000 timepoint 0.87505
 finished parsing run0000_0000 timepoint 0.8751
 finished parsing run0000_0000 timepoint 0.87515
 finished parsing run0000_0000 timepoint 0.8752
 finished parsing run0000_0000 timepoint 0.87525
 finished parsing run0000_0000 timepoint 0.8753
 finished parsing run0000_0000 timepoint 0.87535
 finished parsing run0000_0000 timepoint 0.8754
 finished parsing run0000_0000 timepoint 0.87545
 finished parsing run0000_0000 timepoint 0.8755
 finished parsing run0000_0000 timepoint 0.87555
 finished parsing run0000_0000 timepoint 0.8756
 finished parsing run0000_0000 timepoint 0.87565
 finished parsing run0000_0000 timepoint 0.8757
 finished parsing run0000_0000 timepoint 0.87575
 finished parsing run0000_0000 timepoint 0.8758
 finished parsing run0000_0000 timepoint 0.87585
 finished parsing run0000_0000 timepoint 0.8759
 finished parsing run0000_0000 timepoint 0.87595
 finished parsing run0000_0000 timepoint 0.876
 finished parsing run0000_0000 timepoint 0.87605
 finished parsing run0000_0000 timepoint 0.8761
 finished parsing run0000_0000 timepoint 0.87615
 finished parsing run0000_0000 timepoint 0.8762
 finished parsing run0000_0000 timepoint 0.87625
 finished parsing run0000_0000 timepoint 0.8763
 finished parsing run0000_0000 timepoint 0.87635
 finished parsing run0000_0000 timepoint 0.8764
 finished parsing run0000_0000 timepoint 0.87645
 finished parsing run0000_0000 timepoint 0.8765
 finished parsing run0000_0000 timepoint 0.87655
 finished parsing run0000_0000 timepoint 0.8766
 finished parsing run0000_0000 timepoint 0.87665
 finished parsing run0000_0000 timepoint 0.8767
 finished parsing run0000_0000 timepoint 0.87675
 finished parsing run0000_0000 timepoint 0.8768
 finished parsing run0000_0000 timepoint 0.87685
 finished parsing run0000_0000 timepoint 0.8769
 finished parsing run0000_0000 timepoint 0.87695
 finished parsing run0000_0000 timepoint 0.877
 finished parsing run0000_0000 timepoint 0.87705
 finished parsing run0000_0000 timepoint 0.8771
 finished parsing run0000_0000 timepoint 0.87715
 finished parsing run0000_0000 timepoint 0.8772
 finished parsing run0000_0000 timepoint 0.87725
 finished parsing run0000_0000 timepoint 0.8773
 finished parsing run0000_0000 timepoint 0.87735
 finished parsing run0000_0000 timepoint 0.8774
 finished parsing run0000_0000 timepoint 0.87745
 finished parsing run0000_0000 timepoint 0.8775
 finished parsing run0000_0000 timepoint 0.87755
 finished parsing run0000_0000 timepoint 0.8776
 finished parsing run0000_0000 timepoint 0.87765
 finished parsing run0000_0000 timepoint 0.8777
 finished parsing run0000_0000 timepoint 0.87775
 finished parsing run0000_0000 timepoint 0.8778
 finished parsing run0000_0000 timepoint 0.87785
 finished parsing run0000_0000 timepoint 0.8779
 finished parsing run0000_0000 timepoint 0.87795
 finished parsing run0000_0000 timepoint 0.878
 finished parsing run0000_0000 timepoint 0.87805
 finished parsing run0000_0000 timepoint 0.8781
 finished parsing run0000_0000 timepoint 0.87815
 finished parsing run0000_0000 timepoint 0.8782
 finished parsing run0000_0000 timepoint 0.87825
 finished parsing run0000_0000 timepoint 0.8783
 finished parsing run0000_0000 timepoint 0.87835
 finished parsing run0000_0000 timepoint 0.8784
 finished parsing run0000_0000 timepoint 0.87845
 finished parsing run0000_0000 timepoint 0.8785
 finished parsing run0000_0000 timepoint 0.87855
 finished parsing run0000_0000 timepoint 0.8786
 finished parsing run0000_0000 timepoint 0.87865
 finished parsing run0000_0000 timepoint 0.8787
 finished parsing run0000_0000 timepoint 0.87875
 finished parsing run0000_0000 timepoint 0.8788
 finished parsing run0000_0000 timepoint 0.87885
 finished parsing run0000_0000 timepoint 0.8789
 finished parsing run0000_0000 timepoint 0.87895
 finished parsing run0000_0000 timepoint 0.879
 finished parsing run0000_0000 timepoint 0.87905
 finished parsing run0000_0000 timepoint 0.8791
 finished parsing run0000_0000 timepoint 0.87915
 finished parsing run0000_0000 timepoint 0.8792
 finished parsing run0000_0000 timepoint 0.87925
 finished parsing run0000_0000 timepoint 0.8793
 finished parsing run0000_0000 timepoint 0.87935
 finished parsing run0000_0000 timepoint 0.8794
 finished parsing run0000_0000 timepoint 0.87945
 finished parsing run0000_0000 timepoint 0.8795
 finished parsing run0000_0000 timepoint 0.87955
 finished parsing run0000_0000 timepoint 0.8796
 finished parsing run0000_0000 timepoint 0.87965
 finished parsing run0000_0000 timepoint 0.8797
 finished parsing run0000_0000 timepoint 0.87975
 finished parsing run0000_0000 timepoint 0.8798
 finished parsing run0000_0000 timepoint 0.87985
 finished parsing run0000_0000 timepoint 0.8799
 finished parsing run0000_0000 timepoint 0.87995
 finished parsing run0000_0000 timepoint 0.88
 finished parsing run0000_0000 timepoint 0.88005
 finished parsing run0000_0000 timepoint 0.8801
 finished parsing run0000_0000 timepoint 0.88015
 finished parsing run0000_0000 timepoint 0.8802
 finished parsing run0000_0000 timepoint 0.88025
 finished parsing run0000_0000 timepoint 0.8803
 finished parsing run0000_0000 timepoint 0.88035
 finished parsing run0000_0000 timepoint 0.8804
 finished parsing run0000_0000 timepoint 0.88045
 finished parsing run0000_0000 timepoint 0.8805
 finished parsing run0000_0000 timepoint 0.88055
 finished parsing run0000_0000 timepoint 0.8806
 finished parsing run0000_0000 timepoint 0.88065
 finished parsing run0000_0000 timepoint 0.8807
 finished parsing run0000_0000 timepoint 0.88075
 finished parsing run0000_0000 timepoint 0.8808
 finished parsing run0000_0000 timepoint 0.88085
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8681
 finished parsing run0000_0000 timepoint 0.86815
 finished parsing run0000_0000 timepoint 0.8682
 finished parsing run0000_0000 timepoint 0.86825
 finished parsing run0000_0000 timepoint 0.8683
 finished parsing run0000_0000 timepoint 0.86835
 finished parsing run0000_0000 timepoint 0.8684
 finished parsing run0000_0000 timepoint 0.86845
 finished parsing run0000_0000 timepoint 0.8685
 finished parsing run0000_0000 timepoint 0.86855
 finished parsing run0000_0000 timepoint 0.8686
 finished parsing run0000_0000 timepoint 0.86865
 finished parsing run0000_0000 timepoint 0.8687
 finished parsing run0000_0000 timepoint 0.86875
 finished parsing run0000_0000 timepoint 0.8688
 finished parsing run0000_0000 timepoint 0.86885
 finished parsing run0000_0000 timepoint 0.8689
 finished parsing run0000_0000 timepoint 0.86895
 finished parsing run0000_0000 timepoint 0.869
 finished parsing run0000_0000 timepoint 0.86905
 finished parsing run0000_0000 timepoint 0.8691
 finished parsing run0000_0000 timepoint 0.86915
 finished parsing run0000_0000 timepoint 0.8692
 finished parsing run0000_0000 timepoint 0.86925
 finished parsing run0000_0000 timepoint 0.8693
 finished parsing run0000_0000 timepoint 0.86935
 finished parsing run0000_0000 timepoint 0.8694
 finished parsing run0000_0000 timepoint 0.86945
 finished parsing run0000_0000 timepoint 0.8695
 finished parsing run0000_0000 timepoint 0.86955
 finished parsing run0000_0000 timepoint 0.8696
 finished parsing run0000_0000 timepoint 0.86965
 finished parsing run0000_0000 timepoint 0.8697
 finished parsing run0000_0000 timepoint 0.86975
 finished parsing run0000_0000 timepoint 0.8698
 finished parsing run0000_0000 timepoint 0.86985
 finished parsing run0000_0000 timepoint 0.8699
 finished parsing run0000_0000 timepoint 0.86995
 finished parsing run0000_0000 timepoint 0.87
 finished parsing run0000_0000 timepoint 0.87005
 finished parsing run0000_0000 timepoint 0.8701
 finished parsing run0000_0000 timepoint 0.87015
 finished parsing run0000_0000 timepoint 0.8702
 finished parsing run0000_0000 timepoint 0.87025
 finished parsing run0000_0000 timepoint 0.8703
 finished parsing run0000_0000 timepoint 0.87035
 finished parsing run0000_0000 timepoint 0.8704
 finished parsing run0000_0000 timepoint 0.87045
 finished parsing run0000_0000 timepoint 0.8705
 finished parsing run0000_0000 timepoint 0.87055
 finished parsing run0000_0000 timepoint 0.8706
 finished parsing run0000_0000 timepoint 0.87065
 finished parsing run0000_0000 timepoint 0.8707
 finished parsing run0000_0000 timepoint 0.87075
 finished parsing run0000_0000 timepoint 0.8708
 finished parsing run0000_0000 timepoint 0.87085
 finished parsing run0000_0000 timepoint 0.8709
 finished parsing run0000_0000 timepoint 0.87095
 finished parsing run0000_0000 timepoint 0.871
 finished parsing run0000_0000 timepoint 0.87105
 finished parsing run0000_0000 timepoint 0.8711
 finished parsing run0000_0000 timepoint 0.87115
 finished parsing run0000_0000 timepoint 0.8712
 finished parsing run0000_0000 timepoint 0.87125
 finished parsing run0000_0000 timepoint 0.8713
 finished parsing run0000_0000 timepoint 0.87135
 finished parsing run0000_0000 timepoint 0.8714
 finished parsing run0000_0000 timepoint 0.87145
 finished parsing run0000_0000 timepoint 0.8715
 finished parsing run0000_0000 timepoint 0.87155
 finished parsing run0000_0000 timepoint 0.8716
 finished parsing run0000_0000 timepoint 0.87165
 finished parsing run0000_0000 timepoint 0.8717
 finished parsing run0000_0000 timepoint 0.87175
 finished parsing run0000_0000 timepoint 0.8718
 finished parsing run0000_0000 timepoint 0.87185
 finished parsing run0000_0000 timepoint 0.8719
 finished parsing run0000_0000 timepoint 0.87195
 finished parsing run0000_0000 timepoint 0.872
 finished parsing run0000_0000 timepoint 0.87205
 finished parsing run0000_0000 timepoint 0.8721
 finished parsing run0000_0000 timepoint 0.87215
 finished parsing run0000_0000 timepoint 0.8722
 finished parsing run0000_0000 timepoint 0.87225
 finished parsing run0000_0000 timepoint 0.8723
 finished parsing run0000_0000 timepoint 0.87235
 finished parsing run0000_0000 timepoint 0.8724
 finished parsing run0000_0000 timepoint 0.87245
 finished parsing run0000_0000 timepoint 0.8725
 finished parsing run0000_0000 timepoint 0.87255
 finished parsing run0000_0000 timepoint 0.8726
 finished parsing run0000_0000 timepoint 0.87265
 finished parsing run0000_0000 timepoint 0.8727
 finished parsing run0000_0000 timepoint 0.87275
 finished parsing run0000_0000 timepoint 0.8728
 finished parsing run0000_0000 timepoint 0.87285
 finished parsing run0000_0000 timepoint 0.8729
 finished parsing run0000_0000 timepoint 0.87295
 finished parsing run0000_0000 timepoint 0.873
 finished parsing run0000_0000 timepoint 0.87305
 finished parsing run0000_0000 timepoint 0.8731
 finished parsing run0000_0000 timepoint 0.87315
 finished parsing run0000_0000 timepoint 0.8732
 finished parsing run0000_0000 timepoint 0.87325
 finished parsing run0000_0000 timepoint 0.8733
 finished parsing run0000_0000 timepoint 0.87335
 finished parsing run0000_0000 timepoint 0.8734
 finished parsing run0000_0000 timepoint 0.87345
 finished parsing run0000_0000 timepoint 0.8735
 finished parsing run0000_0000 timepoint 0.87355
 finished parsing run0000_0000 timepoint 0.8736
 finished parsing run0000_0000 timepoint 0.87365
 finished parsing run0000_0000 timepoint 0.8737
 finished parsing run0000_0000 timepoint 0.87375
 finished parsing run0000_0000 timepoint 0.8738
 finished parsing run0000_0000 timepoint 0.87385
 finished parsing run0000_0000 timepoint 0.8739
 finished parsing run0000_0000 timepoint 0.87395
 finished parsing run0000_0000 timepoint 0.874
 finished parsing run0000_0000 timepoint 0.87405
 finished parsing run0000_0000 timepoint 0.8741
 finished parsing run0000_0000 timepoint 0.87415
 finished parsing run0000_0000 timepoint 0.8742
 finished parsing run0000_0000 timepoint 0.87425
 finished parsing run0000_0000 timepoint 0.8743
 finished parsing run0000_0000 timepoint 0.87435
 finished parsing run0000_0000 timepoint 0.8744
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8616
 finished parsing run0000_0000 timepoint 0.86165
 finished parsing run0000_0000 timepoint 0.8617
 finished parsing run0000_0000 timepoint 0.86175
 finished parsing run0000_0000 timepoint 0.8618
 finished parsing run0000_0000 timepoint 0.86185
 finished parsing run0000_0000 timepoint 0.8619
 finished parsing run0000_0000 timepoint 0.86195
 finished parsing run0000_0000 timepoint 0.862
 finished parsing run0000_0000 timepoint 0.86205
 finished parsing run0000_0000 timepoint 0.8621
 finished parsing run0000_0000 timepoint 0.86215
 finished parsing run0000_0000 timepoint 0.8622
 finished parsing run0000_0000 timepoint 0.86225
 finished parsing run0000_0000 timepoint 0.8623
 finished parsing run0000_0000 timepoint 0.86235
 finished parsing run0000_0000 timepoint 0.8624
 finished parsing run0000_0000 timepoint 0.86245
 finished parsing run0000_0000 timepoint 0.8625
 finished parsing run0000_0000 timepoint 0.86255
 finished parsing run0000_0000 timepoint 0.8626
 finished parsing run0000_0000 timepoint 0.86265
 finished parsing run0000_0000 timepoint 0.8627
 finished parsing run0000_0000 timepoint 0.86275
 finished parsing run0000_0000 timepoint 0.8628
 finished parsing run0000_0000 timepoint 0.86285
 finished parsing run0000_0000 timepoint 0.8629
 finished parsing run0000_0000 timepoint 0.86295
 finished parsing run0000_0000 timepoint 0.863
 finished parsing run0000_0000 timepoint 0.86305
 finished parsing run0000_0000 timepoint 0.8631
 finished parsing run0000_0000 timepoint 0.86315
 finished parsing run0000_0000 timepoint 0.8632
 finished parsing run0000_0000 timepoint 0.86325
 finished parsing run0000_0000 timepoint 0.8633
 finished parsing run0000_0000 timepoint 0.86335
 finished parsing run0000_0000 timepoint 0.8634
 finished parsing run0000_0000 timepoint 0.86345
 finished parsing run0000_0000 timepoint 0.8635
 finished parsing run0000_0000 timepoint 0.86355
 finished parsing run0000_0000 timepoint 0.8636
 finished parsing run0000_0000 timepoint 0.86365
 finished parsing run0000_0000 timepoint 0.8637
 finished parsing run0000_0000 timepoint 0.86375
 finished parsing run0000_0000 timepoint 0.8638
 finished parsing run0000_0000 timepoint 0.86385
 finished parsing run0000_0000 timepoint 0.8639
 finished parsing run0000_0000 timepoint 0.86395
 finished parsing run0000_0000 timepoint 0.864
 finished parsing run0000_0000 timepoint 0.86405
 finished parsing run0000_0000 timepoint 0.8641
 finished parsing run0000_0000 timepoint 0.86415
 finished parsing run0000_0000 timepoint 0.8642
 finished parsing run0000_0000 timepoint 0.86425
 finished parsing run0000_0000 timepoint 0.8643
 finished parsing run0000_0000 timepoint 0.86435
 finished parsing run0000_0000 timepoint 0.8644
 finished parsing run0000_0000 timepoint 0.86445
 finished parsing run0000_0000 timepoint 0.8645
 finished parsing run0000_0000 timepoint 0.86455
 finished parsing run0000_0000 timepoint 0.8646
 finished parsing run0000_0000 timepoint 0.86465
 finished parsing run0000_0000 timepoint 0.8647
 finished parsing run0000_0000 timepoint 0.86475
 finished parsing run0000_0000 timepoint 0.8648
 finished parsing run0000_0000 timepoint 0.86485
 finished parsing run0000_0000 timepoint 0.8649
 finished parsing run0000_0000 timepoint 0.86495
 finished parsing run0000_0000 timepoint 0.865
 finished parsing run0000_0000 timepoint 0.86505
 finished parsing run0000_0000 timepoint 0.8651
 finished parsing run0000_0000 timepoint 0.86515
 finished parsing run0000_0000 timepoint 0.8652
 finished parsing run0000_0000 timepoint 0.86525
 finished parsing run0000_0000 timepoint 0.8653
 finished parsing run0000_0000 timepoint 0.86535
 finished parsing run0000_0000 timepoint 0.8654
 finished parsing run0000_0000 timepoint 0.86545
 finished parsing run0000_0000 timepoint 0.8655
 finished parsing run0000_0000 timepoint 0.86555
 finished parsing run0000_0000 timepoint 0.8656
 finished parsing run0000_0000 timepoint 0.86565
 finished parsing run0000_0000 timepoint 0.8657
 finished parsing run0000_0000 timepoint 0.86575
 finished parsing run0000_0000 timepoint 0.8658
 finished parsing run0000_0000 timepoint 0.86585
 finished parsing run0000_0000 timepoint 0.8659
 finished parsing run0000_0000 timepoint 0.86595
 finished parsing run0000_0000 timepoint 0.866
 finished parsing run0000_0000 timepoint 0.86605
 finished parsing run0000_0000 timepoint 0.8661
 finished parsing run0000_0000 timepoint 0.86615
 finished parsing run0000_0000 timepoint 0.8662
 finished parsing run0000_0000 timepoint 0.86625
 finished parsing run0000_0000 timepoint 0.8663
 finished parsing run0000_0000 timepoint 0.86635
 finished parsing run0000_0000 timepoint 0.8664
 finished parsing run0000_0000 timepoint 0.86645
 finished parsing run0000_0000 timepoint 0.8665
 finished parsing run0000_0000 timepoint 0.86655
 finished parsing run0000_0000 timepoint 0.8666
 finished parsing run0000_0000 timepoint 0.86665
 finished parsing run0000_0000 timepoint 0.8667
 finished parsing run0000_0000 timepoint 0.86675
 finished parsing run0000_0000 timepoint 0.8668
 finished parsing run0000_0000 timepoint 0.86685
 finished parsing run0000_0000 timepoint 0.8669
 finished parsing run0000_0000 timepoint 0.86695
 finished parsing run0000_0000 timepoint 0.867
 finished parsing run0000_0000 timepoint 0.86705
 finished parsing run0000_0000 timepoint 0.8671
 finished parsing run0000_0000 timepoint 0.86715
 finished parsing run0000_0000 timepoint 0.8672
 finished parsing run0000_0000 timepoint 0.86725
 finished parsing run0000_0000 timepoint 0.8673
 finished parsing run0000_0000 timepoint 0.86735
 finished parsing run0000_0000 timepoint 0.8674
 finished parsing run0000_0000 timepoint 0.86745
 finished parsing run0000_0000 timepoint 0.8675
 finished parsing run0000_0000 timepoint 0.86755
 finished parsing run0000_0000 timepoint 0.8676
 finished parsing run0000_0000 timepoint 0.86765
 finished parsing run0000_0000 timepoint 0.8677
 finished parsing run0000_0000 timepoint 0.86775
 finished parsing run0000_0000 timepoint 0.8678
 finished parsing run0000_0000 timepoint 0.86785
 finished parsing run0000_0000 timepoint 0.8679
 finished parsing run0000_0000 timepoint 0.86795
 finished parsing run0000_0000 timepoint 0.868
 finished parsing run0000_0000 timepoint 0.86805
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.85495
 finished parsing run0000_0000 timepoint 0.855
 finished parsing run0000_0000 timepoint 0.85505
 finished parsing run0000_0000 timepoint 0.8551
 finished parsing run0000_0000 timepoint 0.85515
 finished parsing run0000_0000 timepoint 0.8552
 finished parsing run0000_0000 timepoint 0.85525
 finished parsing run0000_0000 timepoint 0.8553
 finished parsing run0000_0000 timepoint 0.85535
 finished parsing run0000_0000 timepoint 0.8554
 finished parsing run0000_0000 timepoint 0.85545
 finished parsing run0000_0000 timepoint 0.8555
 finished parsing run0000_0000 timepoint 0.85555
 finished parsing run0000_0000 timepoint 0.8556
 finished parsing run0000_0000 timepoint 0.85565
 finished parsing run0000_0000 timepoint 0.8557
 finished parsing run0000_0000 timepoint 0.85575
 finished parsing run0000_0000 timepoint 0.8558
 finished parsing run0000_0000 timepoint 0.85585
 finished parsing run0000_0000 timepoint 0.8559
 finished parsing run0000_0000 timepoint 0.85595
 finished parsing run0000_0000 timepoint 0.856
 finished parsing run0000_0000 timepoint 0.85605
 finished parsing run0000_0000 timepoint 0.8561
 finished parsing run0000_0000 timepoint 0.85615
 finished parsing run0000_0000 timepoint 0.8562
 finished parsing run0000_0000 timepoint 0.85625
 finished parsing run0000_0000 timepoint 0.8563
 finished parsing run0000_0000 timepoint 0.85635
 finished parsing run0000_0000 timepoint 0.8564
 finished parsing run0000_0000 timepoint 0.85645
 finished parsing run0000_0000 timepoint 0.8565
 finished parsing run0000_0000 timepoint 0.85655
 finished parsing run0000_0000 timepoint 0.8566
 finished parsing run0000_0000 timepoint 0.85665
 finished parsing run0000_0000 timepoint 0.8567
 finished parsing run0000_0000 timepoint 0.85675
 finished parsing run0000_0000 timepoint 0.8568
 finished parsing run0000_0000 timepoint 0.85685
 finished parsing run0000_0000 timepoint 0.8569
 finished parsing run0000_0000 timepoint 0.85695
 finished parsing run0000_0000 timepoint 0.857
 finished parsing run0000_0000 timepoint 0.85705
 finished parsing run0000_0000 timepoint 0.8571
 finished parsing run0000_0000 timepoint 0.85715
 finished parsing run0000_0000 timepoint 0.8572
 finished parsing run0000_0000 timepoint 0.85725
 finished parsing run0000_0000 timepoint 0.8573
 finished parsing run0000_0000 timepoint 0.85735
 finished parsing run0000_0000 timepoint 0.8574
 finished parsing run0000_0000 timepoint 0.85745
 finished parsing run0000_0000 timepoint 0.8575
 finished parsing run0000_0000 timepoint 0.85755
 finished parsing run0000_0000 timepoint 0.8576
 finished parsing run0000_0000 timepoint 0.85765
 finished parsing run0000_0000 timepoint 0.8577
 finished parsing run0000_0000 timepoint 0.85775
 finished parsing run0000_0000 timepoint 0.8578
 finished parsing run0000_0000 timepoint 0.85785
 finished parsing run0000_0000 timepoint 0.8579
 finished parsing run0000_0000 timepoint 0.85795
 finished parsing run0000_0000 timepoint 0.858
 finished parsing run0000_0000 timepoint 0.85805
 finished parsing run0000_0000 timepoint 0.8581
 finished parsing run0000_0000 timepoint 0.85815
 finished parsing run0000_0000 timepoint 0.8582
 finished parsing run0000_0000 timepoint 0.85825
 finished parsing run0000_0000 timepoint 0.8583
 finished parsing run0000_0000 timepoint 0.85835
 finished parsing run0000_0000 timepoint 0.8584
 finished parsing run0000_0000 timepoint 0.85845
 finished parsing run0000_0000 timepoint 0.8585
 finished parsing run0000_0000 timepoint 0.85855
 finished parsing run0000_0000 timepoint 0.8586
 finished parsing run0000_0000 timepoint 0.85865
 finished parsing run0000_0000 timepoint 0.8587
 finished parsing run0000_0000 timepoint 0.85875
 finished parsing run0000_0000 timepoint 0.8588
 finished parsing run0000_0000 timepoint 0.85885
 finished parsing run0000_0000 timepoint 0.8589
 finished parsing run0000_0000 timepoint 0.85895
 finished parsing run0000_0000 timepoint 0.859
 finished parsing run0000_0000 timepoint 0.85905
 finished parsing run0000_0000 timepoint 0.8591
 finished parsing run0000_0000 timepoint 0.85915
 finished parsing run0000_0000 timepoint 0.8592
 finished parsing run0000_0000 timepoint 0.85925
 finished parsing run0000_0000 timepoint 0.8593
 finished parsing run0000_0000 timepoint 0.85935
 finished parsing run0000_0000 timepoint 0.8594
 finished parsing run0000_0000 timepoint 0.85945
 finished parsing run0000_0000 timepoint 0.8595
 finished parsing run0000_0000 timepoint 0.85955
 finished parsing run0000_0000 timepoint 0.8596
 finished parsing run0000_0000 timepoint 0.85965
 finished parsing run0000_0000 timepoint 0.8597
 finished parsing run0000_0000 timepoint 0.85975
 finished parsing run0000_0000 timepoint 0.8598
 finished parsing run0000_0000 timepoint 0.85985
 finished parsing run0000_0000 timepoint 0.8599
 finished parsing run0000_0000 timepoint 0.85995
 finished parsing run0000_0000 timepoint 0.86
 finished parsing run0000_0000 timepoint 0.86005
 finished parsing run0000_0000 timepoint 0.8601
 finished parsing run0000_0000 timepoint 0.86015
 finished parsing run0000_0000 timepoint 0.8602
 finished parsing run0000_0000 timepoint 0.86025
 finished parsing run0000_0000 timepoint 0.8603
 finished parsing run0000_0000 timepoint 0.86035
 finished parsing run0000_0000 timepoint 0.8604
 finished parsing run0000_0000 timepoint 0.86045
 finished parsing run0000_0000 timepoint 0.8605
 finished parsing run0000_0000 timepoint 0.86055
 finished parsing run0000_0000 timepoint 0.8606
 finished parsing run0000_0000 timepoint 0.86065
 finished parsing run0000_0000 timepoint 0.8607
 finished parsing run0000_0000 timepoint 0.86075
 finished parsing run0000_0000 timepoint 0.8608
 finished parsing run0000_0000 timepoint 0.86085
 finished parsing run0000_0000 timepoint 0.8609
 finished parsing run0000_0000 timepoint 0.86095
 finished parsing run0000_0000 timepoint 0.861
 finished parsing run0000_0000 timepoint 0.86105
 finished parsing run0000_0000 timepoint 0.8611
 finished parsing run0000_0000 timepoint 0.86115
 finished parsing run0000_0000 timepoint 0.8612
 finished parsing run0000_0000 timepoint 0.86125
 finished parsing run0000_0000 timepoint 0.8613
 finished parsing run0000_0000 timepoint 0.86135
 finished parsing run0000_0000 timepoint 0.8614
 finished parsing run0000_0000 timepoint 0.86145
 finished parsing run0000_0000 timepoint 0.8615
 finished parsing run0000_0000 timepoint 0.86155
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8483
 finished parsing run0000_0000 timepoint 0.84835
 finished parsing run0000_0000 timepoint 0.8484
 finished parsing run0000_0000 timepoint 0.84845
 finished parsing run0000_0000 timepoint 0.8485
 finished parsing run0000_0000 timepoint 0.84855
 finished parsing run0000_0000 timepoint 0.8486
 finished parsing run0000_0000 timepoint 0.84865
 finished parsing run0000_0000 timepoint 0.8487
 finished parsing run0000_0000 timepoint 0.84875
 finished parsing run0000_0000 timepoint 0.8488
 finished parsing run0000_0000 timepoint 0.84885
 finished parsing run0000_0000 timepoint 0.8489
 finished parsing run0000_0000 timepoint 0.84895
 finished parsing run0000_0000 timepoint 0.849
 finished parsing run0000_0000 timepoint 0.84905
 finished parsing run0000_0000 timepoint 0.8491
 finished parsing run0000_0000 timepoint 0.84915
 finished parsing run0000_0000 timepoint 0.8492
 finished parsing run0000_0000 timepoint 0.84925
 finished parsing run0000_0000 timepoint 0.8493
 finished parsing run0000_0000 timepoint 0.84935
 finished parsing run0000_0000 timepoint 0.8494
 finished parsing run0000_0000 timepoint 0.84945
 finished parsing run0000_0000 timepoint 0.8495
 finished parsing run0000_0000 timepoint 0.84955
 finished parsing run0000_0000 timepoint 0.8496
 finished parsing run0000_0000 timepoint 0.84965
 finished parsing run0000_0000 timepoint 0.8497
 finished parsing run0000_0000 timepoint 0.84975
 finished parsing run0000_0000 timepoint 0.8498
 finished parsing run0000_0000 timepoint 0.84985
 finished parsing run0000_0000 timepoint 0.8499
 finished parsing run0000_0000 timepoint 0.84995
 finished parsing run0000_0000 timepoint 0.85
 finished parsing run0000_0000 timepoint 0.85005
 finished parsing run0000_0000 timepoint 0.8501
 finished parsing run0000_0000 timepoint 0.85015
 finished parsing run0000_0000 timepoint 0.8502
 finished parsing run0000_0000 timepoint 0.85025
 finished parsing run0000_0000 timepoint 0.8503
 finished parsing run0000_0000 timepoint 0.85035
 finished parsing run0000_0000 timepoint 0.8504
 finished parsing run0000_0000 timepoint 0.85045
 finished parsing run0000_0000 timepoint 0.8505
 finished parsing run0000_0000 timepoint 0.85055
 finished parsing run0000_0000 timepoint 0.8506
 finished parsing run0000_0000 timepoint 0.85065
 finished parsing run0000_0000 timepoint 0.8507
 finished parsing run0000_0000 timepoint 0.85075
 finished parsing run0000_0000 timepoint 0.8508
 finished parsing run0000_0000 timepoint 0.85085
 finished parsing run0000_0000 timepoint 0.8509
 finished parsing run0000_0000 timepoint 0.85095
 finished parsing run0000_0000 timepoint 0.851
 finished parsing run0000_0000 timepoint 0.85105
 finished parsing run0000_0000 timepoint 0.8511
 finished parsing run0000_0000 timepoint 0.85115
 finished parsing run0000_0000 timepoint 0.8512
 finished parsing run0000_0000 timepoint 0.85125
 finished parsing run0000_0000 timepoint 0.8513
 finished parsing run0000_0000 timepoint 0.85135
 finished parsing run0000_0000 timepoint 0.8514
 finished parsing run0000_0000 timepoint 0.85145
 finished parsing run0000_0000 timepoint 0.8515
 finished parsing run0000_0000 timepoint 0.85155
 finished parsing run0000_0000 timepoint 0.8516
 finished parsing run0000_0000 timepoint 0.85165
 finished parsing run0000_0000 timepoint 0.8517
 finished parsing run0000_0000 timepoint 0.85175
 finished parsing run0000_0000 timepoint 0.8518
 finished parsing run0000_0000 timepoint 0.85185
 finished parsing run0000_0000 timepoint 0.8519
 finished parsing run0000_0000 timepoint 0.85195
 finished parsing run0000_0000 timepoint 0.852
 finished parsing run0000_0000 timepoint 0.85205
 finished parsing run0000_0000 timepoint 0.8521
 finished parsing run0000_0000 timepoint 0.85215
 finished parsing run0000_0000 timepoint 0.8522
 finished parsing run0000_0000 timepoint 0.85225
 finished parsing run0000_0000 timepoint 0.8523
 finished parsing run0000_0000 timepoint 0.85235
 finished parsing run0000_0000 timepoint 0.8524
 finished parsing run0000_0000 timepoint 0.85245
 finished parsing run0000_0000 timepoint 0.8525
 finished parsing run0000_0000 timepoint 0.85255
 finished parsing run0000_0000 timepoint 0.8526
 finished parsing run0000_0000 timepoint 0.85265
 finished parsing run0000_0000 timepoint 0.8527
 finished parsing run0000_0000 timepoint 0.85275
 finished parsing run0000_0000 timepoint 0.8528
 finished parsing run0000_0000 timepoint 0.85285
 finished parsing run0000_0000 timepoint 0.8529
 finished parsing run0000_0000 timepoint 0.85295
 finished parsing run0000_0000 timepoint 0.853
 finished parsing run0000_0000 timepoint 0.85305
 finished parsing run0000_0000 timepoint 0.8531
 finished parsing run0000_0000 timepoint 0.85315
 finished parsing run0000_0000 timepoint 0.8532
 finished parsing run0000_0000 timepoint 0.85325
 finished parsing run0000_0000 timepoint 0.8533
 finished parsing run0000_0000 timepoint 0.85335
 finished parsing run0000_0000 timepoint 0.8534
 finished parsing run0000_0000 timepoint 0.85345
 finished parsing run0000_0000 timepoint 0.8535
 finished parsing run0000_0000 timepoint 0.85355
 finished parsing run0000_0000 timepoint 0.8536
 finished parsing run0000_0000 timepoint 0.85365
 finished parsing run0000_0000 timepoint 0.8537
 finished parsing run0000_0000 timepoint 0.85375
 finished parsing run0000_0000 timepoint 0.8538
 finished parsing run0000_0000 timepoint 0.85385
 finished parsing run0000_0000 timepoint 0.8539
 finished parsing run0000_0000 timepoint 0.85395
 finished parsing run0000_0000 timepoint 0.854
 finished parsing run0000_0000 timepoint 0.85405
 finished parsing run0000_0000 timepoint 0.8541
 finished parsing run0000_0000 timepoint 0.85415
 finished parsing run0000_0000 timepoint 0.8542
 finished parsing run0000_0000 timepoint 0.85425
 finished parsing run0000_0000 timepoint 0.8543
 finished parsing run0000_0000 timepoint 0.85435
 finished parsing run0000_0000 timepoint 0.8544
 finished parsing run0000_0000 timepoint 0.85445
 finished parsing run0000_0000 timepoint 0.8545
 finished parsing run0000_0000 timepoint 0.85455
 finished parsing run0000_0000 timepoint 0.8546
 finished parsing run0000_0000 timepoint 0.85465
 finished parsing run0000_0000 timepoint 0.8547
 finished parsing run0000_0000 timepoint 0.85475
 finished parsing run0000_0000 timepoint 0.8548
 finished parsing run0000_0000 timepoint 0.85485
 finished parsing run0000_0000 timepoint 0.8549
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8418
 finished parsing run0000_0000 timepoint 0.84185
 finished parsing run0000_0000 timepoint 0.8419
 finished parsing run0000_0000 timepoint 0.84195
 finished parsing run0000_0000 timepoint 0.842
 finished parsing run0000_0000 timepoint 0.84205
 finished parsing run0000_0000 timepoint 0.8421
 finished parsing run0000_0000 timepoint 0.84215
 finished parsing run0000_0000 timepoint 0.8422
 finished parsing run0000_0000 timepoint 0.84225
 finished parsing run0000_0000 timepoint 0.8423
 finished parsing run0000_0000 timepoint 0.84235
 finished parsing run0000_0000 timepoint 0.8424
 finished parsing run0000_0000 timepoint 0.84245
 finished parsing run0000_0000 timepoint 0.8425
 finished parsing run0000_0000 timepoint 0.84255
 finished parsing run0000_0000 timepoint 0.8426
 finished parsing run0000_0000 timepoint 0.84265
 finished parsing run0000_0000 timepoint 0.8427
 finished parsing run0000_0000 timepoint 0.84275
 finished parsing run0000_0000 timepoint 0.8428
 finished parsing run0000_0000 timepoint 0.84285
 finished parsing run0000_0000 timepoint 0.8429
 finished parsing run0000_0000 timepoint 0.84295
 finished parsing run0000_0000 timepoint 0.843
 finished parsing run0000_0000 timepoint 0.84305
 finished parsing run0000_0000 timepoint 0.8431
 finished parsing run0000_0000 timepoint 0.84315
 finished parsing run0000_0000 timepoint 0.8432
 finished parsing run0000_0000 timepoint 0.84325
 finished parsing run0000_0000 timepoint 0.8433
 finished parsing run0000_0000 timepoint 0.84335
 finished parsing run0000_0000 timepoint 0.8434
 finished parsing run0000_0000 timepoint 0.84345
 finished parsing run0000_0000 timepoint 0.8435
 finished parsing run0000_0000 timepoint 0.84355
 finished parsing run0000_0000 timepoint 0.8436
 finished parsing run0000_0000 timepoint 0.84365
 finished parsing run0000_0000 timepoint 0.8437
 finished parsing run0000_0000 timepoint 0.84375
 finished parsing run0000_0000 timepoint 0.8438
 finished parsing run0000_0000 timepoint 0.84385
 finished parsing run0000_0000 timepoint 0.8439
 finished parsing run0000_0000 timepoint 0.84395
 finished parsing run0000_0000 timepoint 0.844
 finished parsing run0000_0000 timepoint 0.84405
 finished parsing run0000_0000 timepoint 0.8441
 finished parsing run0000_0000 timepoint 0.84415
 finished parsing run0000_0000 timepoint 0.8442
 finished parsing run0000_0000 timepoint 0.84425
 finished parsing run0000_0000 timepoint 0.8443
 finished parsing run0000_0000 timepoint 0.84435
 finished parsing run0000_0000 timepoint 0.8444
 finished parsing run0000_0000 timepoint 0.84445
 finished parsing run0000_0000 timepoint 0.8445
 finished parsing run0000_0000 timepoint 0.84455
 finished parsing run0000_0000 timepoint 0.8446
 finished parsing run0000_0000 timepoint 0.84465
 finished parsing run0000_0000 timepoint 0.8447
 finished parsing run0000_0000 timepoint 0.84475
 finished parsing run0000_0000 timepoint 0.8448
 finished parsing run0000_0000 timepoint 0.84485
 finished parsing run0000_0000 timepoint 0.8449
 finished parsing run0000_0000 timepoint 0.84495
 finished parsing run0000_0000 timepoint 0.845
 finished parsing run0000_0000 timepoint 0.84505
 finished parsing run0000_0000 timepoint 0.8451
 finished parsing run0000_0000 timepoint 0.84515
 finished parsing run0000_0000 timepoint 0.8452
 finished parsing run0000_0000 timepoint 0.84525
 finished parsing run0000_0000 timepoint 0.8453
 finished parsing run0000_0000 timepoint 0.84535
 finished parsing run0000_0000 timepoint 0.8454
 finished parsing run0000_0000 timepoint 0.84545
 finished parsing run0000_0000 timepoint 0.8455
 finished parsing run0000_0000 timepoint 0.84555
 finished parsing run0000_0000 timepoint 0.8456
 finished parsing run0000_0000 timepoint 0.84565
 finished parsing run0000_0000 timepoint 0.8457
 finished parsing run0000_0000 timepoint 0.84575
 finished parsing run0000_0000 timepoint 0.8458
 finished parsing run0000_0000 timepoint 0.84585
 finished parsing run0000_0000 timepoint 0.8459
 finished parsing run0000_0000 timepoint 0.84595
 finished parsing run0000_0000 timepoint 0.846
 finished parsing run0000_0000 timepoint 0.84605
 finished parsing run0000_0000 timepoint 0.8461
 finished parsing run0000_0000 timepoint 0.84615
 finished parsing run0000_0000 timepoint 0.8462
 finished parsing run0000_0000 timepoint 0.84625
 finished parsing run0000_0000 timepoint 0.8463
 finished parsing run0000_0000 timepoint 0.84635
 finished parsing run0000_0000 timepoint 0.8464
 finished parsing run0000_0000 timepoint 0.84645
 finished parsing run0000_0000 timepoint 0.8465
 finished parsing run0000_0000 timepoint 0.84655
 finished parsing run0000_0000 timepoint 0.8466
 finished parsing run0000_0000 timepoint 0.84665
 finished parsing run0000_0000 timepoint 0.8467
 finished parsing run0000_0000 timepoint 0.84675
 finished parsing run0000_0000 timepoint 0.8468
 finished parsing run0000_0000 timepoint 0.84685
 finished parsing run0000_0000 timepoint 0.8469
 finished parsing run0000_0000 timepoint 0.84695
 finished parsing run0000_0000 timepoint 0.847
 finished parsing run0000_0000 timepoint 0.84705
 finished parsing run0000_0000 timepoint 0.8471
 finished parsing run0000_0000 timepoint 0.84715
 finished parsing run0000_0000 timepoint 0.8472
 finished parsing run0000_0000 timepoint 0.84725
 finished parsing run0000_0000 timepoint 0.8473
 finished parsing run0000_0000 timepoint 0.84735
 finished parsing run0000_0000 timepoint 0.8474
 finished parsing run0000_0000 timepoint 0.84745
 finished parsing run0000_0000 timepoint 0.8475
 finished parsing run0000_0000 timepoint 0.84755
 finished parsing run0000_0000 timepoint 0.8476
 finished parsing run0000_0000 timepoint 0.84765
 finished parsing run0000_0000 timepoint 0.8477
 finished parsing run0000_0000 timepoint 0.84775
 finished parsing run0000_0000 timepoint 0.8478
 finished parsing run0000_0000 timepoint 0.84785
 finished parsing run0000_0000 timepoint 0.8479
 finished parsing run0000_0000 timepoint 0.84795
 finished parsing run0000_0000 timepoint 0.848
 finished parsing run0000_0000 timepoint 0.84805
 finished parsing run0000_0000 timepoint 0.8481
 finished parsing run0000_0000 timepoint 0.84815
 finished parsing run0000_0000 timepoint 0.8482
 finished parsing run0000_0000 timepoint 0.84825
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8354
 finished parsing run0000_0000 timepoint 0.83545
 finished parsing run0000_0000 timepoint 0.8355
 finished parsing run0000_0000 timepoint 0.83555
 finished parsing run0000_0000 timepoint 0.8356
 finished parsing run0000_0000 timepoint 0.83565
 finished parsing run0000_0000 timepoint 0.8357
 finished parsing run0000_0000 timepoint 0.83575
 finished parsing run0000_0000 timepoint 0.8358
 finished parsing run0000_0000 timepoint 0.83585
 finished parsing run0000_0000 timepoint 0.8359
 finished parsing run0000_0000 timepoint 0.83595
 finished parsing run0000_0000 timepoint 0.836
 finished parsing run0000_0000 timepoint 0.83605
 finished parsing run0000_0000 timepoint 0.8361
 finished parsing run0000_0000 timepoint 0.83615
 finished parsing run0000_0000 timepoint 0.8362
 finished parsing run0000_0000 timepoint 0.83625
 finished parsing run0000_0000 timepoint 0.8363
 finished parsing run0000_0000 timepoint 0.83635
 finished parsing run0000_0000 timepoint 0.8364
 finished parsing run0000_0000 timepoint 0.83645
 finished parsing run0000_0000 timepoint 0.8365
 finished parsing run0000_0000 timepoint 0.83655
 finished parsing run0000_0000 timepoint 0.8366
 finished parsing run0000_0000 timepoint 0.83665
 finished parsing run0000_0000 timepoint 0.8367
 finished parsing run0000_0000 timepoint 0.83675
 finished parsing run0000_0000 timepoint 0.8368
 finished parsing run0000_0000 timepoint 0.83685
 finished parsing run0000_0000 timepoint 0.8369
 finished parsing run0000_0000 timepoint 0.83695
 finished parsing run0000_0000 timepoint 0.837
 finished parsing run0000_0000 timepoint 0.83705
 finished parsing run0000_0000 timepoint 0.8371
 finished parsing run0000_0000 timepoint 0.83715
 finished parsing run0000_0000 timepoint 0.8372
 finished parsing run0000_0000 timepoint 0.83725
 finished parsing run0000_0000 timepoint 0.8373
 finished parsing run0000_0000 timepoint 0.83735
 finished parsing run0000_0000 timepoint 0.8374
 finished parsing run0000_0000 timepoint 0.83745
 finished parsing run0000_0000 timepoint 0.8375
 finished parsing run0000_0000 timepoint 0.83755
 finished parsing run0000_0000 timepoint 0.8376
 finished parsing run0000_0000 timepoint 0.83765
 finished parsing run0000_0000 timepoint 0.8377
 finished parsing run0000_0000 timepoint 0.83775
 finished parsing run0000_0000 timepoint 0.8378
 finished parsing run0000_0000 timepoint 0.83785
 finished parsing run0000_0000 timepoint 0.8379
 finished parsing run0000_0000 timepoint 0.83795
 finished parsing run0000_0000 timepoint 0.838
 finished parsing run0000_0000 timepoint 0.83805
 finished parsing run0000_0000 timepoint 0.8381
 finished parsing run0000_0000 timepoint 0.83815
 finished parsing run0000_0000 timepoint 0.8382
 finished parsing run0000_0000 timepoint 0.83825
 finished parsing run0000_0000 timepoint 0.8383
 finished parsing run0000_0000 timepoint 0.83835
 finished parsing run0000_0000 timepoint 0.8384
 finished parsing run0000_0000 timepoint 0.83845
 finished parsing run0000_0000 timepoint 0.8385
 finished parsing run0000_0000 timepoint 0.83855
 finished parsing run0000_0000 timepoint 0.8386
 finished parsing run0000_0000 timepoint 0.83865
 finished parsing run0000_0000 timepoint 0.8387
 finished parsing run0000_0000 timepoint 0.83875
 finished parsing run0000_0000 timepoint 0.8388
 finished parsing run0000_0000 timepoint 0.83885
 finished parsing run0000_0000 timepoint 0.8389
 finished parsing run0000_0000 timepoint 0.83895
 finished parsing run0000_0000 timepoint 0.839
 finished parsing run0000_0000 timepoint 0.83905
 finished parsing run0000_0000 timepoint 0.8391
 finished parsing run0000_0000 timepoint 0.83915
 finished parsing run0000_0000 timepoint 0.8392
 finished parsing run0000_0000 timepoint 0.83925
 finished parsing run0000_0000 timepoint 0.8393
 finished parsing run0000_0000 timepoint 0.83935
 finished parsing run0000_0000 timepoint 0.8394
 finished parsing run0000_0000 timepoint 0.83945
 finished parsing run0000_0000 timepoint 0.8395
 finished parsing run0000_0000 timepoint 0.83955
 finished parsing run0000_0000 timepoint 0.8396
 finished parsing run0000_0000 timepoint 0.83965
 finished parsing run0000_0000 timepoint 0.8397
 finished parsing run0000_0000 timepoint 0.83975
 finished parsing run0000_0000 timepoint 0.8398
 finished parsing run0000_0000 timepoint 0.83985
 finished parsing run0000_0000 timepoint 0.8399
 finished parsing run0000_0000 timepoint 0.83995
 finished parsing run0000_0000 timepoint 0.84
 finished parsing run0000_0000 timepoint 0.84005
 finished parsing run0000_0000 timepoint 0.8401
 finished parsing run0000_0000 timepoint 0.84015
 finished parsing run0000_0000 timepoint 0.8402
 finished parsing run0000_0000 timepoint 0.84025
 finished parsing run0000_0000 timepoint 0.8403
 finished parsing run0000_0000 timepoint 0.84035
 finished parsing run0000_0000 timepoint 0.8404
 finished parsing run0000_0000 timepoint 0.84045
 finished parsing run0000_0000 timepoint 0.8405
 finished parsing run0000_0000 timepoint 0.84055
 finished parsing run0000_0000 timepoint 0.8406
 finished parsing run0000_0000 timepoint 0.84065
 finished parsing run0000_0000 timepoint 0.8407
 finished parsing run0000_0000 timepoint 0.84075
 finished parsing run0000_0000 timepoint 0.8408
 finished parsing run0000_0000 timepoint 0.84085
 finished parsing run0000_0000 timepoint 0.8409
 finished parsing run0000_0000 timepoint 0.84095
 finished parsing run0000_0000 timepoint 0.841
 finished parsing run0000_0000 timepoint 0.84105
 finished parsing run0000_0000 timepoint 0.8411
 finished parsing run0000_0000 timepoint 0.84115
 finished parsing run0000_0000 timepoint 0.8412
 finished parsing run0000_0000 timepoint 0.84125
 finished parsing run0000_0000 timepoint 0.8413
 finished parsing run0000_0000 timepoint 0.84135
 finished parsing run0000_0000 timepoint 0.8414
 finished parsing run0000_0000 timepoint 0.84145
 finished parsing run0000_0000 timepoint 0.8415
 finished parsing run0000_0000 timepoint 0.84155
 finished parsing run0000_0000 timepoint 0.8416
 finished parsing run0000_0000 timepoint 0.84165
 finished parsing run0000_0000 timepoint 0.8417
 finished parsing run0000_0000 timepoint 0.84175
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8288
 finished parsing run0000_0000 timepoint 0.82885
 finished parsing run0000_0000 timepoint 0.8289
 finished parsing run0000_0000 timepoint 0.82895
 finished parsing run0000_0000 timepoint 0.829
 finished parsing run0000_0000 timepoint 0.82905
 finished parsing run0000_0000 timepoint 0.8291
 finished parsing run0000_0000 timepoint 0.82915
 finished parsing run0000_0000 timepoint 0.8292
 finished parsing run0000_0000 timepoint 0.82925
 finished parsing run0000_0000 timepoint 0.8293
 finished parsing run0000_0000 timepoint 0.82935
 finished parsing run0000_0000 timepoint 0.8294
 finished parsing run0000_0000 timepoint 0.82945
 finished parsing run0000_0000 timepoint 0.8295
 finished parsing run0000_0000 timepoint 0.82955
 finished parsing run0000_0000 timepoint 0.8296
 finished parsing run0000_0000 timepoint 0.82965
 finished parsing run0000_0000 timepoint 0.8297
 finished parsing run0000_0000 timepoint 0.82975
 finished parsing run0000_0000 timepoint 0.8298
 finished parsing run0000_0000 timepoint 0.82985
 finished parsing run0000_0000 timepoint 0.8299
 finished parsing run0000_0000 timepoint 0.82995
 finished parsing run0000_0000 timepoint 0.83
 finished parsing run0000_0000 timepoint 0.83005
 finished parsing run0000_0000 timepoint 0.8301
 finished parsing run0000_0000 timepoint 0.83015
 finished parsing run0000_0000 timepoint 0.8302
 finished parsing run0000_0000 timepoint 0.83025
 finished parsing run0000_0000 timepoint 0.8303
 finished parsing run0000_0000 timepoint 0.83035
 finished parsing run0000_0000 timepoint 0.8304
 finished parsing run0000_0000 timepoint 0.83045
 finished parsing run0000_0000 timepoint 0.8305
 finished parsing run0000_0000 timepoint 0.83055
 finished parsing run0000_0000 timepoint 0.8306
 finished parsing run0000_0000 timepoint 0.83065
 finished parsing run0000_0000 timepoint 0.8307
 finished parsing run0000_0000 timepoint 0.83075
 finished parsing run0000_0000 timepoint 0.8308
 finished parsing run0000_0000 timepoint 0.83085
 finished parsing run0000_0000 timepoint 0.8309
 finished parsing run0000_0000 timepoint 0.83095
 finished parsing run0000_0000 timepoint 0.831
 finished parsing run0000_0000 timepoint 0.83105
 finished parsing run0000_0000 timepoint 0.8311
 finished parsing run0000_0000 timepoint 0.83115
 finished parsing run0000_0000 timepoint 0.8312
 finished parsing run0000_0000 timepoint 0.83125
 finished parsing run0000_0000 timepoint 0.8313
 finished parsing run0000_0000 timepoint 0.83135
 finished parsing run0000_0000 timepoint 0.8314
 finished parsing run0000_0000 timepoint 0.83145
 finished parsing run0000_0000 timepoint 0.8315
 finished parsing run0000_0000 timepoint 0.83155
 finished parsing run0000_0000 timepoint 0.8316
 finished parsing run0000_0000 timepoint 0.83165
 finished parsing run0000_0000 timepoint 0.8317
 finished parsing run0000_0000 timepoint 0.83175
 finished parsing run0000_0000 timepoint 0.8318
 finished parsing run0000_0000 timepoint 0.83185
 finished parsing run0000_0000 timepoint 0.8319
 finished parsing run0000_0000 timepoint 0.83195
 finished parsing run0000_0000 timepoint 0.832
 finished parsing run0000_0000 timepoint 0.83205
 finished parsing run0000_0000 timepoint 0.8321
 finished parsing run0000_0000 timepoint 0.83215
 finished parsing run0000_0000 timepoint 0.8322
 finished parsing run0000_0000 timepoint 0.83225
 finished parsing run0000_0000 timepoint 0.8323
 finished parsing run0000_0000 timepoint 0.83235
 finished parsing run0000_0000 timepoint 0.8324
 finished parsing run0000_0000 timepoint 0.83245
 finished parsing run0000_0000 timepoint 0.8325
 finished parsing run0000_0000 timepoint 0.83255
 finished parsing run0000_0000 timepoint 0.8326
 finished parsing run0000_0000 timepoint 0.83265
 finished parsing run0000_0000 timepoint 0.8327
 finished parsing run0000_0000 timepoint 0.83275
 finished parsing run0000_0000 timepoint 0.8328
 finished parsing run0000_0000 timepoint 0.83285
 finished parsing run0000_0000 timepoint 0.8329
 finished parsing run0000_0000 timepoint 0.83295
 finished parsing run0000_0000 timepoint 0.833
 finished parsing run0000_0000 timepoint 0.83305
 finished parsing run0000_0000 timepoint 0.8331
 finished parsing run0000_0000 timepoint 0.83315
 finished parsing run0000_0000 timepoint 0.8332
 finished parsing run0000_0000 timepoint 0.83325
 finished parsing run0000_0000 timepoint 0.8333
 finished parsing run0000_0000 timepoint 0.83335
 finished parsing run0000_0000 timepoint 0.8334
 finished parsing run0000_0000 timepoint 0.83345
 finished parsing run0000_0000 timepoint 0.8335
 finished parsing run0000_0000 timepoint 0.83355
 finished parsing run0000_0000 timepoint 0.8336
 finished parsing run0000_0000 timepoint 0.83365
 finished parsing run0000_0000 timepoint 0.8337
 finished parsing run0000_0000 timepoint 0.83375
 finished parsing run0000_0000 timepoint 0.8338
 finished parsing run0000_0000 timepoint 0.83385
 finished parsing run0000_0000 timepoint 0.8339
 finished parsing run0000_0000 timepoint 0.83395
 finished parsing run0000_0000 timepoint 0.834
 finished parsing run0000_0000 timepoint 0.83405
 finished parsing run0000_0000 timepoint 0.8341
 finished parsing run0000_0000 timepoint 0.83415
 finished parsing run0000_0000 timepoint 0.8342
 finished parsing run0000_0000 timepoint 0.83425
 finished parsing run0000_0000 timepoint 0.8343
 finished parsing run0000_0000 timepoint 0.83435
 finished parsing run0000_0000 timepoint 0.8344
 finished parsing run0000_0000 timepoint 0.83445
 finished parsing run0000_0000 timepoint 0.8345
 finished parsing run0000_0000 timepoint 0.83455
 finished parsing run0000_0000 timepoint 0.8346
 finished parsing run0000_0000 timepoint 0.83465
 finished parsing run0000_0000 timepoint 0.8347
 finished parsing run0000_0000 timepoint 0.83475
 finished parsing run0000_0000 timepoint 0.8348
 finished parsing run0000_0000 timepoint 0.83485
 finished parsing run0000_0000 timepoint 0.8349
 finished parsing run0000_0000 timepoint 0.83495
 finished parsing run0000_0000 timepoint 0.835
 finished parsing run0000_0000 timepoint 0.83505
 finished parsing run0000_0000 timepoint 0.8351
 finished parsing run0000_0000 timepoint 0.83515
 finished parsing run0000_0000 timepoint 0.8352
 finished parsing run0000_0000 timepoint 0.83525
 finished parsing run0000_0000 timepoint 0.8353
 finished parsing run0000_0000 timepoint 0.83535
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.82235
 finished parsing run0000_0000 timepoint 0.8224
 finished parsing run0000_0000 timepoint 0.82245
 finished parsing run0000_0000 timepoint 0.8225
 finished parsing run0000_0000 timepoint 0.82255
 finished parsing run0000_0000 timepoint 0.8226
 finished parsing run0000_0000 timepoint 0.82265
 finished parsing run0000_0000 timepoint 0.8227
 finished parsing run0000_0000 timepoint 0.82275
 finished parsing run0000_0000 timepoint 0.8228
 finished parsing run0000_0000 timepoint 0.82285
 finished parsing run0000_0000 timepoint 0.8229
 finished parsing run0000_0000 timepoint 0.82295
 finished parsing run0000_0000 timepoint 0.823
 finished parsing run0000_0000 timepoint 0.82305
 finished parsing run0000_0000 timepoint 0.8231
 finished parsing run0000_0000 timepoint 0.82315
 finished parsing run0000_0000 timepoint 0.8232
 finished parsing run0000_0000 timepoint 0.82325
 finished parsing run0000_0000 timepoint 0.8233
 finished parsing run0000_0000 timepoint 0.82335
 finished parsing run0000_0000 timepoint 0.8234
 finished parsing run0000_0000 timepoint 0.82345
 finished parsing run0000_0000 timepoint 0.8235
 finished parsing run0000_0000 timepoint 0.82355
 finished parsing run0000_0000 timepoint 0.8236
 finished parsing run0000_0000 timepoint 0.82365
 finished parsing run0000_0000 timepoint 0.8237
 finished parsing run0000_0000 timepoint 0.82375
 finished parsing run0000_0000 timepoint 0.8238
 finished parsing run0000_0000 timepoint 0.82385
 finished parsing run0000_0000 timepoint 0.8239
 finished parsing run0000_0000 timepoint 0.82395
 finished parsing run0000_0000 timepoint 0.824
 finished parsing run0000_0000 timepoint 0.82405
 finished parsing run0000_0000 timepoint 0.8241
 finished parsing run0000_0000 timepoint 0.82415
 finished parsing run0000_0000 timepoint 0.8242
 finished parsing run0000_0000 timepoint 0.82425
 finished parsing run0000_0000 timepoint 0.8243
 finished parsing run0000_0000 timepoint 0.82435
 finished parsing run0000_0000 timepoint 0.8244
 finished parsing run0000_0000 timepoint 0.82445
 finished parsing run0000_0000 timepoint 0.8245
 finished parsing run0000_0000 timepoint 0.82455
 finished parsing run0000_0000 timepoint 0.8246
 finished parsing run0000_0000 timepoint 0.82465
 finished parsing run0000_0000 timepoint 0.8247
 finished parsing run0000_0000 timepoint 0.82475
 finished parsing run0000_0000 timepoint 0.8248
 finished parsing run0000_0000 timepoint 0.82485
 finished parsing run0000_0000 timepoint 0.8249
 finished parsing run0000_0000 timepoint 0.82495
 finished parsing run0000_0000 timepoint 0.825
 finished parsing run0000_0000 timepoint 0.82505
 finished parsing run0000_0000 timepoint 0.8251
 finished parsing run0000_0000 timepoint 0.82515
 finished parsing run0000_0000 timepoint 0.8252
 finished parsing run0000_0000 timepoint 0.82525
 finished parsing run0000_0000 timepoint 0.8253
 finished parsing run0000_0000 timepoint 0.82535
 finished parsing run0000_0000 timepoint 0.8254
 finished parsing run0000_0000 timepoint 0.82545
 finished parsing run0000_0000 timepoint 0.8255
 finished parsing run0000_0000 timepoint 0.82555
 finished parsing run0000_0000 timepoint 0.8256
 finished parsing run0000_0000 timepoint 0.82565
 finished parsing run0000_0000 timepoint 0.8257
 finished parsing run0000_0000 timepoint 0.82575
 finished parsing run0000_0000 timepoint 0.8258
 finished parsing run0000_0000 timepoint 0.82585
 finished parsing run0000_0000 timepoint 0.8259
 finished parsing run0000_0000 timepoint 0.82595
 finished parsing run0000_0000 timepoint 0.826
 finished parsing run0000_0000 timepoint 0.82605
 finished parsing run0000_0000 timepoint 0.8261
 finished parsing run0000_0000 timepoint 0.82615
 finished parsing run0000_0000 timepoint 0.8262
 finished parsing run0000_0000 timepoint 0.82625
 finished parsing run0000_0000 timepoint 0.8263
 finished parsing run0000_0000 timepoint 0.82635
 finished parsing run0000_0000 timepoint 0.8264
 finished parsing run0000_0000 timepoint 0.82645
 finished parsing run0000_0000 timepoint 0.8265
 finished parsing run0000_0000 timepoint 0.82655
 finished parsing run0000_0000 timepoint 0.8266
 finished parsing run0000_0000 timepoint 0.82665
 finished parsing run0000_0000 timepoint 0.8267
 finished parsing run0000_0000 timepoint 0.82675
 finished parsing run0000_0000 timepoint 0.8268
 finished parsing run0000_0000 timepoint 0.82685
 finished parsing run0000_0000 timepoint 0.8269
 finished parsing run0000_0000 timepoint 0.82695
 finished parsing run0000_0000 timepoint 0.827
 finished parsing run0000_0000 timepoint 0.82705
 finished parsing run0000_0000 timepoint 0.8271
 finished parsing run0000_0000 timepoint 0.82715
 finished parsing run0000_0000 timepoint 0.8272
 finished parsing run0000_0000 timepoint 0.82725
 finished parsing run0000_0000 timepoint 0.8273
 finished parsing run0000_0000 timepoint 0.82735
 finished parsing run0000_0000 timepoint 0.8274
 finished parsing run0000_0000 timepoint 0.82745
 finished parsing run0000_0000 timepoint 0.8275
 finished parsing run0000_0000 timepoint 0.82755
 finished parsing run0000_0000 timepoint 0.8276
 finished parsing run0000_0000 timepoint 0.82765
 finished parsing run0000_0000 timepoint 0.8277
 finished parsing run0000_0000 timepoint 0.82775
 finished parsing run0000_0000 timepoint 0.8278
 finished parsing run0000_0000 timepoint 0.82785
 finished parsing run0000_0000 timepoint 0.8279
 finished parsing run0000_0000 timepoint 0.82795
 finished parsing run0000_0000 timepoint 0.828
 finished parsing run0000_0000 timepoint 0.82805
 finished parsing run0000_0000 timepoint 0.8281
 finished parsing run0000_0000 timepoint 0.82815
 finished parsing run0000_0000 timepoint 0.8282
 finished parsing run0000_0000 timepoint 0.82825
 finished parsing run0000_0000 timepoint 0.8283
 finished parsing run0000_0000 timepoint 0.82835
 finished parsing run0000_0000 timepoint 0.8284
 finished parsing run0000_0000 timepoint 0.82845
 finished parsing run0000_0000 timepoint 0.8285
 finished parsing run0000_0000 timepoint 0.82855
 finished parsing run0000_0000 timepoint 0.8286
 finished parsing run0000_0000 timepoint 0.82865
 finished parsing run0000_0000 timepoint 0.8287
 finished parsing run0000_0000 timepoint 0.82875
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.81575
 finished parsing run0000_0000 timepoint 0.8158
 finished parsing run0000_0000 timepoint 0.81585
 finished parsing run0000_0000 timepoint 0.8159
 finished parsing run0000_0000 timepoint 0.81595
 finished parsing run0000_0000 timepoint 0.816
 finished parsing run0000_0000 timepoint 0.81605
 finished parsing run0000_0000 timepoint 0.8161
 finished parsing run0000_0000 timepoint 0.81615
 finished parsing run0000_0000 timepoint 0.8162
 finished parsing run0000_0000 timepoint 0.81625
 finished parsing run0000_0000 timepoint 0.8163
 finished parsing run0000_0000 timepoint 0.81635
 finished parsing run0000_0000 timepoint 0.8164
 finished parsing run0000_0000 timepoint 0.81645
 finished parsing run0000_0000 timepoint 0.8165
 finished parsing run0000_0000 timepoint 0.81655
 finished parsing run0000_0000 timepoint 0.8166
 finished parsing run0000_0000 timepoint 0.81665
 finished parsing run0000_0000 timepoint 0.8167
 finished parsing run0000_0000 timepoint 0.81675
 finished parsing run0000_0000 timepoint 0.8168
 finished parsing run0000_0000 timepoint 0.81685
 finished parsing run0000_0000 timepoint 0.8169
 finished parsing run0000_0000 timepoint 0.81695
 finished parsing run0000_0000 timepoint 0.817
 finished parsing run0000_0000 timepoint 0.81705
 finished parsing run0000_0000 timepoint 0.8171
 finished parsing run0000_0000 timepoint 0.81715
 finished parsing run0000_0000 timepoint 0.8172
 finished parsing run0000_0000 timepoint 0.81725
 finished parsing run0000_0000 timepoint 0.8173
 finished parsing run0000_0000 timepoint 0.81735
 finished parsing run0000_0000 timepoint 0.8174
 finished parsing run0000_0000 timepoint 0.81745
 finished parsing run0000_0000 timepoint 0.8175
 finished parsing run0000_0000 timepoint 0.81755
 finished parsing run0000_0000 timepoint 0.8176
 finished parsing run0000_0000 timepoint 0.81765
 finished parsing run0000_0000 timepoint 0.8177
 finished parsing run0000_0000 timepoint 0.81775
 finished parsing run0000_0000 timepoint 0.8178
 finished parsing run0000_0000 timepoint 0.81785
 finished parsing run0000_0000 timepoint 0.8179
 finished parsing run0000_0000 timepoint 0.81795
 finished parsing run0000_0000 timepoint 0.818
 finished parsing run0000_0000 timepoint 0.81805
 finished parsing run0000_0000 timepoint 0.8181
 finished parsing run0000_0000 timepoint 0.81815
 finished parsing run0000_0000 timepoint 0.8182
 finished parsing run0000_0000 timepoint 0.81825
 finished parsing run0000_0000 timepoint 0.8183
 finished parsing run0000_0000 timepoint 0.81835
 finished parsing run0000_0000 timepoint 0.8184
 finished parsing run0000_0000 timepoint 0.81845
 finished parsing run0000_0000 timepoint 0.8185
 finished parsing run0000_0000 timepoint 0.81855
 finished parsing run0000_0000 timepoint 0.8186
 finished parsing run0000_0000 timepoint 0.81865
 finished parsing run0000_0000 timepoint 0.8187
 finished parsing run0000_0000 timepoint 0.81875
 finished parsing run0000_0000 timepoint 0.8188
 finished parsing run0000_0000 timepoint 0.81885
 finished parsing run0000_0000 timepoint 0.8189
 finished parsing run0000_0000 timepoint 0.81895
 finished parsing run0000_0000 timepoint 0.819
 finished parsing run0000_0000 timepoint 0.81905
 finished parsing run0000_0000 timepoint 0.8191
 finished parsing run0000_0000 timepoint 0.81915
 finished parsing run0000_0000 timepoint 0.8192
 finished parsing run0000_0000 timepoint 0.81925
 finished parsing run0000_0000 timepoint 0.8193
 finished parsing run0000_0000 timepoint 0.81935
 finished parsing run0000_0000 timepoint 0.8194
 finished parsing run0000_0000 timepoint 0.81945
 finished parsing run0000_0000 timepoint 0.8195
 finished parsing run0000_0000 timepoint 0.81955
 finished parsing run0000_0000 timepoint 0.8196
 finished parsing run0000_0000 timepoint 0.81965
 finished parsing run0000_0000 timepoint 0.8197
 finished parsing run0000_0000 timepoint 0.81975
 finished parsing run0000_0000 timepoint 0.8198
 finished parsing run0000_0000 timepoint 0.81985
 finished parsing run0000_0000 timepoint 0.8199
 finished parsing run0000_0000 timepoint 0.81995
 finished parsing run0000_0000 timepoint 0.82
 finished parsing run0000_0000 timepoint 0.82005
 finished parsing run0000_0000 timepoint 0.8201
 finished parsing run0000_0000 timepoint 0.82015
 finished parsing run0000_0000 timepoint 0.8202
 finished parsing run0000_0000 timepoint 0.82025
 finished parsing run0000_0000 timepoint 0.8203
 finished parsing run0000_0000 timepoint 0.82035
 finished parsing run0000_0000 timepoint 0.8204
 finished parsing run0000_0000 timepoint 0.82045
 finished parsing run0000_0000 timepoint 0.8205
 finished parsing run0000_0000 timepoint 0.82055
 finished parsing run0000_0000 timepoint 0.8206
 finished parsing run0000_0000 timepoint 0.82065
 finished parsing run0000_0000 timepoint 0.8207
 finished parsing run0000_0000 timepoint 0.82075
 finished parsing run0000_0000 timepoint 0.8208
 finished parsing run0000_0000 timepoint 0.82085
 finished parsing run0000_0000 timepoint 0.8209
 finished parsing run0000_0000 timepoint 0.82095
 finished parsing run0000_0000 timepoint 0.821
 finished parsing run0000_0000 timepoint 0.82105
 finished parsing run0000_0000 timepoint 0.8211
 finished parsing run0000_0000 timepoint 0.82115
 finished parsing run0000_0000 timepoint 0.8212
 finished parsing run0000_0000 timepoint 0.82125
 finished parsing run0000_0000 timepoint 0.8213
 finished parsing run0000_0000 timepoint 0.82135
 finished parsing run0000_0000 timepoint 0.8214
 finished parsing run0000_0000 timepoint 0.82145
 finished parsing run0000_0000 timepoint 0.8215
 finished parsing run0000_0000 timepoint 0.82155
 finished parsing run0000_0000 timepoint 0.8216
 finished parsing run0000_0000 timepoint 0.82165
 finished parsing run0000_0000 timepoint 0.8217
 finished parsing run0000_0000 timepoint 0.82175
 finished parsing run0000_0000 timepoint 0.8218
 finished parsing run0000_0000 timepoint 0.82185
 finished parsing run0000_0000 timepoint 0.8219
 finished parsing run0000_0000 timepoint 0.82195
 finished parsing run0000_0000 timepoint 0.822
 finished parsing run0000_0000 timepoint 0.82205
 finished parsing run0000_0000 timepoint 0.8221
 finished parsing run0000_0000 timepoint 0.82215
 finished parsing run0000_0000 timepoint 0.8222
 finished parsing run0000_0000 timepoint 0.82225
 finished parsing run0000_0000 timepoint 0.8223
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.80905
 finished parsing run0000_0000 timepoint 0.8091
 finished parsing run0000_0000 timepoint 0.80915
 finished parsing run0000_0000 timepoint 0.8092
 finished parsing run0000_0000 timepoint 0.80925
 finished parsing run0000_0000 timepoint 0.8093
 finished parsing run0000_0000 timepoint 0.80935
 finished parsing run0000_0000 timepoint 0.8094
 finished parsing run0000_0000 timepoint 0.80945
 finished parsing run0000_0000 timepoint 0.8095
 finished parsing run0000_0000 timepoint 0.80955
 finished parsing run0000_0000 timepoint 0.8096
 finished parsing run0000_0000 timepoint 0.80965
 finished parsing run0000_0000 timepoint 0.8097
 finished parsing run0000_0000 timepoint 0.80975
 finished parsing run0000_0000 timepoint 0.8098
 finished parsing run0000_0000 timepoint 0.80985
 finished parsing run0000_0000 timepoint 0.8099
 finished parsing run0000_0000 timepoint 0.80995
 finished parsing run0000_0000 timepoint 0.81
 finished parsing run0000_0000 timepoint 0.81005
 finished parsing run0000_0000 timepoint 0.8101
 finished parsing run0000_0000 timepoint 0.81015
 finished parsing run0000_0000 timepoint 0.8102
 finished parsing run0000_0000 timepoint 0.81025
 finished parsing run0000_0000 timepoint 0.8103
 finished parsing run0000_0000 timepoint 0.81035
 finished parsing run0000_0000 timepoint 0.8104
 finished parsing run0000_0000 timepoint 0.81045
 finished parsing run0000_0000 timepoint 0.8105
 finished parsing run0000_0000 timepoint 0.81055
 finished parsing run0000_0000 timepoint 0.8106
 finished parsing run0000_0000 timepoint 0.81065
 finished parsing run0000_0000 timepoint 0.8107
 finished parsing run0000_0000 timepoint 0.81075
 finished parsing run0000_0000 timepoint 0.8108
 finished parsing run0000_0000 timepoint 0.81085
 finished parsing run0000_0000 timepoint 0.8109
 finished parsing run0000_0000 timepoint 0.81095
 finished parsing run0000_0000 timepoint 0.811
 finished parsing run0000_0000 timepoint 0.81105
 finished parsing run0000_0000 timepoint 0.8111
 finished parsing run0000_0000 timepoint 0.81115
 finished parsing run0000_0000 timepoint 0.8112
 finished parsing run0000_0000 timepoint 0.81125
 finished parsing run0000_0000 timepoint 0.8113
 finished parsing run0000_0000 timepoint 0.81135
 finished parsing run0000_0000 timepoint 0.8114
 finished parsing run0000_0000 timepoint 0.81145
 finished parsing run0000_0000 timepoint 0.8115
 finished parsing run0000_0000 timepoint 0.81155
 finished parsing run0000_0000 timepoint 0.8116
 finished parsing run0000_0000 timepoint 0.81165
 finished parsing run0000_0000 timepoint 0.8117
 finished parsing run0000_0000 timepoint 0.81175
 finished parsing run0000_0000 timepoint 0.8118
 finished parsing run0000_0000 timepoint 0.81185
 finished parsing run0000_0000 timepoint 0.8119
 finished parsing run0000_0000 timepoint 0.81195
 finished parsing run0000_0000 timepoint 0.812
 finished parsing run0000_0000 timepoint 0.81205
 finished parsing run0000_0000 timepoint 0.8121
 finished parsing run0000_0000 timepoint 0.81215
 finished parsing run0000_0000 timepoint 0.8122
 finished parsing run0000_0000 timepoint 0.81225
 finished parsing run0000_0000 timepoint 0.8123
 finished parsing run0000_0000 timepoint 0.81235
 finished parsing run0000_0000 timepoint 0.8124
 finished parsing run0000_0000 timepoint 0.81245
 finished parsing run0000_0000 timepoint 0.8125
 finished parsing run0000_0000 timepoint 0.81255
 finished parsing run0000_0000 timepoint 0.8126
 finished parsing run0000_0000 timepoint 0.81265
 finished parsing run0000_0000 timepoint 0.8127
 finished parsing run0000_0000 timepoint 0.81275
 finished parsing run0000_0000 timepoint 0.8128
 finished parsing run0000_0000 timepoint 0.81285
 finished parsing run0000_0000 timepoint 0.8129
 finished parsing run0000_0000 timepoint 0.81295
 finished parsing run0000_0000 timepoint 0.813
 finished parsing run0000_0000 timepoint 0.81305
 finished parsing run0000_0000 timepoint 0.8131
 finished parsing run0000_0000 timepoint 0.81315
 finished parsing run0000_0000 timepoint 0.8132
 finished parsing run0000_0000 timepoint 0.81325
 finished parsing run0000_0000 timepoint 0.8133
 finished parsing run0000_0000 timepoint 0.81335
 finished parsing run0000_0000 timepoint 0.8134
 finished parsing run0000_0000 timepoint 0.81345
 finished parsing run0000_0000 timepoint 0.8135
 finished parsing run0000_0000 timepoint 0.81355
 finished parsing run0000_0000 timepoint 0.8136
 finished parsing run0000_0000 timepoint 0.81365
 finished parsing run0000_0000 timepoint 0.8137
 finished parsing run0000_0000 timepoint 0.81375
 finished parsing run0000_0000 timepoint 0.8138
 finished parsing run0000_0000 timepoint 0.81385
 finished parsing run0000_0000 timepoint 0.8139
 finished parsing run0000_0000 timepoint 0.81395
 finished parsing run0000_0000 timepoint 0.814
 finished parsing run0000_0000 timepoint 0.81405
 finished parsing run0000_0000 timepoint 0.8141
 finished parsing run0000_0000 timepoint 0.81415
 finished parsing run0000_0000 timepoint 0.8142
 finished parsing run0000_0000 timepoint 0.81425
 finished parsing run0000_0000 timepoint 0.8143
 finished parsing run0000_0000 timepoint 0.81435
 finished parsing run0000_0000 timepoint 0.8144
 finished parsing run0000_0000 timepoint 0.81445
 finished parsing run0000_0000 timepoint 0.8145
 finished parsing run0000_0000 timepoint 0.81455
 finished parsing run0000_0000 timepoint 0.8146
 finished parsing run0000_0000 timepoint 0.81465
 finished parsing run0000_0000 timepoint 0.8147
 finished parsing run0000_0000 timepoint 0.81475
 finished parsing run0000_0000 timepoint 0.8148
 finished parsing run0000_0000 timepoint 0.81485
 finished parsing run0000_0000 timepoint 0.8149
 finished parsing run0000_0000 timepoint 0.81495
 finished parsing run0000_0000 timepoint 0.815
 finished parsing run0000_0000 timepoint 0.81505
 finished parsing run0000_0000 timepoint 0.8151
 finished parsing run0000_0000 timepoint 0.81515
 finished parsing run0000_0000 timepoint 0.8152
 finished parsing run0000_0000 timepoint 0.81525
 finished parsing run0000_0000 timepoint 0.8153
 finished parsing run0000_0000 timepoint 0.81535
 finished parsing run0000_0000 timepoint 0.8154
 finished parsing run0000_0000 timepoint 0.81545
 finished parsing run0000_0000 timepoint 0.8155
 finished parsing run0000_0000 timepoint 0.81555
 finished parsing run0000_0000 timepoint 0.8156
 finished parsing run0000_0000 timepoint 0.81565
 finished parsing run0000_0000 timepoint 0.8157
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.8023
 finished parsing run0000_0000 timepoint 0.80235
 finished parsing run0000_0000 timepoint 0.8024
 finished parsing run0000_0000 timepoint 0.80245
 finished parsing run0000_0000 timepoint 0.8025
 finished parsing run0000_0000 timepoint 0.80255
 finished parsing run0000_0000 timepoint 0.8026
 finished parsing run0000_0000 timepoint 0.80265
 finished parsing run0000_0000 timepoint 0.8027
 finished parsing run0000_0000 timepoint 0.80275
 finished parsing run0000_0000 timepoint 0.8028
 finished parsing run0000_0000 timepoint 0.80285
 finished parsing run0000_0000 timepoint 0.8029
 finished parsing run0000_0000 timepoint 0.80295
 finished parsing run0000_0000 timepoint 0.803
 finished parsing run0000_0000 timepoint 0.80305
 finished parsing run0000_0000 timepoint 0.8031
 finished parsing run0000_0000 timepoint 0.80315
 finished parsing run0000_0000 timepoint 0.8032
 finished parsing run0000_0000 timepoint 0.80325
 finished parsing run0000_0000 timepoint 0.8033
 finished parsing run0000_0000 timepoint 0.80335
 finished parsing run0000_0000 timepoint 0.8034
 finished parsing run0000_0000 timepoint 0.80345
 finished parsing run0000_0000 timepoint 0.8035
 finished parsing run0000_0000 timepoint 0.80355
 finished parsing run0000_0000 timepoint 0.8036
 finished parsing run0000_0000 timepoint 0.80365
 finished parsing run0000_0000 timepoint 0.8037
 finished parsing run0000_0000 timepoint 0.80375
 finished parsing run0000_0000 timepoint 0.8038
 finished parsing run0000_0000 timepoint 0.80385
 finished parsing run0000_0000 timepoint 0.8039
 finished parsing run0000_0000 timepoint 0.80395
 finished parsing run0000_0000 timepoint 0.804
 finished parsing run0000_0000 timepoint 0.80405
 finished parsing run0000_0000 timepoint 0.8041
 finished parsing run0000_0000 timepoint 0.80415
 finished parsing run0000_0000 timepoint 0.8042
 finished parsing run0000_0000 timepoint 0.80425
 finished parsing run0000_0000 timepoint 0.8043
 finished parsing run0000_0000 timepoint 0.80435
 finished parsing run0000_0000 timepoint 0.8044
 finished parsing run0000_0000 timepoint 0.80445
 finished parsing run0000_0000 timepoint 0.8045
 finished parsing run0000_0000 timepoint 0.80455
 finished parsing run0000_0000 timepoint 0.8046
 finished parsing run0000_0000 timepoint 0.80465
 finished parsing run0000_0000 timepoint 0.8047
 finished parsing run0000_0000 timepoint 0.80475
 finished parsing run0000_0000 timepoint 0.8048
 finished parsing run0000_0000 timepoint 0.80485
 finished parsing run0000_0000 timepoint 0.8049
 finished parsing run0000_0000 timepoint 0.80495
 finished parsing run0000_0000 timepoint 0.805
 finished parsing run0000_0000 timepoint 0.80505
 finished parsing run0000_0000 timepoint 0.8051
 finished parsing run0000_0000 timepoint 0.80515
 finished parsing run0000_0000 timepoint 0.8052
 finished parsing run0000_0000 timepoint 0.80525
 finished parsing run0000_0000 timepoint 0.8053
 finished parsing run0000_0000 timepoint 0.80535
 finished parsing run0000_0000 timepoint 0.8054
 finished parsing run0000_0000 timepoint 0.80545
 finished parsing run0000_0000 timepoint 0.8055
 finished parsing run0000_0000 timepoint 0.80555
 finished parsing run0000_0000 timepoint 0.8056
 finished parsing run0000_0000 timepoint 0.80565
 finished parsing run0000_0000 timepoint 0.8057
 finished parsing run0000_0000 timepoint 0.80575
 finished parsing run0000_0000 timepoint 0.8058
 finished parsing run0000_0000 timepoint 0.80585
 finished parsing run0000_0000 timepoint 0.8059
 finished parsing run0000_0000 timepoint 0.80595
 finished parsing run0000_0000 timepoint 0.806
 finished parsing run0000_0000 timepoint 0.80605
 finished parsing run0000_0000 timepoint 0.8061
 finished parsing run0000_0000 timepoint 0.80615
 finished parsing run0000_0000 timepoint 0.8062
 finished parsing run0000_0000 timepoint 0.80625
 finished parsing run0000_0000 timepoint 0.8063
 finished parsing run0000_0000 timepoint 0.80635
 finished parsing run0000_0000 timepoint 0.8064
 finished parsing run0000_0000 timepoint 0.80645
 finished parsing run0000_0000 timepoint 0.8065
 finished parsing run0000_0000 timepoint 0.80655
 finished parsing run0000_0000 timepoint 0.8066
 finished parsing run0000_0000 timepoint 0.80665
 finished parsing run0000_0000 timepoint 0.8067
 finished parsing run0000_0000 timepoint 0.80675
 finished parsing run0000_0000 timepoint 0.8068
 finished parsing run0000_0000 timepoint 0.80685
 finished parsing run0000_0000 timepoint 0.8069
 finished parsing run0000_0000 timepoint 0.80695
 finished parsing run0000_0000 timepoint 0.807
 finished parsing run0000_0000 timepoint 0.80705
 finished parsing run0000_0000 timepoint 0.8071
 finished parsing run0000_0000 timepoint 0.80715
 finished parsing run0000_0000 timepoint 0.8072
 finished parsing run0000_0000 timepoint 0.80725
 finished parsing run0000_0000 timepoint 0.8073
 finished parsing run0000_0000 timepoint 0.80735
 finished parsing run0000_0000 timepoint 0.8074
 finished parsing run0000_0000 timepoint 0.80745
 finished parsing run0000_0000 timepoint 0.8075
 finished parsing run0000_0000 timepoint 0.80755
 finished parsing run0000_0000 timepoint 0.8076
 finished parsing run0000_0000 timepoint 0.80765
 finished parsing run0000_0000 timepoint 0.8077
 finished parsing run0000_0000 timepoint 0.80775
 finished parsing run0000_0000 timepoint 0.8078
 finished parsing run0000_0000 timepoint 0.80785
 finished parsing run0000_0000 timepoint 0.8079
 finished parsing run0000_0000 timepoint 0.80795
 finished parsing run0000_0000 timepoint 0.808
 finished parsing run0000_0000 timepoint 0.80805
 finished parsing run0000_0000 timepoint 0.8081
 finished parsing run0000_0000 timepoint 0.80815
 finished parsing run0000_0000 timepoint 0.8082
 finished parsing run0000_0000 timepoint 0.80825
 finished parsing run0000_0000 timepoint 0.8083
 finished parsing run0000_0000 timepoint 0.80835
 finished parsing run0000_0000 timepoint 0.8084
 finished parsing run0000_0000 timepoint 0.80845
 finished parsing run0000_0000 timepoint 0.8085
 finished parsing run0000_0000 timepoint 0.80855
 finished parsing run0000_0000 timepoint 0.8086
 finished parsing run0000_0000 timepoint 0.80865
 finished parsing run0000_0000 timepoint 0.8087
 finished parsing run0000_0000 timepoint 0.80875
 finished parsing run0000_0000 timepoint 0.8088
 finished parsing run0000_0000 timepoint 0.80885
 finished parsing run0000_0000 timepoint 0.8089
 finished parsing run0000_0000 timepoint 0.80895
 finished parsing run0000_0000 timepoint 0.809
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7957
 finished parsing run0000_0000 timepoint 0.79575
 finished parsing run0000_0000 timepoint 0.7958
 finished parsing run0000_0000 timepoint 0.79585
 finished parsing run0000_0000 timepoint 0.7959
 finished parsing run0000_0000 timepoint 0.79595
 finished parsing run0000_0000 timepoint 0.796
 finished parsing run0000_0000 timepoint 0.79605
 finished parsing run0000_0000 timepoint 0.7961
 finished parsing run0000_0000 timepoint 0.79615
 finished parsing run0000_0000 timepoint 0.7962
 finished parsing run0000_0000 timepoint 0.79625
 finished parsing run0000_0000 timepoint 0.7963
 finished parsing run0000_0000 timepoint 0.79635
 finished parsing run0000_0000 timepoint 0.7964
 finished parsing run0000_0000 timepoint 0.79645
 finished parsing run0000_0000 timepoint 0.7965
 finished parsing run0000_0000 timepoint 0.79655
 finished parsing run0000_0000 timepoint 0.7966
 finished parsing run0000_0000 timepoint 0.79665
 finished parsing run0000_0000 timepoint 0.7967
 finished parsing run0000_0000 timepoint 0.79675
 finished parsing run0000_0000 timepoint 0.7968
 finished parsing run0000_0000 timepoint 0.79685
 finished parsing run0000_0000 timepoint 0.7969
 finished parsing run0000_0000 timepoint 0.79695
 finished parsing run0000_0000 timepoint 0.797
 finished parsing run0000_0000 timepoint 0.79705
 finished parsing run0000_0000 timepoint 0.7971
 finished parsing run0000_0000 timepoint 0.79715
 finished parsing run0000_0000 timepoint 0.7972
 finished parsing run0000_0000 timepoint 0.79725
 finished parsing run0000_0000 timepoint 0.7973
 finished parsing run0000_0000 timepoint 0.79735
 finished parsing run0000_0000 timepoint 0.7974
 finished parsing run0000_0000 timepoint 0.79745
 finished parsing run0000_0000 timepoint 0.7975
 finished parsing run0000_0000 timepoint 0.79755
 finished parsing run0000_0000 timepoint 0.7976
 finished parsing run0000_0000 timepoint 0.79765
 finished parsing run0000_0000 timepoint 0.7977
 finished parsing run0000_0000 timepoint 0.79775
 finished parsing run0000_0000 timepoint 0.7978
 finished parsing run0000_0000 timepoint 0.79785
 finished parsing run0000_0000 timepoint 0.7979
 finished parsing run0000_0000 timepoint 0.79795
 finished parsing run0000_0000 timepoint 0.798
 finished parsing run0000_0000 timepoint 0.79805
 finished parsing run0000_0000 timepoint 0.7981
 finished parsing run0000_0000 timepoint 0.79815
 finished parsing run0000_0000 timepoint 0.7982
 finished parsing run0000_0000 timepoint 0.79825
 finished parsing run0000_0000 timepoint 0.7983
 finished parsing run0000_0000 timepoint 0.79835
 finished parsing run0000_0000 timepoint 0.7984
 finished parsing run0000_0000 timepoint 0.79845
 finished parsing run0000_0000 timepoint 0.7985
 finished parsing run0000_0000 timepoint 0.79855
 finished parsing run0000_0000 timepoint 0.7986
 finished parsing run0000_0000 timepoint 0.79865
 finished parsing run0000_0000 timepoint 0.7987
 finished parsing run0000_0000 timepoint 0.79875
 finished parsing run0000_0000 timepoint 0.7988
 finished parsing run0000_0000 timepoint 0.79885
 finished parsing run0000_0000 timepoint 0.7989
 finished parsing run0000_0000 timepoint 0.79895
 finished parsing run0000_0000 timepoint 0.799
 finished parsing run0000_0000 timepoint 0.79905
 finished parsing run0000_0000 timepoint 0.7991
 finished parsing run0000_0000 timepoint 0.79915
 finished parsing run0000_0000 timepoint 0.7992
 finished parsing run0000_0000 timepoint 0.79925
 finished parsing run0000_0000 timepoint 0.7993
 finished parsing run0000_0000 timepoint 0.79935
 finished parsing run0000_0000 timepoint 0.7994
 finished parsing run0000_0000 timepoint 0.79945
 finished parsing run0000_0000 timepoint 0.7995
 finished parsing run0000_0000 timepoint 0.79955
 finished parsing run0000_0000 timepoint 0.7996
 finished parsing run0000_0000 timepoint 0.79965
 finished parsing run0000_0000 timepoint 0.7997
 finished parsing run0000_0000 timepoint 0.79975
 finished parsing run0000_0000 timepoint 0.7998
 finished parsing run0000_0000 timepoint 0.79985
 finished parsing run0000_0000 timepoint 0.7999
 finished parsing run0000_0000 timepoint 0.79995
 finished parsing run0000_0000 timepoint 0.8
 finished parsing run0000_0000 timepoint 0.80005
 finished parsing run0000_0000 timepoint 0.8001
 finished parsing run0000_0000 timepoint 0.80015
 finished parsing run0000_0000 timepoint 0.8002
 finished parsing run0000_0000 timepoint 0.80025
 finished parsing run0000_0000 timepoint 0.8003
 finished parsing run0000_0000 timepoint 0.80035
 finished parsing run0000_0000 timepoint 0.8004
 finished parsing run0000_0000 timepoint 0.80045
 finished parsing run0000_0000 timepoint 0.8005
 finished parsing run0000_0000 timepoint 0.80055
 finished parsing run0000_0000 timepoint 0.8006
 finished parsing run0000_0000 timepoint 0.80065
 finished parsing run0000_0000 timepoint 0.8007
 finished parsing run0000_0000 timepoint 0.80075
 finished parsing run0000_0000 timepoint 0.8008
 finished parsing run0000_0000 timepoint 0.80085
 finished parsing run0000_0000 timepoint 0.8009
 finished parsing run0000_0000 timepoint 0.80095
 finished parsing run0000_0000 timepoint 0.801
 finished parsing run0000_0000 timepoint 0.80105
 finished parsing run0000_0000 timepoint 0.8011
 finished parsing run0000_0000 timepoint 0.80115
 finished parsing run0000_0000 timepoint 0.8012
 finished parsing run0000_0000 timepoint 0.80125
 finished parsing run0000_0000 timepoint 0.8013
 finished parsing run0000_0000 timepoint 0.80135
 finished parsing run0000_0000 timepoint 0.8014
 finished parsing run0000_0000 timepoint 0.80145
 finished parsing run0000_0000 timepoint 0.8015
 finished parsing run0000_0000 timepoint 0.80155
 finished parsing run0000_0000 timepoint 0.8016
 finished parsing run0000_0000 timepoint 0.80165
 finished parsing run0000_0000 timepoint 0.8017
 finished parsing run0000_0000 timepoint 0.80175
 finished parsing run0000_0000 timepoint 0.8018
 finished parsing run0000_0000 timepoint 0.80185
 finished parsing run0000_0000 timepoint 0.8019
 finished parsing run0000_0000 timepoint 0.80195
 finished parsing run0000_0000 timepoint 0.802
 finished parsing run0000_0000 timepoint 0.80205
 finished parsing run0000_0000 timepoint 0.8021
 finished parsing run0000_0000 timepoint 0.80215
 finished parsing run0000_0000 timepoint 0.8022
 finished parsing run0000_0000 timepoint 0.80225
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7893
 finished parsing run0000_0000 timepoint 0.78935
 finished parsing run0000_0000 timepoint 0.7894
 finished parsing run0000_0000 timepoint 0.78945
 finished parsing run0000_0000 timepoint 0.7895
 finished parsing run0000_0000 timepoint 0.78955
 finished parsing run0000_0000 timepoint 0.7896
 finished parsing run0000_0000 timepoint 0.78965
 finished parsing run0000_0000 timepoint 0.7897
 finished parsing run0000_0000 timepoint 0.78975
 finished parsing run0000_0000 timepoint 0.7898
 finished parsing run0000_0000 timepoint 0.78985
 finished parsing run0000_0000 timepoint 0.7899
 finished parsing run0000_0000 timepoint 0.78995
 finished parsing run0000_0000 timepoint 0.79
 finished parsing run0000_0000 timepoint 0.79005
 finished parsing run0000_0000 timepoint 0.7901
 finished parsing run0000_0000 timepoint 0.79015
 finished parsing run0000_0000 timepoint 0.7902
 finished parsing run0000_0000 timepoint 0.79025
 finished parsing run0000_0000 timepoint 0.7903
 finished parsing run0000_0000 timepoint 0.79035
 finished parsing run0000_0000 timepoint 0.7904
 finished parsing run0000_0000 timepoint 0.79045
 finished parsing run0000_0000 timepoint 0.7905
 finished parsing run0000_0000 timepoint 0.79055
 finished parsing run0000_0000 timepoint 0.7906
 finished parsing run0000_0000 timepoint 0.79065
 finished parsing run0000_0000 timepoint 0.7907
 finished parsing run0000_0000 timepoint 0.79075
 finished parsing run0000_0000 timepoint 0.7908
 finished parsing run0000_0000 timepoint 0.79085
 finished parsing run0000_0000 timepoint 0.7909
 finished parsing run0000_0000 timepoint 0.79095
 finished parsing run0000_0000 timepoint 0.791
 finished parsing run0000_0000 timepoint 0.79105
 finished parsing run0000_0000 timepoint 0.7911
 finished parsing run0000_0000 timepoint 0.79115
 finished parsing run0000_0000 timepoint 0.7912
 finished parsing run0000_0000 timepoint 0.79125
 finished parsing run0000_0000 timepoint 0.7913
 finished parsing run0000_0000 timepoint 0.79135
 finished parsing run0000_0000 timepoint 0.7914
 finished parsing run0000_0000 timepoint 0.79145
 finished parsing run0000_0000 timepoint 0.7915
 finished parsing run0000_0000 timepoint 0.79155
 finished parsing run0000_0000 timepoint 0.7916
 finished parsing run0000_0000 timepoint 0.79165
 finished parsing run0000_0000 timepoint 0.7917
 finished parsing run0000_0000 timepoint 0.79175
 finished parsing run0000_0000 timepoint 0.7918
 finished parsing run0000_0000 timepoint 0.79185
 finished parsing run0000_0000 timepoint 0.7919
 finished parsing run0000_0000 timepoint 0.79195
 finished parsing run0000_0000 timepoint 0.792
 finished parsing run0000_0000 timepoint 0.79205
 finished parsing run0000_0000 timepoint 0.7921
 finished parsing run0000_0000 timepoint 0.79215
 finished parsing run0000_0000 timepoint 0.7922
 finished parsing run0000_0000 timepoint 0.79225
 finished parsing run0000_0000 timepoint 0.7923
 finished parsing run0000_0000 timepoint 0.79235
 finished parsing run0000_0000 timepoint 0.7924
 finished parsing run0000_0000 timepoint 0.79245
 finished parsing run0000_0000 timepoint 0.7925
 finished parsing run0000_0000 timepoint 0.79255
 finished parsing run0000_0000 timepoint 0.7926
 finished parsing run0000_0000 timepoint 0.79265
 finished parsing run0000_0000 timepoint 0.7927
 finished parsing run0000_0000 timepoint 0.79275
 finished parsing run0000_0000 timepoint 0.7928
 finished parsing run0000_0000 timepoint 0.79285
 finished parsing run0000_0000 timepoint 0.7929
 finished parsing run0000_0000 timepoint 0.79295
 finished parsing run0000_0000 timepoint 0.793
 finished parsing run0000_0000 timepoint 0.79305
 finished parsing run0000_0000 timepoint 0.7931
 finished parsing run0000_0000 timepoint 0.79315
 finished parsing run0000_0000 timepoint 0.7932
 finished parsing run0000_0000 timepoint 0.79325
 finished parsing run0000_0000 timepoint 0.7933
 finished parsing run0000_0000 timepoint 0.79335
 finished parsing run0000_0000 timepoint 0.7934
 finished parsing run0000_0000 timepoint 0.79345
 finished parsing run0000_0000 timepoint 0.7935
 finished parsing run0000_0000 timepoint 0.79355
 finished parsing run0000_0000 timepoint 0.7936
 finished parsing run0000_0000 timepoint 0.79365
 finished parsing run0000_0000 timepoint 0.7937
 finished parsing run0000_0000 timepoint 0.79375
 finished parsing run0000_0000 timepoint 0.7938
 finished parsing run0000_0000 timepoint 0.79385
 finished parsing run0000_0000 timepoint 0.7939
 finished parsing run0000_0000 timepoint 0.79395
 finished parsing run0000_0000 timepoint 0.794
 finished parsing run0000_0000 timepoint 0.79405
 finished parsing run0000_0000 timepoint 0.7941
 finished parsing run0000_0000 timepoint 0.79415
 finished parsing run0000_0000 timepoint 0.7942
 finished parsing run0000_0000 timepoint 0.79425
 finished parsing run0000_0000 timepoint 0.7943
 finished parsing run0000_0000 timepoint 0.79435
 finished parsing run0000_0000 timepoint 0.7944
 finished parsing run0000_0000 timepoint 0.79445
 finished parsing run0000_0000 timepoint 0.7945
 finished parsing run0000_0000 timepoint 0.79455
 finished parsing run0000_0000 timepoint 0.7946
 finished parsing run0000_0000 timepoint 0.79465
 finished parsing run0000_0000 timepoint 0.7947
 finished parsing run0000_0000 timepoint 0.79475
 finished parsing run0000_0000 timepoint 0.7948
 finished parsing run0000_0000 timepoint 0.79485
 finished parsing run0000_0000 timepoint 0.7949
 finished parsing run0000_0000 timepoint 0.79495
 finished parsing run0000_0000 timepoint 0.795
 finished parsing run0000_0000 timepoint 0.79505
 finished parsing run0000_0000 timepoint 0.7951
 finished parsing run0000_0000 timepoint 0.79515
 finished parsing run0000_0000 timepoint 0.7952
 finished parsing run0000_0000 timepoint 0.79525
 finished parsing run0000_0000 timepoint 0.7953
 finished parsing run0000_0000 timepoint 0.79535
 finished parsing run0000_0000 timepoint 0.7954
 finished parsing run0000_0000 timepoint 0.79545
 finished parsing run0000_0000 timepoint 0.7955
 finished parsing run0000_0000 timepoint 0.79555
 finished parsing run0000_0000 timepoint 0.7956
 finished parsing run0000_0000 timepoint 0.79565
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7826
 finished parsing run0000_0000 timepoint 0.78265
 finished parsing run0000_0000 timepoint 0.7827
 finished parsing run0000_0000 timepoint 0.78275
 finished parsing run0000_0000 timepoint 0.7828
 finished parsing run0000_0000 timepoint 0.78285
 finished parsing run0000_0000 timepoint 0.7829
 finished parsing run0000_0000 timepoint 0.78295
 finished parsing run0000_0000 timepoint 0.783
 finished parsing run0000_0000 timepoint 0.78305
 finished parsing run0000_0000 timepoint 0.7831
 finished parsing run0000_0000 timepoint 0.78315
 finished parsing run0000_0000 timepoint 0.7832
 finished parsing run0000_0000 timepoint 0.78325
 finished parsing run0000_0000 timepoint 0.7833
 finished parsing run0000_0000 timepoint 0.78335
 finished parsing run0000_0000 timepoint 0.7834
 finished parsing run0000_0000 timepoint 0.78345
 finished parsing run0000_0000 timepoint 0.7835
 finished parsing run0000_0000 timepoint 0.78355
 finished parsing run0000_0000 timepoint 0.7836
 finished parsing run0000_0000 timepoint 0.78365
 finished parsing run0000_0000 timepoint 0.7837
 finished parsing run0000_0000 timepoint 0.78375
 finished parsing run0000_0000 timepoint 0.7838
 finished parsing run0000_0000 timepoint 0.78385
 finished parsing run0000_0000 timepoint 0.7839
 finished parsing run0000_0000 timepoint 0.78395
 finished parsing run0000_0000 timepoint 0.784
 finished parsing run0000_0000 timepoint 0.78405
 finished parsing run0000_0000 timepoint 0.7841
 finished parsing run0000_0000 timepoint 0.78415
 finished parsing run0000_0000 timepoint 0.7842
 finished parsing run0000_0000 timepoint 0.78425
 finished parsing run0000_0000 timepoint 0.7843
 finished parsing run0000_0000 timepoint 0.78435
 finished parsing run0000_0000 timepoint 0.7844
 finished parsing run0000_0000 timepoint 0.78445
 finished parsing run0000_0000 timepoint 0.7845
 finished parsing run0000_0000 timepoint 0.78455
 finished parsing run0000_0000 timepoint 0.7846
 finished parsing run0000_0000 timepoint 0.78465
 finished parsing run0000_0000 timepoint 0.7847
 finished parsing run0000_0000 timepoint 0.78475
 finished parsing run0000_0000 timepoint 0.7848
 finished parsing run0000_0000 timepoint 0.78485
 finished parsing run0000_0000 timepoint 0.7849
 finished parsing run0000_0000 timepoint 0.78495
 finished parsing run0000_0000 timepoint 0.785
 finished parsing run0000_0000 timepoint 0.78505
 finished parsing run0000_0000 timepoint 0.7851
 finished parsing run0000_0000 timepoint 0.78515
 finished parsing run0000_0000 timepoint 0.7852
 finished parsing run0000_0000 timepoint 0.78525
 finished parsing run0000_0000 timepoint 0.7853
 finished parsing run0000_0000 timepoint 0.78535
 finished parsing run0000_0000 timepoint 0.7854
 finished parsing run0000_0000 timepoint 0.78545
 finished parsing run0000_0000 timepoint 0.7855
 finished parsing run0000_0000 timepoint 0.78555
 finished parsing run0000_0000 timepoint 0.7856
 finished parsing run0000_0000 timepoint 0.78565
 finished parsing run0000_0000 timepoint 0.7857
 finished parsing run0000_0000 timepoint 0.78575
 finished parsing run0000_0000 timepoint 0.7858
 finished parsing run0000_0000 timepoint 0.78585
 finished parsing run0000_0000 timepoint 0.7859
 finished parsing run0000_0000 timepoint 0.78595
 finished parsing run0000_0000 timepoint 0.786
 finished parsing run0000_0000 timepoint 0.78605
 finished parsing run0000_0000 timepoint 0.7861
 finished parsing run0000_0000 timepoint 0.78615
 finished parsing run0000_0000 timepoint 0.7862
 finished parsing run0000_0000 timepoint 0.78625
 finished parsing run0000_0000 timepoint 0.7863
 finished parsing run0000_0000 timepoint 0.78635
 finished parsing run0000_0000 timepoint 0.7864
 finished parsing run0000_0000 timepoint 0.78645
 finished parsing run0000_0000 timepoint 0.7865
 finished parsing run0000_0000 timepoint 0.78655
 finished parsing run0000_0000 timepoint 0.7866
 finished parsing run0000_0000 timepoint 0.78665
 finished parsing run0000_0000 timepoint 0.7867
 finished parsing run0000_0000 timepoint 0.78675
 finished parsing run0000_0000 timepoint 0.7868
 finished parsing run0000_0000 timepoint 0.78685
 finished parsing run0000_0000 timepoint 0.7869
 finished parsing run0000_0000 timepoint 0.78695
 finished parsing run0000_0000 timepoint 0.787
 finished parsing run0000_0000 timepoint 0.78705
 finished parsing run0000_0000 timepoint 0.7871
 finished parsing run0000_0000 timepoint 0.78715
 finished parsing run0000_0000 timepoint 0.7872
 finished parsing run0000_0000 timepoint 0.78725
 finished parsing run0000_0000 timepoint 0.7873
 finished parsing run0000_0000 timepoint 0.78735
 finished parsing run0000_0000 timepoint 0.7874
 finished parsing run0000_0000 timepoint 0.78745
 finished parsing run0000_0000 timepoint 0.7875
 finished parsing run0000_0000 timepoint 0.78755
 finished parsing run0000_0000 timepoint 0.7876
 finished parsing run0000_0000 timepoint 0.78765
 finished parsing run0000_0000 timepoint 0.7877
 finished parsing run0000_0000 timepoint 0.78775
 finished parsing run0000_0000 timepoint 0.7878
 finished parsing run0000_0000 timepoint 0.78785
 finished parsing run0000_0000 timepoint 0.7879
 finished parsing run0000_0000 timepoint 0.78795
 finished parsing run0000_0000 timepoint 0.788
 finished parsing run0000_0000 timepoint 0.78805
 finished parsing run0000_0000 timepoint 0.7881
 finished parsing run0000_0000 timepoint 0.78815
 finished parsing run0000_0000 timepoint 0.7882
 finished parsing run0000_0000 timepoint 0.78825
 finished parsing run0000_0000 timepoint 0.7883
 finished parsing run0000_0000 timepoint 0.78835
 finished parsing run0000_0000 timepoint 0.7884
 finished parsing run0000_0000 timepoint 0.78845
 finished parsing run0000_0000 timepoint 0.7885
 finished parsing run0000_0000 timepoint 0.78855
 finished parsing run0000_0000 timepoint 0.7886
 finished parsing run0000_0000 timepoint 0.78865
 finished parsing run0000_0000 timepoint 0.7887
 finished parsing run0000_0000 timepoint 0.78875
 finished parsing run0000_0000 timepoint 0.7888
 finished parsing run0000_0000 timepoint 0.78885
 finished parsing run0000_0000 timepoint 0.7889
 finished parsing run0000_0000 timepoint 0.78895
 finished parsing run0000_0000 timepoint 0.789
 finished parsing run0000_0000 timepoint 0.78905
 finished parsing run0000_0000 timepoint 0.7891
 finished parsing run0000_0000 timepoint 0.78915
 finished parsing run0000_0000 timepoint 0.7892
 finished parsing run0000_0000 timepoint 0.78925
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.77605
 finished parsing run0000_0000 timepoint 0.7761
 finished parsing run0000_0000 timepoint 0.77615
 finished parsing run0000_0000 timepoint 0.7762
 finished parsing run0000_0000 timepoint 0.77625
 finished parsing run0000_0000 timepoint 0.7763
 finished parsing run0000_0000 timepoint 0.77635
 finished parsing run0000_0000 timepoint 0.7764
 finished parsing run0000_0000 timepoint 0.77645
 finished parsing run0000_0000 timepoint 0.7765
 finished parsing run0000_0000 timepoint 0.77655
 finished parsing run0000_0000 timepoint 0.7766
 finished parsing run0000_0000 timepoint 0.77665
 finished parsing run0000_0000 timepoint 0.7767
 finished parsing run0000_0000 timepoint 0.77675
 finished parsing run0000_0000 timepoint 0.7768
 finished parsing run0000_0000 timepoint 0.77685
 finished parsing run0000_0000 timepoint 0.7769
 finished parsing run0000_0000 timepoint 0.77695
 finished parsing run0000_0000 timepoint 0.777
 finished parsing run0000_0000 timepoint 0.77705
 finished parsing run0000_0000 timepoint 0.7771
 finished parsing run0000_0000 timepoint 0.77715
 finished parsing run0000_0000 timepoint 0.7772
 finished parsing run0000_0000 timepoint 0.77725
 finished parsing run0000_0000 timepoint 0.7773
 finished parsing run0000_0000 timepoint 0.77735
 finished parsing run0000_0000 timepoint 0.7774
 finished parsing run0000_0000 timepoint 0.77745
 finished parsing run0000_0000 timepoint 0.7775
 finished parsing run0000_0000 timepoint 0.77755
 finished parsing run0000_0000 timepoint 0.7776
 finished parsing run0000_0000 timepoint 0.77765
 finished parsing run0000_0000 timepoint 0.7777
 finished parsing run0000_0000 timepoint 0.77775
 finished parsing run0000_0000 timepoint 0.7778
 finished parsing run0000_0000 timepoint 0.77785
 finished parsing run0000_0000 timepoint 0.7779
 finished parsing run0000_0000 timepoint 0.77795
 finished parsing run0000_0000 timepoint 0.778
 finished parsing run0000_0000 timepoint 0.77805
 finished parsing run0000_0000 timepoint 0.7781
 finished parsing run0000_0000 timepoint 0.77815
 finished parsing run0000_0000 timepoint 0.7782
 finished parsing run0000_0000 timepoint 0.77825
 finished parsing run0000_0000 timepoint 0.7783
 finished parsing run0000_0000 timepoint 0.77835
 finished parsing run0000_0000 timepoint 0.7784
 finished parsing run0000_0000 timepoint 0.77845
 finished parsing run0000_0000 timepoint 0.7785
 finished parsing run0000_0000 timepoint 0.77855
 finished parsing run0000_0000 timepoint 0.7786
 finished parsing run0000_0000 timepoint 0.77865
 finished parsing run0000_0000 timepoint 0.7787
 finished parsing run0000_0000 timepoint 0.77875
 finished parsing run0000_0000 timepoint 0.7788
 finished parsing run0000_0000 timepoint 0.77885
 finished parsing run0000_0000 timepoint 0.7789
 finished parsing run0000_0000 timepoint 0.77895
 finished parsing run0000_0000 timepoint 0.779
 finished parsing run0000_0000 timepoint 0.77905
 finished parsing run0000_0000 timepoint 0.7791
 finished parsing run0000_0000 timepoint 0.77915
 finished parsing run0000_0000 timepoint 0.7792
 finished parsing run0000_0000 timepoint 0.77925
 finished parsing run0000_0000 timepoint 0.7793
 finished parsing run0000_0000 timepoint 0.77935
 finished parsing run0000_0000 timepoint 0.7794
 finished parsing run0000_0000 timepoint 0.77945
 finished parsing run0000_0000 timepoint 0.7795
 finished parsing run0000_0000 timepoint 0.77955
 finished parsing run0000_0000 timepoint 0.7796
 finished parsing run0000_0000 timepoint 0.77965
 finished parsing run0000_0000 timepoint 0.7797
 finished parsing run0000_0000 timepoint 0.77975
 finished parsing run0000_0000 timepoint 0.7798
 finished parsing run0000_0000 timepoint 0.77985
 finished parsing run0000_0000 timepoint 0.7799
 finished parsing run0000_0000 timepoint 0.77995
 finished parsing run0000_0000 timepoint 0.78
 finished parsing run0000_0000 timepoint 0.78005
 finished parsing run0000_0000 timepoint 0.7801
 finished parsing run0000_0000 timepoint 0.78015
 finished parsing run0000_0000 timepoint 0.7802
 finished parsing run0000_0000 timepoint 0.78025
 finished parsing run0000_0000 timepoint 0.7803
 finished parsing run0000_0000 timepoint 0.78035
 finished parsing run0000_0000 timepoint 0.7804
 finished parsing run0000_0000 timepoint 0.78045
 finished parsing run0000_0000 timepoint 0.7805
 finished parsing run0000_0000 timepoint 0.78055
 finished parsing run0000_0000 timepoint 0.7806
 finished parsing run0000_0000 timepoint 0.78065
 finished parsing run0000_0000 timepoint 0.7807
 finished parsing run0000_0000 timepoint 0.78075
 finished parsing run0000_0000 timepoint 0.7808
 finished parsing run0000_0000 timepoint 0.78085
 finished parsing run0000_0000 timepoint 0.7809
 finished parsing run0000_0000 timepoint 0.78095
 finished parsing run0000_0000 timepoint 0.781
 finished parsing run0000_0000 timepoint 0.78105
 finished parsing run0000_0000 timepoint 0.7811
 finished parsing run0000_0000 timepoint 0.78115
 finished parsing run0000_0000 timepoint 0.7812
 finished parsing run0000_0000 timepoint 0.78125
 finished parsing run0000_0000 timepoint 0.7813
 finished parsing run0000_0000 timepoint 0.78135
 finished parsing run0000_0000 timepoint 0.7814
 finished parsing run0000_0000 timepoint 0.78145
 finished parsing run0000_0000 timepoint 0.7815
 finished parsing run0000_0000 timepoint 0.78155
 finished parsing run0000_0000 timepoint 0.7816
 finished parsing run0000_0000 timepoint 0.78165
 finished parsing run0000_0000 timepoint 0.7817
 finished parsing run0000_0000 timepoint 0.78175
 finished parsing run0000_0000 timepoint 0.7818
 finished parsing run0000_0000 timepoint 0.78185
 finished parsing run0000_0000 timepoint 0.7819
 finished parsing run0000_0000 timepoint 0.78195
 finished parsing run0000_0000 timepoint 0.782
 finished parsing run0000_0000 timepoint 0.78205
 finished parsing run0000_0000 timepoint 0.7821
 finished parsing run0000_0000 timepoint 0.78215
 finished parsing run0000_0000 timepoint 0.7822
 finished parsing run0000_0000 timepoint 0.78225
 finished parsing run0000_0000 timepoint 0.7823
 finished parsing run0000_0000 timepoint 0.78235
 finished parsing run0000_0000 timepoint 0.7824
 finished parsing run0000_0000 timepoint 0.78245
 finished parsing run0000_0000 timepoint 0.7825
 finished parsing run0000_0000 timepoint 0.78255
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.76925
 finished parsing run0000_0000 timepoint 0.7693
 finished parsing run0000_0000 timepoint 0.76935
 finished parsing run0000_0000 timepoint 0.7694
 finished parsing run0000_0000 timepoint 0.76945
 finished parsing run0000_0000 timepoint 0.7695
 finished parsing run0000_0000 timepoint 0.76955
 finished parsing run0000_0000 timepoint 0.7696
 finished parsing run0000_0000 timepoint 0.76965
 finished parsing run0000_0000 timepoint 0.7697
 finished parsing run0000_0000 timepoint 0.76975
 finished parsing run0000_0000 timepoint 0.7698
 finished parsing run0000_0000 timepoint 0.76985
 finished parsing run0000_0000 timepoint 0.7699
 finished parsing run0000_0000 timepoint 0.76995
 finished parsing run0000_0000 timepoint 0.77
 finished parsing run0000_0000 timepoint 0.77005
 finished parsing run0000_0000 timepoint 0.7701
 finished parsing run0000_0000 timepoint 0.77015
 finished parsing run0000_0000 timepoint 0.7702
 finished parsing run0000_0000 timepoint 0.77025
 finished parsing run0000_0000 timepoint 0.7703
 finished parsing run0000_0000 timepoint 0.77035
 finished parsing run0000_0000 timepoint 0.7704
 finished parsing run0000_0000 timepoint 0.77045
 finished parsing run0000_0000 timepoint 0.7705
 finished parsing run0000_0000 timepoint 0.77055
 finished parsing run0000_0000 timepoint 0.7706
 finished parsing run0000_0000 timepoint 0.77065
 finished parsing run0000_0000 timepoint 0.7707
 finished parsing run0000_0000 timepoint 0.77075
 finished parsing run0000_0000 timepoint 0.7708
 finished parsing run0000_0000 timepoint 0.77085
 finished parsing run0000_0000 timepoint 0.7709
 finished parsing run0000_0000 timepoint 0.77095
 finished parsing run0000_0000 timepoint 0.771
 finished parsing run0000_0000 timepoint 0.77105
 finished parsing run0000_0000 timepoint 0.7711
 finished parsing run0000_0000 timepoint 0.77115
 finished parsing run0000_0000 timepoint 0.7712
 finished parsing run0000_0000 timepoint 0.77125
 finished parsing run0000_0000 timepoint 0.7713
 finished parsing run0000_0000 timepoint 0.77135
 finished parsing run0000_0000 timepoint 0.7714
 finished parsing run0000_0000 timepoint 0.77145
 finished parsing run0000_0000 timepoint 0.7715
 finished parsing run0000_0000 timepoint 0.77155
 finished parsing run0000_0000 timepoint 0.7716
 finished parsing run0000_0000 timepoint 0.77165
 finished parsing run0000_0000 timepoint 0.7717
 finished parsing run0000_0000 timepoint 0.77175
 finished parsing run0000_0000 timepoint 0.7718
 finished parsing run0000_0000 timepoint 0.77185
 finished parsing run0000_0000 timepoint 0.7719
 finished parsing run0000_0000 timepoint 0.77195
 finished parsing run0000_0000 timepoint 0.772
 finished parsing run0000_0000 timepoint 0.77205
 finished parsing run0000_0000 timepoint 0.7721
 finished parsing run0000_0000 timepoint 0.77215
 finished parsing run0000_0000 timepoint 0.7722
 finished parsing run0000_0000 timepoint 0.77225
 finished parsing run0000_0000 timepoint 0.7723
 finished parsing run0000_0000 timepoint 0.77235
 finished parsing run0000_0000 timepoint 0.7724
 finished parsing run0000_0000 timepoint 0.77245
 finished parsing run0000_0000 timepoint 0.7725
 finished parsing run0000_0000 timepoint 0.77255
 finished parsing run0000_0000 timepoint 0.7726
 finished parsing run0000_0000 timepoint 0.77265
 finished parsing run0000_0000 timepoint 0.7727
 finished parsing run0000_0000 timepoint 0.77275
 finished parsing run0000_0000 timepoint 0.7728
 finished parsing run0000_0000 timepoint 0.77285
 finished parsing run0000_0000 timepoint 0.7729
 finished parsing run0000_0000 timepoint 0.77295
 finished parsing run0000_0000 timepoint 0.773
 finished parsing run0000_0000 timepoint 0.77305
 finished parsing run0000_0000 timepoint 0.7731
 finished parsing run0000_0000 timepoint 0.77315
 finished parsing run0000_0000 timepoint 0.7732
 finished parsing run0000_0000 timepoint 0.77325
 finished parsing run0000_0000 timepoint 0.7733
 finished parsing run0000_0000 timepoint 0.77335
 finished parsing run0000_0000 timepoint 0.7734
 finished parsing run0000_0000 timepoint 0.77345
 finished parsing run0000_0000 timepoint 0.7735
 finished parsing run0000_0000 timepoint 0.77355
 finished parsing run0000_0000 timepoint 0.7736
 finished parsing run0000_0000 timepoint 0.77365
 finished parsing run0000_0000 timepoint 0.7737
 finished parsing run0000_0000 timepoint 0.77375
 finished parsing run0000_0000 timepoint 0.7738
 finished parsing run0000_0000 timepoint 0.77385
 finished parsing run0000_0000 timepoint 0.7739
 finished parsing run0000_0000 timepoint 0.77395
 finished parsing run0000_0000 timepoint 0.774
 finished parsing run0000_0000 timepoint 0.77405
 finished parsing run0000_0000 timepoint 0.7741
 finished parsing run0000_0000 timepoint 0.77415
 finished parsing run0000_0000 timepoint 0.7742
 finished parsing run0000_0000 timepoint 0.77425
 finished parsing run0000_0000 timepoint 0.7743
 finished parsing run0000_0000 timepoint 0.77435
 finished parsing run0000_0000 timepoint 0.7744
 finished parsing run0000_0000 timepoint 0.77445
 finished parsing run0000_0000 timepoint 0.7745
 finished parsing run0000_0000 timepoint 0.77455
 finished parsing run0000_0000 timepoint 0.7746
 finished parsing run0000_0000 timepoint 0.77465
 finished parsing run0000_0000 timepoint 0.7747
 finished parsing run0000_0000 timepoint 0.77475
 finished parsing run0000_0000 timepoint 0.7748
 finished parsing run0000_0000 timepoint 0.77485
 finished parsing run0000_0000 timepoint 0.7749
 finished parsing run0000_0000 timepoint 0.77495
 finished parsing run0000_0000 timepoint 0.775
 finished parsing run0000_0000 timepoint 0.77505
 finished parsing run0000_0000 timepoint 0.7751
 finished parsing run0000_0000 timepoint 0.77515
 finished parsing run0000_0000 timepoint 0.7752
 finished parsing run0000_0000 timepoint 0.77525
 finished parsing run0000_0000 timepoint 0.7753
 finished parsing run0000_0000 timepoint 0.77535
 finished parsing run0000_0000 timepoint 0.7754
 finished parsing run0000_0000 timepoint 0.77545
 finished parsing run0000_0000 timepoint 0.7755
 finished parsing run0000_0000 timepoint 0.77555
 finished parsing run0000_0000 timepoint 0.7756
 finished parsing run0000_0000 timepoint 0.77565
 finished parsing run0000_0000 timepoint 0.7757
 finished parsing run0000_0000 timepoint 0.77575
 finished parsing run0000_0000 timepoint 0.7758
 finished parsing run0000_0000 timepoint 0.77585
 finished parsing run0000_0000 timepoint 0.7759
 finished parsing run0000_0000 timepoint 0.77595
 finished parsing run0000_0000 timepoint 0.776
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7627
 finished parsing run0000_0000 timepoint 0.76275
 finished parsing run0000_0000 timepoint 0.7628
 finished parsing run0000_0000 timepoint 0.76285
 finished parsing run0000_0000 timepoint 0.7629
 finished parsing run0000_0000 timepoint 0.76295
 finished parsing run0000_0000 timepoint 0.763
 finished parsing run0000_0000 timepoint 0.76305
 finished parsing run0000_0000 timepoint 0.7631
 finished parsing run0000_0000 timepoint 0.76315
 finished parsing run0000_0000 timepoint 0.7632
 finished parsing run0000_0000 timepoint 0.76325
 finished parsing run0000_0000 timepoint 0.7633
 finished parsing run0000_0000 timepoint 0.76335
 finished parsing run0000_0000 timepoint 0.7634
 finished parsing run0000_0000 timepoint 0.76345
 finished parsing run0000_0000 timepoint 0.7635
 finished parsing run0000_0000 timepoint 0.76355
 finished parsing run0000_0000 timepoint 0.7636
 finished parsing run0000_0000 timepoint 0.76365
 finished parsing run0000_0000 timepoint 0.7637
 finished parsing run0000_0000 timepoint 0.76375
 finished parsing run0000_0000 timepoint 0.7638
 finished parsing run0000_0000 timepoint 0.76385
 finished parsing run0000_0000 timepoint 0.7639
 finished parsing run0000_0000 timepoint 0.76395
 finished parsing run0000_0000 timepoint 0.764
 finished parsing run0000_0000 timepoint 0.76405
 finished parsing run0000_0000 timepoint 0.7641
 finished parsing run0000_0000 timepoint 0.76415
 finished parsing run0000_0000 timepoint 0.7642
 finished parsing run0000_0000 timepoint 0.76425
 finished parsing run0000_0000 timepoint 0.7643
 finished parsing run0000_0000 timepoint 0.76435
 finished parsing run0000_0000 timepoint 0.7644
 finished parsing run0000_0000 timepoint 0.76445
 finished parsing run0000_0000 timepoint 0.7645
 finished parsing run0000_0000 timepoint 0.76455
 finished parsing run0000_0000 timepoint 0.7646
 finished parsing run0000_0000 timepoint 0.76465
 finished parsing run0000_0000 timepoint 0.7647
 finished parsing run0000_0000 timepoint 0.76475
 finished parsing run0000_0000 timepoint 0.7648
 finished parsing run0000_0000 timepoint 0.76485
 finished parsing run0000_0000 timepoint 0.7649
 finished parsing run0000_0000 timepoint 0.76495
 finished parsing run0000_0000 timepoint 0.765
 finished parsing run0000_0000 timepoint 0.76505
 finished parsing run0000_0000 timepoint 0.7651
 finished parsing run0000_0000 timepoint 0.76515
 finished parsing run0000_0000 timepoint 0.7652
 finished parsing run0000_0000 timepoint 0.76525
 finished parsing run0000_0000 timepoint 0.7653
 finished parsing run0000_0000 timepoint 0.76535
 finished parsing run0000_0000 timepoint 0.7654
 finished parsing run0000_0000 timepoint 0.76545
 finished parsing run0000_0000 timepoint 0.7655
 finished parsing run0000_0000 timepoint 0.76555
 finished parsing run0000_0000 timepoint 0.7656
 finished parsing run0000_0000 timepoint 0.76565
 finished parsing run0000_0000 timepoint 0.7657
 finished parsing run0000_0000 timepoint 0.76575
 finished parsing run0000_0000 timepoint 0.7658
 finished parsing run0000_0000 timepoint 0.76585
 finished parsing run0000_0000 timepoint 0.7659
 finished parsing run0000_0000 timepoint 0.76595
 finished parsing run0000_0000 timepoint 0.766
 finished parsing run0000_0000 timepoint 0.76605
 finished parsing run0000_0000 timepoint 0.7661
 finished parsing run0000_0000 timepoint 0.76615
 finished parsing run0000_0000 timepoint 0.7662
 finished parsing run0000_0000 timepoint 0.76625
 finished parsing run0000_0000 timepoint 0.7663
 finished parsing run0000_0000 timepoint 0.76635
 finished parsing run0000_0000 timepoint 0.7664
 finished parsing run0000_0000 timepoint 0.76645
 finished parsing run0000_0000 timepoint 0.7665
 finished parsing run0000_0000 timepoint 0.76655
 finished parsing run0000_0000 timepoint 0.7666
 finished parsing run0000_0000 timepoint 0.76665
 finished parsing run0000_0000 timepoint 0.7667
 finished parsing run0000_0000 timepoint 0.76675
 finished parsing run0000_0000 timepoint 0.7668
 finished parsing run0000_0000 timepoint 0.76685
 finished parsing run0000_0000 timepoint 0.7669
 finished parsing run0000_0000 timepoint 0.76695
 finished parsing run0000_0000 timepoint 0.767
 finished parsing run0000_0000 timepoint 0.76705
 finished parsing run0000_0000 timepoint 0.7671
 finished parsing run0000_0000 timepoint 0.76715
 finished parsing run0000_0000 timepoint 0.7672
 finished parsing run0000_0000 timepoint 0.76725
 finished parsing run0000_0000 timepoint 0.7673
 finished parsing run0000_0000 timepoint 0.76735
 finished parsing run0000_0000 timepoint 0.7674
 finished parsing run0000_0000 timepoint 0.76745
 finished parsing run0000_0000 timepoint 0.7675
 finished parsing run0000_0000 timepoint 0.76755
 finished parsing run0000_0000 timepoint 0.7676
 finished parsing run0000_0000 timepoint 0.76765
 finished parsing run0000_0000 timepoint 0.7677
 finished parsing run0000_0000 timepoint 0.76775
 finished parsing run0000_0000 timepoint 0.7678
 finished parsing run0000_0000 timepoint 0.76785
 finished parsing run0000_0000 timepoint 0.7679
 finished parsing run0000_0000 timepoint 0.76795
 finished parsing run0000_0000 timepoint 0.768
 finished parsing run0000_0000 timepoint 0.76805
 finished parsing run0000_0000 timepoint 0.7681
 finished parsing run0000_0000 timepoint 0.76815
 finished parsing run0000_0000 timepoint 0.7682
 finished parsing run0000_0000 timepoint 0.76825
 finished parsing run0000_0000 timepoint 0.7683
 finished parsing run0000_0000 timepoint 0.76835
 finished parsing run0000_0000 timepoint 0.7684
 finished parsing run0000_0000 timepoint 0.76845
 finished parsing run0000_0000 timepoint 0.7685
 finished parsing run0000_0000 timepoint 0.76855
 finished parsing run0000_0000 timepoint 0.7686
 finished parsing run0000_0000 timepoint 0.76865
 finished parsing run0000_0000 timepoint 0.7687
 finished parsing run0000_0000 timepoint 0.76875
 finished parsing run0000_0000 timepoint 0.7688
 finished parsing run0000_0000 timepoint 0.76885
 finished parsing run0000_0000 timepoint 0.7689
 finished parsing run0000_0000 timepoint 0.76895
 finished parsing run0000_0000 timepoint 0.769
 finished parsing run0000_0000 timepoint 0.76905
 finished parsing run0000_0000 timepoint 0.7691
 finished parsing run0000_0000 timepoint 0.76915
 finished parsing run0000_0000 timepoint 0.7692
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7562
 finished parsing run0000_0000 timepoint 0.75625
 finished parsing run0000_0000 timepoint 0.7563
 finished parsing run0000_0000 timepoint 0.75635
 finished parsing run0000_0000 timepoint 0.7564
 finished parsing run0000_0000 timepoint 0.75645
 finished parsing run0000_0000 timepoint 0.7565
 finished parsing run0000_0000 timepoint 0.75655
 finished parsing run0000_0000 timepoint 0.7566
 finished parsing run0000_0000 timepoint 0.75665
 finished parsing run0000_0000 timepoint 0.7567
 finished parsing run0000_0000 timepoint 0.75675
 finished parsing run0000_0000 timepoint 0.7568
 finished parsing run0000_0000 timepoint 0.75685
 finished parsing run0000_0000 timepoint 0.7569
 finished parsing run0000_0000 timepoint 0.75695
 finished parsing run0000_0000 timepoint 0.757
 finished parsing run0000_0000 timepoint 0.75705
 finished parsing run0000_0000 timepoint 0.7571
 finished parsing run0000_0000 timepoint 0.75715
 finished parsing run0000_0000 timepoint 0.7572
 finished parsing run0000_0000 timepoint 0.75725
 finished parsing run0000_0000 timepoint 0.7573
 finished parsing run0000_0000 timepoint 0.75735
 finished parsing run0000_0000 timepoint 0.7574
 finished parsing run0000_0000 timepoint 0.75745
 finished parsing run0000_0000 timepoint 0.7575
 finished parsing run0000_0000 timepoint 0.75755
 finished parsing run0000_0000 timepoint 0.7576
 finished parsing run0000_0000 timepoint 0.75765
 finished parsing run0000_0000 timepoint 0.7577
 finished parsing run0000_0000 timepoint 0.75775
 finished parsing run0000_0000 timepoint 0.7578
 finished parsing run0000_0000 timepoint 0.75785
 finished parsing run0000_0000 timepoint 0.7579
 finished parsing run0000_0000 timepoint 0.75795
 finished parsing run0000_0000 timepoint 0.758
 finished parsing run0000_0000 timepoint 0.75805
 finished parsing run0000_0000 timepoint 0.7581
 finished parsing run0000_0000 timepoint 0.75815
 finished parsing run0000_0000 timepoint 0.7582
 finished parsing run0000_0000 timepoint 0.75825
 finished parsing run0000_0000 timepoint 0.7583
 finished parsing run0000_0000 timepoint 0.75835
 finished parsing run0000_0000 timepoint 0.7584
 finished parsing run0000_0000 timepoint 0.75845
 finished parsing run0000_0000 timepoint 0.7585
 finished parsing run0000_0000 timepoint 0.75855
 finished parsing run0000_0000 timepoint 0.7586
 finished parsing run0000_0000 timepoint 0.75865
 finished parsing run0000_0000 timepoint 0.7587
 finished parsing run0000_0000 timepoint 0.75875
 finished parsing run0000_0000 timepoint 0.7588
 finished parsing run0000_0000 timepoint 0.75885
 finished parsing run0000_0000 timepoint 0.7589
 finished parsing run0000_0000 timepoint 0.75895
 finished parsing run0000_0000 timepoint 0.759
 finished parsing run0000_0000 timepoint 0.75905
 finished parsing run0000_0000 timepoint 0.7591
 finished parsing run0000_0000 timepoint 0.75915
 finished parsing run0000_0000 timepoint 0.7592
 finished parsing run0000_0000 timepoint 0.75925
 finished parsing run0000_0000 timepoint 0.7593
 finished parsing run0000_0000 timepoint 0.75935
 finished parsing run0000_0000 timepoint 0.7594
 finished parsing run0000_0000 timepoint 0.75945
 finished parsing run0000_0000 timepoint 0.7595
 finished parsing run0000_0000 timepoint 0.75955
 finished parsing run0000_0000 timepoint 0.7596
 finished parsing run0000_0000 timepoint 0.75965
 finished parsing run0000_0000 timepoint 0.7597
 finished parsing run0000_0000 timepoint 0.75975
 finished parsing run0000_0000 timepoint 0.7598
 finished parsing run0000_0000 timepoint 0.75985
 finished parsing run0000_0000 timepoint 0.7599
 finished parsing run0000_0000 timepoint 0.75995
 finished parsing run0000_0000 timepoint 0.76
 finished parsing run0000_0000 timepoint 0.76005
 finished parsing run0000_0000 timepoint 0.7601
 finished parsing run0000_0000 timepoint 0.76015
 finished parsing run0000_0000 timepoint 0.7602
 finished parsing run0000_0000 timepoint 0.76025
 finished parsing run0000_0000 timepoint 0.7603
 finished parsing run0000_0000 timepoint 0.76035
 finished parsing run0000_0000 timepoint 0.7604
 finished parsing run0000_0000 timepoint 0.76045
 finished parsing run0000_0000 timepoint 0.7605
 finished parsing run0000_0000 timepoint 0.76055
 finished parsing run0000_0000 timepoint 0.7606
 finished parsing run0000_0000 timepoint 0.76065
 finished parsing run0000_0000 timepoint 0.7607
 finished parsing run0000_0000 timepoint 0.76075
 finished parsing run0000_0000 timepoint 0.7608
 finished parsing run0000_0000 timepoint 0.76085
 finished parsing run0000_0000 timepoint 0.7609
 finished parsing run0000_0000 timepoint 0.76095
 finished parsing run0000_0000 timepoint 0.761
 finished parsing run0000_0000 timepoint 0.76105
 finished parsing run0000_0000 timepoint 0.7611
 finished parsing run0000_0000 timepoint 0.76115
 finished parsing run0000_0000 timepoint 0.7612
 finished parsing run0000_0000 timepoint 0.76125
 finished parsing run0000_0000 timepoint 0.7613
 finished parsing run0000_0000 timepoint 0.76135
 finished parsing run0000_0000 timepoint 0.7614
 finished parsing run0000_0000 timepoint 0.76145
 finished parsing run0000_0000 timepoint 0.7615
 finished parsing run0000_0000 timepoint 0.76155
 finished parsing run0000_0000 timepoint 0.7616
 finished parsing run0000_0000 timepoint 0.76165
 finished parsing run0000_0000 timepoint 0.7617
 finished parsing run0000_0000 timepoint 0.76175
 finished parsing run0000_0000 timepoint 0.7618
 finished parsing run0000_0000 timepoint 0.76185
 finished parsing run0000_0000 timepoint 0.7619
 finished parsing run0000_0000 timepoint 0.76195
 finished parsing run0000_0000 timepoint 0.762
 finished parsing run0000_0000 timepoint 0.76205
 finished parsing run0000_0000 timepoint 0.7621
 finished parsing run0000_0000 timepoint 0.76215
 finished parsing run0000_0000 timepoint 0.7622
 finished parsing run0000_0000 timepoint 0.76225
 finished parsing run0000_0000 timepoint 0.7623
 finished parsing run0000_0000 timepoint 0.76235
 finished parsing run0000_0000 timepoint 0.7624
 finished parsing run0000_0000 timepoint 0.76245
 finished parsing run0000_0000 timepoint 0.7625
 finished parsing run0000_0000 timepoint 0.76255
 finished parsing run0000_0000 timepoint 0.7626
 finished parsing run0000_0000 timepoint 0.76265
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.74965
 finished parsing run0000_0000 timepoint 0.7497
 finished parsing run0000_0000 timepoint 0.74975
 finished parsing run0000_0000 timepoint 0.7498
 finished parsing run0000_0000 timepoint 0.74985
 finished parsing run0000_0000 timepoint 0.7499
 finished parsing run0000_0000 timepoint 0.74995
 finished parsing run0000_0000 timepoint 0.75
 finished parsing run0000_0000 timepoint 0.75005
 finished parsing run0000_0000 timepoint 0.7501
 finished parsing run0000_0000 timepoint 0.75015
 finished parsing run0000_0000 timepoint 0.7502
 finished parsing run0000_0000 timepoint 0.75025
 finished parsing run0000_0000 timepoint 0.7503
 finished parsing run0000_0000 timepoint 0.75035
 finished parsing run0000_0000 timepoint 0.7504
 finished parsing run0000_0000 timepoint 0.75045
 finished parsing run0000_0000 timepoint 0.7505
 finished parsing run0000_0000 timepoint 0.75055
 finished parsing run0000_0000 timepoint 0.7506
 finished parsing run0000_0000 timepoint 0.75065
 finished parsing run0000_0000 timepoint 0.7507
 finished parsing run0000_0000 timepoint 0.75075
 finished parsing run0000_0000 timepoint 0.7508
 finished parsing run0000_0000 timepoint 0.75085
 finished parsing run0000_0000 timepoint 0.7509
 finished parsing run0000_0000 timepoint 0.75095
 finished parsing run0000_0000 timepoint 0.751
 finished parsing run0000_0000 timepoint 0.75105
 finished parsing run0000_0000 timepoint 0.7511
 finished parsing run0000_0000 timepoint 0.75115
 finished parsing run0000_0000 timepoint 0.7512
 finished parsing run0000_0000 timepoint 0.75125
 finished parsing run0000_0000 timepoint 0.7513
 finished parsing run0000_0000 timepoint 0.75135
 finished parsing run0000_0000 timepoint 0.7514
 finished parsing run0000_0000 timepoint 0.75145
 finished parsing run0000_0000 timepoint 0.7515
 finished parsing run0000_0000 timepoint 0.75155
 finished parsing run0000_0000 timepoint 0.7516
 finished parsing run0000_0000 timepoint 0.75165
 finished parsing run0000_0000 timepoint 0.7517
 finished parsing run0000_0000 timepoint 0.75175
 finished parsing run0000_0000 timepoint 0.7518
 finished parsing run0000_0000 timepoint 0.75185
 finished parsing run0000_0000 timepoint 0.7519
 finished parsing run0000_0000 timepoint 0.75195
 finished parsing run0000_0000 timepoint 0.752
 finished parsing run0000_0000 timepoint 0.75205
 finished parsing run0000_0000 timepoint 0.7521
 finished parsing run0000_0000 timepoint 0.75215
 finished parsing run0000_0000 timepoint 0.7522
 finished parsing run0000_0000 timepoint 0.75225
 finished parsing run0000_0000 timepoint 0.7523
 finished parsing run0000_0000 timepoint 0.75235
 finished parsing run0000_0000 timepoint 0.7524
 finished parsing run0000_0000 timepoint 0.75245
 finished parsing run0000_0000 timepoint 0.7525
 finished parsing run0000_0000 timepoint 0.75255
 finished parsing run0000_0000 timepoint 0.7526
 finished parsing run0000_0000 timepoint 0.75265
 finished parsing run0000_0000 timepoint 0.7527
 finished parsing run0000_0000 timepoint 0.75275
 finished parsing run0000_0000 timepoint 0.7528
 finished parsing run0000_0000 timepoint 0.75285
 finished parsing run0000_0000 timepoint 0.7529
 finished parsing run0000_0000 timepoint 0.75295
 finished parsing run0000_0000 timepoint 0.753
 finished parsing run0000_0000 timepoint 0.75305
 finished parsing run0000_0000 timepoint 0.7531
 finished parsing run0000_0000 timepoint 0.75315
 finished parsing run0000_0000 timepoint 0.7532
 finished parsing run0000_0000 timepoint 0.75325
 finished parsing run0000_0000 timepoint 0.7533
 finished parsing run0000_0000 timepoint 0.75335
 finished parsing run0000_0000 timepoint 0.7534
 finished parsing run0000_0000 timepoint 0.75345
 finished parsing run0000_0000 timepoint 0.7535
 finished parsing run0000_0000 timepoint 0.75355
 finished parsing run0000_0000 timepoint 0.7536
 finished parsing run0000_0000 timepoint 0.75365
 finished parsing run0000_0000 timepoint 0.7537
 finished parsing run0000_0000 timepoint 0.75375
 finished parsing run0000_0000 timepoint 0.7538
 finished parsing run0000_0000 timepoint 0.75385
 finished parsing run0000_0000 timepoint 0.7539
 finished parsing run0000_0000 timepoint 0.75395
 finished parsing run0000_0000 timepoint 0.754
 finished parsing run0000_0000 timepoint 0.75405
 finished parsing run0000_0000 timepoint 0.7541
 finished parsing run0000_0000 timepoint 0.75415
 finished parsing run0000_0000 timepoint 0.7542
 finished parsing run0000_0000 timepoint 0.75425
 finished parsing run0000_0000 timepoint 0.7543
 finished parsing run0000_0000 timepoint 0.75435
 finished parsing run0000_0000 timepoint 0.7544
 finished parsing run0000_0000 timepoint 0.75445
 finished parsing run0000_0000 timepoint 0.7545
 finished parsing run0000_0000 timepoint 0.75455
 finished parsing run0000_0000 timepoint 0.7546
 finished parsing run0000_0000 timepoint 0.75465
 finished parsing run0000_0000 timepoint 0.7547
 finished parsing run0000_0000 timepoint 0.75475
 finished parsing run0000_0000 timepoint 0.7548
 finished parsing run0000_0000 timepoint 0.75485
 finished parsing run0000_0000 timepoint 0.7549
 finished parsing run0000_0000 timepoint 0.75495
 finished parsing run0000_0000 timepoint 0.755
 finished parsing run0000_0000 timepoint 0.75505
 finished parsing run0000_0000 timepoint 0.7551
 finished parsing run0000_0000 timepoint 0.75515
 finished parsing run0000_0000 timepoint 0.7552
 finished parsing run0000_0000 timepoint 0.75525
 finished parsing run0000_0000 timepoint 0.7553
 finished parsing run0000_0000 timepoint 0.75535
 finished parsing run0000_0000 timepoint 0.7554
 finished parsing run0000_0000 timepoint 0.75545
 finished parsing run0000_0000 timepoint 0.7555
 finished parsing run0000_0000 timepoint 0.75555
 finished parsing run0000_0000 timepoint 0.7556
 finished parsing run0000_0000 timepoint 0.75565
 finished parsing run0000_0000 timepoint 0.7557
 finished parsing run0000_0000 timepoint 0.75575
 finished parsing run0000_0000 timepoint 0.7558
 finished parsing run0000_0000 timepoint 0.75585
 finished parsing run0000_0000 timepoint 0.7559
 finished parsing run0000_0000 timepoint 0.75595
 finished parsing run0000_0000 timepoint 0.756
 finished parsing run0000_0000 timepoint 0.75605
 finished parsing run0000_0000 timepoint 0.7561
 finished parsing run0000_0000 timepoint 0.75615
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.74315
 finished parsing run0000_0000 timepoint 0.7432
 finished parsing run0000_0000 timepoint 0.74325
 finished parsing run0000_0000 timepoint 0.7433
 finished parsing run0000_0000 timepoint 0.74335
 finished parsing run0000_0000 timepoint 0.7434
 finished parsing run0000_0000 timepoint 0.74345
 finished parsing run0000_0000 timepoint 0.7435
 finished parsing run0000_0000 timepoint 0.74355
 finished parsing run0000_0000 timepoint 0.7436
 finished parsing run0000_0000 timepoint 0.74365
 finished parsing run0000_0000 timepoint 0.7437
 finished parsing run0000_0000 timepoint 0.74375
 finished parsing run0000_0000 timepoint 0.7438
 finished parsing run0000_0000 timepoint 0.74385
 finished parsing run0000_0000 timepoint 0.7439
 finished parsing run0000_0000 timepoint 0.74395
 finished parsing run0000_0000 timepoint 0.744
 finished parsing run0000_0000 timepoint 0.74405
 finished parsing run0000_0000 timepoint 0.7441
 finished parsing run0000_0000 timepoint 0.74415
 finished parsing run0000_0000 timepoint 0.7442
 finished parsing run0000_0000 timepoint 0.74425
 finished parsing run0000_0000 timepoint 0.7443
 finished parsing run0000_0000 timepoint 0.74435
 finished parsing run0000_0000 timepoint 0.7444
 finished parsing run0000_0000 timepoint 0.74445
 finished parsing run0000_0000 timepoint 0.7445
 finished parsing run0000_0000 timepoint 0.74455
 finished parsing run0000_0000 timepoint 0.7446
 finished parsing run0000_0000 timepoint 0.74465
 finished parsing run0000_0000 timepoint 0.7447
 finished parsing run0000_0000 timepoint 0.74475
 finished parsing run0000_0000 timepoint 0.7448
 finished parsing run0000_0000 timepoint 0.74485
 finished parsing run0000_0000 timepoint 0.7449
 finished parsing run0000_0000 timepoint 0.74495
 finished parsing run0000_0000 timepoint 0.745
 finished parsing run0000_0000 timepoint 0.74505
 finished parsing run0000_0000 timepoint 0.7451
 finished parsing run0000_0000 timepoint 0.74515
 finished parsing run0000_0000 timepoint 0.7452
 finished parsing run0000_0000 timepoint 0.74525
 finished parsing run0000_0000 timepoint 0.7453
 finished parsing run0000_0000 timepoint 0.74535
 finished parsing run0000_0000 timepoint 0.7454
 finished parsing run0000_0000 timepoint 0.74545
 finished parsing run0000_0000 timepoint 0.7455
 finished parsing run0000_0000 timepoint 0.74555
 finished parsing run0000_0000 timepoint 0.7456
 finished parsing run0000_0000 timepoint 0.74565
 finished parsing run0000_0000 timepoint 0.7457
 finished parsing run0000_0000 timepoint 0.74575
 finished parsing run0000_0000 timepoint 0.7458
 finished parsing run0000_0000 timepoint 0.74585
 finished parsing run0000_0000 timepoint 0.7459
 finished parsing run0000_0000 timepoint 0.74595
 finished parsing run0000_0000 timepoint 0.746
 finished parsing run0000_0000 timepoint 0.74605
 finished parsing run0000_0000 timepoint 0.7461
 finished parsing run0000_0000 timepoint 0.74615
 finished parsing run0000_0000 timepoint 0.7462
 finished parsing run0000_0000 timepoint 0.74625
 finished parsing run0000_0000 timepoint 0.7463
 finished parsing run0000_0000 timepoint 0.74635
 finished parsing run0000_0000 timepoint 0.7464
 finished parsing run0000_0000 timepoint 0.74645
 finished parsing run0000_0000 timepoint 0.7465
 finished parsing run0000_0000 timepoint 0.74655
 finished parsing run0000_0000 timepoint 0.7466
 finished parsing run0000_0000 timepoint 0.74665
 finished parsing run0000_0000 timepoint 0.7467
 finished parsing run0000_0000 timepoint 0.74675
 finished parsing run0000_0000 timepoint 0.7468
 finished parsing run0000_0000 timepoint 0.74685
 finished parsing run0000_0000 timepoint 0.7469
 finished parsing run0000_0000 timepoint 0.74695
 finished parsing run0000_0000 timepoint 0.747
 finished parsing run0000_0000 timepoint 0.74705
 finished parsing run0000_0000 timepoint 0.7471
 finished parsing run0000_0000 timepoint 0.74715
 finished parsing run0000_0000 timepoint 0.7472
 finished parsing run0000_0000 timepoint 0.74725
 finished parsing run0000_0000 timepoint 0.7473
 finished parsing run0000_0000 timepoint 0.74735
 finished parsing run0000_0000 timepoint 0.7474
 finished parsing run0000_0000 timepoint 0.74745
 finished parsing run0000_0000 timepoint 0.7475
 finished parsing run0000_0000 timepoint 0.74755
 finished parsing run0000_0000 timepoint 0.7476
 finished parsing run0000_0000 timepoint 0.74765
 finished parsing run0000_0000 timepoint 0.7477
 finished parsing run0000_0000 timepoint 0.74775
 finished parsing run0000_0000 timepoint 0.7478
 finished parsing run0000_0000 timepoint 0.74785
 finished parsing run0000_0000 timepoint 0.7479
 finished parsing run0000_0000 timepoint 0.74795
 finished parsing run0000_0000 timepoint 0.748
 finished parsing run0000_0000 timepoint 0.74805
 finished parsing run0000_0000 timepoint 0.7481
 finished parsing run0000_0000 timepoint 0.74815
 finished parsing run0000_0000 timepoint 0.7482
 finished parsing run0000_0000 timepoint 0.74825
 finished parsing run0000_0000 timepoint 0.7483
 finished parsing run0000_0000 timepoint 0.74835
 finished parsing run0000_0000 timepoint 0.7484
 finished parsing run0000_0000 timepoint 0.74845
 finished parsing run0000_0000 timepoint 0.7485
 finished parsing run0000_0000 timepoint 0.74855
 finished parsing run0000_0000 timepoint 0.7486
 finished parsing run0000_0000 timepoint 0.74865
 finished parsing run0000_0000 timepoint 0.7487
 finished parsing run0000_0000 timepoint 0.74875
 finished parsing run0000_0000 timepoint 0.7488
 finished parsing run0000_0000 timepoint 0.74885
 finished parsing run0000_0000 timepoint 0.7489
 finished parsing run0000_0000 timepoint 0.74895
 finished parsing run0000_0000 timepoint 0.749
 finished parsing run0000_0000 timepoint 0.74905
 finished parsing run0000_0000 timepoint 0.7491
 finished parsing run0000_0000 timepoint 0.74915
 finished parsing run0000_0000 timepoint 0.7492
 finished parsing run0000_0000 timepoint 0.74925
 finished parsing run0000_0000 timepoint 0.7493
 finished parsing run0000_0000 timepoint 0.74935
 finished parsing run0000_0000 timepoint 0.7494
 finished parsing run0000_0000 timepoint 0.74945
 finished parsing run0000_0000 timepoint 0.7495
 finished parsing run0000_0000 timepoint 0.74955
 finished parsing run0000_0000 timepoint 0.7496
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.73635
 finished parsing run0000_0000 timepoint 0.7364
 finished parsing run0000_0000 timepoint 0.73645
 finished parsing run0000_0000 timepoint 0.7365
 finished parsing run0000_0000 timepoint 0.73655
 finished parsing run0000_0000 timepoint 0.7366
 finished parsing run0000_0000 timepoint 0.73665
 finished parsing run0000_0000 timepoint 0.7367
 finished parsing run0000_0000 timepoint 0.73675
 finished parsing run0000_0000 timepoint 0.7368
 finished parsing run0000_0000 timepoint 0.73685
 finished parsing run0000_0000 timepoint 0.7369
 finished parsing run0000_0000 timepoint 0.73695
 finished parsing run0000_0000 timepoint 0.737
 finished parsing run0000_0000 timepoint 0.73705
 finished parsing run0000_0000 timepoint 0.7371
 finished parsing run0000_0000 timepoint 0.73715
 finished parsing run0000_0000 timepoint 0.7372
 finished parsing run0000_0000 timepoint 0.73725
 finished parsing run0000_0000 timepoint 0.7373
 finished parsing run0000_0000 timepoint 0.73735
 finished parsing run0000_0000 timepoint 0.7374
 finished parsing run0000_0000 timepoint 0.73745
 finished parsing run0000_0000 timepoint 0.7375
 finished parsing run0000_0000 timepoint 0.73755
 finished parsing run0000_0000 timepoint 0.7376
 finished parsing run0000_0000 timepoint 0.73765
 finished parsing run0000_0000 timepoint 0.7377
 finished parsing run0000_0000 timepoint 0.73775
 finished parsing run0000_0000 timepoint 0.7378
 finished parsing run0000_0000 timepoint 0.73785
 finished parsing run0000_0000 timepoint 0.7379
 finished parsing run0000_0000 timepoint 0.73795
 finished parsing run0000_0000 timepoint 0.738
 finished parsing run0000_0000 timepoint 0.73805
 finished parsing run0000_0000 timepoint 0.7381
 finished parsing run0000_0000 timepoint 0.73815
 finished parsing run0000_0000 timepoint 0.7382
 finished parsing run0000_0000 timepoint 0.73825
 finished parsing run0000_0000 timepoint 0.7383
 finished parsing run0000_0000 timepoint 0.73835
 finished parsing run0000_0000 timepoint 0.7384
 finished parsing run0000_0000 timepoint 0.73845
 finished parsing run0000_0000 timepoint 0.7385
 finished parsing run0000_0000 timepoint 0.73855
 finished parsing run0000_0000 timepoint 0.7386
 finished parsing run0000_0000 timepoint 0.73865
 finished parsing run0000_0000 timepoint 0.7387
 finished parsing run0000_0000 timepoint 0.73875
 finished parsing run0000_0000 timepoint 0.7388
 finished parsing run0000_0000 timepoint 0.73885
 finished parsing run0000_0000 timepoint 0.7389
 finished parsing run0000_0000 timepoint 0.73895
 finished parsing run0000_0000 timepoint 0.739
 finished parsing run0000_0000 timepoint 0.73905
 finished parsing run0000_0000 timepoint 0.7391
 finished parsing run0000_0000 timepoint 0.73915
 finished parsing run0000_0000 timepoint 0.7392
 finished parsing run0000_0000 timepoint 0.73925
 finished parsing run0000_0000 timepoint 0.7393
 finished parsing run0000_0000 timepoint 0.73935
 finished parsing run0000_0000 timepoint 0.7394
 finished parsing run0000_0000 timepoint 0.73945
 finished parsing run0000_0000 timepoint 0.7395
 finished parsing run0000_0000 timepoint 0.73955
 finished parsing run0000_0000 timepoint 0.7396
 finished parsing run0000_0000 timepoint 0.73965
 finished parsing run0000_0000 timepoint 0.7397
 finished parsing run0000_0000 timepoint 0.73975
 finished parsing run0000_0000 timepoint 0.7398
 finished parsing run0000_0000 timepoint 0.73985
 finished parsing run0000_0000 timepoint 0.7399
 finished parsing run0000_0000 timepoint 0.73995
 finished parsing run0000_0000 timepoint 0.74
 finished parsing run0000_0000 timepoint 0.74005
 finished parsing run0000_0000 timepoint 0.7401
 finished parsing run0000_0000 timepoint 0.74015
 finished parsing run0000_0000 timepoint 0.7402
 finished parsing run0000_0000 timepoint 0.74025
 finished parsing run0000_0000 timepoint 0.7403
 finished parsing run0000_0000 timepoint 0.74035
 finished parsing run0000_0000 timepoint 0.7404
 finished parsing run0000_0000 timepoint 0.74045
 finished parsing run0000_0000 timepoint 0.7405
 finished parsing run0000_0000 timepoint 0.74055
 finished parsing run0000_0000 timepoint 0.7406
 finished parsing run0000_0000 timepoint 0.74065
 finished parsing run0000_0000 timepoint 0.7407
 finished parsing run0000_0000 timepoint 0.74075
 finished parsing run0000_0000 timepoint 0.7408
 finished parsing run0000_0000 timepoint 0.74085
 finished parsing run0000_0000 timepoint 0.7409
 finished parsing run0000_0000 timepoint 0.74095
 finished parsing run0000_0000 timepoint 0.741
 finished parsing run0000_0000 timepoint 0.74105
 finished parsing run0000_0000 timepoint 0.7411
 finished parsing run0000_0000 timepoint 0.74115
 finished parsing run0000_0000 timepoint 0.7412
 finished parsing run0000_0000 timepoint 0.74125
 finished parsing run0000_0000 timepoint 0.7413
 finished parsing run0000_0000 timepoint 0.74135
 finished parsing run0000_0000 timepoint 0.7414
 finished parsing run0000_0000 timepoint 0.74145
 finished parsing run0000_0000 timepoint 0.7415
 finished parsing run0000_0000 timepoint 0.74155
 finished parsing run0000_0000 timepoint 0.7416
 finished parsing run0000_0000 timepoint 0.74165
 finished parsing run0000_0000 timepoint 0.7417
 finished parsing run0000_0000 timepoint 0.74175
 finished parsing run0000_0000 timepoint 0.7418
 finished parsing run0000_0000 timepoint 0.74185
 finished parsing run0000_0000 timepoint 0.7419
 finished parsing run0000_0000 timepoint 0.74195
 finished parsing run0000_0000 timepoint 0.742
 finished parsing run0000_0000 timepoint 0.74205
 finished parsing run0000_0000 timepoint 0.7421
 finished parsing run0000_0000 timepoint 0.74215
 finished parsing run0000_0000 timepoint 0.7422
 finished parsing run0000_0000 timepoint 0.74225
 finished parsing run0000_0000 timepoint 0.7423
 finished parsing run0000_0000 timepoint 0.74235
 finished parsing run0000_0000 timepoint 0.7424
 finished parsing run0000_0000 timepoint 0.74245
 finished parsing run0000_0000 timepoint 0.7425
 finished parsing run0000_0000 timepoint 0.74255
 finished parsing run0000_0000 timepoint 0.7426
 finished parsing run0000_0000 timepoint 0.74265
 finished parsing run0000_0000 timepoint 0.7427
 finished parsing run0000_0000 timepoint 0.74275
 finished parsing run0000_0000 timepoint 0.7428
 finished parsing run0000_0000 timepoint 0.74285
 finished parsing run0000_0000 timepoint 0.7429
 finished parsing run0000_0000 timepoint 0.74295
 finished parsing run0000_0000 timepoint 0.743
 finished parsing run0000_0000 timepoint 0.74305
 finished parsing run0000_0000 timepoint 0.7431
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.72995
 finished parsing run0000_0000 timepoint 0.73
 finished parsing run0000_0000 timepoint 0.73005
 finished parsing run0000_0000 timepoint 0.7301
 finished parsing run0000_0000 timepoint 0.73015
 finished parsing run0000_0000 timepoint 0.7302
 finished parsing run0000_0000 timepoint 0.73025
 finished parsing run0000_0000 timepoint 0.7303
 finished parsing run0000_0000 timepoint 0.73035
 finished parsing run0000_0000 timepoint 0.7304
 finished parsing run0000_0000 timepoint 0.73045
 finished parsing run0000_0000 timepoint 0.7305
 finished parsing run0000_0000 timepoint 0.73055
 finished parsing run0000_0000 timepoint 0.7306
 finished parsing run0000_0000 timepoint 0.73065
 finished parsing run0000_0000 timepoint 0.7307
 finished parsing run0000_0000 timepoint 0.73075
 finished parsing run0000_0000 timepoint 0.7308
 finished parsing run0000_0000 timepoint 0.73085
 finished parsing run0000_0000 timepoint 0.7309
 finished parsing run0000_0000 timepoint 0.73095
 finished parsing run0000_0000 timepoint 0.731
 finished parsing run0000_0000 timepoint 0.73105
 finished parsing run0000_0000 timepoint 0.7311
 finished parsing run0000_0000 timepoint 0.73115
 finished parsing run0000_0000 timepoint 0.7312
 finished parsing run0000_0000 timepoint 0.73125
 finished parsing run0000_0000 timepoint 0.7313
 finished parsing run0000_0000 timepoint 0.73135
 finished parsing run0000_0000 timepoint 0.7314
 finished parsing run0000_0000 timepoint 0.73145
 finished parsing run0000_0000 timepoint 0.7315
 finished parsing run0000_0000 timepoint 0.73155
 finished parsing run0000_0000 timepoint 0.7316
 finished parsing run0000_0000 timepoint 0.73165
 finished parsing run0000_0000 timepoint 0.7317
 finished parsing run0000_0000 timepoint 0.73175
 finished parsing run0000_0000 timepoint 0.7318
 finished parsing run0000_0000 timepoint 0.73185
 finished parsing run0000_0000 timepoint 0.7319
 finished parsing run0000_0000 timepoint 0.73195
 finished parsing run0000_0000 timepoint 0.732
 finished parsing run0000_0000 timepoint 0.73205
 finished parsing run0000_0000 timepoint 0.7321
 finished parsing run0000_0000 timepoint 0.73215
 finished parsing run0000_0000 timepoint 0.7322
 finished parsing run0000_0000 timepoint 0.73225
 finished parsing run0000_0000 timepoint 0.7323
 finished parsing run0000_0000 timepoint 0.73235
 finished parsing run0000_0000 timepoint 0.7324
 finished parsing run0000_0000 timepoint 0.73245
 finished parsing run0000_0000 timepoint 0.7325
 finished parsing run0000_0000 timepoint 0.73255
 finished parsing run0000_0000 timepoint 0.7326
 finished parsing run0000_0000 timepoint 0.73265
 finished parsing run0000_0000 timepoint 0.7327
 finished parsing run0000_0000 timepoint 0.73275
 finished parsing run0000_0000 timepoint 0.7328
 finished parsing run0000_0000 timepoint 0.73285
 finished parsing run0000_0000 timepoint 0.7329
 finished parsing run0000_0000 timepoint 0.73295
 finished parsing run0000_0000 timepoint 0.733
 finished parsing run0000_0000 timepoint 0.73305
 finished parsing run0000_0000 timepoint 0.7331
 finished parsing run0000_0000 timepoint 0.73315
 finished parsing run0000_0000 timepoint 0.7332
 finished parsing run0000_0000 timepoint 0.73325
 finished parsing run0000_0000 timepoint 0.7333
 finished parsing run0000_0000 timepoint 0.73335
 finished parsing run0000_0000 timepoint 0.7334
 finished parsing run0000_0000 timepoint 0.73345
 finished parsing run0000_0000 timepoint 0.7335
 finished parsing run0000_0000 timepoint 0.73355
 finished parsing run0000_0000 timepoint 0.7336
 finished parsing run0000_0000 timepoint 0.73365
 finished parsing run0000_0000 timepoint 0.7337
 finished parsing run0000_0000 timepoint 0.73375
 finished parsing run0000_0000 timepoint 0.7338
 finished parsing run0000_0000 timepoint 0.73385
 finished parsing run0000_0000 timepoint 0.7339
 finished parsing run0000_0000 timepoint 0.73395
 finished parsing run0000_0000 timepoint 0.734
 finished parsing run0000_0000 timepoint 0.73405
 finished parsing run0000_0000 timepoint 0.7341
 finished parsing run0000_0000 timepoint 0.73415
 finished parsing run0000_0000 timepoint 0.7342
 finished parsing run0000_0000 timepoint 0.73425
 finished parsing run0000_0000 timepoint 0.7343
 finished parsing run0000_0000 timepoint 0.73435
 finished parsing run0000_0000 timepoint 0.7344
 finished parsing run0000_0000 timepoint 0.73445
 finished parsing run0000_0000 timepoint 0.7345
 finished parsing run0000_0000 timepoint 0.73455
 finished parsing run0000_0000 timepoint 0.7346
 finished parsing run0000_0000 timepoint 0.73465
 finished parsing run0000_0000 timepoint 0.7347
 finished parsing run0000_0000 timepoint 0.73475
 finished parsing run0000_0000 timepoint 0.7348
 finished parsing run0000_0000 timepoint 0.73485
 finished parsing run0000_0000 timepoint 0.7349
 finished parsing run0000_0000 timepoint 0.73495
 finished parsing run0000_0000 timepoint 0.735
 finished parsing run0000_0000 timepoint 0.73505
 finished parsing run0000_0000 timepoint 0.7351
 finished parsing run0000_0000 timepoint 0.73515
 finished parsing run0000_0000 timepoint 0.7352
 finished parsing run0000_0000 timepoint 0.73525
 finished parsing run0000_0000 timepoint 0.7353
 finished parsing run0000_0000 timepoint 0.73535
 finished parsing run0000_0000 timepoint 0.7354
 finished parsing run0000_0000 timepoint 0.73545
 finished parsing run0000_0000 timepoint 0.7355
 finished parsing run0000_0000 timepoint 0.73555
 finished parsing run0000_0000 timepoint 0.7356
 finished parsing run0000_0000 timepoint 0.73565
 finished parsing run0000_0000 timepoint 0.7357
 finished parsing run0000_0000 timepoint 0.73575
 finished parsing run0000_0000 timepoint 0.7358
 finished parsing run0000_0000 timepoint 0.73585
 finished parsing run0000_0000 timepoint 0.7359
 finished parsing run0000_0000 timepoint 0.73595
 finished parsing run0000_0000 timepoint 0.736
 finished parsing run0000_0000 timepoint 0.73605
 finished parsing run0000_0000 timepoint 0.7361
 finished parsing run0000_0000 timepoint 0.73615
 finished parsing run0000_0000 timepoint 0.7362
 finished parsing run0000_0000 timepoint 0.73625
 finished parsing run0000_0000 timepoint 0.7363
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7235
 finished parsing run0000_0000 timepoint 0.72355
 finished parsing run0000_0000 timepoint 0.7236
 finished parsing run0000_0000 timepoint 0.72365
 finished parsing run0000_0000 timepoint 0.7237
 finished parsing run0000_0000 timepoint 0.72375
 finished parsing run0000_0000 timepoint 0.7238
 finished parsing run0000_0000 timepoint 0.72385
 finished parsing run0000_0000 timepoint 0.7239
 finished parsing run0000_0000 timepoint 0.72395
 finished parsing run0000_0000 timepoint 0.724
 finished parsing run0000_0000 timepoint 0.72405
 finished parsing run0000_0000 timepoint 0.7241
 finished parsing run0000_0000 timepoint 0.72415
 finished parsing run0000_0000 timepoint 0.7242
 finished parsing run0000_0000 timepoint 0.72425
 finished parsing run0000_0000 timepoint 0.7243
 finished parsing run0000_0000 timepoint 0.72435
 finished parsing run0000_0000 timepoint 0.7244
 finished parsing run0000_0000 timepoint 0.72445
 finished parsing run0000_0000 timepoint 0.7245
 finished parsing run0000_0000 timepoint 0.72455
 finished parsing run0000_0000 timepoint 0.7246
 finished parsing run0000_0000 timepoint 0.72465
 finished parsing run0000_0000 timepoint 0.7247
 finished parsing run0000_0000 timepoint 0.72475
 finished parsing run0000_0000 timepoint 0.7248
 finished parsing run0000_0000 timepoint 0.72485
 finished parsing run0000_0000 timepoint 0.7249
 finished parsing run0000_0000 timepoint 0.72495
 finished parsing run0000_0000 timepoint 0.725
 finished parsing run0000_0000 timepoint 0.72505
 finished parsing run0000_0000 timepoint 0.7251
 finished parsing run0000_0000 timepoint 0.72515
 finished parsing run0000_0000 timepoint 0.7252
 finished parsing run0000_0000 timepoint 0.72525
 finished parsing run0000_0000 timepoint 0.7253
 finished parsing run0000_0000 timepoint 0.72535
 finished parsing run0000_0000 timepoint 0.7254
 finished parsing run0000_0000 timepoint 0.72545
 finished parsing run0000_0000 timepoint 0.7255
 finished parsing run0000_0000 timepoint 0.72555
 finished parsing run0000_0000 timepoint 0.7256
 finished parsing run0000_0000 timepoint 0.72565
 finished parsing run0000_0000 timepoint 0.7257
 finished parsing run0000_0000 timepoint 0.72575
 finished parsing run0000_0000 timepoint 0.7258
 finished parsing run0000_0000 timepoint 0.72585
 finished parsing run0000_0000 timepoint 0.7259
 finished parsing run0000_0000 timepoint 0.72595
 finished parsing run0000_0000 timepoint 0.726
 finished parsing run0000_0000 timepoint 0.72605
 finished parsing run0000_0000 timepoint 0.7261
 finished parsing run0000_0000 timepoint 0.72615
 finished parsing run0000_0000 timepoint 0.7262
 finished parsing run0000_0000 timepoint 0.72625
 finished parsing run0000_0000 timepoint 0.7263
 finished parsing run0000_0000 timepoint 0.72635
 finished parsing run0000_0000 timepoint 0.7264
 finished parsing run0000_0000 timepoint 0.72645
 finished parsing run0000_0000 timepoint 0.7265
 finished parsing run0000_0000 timepoint 0.72655
 finished parsing run0000_0000 timepoint 0.7266
 finished parsing run0000_0000 timepoint 0.72665
 finished parsing run0000_0000 timepoint 0.7267
 finished parsing run0000_0000 timepoint 0.72675
 finished parsing run0000_0000 timepoint 0.7268
 finished parsing run0000_0000 timepoint 0.72685
 finished parsing run0000_0000 timepoint 0.7269
 finished parsing run0000_0000 timepoint 0.72695
 finished parsing run0000_0000 timepoint 0.727
 finished parsing run0000_0000 timepoint 0.72705
 finished parsing run0000_0000 timepoint 0.7271
 finished parsing run0000_0000 timepoint 0.72715
 finished parsing run0000_0000 timepoint 0.7272
 finished parsing run0000_0000 timepoint 0.72725
 finished parsing run0000_0000 timepoint 0.7273
 finished parsing run0000_0000 timepoint 0.72735
 finished parsing run0000_0000 timepoint 0.7274
 finished parsing run0000_0000 timepoint 0.72745
 finished parsing run0000_0000 timepoint 0.7275
 finished parsing run0000_0000 timepoint 0.72755
 finished parsing run0000_0000 timepoint 0.7276
 finished parsing run0000_0000 timepoint 0.72765
 finished parsing run0000_0000 timepoint 0.7277
 finished parsing run0000_0000 timepoint 0.72775
 finished parsing run0000_0000 timepoint 0.7278
 finished parsing run0000_0000 timepoint 0.72785
 finished parsing run0000_0000 timepoint 0.7279
 finished parsing run0000_0000 timepoint 0.72795
 finished parsing run0000_0000 timepoint 0.728
 finished parsing run0000_0000 timepoint 0.72805
 finished parsing run0000_0000 timepoint 0.7281
 finished parsing run0000_0000 timepoint 0.72815
 finished parsing run0000_0000 timepoint 0.7282
 finished parsing run0000_0000 timepoint 0.72825
 finished parsing run0000_0000 timepoint 0.7283
 finished parsing run0000_0000 timepoint 0.72835
 finished parsing run0000_0000 timepoint 0.7284
 finished parsing run0000_0000 timepoint 0.72845
 finished parsing run0000_0000 timepoint 0.7285
 finished parsing run0000_0000 timepoint 0.72855
 finished parsing run0000_0000 timepoint 0.7286
 finished parsing run0000_0000 timepoint 0.72865
 finished parsing run0000_0000 timepoint 0.7287
 finished parsing run0000_0000 timepoint 0.72875
 finished parsing run0000_0000 timepoint 0.7288
 finished parsing run0000_0000 timepoint 0.72885
 finished parsing run0000_0000 timepoint 0.7289
 finished parsing run0000_0000 timepoint 0.72895
 finished parsing run0000_0000 timepoint 0.729
 finished parsing run0000_0000 timepoint 0.72905
 finished parsing run0000_0000 timepoint 0.7291
 finished parsing run0000_0000 timepoint 0.72915
 finished parsing run0000_0000 timepoint 0.7292
 finished parsing run0000_0000 timepoint 0.72925
 finished parsing run0000_0000 timepoint 0.7293
 finished parsing run0000_0000 timepoint 0.72935
 finished parsing run0000_0000 timepoint 0.7294
 finished parsing run0000_0000 timepoint 0.72945
 finished parsing run0000_0000 timepoint 0.7295
 finished parsing run0000_0000 timepoint 0.72955
 finished parsing run0000_0000 timepoint 0.7296
 finished parsing run0000_0000 timepoint 0.72965
 finished parsing run0000_0000 timepoint 0.7297
 finished parsing run0000_0000 timepoint 0.72975
 finished parsing run0000_0000 timepoint 0.7298
 finished parsing run0000_0000 timepoint 0.72985
 finished parsing run0000_0000 timepoint 0.7299
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.71705
 finished parsing run0000_0000 timepoint 0.7171
 finished parsing run0000_0000 timepoint 0.71715
 finished parsing run0000_0000 timepoint 0.7172
 finished parsing run0000_0000 timepoint 0.71725
 finished parsing run0000_0000 timepoint 0.7173
 finished parsing run0000_0000 timepoint 0.71735
 finished parsing run0000_0000 timepoint 0.7174
 finished parsing run0000_0000 timepoint 0.71745
 finished parsing run0000_0000 timepoint 0.7175
 finished parsing run0000_0000 timepoint 0.71755
 finished parsing run0000_0000 timepoint 0.7176
 finished parsing run0000_0000 timepoint 0.71765
 finished parsing run0000_0000 timepoint 0.7177
 finished parsing run0000_0000 timepoint 0.71775
 finished parsing run0000_0000 timepoint 0.7178
 finished parsing run0000_0000 timepoint 0.71785
 finished parsing run0000_0000 timepoint 0.7179
 finished parsing run0000_0000 timepoint 0.71795
 finished parsing run0000_0000 timepoint 0.718
 finished parsing run0000_0000 timepoint 0.71805
 finished parsing run0000_0000 timepoint 0.7181
 finished parsing run0000_0000 timepoint 0.71815
 finished parsing run0000_0000 timepoint 0.7182
 finished parsing run0000_0000 timepoint 0.71825
 finished parsing run0000_0000 timepoint 0.7183
 finished parsing run0000_0000 timepoint 0.71835
 finished parsing run0000_0000 timepoint 0.7184
 finished parsing run0000_0000 timepoint 0.71845
 finished parsing run0000_0000 timepoint 0.7185
 finished parsing run0000_0000 timepoint 0.71855
 finished parsing run0000_0000 timepoint 0.7186
 finished parsing run0000_0000 timepoint 0.71865
 finished parsing run0000_0000 timepoint 0.7187
 finished parsing run0000_0000 timepoint 0.71875
 finished parsing run0000_0000 timepoint 0.7188
 finished parsing run0000_0000 timepoint 0.71885
 finished parsing run0000_0000 timepoint 0.7189
 finished parsing run0000_0000 timepoint 0.71895
 finished parsing run0000_0000 timepoint 0.719
 finished parsing run0000_0000 timepoint 0.71905
 finished parsing run0000_0000 timepoint 0.7191
 finished parsing run0000_0000 timepoint 0.71915
 finished parsing run0000_0000 timepoint 0.7192
 finished parsing run0000_0000 timepoint 0.71925
 finished parsing run0000_0000 timepoint 0.7193
 finished parsing run0000_0000 timepoint 0.71935
 finished parsing run0000_0000 timepoint 0.7194
 finished parsing run0000_0000 timepoint 0.71945
 finished parsing run0000_0000 timepoint 0.7195
 finished parsing run0000_0000 timepoint 0.71955
 finished parsing run0000_0000 timepoint 0.7196
 finished parsing run0000_0000 timepoint 0.71965
 finished parsing run0000_0000 timepoint 0.7197
 finished parsing run0000_0000 timepoint 0.71975
 finished parsing run0000_0000 timepoint 0.7198
 finished parsing run0000_0000 timepoint 0.71985
 finished parsing run0000_0000 timepoint 0.7199
 finished parsing run0000_0000 timepoint 0.71995
 finished parsing run0000_0000 timepoint 0.72
 finished parsing run0000_0000 timepoint 0.72005
 finished parsing run0000_0000 timepoint 0.7201
 finished parsing run0000_0000 timepoint 0.72015
 finished parsing run0000_0000 timepoint 0.7202
 finished parsing run0000_0000 timepoint 0.72025
 finished parsing run0000_0000 timepoint 0.7203
 finished parsing run0000_0000 timepoint 0.72035
 finished parsing run0000_0000 timepoint 0.7204
 finished parsing run0000_0000 timepoint 0.72045
 finished parsing run0000_0000 timepoint 0.7205
 finished parsing run0000_0000 timepoint 0.72055
 finished parsing run0000_0000 timepoint 0.7206
 finished parsing run0000_0000 timepoint 0.72065
 finished parsing run0000_0000 timepoint 0.7207
 finished parsing run0000_0000 timepoint 0.72075
 finished parsing run0000_0000 timepoint 0.7208
 finished parsing run0000_0000 timepoint 0.72085
 finished parsing run0000_0000 timepoint 0.7209
 finished parsing run0000_0000 timepoint 0.72095
 finished parsing run0000_0000 timepoint 0.721
 finished parsing run0000_0000 timepoint 0.72105
 finished parsing run0000_0000 timepoint 0.7211
 finished parsing run0000_0000 timepoint 0.72115
 finished parsing run0000_0000 timepoint 0.7212
 finished parsing run0000_0000 timepoint 0.72125
 finished parsing run0000_0000 timepoint 0.7213
 finished parsing run0000_0000 timepoint 0.72135
 finished parsing run0000_0000 timepoint 0.7214
 finished parsing run0000_0000 timepoint 0.72145
 finished parsing run0000_0000 timepoint 0.7215
 finished parsing run0000_0000 timepoint 0.72155
 finished parsing run0000_0000 timepoint 0.7216
 finished parsing run0000_0000 timepoint 0.72165
 finished parsing run0000_0000 timepoint 0.7217
 finished parsing run0000_0000 timepoint 0.72175
 finished parsing run0000_0000 timepoint 0.7218
 finished parsing run0000_0000 timepoint 0.72185
 finished parsing run0000_0000 timepoint 0.7219
 finished parsing run0000_0000 timepoint 0.72195
 finished parsing run0000_0000 timepoint 0.722
 finished parsing run0000_0000 timepoint 0.72205
 finished parsing run0000_0000 timepoint 0.7221
 finished parsing run0000_0000 timepoint 0.72215
 finished parsing run0000_0000 timepoint 0.7222
 finished parsing run0000_0000 timepoint 0.72225
 finished parsing run0000_0000 timepoint 0.7223
 finished parsing run0000_0000 timepoint 0.72235
 finished parsing run0000_0000 timepoint 0.7224
 finished parsing run0000_0000 timepoint 0.72245
 finished parsing run0000_0000 timepoint 0.7225
 finished parsing run0000_0000 timepoint 0.72255
 finished parsing run0000_0000 timepoint 0.7226
 finished parsing run0000_0000 timepoint 0.72265
 finished parsing run0000_0000 timepoint 0.7227
 finished parsing run0000_0000 timepoint 0.72275
 finished parsing run0000_0000 timepoint 0.7228
 finished parsing run0000_0000 timepoint 0.72285
 finished parsing run0000_0000 timepoint 0.7229
 finished parsing run0000_0000 timepoint 0.72295
 finished parsing run0000_0000 timepoint 0.723
 finished parsing run0000_0000 timepoint 0.72305
 finished parsing run0000_0000 timepoint 0.7231
 finished parsing run0000_0000 timepoint 0.72315
 finished parsing run0000_0000 timepoint 0.7232
 finished parsing run0000_0000 timepoint 0.72325
 finished parsing run0000_0000 timepoint 0.7233
 finished parsing run0000_0000 timepoint 0.72335
 finished parsing run0000_0000 timepoint 0.7234
 finished parsing run0000_0000 timepoint 0.72345
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.7103
 finished parsing run0000_0000 timepoint 0.71035
 finished parsing run0000_0000 timepoint 0.7104
 finished parsing run0000_0000 timepoint 0.71045
 finished parsing run0000_0000 timepoint 0.7105
 finished parsing run0000_0000 timepoint 0.71055
 finished parsing run0000_0000 timepoint 0.7106
 finished parsing run0000_0000 timepoint 0.71065
 finished parsing run0000_0000 timepoint 0.7107
 finished parsing run0000_0000 timepoint 0.71075
 finished parsing run0000_0000 timepoint 0.7108
 finished parsing run0000_0000 timepoint 0.71085
 finished parsing run0000_0000 timepoint 0.7109
 finished parsing run0000_0000 timepoint 0.71095
 finished parsing run0000_0000 timepoint 0.711
 finished parsing run0000_0000 timepoint 0.71105
 finished parsing run0000_0000 timepoint 0.7111
 finished parsing run0000_0000 timepoint 0.71115
 finished parsing run0000_0000 timepoint 0.7112
 finished parsing run0000_0000 timepoint 0.71125
 finished parsing run0000_0000 timepoint 0.7113
 finished parsing run0000_0000 timepoint 0.71135
 finished parsing run0000_0000 timepoint 0.7114
 finished parsing run0000_0000 timepoint 0.71145
 finished parsing run0000_0000 timepoint 0.7115
 finished parsing run0000_0000 timepoint 0.71155
 finished parsing run0000_0000 timepoint 0.7116
 finished parsing run0000_0000 timepoint 0.71165
 finished parsing run0000_0000 timepoint 0.7117
 finished parsing run0000_0000 timepoint 0.71175
 finished parsing run0000_0000 timepoint 0.7118
 finished parsing run0000_0000 timepoint 0.71185
 finished parsing run0000_0000 timepoint 0.7119
 finished parsing run0000_0000 timepoint 0.71195
 finished parsing run0000_0000 timepoint 0.712
 finished parsing run0000_0000 timepoint 0.71205
 finished parsing run0000_0000 timepoint 0.7121
 finished parsing run0000_0000 timepoint 0.71215
 finished parsing run0000_0000 timepoint 0.7122
 finished parsing run0000_0000 timepoint 0.71225
 finished parsing run0000_0000 timepoint 0.7123
 finished parsing run0000_0000 timepoint 0.71235
 finished parsing run0000_0000 timepoint 0.7124
 finished parsing run0000_0000 timepoint 0.71245
 finished parsing run0000_0000 timepoint 0.7125
 finished parsing run0000_0000 timepoint 0.71255
 finished parsing run0000_0000 timepoint 0.7126
 finished parsing run0000_0000 timepoint 0.71265
 finished parsing run0000_0000 timepoint 0.7127
 finished parsing run0000_0000 timepoint 0.71275
 finished parsing run0000_0000 timepoint 0.7128
 finished parsing run0000_0000 timepoint 0.71285
 finished parsing run0000_0000 timepoint 0.7129
 finished parsing run0000_0000 timepoint 0.71295
 finished parsing run0000_0000 timepoint 0.713
 finished parsing run0000_0000 timepoint 0.71305
 finished parsing run0000_0000 timepoint 0.7131
 finished parsing run0000_0000 timepoint 0.71315
 finished parsing run0000_0000 timepoint 0.7132
 finished parsing run0000_0000 timepoint 0.71325
 finished parsing run0000_0000 timepoint 0.7133
 finished parsing run0000_0000 timepoint 0.71335
 finished parsing run0000_0000 timepoint 0.7134
 finished parsing run0000_0000 timepoint 0.71345
 finished parsing run0000_0000 timepoint 0.7135
 finished parsing run0000_0000 timepoint 0.71355
 finished parsing run0000_0000 timepoint 0.7136
 finished parsing run0000_0000 timepoint 0.71365
 finished parsing run0000_0000 timepoint 0.7137
 finished parsing run0000_0000 timepoint 0.71375
 finished parsing run0000_0000 timepoint 0.7138
 finished parsing run0000_0000 timepoint 0.71385
 finished parsing run0000_0000 timepoint 0.7139
 finished parsing run0000_0000 timepoint 0.71395
 finished parsing run0000_0000 timepoint 0.714
 finished parsing run0000_0000 timepoint 0.71405
 finished parsing run0000_0000 timepoint 0.7141
 finished parsing run0000_0000 timepoint 0.71415
 finished parsing run0000_0000 timepoint 0.7142
 finished parsing run0000_0000 timepoint 0.71425
 finished parsing run0000_0000 timepoint 0.7143
 finished parsing run0000_0000 timepoint 0.71435
 finished parsing run0000_0000 timepoint 0.7144
 finished parsing run0000_0000 timepoint 0.71445
 finished parsing run0000_0000 timepoint 0.7145
 finished parsing run0000_0000 timepoint 0.71455
 finished parsing run0000_0000 timepoint 0.7146
 finished parsing run0000_0000 timepoint 0.71465
 finished parsing run0000_0000 timepoint 0.7147
 finished parsing run0000_0000 timepoint 0.71475
 finished parsing run0000_0000 timepoint 0.7148
 finished parsing run0000_0000 timepoint 0.71485
 finished parsing run0000_0000 timepoint 0.7149
 finished parsing run0000_0000 timepoint 0.71495
 finished parsing run0000_0000 timepoint 0.715
 finished parsing run0000_0000 timepoint 0.71505
 finished parsing run0000_0000 timepoint 0.7151
 finished parsing run0000_0000 timepoint 0.71515
 finished parsing run0000_0000 timepoint 0.7152
 finished parsing run0000_0000 timepoint 0.71525
 finished parsing run0000_0000 timepoint 0.7153
 finished parsing run0000_0000 timepoint 0.71535
 finished parsing run0000_0000 timepoint 0.7154
 finished parsing run0000_0000 timepoint 0.71545
 finished parsing run0000_0000 timepoint 0.7155
 finished parsing run0000_0000 timepoint 0.71555
 finished parsing run0000_0000 timepoint 0.7156
 finished parsing run0000_0000 timepoint 0.71565
 finished parsing run0000_0000 timepoint 0.7157
 finished parsing run0000_0000 timepoint 0.71575
 finished parsing run0000_0000 timepoint 0.7158
 finished parsing run0000_0000 timepoint 0.71585
 finished parsing run0000_0000 timepoint 0.7159
 finished parsing run0000_0000 timepoint 0.71595
 finished parsing run0000_0000 timepoint 0.716
 finished parsing run0000_0000 timepoint 0.71605
 finished parsing run0000_0000 timepoint 0.7161
 finished parsing run0000_0000 timepoint 0.71615
 finished parsing run0000_0000 timepoint 0.7162
 finished parsing run0000_0000 timepoint 0.71625
 finished parsing run0000_0000 timepoint 0.7163
 finished parsing run0000_0000 timepoint 0.71635
 finished parsing run0000_0000 timepoint 0.7164
 finished parsing run0000_0000 timepoint 0.71645
 finished parsing run0000_0000 timepoint 0.7165
 finished parsing run0000_0000 timepoint 0.71655
 finished parsing run0000_0000 timepoint 0.7166
 finished parsing run0000_0000 timepoint 0.71665
 finished parsing run0000_0000 timepoint 0.7167
 finished parsing run0000_0000 timepoint 0.71675
 finished parsing run0000_0000 timepoint 0.7168
 finished parsing run0000_0000 timepoint 0.71685
 finished parsing run0000_0000 timepoint 0.7169
 finished parsing run0000_0000 timepoint 0.71695
 finished parsing run0000_0000 timepoint 0.717
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.70385
 finished parsing run0000_0000 timepoint 0.7039
 finished parsing run0000_0000 timepoint 0.70395
 finished parsing run0000_0000 timepoint 0.704
 finished parsing run0000_0000 timepoint 0.70405
 finished parsing run0000_0000 timepoint 0.7041
 finished parsing run0000_0000 timepoint 0.70415
 finished parsing run0000_0000 timepoint 0.7042
 finished parsing run0000_0000 timepoint 0.70425
 finished parsing run0000_0000 timepoint 0.7043
 finished parsing run0000_0000 timepoint 0.70435
 finished parsing run0000_0000 timepoint 0.7044
 finished parsing run0000_0000 timepoint 0.70445
 finished parsing run0000_0000 timepoint 0.7045
 finished parsing run0000_0000 timepoint 0.70455
 finished parsing run0000_0000 timepoint 0.7046
 finished parsing run0000_0000 timepoint 0.70465
 finished parsing run0000_0000 timepoint 0.7047
 finished parsing run0000_0000 timepoint 0.70475
 finished parsing run0000_0000 timepoint 0.7048
 finished parsing run0000_0000 timepoint 0.70485
 finished parsing run0000_0000 timepoint 0.7049
 finished parsing run0000_0000 timepoint 0.70495
 finished parsing run0000_0000 timepoint 0.705
 finished parsing run0000_0000 timepoint 0.70505
 finished parsing run0000_0000 timepoint 0.7051
 finished parsing run0000_0000 timepoint 0.70515
 finished parsing run0000_0000 timepoint 0.7052
 finished parsing run0000_0000 timepoint 0.70525
 finished parsing run0000_0000 timepoint 0.7053
 finished parsing run0000_0000 timepoint 0.70535
 finished parsing run0000_0000 timepoint 0.7054
 finished parsing run0000_0000 timepoint 0.70545
 finished parsing run0000_0000 timepoint 0.7055
 finished parsing run0000_0000 timepoint 0.70555
 finished parsing run0000_0000 timepoint 0.7056
 finished parsing run0000_0000 timepoint 0.70565
 finished parsing run0000_0000 timepoint 0.7057
 finished parsing run0000_0000 timepoint 0.70575
 finished parsing run0000_0000 timepoint 0.7058
 finished parsing run0000_0000 timepoint 0.70585
 finished parsing run0000_0000 timepoint 0.7059
 finished parsing run0000_0000 timepoint 0.70595
 finished parsing run0000_0000 timepoint 0.706
 finished parsing run0000_0000 timepoint 0.70605
 finished parsing run0000_0000 timepoint 0.7061
 finished parsing run0000_0000 timepoint 0.70615
 finished parsing run0000_0000 timepoint 0.7062
 finished parsing run0000_0000 timepoint 0.70625
 finished parsing run0000_0000 timepoint 0.7063
 finished parsing run0000_0000 timepoint 0.70635
 finished parsing run0000_0000 timepoint 0.7064
 finished parsing run0000_0000 timepoint 0.70645
 finished parsing run0000_0000 timepoint 0.7065
 finished parsing run0000_0000 timepoint 0.70655
 finished parsing run0000_0000 timepoint 0.7066
 finished parsing run0000_0000 timepoint 0.70665
 finished parsing run0000_0000 timepoint 0.7067
 finished parsing run0000_0000 timepoint 0.70675
 finished parsing run0000_0000 timepoint 0.7068
 finished parsing run0000_0000 timepoint 0.70685
 finished parsing run0000_0000 timepoint 0.7069
 finished parsing run0000_0000 timepoint 0.70695
 finished parsing run0000_0000 timepoint 0.707
 finished parsing run0000_0000 timepoint 0.70705
 finished parsing run0000_0000 timepoint 0.7071
 finished parsing run0000_0000 timepoint 0.70715
 finished parsing run0000_0000 timepoint 0.7072
 finished parsing run0000_0000 timepoint 0.70725
 finished parsing run0000_0000 timepoint 0.7073
 finished parsing run0000_0000 timepoint 0.70735
 finished parsing run0000_0000 timepoint 0.7074
 finished parsing run0000_0000 timepoint 0.70745
 finished parsing run0000_0000 timepoint 0.7075
 finished parsing run0000_0000 timepoint 0.70755
 finished parsing run0000_0000 timepoint 0.7076
 finished parsing run0000_0000 timepoint 0.70765
 finished parsing run0000_0000 timepoint 0.7077
 finished parsing run0000_0000 timepoint 0.70775
 finished parsing run0000_0000 timepoint 0.7078
 finished parsing run0000_0000 timepoint 0.70785
 finished parsing run0000_0000 timepoint 0.7079
 finished parsing run0000_0000 timepoint 0.70795
 finished parsing run0000_0000 timepoint 0.708
 finished parsing run0000_0000 timepoint 0.70805
 finished parsing run0000_0000 timepoint 0.7081
 finished parsing run0000_0000 timepoint 0.70815
 finished parsing run0000_0000 timepoint 0.7082
 finished parsing run0000_0000 timepoint 0.70825
 finished parsing run0000_0000 timepoint 0.7083
 finished parsing run0000_0000 timepoint 0.70835
 finished parsing run0000_0000 timepoint 0.7084
 finished parsing run0000_0000 timepoint 0.70845
 finished parsing run0000_0000 timepoint 0.7085
 finished parsing run0000_0000 timepoint 0.70855
 finished parsing run0000_0000 timepoint 0.7086
 finished parsing run0000_0000 timepoint 0.70865
 finished parsing run0000_0000 timepoint 0.7087
 finished parsing run0000_0000 timepoint 0.70875
 finished parsing run0000_0000 timepoint 0.7088
 finished parsing run0000_0000 timepoint 0.70885
 finished parsing run0000_0000 timepoint 0.7089
 finished parsing run0000_0000 timepoint 0.70895
 finished parsing run0000_0000 timepoint 0.709
 finished parsing run0000_0000 timepoint 0.70905
 finished parsing run0000_0000 timepoint 0.7091
 finished parsing run0000_0000 timepoint 0.70915
 finished parsing run0000_0000 timepoint 0.7092
 finished parsing run0000_0000 timepoint 0.70925
 finished parsing run0000_0000 timepoint 0.7093
 finished parsing run0000_0000 timepoint 0.70935
 finished parsing run0000_0000 timepoint 0.7094
 finished parsing run0000_0000 timepoint 0.70945
 finished parsing run0000_0000 timepoint 0.7095
 finished parsing run0000_0000 timepoint 0.70955
 finished parsing run0000_0000 timepoint 0.7096
 finished parsing run0000_0000 timepoint 0.70965
 finished parsing run0000_0000 timepoint 0.7097
 finished parsing run0000_0000 timepoint 0.70975
 finished parsing run0000_0000 timepoint 0.7098
 finished parsing run0000_0000 timepoint 0.70985
 finished parsing run0000_0000 timepoint 0.7099
 finished parsing run0000_0000 timepoint 0.70995
 finished parsing run0000_0000 timepoint 0.71
 finished parsing run0000_0000 timepoint 0.71005
 finished parsing run0000_0000 timepoint 0.7101
 finished parsing run0000_0000 timepoint 0.71015
 finished parsing run0000_0000 timepoint 0.7102
 finished parsing run0000_0000 timepoint 0.71025
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6973
 finished parsing run0000_0000 timepoint 0.69735
 finished parsing run0000_0000 timepoint 0.6974
 finished parsing run0000_0000 timepoint 0.69745
 finished parsing run0000_0000 timepoint 0.6975
 finished parsing run0000_0000 timepoint 0.69755
 finished parsing run0000_0000 timepoint 0.6976
 finished parsing run0000_0000 timepoint 0.69765
 finished parsing run0000_0000 timepoint 0.6977
 finished parsing run0000_0000 timepoint 0.69775
 finished parsing run0000_0000 timepoint 0.6978
 finished parsing run0000_0000 timepoint 0.69785
 finished parsing run0000_0000 timepoint 0.6979
 finished parsing run0000_0000 timepoint 0.69795
 finished parsing run0000_0000 timepoint 0.698
 finished parsing run0000_0000 timepoint 0.69805
 finished parsing run0000_0000 timepoint 0.6981
 finished parsing run0000_0000 timepoint 0.69815
 finished parsing run0000_0000 timepoint 0.6982
 finished parsing run0000_0000 timepoint 0.69825
 finished parsing run0000_0000 timepoint 0.6983
 finished parsing run0000_0000 timepoint 0.69835
 finished parsing run0000_0000 timepoint 0.6984
 finished parsing run0000_0000 timepoint 0.69845
 finished parsing run0000_0000 timepoint 0.6985
 finished parsing run0000_0000 timepoint 0.69855
 finished parsing run0000_0000 timepoint 0.6986
 finished parsing run0000_0000 timepoint 0.69865
 finished parsing run0000_0000 timepoint 0.6987
 finished parsing run0000_0000 timepoint 0.69875
 finished parsing run0000_0000 timepoint 0.6988
 finished parsing run0000_0000 timepoint 0.69885
 finished parsing run0000_0000 timepoint 0.6989
 finished parsing run0000_0000 timepoint 0.69895
 finished parsing run0000_0000 timepoint 0.699
 finished parsing run0000_0000 timepoint 0.69905
 finished parsing run0000_0000 timepoint 0.6991
 finished parsing run0000_0000 timepoint 0.69915
 finished parsing run0000_0000 timepoint 0.6992
 finished parsing run0000_0000 timepoint 0.69925
 finished parsing run0000_0000 timepoint 0.6993
 finished parsing run0000_0000 timepoint 0.69935
 finished parsing run0000_0000 timepoint 0.6994
 finished parsing run0000_0000 timepoint 0.69945
 finished parsing run0000_0000 timepoint 0.6995
 finished parsing run0000_0000 timepoint 0.69955
 finished parsing run0000_0000 timepoint 0.6996
 finished parsing run0000_0000 timepoint 0.69965
 finished parsing run0000_0000 timepoint 0.6997
 finished parsing run0000_0000 timepoint 0.69975
 finished parsing run0000_0000 timepoint 0.6998
 finished parsing run0000_0000 timepoint 0.69985
 finished parsing run0000_0000 timepoint 0.6999
 finished parsing run0000_0000 timepoint 0.69995
 finished parsing run0000_0000 timepoint 0.7
 finished parsing run0000_0000 timepoint 0.70005
 finished parsing run0000_0000 timepoint 0.7001
 finished parsing run0000_0000 timepoint 0.70015
 finished parsing run0000_0000 timepoint 0.7002
 finished parsing run0000_0000 timepoint 0.70025
 finished parsing run0000_0000 timepoint 0.7003
 finished parsing run0000_0000 timepoint 0.70035
 finished parsing run0000_0000 timepoint 0.7004
 finished parsing run0000_0000 timepoint 0.70045
 finished parsing run0000_0000 timepoint 0.7005
 finished parsing run0000_0000 timepoint 0.70055
 finished parsing run0000_0000 timepoint 0.7006
 finished parsing run0000_0000 timepoint 0.70065
 finished parsing run0000_0000 timepoint 0.7007
 finished parsing run0000_0000 timepoint 0.70075
 finished parsing run0000_0000 timepoint 0.7008
 finished parsing run0000_0000 timepoint 0.70085
 finished parsing run0000_0000 timepoint 0.7009
 finished parsing run0000_0000 timepoint 0.70095
 finished parsing run0000_0000 timepoint 0.701
 finished parsing run0000_0000 timepoint 0.70105
 finished parsing run0000_0000 timepoint 0.7011
 finished parsing run0000_0000 timepoint 0.70115
 finished parsing run0000_0000 timepoint 0.7012
 finished parsing run0000_0000 timepoint 0.70125
 finished parsing run0000_0000 timepoint 0.7013
 finished parsing run0000_0000 timepoint 0.70135
 finished parsing run0000_0000 timepoint 0.7014
 finished parsing run0000_0000 timepoint 0.70145
 finished parsing run0000_0000 timepoint 0.7015
 finished parsing run0000_0000 timepoint 0.70155
 finished parsing run0000_0000 timepoint 0.7016
 finished parsing run0000_0000 timepoint 0.70165
 finished parsing run0000_0000 timepoint 0.7017
 finished parsing run0000_0000 timepoint 0.70175
 finished parsing run0000_0000 timepoint 0.7018
 finished parsing run0000_0000 timepoint 0.70185
 finished parsing run0000_0000 timepoint 0.7019
 finished parsing run0000_0000 timepoint 0.70195
 finished parsing run0000_0000 timepoint 0.702
 finished parsing run0000_0000 timepoint 0.70205
 finished parsing run0000_0000 timepoint 0.7021
 finished parsing run0000_0000 timepoint 0.70215
 finished parsing run0000_0000 timepoint 0.7022
 finished parsing run0000_0000 timepoint 0.70225
 finished parsing run0000_0000 timepoint 0.7023
 finished parsing run0000_0000 timepoint 0.70235
 finished parsing run0000_0000 timepoint 0.7024
 finished parsing run0000_0000 timepoint 0.70245
 finished parsing run0000_0000 timepoint 0.7025
 finished parsing run0000_0000 timepoint 0.70255
 finished parsing run0000_0000 timepoint 0.7026
 finished parsing run0000_0000 timepoint 0.70265
 finished parsing run0000_0000 timepoint 0.7027
 finished parsing run0000_0000 timepoint 0.70275
 finished parsing run0000_0000 timepoint 0.7028
 finished parsing run0000_0000 timepoint 0.70285
 finished parsing run0000_0000 timepoint 0.7029
 finished parsing run0000_0000 timepoint 0.70295
 finished parsing run0000_0000 timepoint 0.703
 finished parsing run0000_0000 timepoint 0.70305
 finished parsing run0000_0000 timepoint 0.7031
 finished parsing run0000_0000 timepoint 0.70315
 finished parsing run0000_0000 timepoint 0.7032
 finished parsing run0000_0000 timepoint 0.70325
 finished parsing run0000_0000 timepoint 0.7033
 finished parsing run0000_0000 timepoint 0.70335
 finished parsing run0000_0000 timepoint 0.7034
 finished parsing run0000_0000 timepoint 0.70345
 finished parsing run0000_0000 timepoint 0.7035
 finished parsing run0000_0000 timepoint 0.70355
 finished parsing run0000_0000 timepoint 0.7036
 finished parsing run0000_0000 timepoint 0.70365
 finished parsing run0000_0000 timepoint 0.7037
 finished parsing run0000_0000 timepoint 0.70375
 finished parsing run0000_0000 timepoint 0.7038
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6907
 finished parsing run0000_0000 timepoint 0.69075
 finished parsing run0000_0000 timepoint 0.6908
 finished parsing run0000_0000 timepoint 0.69085
 finished parsing run0000_0000 timepoint 0.6909
 finished parsing run0000_0000 timepoint 0.69095
 finished parsing run0000_0000 timepoint 0.691
 finished parsing run0000_0000 timepoint 0.69105
 finished parsing run0000_0000 timepoint 0.6911
 finished parsing run0000_0000 timepoint 0.69115
 finished parsing run0000_0000 timepoint 0.6912
 finished parsing run0000_0000 timepoint 0.69125
 finished parsing run0000_0000 timepoint 0.6913
 finished parsing run0000_0000 timepoint 0.69135
 finished parsing run0000_0000 timepoint 0.6914
 finished parsing run0000_0000 timepoint 0.69145
 finished parsing run0000_0000 timepoint 0.6915
 finished parsing run0000_0000 timepoint 0.69155
 finished parsing run0000_0000 timepoint 0.6916
 finished parsing run0000_0000 timepoint 0.69165
 finished parsing run0000_0000 timepoint 0.6917
 finished parsing run0000_0000 timepoint 0.69175
 finished parsing run0000_0000 timepoint 0.6918
 finished parsing run0000_0000 timepoint 0.69185
 finished parsing run0000_0000 timepoint 0.6919
 finished parsing run0000_0000 timepoint 0.69195
 finished parsing run0000_0000 timepoint 0.692
 finished parsing run0000_0000 timepoint 0.69205
 finished parsing run0000_0000 timepoint 0.6921
 finished parsing run0000_0000 timepoint 0.69215
 finished parsing run0000_0000 timepoint 0.6922
 finished parsing run0000_0000 timepoint 0.69225
 finished parsing run0000_0000 timepoint 0.6923
 finished parsing run0000_0000 timepoint 0.69235
 finished parsing run0000_0000 timepoint 0.6924
 finished parsing run0000_0000 timepoint 0.69245
 finished parsing run0000_0000 timepoint 0.6925
 finished parsing run0000_0000 timepoint 0.69255
 finished parsing run0000_0000 timepoint 0.6926
 finished parsing run0000_0000 timepoint 0.69265
 finished parsing run0000_0000 timepoint 0.6927
 finished parsing run0000_0000 timepoint 0.69275
 finished parsing run0000_0000 timepoint 0.6928
 finished parsing run0000_0000 timepoint 0.69285
 finished parsing run0000_0000 timepoint 0.6929
 finished parsing run0000_0000 timepoint 0.69295
 finished parsing run0000_0000 timepoint 0.693
 finished parsing run0000_0000 timepoint 0.69305
 finished parsing run0000_0000 timepoint 0.6931
 finished parsing run0000_0000 timepoint 0.69315
 finished parsing run0000_0000 timepoint 0.6932
 finished parsing run0000_0000 timepoint 0.69325
 finished parsing run0000_0000 timepoint 0.6933
 finished parsing run0000_0000 timepoint 0.69335
 finished parsing run0000_0000 timepoint 0.6934
 finished parsing run0000_0000 timepoint 0.69345
 finished parsing run0000_0000 timepoint 0.6935
 finished parsing run0000_0000 timepoint 0.69355
 finished parsing run0000_0000 timepoint 0.6936
 finished parsing run0000_0000 timepoint 0.69365
 finished parsing run0000_0000 timepoint 0.6937
 finished parsing run0000_0000 timepoint 0.69375
 finished parsing run0000_0000 timepoint 0.6938
 finished parsing run0000_0000 timepoint 0.69385
 finished parsing run0000_0000 timepoint 0.6939
 finished parsing run0000_0000 timepoint 0.69395
 finished parsing run0000_0000 timepoint 0.694
 finished parsing run0000_0000 timepoint 0.69405
 finished parsing run0000_0000 timepoint 0.6941
 finished parsing run0000_0000 timepoint 0.69415
 finished parsing run0000_0000 timepoint 0.6942
 finished parsing run0000_0000 timepoint 0.69425
 finished parsing run0000_0000 timepoint 0.6943
 finished parsing run0000_0000 timepoint 0.69435
 finished parsing run0000_0000 timepoint 0.6944
 finished parsing run0000_0000 timepoint 0.69445
 finished parsing run0000_0000 timepoint 0.6945
 finished parsing run0000_0000 timepoint 0.69455
 finished parsing run0000_0000 timepoint 0.6946
 finished parsing run0000_0000 timepoint 0.69465
 finished parsing run0000_0000 timepoint 0.6947
 finished parsing run0000_0000 timepoint 0.69475
 finished parsing run0000_0000 timepoint 0.6948
 finished parsing run0000_0000 timepoint 0.69485
 finished parsing run0000_0000 timepoint 0.6949
 finished parsing run0000_0000 timepoint 0.69495
 finished parsing run0000_0000 timepoint 0.695
 finished parsing run0000_0000 timepoint 0.69505
 finished parsing run0000_0000 timepoint 0.6951
 finished parsing run0000_0000 timepoint 0.69515
 finished parsing run0000_0000 timepoint 0.6952
 finished parsing run0000_0000 timepoint 0.69525
 finished parsing run0000_0000 timepoint 0.6953
 finished parsing run0000_0000 timepoint 0.69535
 finished parsing run0000_0000 timepoint 0.6954
 finished parsing run0000_0000 timepoint 0.69545
 finished parsing run0000_0000 timepoint 0.6955
 finished parsing run0000_0000 timepoint 0.69555
 finished parsing run0000_0000 timepoint 0.6956
 finished parsing run0000_0000 timepoint 0.69565
 finished parsing run0000_0000 timepoint 0.6957
 finished parsing run0000_0000 timepoint 0.69575
 finished parsing run0000_0000 timepoint 0.6958
 finished parsing run0000_0000 timepoint 0.69585
 finished parsing run0000_0000 timepoint 0.6959
 finished parsing run0000_0000 timepoint 0.69595
 finished parsing run0000_0000 timepoint 0.696
 finished parsing run0000_0000 timepoint 0.69605
 finished parsing run0000_0000 timepoint 0.6961
 finished parsing run0000_0000 timepoint 0.69615
 finished parsing run0000_0000 timepoint 0.6962
 finished parsing run0000_0000 timepoint 0.69625
 finished parsing run0000_0000 timepoint 0.6963
 finished parsing run0000_0000 timepoint 0.69635
 finished parsing run0000_0000 timepoint 0.6964
 finished parsing run0000_0000 timepoint 0.69645
 finished parsing run0000_0000 timepoint 0.6965
 finished parsing run0000_0000 timepoint 0.69655
 finished parsing run0000_0000 timepoint 0.6966
 finished parsing run0000_0000 timepoint 0.69665
 finished parsing run0000_0000 timepoint 0.6967
 finished parsing run0000_0000 timepoint 0.69675
 finished parsing run0000_0000 timepoint 0.6968
 finished parsing run0000_0000 timepoint 0.69685
 finished parsing run0000_0000 timepoint 0.6969
 finished parsing run0000_0000 timepoint 0.69695
 finished parsing run0000_0000 timepoint 0.697
 finished parsing run0000_0000 timepoint 0.69705
 finished parsing run0000_0000 timepoint 0.6971
 finished parsing run0000_0000 timepoint 0.69715
 finished parsing run0000_0000 timepoint 0.6972
 finished parsing run0000_0000 timepoint 0.69725
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.68405
 finished parsing run0000_0000 timepoint 0.6841
 finished parsing run0000_0000 timepoint 0.68415
 finished parsing run0000_0000 timepoint 0.6842
 finished parsing run0000_0000 timepoint 0.68425
 finished parsing run0000_0000 timepoint 0.6843
 finished parsing run0000_0000 timepoint 0.68435
 finished parsing run0000_0000 timepoint 0.6844
 finished parsing run0000_0000 timepoint 0.68445
 finished parsing run0000_0000 timepoint 0.6845
 finished parsing run0000_0000 timepoint 0.68455
 finished parsing run0000_0000 timepoint 0.6846
 finished parsing run0000_0000 timepoint 0.68465
 finished parsing run0000_0000 timepoint 0.6847
 finished parsing run0000_0000 timepoint 0.68475
 finished parsing run0000_0000 timepoint 0.6848
 finished parsing run0000_0000 timepoint 0.68485
 finished parsing run0000_0000 timepoint 0.6849
 finished parsing run0000_0000 timepoint 0.68495
 finished parsing run0000_0000 timepoint 0.685
 finished parsing run0000_0000 timepoint 0.68505
 finished parsing run0000_0000 timepoint 0.6851
 finished parsing run0000_0000 timepoint 0.68515
 finished parsing run0000_0000 timepoint 0.6852
 finished parsing run0000_0000 timepoint 0.68525
 finished parsing run0000_0000 timepoint 0.6853
 finished parsing run0000_0000 timepoint 0.68535
 finished parsing run0000_0000 timepoint 0.6854
 finished parsing run0000_0000 timepoint 0.68545
 finished parsing run0000_0000 timepoint 0.6855
 finished parsing run0000_0000 timepoint 0.68555
 finished parsing run0000_0000 timepoint 0.6856
 finished parsing run0000_0000 timepoint 0.68565
 finished parsing run0000_0000 timepoint 0.6857
 finished parsing run0000_0000 timepoint 0.68575
 finished parsing run0000_0000 timepoint 0.6858
 finished parsing run0000_0000 timepoint 0.68585
 finished parsing run0000_0000 timepoint 0.6859
 finished parsing run0000_0000 timepoint 0.68595
 finished parsing run0000_0000 timepoint 0.686
 finished parsing run0000_0000 timepoint 0.68605
 finished parsing run0000_0000 timepoint 0.6861
 finished parsing run0000_0000 timepoint 0.68615
 finished parsing run0000_0000 timepoint 0.6862
 finished parsing run0000_0000 timepoint 0.68625
 finished parsing run0000_0000 timepoint 0.6863
 finished parsing run0000_0000 timepoint 0.68635
 finished parsing run0000_0000 timepoint 0.6864
 finished parsing run0000_0000 timepoint 0.68645
 finished parsing run0000_0000 timepoint 0.6865
 finished parsing run0000_0000 timepoint 0.68655
 finished parsing run0000_0000 timepoint 0.6866
 finished parsing run0000_0000 timepoint 0.68665
 finished parsing run0000_0000 timepoint 0.6867
 finished parsing run0000_0000 timepoint 0.68675
 finished parsing run0000_0000 timepoint 0.6868
 finished parsing run0000_0000 timepoint 0.68685
 finished parsing run0000_0000 timepoint 0.6869
 finished parsing run0000_0000 timepoint 0.68695
 finished parsing run0000_0000 timepoint 0.687
 finished parsing run0000_0000 timepoint 0.68705
 finished parsing run0000_0000 timepoint 0.6871
 finished parsing run0000_0000 timepoint 0.68715
 finished parsing run0000_0000 timepoint 0.6872
 finished parsing run0000_0000 timepoint 0.68725
 finished parsing run0000_0000 timepoint 0.6873
 finished parsing run0000_0000 timepoint 0.68735
 finished parsing run0000_0000 timepoint 0.6874
 finished parsing run0000_0000 timepoint 0.68745
 finished parsing run0000_0000 timepoint 0.6875
 finished parsing run0000_0000 timepoint 0.68755
 finished parsing run0000_0000 timepoint 0.6876
 finished parsing run0000_0000 timepoint 0.68765
 finished parsing run0000_0000 timepoint 0.6877
 finished parsing run0000_0000 timepoint 0.68775
 finished parsing run0000_0000 timepoint 0.6878
 finished parsing run0000_0000 timepoint 0.68785
 finished parsing run0000_0000 timepoint 0.6879
 finished parsing run0000_0000 timepoint 0.68795
 finished parsing run0000_0000 timepoint 0.688
 finished parsing run0000_0000 timepoint 0.68805
 finished parsing run0000_0000 timepoint 0.6881
 finished parsing run0000_0000 timepoint 0.68815
 finished parsing run0000_0000 timepoint 0.6882
 finished parsing run0000_0000 timepoint 0.68825
 finished parsing run0000_0000 timepoint 0.6883
 finished parsing run0000_0000 timepoint 0.68835
 finished parsing run0000_0000 timepoint 0.6884
 finished parsing run0000_0000 timepoint 0.68845
 finished parsing run0000_0000 timepoint 0.6885
 finished parsing run0000_0000 timepoint 0.68855
 finished parsing run0000_0000 timepoint 0.6886
 finished parsing run0000_0000 timepoint 0.68865
 finished parsing run0000_0000 timepoint 0.6887
 finished parsing run0000_0000 timepoint 0.68875
 finished parsing run0000_0000 timepoint 0.6888
 finished parsing run0000_0000 timepoint 0.68885
 finished parsing run0000_0000 timepoint 0.6889
 finished parsing run0000_0000 timepoint 0.68895
 finished parsing run0000_0000 timepoint 0.689
 finished parsing run0000_0000 timepoint 0.68905
 finished parsing run0000_0000 timepoint 0.6891
 finished parsing run0000_0000 timepoint 0.68915
 finished parsing run0000_0000 timepoint 0.6892
 finished parsing run0000_0000 timepoint 0.68925
 finished parsing run0000_0000 timepoint 0.6893
 finished parsing run0000_0000 timepoint 0.68935
 finished parsing run0000_0000 timepoint 0.6894
 finished parsing run0000_0000 timepoint 0.68945
 finished parsing run0000_0000 timepoint 0.6895
 finished parsing run0000_0000 timepoint 0.68955
 finished parsing run0000_0000 timepoint 0.6896
 finished parsing run0000_0000 timepoint 0.68965
 finished parsing run0000_0000 timepoint 0.6897
 finished parsing run0000_0000 timepoint 0.68975
 finished parsing run0000_0000 timepoint 0.6898
 finished parsing run0000_0000 timepoint 0.68985
 finished parsing run0000_0000 timepoint 0.6899
 finished parsing run0000_0000 timepoint 0.68995
 finished parsing run0000_0000 timepoint 0.69
 finished parsing run0000_0000 timepoint 0.69005
 finished parsing run0000_0000 timepoint 0.6901
 finished parsing run0000_0000 timepoint 0.69015
 finished parsing run0000_0000 timepoint 0.6902
 finished parsing run0000_0000 timepoint 0.69025
 finished parsing run0000_0000 timepoint 0.6903
 finished parsing run0000_0000 timepoint 0.69035
 finished parsing run0000_0000 timepoint 0.6904
 finished parsing run0000_0000 timepoint 0.69045
 finished parsing run0000_0000 timepoint 0.6905
 finished parsing run0000_0000 timepoint 0.69055
 finished parsing run0000_0000 timepoint 0.6906
 finished parsing run0000_0000 timepoint 0.69065
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.67745
 finished parsing run0000_0000 timepoint 0.6775
 finished parsing run0000_0000 timepoint 0.67755
 finished parsing run0000_0000 timepoint 0.6776
 finished parsing run0000_0000 timepoint 0.67765
 finished parsing run0000_0000 timepoint 0.6777
 finished parsing run0000_0000 timepoint 0.67775
 finished parsing run0000_0000 timepoint 0.6778
 finished parsing run0000_0000 timepoint 0.67785
 finished parsing run0000_0000 timepoint 0.6779
 finished parsing run0000_0000 timepoint 0.67795
 finished parsing run0000_0000 timepoint 0.678
 finished parsing run0000_0000 timepoint 0.67805
 finished parsing run0000_0000 timepoint 0.6781
 finished parsing run0000_0000 timepoint 0.67815
 finished parsing run0000_0000 timepoint 0.6782
 finished parsing run0000_0000 timepoint 0.67825
 finished parsing run0000_0000 timepoint 0.6783
 finished parsing run0000_0000 timepoint 0.67835
 finished parsing run0000_0000 timepoint 0.6784
 finished parsing run0000_0000 timepoint 0.67845
 finished parsing run0000_0000 timepoint 0.6785
 finished parsing run0000_0000 timepoint 0.67855
 finished parsing run0000_0000 timepoint 0.6786
 finished parsing run0000_0000 timepoint 0.67865
 finished parsing run0000_0000 timepoint 0.6787
 finished parsing run0000_0000 timepoint 0.67875
 finished parsing run0000_0000 timepoint 0.6788
 finished parsing run0000_0000 timepoint 0.67885
 finished parsing run0000_0000 timepoint 0.6789
 finished parsing run0000_0000 timepoint 0.67895
 finished parsing run0000_0000 timepoint 0.679
 finished parsing run0000_0000 timepoint 0.67905
 finished parsing run0000_0000 timepoint 0.6791
 finished parsing run0000_0000 timepoint 0.67915
 finished parsing run0000_0000 timepoint 0.6792
 finished parsing run0000_0000 timepoint 0.67925
 finished parsing run0000_0000 timepoint 0.6793
 finished parsing run0000_0000 timepoint 0.67935
 finished parsing run0000_0000 timepoint 0.6794
 finished parsing run0000_0000 timepoint 0.67945
 finished parsing run0000_0000 timepoint 0.6795
 finished parsing run0000_0000 timepoint 0.67955
 finished parsing run0000_0000 timepoint 0.6796
 finished parsing run0000_0000 timepoint 0.67965
 finished parsing run0000_0000 timepoint 0.6797
 finished parsing run0000_0000 timepoint 0.67975
 finished parsing run0000_0000 timepoint 0.6798
 finished parsing run0000_0000 timepoint 0.67985
 finished parsing run0000_0000 timepoint 0.6799
 finished parsing run0000_0000 timepoint 0.67995
 finished parsing run0000_0000 timepoint 0.68
 finished parsing run0000_0000 timepoint 0.68005
 finished parsing run0000_0000 timepoint 0.6801
 finished parsing run0000_0000 timepoint 0.68015
 finished parsing run0000_0000 timepoint 0.6802
 finished parsing run0000_0000 timepoint 0.68025
 finished parsing run0000_0000 timepoint 0.6803
 finished parsing run0000_0000 timepoint 0.68035
 finished parsing run0000_0000 timepoint 0.6804
 finished parsing run0000_0000 timepoint 0.68045
 finished parsing run0000_0000 timepoint 0.6805
 finished parsing run0000_0000 timepoint 0.68055
 finished parsing run0000_0000 timepoint 0.6806
 finished parsing run0000_0000 timepoint 0.68065
 finished parsing run0000_0000 timepoint 0.6807
 finished parsing run0000_0000 timepoint 0.68075
 finished parsing run0000_0000 timepoint 0.6808
 finished parsing run0000_0000 timepoint 0.68085
 finished parsing run0000_0000 timepoint 0.6809
 finished parsing run0000_0000 timepoint 0.68095
 finished parsing run0000_0000 timepoint 0.681
 finished parsing run0000_0000 timepoint 0.68105
 finished parsing run0000_0000 timepoint 0.6811
 finished parsing run0000_0000 timepoint 0.68115
 finished parsing run0000_0000 timepoint 0.6812
 finished parsing run0000_0000 timepoint 0.68125
 finished parsing run0000_0000 timepoint 0.6813
 finished parsing run0000_0000 timepoint 0.68135
 finished parsing run0000_0000 timepoint 0.6814
 finished parsing run0000_0000 timepoint 0.68145
 finished parsing run0000_0000 timepoint 0.6815
 finished parsing run0000_0000 timepoint 0.68155
 finished parsing run0000_0000 timepoint 0.6816
 finished parsing run0000_0000 timepoint 0.68165
 finished parsing run0000_0000 timepoint 0.6817
 finished parsing run0000_0000 timepoint 0.68175
 finished parsing run0000_0000 timepoint 0.6818
 finished parsing run0000_0000 timepoint 0.68185
 finished parsing run0000_0000 timepoint 0.6819
 finished parsing run0000_0000 timepoint 0.68195
 finished parsing run0000_0000 timepoint 0.682
 finished parsing run0000_0000 timepoint 0.68205
 finished parsing run0000_0000 timepoint 0.6821
 finished parsing run0000_0000 timepoint 0.68215
 finished parsing run0000_0000 timepoint 0.6822
 finished parsing run0000_0000 timepoint 0.68225
 finished parsing run0000_0000 timepoint 0.6823
 finished parsing run0000_0000 timepoint 0.68235
 finished parsing run0000_0000 timepoint 0.6824
 finished parsing run0000_0000 timepoint 0.68245
 finished parsing run0000_0000 timepoint 0.6825
 finished parsing run0000_0000 timepoint 0.68255
 finished parsing run0000_0000 timepoint 0.6826
 finished parsing run0000_0000 timepoint 0.68265
 finished parsing run0000_0000 timepoint 0.6827
 finished parsing run0000_0000 timepoint 0.68275
 finished parsing run0000_0000 timepoint 0.6828
 finished parsing run0000_0000 timepoint 0.68285
 finished parsing run0000_0000 timepoint 0.6829
 finished parsing run0000_0000 timepoint 0.68295
 finished parsing run0000_0000 timepoint 0.683
 finished parsing run0000_0000 timepoint 0.68305
 finished parsing run0000_0000 timepoint 0.6831
 finished parsing run0000_0000 timepoint 0.68315
 finished parsing run0000_0000 timepoint 0.6832
 finished parsing run0000_0000 timepoint 0.68325
 finished parsing run0000_0000 timepoint 0.6833
 finished parsing run0000_0000 timepoint 0.68335
 finished parsing run0000_0000 timepoint 0.6834
 finished parsing run0000_0000 timepoint 0.68345
 finished parsing run0000_0000 timepoint 0.6835
 finished parsing run0000_0000 timepoint 0.68355
 finished parsing run0000_0000 timepoint 0.6836
 finished parsing run0000_0000 timepoint 0.68365
 finished parsing run0000_0000 timepoint 0.6837
 finished parsing run0000_0000 timepoint 0.68375
 finished parsing run0000_0000 timepoint 0.6838
 finished parsing run0000_0000 timepoint 0.68385
 finished parsing run0000_0000 timepoint 0.6839
 finished parsing run0000_0000 timepoint 0.68395
 finished parsing run0000_0000 timepoint 0.684
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.67105
 finished parsing run0000_0000 timepoint 0.6711
 finished parsing run0000_0000 timepoint 0.67115
 finished parsing run0000_0000 timepoint 0.6712
 finished parsing run0000_0000 timepoint 0.67125
 finished parsing run0000_0000 timepoint 0.6713
 finished parsing run0000_0000 timepoint 0.67135
 finished parsing run0000_0000 timepoint 0.6714
 finished parsing run0000_0000 timepoint 0.67145
 finished parsing run0000_0000 timepoint 0.6715
 finished parsing run0000_0000 timepoint 0.67155
 finished parsing run0000_0000 timepoint 0.6716
 finished parsing run0000_0000 timepoint 0.67165
 finished parsing run0000_0000 timepoint 0.6717
 finished parsing run0000_0000 timepoint 0.67175
 finished parsing run0000_0000 timepoint 0.6718
 finished parsing run0000_0000 timepoint 0.67185
 finished parsing run0000_0000 timepoint 0.6719
 finished parsing run0000_0000 timepoint 0.67195
 finished parsing run0000_0000 timepoint 0.672
 finished parsing run0000_0000 timepoint 0.67205
 finished parsing run0000_0000 timepoint 0.6721
 finished parsing run0000_0000 timepoint 0.67215
 finished parsing run0000_0000 timepoint 0.6722
 finished parsing run0000_0000 timepoint 0.67225
 finished parsing run0000_0000 timepoint 0.6723
 finished parsing run0000_0000 timepoint 0.67235
 finished parsing run0000_0000 timepoint 0.6724
 finished parsing run0000_0000 timepoint 0.67245
 finished parsing run0000_0000 timepoint 0.6725
 finished parsing run0000_0000 timepoint 0.67255
 finished parsing run0000_0000 timepoint 0.6726
 finished parsing run0000_0000 timepoint 0.67265
 finished parsing run0000_0000 timepoint 0.6727
 finished parsing run0000_0000 timepoint 0.67275
 finished parsing run0000_0000 timepoint 0.6728
 finished parsing run0000_0000 timepoint 0.67285
 finished parsing run0000_0000 timepoint 0.6729
 finished parsing run0000_0000 timepoint 0.67295
 finished parsing run0000_0000 timepoint 0.673
 finished parsing run0000_0000 timepoint 0.67305
 finished parsing run0000_0000 timepoint 0.6731
 finished parsing run0000_0000 timepoint 0.67315
 finished parsing run0000_0000 timepoint 0.6732
 finished parsing run0000_0000 timepoint 0.67325
 finished parsing run0000_0000 timepoint 0.6733
 finished parsing run0000_0000 timepoint 0.67335
 finished parsing run0000_0000 timepoint 0.6734
 finished parsing run0000_0000 timepoint 0.67345
 finished parsing run0000_0000 timepoint 0.6735
 finished parsing run0000_0000 timepoint 0.67355
 finished parsing run0000_0000 timepoint 0.6736
 finished parsing run0000_0000 timepoint 0.67365
 finished parsing run0000_0000 timepoint 0.6737
 finished parsing run0000_0000 timepoint 0.67375
 finished parsing run0000_0000 timepoint 0.6738
 finished parsing run0000_0000 timepoint 0.67385
 finished parsing run0000_0000 timepoint 0.6739
 finished parsing run0000_0000 timepoint 0.67395
 finished parsing run0000_0000 timepoint 0.674
 finished parsing run0000_0000 timepoint 0.67405
 finished parsing run0000_0000 timepoint 0.6741
 finished parsing run0000_0000 timepoint 0.67415
 finished parsing run0000_0000 timepoint 0.6742
 finished parsing run0000_0000 timepoint 0.67425
 finished parsing run0000_0000 timepoint 0.6743
 finished parsing run0000_0000 timepoint 0.67435
 finished parsing run0000_0000 timepoint 0.6744
 finished parsing run0000_0000 timepoint 0.67445
 finished parsing run0000_0000 timepoint 0.6745
 finished parsing run0000_0000 timepoint 0.67455
 finished parsing run0000_0000 timepoint 0.6746
 finished parsing run0000_0000 timepoint 0.67465
 finished parsing run0000_0000 timepoint 0.6747
 finished parsing run0000_0000 timepoint 0.67475
 finished parsing run0000_0000 timepoint 0.6748
 finished parsing run0000_0000 timepoint 0.67485
 finished parsing run0000_0000 timepoint 0.6749
 finished parsing run0000_0000 timepoint 0.67495
 finished parsing run0000_0000 timepoint 0.675
 finished parsing run0000_0000 timepoint 0.67505
 finished parsing run0000_0000 timepoint 0.6751
 finished parsing run0000_0000 timepoint 0.67515
 finished parsing run0000_0000 timepoint 0.6752
 finished parsing run0000_0000 timepoint 0.67525
 finished parsing run0000_0000 timepoint 0.6753
 finished parsing run0000_0000 timepoint 0.67535
 finished parsing run0000_0000 timepoint 0.6754
 finished parsing run0000_0000 timepoint 0.67545
 finished parsing run0000_0000 timepoint 0.6755
 finished parsing run0000_0000 timepoint 0.67555
 finished parsing run0000_0000 timepoint 0.6756
 finished parsing run0000_0000 timepoint 0.67565
 finished parsing run0000_0000 timepoint 0.6757
 finished parsing run0000_0000 timepoint 0.67575
 finished parsing run0000_0000 timepoint 0.6758
 finished parsing run0000_0000 timepoint 0.67585
 finished parsing run0000_0000 timepoint 0.6759
 finished parsing run0000_0000 timepoint 0.67595
 finished parsing run0000_0000 timepoint 0.676
 finished parsing run0000_0000 timepoint 0.67605
 finished parsing run0000_0000 timepoint 0.6761
 finished parsing run0000_0000 timepoint 0.67615
 finished parsing run0000_0000 timepoint 0.6762
 finished parsing run0000_0000 timepoint 0.67625
 finished parsing run0000_0000 timepoint 0.6763
 finished parsing run0000_0000 timepoint 0.67635
 finished parsing run0000_0000 timepoint 0.6764
 finished parsing run0000_0000 timepoint 0.67645
 finished parsing run0000_0000 timepoint 0.6765
 finished parsing run0000_0000 timepoint 0.67655
 finished parsing run0000_0000 timepoint 0.6766
 finished parsing run0000_0000 timepoint 0.67665
 finished parsing run0000_0000 timepoint 0.6767
 finished parsing run0000_0000 timepoint 0.67675
 finished parsing run0000_0000 timepoint 0.6768
 finished parsing run0000_0000 timepoint 0.67685
 finished parsing run0000_0000 timepoint 0.6769
 finished parsing run0000_0000 timepoint 0.67695
 finished parsing run0000_0000 timepoint 0.677
 finished parsing run0000_0000 timepoint 0.67705
 finished parsing run0000_0000 timepoint 0.6771
 finished parsing run0000_0000 timepoint 0.67715
 finished parsing run0000_0000 timepoint 0.6772
 finished parsing run0000_0000 timepoint 0.67725
 finished parsing run0000_0000 timepoint 0.6773
 finished parsing run0000_0000 timepoint 0.67735
 finished parsing run0000_0000 timepoint 0.6774
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6644
 finished parsing run0000_0000 timepoint 0.66445
 finished parsing run0000_0000 timepoint 0.6645
 finished parsing run0000_0000 timepoint 0.66455
 finished parsing run0000_0000 timepoint 0.6646
 finished parsing run0000_0000 timepoint 0.66465
 finished parsing run0000_0000 timepoint 0.6647
 finished parsing run0000_0000 timepoint 0.66475
 finished parsing run0000_0000 timepoint 0.6648
 finished parsing run0000_0000 timepoint 0.66485
 finished parsing run0000_0000 timepoint 0.6649
 finished parsing run0000_0000 timepoint 0.66495
 finished parsing run0000_0000 timepoint 0.665
 finished parsing run0000_0000 timepoint 0.66505
 finished parsing run0000_0000 timepoint 0.6651
 finished parsing run0000_0000 timepoint 0.66515
 finished parsing run0000_0000 timepoint 0.6652
 finished parsing run0000_0000 timepoint 0.66525
 finished parsing run0000_0000 timepoint 0.6653
 finished parsing run0000_0000 timepoint 0.66535
 finished parsing run0000_0000 timepoint 0.6654
 finished parsing run0000_0000 timepoint 0.66545
 finished parsing run0000_0000 timepoint 0.6655
 finished parsing run0000_0000 timepoint 0.66555
 finished parsing run0000_0000 timepoint 0.6656
 finished parsing run0000_0000 timepoint 0.66565
 finished parsing run0000_0000 timepoint 0.6657
 finished parsing run0000_0000 timepoint 0.66575
 finished parsing run0000_0000 timepoint 0.6658
 finished parsing run0000_0000 timepoint 0.66585
 finished parsing run0000_0000 timepoint 0.6659
 finished parsing run0000_0000 timepoint 0.66595
 finished parsing run0000_0000 timepoint 0.666
 finished parsing run0000_0000 timepoint 0.66605
 finished parsing run0000_0000 timepoint 0.6661
 finished parsing run0000_0000 timepoint 0.66615
 finished parsing run0000_0000 timepoint 0.6662
 finished parsing run0000_0000 timepoint 0.66625
 finished parsing run0000_0000 timepoint 0.6663
 finished parsing run0000_0000 timepoint 0.66635
 finished parsing run0000_0000 timepoint 0.6664
 finished parsing run0000_0000 timepoint 0.66645
 finished parsing run0000_0000 timepoint 0.6665
 finished parsing run0000_0000 timepoint 0.66655
 finished parsing run0000_0000 timepoint 0.6666
 finished parsing run0000_0000 timepoint 0.66665
 finished parsing run0000_0000 timepoint 0.6667
 finished parsing run0000_0000 timepoint 0.66675
 finished parsing run0000_0000 timepoint 0.6668
 finished parsing run0000_0000 timepoint 0.66685
 finished parsing run0000_0000 timepoint 0.6669
 finished parsing run0000_0000 timepoint 0.66695
 finished parsing run0000_0000 timepoint 0.667
 finished parsing run0000_0000 timepoint 0.66705
 finished parsing run0000_0000 timepoint 0.6671
 finished parsing run0000_0000 timepoint 0.66715
 finished parsing run0000_0000 timepoint 0.6672
 finished parsing run0000_0000 timepoint 0.66725
 finished parsing run0000_0000 timepoint 0.6673
 finished parsing run0000_0000 timepoint 0.66735
 finished parsing run0000_0000 timepoint 0.6674
 finished parsing run0000_0000 timepoint 0.66745
 finished parsing run0000_0000 timepoint 0.6675
 finished parsing run0000_0000 timepoint 0.66755
 finished parsing run0000_0000 timepoint 0.6676
 finished parsing run0000_0000 timepoint 0.66765
 finished parsing run0000_0000 timepoint 0.6677
 finished parsing run0000_0000 timepoint 0.66775
 finished parsing run0000_0000 timepoint 0.6678
 finished parsing run0000_0000 timepoint 0.66785
 finished parsing run0000_0000 timepoint 0.6679
 finished parsing run0000_0000 timepoint 0.66795
 finished parsing run0000_0000 timepoint 0.668
 finished parsing run0000_0000 timepoint 0.66805
 finished parsing run0000_0000 timepoint 0.6681
 finished parsing run0000_0000 timepoint 0.66815
 finished parsing run0000_0000 timepoint 0.6682
 finished parsing run0000_0000 timepoint 0.66825
 finished parsing run0000_0000 timepoint 0.6683
 finished parsing run0000_0000 timepoint 0.66835
 finished parsing run0000_0000 timepoint 0.6684
 finished parsing run0000_0000 timepoint 0.66845
 finished parsing run0000_0000 timepoint 0.6685
 finished parsing run0000_0000 timepoint 0.66855
 finished parsing run0000_0000 timepoint 0.6686
 finished parsing run0000_0000 timepoint 0.66865
 finished parsing run0000_0000 timepoint 0.6687
 finished parsing run0000_0000 timepoint 0.66875
 finished parsing run0000_0000 timepoint 0.6688
 finished parsing run0000_0000 timepoint 0.66885
 finished parsing run0000_0000 timepoint 0.6689
 finished parsing run0000_0000 timepoint 0.66895
 finished parsing run0000_0000 timepoint 0.669
 finished parsing run0000_0000 timepoint 0.66905
 finished parsing run0000_0000 timepoint 0.6691
 finished parsing run0000_0000 timepoint 0.66915
 finished parsing run0000_0000 timepoint 0.6692
 finished parsing run0000_0000 timepoint 0.66925
 finished parsing run0000_0000 timepoint 0.6693
 finished parsing run0000_0000 timepoint 0.66935
 finished parsing run0000_0000 timepoint 0.6694
 finished parsing run0000_0000 timepoint 0.66945
 finished parsing run0000_0000 timepoint 0.6695
 finished parsing run0000_0000 timepoint 0.66955
 finished parsing run0000_0000 timepoint 0.6696
 finished parsing run0000_0000 timepoint 0.66965
 finished parsing run0000_0000 timepoint 0.6697
 finished parsing run0000_0000 timepoint 0.66975
 finished parsing run0000_0000 timepoint 0.6698
 finished parsing run0000_0000 timepoint 0.66985
 finished parsing run0000_0000 timepoint 0.6699
 finished parsing run0000_0000 timepoint 0.66995
 finished parsing run0000_0000 timepoint 0.67
 finished parsing run0000_0000 timepoint 0.67005
 finished parsing run0000_0000 timepoint 0.6701
 finished parsing run0000_0000 timepoint 0.67015
 finished parsing run0000_0000 timepoint 0.6702
 finished parsing run0000_0000 timepoint 0.67025
 finished parsing run0000_0000 timepoint 0.6703
 finished parsing run0000_0000 timepoint 0.67035
 finished parsing run0000_0000 timepoint 0.6704
 finished parsing run0000_0000 timepoint 0.67045
 finished parsing run0000_0000 timepoint 0.6705
 finished parsing run0000_0000 timepoint 0.67055
 finished parsing run0000_0000 timepoint 0.6706
 finished parsing run0000_0000 timepoint 0.67065
 finished parsing run0000_0000 timepoint 0.6707
 finished parsing run0000_0000 timepoint 0.67075
 finished parsing run0000_0000 timepoint 0.6708
 finished parsing run0000_0000 timepoint 0.67085
 finished parsing run0000_0000 timepoint 0.6709
 finished parsing run0000_0000 timepoint 0.67095
 finished parsing run0000_0000 timepoint 0.671
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.65795
 finished parsing run0000_0000 timepoint 0.658
 finished parsing run0000_0000 timepoint 0.65805
 finished parsing run0000_0000 timepoint 0.6581
 finished parsing run0000_0000 timepoint 0.65815
 finished parsing run0000_0000 timepoint 0.6582
 finished parsing run0000_0000 timepoint 0.65825
 finished parsing run0000_0000 timepoint 0.6583
 finished parsing run0000_0000 timepoint 0.65835
 finished parsing run0000_0000 timepoint 0.6584
 finished parsing run0000_0000 timepoint 0.65845
 finished parsing run0000_0000 timepoint 0.6585
 finished parsing run0000_0000 timepoint 0.65855
 finished parsing run0000_0000 timepoint 0.6586
 finished parsing run0000_0000 timepoint 0.65865
 finished parsing run0000_0000 timepoint 0.6587
 finished parsing run0000_0000 timepoint 0.65875
 finished parsing run0000_0000 timepoint 0.6588
 finished parsing run0000_0000 timepoint 0.65885
 finished parsing run0000_0000 timepoint 0.6589
 finished parsing run0000_0000 timepoint 0.65895
 finished parsing run0000_0000 timepoint 0.659
 finished parsing run0000_0000 timepoint 0.65905
 finished parsing run0000_0000 timepoint 0.6591
 finished parsing run0000_0000 timepoint 0.65915
 finished parsing run0000_0000 timepoint 0.6592
 finished parsing run0000_0000 timepoint 0.65925
 finished parsing run0000_0000 timepoint 0.6593
 finished parsing run0000_0000 timepoint 0.65935
 finished parsing run0000_0000 timepoint 0.6594
 finished parsing run0000_0000 timepoint 0.65945
 finished parsing run0000_0000 timepoint 0.6595
 finished parsing run0000_0000 timepoint 0.65955
 finished parsing run0000_0000 timepoint 0.6596
 finished parsing run0000_0000 timepoint 0.65965
 finished parsing run0000_0000 timepoint 0.6597
 finished parsing run0000_0000 timepoint 0.65975
 finished parsing run0000_0000 timepoint 0.6598
 finished parsing run0000_0000 timepoint 0.65985
 finished parsing run0000_0000 timepoint 0.6599
 finished parsing run0000_0000 timepoint 0.65995
 finished parsing run0000_0000 timepoint 0.66
 finished parsing run0000_0000 timepoint 0.66005
 finished parsing run0000_0000 timepoint 0.6601
 finished parsing run0000_0000 timepoint 0.66015
 finished parsing run0000_0000 timepoint 0.6602
 finished parsing run0000_0000 timepoint 0.66025
 finished parsing run0000_0000 timepoint 0.6603
 finished parsing run0000_0000 timepoint 0.66035
 finished parsing run0000_0000 timepoint 0.6604
 finished parsing run0000_0000 timepoint 0.66045
 finished parsing run0000_0000 timepoint 0.6605
 finished parsing run0000_0000 timepoint 0.66055
 finished parsing run0000_0000 timepoint 0.6606
 finished parsing run0000_0000 timepoint 0.66065
 finished parsing run0000_0000 timepoint 0.6607
 finished parsing run0000_0000 timepoint 0.66075
 finished parsing run0000_0000 timepoint 0.6608
 finished parsing run0000_0000 timepoint 0.66085
 finished parsing run0000_0000 timepoint 0.6609
 finished parsing run0000_0000 timepoint 0.66095
 finished parsing run0000_0000 timepoint 0.661
 finished parsing run0000_0000 timepoint 0.66105
 finished parsing run0000_0000 timepoint 0.6611
 finished parsing run0000_0000 timepoint 0.66115
 finished parsing run0000_0000 timepoint 0.6612
 finished parsing run0000_0000 timepoint 0.66125
 finished parsing run0000_0000 timepoint 0.6613
 finished parsing run0000_0000 timepoint 0.66135
 finished parsing run0000_0000 timepoint 0.6614
 finished parsing run0000_0000 timepoint 0.66145
 finished parsing run0000_0000 timepoint 0.6615
 finished parsing run0000_0000 timepoint 0.66155
 finished parsing run0000_0000 timepoint 0.6616
 finished parsing run0000_0000 timepoint 0.66165
 finished parsing run0000_0000 timepoint 0.6617
 finished parsing run0000_0000 timepoint 0.66175
 finished parsing run0000_0000 timepoint 0.6618
 finished parsing run0000_0000 timepoint 0.66185
 finished parsing run0000_0000 timepoint 0.6619
 finished parsing run0000_0000 timepoint 0.66195
 finished parsing run0000_0000 timepoint 0.662
 finished parsing run0000_0000 timepoint 0.66205
 finished parsing run0000_0000 timepoint 0.6621
 finished parsing run0000_0000 timepoint 0.66215
 finished parsing run0000_0000 timepoint 0.6622
 finished parsing run0000_0000 timepoint 0.66225
 finished parsing run0000_0000 timepoint 0.6623
 finished parsing run0000_0000 timepoint 0.66235
 finished parsing run0000_0000 timepoint 0.6624
 finished parsing run0000_0000 timepoint 0.66245
 finished parsing run0000_0000 timepoint 0.6625
 finished parsing run0000_0000 timepoint 0.66255
 finished parsing run0000_0000 timepoint 0.6626
 finished parsing run0000_0000 timepoint 0.66265
 finished parsing run0000_0000 timepoint 0.6627
 finished parsing run0000_0000 timepoint 0.66275
 finished parsing run0000_0000 timepoint 0.6628
 finished parsing run0000_0000 timepoint 0.66285
 finished parsing run0000_0000 timepoint 0.6629
 finished parsing run0000_0000 timepoint 0.66295
 finished parsing run0000_0000 timepoint 0.663
 finished parsing run0000_0000 timepoint 0.66305
 finished parsing run0000_0000 timepoint 0.6631
 finished parsing run0000_0000 timepoint 0.66315
 finished parsing run0000_0000 timepoint 0.6632
 finished parsing run0000_0000 timepoint 0.66325
 finished parsing run0000_0000 timepoint 0.6633
 finished parsing run0000_0000 timepoint 0.66335
 finished parsing run0000_0000 timepoint 0.6634
 finished parsing run0000_0000 timepoint 0.66345
 finished parsing run0000_0000 timepoint 0.6635
 finished parsing run0000_0000 timepoint 0.66355
 finished parsing run0000_0000 timepoint 0.6636
 finished parsing run0000_0000 timepoint 0.66365
 finished parsing run0000_0000 timepoint 0.6637
 finished parsing run0000_0000 timepoint 0.66375
 finished parsing run0000_0000 timepoint 0.6638
 finished parsing run0000_0000 timepoint 0.66385
 finished parsing run0000_0000 timepoint 0.6639
 finished parsing run0000_0000 timepoint 0.66395
 finished parsing run0000_0000 timepoint 0.664
 finished parsing run0000_0000 timepoint 0.66405
 finished parsing run0000_0000 timepoint 0.6641
 finished parsing run0000_0000 timepoint 0.66415
 finished parsing run0000_0000 timepoint 0.6642
 finished parsing run0000_0000 timepoint 0.66425
 finished parsing run0000_0000 timepoint 0.6643
 finished parsing run0000_0000 timepoint 0.66435
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6515
 finished parsing run0000_0000 timepoint 0.65155
 finished parsing run0000_0000 timepoint 0.6516
 finished parsing run0000_0000 timepoint 0.65165
 finished parsing run0000_0000 timepoint 0.6517
 finished parsing run0000_0000 timepoint 0.65175
 finished parsing run0000_0000 timepoint 0.6518
 finished parsing run0000_0000 timepoint 0.65185
 finished parsing run0000_0000 timepoint 0.6519
 finished parsing run0000_0000 timepoint 0.65195
 finished parsing run0000_0000 timepoint 0.652
 finished parsing run0000_0000 timepoint 0.65205
 finished parsing run0000_0000 timepoint 0.6521
 finished parsing run0000_0000 timepoint 0.65215
 finished parsing run0000_0000 timepoint 0.6522
 finished parsing run0000_0000 timepoint 0.65225
 finished parsing run0000_0000 timepoint 0.6523
 finished parsing run0000_0000 timepoint 0.65235
 finished parsing run0000_0000 timepoint 0.6524
 finished parsing run0000_0000 timepoint 0.65245
 finished parsing run0000_0000 timepoint 0.6525
 finished parsing run0000_0000 timepoint 0.65255
 finished parsing run0000_0000 timepoint 0.6526
 finished parsing run0000_0000 timepoint 0.65265
 finished parsing run0000_0000 timepoint 0.6527
 finished parsing run0000_0000 timepoint 0.65275
 finished parsing run0000_0000 timepoint 0.6528
 finished parsing run0000_0000 timepoint 0.65285
 finished parsing run0000_0000 timepoint 0.6529
 finished parsing run0000_0000 timepoint 0.65295
 finished parsing run0000_0000 timepoint 0.653
 finished parsing run0000_0000 timepoint 0.65305
 finished parsing run0000_0000 timepoint 0.6531
 finished parsing run0000_0000 timepoint 0.65315
 finished parsing run0000_0000 timepoint 0.6532
 finished parsing run0000_0000 timepoint 0.65325
 finished parsing run0000_0000 timepoint 0.6533
 finished parsing run0000_0000 timepoint 0.65335
 finished parsing run0000_0000 timepoint 0.6534
 finished parsing run0000_0000 timepoint 0.65345
 finished parsing run0000_0000 timepoint 0.6535
 finished parsing run0000_0000 timepoint 0.65355
 finished parsing run0000_0000 timepoint 0.6536
 finished parsing run0000_0000 timepoint 0.65365
 finished parsing run0000_0000 timepoint 0.6537
 finished parsing run0000_0000 timepoint 0.65375
 finished parsing run0000_0000 timepoint 0.6538
 finished parsing run0000_0000 timepoint 0.65385
 finished parsing run0000_0000 timepoint 0.6539
 finished parsing run0000_0000 timepoint 0.65395
 finished parsing run0000_0000 timepoint 0.654
 finished parsing run0000_0000 timepoint 0.65405
 finished parsing run0000_0000 timepoint 0.6541
 finished parsing run0000_0000 timepoint 0.65415
 finished parsing run0000_0000 timepoint 0.6542
 finished parsing run0000_0000 timepoint 0.65425
 finished parsing run0000_0000 timepoint 0.6543
 finished parsing run0000_0000 timepoint 0.65435
 finished parsing run0000_0000 timepoint 0.6544
 finished parsing run0000_0000 timepoint 0.65445
 finished parsing run0000_0000 timepoint 0.6545
 finished parsing run0000_0000 timepoint 0.65455
 finished parsing run0000_0000 timepoint 0.6546
 finished parsing run0000_0000 timepoint 0.65465
 finished parsing run0000_0000 timepoint 0.6547
 finished parsing run0000_0000 timepoint 0.65475
 finished parsing run0000_0000 timepoint 0.6548
 finished parsing run0000_0000 timepoint 0.65485
 finished parsing run0000_0000 timepoint 0.6549
 finished parsing run0000_0000 timepoint 0.65495
 finished parsing run0000_0000 timepoint 0.655
 finished parsing run0000_0000 timepoint 0.65505
 finished parsing run0000_0000 timepoint 0.6551
 finished parsing run0000_0000 timepoint 0.65515
 finished parsing run0000_0000 timepoint 0.6552
 finished parsing run0000_0000 timepoint 0.65525
 finished parsing run0000_0000 timepoint 0.6553
 finished parsing run0000_0000 timepoint 0.65535
 finished parsing run0000_0000 timepoint 0.6554
 finished parsing run0000_0000 timepoint 0.65545
 finished parsing run0000_0000 timepoint 0.6555
 finished parsing run0000_0000 timepoint 0.65555
 finished parsing run0000_0000 timepoint 0.6556
 finished parsing run0000_0000 timepoint 0.65565
 finished parsing run0000_0000 timepoint 0.6557
 finished parsing run0000_0000 timepoint 0.65575
 finished parsing run0000_0000 timepoint 0.6558
 finished parsing run0000_0000 timepoint 0.65585
 finished parsing run0000_0000 timepoint 0.6559
 finished parsing run0000_0000 timepoint 0.65595
 finished parsing run0000_0000 timepoint 0.656
 finished parsing run0000_0000 timepoint 0.65605
 finished parsing run0000_0000 timepoint 0.6561
 finished parsing run0000_0000 timepoint 0.65615
 finished parsing run0000_0000 timepoint 0.6562
 finished parsing run0000_0000 timepoint 0.65625
 finished parsing run0000_0000 timepoint 0.6563
 finished parsing run0000_0000 timepoint 0.65635
 finished parsing run0000_0000 timepoint 0.6564
 finished parsing run0000_0000 timepoint 0.65645
 finished parsing run0000_0000 timepoint 0.6565
 finished parsing run0000_0000 timepoint 0.65655
 finished parsing run0000_0000 timepoint 0.6566
 finished parsing run0000_0000 timepoint 0.65665
 finished parsing run0000_0000 timepoint 0.6567
 finished parsing run0000_0000 timepoint 0.65675
 finished parsing run0000_0000 timepoint 0.6568
 finished parsing run0000_0000 timepoint 0.65685
 finished parsing run0000_0000 timepoint 0.6569
 finished parsing run0000_0000 timepoint 0.65695
 finished parsing run0000_0000 timepoint 0.657
 finished parsing run0000_0000 timepoint 0.65705
 finished parsing run0000_0000 timepoint 0.6571
 finished parsing run0000_0000 timepoint 0.65715
 finished parsing run0000_0000 timepoint 0.6572
 finished parsing run0000_0000 timepoint 0.65725
 finished parsing run0000_0000 timepoint 0.6573
 finished parsing run0000_0000 timepoint 0.65735
 finished parsing run0000_0000 timepoint 0.6574
 finished parsing run0000_0000 timepoint 0.65745
 finished parsing run0000_0000 timepoint 0.6575
 finished parsing run0000_0000 timepoint 0.65755
 finished parsing run0000_0000 timepoint 0.6576
 finished parsing run0000_0000 timepoint 0.65765
 finished parsing run0000_0000 timepoint 0.6577
 finished parsing run0000_0000 timepoint 0.65775
 finished parsing run0000_0000 timepoint 0.6578
 finished parsing run0000_0000 timepoint 0.65785
 finished parsing run0000_0000 timepoint 0.6579
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6449
 finished parsing run0000_0000 timepoint 0.64495
 finished parsing run0000_0000 timepoint 0.645
 finished parsing run0000_0000 timepoint 0.64505
 finished parsing run0000_0000 timepoint 0.6451
 finished parsing run0000_0000 timepoint 0.64515
 finished parsing run0000_0000 timepoint 0.6452
 finished parsing run0000_0000 timepoint 0.64525
 finished parsing run0000_0000 timepoint 0.6453
 finished parsing run0000_0000 timepoint 0.64535
 finished parsing run0000_0000 timepoint 0.6454
 finished parsing run0000_0000 timepoint 0.64545
 finished parsing run0000_0000 timepoint 0.6455
 finished parsing run0000_0000 timepoint 0.64555
 finished parsing run0000_0000 timepoint 0.6456
 finished parsing run0000_0000 timepoint 0.64565
 finished parsing run0000_0000 timepoint 0.6457
 finished parsing run0000_0000 timepoint 0.64575
 finished parsing run0000_0000 timepoint 0.6458
 finished parsing run0000_0000 timepoint 0.64585
 finished parsing run0000_0000 timepoint 0.6459
 finished parsing run0000_0000 timepoint 0.64595
 finished parsing run0000_0000 timepoint 0.646
 finished parsing run0000_0000 timepoint 0.64605
 finished parsing run0000_0000 timepoint 0.6461
 finished parsing run0000_0000 timepoint 0.64615
 finished parsing run0000_0000 timepoint 0.6462
 finished parsing run0000_0000 timepoint 0.64625
 finished parsing run0000_0000 timepoint 0.6463
 finished parsing run0000_0000 timepoint 0.64635
 finished parsing run0000_0000 timepoint 0.6464
 finished parsing run0000_0000 timepoint 0.64645
 finished parsing run0000_0000 timepoint 0.6465
 finished parsing run0000_0000 timepoint 0.64655
 finished parsing run0000_0000 timepoint 0.6466
 finished parsing run0000_0000 timepoint 0.64665
 finished parsing run0000_0000 timepoint 0.6467
 finished parsing run0000_0000 timepoint 0.64675
 finished parsing run0000_0000 timepoint 0.6468
 finished parsing run0000_0000 timepoint 0.64685
 finished parsing run0000_0000 timepoint 0.6469
 finished parsing run0000_0000 timepoint 0.64695
 finished parsing run0000_0000 timepoint 0.647
 finished parsing run0000_0000 timepoint 0.64705
 finished parsing run0000_0000 timepoint 0.6471
 finished parsing run0000_0000 timepoint 0.64715
 finished parsing run0000_0000 timepoint 0.6472
 finished parsing run0000_0000 timepoint 0.64725
 finished parsing run0000_0000 timepoint 0.6473
 finished parsing run0000_0000 timepoint 0.64735
 finished parsing run0000_0000 timepoint 0.6474
 finished parsing run0000_0000 timepoint 0.64745
 finished parsing run0000_0000 timepoint 0.6475
 finished parsing run0000_0000 timepoint 0.64755
 finished parsing run0000_0000 timepoint 0.6476
 finished parsing run0000_0000 timepoint 0.64765
 finished parsing run0000_0000 timepoint 0.6477
 finished parsing run0000_0000 timepoint 0.64775
 finished parsing run0000_0000 timepoint 0.6478
 finished parsing run0000_0000 timepoint 0.64785
 finished parsing run0000_0000 timepoint 0.6479
 finished parsing run0000_0000 timepoint 0.64795
 finished parsing run0000_0000 timepoint 0.648
 finished parsing run0000_0000 timepoint 0.64805
 finished parsing run0000_0000 timepoint 0.6481
 finished parsing run0000_0000 timepoint 0.64815
 finished parsing run0000_0000 timepoint 0.6482
 finished parsing run0000_0000 timepoint 0.64825
 finished parsing run0000_0000 timepoint 0.6483
 finished parsing run0000_0000 timepoint 0.64835
 finished parsing run0000_0000 timepoint 0.6484
 finished parsing run0000_0000 timepoint 0.64845
 finished parsing run0000_0000 timepoint 0.6485
 finished parsing run0000_0000 timepoint 0.64855
 finished parsing run0000_0000 timepoint 0.6486
 finished parsing run0000_0000 timepoint 0.64865
 finished parsing run0000_0000 timepoint 0.6487
 finished parsing run0000_0000 timepoint 0.64875
 finished parsing run0000_0000 timepoint 0.6488
 finished parsing run0000_0000 timepoint 0.64885
 finished parsing run0000_0000 timepoint 0.6489
 finished parsing run0000_0000 timepoint 0.64895
 finished parsing run0000_0000 timepoint 0.649
 finished parsing run0000_0000 timepoint 0.64905
 finished parsing run0000_0000 timepoint 0.6491
 finished parsing run0000_0000 timepoint 0.64915
 finished parsing run0000_0000 timepoint 0.6492
 finished parsing run0000_0000 timepoint 0.64925
 finished parsing run0000_0000 timepoint 0.6493
 finished parsing run0000_0000 timepoint 0.64935
 finished parsing run0000_0000 timepoint 0.6494
 finished parsing run0000_0000 timepoint 0.64945
 finished parsing run0000_0000 timepoint 0.6495
 finished parsing run0000_0000 timepoint 0.64955
 finished parsing run0000_0000 timepoint 0.6496
 finished parsing run0000_0000 timepoint 0.64965
 finished parsing run0000_0000 timepoint 0.6497
 finished parsing run0000_0000 timepoint 0.64975
 finished parsing run0000_0000 timepoint 0.6498
 finished parsing run0000_0000 timepoint 0.64985
 finished parsing run0000_0000 timepoint 0.6499
 finished parsing run0000_0000 timepoint 0.64995
 finished parsing run0000_0000 timepoint 0.65
 finished parsing run0000_0000 timepoint 0.65005
 finished parsing run0000_0000 timepoint 0.6501
 finished parsing run0000_0000 timepoint 0.65015
 finished parsing run0000_0000 timepoint 0.6502
 finished parsing run0000_0000 timepoint 0.65025
 finished parsing run0000_0000 timepoint 0.6503
 finished parsing run0000_0000 timepoint 0.65035
 finished parsing run0000_0000 timepoint 0.6504
 finished parsing run0000_0000 timepoint 0.65045
 finished parsing run0000_0000 timepoint 0.6505
 finished parsing run0000_0000 timepoint 0.65055
 finished parsing run0000_0000 timepoint 0.6506
 finished parsing run0000_0000 timepoint 0.65065
 finished parsing run0000_0000 timepoint 0.6507
 finished parsing run0000_0000 timepoint 0.65075
 finished parsing run0000_0000 timepoint 0.6508
 finished parsing run0000_0000 timepoint 0.65085
 finished parsing run0000_0000 timepoint 0.6509
 finished parsing run0000_0000 timepoint 0.65095
 finished parsing run0000_0000 timepoint 0.651
 finished parsing run0000_0000 timepoint 0.65105
 finished parsing run0000_0000 timepoint 0.6511
 finished parsing run0000_0000 timepoint 0.65115
 finished parsing run0000_0000 timepoint 0.6512
 finished parsing run0000_0000 timepoint 0.65125
 finished parsing run0000_0000 timepoint 0.6513
 finished parsing run0000_0000 timepoint 0.65135
 finished parsing run0000_0000 timepoint 0.6514
 finished parsing run0000_0000 timepoint 0.65145
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.63855
 finished parsing run0000_0000 timepoint 0.6386
 finished parsing run0000_0000 timepoint 0.63865
 finished parsing run0000_0000 timepoint 0.6387
 finished parsing run0000_0000 timepoint 0.63875
 finished parsing run0000_0000 timepoint 0.6388
 finished parsing run0000_0000 timepoint 0.63885
 finished parsing run0000_0000 timepoint 0.6389
 finished parsing run0000_0000 timepoint 0.63895
 finished parsing run0000_0000 timepoint 0.639
 finished parsing run0000_0000 timepoint 0.63905
 finished parsing run0000_0000 timepoint 0.6391
 finished parsing run0000_0000 timepoint 0.63915
 finished parsing run0000_0000 timepoint 0.6392
 finished parsing run0000_0000 timepoint 0.63925
 finished parsing run0000_0000 timepoint 0.6393
 finished parsing run0000_0000 timepoint 0.63935
 finished parsing run0000_0000 timepoint 0.6394
 finished parsing run0000_0000 timepoint 0.63945
 finished parsing run0000_0000 timepoint 0.6395
 finished parsing run0000_0000 timepoint 0.63955
 finished parsing run0000_0000 timepoint 0.6396
 finished parsing run0000_0000 timepoint 0.63965
 finished parsing run0000_0000 timepoint 0.6397
 finished parsing run0000_0000 timepoint 0.63975
 finished parsing run0000_0000 timepoint 0.6398
 finished parsing run0000_0000 timepoint 0.63985
 finished parsing run0000_0000 timepoint 0.6399
 finished parsing run0000_0000 timepoint 0.63995
 finished parsing run0000_0000 timepoint 0.64
 finished parsing run0000_0000 timepoint 0.64005
 finished parsing run0000_0000 timepoint 0.6401
 finished parsing run0000_0000 timepoint 0.64015
 finished parsing run0000_0000 timepoint 0.6402
 finished parsing run0000_0000 timepoint 0.64025
 finished parsing run0000_0000 timepoint 0.6403
 finished parsing run0000_0000 timepoint 0.64035
 finished parsing run0000_0000 timepoint 0.6404
 finished parsing run0000_0000 timepoint 0.64045
 finished parsing run0000_0000 timepoint 0.6405
 finished parsing run0000_0000 timepoint 0.64055
 finished parsing run0000_0000 timepoint 0.6406
 finished parsing run0000_0000 timepoint 0.64065
 finished parsing run0000_0000 timepoint 0.6407
 finished parsing run0000_0000 timepoint 0.64075
 finished parsing run0000_0000 timepoint 0.6408
 finished parsing run0000_0000 timepoint 0.64085
 finished parsing run0000_0000 timepoint 0.6409
 finished parsing run0000_0000 timepoint 0.64095
 finished parsing run0000_0000 timepoint 0.641
 finished parsing run0000_0000 timepoint 0.64105
 finished parsing run0000_0000 timepoint 0.6411
 finished parsing run0000_0000 timepoint 0.64115
 finished parsing run0000_0000 timepoint 0.6412
 finished parsing run0000_0000 timepoint 0.64125
 finished parsing run0000_0000 timepoint 0.6413
 finished parsing run0000_0000 timepoint 0.64135
 finished parsing run0000_0000 timepoint 0.6414
 finished parsing run0000_0000 timepoint 0.64145
 finished parsing run0000_0000 timepoint 0.6415
 finished parsing run0000_0000 timepoint 0.64155
 finished parsing run0000_0000 timepoint 0.6416
 finished parsing run0000_0000 timepoint 0.64165
 finished parsing run0000_0000 timepoint 0.6417
 finished parsing run0000_0000 timepoint 0.64175
 finished parsing run0000_0000 timepoint 0.6418
 finished parsing run0000_0000 timepoint 0.64185
 finished parsing run0000_0000 timepoint 0.6419
 finished parsing run0000_0000 timepoint 0.64195
 finished parsing run0000_0000 timepoint 0.642
 finished parsing run0000_0000 timepoint 0.64205
 finished parsing run0000_0000 timepoint 0.6421
 finished parsing run0000_0000 timepoint 0.64215
 finished parsing run0000_0000 timepoint 0.6422
 finished parsing run0000_0000 timepoint 0.64225
 finished parsing run0000_0000 timepoint 0.6423
 finished parsing run0000_0000 timepoint 0.64235
 finished parsing run0000_0000 timepoint 0.6424
 finished parsing run0000_0000 timepoint 0.64245
 finished parsing run0000_0000 timepoint 0.6425
 finished parsing run0000_0000 timepoint 0.64255
 finished parsing run0000_0000 timepoint 0.6426
 finished parsing run0000_0000 timepoint 0.64265
 finished parsing run0000_0000 timepoint 0.6427
 finished parsing run0000_0000 timepoint 0.64275
 finished parsing run0000_0000 timepoint 0.6428
 finished parsing run0000_0000 timepoint 0.64285
 finished parsing run0000_0000 timepoint 0.6429
 finished parsing run0000_0000 timepoint 0.64295
 finished parsing run0000_0000 timepoint 0.643
 finished parsing run0000_0000 timepoint 0.64305
 finished parsing run0000_0000 timepoint 0.6431
 finished parsing run0000_0000 timepoint 0.64315
 finished parsing run0000_0000 timepoint 0.6432
 finished parsing run0000_0000 timepoint 0.64325
 finished parsing run0000_0000 timepoint 0.6433
 finished parsing run0000_0000 timepoint 0.64335
 finished parsing run0000_0000 timepoint 0.6434
 finished parsing run0000_0000 timepoint 0.64345
 finished parsing run0000_0000 timepoint 0.6435
 finished parsing run0000_0000 timepoint 0.64355
 finished parsing run0000_0000 timepoint 0.6436
 finished parsing run0000_0000 timepoint 0.64365
 finished parsing run0000_0000 timepoint 0.6437
 finished parsing run0000_0000 timepoint 0.64375
 finished parsing run0000_0000 timepoint 0.6438
 finished parsing run0000_0000 timepoint 0.64385
 finished parsing run0000_0000 timepoint 0.6439
 finished parsing run0000_0000 timepoint 0.64395
 finished parsing run0000_0000 timepoint 0.644
 finished parsing run0000_0000 timepoint 0.64405
 finished parsing run0000_0000 timepoint 0.6441
 finished parsing run0000_0000 timepoint 0.64415
 finished parsing run0000_0000 timepoint 0.6442
 finished parsing run0000_0000 timepoint 0.64425
 finished parsing run0000_0000 timepoint 0.6443
 finished parsing run0000_0000 timepoint 0.64435
 finished parsing run0000_0000 timepoint 0.6444
 finished parsing run0000_0000 timepoint 0.64445
 finished parsing run0000_0000 timepoint 0.6445
 finished parsing run0000_0000 timepoint 0.64455
 finished parsing run0000_0000 timepoint 0.6446
 finished parsing run0000_0000 timepoint 0.64465
 finished parsing run0000_0000 timepoint 0.6447
 finished parsing run0000_0000 timepoint 0.64475
 finished parsing run0000_0000 timepoint 0.6448
 finished parsing run0000_0000 timepoint 0.64485
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.632
 finished parsing run0000_0000 timepoint 0.63205
 finished parsing run0000_0000 timepoint 0.6321
 finished parsing run0000_0000 timepoint 0.63215
 finished parsing run0000_0000 timepoint 0.6322
 finished parsing run0000_0000 timepoint 0.63225
 finished parsing run0000_0000 timepoint 0.6323
 finished parsing run0000_0000 timepoint 0.63235
 finished parsing run0000_0000 timepoint 0.6324
 finished parsing run0000_0000 timepoint 0.63245
 finished parsing run0000_0000 timepoint 0.6325
 finished parsing run0000_0000 timepoint 0.63255
 finished parsing run0000_0000 timepoint 0.6326
 finished parsing run0000_0000 timepoint 0.63265
 finished parsing run0000_0000 timepoint 0.6327
 finished parsing run0000_0000 timepoint 0.63275
 finished parsing run0000_0000 timepoint 0.6328
 finished parsing run0000_0000 timepoint 0.63285
 finished parsing run0000_0000 timepoint 0.6329
 finished parsing run0000_0000 timepoint 0.63295
 finished parsing run0000_0000 timepoint 0.633
 finished parsing run0000_0000 timepoint 0.63305
 finished parsing run0000_0000 timepoint 0.6331
 finished parsing run0000_0000 timepoint 0.63315
 finished parsing run0000_0000 timepoint 0.6332
 finished parsing run0000_0000 timepoint 0.63325
 finished parsing run0000_0000 timepoint 0.6333
 finished parsing run0000_0000 timepoint 0.63335
 finished parsing run0000_0000 timepoint 0.6334
 finished parsing run0000_0000 timepoint 0.63345
 finished parsing run0000_0000 timepoint 0.6335
 finished parsing run0000_0000 timepoint 0.63355
 finished parsing run0000_0000 timepoint 0.6336
 finished parsing run0000_0000 timepoint 0.63365
 finished parsing run0000_0000 timepoint 0.6337
 finished parsing run0000_0000 timepoint 0.63375
 finished parsing run0000_0000 timepoint 0.6338
 finished parsing run0000_0000 timepoint 0.63385
 finished parsing run0000_0000 timepoint 0.6339
 finished parsing run0000_0000 timepoint 0.63395
 finished parsing run0000_0000 timepoint 0.634
 finished parsing run0000_0000 timepoint 0.63405
 finished parsing run0000_0000 timepoint 0.6341
 finished parsing run0000_0000 timepoint 0.63415
 finished parsing run0000_0000 timepoint 0.6342
 finished parsing run0000_0000 timepoint 0.63425
 finished parsing run0000_0000 timepoint 0.6343
 finished parsing run0000_0000 timepoint 0.63435
 finished parsing run0000_0000 timepoint 0.6344
 finished parsing run0000_0000 timepoint 0.63445
 finished parsing run0000_0000 timepoint 0.6345
 finished parsing run0000_0000 timepoint 0.63455
 finished parsing run0000_0000 timepoint 0.6346
 finished parsing run0000_0000 timepoint 0.63465
 finished parsing run0000_0000 timepoint 0.6347
 finished parsing run0000_0000 timepoint 0.63475
 finished parsing run0000_0000 timepoint 0.6348
 finished parsing run0000_0000 timepoint 0.63485
 finished parsing run0000_0000 timepoint 0.6349
 finished parsing run0000_0000 timepoint 0.63495
 finished parsing run0000_0000 timepoint 0.635
 finished parsing run0000_0000 timepoint 0.63505
 finished parsing run0000_0000 timepoint 0.6351
 finished parsing run0000_0000 timepoint 0.63515
 finished parsing run0000_0000 timepoint 0.6352
 finished parsing run0000_0000 timepoint 0.63525
 finished parsing run0000_0000 timepoint 0.6353
 finished parsing run0000_0000 timepoint 0.63535
 finished parsing run0000_0000 timepoint 0.6354
 finished parsing run0000_0000 timepoint 0.63545
 finished parsing run0000_0000 timepoint 0.6355
 finished parsing run0000_0000 timepoint 0.63555
 finished parsing run0000_0000 timepoint 0.6356
 finished parsing run0000_0000 timepoint 0.63565
 finished parsing run0000_0000 timepoint 0.6357
 finished parsing run0000_0000 timepoint 0.63575
 finished parsing run0000_0000 timepoint 0.6358
 finished parsing run0000_0000 timepoint 0.63585
 finished parsing run0000_0000 timepoint 0.6359
 finished parsing run0000_0000 timepoint 0.63595
 finished parsing run0000_0000 timepoint 0.636
 finished parsing run0000_0000 timepoint 0.63605
 finished parsing run0000_0000 timepoint 0.6361
 finished parsing run0000_0000 timepoint 0.63615
 finished parsing run0000_0000 timepoint 0.6362
 finished parsing run0000_0000 timepoint 0.63625
 finished parsing run0000_0000 timepoint 0.6363
 finished parsing run0000_0000 timepoint 0.63635
 finished parsing run0000_0000 timepoint 0.6364
 finished parsing run0000_0000 timepoint 0.63645
 finished parsing run0000_0000 timepoint 0.6365
 finished parsing run0000_0000 timepoint 0.63655
 finished parsing run0000_0000 timepoint 0.6366
 finished parsing run0000_0000 timepoint 0.63665
 finished parsing run0000_0000 timepoint 0.6367
 finished parsing run0000_0000 timepoint 0.63675
 finished parsing run0000_0000 timepoint 0.6368
 finished parsing run0000_0000 timepoint 0.63685
 finished parsing run0000_0000 timepoint 0.6369
 finished parsing run0000_0000 timepoint 0.63695
 finished parsing run0000_0000 timepoint 0.637
 finished parsing run0000_0000 timepoint 0.63705
 finished parsing run0000_0000 timepoint 0.6371
 finished parsing run0000_0000 timepoint 0.63715
 finished parsing run0000_0000 timepoint 0.6372
 finished parsing run0000_0000 timepoint 0.63725
 finished parsing run0000_0000 timepoint 0.6373
 finished parsing run0000_0000 timepoint 0.63735
 finished parsing run0000_0000 timepoint 0.6374
 finished parsing run0000_0000 timepoint 0.63745
 finished parsing run0000_0000 timepoint 0.6375
 finished parsing run0000_0000 timepoint 0.63755
 finished parsing run0000_0000 timepoint 0.6376
 finished parsing run0000_0000 timepoint 0.63765
 finished parsing run0000_0000 timepoint 0.6377
 finished parsing run0000_0000 timepoint 0.63775
 finished parsing run0000_0000 timepoint 0.6378
 finished parsing run0000_0000 timepoint 0.63785
 finished parsing run0000_0000 timepoint 0.6379
 finished parsing run0000_0000 timepoint 0.63795
 finished parsing run0000_0000 timepoint 0.638
 finished parsing run0000_0000 timepoint 0.63805
 finished parsing run0000_0000 timepoint 0.6381
 finished parsing run0000_0000 timepoint 0.63815
 finished parsing run0000_0000 timepoint 0.6382
 finished parsing run0000_0000 timepoint 0.63825
 finished parsing run0000_0000 timepoint 0.6383
 finished parsing run0000_0000 timepoint 0.63835
 finished parsing run0000_0000 timepoint 0.6384
 finished parsing run0000_0000 timepoint 0.63845
 finished parsing run0000_0000 timepoint 0.6385
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6255
 finished parsing run0000_0000 timepoint 0.62555
 finished parsing run0000_0000 timepoint 0.6256
 finished parsing run0000_0000 timepoint 0.62565
 finished parsing run0000_0000 timepoint 0.6257
 finished parsing run0000_0000 timepoint 0.62575
 finished parsing run0000_0000 timepoint 0.6258
 finished parsing run0000_0000 timepoint 0.62585
 finished parsing run0000_0000 timepoint 0.6259
 finished parsing run0000_0000 timepoint 0.62595
 finished parsing run0000_0000 timepoint 0.626
 finished parsing run0000_0000 timepoint 0.62605
 finished parsing run0000_0000 timepoint 0.6261
 finished parsing run0000_0000 timepoint 0.62615
 finished parsing run0000_0000 timepoint 0.6262
 finished parsing run0000_0000 timepoint 0.62625
 finished parsing run0000_0000 timepoint 0.6263
 finished parsing run0000_0000 timepoint 0.62635
 finished parsing run0000_0000 timepoint 0.6264
 finished parsing run0000_0000 timepoint 0.62645
 finished parsing run0000_0000 timepoint 0.6265
 finished parsing run0000_0000 timepoint 0.62655
 finished parsing run0000_0000 timepoint 0.6266
 finished parsing run0000_0000 timepoint 0.62665
 finished parsing run0000_0000 timepoint 0.6267
 finished parsing run0000_0000 timepoint 0.62675
 finished parsing run0000_0000 timepoint 0.6268
 finished parsing run0000_0000 timepoint 0.62685
 finished parsing run0000_0000 timepoint 0.6269
 finished parsing run0000_0000 timepoint 0.62695
 finished parsing run0000_0000 timepoint 0.627
 finished parsing run0000_0000 timepoint 0.62705
 finished parsing run0000_0000 timepoint 0.6271
 finished parsing run0000_0000 timepoint 0.62715
 finished parsing run0000_0000 timepoint 0.6272
 finished parsing run0000_0000 timepoint 0.62725
 finished parsing run0000_0000 timepoint 0.6273
 finished parsing run0000_0000 timepoint 0.62735
 finished parsing run0000_0000 timepoint 0.6274
 finished parsing run0000_0000 timepoint 0.62745
 finished parsing run0000_0000 timepoint 0.6275
 finished parsing run0000_0000 timepoint 0.62755
 finished parsing run0000_0000 timepoint 0.6276
 finished parsing run0000_0000 timepoint 0.62765
 finished parsing run0000_0000 timepoint 0.6277
 finished parsing run0000_0000 timepoint 0.62775
 finished parsing run0000_0000 timepoint 0.6278
 finished parsing run0000_0000 timepoint 0.62785
 finished parsing run0000_0000 timepoint 0.6279
 finished parsing run0000_0000 timepoint 0.62795
 finished parsing run0000_0000 timepoint 0.628
 finished parsing run0000_0000 timepoint 0.62805
 finished parsing run0000_0000 timepoint 0.6281
 finished parsing run0000_0000 timepoint 0.62815
 finished parsing run0000_0000 timepoint 0.6282
 finished parsing run0000_0000 timepoint 0.62825
 finished parsing run0000_0000 timepoint 0.6283
 finished parsing run0000_0000 timepoint 0.62835
 finished parsing run0000_0000 timepoint 0.6284
 finished parsing run0000_0000 timepoint 0.62845
 finished parsing run0000_0000 timepoint 0.6285
 finished parsing run0000_0000 timepoint 0.62855
 finished parsing run0000_0000 timepoint 0.6286
 finished parsing run0000_0000 timepoint 0.62865
 finished parsing run0000_0000 timepoint 0.6287
 finished parsing run0000_0000 timepoint 0.62875
 finished parsing run0000_0000 timepoint 0.6288
 finished parsing run0000_0000 timepoint 0.62885
 finished parsing run0000_0000 timepoint 0.6289
 finished parsing run0000_0000 timepoint 0.62895
 finished parsing run0000_0000 timepoint 0.629
 finished parsing run0000_0000 timepoint 0.62905
 finished parsing run0000_0000 timepoint 0.6291
 finished parsing run0000_0000 timepoint 0.62915
 finished parsing run0000_0000 timepoint 0.6292
 finished parsing run0000_0000 timepoint 0.62925
 finished parsing run0000_0000 timepoint 0.6293
 finished parsing run0000_0000 timepoint 0.62935
 finished parsing run0000_0000 timepoint 0.6294
 finished parsing run0000_0000 timepoint 0.62945
 finished parsing run0000_0000 timepoint 0.6295
 finished parsing run0000_0000 timepoint 0.62955
 finished parsing run0000_0000 timepoint 0.6296
 finished parsing run0000_0000 timepoint 0.62965
 finished parsing run0000_0000 timepoint 0.6297
 finished parsing run0000_0000 timepoint 0.62975
 finished parsing run0000_0000 timepoint 0.6298
 finished parsing run0000_0000 timepoint 0.62985
 finished parsing run0000_0000 timepoint 0.6299
 finished parsing run0000_0000 timepoint 0.62995
 finished parsing run0000_0000 timepoint 0.63
 finished parsing run0000_0000 timepoint 0.63005
 finished parsing run0000_0000 timepoint 0.6301
 finished parsing run0000_0000 timepoint 0.63015
 finished parsing run0000_0000 timepoint 0.6302
 finished parsing run0000_0000 timepoint 0.63025
 finished parsing run0000_0000 timepoint 0.6303
 finished parsing run0000_0000 timepoint 0.63035
 finished parsing run0000_0000 timepoint 0.6304
 finished parsing run0000_0000 timepoint 0.63045
 finished parsing run0000_0000 timepoint 0.6305
 finished parsing run0000_0000 timepoint 0.63055
 finished parsing run0000_0000 timepoint 0.6306
 finished parsing run0000_0000 timepoint 0.63065
 finished parsing run0000_0000 timepoint 0.6307
 finished parsing run0000_0000 timepoint 0.63075
 finished parsing run0000_0000 timepoint 0.6308
 finished parsing run0000_0000 timepoint 0.63085
 finished parsing run0000_0000 timepoint 0.6309
 finished parsing run0000_0000 timepoint 0.63095
 finished parsing run0000_0000 timepoint 0.631
 finished parsing run0000_0000 timepoint 0.63105
 finished parsing run0000_0000 timepoint 0.6311
 finished parsing run0000_0000 timepoint 0.63115
 finished parsing run0000_0000 timepoint 0.6312
 finished parsing run0000_0000 timepoint 0.63125
 finished parsing run0000_0000 timepoint 0.6313
 finished parsing run0000_0000 timepoint 0.63135
 finished parsing run0000_0000 timepoint 0.6314
 finished parsing run0000_0000 timepoint 0.63145
 finished parsing run0000_0000 timepoint 0.6315
 finished parsing run0000_0000 timepoint 0.63155
 finished parsing run0000_0000 timepoint 0.6316
 finished parsing run0000_0000 timepoint 0.63165
 finished parsing run0000_0000 timepoint 0.6317
 finished parsing run0000_0000 timepoint 0.63175
 finished parsing run0000_0000 timepoint 0.6318
 finished parsing run0000_0000 timepoint 0.63185
 finished parsing run0000_0000 timepoint 0.6319
 finished parsing run0000_0000 timepoint 0.63195
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6191
 finished parsing run0000_0000 timepoint 0.61915
 finished parsing run0000_0000 timepoint 0.6192
 finished parsing run0000_0000 timepoint 0.61925
 finished parsing run0000_0000 timepoint 0.6193
 finished parsing run0000_0000 timepoint 0.61935
 finished parsing run0000_0000 timepoint 0.6194
 finished parsing run0000_0000 timepoint 0.61945
 finished parsing run0000_0000 timepoint 0.6195
 finished parsing run0000_0000 timepoint 0.61955
 finished parsing run0000_0000 timepoint 0.6196
 finished parsing run0000_0000 timepoint 0.61965
 finished parsing run0000_0000 timepoint 0.6197
 finished parsing run0000_0000 timepoint 0.61975
 finished parsing run0000_0000 timepoint 0.6198
 finished parsing run0000_0000 timepoint 0.61985
 finished parsing run0000_0000 timepoint 0.6199
 finished parsing run0000_0000 timepoint 0.61995
 finished parsing run0000_0000 timepoint 0.62
 finished parsing run0000_0000 timepoint 0.62005
 finished parsing run0000_0000 timepoint 0.6201
 finished parsing run0000_0000 timepoint 0.62015
 finished parsing run0000_0000 timepoint 0.6202
 finished parsing run0000_0000 timepoint 0.62025
 finished parsing run0000_0000 timepoint 0.6203
 finished parsing run0000_0000 timepoint 0.62035
 finished parsing run0000_0000 timepoint 0.6204
 finished parsing run0000_0000 timepoint 0.62045
 finished parsing run0000_0000 timepoint 0.6205
 finished parsing run0000_0000 timepoint 0.62055
 finished parsing run0000_0000 timepoint 0.6206
 finished parsing run0000_0000 timepoint 0.62065
 finished parsing run0000_0000 timepoint 0.6207
 finished parsing run0000_0000 timepoint 0.62075
 finished parsing run0000_0000 timepoint 0.6208
 finished parsing run0000_0000 timepoint 0.62085
 finished parsing run0000_0000 timepoint 0.6209
 finished parsing run0000_0000 timepoint 0.62095
 finished parsing run0000_0000 timepoint 0.621
 finished parsing run0000_0000 timepoint 0.62105
 finished parsing run0000_0000 timepoint 0.6211
 finished parsing run0000_0000 timepoint 0.62115
 finished parsing run0000_0000 timepoint 0.6212
 finished parsing run0000_0000 timepoint 0.62125
 finished parsing run0000_0000 timepoint 0.6213
 finished parsing run0000_0000 timepoint 0.62135
 finished parsing run0000_0000 timepoint 0.6214
 finished parsing run0000_0000 timepoint 0.62145
 finished parsing run0000_0000 timepoint 0.6215
 finished parsing run0000_0000 timepoint 0.62155
 finished parsing run0000_0000 timepoint 0.6216
 finished parsing run0000_0000 timepoint 0.62165
 finished parsing run0000_0000 timepoint 0.6217
 finished parsing run0000_0000 timepoint 0.62175
 finished parsing run0000_0000 timepoint 0.6218
 finished parsing run0000_0000 timepoint 0.62185
 finished parsing run0000_0000 timepoint 0.6219
 finished parsing run0000_0000 timepoint 0.62195
 finished parsing run0000_0000 timepoint 0.622
 finished parsing run0000_0000 timepoint 0.62205
 finished parsing run0000_0000 timepoint 0.6221
 finished parsing run0000_0000 timepoint 0.62215
 finished parsing run0000_0000 timepoint 0.6222
 finished parsing run0000_0000 timepoint 0.62225
 finished parsing run0000_0000 timepoint 0.6223
 finished parsing run0000_0000 timepoint 0.62235
 finished parsing run0000_0000 timepoint 0.6224
 finished parsing run0000_0000 timepoint 0.62245
 finished parsing run0000_0000 timepoint 0.6225
 finished parsing run0000_0000 timepoint 0.62255
 finished parsing run0000_0000 timepoint 0.6226
 finished parsing run0000_0000 timepoint 0.62265
 finished parsing run0000_0000 timepoint 0.6227
 finished parsing run0000_0000 timepoint 0.62275
 finished parsing run0000_0000 timepoint 0.6228
 finished parsing run0000_0000 timepoint 0.62285
 finished parsing run0000_0000 timepoint 0.6229
 finished parsing run0000_0000 timepoint 0.62295
 finished parsing run0000_0000 timepoint 0.623
 finished parsing run0000_0000 timepoint 0.62305
 finished parsing run0000_0000 timepoint 0.6231
 finished parsing run0000_0000 timepoint 0.62315
 finished parsing run0000_0000 timepoint 0.6232
 finished parsing run0000_0000 timepoint 0.62325
 finished parsing run0000_0000 timepoint 0.6233
 finished parsing run0000_0000 timepoint 0.62335
 finished parsing run0000_0000 timepoint 0.6234
 finished parsing run0000_0000 timepoint 0.62345
 finished parsing run0000_0000 timepoint 0.6235
 finished parsing run0000_0000 timepoint 0.62355
 finished parsing run0000_0000 timepoint 0.6236
 finished parsing run0000_0000 timepoint 0.62365
 finished parsing run0000_0000 timepoint 0.6237
 finished parsing run0000_0000 timepoint 0.62375
 finished parsing run0000_0000 timepoint 0.6238
 finished parsing run0000_0000 timepoint 0.62385
 finished parsing run0000_0000 timepoint 0.6239
 finished parsing run0000_0000 timepoint 0.62395
 finished parsing run0000_0000 timepoint 0.624
 finished parsing run0000_0000 timepoint 0.62405
 finished parsing run0000_0000 timepoint 0.6241
 finished parsing run0000_0000 timepoint 0.62415
 finished parsing run0000_0000 timepoint 0.6242
 finished parsing run0000_0000 timepoint 0.62425
 finished parsing run0000_0000 timepoint 0.6243
 finished parsing run0000_0000 timepoint 0.62435
 finished parsing run0000_0000 timepoint 0.6244
 finished parsing run0000_0000 timepoint 0.62445
 finished parsing run0000_0000 timepoint 0.6245
 finished parsing run0000_0000 timepoint 0.62455
 finished parsing run0000_0000 timepoint 0.6246
 finished parsing run0000_0000 timepoint 0.62465
 finished parsing run0000_0000 timepoint 0.6247
 finished parsing run0000_0000 timepoint 0.62475
 finished parsing run0000_0000 timepoint 0.6248
 finished parsing run0000_0000 timepoint 0.62485
 finished parsing run0000_0000 timepoint 0.6249
 finished parsing run0000_0000 timepoint 0.62495
 finished parsing run0000_0000 timepoint 0.625
 finished parsing run0000_0000 timepoint 0.62505
 finished parsing run0000_0000 timepoint 0.6251
 finished parsing run0000_0000 timepoint 0.62515
 finished parsing run0000_0000 timepoint 0.6252
 finished parsing run0000_0000 timepoint 0.62525
 finished parsing run0000_0000 timepoint 0.6253
 finished parsing run0000_0000 timepoint 0.62535
 finished parsing run0000_0000 timepoint 0.6254
 finished parsing run0000_0000 timepoint 0.62545
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.6126
 finished parsing run0000_0000 timepoint 0.61265
 finished parsing run0000_0000 timepoint 0.6127
 finished parsing run0000_0000 timepoint 0.61275
 finished parsing run0000_0000 timepoint 0.6128
 finished parsing run0000_0000 timepoint 0.61285
 finished parsing run0000_0000 timepoint 0.6129
 finished parsing run0000_0000 timepoint 0.61295
 finished parsing run0000_0000 timepoint 0.613
 finished parsing run0000_0000 timepoint 0.61305
 finished parsing run0000_0000 timepoint 0.6131
 finished parsing run0000_0000 timepoint 0.61315
 finished parsing run0000_0000 timepoint 0.6132
 finished parsing run0000_0000 timepoint 0.61325
 finished parsing run0000_0000 timepoint 0.6133
 finished parsing run0000_0000 timepoint 0.61335
 finished parsing run0000_0000 timepoint 0.6134
 finished parsing run0000_0000 timepoint 0.61345
 finished parsing run0000_0000 timepoint 0.6135
 finished parsing run0000_0000 timepoint 0.61355
 finished parsing run0000_0000 timepoint 0.6136
 finished parsing run0000_0000 timepoint 0.61365
 finished parsing run0000_0000 timepoint 0.6137
 finished parsing run0000_0000 timepoint 0.61375
 finished parsing run0000_0000 timepoint 0.6138
 finished parsing run0000_0000 timepoint 0.61385
 finished parsing run0000_0000 timepoint 0.6139
 finished parsing run0000_0000 timepoint 0.61395
 finished parsing run0000_0000 timepoint 0.614
 finished parsing run0000_0000 timepoint 0.61405
 finished parsing run0000_0000 timepoint 0.6141
 finished parsing run0000_0000 timepoint 0.61415
 finished parsing run0000_0000 timepoint 0.6142
 finished parsing run0000_0000 timepoint 0.61425
 finished parsing run0000_0000 timepoint 0.6143
 finished parsing run0000_0000 timepoint 0.61435
 finished parsing run0000_0000 timepoint 0.6144
 finished parsing run0000_0000 timepoint 0.61445
 finished parsing run0000_0000 timepoint 0.6145
 finished parsing run0000_0000 timepoint 0.61455
 finished parsing run0000_0000 timepoint 0.6146
 finished parsing run0000_0000 timepoint 0.61465
 finished parsing run0000_0000 timepoint 0.6147
 finished parsing run0000_0000 timepoint 0.61475
 finished parsing run0000_0000 timepoint 0.6148
 finished parsing run0000_0000 timepoint 0.61485
 finished parsing run0000_0000 timepoint 0.6149
 finished parsing run0000_0000 timepoint 0.61495
 finished parsing run0000_0000 timepoint 0.615
 finished parsing run0000_0000 timepoint 0.61505
 finished parsing run0000_0000 timepoint 0.6151
 finished parsing run0000_0000 timepoint 0.61515
 finished parsing run0000_0000 timepoint 0.6152
 finished parsing run0000_0000 timepoint 0.61525
 finished parsing run0000_0000 timepoint 0.6153
 finished parsing run0000_0000 timepoint 0.61535
 finished parsing run0000_0000 timepoint 0.6154
 finished parsing run0000_0000 timepoint 0.61545
 finished parsing run0000_0000 timepoint 0.6155
 finished parsing run0000_0000 timepoint 0.61555
 finished parsing run0000_0000 timepoint 0.6156
 finished parsing run0000_0000 timepoint 0.61565
 finished parsing run0000_0000 timepoint 0.6157
 finished parsing run0000_0000 timepoint 0.61575
 finished parsing run0000_0000 timepoint 0.6158
 finished parsing run0000_0000 timepoint 0.61585
 finished parsing run0000_0000 timepoint 0.6159
 finished parsing run0000_0000 timepoint 0.61595
 finished parsing run0000_0000 timepoint 0.616
 finished parsing run0000_0000 timepoint 0.61605
 finished parsing run0000_0000 timepoint 0.6161
 finished parsing run0000_0000 timepoint 0.61615
 finished parsing run0000_0000 timepoint 0.6162
 finished parsing run0000_0000 timepoint 0.61625
 finished parsing run0000_0000 timepoint 0.6163
 finished parsing run0000_0000 timepoint 0.61635
 finished parsing run0000_0000 timepoint 0.6164
 finished parsing run0000_0000 timepoint 0.61645
 finished parsing run0000_0000 timepoint 0.6165
 finished parsing run0000_0000 timepoint 0.61655
 finished parsing run0000_0000 timepoint 0.6166
 finished parsing run0000_0000 timepoint 0.61665
 finished parsing run0000_0000 timepoint 0.6167
 finished parsing run0000_0000 timepoint 0.61675
 finished parsing run0000_0000 timepoint 0.6168
 finished parsing run0000_0000 timepoint 0.61685
 finished parsing run0000_0000 timepoint 0.6169
 finished parsing run0000_0000 timepoint 0.61695
 finished parsing run0000_0000 timepoint 0.617
 finished parsing run0000_0000 timepoint 0.61705
 finished parsing run0000_0000 timepoint 0.6171
 finished parsing run0000_0000 timepoint 0.61715
 finished parsing run0000_0000 timepoint 0.6172
 finished parsing run0000_0000 timepoint 0.61725
 finished parsing run0000_0000 timepoint 0.6173
 finished parsing run0000_0000 timepoint 0.61735
 finished parsing run0000_0000 timepoint 0.6174
 finished parsing run0000_0000 timepoint 0.61745
 finished parsing run0000_0000 timepoint 0.6175
 finished parsing run0000_0000 timepoint 0.61755
 finished parsing run0000_0000 timepoint 0.6176
 finished parsing run0000_0000 timepoint 0.61765
 finished parsing run0000_0000 timepoint 0.6177
 finished parsing run0000_0000 timepoint 0.61775
 finished parsing run0000_0000 timepoint 0.6178
 finished parsing run0000_0000 timepoint 0.61785
 finished parsing run0000_0000 timepoint 0.6179
 finished parsing run0000_0000 timepoint 0.61795
 finished parsing run0000_0000 timepoint 0.618
 finished parsing run0000_0000 timepoint 0.61805
 finished parsing run0000_0000 timepoint 0.6181
 finished parsing run0000_0000 timepoint 0.61815
 finished parsing run0000_0000 timepoint 0.6182
 finished parsing run0000_0000 timepoint 0.61825
 finished parsing run0000_0000 timepoint 0.6183
 finished parsing run0000_0000 timepoint 0.61835
 finished parsing run0000_0000 timepoint 0.6184
 finished parsing run0000_0000 timepoint 0.61845
 finished parsing run0000_0000 timepoint 0.6185
 finished parsing run0000_0000 timepoint 0.61855
 finished parsing run0000_0000 timepoint 0.6186
 finished parsing run0000_0000 timepoint 0.61865
 finished parsing run0000_0000 timepoint 0.6187
 finished parsing run0000_0000 timepoint 0.61875
 finished parsing run0000_0000 timepoint 0.6188
 finished parsing run0000_0000 timepoint 0.61885
 finished parsing run0000_0000 timepoint 0.6189
 finished parsing run0000_0000 timepoint 0.61895
 finished parsing run0000_0000 timepoint 0.619
 finished parsing run0000_0000 timepoint 0.61905
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.606
 finished parsing run0000_0000 timepoint 0.60605
 finished parsing run0000_0000 timepoint 0.6061
 finished parsing run0000_0000 timepoint 0.60615
 finished parsing run0000_0000 timepoint 0.6062
 finished parsing run0000_0000 timepoint 0.60625
 finished parsing run0000_0000 timepoint 0.6063
 finished parsing run0000_0000 timepoint 0.60635
 finished parsing run0000_0000 timepoint 0.6064
 finished parsing run0000_0000 timepoint 0.60645
 finished parsing run0000_0000 timepoint 0.6065
 finished parsing run0000_0000 timepoint 0.60655
 finished parsing run0000_0000 timepoint 0.6066
 finished parsing run0000_0000 timepoint 0.60665
 finished parsing run0000_0000 timepoint 0.6067
 finished parsing run0000_0000 timepoint 0.60675
 finished parsing run0000_0000 timepoint 0.6068
 finished parsing run0000_0000 timepoint 0.60685
 finished parsing run0000_0000 timepoint 0.6069
 finished parsing run0000_0000 timepoint 0.60695
 finished parsing run0000_0000 timepoint 0.607
 finished parsing run0000_0000 timepoint 0.60705
 finished parsing run0000_0000 timepoint 0.6071
 finished parsing run0000_0000 timepoint 0.60715
 finished parsing run0000_0000 timepoint 0.6072
 finished parsing run0000_0000 timepoint 0.60725
 finished parsing run0000_0000 timepoint 0.6073
 finished parsing run0000_0000 timepoint 0.60735
 finished parsing run0000_0000 timepoint 0.6074
 finished parsing run0000_0000 timepoint 0.60745
 finished parsing run0000_0000 timepoint 0.6075
 finished parsing run0000_0000 timepoint 0.60755
 finished parsing run0000_0000 timepoint 0.6076
 finished parsing run0000_0000 timepoint 0.60765
 finished parsing run0000_0000 timepoint 0.6077
 finished parsing run0000_0000 timepoint 0.60775
 finished parsing run0000_0000 timepoint 0.6078
 finished parsing run0000_0000 timepoint 0.60785
 finished parsing run0000_0000 timepoint 0.6079
 finished parsing run0000_0000 timepoint 0.60795
 finished parsing run0000_0000 timepoint 0.608
 finished parsing run0000_0000 timepoint 0.60805
 finished parsing run0000_0000 timepoint 0.6081
 finished parsing run0000_0000 timepoint 0.60815
 finished parsing run0000_0000 timepoint 0.6082
 finished parsing run0000_0000 timepoint 0.60825
 finished parsing run0000_0000 timepoint 0.6083
 finished parsing run0000_0000 timepoint 0.60835
 finished parsing run0000_0000 timepoint 0.6084
 finished parsing run0000_0000 timepoint 0.60845
 finished parsing run0000_0000 timepoint 0.6085
 finished parsing run0000_0000 timepoint 0.60855
 finished parsing run0000_0000 timepoint 0.6086
 finished parsing run0000_0000 timepoint 0.60865
 finished parsing run0000_0000 timepoint 0.6087
 finished parsing run0000_0000 timepoint 0.60875
 finished parsing run0000_0000 timepoint 0.6088
 finished parsing run0000_0000 timepoint 0.60885
 finished parsing run0000_0000 timepoint 0.6089
 finished parsing run0000_0000 timepoint 0.60895
 finished parsing run0000_0000 timepoint 0.609
 finished parsing run0000_0000 timepoint 0.60905
 finished parsing run0000_0000 timepoint 0.6091
 finished parsing run0000_0000 timepoint 0.60915
 finished parsing run0000_0000 timepoint 0.6092
 finished parsing run0000_0000 timepoint 0.60925
 finished parsing run0000_0000 timepoint 0.6093
 finished parsing run0000_0000 timepoint 0.60935
 finished parsing run0000_0000 timepoint 0.6094
 finished parsing run0000_0000 timepoint 0.60945
 finished parsing run0000_0000 timepoint 0.6095
 finished parsing run0000_0000 timepoint 0.60955
 finished parsing run0000_0000 timepoint 0.6096
 finished parsing run0000_0000 timepoint 0.60965
 finished parsing run0000_0000 timepoint 0.6097
 finished parsing run0000_0000 timepoint 0.60975
 finished parsing run0000_0000 timepoint 0.6098
 finished parsing run0000_0000 timepoint 0.60985
 finished parsing run0000_0000 timepoint 0.6099
 finished parsing run0000_0000 timepoint 0.60995
 finished parsing run0000_0000 timepoint 0.61
 finished parsing run0000_0000 timepoint 0.61005
 finished parsing run0000_0000 timepoint 0.6101
 finished parsing run0000_0000 timepoint 0.61015
 finished parsing run0000_0000 timepoint 0.6102
 finished parsing run0000_0000 timepoint 0.61025
 finished parsing run0000_0000 timepoint 0.6103
 finished parsing run0000_0000 timepoint 0.61035
 finished parsing run0000_0000 timepoint 0.6104
 finished parsing run0000_0000 timepoint 0.61045
 finished parsing run0000_0000 timepoint 0.6105
 finished parsing run0000_0000 timepoint 0.61055
 finished parsing run0000_0000 timepoint 0.6106
 finished parsing run0000_0000 timepoint 0.61065
 finished parsing run0000_0000 timepoint 0.6107
 finished parsing run0000_0000 timepoint 0.61075
 finished parsing run0000_0000 timepoint 0.6108
 finished parsing run0000_0000 timepoint 0.61085
 finished parsing run0000_0000 timepoint 0.6109
 finished parsing run0000_0000 timepoint 0.61095
 finished parsing run0000_0000 timepoint 0.611
 finished parsing run0000_0000 timepoint 0.61105
 finished parsing run0000_0000 timepoint 0.6111
 finished parsing run0000_0000 timepoint 0.61115
 finished parsing run0000_0000 timepoint 0.6112
 finished parsing run0000_0000 timepoint 0.61125
 finished parsing run0000_0000 timepoint 0.6113
 finished parsing run0000_0000 timepoint 0.61135
 finished parsing run0000_0000 timepoint 0.6114
 finished parsing run0000_0000 timepoint 0.61145
 finished parsing run0000_0000 timepoint 0.6115
 finished parsing run0000_0000 timepoint 0.61155
 finished parsing run0000_0000 timepoint 0.6116
 finished parsing run0000_0000 timepoint 0.61165
 finished parsing run0000_0000 timepoint 0.6117
 finished parsing run0000_0000 timepoint 0.61175
 finished parsing run0000_0000 timepoint 0.6118
 finished parsing run0000_0000 timepoint 0.61185
 finished parsing run0000_0000 timepoint 0.6119
 finished parsing run0000_0000 timepoint 0.61195
 finished parsing run0000_0000 timepoint 0.612
 finished parsing run0000_0000 timepoint 0.61205
 finished parsing run0000_0000 timepoint 0.6121
 finished parsing run0000_0000 timepoint 0.61215
 finished parsing run0000_0000 timepoint 0.6122
 finished parsing run0000_0000 timepoint 0.61225
 finished parsing run0000_0000 timepoint 0.6123
 finished parsing run0000_0000 timepoint 0.61235
 finished parsing run0000_0000 timepoint 0.6124
 finished parsing run0000_0000 timepoint 0.61245
 finished parsing run0000_0000 timepoint 0.6125
 finished parsing run0000_0000 timepoint 0.61255
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.59945
 finished parsing run0000_0000 timepoint 0.5995
 finished parsing run0000_0000 timepoint 0.59955
 finished parsing run0000_0000 timepoint 0.5996
 finished parsing run0000_0000 timepoint 0.59965
 finished parsing run0000_0000 timepoint 0.5997
 finished parsing run0000_0000 timepoint 0.59975
 finished parsing run0000_0000 timepoint 0.5998
 finished parsing run0000_0000 timepoint 0.59985
 finished parsing run0000_0000 timepoint 0.5999
 finished parsing run0000_0000 timepoint 0.59995
 finished parsing run0000_0000 timepoint 0.6
 finished parsing run0000_0000 timepoint 0.60005
 finished parsing run0000_0000 timepoint 0.6001
 finished parsing run0000_0000 timepoint 0.60015
 finished parsing run0000_0000 timepoint 0.6002
 finished parsing run0000_0000 timepoint 0.60025
 finished parsing run0000_0000 timepoint 0.6003
 finished parsing run0000_0000 timepoint 0.60035
 finished parsing run0000_0000 timepoint 0.6004
 finished parsing run0000_0000 timepoint 0.60045
 finished parsing run0000_0000 timepoint 0.6005
 finished parsing run0000_0000 timepoint 0.60055
 finished parsing run0000_0000 timepoint 0.6006
 finished parsing run0000_0000 timepoint 0.60065
 finished parsing run0000_0000 timepoint 0.6007
 finished parsing run0000_0000 timepoint 0.60075
 finished parsing run0000_0000 timepoint 0.6008
 finished parsing run0000_0000 timepoint 0.60085
 finished parsing run0000_0000 timepoint 0.6009
 finished parsing run0000_0000 timepoint 0.60095
 finished parsing run0000_0000 timepoint 0.601
 finished parsing run0000_0000 timepoint 0.60105
 finished parsing run0000_0000 timepoint 0.6011
 finished parsing run0000_0000 timepoint 0.60115
 finished parsing run0000_0000 timepoint 0.6012
 finished parsing run0000_0000 timepoint 0.60125
 finished parsing run0000_0000 timepoint 0.6013
 finished parsing run0000_0000 timepoint 0.60135
 finished parsing run0000_0000 timepoint 0.6014
 finished parsing run0000_0000 timepoint 0.60145
 finished parsing run0000_0000 timepoint 0.6015
 finished parsing run0000_0000 timepoint 0.60155
 finished parsing run0000_0000 timepoint 0.6016
 finished parsing run0000_0000 timepoint 0.60165
 finished parsing run0000_0000 timepoint 0.6017
 finished parsing run0000_0000 timepoint 0.60175
 finished parsing run0000_0000 timepoint 0.6018
 finished parsing run0000_0000 timepoint 0.60185
 finished parsing run0000_0000 timepoint 0.6019
 finished parsing run0000_0000 timepoint 0.60195
 finished parsing run0000_0000 timepoint 0.602
 finished parsing run0000_0000 timepoint 0.60205
 finished parsing run0000_0000 timepoint 0.6021
 finished parsing run0000_0000 timepoint 0.60215
 finished parsing run0000_0000 timepoint 0.6022
 finished parsing run0000_0000 timepoint 0.60225
 finished parsing run0000_0000 timepoint 0.6023
 finished parsing run0000_0000 timepoint 0.60235
 finished parsing run0000_0000 timepoint 0.6024
 finished parsing run0000_0000 timepoint 0.60245
 finished parsing run0000_0000 timepoint 0.6025
 finished parsing run0000_0000 timepoint 0.60255
 finished parsing run0000_0000 timepoint 0.6026
 finished parsing run0000_0000 timepoint 0.60265
 finished parsing run0000_0000 timepoint 0.6027
 finished parsing run0000_0000 timepoint 0.60275
 finished parsing run0000_0000 timepoint 0.6028
 finished parsing run0000_0000 timepoint 0.60285
 finished parsing run0000_0000 timepoint 0.6029
 finished parsing run0000_0000 timepoint 0.60295
 finished parsing run0000_0000 timepoint 0.603
 finished parsing run0000_0000 timepoint 0.60305
 finished parsing run0000_0000 timepoint 0.6031
 finished parsing run0000_0000 timepoint 0.60315
 finished parsing run0000_0000 timepoint 0.6032
 finished parsing run0000_0000 timepoint 0.60325
 finished parsing run0000_0000 timepoint 0.6033
 finished parsing run0000_0000 timepoint 0.60335
 finished parsing run0000_0000 timepoint 0.6034
 finished parsing run0000_0000 timepoint 0.60345
 finished parsing run0000_0000 timepoint 0.6035
 finished parsing run0000_0000 timepoint 0.60355
 finished parsing run0000_0000 timepoint 0.6036
 finished parsing run0000_0000 timepoint 0.60365
 finished parsing run0000_0000 timepoint 0.6037
 finished parsing run0000_0000 timepoint 0.60375
 finished parsing run0000_0000 timepoint 0.6038
 finished parsing run0000_0000 timepoint 0.60385
 finished parsing run0000_0000 timepoint 0.6039
 finished parsing run0000_0000 timepoint 0.60395
 finished parsing run0000_0000 timepoint 0.604
 finished parsing run0000_0000 timepoint 0.60405
 finished parsing run0000_0000 timepoint 0.6041
 finished parsing run0000_0000 timepoint 0.60415
 finished parsing run0000_0000 timepoint 0.6042
 finished parsing run0000_0000 timepoint 0.60425
 finished parsing run0000_0000 timepoint 0.6043
 finished parsing run0000_0000 timepoint 0.60435
 finished parsing run0000_0000 timepoint 0.6044
 finished parsing run0000_0000 timepoint 0.60445
 finished parsing run0000_0000 timepoint 0.6045
 finished parsing run0000_0000 timepoint 0.60455
 finished parsing run0000_0000 timepoint 0.6046
 finished parsing run0000_0000 timepoint 0.60465
 finished parsing run0000_0000 timepoint 0.6047
 finished parsing run0000_0000 timepoint 0.60475
 finished parsing run0000_0000 timepoint 0.6048
 finished parsing run0000_0000 timepoint 0.60485
 finished parsing run0000_0000 timepoint 0.6049
 finished parsing run0000_0000 timepoint 0.60495
 finished parsing run0000_0000 timepoint 0.605
 finished parsing run0000_0000 timepoint 0.60505
 finished parsing run0000_0000 timepoint 0.6051
 finished parsing run0000_0000 timepoint 0.60515
 finished parsing run0000_0000 timepoint 0.6052
 finished parsing run0000_0000 timepoint 0.60525
 finished parsing run0000_0000 timepoint 0.6053
 finished parsing run0000_0000 timepoint 0.60535
 finished parsing run0000_0000 timepoint 0.6054
 finished parsing run0000_0000 timepoint 0.60545
 finished parsing run0000_0000 timepoint 0.6055
 finished parsing run0000_0000 timepoint 0.60555
 finished parsing run0000_0000 timepoint 0.6056
 finished parsing run0000_0000 timepoint 0.60565
 finished parsing run0000_0000 timepoint 0.6057
 finished parsing run0000_0000 timepoint 0.60575
 finished parsing run0000_0000 timepoint 0.6058
 finished parsing run0000_0000 timepoint 0.60585
 finished parsing run0000_0000 timepoint 0.6059
 finished parsing run0000_0000 timepoint 0.60595
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.59295
 finished parsing run0000_0000 timepoint 0.593
 finished parsing run0000_0000 timepoint 0.59305
 finished parsing run0000_0000 timepoint 0.5931
 finished parsing run0000_0000 timepoint 0.59315
 finished parsing run0000_0000 timepoint 0.5932
 finished parsing run0000_0000 timepoint 0.59325
 finished parsing run0000_0000 timepoint 0.5933
 finished parsing run0000_0000 timepoint 0.59335
 finished parsing run0000_0000 timepoint 0.5934
 finished parsing run0000_0000 timepoint 0.59345
 finished parsing run0000_0000 timepoint 0.5935
 finished parsing run0000_0000 timepoint 0.59355
 finished parsing run0000_0000 timepoint 0.5936
 finished parsing run0000_0000 timepoint 0.59365
 finished parsing run0000_0000 timepoint 0.5937
 finished parsing run0000_0000 timepoint 0.59375
 finished parsing run0000_0000 timepoint 0.5938
 finished parsing run0000_0000 timepoint 0.59385
 finished parsing run0000_0000 timepoint 0.5939
 finished parsing run0000_0000 timepoint 0.59395
 finished parsing run0000_0000 timepoint 0.594
 finished parsing run0000_0000 timepoint 0.59405
 finished parsing run0000_0000 timepoint 0.5941
 finished parsing run0000_0000 timepoint 0.59415
 finished parsing run0000_0000 timepoint 0.5942
 finished parsing run0000_0000 timepoint 0.59425
 finished parsing run0000_0000 timepoint 0.5943
 finished parsing run0000_0000 timepoint 0.59435
 finished parsing run0000_0000 timepoint 0.5944
 finished parsing run0000_0000 timepoint 0.59445
 finished parsing run0000_0000 timepoint 0.5945
 finished parsing run0000_0000 timepoint 0.59455
 finished parsing run0000_0000 timepoint 0.5946
 finished parsing run0000_0000 timepoint 0.59465
 finished parsing run0000_0000 timepoint 0.5947
 finished parsing run0000_0000 timepoint 0.59475
 finished parsing run0000_0000 timepoint 0.5948
 finished parsing run0000_0000 timepoint 0.59485
 finished parsing run0000_0000 timepoint 0.5949
 finished parsing run0000_0000 timepoint 0.59495
 finished parsing run0000_0000 timepoint 0.595
 finished parsing run0000_0000 timepoint 0.59505
 finished parsing run0000_0000 timepoint 0.5951
 finished parsing run0000_0000 timepoint 0.59515
 finished parsing run0000_0000 timepoint 0.5952
 finished parsing run0000_0000 timepoint 0.59525
 finished parsing run0000_0000 timepoint 0.5953
 finished parsing run0000_0000 timepoint 0.59535
 finished parsing run0000_0000 timepoint 0.5954
 finished parsing run0000_0000 timepoint 0.59545
 finished parsing run0000_0000 timepoint 0.5955
 finished parsing run0000_0000 timepoint 0.59555
 finished parsing run0000_0000 timepoint 0.5956
 finished parsing run0000_0000 timepoint 0.59565
 finished parsing run0000_0000 timepoint 0.5957
 finished parsing run0000_0000 timepoint 0.59575
 finished parsing run0000_0000 timepoint 0.5958
 finished parsing run0000_0000 timepoint 0.59585
 finished parsing run0000_0000 timepoint 0.5959
 finished parsing run0000_0000 timepoint 0.59595
 finished parsing run0000_0000 timepoint 0.596
 finished parsing run0000_0000 timepoint 0.59605
 finished parsing run0000_0000 timepoint 0.5961
 finished parsing run0000_0000 timepoint 0.59615
 finished parsing run0000_0000 timepoint 0.5962
 finished parsing run0000_0000 timepoint 0.59625
 finished parsing run0000_0000 timepoint 0.5963
 finished parsing run0000_0000 timepoint 0.59635
 finished parsing run0000_0000 timepoint 0.5964
 finished parsing run0000_0000 timepoint 0.59645
 finished parsing run0000_0000 timepoint 0.5965
 finished parsing run0000_0000 timepoint 0.59655
 finished parsing run0000_0000 timepoint 0.5966
 finished parsing run0000_0000 timepoint 0.59665
 finished parsing run0000_0000 timepoint 0.5967
 finished parsing run0000_0000 timepoint 0.59675
 finished parsing run0000_0000 timepoint 0.5968
 finished parsing run0000_0000 timepoint 0.59685
 finished parsing run0000_0000 timepoint 0.5969
 finished parsing run0000_0000 timepoint 0.59695
 finished parsing run0000_0000 timepoint 0.597
 finished parsing run0000_0000 timepoint 0.59705
 finished parsing run0000_0000 timepoint 0.5971
 finished parsing run0000_0000 timepoint 0.59715
 finished parsing run0000_0000 timepoint 0.5972
 finished parsing run0000_0000 timepoint 0.59725
 finished parsing run0000_0000 timepoint 0.5973
 finished parsing run0000_0000 timepoint 0.59735
 finished parsing run0000_0000 timepoint 0.5974
 finished parsing run0000_0000 timepoint 0.59745
 finished parsing run0000_0000 timepoint 0.5975
 finished parsing run0000_0000 timepoint 0.59755
 finished parsing run0000_0000 timepoint 0.5976
 finished parsing run0000_0000 timepoint 0.59765
 finished parsing run0000_0000 timepoint 0.5977
 finished parsing run0000_0000 timepoint 0.59775
 finished parsing run0000_0000 timepoint 0.5978
 finished parsing run0000_0000 timepoint 0.59785
 finished parsing run0000_0000 timepoint 0.5979
 finished parsing run0000_0000 timepoint 0.59795
 finished parsing run0000_0000 timepoint 0.598
 finished parsing run0000_0000 timepoint 0.59805
 finished parsing run0000_0000 timepoint 0.5981
 finished parsing run0000_0000 timepoint 0.59815
 finished parsing run0000_0000 timepoint 0.5982
 finished parsing run0000_0000 timepoint 0.59825
 finished parsing run0000_0000 timepoint 0.5983
 finished parsing run0000_0000 timepoint 0.59835
 finished parsing run0000_0000 timepoint 0.5984
 finished parsing run0000_0000 timepoint 0.59845
 finished parsing run0000_0000 timepoint 0.5985
 finished parsing run0000_0000 timepoint 0.59855
 finished parsing run0000_0000 timepoint 0.5986
 finished parsing run0000_0000 timepoint 0.59865
 finished parsing run0000_0000 timepoint 0.5987
 finished parsing run0000_0000 timepoint 0.59875
 finished parsing run0000_0000 timepoint 0.5988
 finished parsing run0000_0000 timepoint 0.59885
 finished parsing run0000_0000 timepoint 0.5989
 finished parsing run0000_0000 timepoint 0.59895
 finished parsing run0000_0000 timepoint 0.599
 finished parsing run0000_0000 timepoint 0.59905
 finished parsing run0000_0000 timepoint 0.5991
 finished parsing run0000_0000 timepoint 0.59915
 finished parsing run0000_0000 timepoint 0.5992
 finished parsing run0000_0000 timepoint 0.59925
 finished parsing run0000_0000 timepoint 0.5993
 finished parsing run0000_0000 timepoint 0.59935
 finished parsing run0000_0000 timepoint 0.5994
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5863
 finished parsing run0000_0000 timepoint 0.58635
 finished parsing run0000_0000 timepoint 0.5864
 finished parsing run0000_0000 timepoint 0.58645
 finished parsing run0000_0000 timepoint 0.5865
 finished parsing run0000_0000 timepoint 0.58655
 finished parsing run0000_0000 timepoint 0.5866
 finished parsing run0000_0000 timepoint 0.58665
 finished parsing run0000_0000 timepoint 0.5867
 finished parsing run0000_0000 timepoint 0.58675
 finished parsing run0000_0000 timepoint 0.5868
 finished parsing run0000_0000 timepoint 0.58685
 finished parsing run0000_0000 timepoint 0.5869
 finished parsing run0000_0000 timepoint 0.58695
 finished parsing run0000_0000 timepoint 0.587
 finished parsing run0000_0000 timepoint 0.58705
 finished parsing run0000_0000 timepoint 0.5871
 finished parsing run0000_0000 timepoint 0.58715
 finished parsing run0000_0000 timepoint 0.5872
 finished parsing run0000_0000 timepoint 0.58725
 finished parsing run0000_0000 timepoint 0.5873
 finished parsing run0000_0000 timepoint 0.58735
 finished parsing run0000_0000 timepoint 0.5874
 finished parsing run0000_0000 timepoint 0.58745
 finished parsing run0000_0000 timepoint 0.5875
 finished parsing run0000_0000 timepoint 0.58755
 finished parsing run0000_0000 timepoint 0.5876
 finished parsing run0000_0000 timepoint 0.58765
 finished parsing run0000_0000 timepoint 0.5877
 finished parsing run0000_0000 timepoint 0.58775
 finished parsing run0000_0000 timepoint 0.5878
 finished parsing run0000_0000 timepoint 0.58785
 finished parsing run0000_0000 timepoint 0.5879
 finished parsing run0000_0000 timepoint 0.58795
 finished parsing run0000_0000 timepoint 0.588
 finished parsing run0000_0000 timepoint 0.58805
 finished parsing run0000_0000 timepoint 0.5881
 finished parsing run0000_0000 timepoint 0.58815
 finished parsing run0000_0000 timepoint 0.5882
 finished parsing run0000_0000 timepoint 0.58825
 finished parsing run0000_0000 timepoint 0.5883
 finished parsing run0000_0000 timepoint 0.58835
 finished parsing run0000_0000 timepoint 0.5884
 finished parsing run0000_0000 timepoint 0.58845
 finished parsing run0000_0000 timepoint 0.5885
 finished parsing run0000_0000 timepoint 0.58855
 finished parsing run0000_0000 timepoint 0.5886
 finished parsing run0000_0000 timepoint 0.58865
 finished parsing run0000_0000 timepoint 0.5887
 finished parsing run0000_0000 timepoint 0.58875
 finished parsing run0000_0000 timepoint 0.5888
 finished parsing run0000_0000 timepoint 0.58885
 finished parsing run0000_0000 timepoint 0.5889
 finished parsing run0000_0000 timepoint 0.58895
 finished parsing run0000_0000 timepoint 0.589
 finished parsing run0000_0000 timepoint 0.58905
 finished parsing run0000_0000 timepoint 0.5891
 finished parsing run0000_0000 timepoint 0.58915
 finished parsing run0000_0000 timepoint 0.5892
 finished parsing run0000_0000 timepoint 0.58925
 finished parsing run0000_0000 timepoint 0.5893
 finished parsing run0000_0000 timepoint 0.58935
 finished parsing run0000_0000 timepoint 0.5894
 finished parsing run0000_0000 timepoint 0.58945
 finished parsing run0000_0000 timepoint 0.5895
 finished parsing run0000_0000 timepoint 0.58955
 finished parsing run0000_0000 timepoint 0.5896
 finished parsing run0000_0000 timepoint 0.58965
 finished parsing run0000_0000 timepoint 0.5897
 finished parsing run0000_0000 timepoint 0.58975
 finished parsing run0000_0000 timepoint 0.5898
 finished parsing run0000_0000 timepoint 0.58985
 finished parsing run0000_0000 timepoint 0.5899
 finished parsing run0000_0000 timepoint 0.58995
 finished parsing run0000_0000 timepoint 0.59
 finished parsing run0000_0000 timepoint 0.59005
 finished parsing run0000_0000 timepoint 0.5901
 finished parsing run0000_0000 timepoint 0.59015
 finished parsing run0000_0000 timepoint 0.5902
 finished parsing run0000_0000 timepoint 0.59025
 finished parsing run0000_0000 timepoint 0.5903
 finished parsing run0000_0000 timepoint 0.59035
 finished parsing run0000_0000 timepoint 0.5904
 finished parsing run0000_0000 timepoint 0.59045
 finished parsing run0000_0000 timepoint 0.5905
 finished parsing run0000_0000 timepoint 0.59055
 finished parsing run0000_0000 timepoint 0.5906
 finished parsing run0000_0000 timepoint 0.59065
 finished parsing run0000_0000 timepoint 0.5907
 finished parsing run0000_0000 timepoint 0.59075
 finished parsing run0000_0000 timepoint 0.5908
 finished parsing run0000_0000 timepoint 0.59085
 finished parsing run0000_0000 timepoint 0.5909
 finished parsing run0000_0000 timepoint 0.59095
 finished parsing run0000_0000 timepoint 0.591
 finished parsing run0000_0000 timepoint 0.59105
 finished parsing run0000_0000 timepoint 0.5911
 finished parsing run0000_0000 timepoint 0.59115
 finished parsing run0000_0000 timepoint 0.5912
 finished parsing run0000_0000 timepoint 0.59125
 finished parsing run0000_0000 timepoint 0.5913
 finished parsing run0000_0000 timepoint 0.59135
 finished parsing run0000_0000 timepoint 0.5914
 finished parsing run0000_0000 timepoint 0.59145
 finished parsing run0000_0000 timepoint 0.5915
 finished parsing run0000_0000 timepoint 0.59155
 finished parsing run0000_0000 timepoint 0.5916
 finished parsing run0000_0000 timepoint 0.59165
 finished parsing run0000_0000 timepoint 0.5917
 finished parsing run0000_0000 timepoint 0.59175
 finished parsing run0000_0000 timepoint 0.5918
 finished parsing run0000_0000 timepoint 0.59185
 finished parsing run0000_0000 timepoint 0.5919
 finished parsing run0000_0000 timepoint 0.59195
 finished parsing run0000_0000 timepoint 0.592
 finished parsing run0000_0000 timepoint 0.59205
 finished parsing run0000_0000 timepoint 0.5921
 finished parsing run0000_0000 timepoint 0.59215
 finished parsing run0000_0000 timepoint 0.5922
 finished parsing run0000_0000 timepoint 0.59225
 finished parsing run0000_0000 timepoint 0.5923
 finished parsing run0000_0000 timepoint 0.59235
 finished parsing run0000_0000 timepoint 0.5924
 finished parsing run0000_0000 timepoint 0.59245
 finished parsing run0000_0000 timepoint 0.5925
 finished parsing run0000_0000 timepoint 0.59255
 finished parsing run0000_0000 timepoint 0.5926
 finished parsing run0000_0000 timepoint 0.59265
 finished parsing run0000_0000 timepoint 0.5927
 finished parsing run0000_0000 timepoint 0.59275
 finished parsing run0000_0000 timepoint 0.5928
 finished parsing run0000_0000 timepoint 0.59285
 finished parsing run0000_0000 timepoint 0.5929
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.57975
 finished parsing run0000_0000 timepoint 0.5798
 finished parsing run0000_0000 timepoint 0.57985
 finished parsing run0000_0000 timepoint 0.5799
 finished parsing run0000_0000 timepoint 0.57995
 finished parsing run0000_0000 timepoint 0.58
 finished parsing run0000_0000 timepoint 0.58005
 finished parsing run0000_0000 timepoint 0.5801
 finished parsing run0000_0000 timepoint 0.58015
 finished parsing run0000_0000 timepoint 0.5802
 finished parsing run0000_0000 timepoint 0.58025
 finished parsing run0000_0000 timepoint 0.5803
 finished parsing run0000_0000 timepoint 0.58035
 finished parsing run0000_0000 timepoint 0.5804
 finished parsing run0000_0000 timepoint 0.58045
 finished parsing run0000_0000 timepoint 0.5805
 finished parsing run0000_0000 timepoint 0.58055
 finished parsing run0000_0000 timepoint 0.5806
 finished parsing run0000_0000 timepoint 0.58065
 finished parsing run0000_0000 timepoint 0.5807
 finished parsing run0000_0000 timepoint 0.58075
 finished parsing run0000_0000 timepoint 0.5808
 finished parsing run0000_0000 timepoint 0.58085
 finished parsing run0000_0000 timepoint 0.5809
 finished parsing run0000_0000 timepoint 0.58095
 finished parsing run0000_0000 timepoint 0.581
 finished parsing run0000_0000 timepoint 0.58105
 finished parsing run0000_0000 timepoint 0.5811
 finished parsing run0000_0000 timepoint 0.58115
 finished parsing run0000_0000 timepoint 0.5812
 finished parsing run0000_0000 timepoint 0.58125
 finished parsing run0000_0000 timepoint 0.5813
 finished parsing run0000_0000 timepoint 0.58135
 finished parsing run0000_0000 timepoint 0.5814
 finished parsing run0000_0000 timepoint 0.58145
 finished parsing run0000_0000 timepoint 0.5815
 finished parsing run0000_0000 timepoint 0.58155
 finished parsing run0000_0000 timepoint 0.5816
 finished parsing run0000_0000 timepoint 0.58165
 finished parsing run0000_0000 timepoint 0.5817
 finished parsing run0000_0000 timepoint 0.58175
 finished parsing run0000_0000 timepoint 0.5818
 finished parsing run0000_0000 timepoint 0.58185
 finished parsing run0000_0000 timepoint 0.5819
 finished parsing run0000_0000 timepoint 0.58195
 finished parsing run0000_0000 timepoint 0.582
 finished parsing run0000_0000 timepoint 0.58205
 finished parsing run0000_0000 timepoint 0.5821
 finished parsing run0000_0000 timepoint 0.58215
 finished parsing run0000_0000 timepoint 0.5822
 finished parsing run0000_0000 timepoint 0.58225
 finished parsing run0000_0000 timepoint 0.5823
 finished parsing run0000_0000 timepoint 0.58235
 finished parsing run0000_0000 timepoint 0.5824
 finished parsing run0000_0000 timepoint 0.58245
 finished parsing run0000_0000 timepoint 0.5825
 finished parsing run0000_0000 timepoint 0.58255
 finished parsing run0000_0000 timepoint 0.5826
 finished parsing run0000_0000 timepoint 0.58265
 finished parsing run0000_0000 timepoint 0.5827
 finished parsing run0000_0000 timepoint 0.58275
 finished parsing run0000_0000 timepoint 0.5828
 finished parsing run0000_0000 timepoint 0.58285
 finished parsing run0000_0000 timepoint 0.5829
 finished parsing run0000_0000 timepoint 0.58295
 finished parsing run0000_0000 timepoint 0.583
 finished parsing run0000_0000 timepoint 0.58305
 finished parsing run0000_0000 timepoint 0.5831
 finished parsing run0000_0000 timepoint 0.58315
 finished parsing run0000_0000 timepoint 0.5832
 finished parsing run0000_0000 timepoint 0.58325
 finished parsing run0000_0000 timepoint 0.5833
 finished parsing run0000_0000 timepoint 0.58335
 finished parsing run0000_0000 timepoint 0.5834
 finished parsing run0000_0000 timepoint 0.58345
 finished parsing run0000_0000 timepoint 0.5835
 finished parsing run0000_0000 timepoint 0.58355
 finished parsing run0000_0000 timepoint 0.5836
 finished parsing run0000_0000 timepoint 0.58365
 finished parsing run0000_0000 timepoint 0.5837
 finished parsing run0000_0000 timepoint 0.58375
 finished parsing run0000_0000 timepoint 0.5838
 finished parsing run0000_0000 timepoint 0.58385
 finished parsing run0000_0000 timepoint 0.5839
 finished parsing run0000_0000 timepoint 0.58395
 finished parsing run0000_0000 timepoint 0.584
 finished parsing run0000_0000 timepoint 0.58405
 finished parsing run0000_0000 timepoint 0.5841
 finished parsing run0000_0000 timepoint 0.58415
 finished parsing run0000_0000 timepoint 0.5842
 finished parsing run0000_0000 timepoint 0.58425
 finished parsing run0000_0000 timepoint 0.5843
 finished parsing run0000_0000 timepoint 0.58435
 finished parsing run0000_0000 timepoint 0.5844
 finished parsing run0000_0000 timepoint 0.58445
 finished parsing run0000_0000 timepoint 0.5845
 finished parsing run0000_0000 timepoint 0.58455
 finished parsing run0000_0000 timepoint 0.5846
 finished parsing run0000_0000 timepoint 0.58465
 finished parsing run0000_0000 timepoint 0.5847
 finished parsing run0000_0000 timepoint 0.58475
 finished parsing run0000_0000 timepoint 0.5848
 finished parsing run0000_0000 timepoint 0.58485
 finished parsing run0000_0000 timepoint 0.5849
 finished parsing run0000_0000 timepoint 0.58495
 finished parsing run0000_0000 timepoint 0.585
 finished parsing run0000_0000 timepoint 0.58505
 finished parsing run0000_0000 timepoint 0.5851
 finished parsing run0000_0000 timepoint 0.58515
 finished parsing run0000_0000 timepoint 0.5852
 finished parsing run0000_0000 timepoint 0.58525
 finished parsing run0000_0000 timepoint 0.5853
 finished parsing run0000_0000 timepoint 0.58535
 finished parsing run0000_0000 timepoint 0.5854
 finished parsing run0000_0000 timepoint 0.58545
 finished parsing run0000_0000 timepoint 0.5855
 finished parsing run0000_0000 timepoint 0.58555
 finished parsing run0000_0000 timepoint 0.5856
 finished parsing run0000_0000 timepoint 0.58565
 finished parsing run0000_0000 timepoint 0.5857
 finished parsing run0000_0000 timepoint 0.58575
 finished parsing run0000_0000 timepoint 0.5858
 finished parsing run0000_0000 timepoint 0.58585
 finished parsing run0000_0000 timepoint 0.5859
 finished parsing run0000_0000 timepoint 0.58595
 finished parsing run0000_0000 timepoint 0.586
 finished parsing run0000_0000 timepoint 0.58605
 finished parsing run0000_0000 timepoint 0.5861
 finished parsing run0000_0000 timepoint 0.58615
 finished parsing run0000_0000 timepoint 0.5862
 finished parsing run0000_0000 timepoint 0.58625
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5732
 finished parsing run0000_0000 timepoint 0.57325
 finished parsing run0000_0000 timepoint 0.5733
 finished parsing run0000_0000 timepoint 0.57335
 finished parsing run0000_0000 timepoint 0.5734
 finished parsing run0000_0000 timepoint 0.57345
 finished parsing run0000_0000 timepoint 0.5735
 finished parsing run0000_0000 timepoint 0.57355
 finished parsing run0000_0000 timepoint 0.5736
 finished parsing run0000_0000 timepoint 0.57365
 finished parsing run0000_0000 timepoint 0.5737
 finished parsing run0000_0000 timepoint 0.57375
 finished parsing run0000_0000 timepoint 0.5738
 finished parsing run0000_0000 timepoint 0.57385
 finished parsing run0000_0000 timepoint 0.5739
 finished parsing run0000_0000 timepoint 0.57395
 finished parsing run0000_0000 timepoint 0.574
 finished parsing run0000_0000 timepoint 0.57405
 finished parsing run0000_0000 timepoint 0.5741
 finished parsing run0000_0000 timepoint 0.57415
 finished parsing run0000_0000 timepoint 0.5742
 finished parsing run0000_0000 timepoint 0.57425
 finished parsing run0000_0000 timepoint 0.5743
 finished parsing run0000_0000 timepoint 0.57435
 finished parsing run0000_0000 timepoint 0.5744
 finished parsing run0000_0000 timepoint 0.57445
 finished parsing run0000_0000 timepoint 0.5745
 finished parsing run0000_0000 timepoint 0.57455
 finished parsing run0000_0000 timepoint 0.5746
 finished parsing run0000_0000 timepoint 0.57465
 finished parsing run0000_0000 timepoint 0.5747
 finished parsing run0000_0000 timepoint 0.57475
 finished parsing run0000_0000 timepoint 0.5748
 finished parsing run0000_0000 timepoint 0.57485
 finished parsing run0000_0000 timepoint 0.5749
 finished parsing run0000_0000 timepoint 0.57495
 finished parsing run0000_0000 timepoint 0.575
 finished parsing run0000_0000 timepoint 0.57505
 finished parsing run0000_0000 timepoint 0.5751
 finished parsing run0000_0000 timepoint 0.57515
 finished parsing run0000_0000 timepoint 0.5752
 finished parsing run0000_0000 timepoint 0.57525
 finished parsing run0000_0000 timepoint 0.5753
 finished parsing run0000_0000 timepoint 0.57535
 finished parsing run0000_0000 timepoint 0.5754
 finished parsing run0000_0000 timepoint 0.57545
 finished parsing run0000_0000 timepoint 0.5755
 finished parsing run0000_0000 timepoint 0.57555
 finished parsing run0000_0000 timepoint 0.5756
 finished parsing run0000_0000 timepoint 0.57565
 finished parsing run0000_0000 timepoint 0.5757
 finished parsing run0000_0000 timepoint 0.57575
 finished parsing run0000_0000 timepoint 0.5758
 finished parsing run0000_0000 timepoint 0.57585
 finished parsing run0000_0000 timepoint 0.5759
 finished parsing run0000_0000 timepoint 0.57595
 finished parsing run0000_0000 timepoint 0.576
 finished parsing run0000_0000 timepoint 0.57605
 finished parsing run0000_0000 timepoint 0.5761
 finished parsing run0000_0000 timepoint 0.57615
 finished parsing run0000_0000 timepoint 0.5762
 finished parsing run0000_0000 timepoint 0.57625
 finished parsing run0000_0000 timepoint 0.5763
 finished parsing run0000_0000 timepoint 0.57635
 finished parsing run0000_0000 timepoint 0.5764
 finished parsing run0000_0000 timepoint 0.57645
 finished parsing run0000_0000 timepoint 0.5765
 finished parsing run0000_0000 timepoint 0.57655
 finished parsing run0000_0000 timepoint 0.5766
 finished parsing run0000_0000 timepoint 0.57665
 finished parsing run0000_0000 timepoint 0.5767
 finished parsing run0000_0000 timepoint 0.57675
 finished parsing run0000_0000 timepoint 0.5768
 finished parsing run0000_0000 timepoint 0.57685
 finished parsing run0000_0000 timepoint 0.5769
 finished parsing run0000_0000 timepoint 0.57695
 finished parsing run0000_0000 timepoint 0.577
 finished parsing run0000_0000 timepoint 0.57705
 finished parsing run0000_0000 timepoint 0.5771
 finished parsing run0000_0000 timepoint 0.57715
 finished parsing run0000_0000 timepoint 0.5772
 finished parsing run0000_0000 timepoint 0.57725
 finished parsing run0000_0000 timepoint 0.5773
 finished parsing run0000_0000 timepoint 0.57735
 finished parsing run0000_0000 timepoint 0.5774
 finished parsing run0000_0000 timepoint 0.57745
 finished parsing run0000_0000 timepoint 0.5775
 finished parsing run0000_0000 timepoint 0.57755
 finished parsing run0000_0000 timepoint 0.5776
 finished parsing run0000_0000 timepoint 0.57765
 finished parsing run0000_0000 timepoint 0.5777
 finished parsing run0000_0000 timepoint 0.57775
 finished parsing run0000_0000 timepoint 0.5778
 finished parsing run0000_0000 timepoint 0.57785
 finished parsing run0000_0000 timepoint 0.5779
 finished parsing run0000_0000 timepoint 0.57795
 finished parsing run0000_0000 timepoint 0.578
 finished parsing run0000_0000 timepoint 0.57805
 finished parsing run0000_0000 timepoint 0.5781
 finished parsing run0000_0000 timepoint 0.57815
 finished parsing run0000_0000 timepoint 0.5782
 finished parsing run0000_0000 timepoint 0.57825
 finished parsing run0000_0000 timepoint 0.5783
 finished parsing run0000_0000 timepoint 0.57835
 finished parsing run0000_0000 timepoint 0.5784
 finished parsing run0000_0000 timepoint 0.57845
 finished parsing run0000_0000 timepoint 0.5785
 finished parsing run0000_0000 timepoint 0.57855
 finished parsing run0000_0000 timepoint 0.5786
 finished parsing run0000_0000 timepoint 0.57865
 finished parsing run0000_0000 timepoint 0.5787
 finished parsing run0000_0000 timepoint 0.57875
 finished parsing run0000_0000 timepoint 0.5788
 finished parsing run0000_0000 timepoint 0.57885
 finished parsing run0000_0000 timepoint 0.5789
 finished parsing run0000_0000 timepoint 0.57895
 finished parsing run0000_0000 timepoint 0.579
 finished parsing run0000_0000 timepoint 0.57905
 finished parsing run0000_0000 timepoint 0.5791
 finished parsing run0000_0000 timepoint 0.57915
 finished parsing run0000_0000 timepoint 0.5792
 finished parsing run0000_0000 timepoint 0.57925
 finished parsing run0000_0000 timepoint 0.5793
 finished parsing run0000_0000 timepoint 0.57935
 finished parsing run0000_0000 timepoint 0.5794
 finished parsing run0000_0000 timepoint 0.57945
 finished parsing run0000_0000 timepoint 0.5795
 finished parsing run0000_0000 timepoint 0.57955
 finished parsing run0000_0000 timepoint 0.5796
 finished parsing run0000_0000 timepoint 0.57965
 finished parsing run0000_0000 timepoint 0.5797
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5667
 finished parsing run0000_0000 timepoint 0.56675
 finished parsing run0000_0000 timepoint 0.5668
 finished parsing run0000_0000 timepoint 0.56685
 finished parsing run0000_0000 timepoint 0.5669
 finished parsing run0000_0000 timepoint 0.56695
 finished parsing run0000_0000 timepoint 0.567
 finished parsing run0000_0000 timepoint 0.56705
 finished parsing run0000_0000 timepoint 0.5671
 finished parsing run0000_0000 timepoint 0.56715
 finished parsing run0000_0000 timepoint 0.5672
 finished parsing run0000_0000 timepoint 0.56725
 finished parsing run0000_0000 timepoint 0.5673
 finished parsing run0000_0000 timepoint 0.56735
 finished parsing run0000_0000 timepoint 0.5674
 finished parsing run0000_0000 timepoint 0.56745
 finished parsing run0000_0000 timepoint 0.5675
 finished parsing run0000_0000 timepoint 0.56755
 finished parsing run0000_0000 timepoint 0.5676
 finished parsing run0000_0000 timepoint 0.56765
 finished parsing run0000_0000 timepoint 0.5677
 finished parsing run0000_0000 timepoint 0.56775
 finished parsing run0000_0000 timepoint 0.5678
 finished parsing run0000_0000 timepoint 0.56785
 finished parsing run0000_0000 timepoint 0.5679
 finished parsing run0000_0000 timepoint 0.56795
 finished parsing run0000_0000 timepoint 0.568
 finished parsing run0000_0000 timepoint 0.56805
 finished parsing run0000_0000 timepoint 0.5681
 finished parsing run0000_0000 timepoint 0.56815
 finished parsing run0000_0000 timepoint 0.5682
 finished parsing run0000_0000 timepoint 0.56825
 finished parsing run0000_0000 timepoint 0.5683
 finished parsing run0000_0000 timepoint 0.56835
 finished parsing run0000_0000 timepoint 0.5684
 finished parsing run0000_0000 timepoint 0.56845
 finished parsing run0000_0000 timepoint 0.5685
 finished parsing run0000_0000 timepoint 0.56855
 finished parsing run0000_0000 timepoint 0.5686
 finished parsing run0000_0000 timepoint 0.56865
 finished parsing run0000_0000 timepoint 0.5687
 finished parsing run0000_0000 timepoint 0.56875
 finished parsing run0000_0000 timepoint 0.5688
 finished parsing run0000_0000 timepoint 0.56885
 finished parsing run0000_0000 timepoint 0.5689
 finished parsing run0000_0000 timepoint 0.56895
 finished parsing run0000_0000 timepoint 0.569
 finished parsing run0000_0000 timepoint 0.56905
 finished parsing run0000_0000 timepoint 0.5691
 finished parsing run0000_0000 timepoint 0.56915
 finished parsing run0000_0000 timepoint 0.5692
 finished parsing run0000_0000 timepoint 0.56925
 finished parsing run0000_0000 timepoint 0.5693
 finished parsing run0000_0000 timepoint 0.56935
 finished parsing run0000_0000 timepoint 0.5694
 finished parsing run0000_0000 timepoint 0.56945
 finished parsing run0000_0000 timepoint 0.5695
 finished parsing run0000_0000 timepoint 0.56955
 finished parsing run0000_0000 timepoint 0.5696
 finished parsing run0000_0000 timepoint 0.56965
 finished parsing run0000_0000 timepoint 0.5697
 finished parsing run0000_0000 timepoint 0.56975
 finished parsing run0000_0000 timepoint 0.5698
 finished parsing run0000_0000 timepoint 0.56985
 finished parsing run0000_0000 timepoint 0.5699
 finished parsing run0000_0000 timepoint 0.56995
 finished parsing run0000_0000 timepoint 0.57
 finished parsing run0000_0000 timepoint 0.57005
 finished parsing run0000_0000 timepoint 0.5701
 finished parsing run0000_0000 timepoint 0.57015
 finished parsing run0000_0000 timepoint 0.5702
 finished parsing run0000_0000 timepoint 0.57025
 finished parsing run0000_0000 timepoint 0.5703
 finished parsing run0000_0000 timepoint 0.57035
 finished parsing run0000_0000 timepoint 0.5704
 finished parsing run0000_0000 timepoint 0.57045
 finished parsing run0000_0000 timepoint 0.5705
 finished parsing run0000_0000 timepoint 0.57055
 finished parsing run0000_0000 timepoint 0.5706
 finished parsing run0000_0000 timepoint 0.57065
 finished parsing run0000_0000 timepoint 0.5707
 finished parsing run0000_0000 timepoint 0.57075
 finished parsing run0000_0000 timepoint 0.5708
 finished parsing run0000_0000 timepoint 0.57085
 finished parsing run0000_0000 timepoint 0.5709
 finished parsing run0000_0000 timepoint 0.57095
 finished parsing run0000_0000 timepoint 0.571
 finished parsing run0000_0000 timepoint 0.57105
 finished parsing run0000_0000 timepoint 0.5711
 finished parsing run0000_0000 timepoint 0.57115
 finished parsing run0000_0000 timepoint 0.5712
 finished parsing run0000_0000 timepoint 0.57125
 finished parsing run0000_0000 timepoint 0.5713
 finished parsing run0000_0000 timepoint 0.57135
 finished parsing run0000_0000 timepoint 0.5714
 finished parsing run0000_0000 timepoint 0.57145
 finished parsing run0000_0000 timepoint 0.5715
 finished parsing run0000_0000 timepoint 0.57155
 finished parsing run0000_0000 timepoint 0.5716
 finished parsing run0000_0000 timepoint 0.57165
 finished parsing run0000_0000 timepoint 0.5717
 finished parsing run0000_0000 timepoint 0.57175
 finished parsing run0000_0000 timepoint 0.5718
 finished parsing run0000_0000 timepoint 0.57185
 finished parsing run0000_0000 timepoint 0.5719
 finished parsing run0000_0000 timepoint 0.57195
 finished parsing run0000_0000 timepoint 0.572
 finished parsing run0000_0000 timepoint 0.57205
 finished parsing run0000_0000 timepoint 0.5721
 finished parsing run0000_0000 timepoint 0.57215
 finished parsing run0000_0000 timepoint 0.5722
 finished parsing run0000_0000 timepoint 0.57225
 finished parsing run0000_0000 timepoint 0.5723
 finished parsing run0000_0000 timepoint 0.57235
 finished parsing run0000_0000 timepoint 0.5724
 finished parsing run0000_0000 timepoint 0.57245
 finished parsing run0000_0000 timepoint 0.5725
 finished parsing run0000_0000 timepoint 0.57255
 finished parsing run0000_0000 timepoint 0.5726
 finished parsing run0000_0000 timepoint 0.57265
 finished parsing run0000_0000 timepoint 0.5727
 finished parsing run0000_0000 timepoint 0.57275
 finished parsing run0000_0000 timepoint 0.5728
 finished parsing run0000_0000 timepoint 0.57285
 finished parsing run0000_0000 timepoint 0.5729
 finished parsing run0000_0000 timepoint 0.57295
 finished parsing run0000_0000 timepoint 0.573
 finished parsing run0000_0000 timepoint 0.57305
 finished parsing run0000_0000 timepoint 0.5731
 finished parsing run0000_0000 timepoint 0.57315
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.56035
 finished parsing run0000_0000 timepoint 0.5604
 finished parsing run0000_0000 timepoint 0.56045
 finished parsing run0000_0000 timepoint 0.5605
 finished parsing run0000_0000 timepoint 0.56055
 finished parsing run0000_0000 timepoint 0.5606
 finished parsing run0000_0000 timepoint 0.56065
 finished parsing run0000_0000 timepoint 0.5607
 finished parsing run0000_0000 timepoint 0.56075
 finished parsing run0000_0000 timepoint 0.5608
 finished parsing run0000_0000 timepoint 0.56085
 finished parsing run0000_0000 timepoint 0.5609
 finished parsing run0000_0000 timepoint 0.56095
 finished parsing run0000_0000 timepoint 0.561
 finished parsing run0000_0000 timepoint 0.56105
 finished parsing run0000_0000 timepoint 0.5611
 finished parsing run0000_0000 timepoint 0.56115
 finished parsing run0000_0000 timepoint 0.5612
 finished parsing run0000_0000 timepoint 0.56125
 finished parsing run0000_0000 timepoint 0.5613
 finished parsing run0000_0000 timepoint 0.56135
 finished parsing run0000_0000 timepoint 0.5614
 finished parsing run0000_0000 timepoint 0.56145
 finished parsing run0000_0000 timepoint 0.5615
 finished parsing run0000_0000 timepoint 0.56155
 finished parsing run0000_0000 timepoint 0.5616
 finished parsing run0000_0000 timepoint 0.56165
 finished parsing run0000_0000 timepoint 0.5617
 finished parsing run0000_0000 timepoint 0.56175
 finished parsing run0000_0000 timepoint 0.5618
 finished parsing run0000_0000 timepoint 0.56185
 finished parsing run0000_0000 timepoint 0.5619
 finished parsing run0000_0000 timepoint 0.56195
 finished parsing run0000_0000 timepoint 0.562
 finished parsing run0000_0000 timepoint 0.56205
 finished parsing run0000_0000 timepoint 0.5621
 finished parsing run0000_0000 timepoint 0.56215
 finished parsing run0000_0000 timepoint 0.5622
 finished parsing run0000_0000 timepoint 0.56225
 finished parsing run0000_0000 timepoint 0.5623
 finished parsing run0000_0000 timepoint 0.56235
 finished parsing run0000_0000 timepoint 0.5624
 finished parsing run0000_0000 timepoint 0.56245
 finished parsing run0000_0000 timepoint 0.5625
 finished parsing run0000_0000 timepoint 0.56255
 finished parsing run0000_0000 timepoint 0.5626
 finished parsing run0000_0000 timepoint 0.56265
 finished parsing run0000_0000 timepoint 0.5627
 finished parsing run0000_0000 timepoint 0.56275
 finished parsing run0000_0000 timepoint 0.5628
 finished parsing run0000_0000 timepoint 0.56285
 finished parsing run0000_0000 timepoint 0.5629
 finished parsing run0000_0000 timepoint 0.56295
 finished parsing run0000_0000 timepoint 0.563
 finished parsing run0000_0000 timepoint 0.56305
 finished parsing run0000_0000 timepoint 0.5631
 finished parsing run0000_0000 timepoint 0.56315
 finished parsing run0000_0000 timepoint 0.5632
 finished parsing run0000_0000 timepoint 0.56325
 finished parsing run0000_0000 timepoint 0.5633
 finished parsing run0000_0000 timepoint 0.56335
 finished parsing run0000_0000 timepoint 0.5634
 finished parsing run0000_0000 timepoint 0.56345
 finished parsing run0000_0000 timepoint 0.5635
 finished parsing run0000_0000 timepoint 0.56355
 finished parsing run0000_0000 timepoint 0.5636
 finished parsing run0000_0000 timepoint 0.56365
 finished parsing run0000_0000 timepoint 0.5637
 finished parsing run0000_0000 timepoint 0.56375
 finished parsing run0000_0000 timepoint 0.5638
 finished parsing run0000_0000 timepoint 0.56385
 finished parsing run0000_0000 timepoint 0.5639
 finished parsing run0000_0000 timepoint 0.56395
 finished parsing run0000_0000 timepoint 0.564
 finished parsing run0000_0000 timepoint 0.56405
 finished parsing run0000_0000 timepoint 0.5641
 finished parsing run0000_0000 timepoint 0.56415
 finished parsing run0000_0000 timepoint 0.5642
 finished parsing run0000_0000 timepoint 0.56425
 finished parsing run0000_0000 timepoint 0.5643
 finished parsing run0000_0000 timepoint 0.56435
 finished parsing run0000_0000 timepoint 0.5644
 finished parsing run0000_0000 timepoint 0.56445
 finished parsing run0000_0000 timepoint 0.5645
 finished parsing run0000_0000 timepoint 0.56455
 finished parsing run0000_0000 timepoint 0.5646
 finished parsing run0000_0000 timepoint 0.56465
 finished parsing run0000_0000 timepoint 0.5647
 finished parsing run0000_0000 timepoint 0.56475
 finished parsing run0000_0000 timepoint 0.5648
 finished parsing run0000_0000 timepoint 0.56485
 finished parsing run0000_0000 timepoint 0.5649
 finished parsing run0000_0000 timepoint 0.56495
 finished parsing run0000_0000 timepoint 0.565
 finished parsing run0000_0000 timepoint 0.56505
 finished parsing run0000_0000 timepoint 0.5651
 finished parsing run0000_0000 timepoint 0.56515
 finished parsing run0000_0000 timepoint 0.5652
 finished parsing run0000_0000 timepoint 0.56525
 finished parsing run0000_0000 timepoint 0.5653
 finished parsing run0000_0000 timepoint 0.56535
 finished parsing run0000_0000 timepoint 0.5654
 finished parsing run0000_0000 timepoint 0.56545
 finished parsing run0000_0000 timepoint 0.5655
 finished parsing run0000_0000 timepoint 0.56555
 finished parsing run0000_0000 timepoint 0.5656
 finished parsing run0000_0000 timepoint 0.56565
 finished parsing run0000_0000 timepoint 0.5657
 finished parsing run0000_0000 timepoint 0.56575
 finished parsing run0000_0000 timepoint 0.5658
 finished parsing run0000_0000 timepoint 0.56585
 finished parsing run0000_0000 timepoint 0.5659
 finished parsing run0000_0000 timepoint 0.56595
 finished parsing run0000_0000 timepoint 0.566
 finished parsing run0000_0000 timepoint 0.56605
 finished parsing run0000_0000 timepoint 0.5661
 finished parsing run0000_0000 timepoint 0.56615
 finished parsing run0000_0000 timepoint 0.5662
 finished parsing run0000_0000 timepoint 0.56625
 finished parsing run0000_0000 timepoint 0.5663
 finished parsing run0000_0000 timepoint 0.56635
 finished parsing run0000_0000 timepoint 0.5664
 finished parsing run0000_0000 timepoint 0.56645
 finished parsing run0000_0000 timepoint 0.5665
 finished parsing run0000_0000 timepoint 0.56655
 finished parsing run0000_0000 timepoint 0.5666
 finished parsing run0000_0000 timepoint 0.56665
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.55375
 finished parsing run0000_0000 timepoint 0.5538
 finished parsing run0000_0000 timepoint 0.55385
 finished parsing run0000_0000 timepoint 0.5539
 finished parsing run0000_0000 timepoint 0.55395
 finished parsing run0000_0000 timepoint 0.554
 finished parsing run0000_0000 timepoint 0.55405
 finished parsing run0000_0000 timepoint 0.5541
 finished parsing run0000_0000 timepoint 0.55415
 finished parsing run0000_0000 timepoint 0.5542
 finished parsing run0000_0000 timepoint 0.55425
 finished parsing run0000_0000 timepoint 0.5543
 finished parsing run0000_0000 timepoint 0.55435
 finished parsing run0000_0000 timepoint 0.5544
 finished parsing run0000_0000 timepoint 0.55445
 finished parsing run0000_0000 timepoint 0.5545
 finished parsing run0000_0000 timepoint 0.55455
 finished parsing run0000_0000 timepoint 0.5546
 finished parsing run0000_0000 timepoint 0.55465
 finished parsing run0000_0000 timepoint 0.5547
 finished parsing run0000_0000 timepoint 0.55475
 finished parsing run0000_0000 timepoint 0.5548
 finished parsing run0000_0000 timepoint 0.55485
 finished parsing run0000_0000 timepoint 0.5549
 finished parsing run0000_0000 timepoint 0.55495
 finished parsing run0000_0000 timepoint 0.555
 finished parsing run0000_0000 timepoint 0.55505
 finished parsing run0000_0000 timepoint 0.5551
 finished parsing run0000_0000 timepoint 0.55515
 finished parsing run0000_0000 timepoint 0.5552
 finished parsing run0000_0000 timepoint 0.55525
 finished parsing run0000_0000 timepoint 0.5553
 finished parsing run0000_0000 timepoint 0.55535
 finished parsing run0000_0000 timepoint 0.5554
 finished parsing run0000_0000 timepoint 0.55545
 finished parsing run0000_0000 timepoint 0.5555
 finished parsing run0000_0000 timepoint 0.55555
 finished parsing run0000_0000 timepoint 0.5556
 finished parsing run0000_0000 timepoint 0.55565
 finished parsing run0000_0000 timepoint 0.5557
 finished parsing run0000_0000 timepoint 0.55575
 finished parsing run0000_0000 timepoint 0.5558
 finished parsing run0000_0000 timepoint 0.55585
 finished parsing run0000_0000 timepoint 0.5559
 finished parsing run0000_0000 timepoint 0.55595
 finished parsing run0000_0000 timepoint 0.556
 finished parsing run0000_0000 timepoint 0.55605
 finished parsing run0000_0000 timepoint 0.5561
 finished parsing run0000_0000 timepoint 0.55615
 finished parsing run0000_0000 timepoint 0.5562
 finished parsing run0000_0000 timepoint 0.55625
 finished parsing run0000_0000 timepoint 0.5563
 finished parsing run0000_0000 timepoint 0.55635
 finished parsing run0000_0000 timepoint 0.5564
 finished parsing run0000_0000 timepoint 0.55645
 finished parsing run0000_0000 timepoint 0.5565
 finished parsing run0000_0000 timepoint 0.55655
 finished parsing run0000_0000 timepoint 0.5566
 finished parsing run0000_0000 timepoint 0.55665
 finished parsing run0000_0000 timepoint 0.5567
 finished parsing run0000_0000 timepoint 0.55675
 finished parsing run0000_0000 timepoint 0.5568
 finished parsing run0000_0000 timepoint 0.55685
 finished parsing run0000_0000 timepoint 0.5569
 finished parsing run0000_0000 timepoint 0.55695
 finished parsing run0000_0000 timepoint 0.557
 finished parsing run0000_0000 timepoint 0.55705
 finished parsing run0000_0000 timepoint 0.5571
 finished parsing run0000_0000 timepoint 0.55715
 finished parsing run0000_0000 timepoint 0.5572
 finished parsing run0000_0000 timepoint 0.55725
 finished parsing run0000_0000 timepoint 0.5573
 finished parsing run0000_0000 timepoint 0.55735
 finished parsing run0000_0000 timepoint 0.5574
 finished parsing run0000_0000 timepoint 0.55745
 finished parsing run0000_0000 timepoint 0.5575
 finished parsing run0000_0000 timepoint 0.55755
 finished parsing run0000_0000 timepoint 0.5576
 finished parsing run0000_0000 timepoint 0.55765
 finished parsing run0000_0000 timepoint 0.5577
 finished parsing run0000_0000 timepoint 0.55775
 finished parsing run0000_0000 timepoint 0.5578
 finished parsing run0000_0000 timepoint 0.55785
 finished parsing run0000_0000 timepoint 0.5579
 finished parsing run0000_0000 timepoint 0.55795
 finished parsing run0000_0000 timepoint 0.558
 finished parsing run0000_0000 timepoint 0.55805
 finished parsing run0000_0000 timepoint 0.5581
 finished parsing run0000_0000 timepoint 0.55815
 finished parsing run0000_0000 timepoint 0.5582
 finished parsing run0000_0000 timepoint 0.55825
 finished parsing run0000_0000 timepoint 0.5583
 finished parsing run0000_0000 timepoint 0.55835
 finished parsing run0000_0000 timepoint 0.5584
 finished parsing run0000_0000 timepoint 0.55845
 finished parsing run0000_0000 timepoint 0.5585
 finished parsing run0000_0000 timepoint 0.55855
 finished parsing run0000_0000 timepoint 0.5586
 finished parsing run0000_0000 timepoint 0.55865
 finished parsing run0000_0000 timepoint 0.5587
 finished parsing run0000_0000 timepoint 0.55875
 finished parsing run0000_0000 timepoint 0.5588
 finished parsing run0000_0000 timepoint 0.55885
 finished parsing run0000_0000 timepoint 0.5589
 finished parsing run0000_0000 timepoint 0.55895
 finished parsing run0000_0000 timepoint 0.559
 finished parsing run0000_0000 timepoint 0.55905
 finished parsing run0000_0000 timepoint 0.5591
 finished parsing run0000_0000 timepoint 0.55915
 finished parsing run0000_0000 timepoint 0.5592
 finished parsing run0000_0000 timepoint 0.55925
 finished parsing run0000_0000 timepoint 0.5593
 finished parsing run0000_0000 timepoint 0.55935
 finished parsing run0000_0000 timepoint 0.5594
 finished parsing run0000_0000 timepoint 0.55945
 finished parsing run0000_0000 timepoint 0.5595
 finished parsing run0000_0000 timepoint 0.55955
 finished parsing run0000_0000 timepoint 0.5596
 finished parsing run0000_0000 timepoint 0.55965
 finished parsing run0000_0000 timepoint 0.5597
 finished parsing run0000_0000 timepoint 0.55975
 finished parsing run0000_0000 timepoint 0.5598
 finished parsing run0000_0000 timepoint 0.55985
 finished parsing run0000_0000 timepoint 0.5599
 finished parsing run0000_0000 timepoint 0.55995
 finished parsing run0000_0000 timepoint 0.56
 finished parsing run0000_0000 timepoint 0.56005
 finished parsing run0000_0000 timepoint 0.5601
 finished parsing run0000_0000 timepoint 0.56015
 finished parsing run0000_0000 timepoint 0.5602
 finished parsing run0000_0000 timepoint 0.56025
 finished parsing run0000_0000 timepoint 0.5603
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5474
 finished parsing run0000_0000 timepoint 0.54745
 finished parsing run0000_0000 timepoint 0.5475
 finished parsing run0000_0000 timepoint 0.54755
 finished parsing run0000_0000 timepoint 0.5476
 finished parsing run0000_0000 timepoint 0.54765
 finished parsing run0000_0000 timepoint 0.5477
 finished parsing run0000_0000 timepoint 0.54775
 finished parsing run0000_0000 timepoint 0.5478
 finished parsing run0000_0000 timepoint 0.54785
 finished parsing run0000_0000 timepoint 0.5479
 finished parsing run0000_0000 timepoint 0.54795
 finished parsing run0000_0000 timepoint 0.548
 finished parsing run0000_0000 timepoint 0.54805
 finished parsing run0000_0000 timepoint 0.5481
 finished parsing run0000_0000 timepoint 0.54815
 finished parsing run0000_0000 timepoint 0.5482
 finished parsing run0000_0000 timepoint 0.54825
 finished parsing run0000_0000 timepoint 0.5483
 finished parsing run0000_0000 timepoint 0.54835
 finished parsing run0000_0000 timepoint 0.5484
 finished parsing run0000_0000 timepoint 0.54845
 finished parsing run0000_0000 timepoint 0.5485
 finished parsing run0000_0000 timepoint 0.54855
 finished parsing run0000_0000 timepoint 0.5486
 finished parsing run0000_0000 timepoint 0.54865
 finished parsing run0000_0000 timepoint 0.5487
 finished parsing run0000_0000 timepoint 0.54875
 finished parsing run0000_0000 timepoint 0.5488
 finished parsing run0000_0000 timepoint 0.54885
 finished parsing run0000_0000 timepoint 0.5489
 finished parsing run0000_0000 timepoint 0.54895
 finished parsing run0000_0000 timepoint 0.549
 finished parsing run0000_0000 timepoint 0.54905
 finished parsing run0000_0000 timepoint 0.5491
 finished parsing run0000_0000 timepoint 0.54915
 finished parsing run0000_0000 timepoint 0.5492
 finished parsing run0000_0000 timepoint 0.54925
 finished parsing run0000_0000 timepoint 0.5493
 finished parsing run0000_0000 timepoint 0.54935
 finished parsing run0000_0000 timepoint 0.5494
 finished parsing run0000_0000 timepoint 0.54945
 finished parsing run0000_0000 timepoint 0.5495
 finished parsing run0000_0000 timepoint 0.54955
 finished parsing run0000_0000 timepoint 0.5496
 finished parsing run0000_0000 timepoint 0.54965
 finished parsing run0000_0000 timepoint 0.5497
 finished parsing run0000_0000 timepoint 0.54975
 finished parsing run0000_0000 timepoint 0.5498
 finished parsing run0000_0000 timepoint 0.54985
 finished parsing run0000_0000 timepoint 0.5499
 finished parsing run0000_0000 timepoint 0.54995
 finished parsing run0000_0000 timepoint 0.55
 finished parsing run0000_0000 timepoint 0.55005
 finished parsing run0000_0000 timepoint 0.5501
 finished parsing run0000_0000 timepoint 0.55015
 finished parsing run0000_0000 timepoint 0.5502
 finished parsing run0000_0000 timepoint 0.55025
 finished parsing run0000_0000 timepoint 0.5503
 finished parsing run0000_0000 timepoint 0.55035
 finished parsing run0000_0000 timepoint 0.5504
 finished parsing run0000_0000 timepoint 0.55045
 finished parsing run0000_0000 timepoint 0.5505
 finished parsing run0000_0000 timepoint 0.55055
 finished parsing run0000_0000 timepoint 0.5506
 finished parsing run0000_0000 timepoint 0.55065
 finished parsing run0000_0000 timepoint 0.5507
 finished parsing run0000_0000 timepoint 0.55075
 finished parsing run0000_0000 timepoint 0.5508
 finished parsing run0000_0000 timepoint 0.55085
 finished parsing run0000_0000 timepoint 0.5509
 finished parsing run0000_0000 timepoint 0.55095
 finished parsing run0000_0000 timepoint 0.551
 finished parsing run0000_0000 timepoint 0.55105
 finished parsing run0000_0000 timepoint 0.5511
 finished parsing run0000_0000 timepoint 0.55115
 finished parsing run0000_0000 timepoint 0.5512
 finished parsing run0000_0000 timepoint 0.55125
 finished parsing run0000_0000 timepoint 0.5513
 finished parsing run0000_0000 timepoint 0.55135
 finished parsing run0000_0000 timepoint 0.5514
 finished parsing run0000_0000 timepoint 0.55145
 finished parsing run0000_0000 timepoint 0.5515
 finished parsing run0000_0000 timepoint 0.55155
 finished parsing run0000_0000 timepoint 0.5516
 finished parsing run0000_0000 timepoint 0.55165
 finished parsing run0000_0000 timepoint 0.5517
 finished parsing run0000_0000 timepoint 0.55175
 finished parsing run0000_0000 timepoint 0.5518
 finished parsing run0000_0000 timepoint 0.55185
 finished parsing run0000_0000 timepoint 0.5519
 finished parsing run0000_0000 timepoint 0.55195
 finished parsing run0000_0000 timepoint 0.552
 finished parsing run0000_0000 timepoint 0.55205
 finished parsing run0000_0000 timepoint 0.5521
 finished parsing run0000_0000 timepoint 0.55215
 finished parsing run0000_0000 timepoint 0.5522
 finished parsing run0000_0000 timepoint 0.55225
 finished parsing run0000_0000 timepoint 0.5523
 finished parsing run0000_0000 timepoint 0.55235
 finished parsing run0000_0000 timepoint 0.5524
 finished parsing run0000_0000 timepoint 0.55245
 finished parsing run0000_0000 timepoint 0.5525
 finished parsing run0000_0000 timepoint 0.55255
 finished parsing run0000_0000 timepoint 0.5526
 finished parsing run0000_0000 timepoint 0.55265
 finished parsing run0000_0000 timepoint 0.5527
 finished parsing run0000_0000 timepoint 0.55275
 finished parsing run0000_0000 timepoint 0.5528
 finished parsing run0000_0000 timepoint 0.55285
 finished parsing run0000_0000 timepoint 0.5529
 finished parsing run0000_0000 timepoint 0.55295
 finished parsing run0000_0000 timepoint 0.553
 finished parsing run0000_0000 timepoint 0.55305
 finished parsing run0000_0000 timepoint 0.5531
 finished parsing run0000_0000 timepoint 0.55315
 finished parsing run0000_0000 timepoint 0.5532
 finished parsing run0000_0000 timepoint 0.55325
 finished parsing run0000_0000 timepoint 0.5533
 finished parsing run0000_0000 timepoint 0.55335
 finished parsing run0000_0000 timepoint 0.5534
 finished parsing run0000_0000 timepoint 0.55345
 finished parsing run0000_0000 timepoint 0.5535
 finished parsing run0000_0000 timepoint 0.55355
 finished parsing run0000_0000 timepoint 0.5536
 finished parsing run0000_0000 timepoint 0.55365
 finished parsing run0000_0000 timepoint 0.5537
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.54095
 finished parsing run0000_0000 timepoint 0.541
 finished parsing run0000_0000 timepoint 0.54105
 finished parsing run0000_0000 timepoint 0.5411
 finished parsing run0000_0000 timepoint 0.54115
 finished parsing run0000_0000 timepoint 0.5412
 finished parsing run0000_0000 timepoint 0.54125
 finished parsing run0000_0000 timepoint 0.5413
 finished parsing run0000_0000 timepoint 0.54135
 finished parsing run0000_0000 timepoint 0.5414
 finished parsing run0000_0000 timepoint 0.54145
 finished parsing run0000_0000 timepoint 0.5415
 finished parsing run0000_0000 timepoint 0.54155
 finished parsing run0000_0000 timepoint 0.5416
 finished parsing run0000_0000 timepoint 0.54165
 finished parsing run0000_0000 timepoint 0.5417
 finished parsing run0000_0000 timepoint 0.54175
 finished parsing run0000_0000 timepoint 0.5418
 finished parsing run0000_0000 timepoint 0.54185
 finished parsing run0000_0000 timepoint 0.5419
 finished parsing run0000_0000 timepoint 0.54195
 finished parsing run0000_0000 timepoint 0.542
 finished parsing run0000_0000 timepoint 0.54205
 finished parsing run0000_0000 timepoint 0.5421
 finished parsing run0000_0000 timepoint 0.54215
 finished parsing run0000_0000 timepoint 0.5422
 finished parsing run0000_0000 timepoint 0.54225
 finished parsing run0000_0000 timepoint 0.5423
 finished parsing run0000_0000 timepoint 0.54235
 finished parsing run0000_0000 timepoint 0.5424
 finished parsing run0000_0000 timepoint 0.54245
 finished parsing run0000_0000 timepoint 0.5425
 finished parsing run0000_0000 timepoint 0.54255
 finished parsing run0000_0000 timepoint 0.5426
 finished parsing run0000_0000 timepoint 0.54265
 finished parsing run0000_0000 timepoint 0.5427
 finished parsing run0000_0000 timepoint 0.54275
 finished parsing run0000_0000 timepoint 0.5428
 finished parsing run0000_0000 timepoint 0.54285
 finished parsing run0000_0000 timepoint 0.5429
 finished parsing run0000_0000 timepoint 0.54295
 finished parsing run0000_0000 timepoint 0.543
 finished parsing run0000_0000 timepoint 0.54305
 finished parsing run0000_0000 timepoint 0.5431
 finished parsing run0000_0000 timepoint 0.54315
 finished parsing run0000_0000 timepoint 0.5432
 finished parsing run0000_0000 timepoint 0.54325
 finished parsing run0000_0000 timepoint 0.5433
 finished parsing run0000_0000 timepoint 0.54335
 finished parsing run0000_0000 timepoint 0.5434
 finished parsing run0000_0000 timepoint 0.54345
 finished parsing run0000_0000 timepoint 0.5435
 finished parsing run0000_0000 timepoint 0.54355
 finished parsing run0000_0000 timepoint 0.5436
 finished parsing run0000_0000 timepoint 0.54365
 finished parsing run0000_0000 timepoint 0.5437
 finished parsing run0000_0000 timepoint 0.54375
 finished parsing run0000_0000 timepoint 0.5438
 finished parsing run0000_0000 timepoint 0.54385
 finished parsing run0000_0000 timepoint 0.5439
 finished parsing run0000_0000 timepoint 0.54395
 finished parsing run0000_0000 timepoint 0.544
 finished parsing run0000_0000 timepoint 0.54405
 finished parsing run0000_0000 timepoint 0.5441
 finished parsing run0000_0000 timepoint 0.54415
 finished parsing run0000_0000 timepoint 0.5442
 finished parsing run0000_0000 timepoint 0.54425
 finished parsing run0000_0000 timepoint 0.5443
 finished parsing run0000_0000 timepoint 0.54435
 finished parsing run0000_0000 timepoint 0.5444
 finished parsing run0000_0000 timepoint 0.54445
 finished parsing run0000_0000 timepoint 0.5445
 finished parsing run0000_0000 timepoint 0.54455
 finished parsing run0000_0000 timepoint 0.5446
 finished parsing run0000_0000 timepoint 0.54465
 finished parsing run0000_0000 timepoint 0.5447
 finished parsing run0000_0000 timepoint 0.54475
 finished parsing run0000_0000 timepoint 0.5448
 finished parsing run0000_0000 timepoint 0.54485
 finished parsing run0000_0000 timepoint 0.5449
 finished parsing run0000_0000 timepoint 0.54495
 finished parsing run0000_0000 timepoint 0.545
 finished parsing run0000_0000 timepoint 0.54505
 finished parsing run0000_0000 timepoint 0.5451
 finished parsing run0000_0000 timepoint 0.54515
 finished parsing run0000_0000 timepoint 0.5452
 finished parsing run0000_0000 timepoint 0.54525
 finished parsing run0000_0000 timepoint 0.5453
 finished parsing run0000_0000 timepoint 0.54535
 finished parsing run0000_0000 timepoint 0.5454
 finished parsing run0000_0000 timepoint 0.54545
 finished parsing run0000_0000 timepoint 0.5455
 finished parsing run0000_0000 timepoint 0.54555
 finished parsing run0000_0000 timepoint 0.5456
 finished parsing run0000_0000 timepoint 0.54565
 finished parsing run0000_0000 timepoint 0.5457
 finished parsing run0000_0000 timepoint 0.54575
 finished parsing run0000_0000 timepoint 0.5458
 finished parsing run0000_0000 timepoint 0.54585
 finished parsing run0000_0000 timepoint 0.5459
 finished parsing run0000_0000 timepoint 0.54595
 finished parsing run0000_0000 timepoint 0.546
 finished parsing run0000_0000 timepoint 0.54605
 finished parsing run0000_0000 timepoint 0.5461
 finished parsing run0000_0000 timepoint 0.54615
 finished parsing run0000_0000 timepoint 0.5462
 finished parsing run0000_0000 timepoint 0.54625
 finished parsing run0000_0000 timepoint 0.5463
 finished parsing run0000_0000 timepoint 0.54635
 finished parsing run0000_0000 timepoint 0.5464
 finished parsing run0000_0000 timepoint 0.54645
 finished parsing run0000_0000 timepoint 0.5465
 finished parsing run0000_0000 timepoint 0.54655
 finished parsing run0000_0000 timepoint 0.5466
 finished parsing run0000_0000 timepoint 0.54665
 finished parsing run0000_0000 timepoint 0.5467
 finished parsing run0000_0000 timepoint 0.54675
 finished parsing run0000_0000 timepoint 0.5468
 finished parsing run0000_0000 timepoint 0.54685
 finished parsing run0000_0000 timepoint 0.5469
 finished parsing run0000_0000 timepoint 0.54695
 finished parsing run0000_0000 timepoint 0.547
 finished parsing run0000_0000 timepoint 0.54705
 finished parsing run0000_0000 timepoint 0.5471
 finished parsing run0000_0000 timepoint 0.54715
 finished parsing run0000_0000 timepoint 0.5472
 finished parsing run0000_0000 timepoint 0.54725
 finished parsing run0000_0000 timepoint 0.5473
 finished parsing run0000_0000 timepoint 0.54735
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.53445
 finished parsing run0000_0000 timepoint 0.5345
 finished parsing run0000_0000 timepoint 0.53455
 finished parsing run0000_0000 timepoint 0.5346
 finished parsing run0000_0000 timepoint 0.53465
 finished parsing run0000_0000 timepoint 0.5347
 finished parsing run0000_0000 timepoint 0.53475
 finished parsing run0000_0000 timepoint 0.5348
 finished parsing run0000_0000 timepoint 0.53485
 finished parsing run0000_0000 timepoint 0.5349
 finished parsing run0000_0000 timepoint 0.53495
 finished parsing run0000_0000 timepoint 0.535
 finished parsing run0000_0000 timepoint 0.53505
 finished parsing run0000_0000 timepoint 0.5351
 finished parsing run0000_0000 timepoint 0.53515
 finished parsing run0000_0000 timepoint 0.5352
 finished parsing run0000_0000 timepoint 0.53525
 finished parsing run0000_0000 timepoint 0.5353
 finished parsing run0000_0000 timepoint 0.53535
 finished parsing run0000_0000 timepoint 0.5354
 finished parsing run0000_0000 timepoint 0.53545
 finished parsing run0000_0000 timepoint 0.5355
 finished parsing run0000_0000 timepoint 0.53555
 finished parsing run0000_0000 timepoint 0.5356
 finished parsing run0000_0000 timepoint 0.53565
 finished parsing run0000_0000 timepoint 0.5357
 finished parsing run0000_0000 timepoint 0.53575
 finished parsing run0000_0000 timepoint 0.5358
 finished parsing run0000_0000 timepoint 0.53585
 finished parsing run0000_0000 timepoint 0.5359
 finished parsing run0000_0000 timepoint 0.53595
 finished parsing run0000_0000 timepoint 0.536
 finished parsing run0000_0000 timepoint 0.53605
 finished parsing run0000_0000 timepoint 0.5361
 finished parsing run0000_0000 timepoint 0.53615
 finished parsing run0000_0000 timepoint 0.5362
 finished parsing run0000_0000 timepoint 0.53625
 finished parsing run0000_0000 timepoint 0.5363
 finished parsing run0000_0000 timepoint 0.53635
 finished parsing run0000_0000 timepoint 0.5364
 finished parsing run0000_0000 timepoint 0.53645
 finished parsing run0000_0000 timepoint 0.5365
 finished parsing run0000_0000 timepoint 0.53655
 finished parsing run0000_0000 timepoint 0.5366
 finished parsing run0000_0000 timepoint 0.53665
 finished parsing run0000_0000 timepoint 0.5367
 finished parsing run0000_0000 timepoint 0.53675
 finished parsing run0000_0000 timepoint 0.5368
 finished parsing run0000_0000 timepoint 0.53685
 finished parsing run0000_0000 timepoint 0.5369
 finished parsing run0000_0000 timepoint 0.53695
 finished parsing run0000_0000 timepoint 0.537
 finished parsing run0000_0000 timepoint 0.53705
 finished parsing run0000_0000 timepoint 0.5371
 finished parsing run0000_0000 timepoint 0.53715
 finished parsing run0000_0000 timepoint 0.5372
 finished parsing run0000_0000 timepoint 0.53725
 finished parsing run0000_0000 timepoint 0.5373
 finished parsing run0000_0000 timepoint 0.53735
 finished parsing run0000_0000 timepoint 0.5374
 finished parsing run0000_0000 timepoint 0.53745
 finished parsing run0000_0000 timepoint 0.5375
 finished parsing run0000_0000 timepoint 0.53755
 finished parsing run0000_0000 timepoint 0.5376
 finished parsing run0000_0000 timepoint 0.53765
 finished parsing run0000_0000 timepoint 0.5377
 finished parsing run0000_0000 timepoint 0.53775
 finished parsing run0000_0000 timepoint 0.5378
 finished parsing run0000_0000 timepoint 0.53785
 finished parsing run0000_0000 timepoint 0.5379
 finished parsing run0000_0000 timepoint 0.53795
 finished parsing run0000_0000 timepoint 0.538
 finished parsing run0000_0000 timepoint 0.53805
 finished parsing run0000_0000 timepoint 0.5381
 finished parsing run0000_0000 timepoint 0.53815
 finished parsing run0000_0000 timepoint 0.5382
 finished parsing run0000_0000 timepoint 0.53825
 finished parsing run0000_0000 timepoint 0.5383
 finished parsing run0000_0000 timepoint 0.53835
 finished parsing run0000_0000 timepoint 0.5384
 finished parsing run0000_0000 timepoint 0.53845
 finished parsing run0000_0000 timepoint 0.5385
 finished parsing run0000_0000 timepoint 0.53855
 finished parsing run0000_0000 timepoint 0.5386
 finished parsing run0000_0000 timepoint 0.53865
 finished parsing run0000_0000 timepoint 0.5387
 finished parsing run0000_0000 timepoint 0.53875
 finished parsing run0000_0000 timepoint 0.5388
 finished parsing run0000_0000 timepoint 0.53885
 finished parsing run0000_0000 timepoint 0.5389
 finished parsing run0000_0000 timepoint 0.53895
 finished parsing run0000_0000 timepoint 0.539
 finished parsing run0000_0000 timepoint 0.53905
 finished parsing run0000_0000 timepoint 0.5391
 finished parsing run0000_0000 timepoint 0.53915
 finished parsing run0000_0000 timepoint 0.5392
 finished parsing run0000_0000 timepoint 0.53925
 finished parsing run0000_0000 timepoint 0.5393
 finished parsing run0000_0000 timepoint 0.53935
 finished parsing run0000_0000 timepoint 0.5394
 finished parsing run0000_0000 timepoint 0.53945
 finished parsing run0000_0000 timepoint 0.5395
 finished parsing run0000_0000 timepoint 0.53955
 finished parsing run0000_0000 timepoint 0.5396
 finished parsing run0000_0000 timepoint 0.53965
 finished parsing run0000_0000 timepoint 0.5397
 finished parsing run0000_0000 timepoint 0.53975
 finished parsing run0000_0000 timepoint 0.5398
 finished parsing run0000_0000 timepoint 0.53985
 finished parsing run0000_0000 timepoint 0.5399
 finished parsing run0000_0000 timepoint 0.53995
 finished parsing run0000_0000 timepoint 0.54
 finished parsing run0000_0000 timepoint 0.54005
 finished parsing run0000_0000 timepoint 0.5401
 finished parsing run0000_0000 timepoint 0.54015
 finished parsing run0000_0000 timepoint 0.5402
 finished parsing run0000_0000 timepoint 0.54025
 finished parsing run0000_0000 timepoint 0.5403
 finished parsing run0000_0000 timepoint 0.54035
 finished parsing run0000_0000 timepoint 0.5404
 finished parsing run0000_0000 timepoint 0.54045
 finished parsing run0000_0000 timepoint 0.5405
 finished parsing run0000_0000 timepoint 0.54055
 finished parsing run0000_0000 timepoint 0.5406
 finished parsing run0000_0000 timepoint 0.54065
 finished parsing run0000_0000 timepoint 0.5407
 finished parsing run0000_0000 timepoint 0.54075
 finished parsing run0000_0000 timepoint 0.5408
 finished parsing run0000_0000 timepoint 0.54085
 finished parsing run0000_0000 timepoint 0.5409
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.52805
 finished parsing run0000_0000 timepoint 0.5281
 finished parsing run0000_0000 timepoint 0.52815
 finished parsing run0000_0000 timepoint 0.5282
 finished parsing run0000_0000 timepoint 0.52825
 finished parsing run0000_0000 timepoint 0.5283
 finished parsing run0000_0000 timepoint 0.52835
 finished parsing run0000_0000 timepoint 0.5284
 finished parsing run0000_0000 timepoint 0.52845
 finished parsing run0000_0000 timepoint 0.5285
 finished parsing run0000_0000 timepoint 0.52855
 finished parsing run0000_0000 timepoint 0.5286
 finished parsing run0000_0000 timepoint 0.52865
 finished parsing run0000_0000 timepoint 0.5287
 finished parsing run0000_0000 timepoint 0.52875
 finished parsing run0000_0000 timepoint 0.5288
 finished parsing run0000_0000 timepoint 0.52885
 finished parsing run0000_0000 timepoint 0.5289
 finished parsing run0000_0000 timepoint 0.52895
 finished parsing run0000_0000 timepoint 0.529
 finished parsing run0000_0000 timepoint 0.52905
 finished parsing run0000_0000 timepoint 0.5291
 finished parsing run0000_0000 timepoint 0.52915
 finished parsing run0000_0000 timepoint 0.5292
 finished parsing run0000_0000 timepoint 0.52925
 finished parsing run0000_0000 timepoint 0.5293
 finished parsing run0000_0000 timepoint 0.52935
 finished parsing run0000_0000 timepoint 0.5294
 finished parsing run0000_0000 timepoint 0.52945
 finished parsing run0000_0000 timepoint 0.5295
 finished parsing run0000_0000 timepoint 0.52955
 finished parsing run0000_0000 timepoint 0.5296
 finished parsing run0000_0000 timepoint 0.52965
 finished parsing run0000_0000 timepoint 0.5297
 finished parsing run0000_0000 timepoint 0.52975
 finished parsing run0000_0000 timepoint 0.5298
 finished parsing run0000_0000 timepoint 0.52985
 finished parsing run0000_0000 timepoint 0.5299
 finished parsing run0000_0000 timepoint 0.52995
 finished parsing run0000_0000 timepoint 0.53
 finished parsing run0000_0000 timepoint 0.53005
 finished parsing run0000_0000 timepoint 0.5301
 finished parsing run0000_0000 timepoint 0.53015
 finished parsing run0000_0000 timepoint 0.5302
 finished parsing run0000_0000 timepoint 0.53025
 finished parsing run0000_0000 timepoint 0.5303
 finished parsing run0000_0000 timepoint 0.53035
 finished parsing run0000_0000 timepoint 0.5304
 finished parsing run0000_0000 timepoint 0.53045
 finished parsing run0000_0000 timepoint 0.5305
 finished parsing run0000_0000 timepoint 0.53055
 finished parsing run0000_0000 timepoint 0.5306
 finished parsing run0000_0000 timepoint 0.53065
 finished parsing run0000_0000 timepoint 0.5307
 finished parsing run0000_0000 timepoint 0.53075
 finished parsing run0000_0000 timepoint 0.5308
 finished parsing run0000_0000 timepoint 0.53085
 finished parsing run0000_0000 timepoint 0.5309
 finished parsing run0000_0000 timepoint 0.53095
 finished parsing run0000_0000 timepoint 0.531
 finished parsing run0000_0000 timepoint 0.53105
 finished parsing run0000_0000 timepoint 0.5311
 finished parsing run0000_0000 timepoint 0.53115
 finished parsing run0000_0000 timepoint 0.5312
 finished parsing run0000_0000 timepoint 0.53125
 finished parsing run0000_0000 timepoint 0.5313
 finished parsing run0000_0000 timepoint 0.53135
 finished parsing run0000_0000 timepoint 0.5314
 finished parsing run0000_0000 timepoint 0.53145
 finished parsing run0000_0000 timepoint 0.5315
 finished parsing run0000_0000 timepoint 0.53155
 finished parsing run0000_0000 timepoint 0.5316
 finished parsing run0000_0000 timepoint 0.53165
 finished parsing run0000_0000 timepoint 0.5317
 finished parsing run0000_0000 timepoint 0.53175
 finished parsing run0000_0000 timepoint 0.5318
 finished parsing run0000_0000 timepoint 0.53185
 finished parsing run0000_0000 timepoint 0.5319
 finished parsing run0000_0000 timepoint 0.53195
 finished parsing run0000_0000 timepoint 0.532
 finished parsing run0000_0000 timepoint 0.53205
 finished parsing run0000_0000 timepoint 0.5321
 finished parsing run0000_0000 timepoint 0.53215
 finished parsing run0000_0000 timepoint 0.5322
 finished parsing run0000_0000 timepoint 0.53225
 finished parsing run0000_0000 timepoint 0.5323
 finished parsing run0000_0000 timepoint 0.53235
 finished parsing run0000_0000 timepoint 0.5324
 finished parsing run0000_0000 timepoint 0.53245
 finished parsing run0000_0000 timepoint 0.5325
 finished parsing run0000_0000 timepoint 0.53255
 finished parsing run0000_0000 timepoint 0.5326
 finished parsing run0000_0000 timepoint 0.53265
 finished parsing run0000_0000 timepoint 0.5327
 finished parsing run0000_0000 timepoint 0.53275
 finished parsing run0000_0000 timepoint 0.5328
 finished parsing run0000_0000 timepoint 0.53285
 finished parsing run0000_0000 timepoint 0.5329
 finished parsing run0000_0000 timepoint 0.53295
 finished parsing run0000_0000 timepoint 0.533
 finished parsing run0000_0000 timepoint 0.53305
 finished parsing run0000_0000 timepoint 0.5331
 finished parsing run0000_0000 timepoint 0.53315
 finished parsing run0000_0000 timepoint 0.5332
 finished parsing run0000_0000 timepoint 0.53325
 finished parsing run0000_0000 timepoint 0.5333
 finished parsing run0000_0000 timepoint 0.53335
 finished parsing run0000_0000 timepoint 0.5334
 finished parsing run0000_0000 timepoint 0.53345
 finished parsing run0000_0000 timepoint 0.5335
 finished parsing run0000_0000 timepoint 0.53355
 finished parsing run0000_0000 timepoint 0.5336
 finished parsing run0000_0000 timepoint 0.53365
 finished parsing run0000_0000 timepoint 0.5337
 finished parsing run0000_0000 timepoint 0.53375
 finished parsing run0000_0000 timepoint 0.5338
 finished parsing run0000_0000 timepoint 0.53385
 finished parsing run0000_0000 timepoint 0.5339
 finished parsing run0000_0000 timepoint 0.53395
 finished parsing run0000_0000 timepoint 0.534
 finished parsing run0000_0000 timepoint 0.53405
 finished parsing run0000_0000 timepoint 0.5341
 finished parsing run0000_0000 timepoint 0.53415
 finished parsing run0000_0000 timepoint 0.5342
 finished parsing run0000_0000 timepoint 0.53425
 finished parsing run0000_0000 timepoint 0.5343
 finished parsing run0000_0000 timepoint 0.53435
 finished parsing run0000_0000 timepoint 0.5344
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5215
 finished parsing run0000_0000 timepoint 0.52155
 finished parsing run0000_0000 timepoint 0.5216
 finished parsing run0000_0000 timepoint 0.52165
 finished parsing run0000_0000 timepoint 0.5217
 finished parsing run0000_0000 timepoint 0.52175
 finished parsing run0000_0000 timepoint 0.5218
 finished parsing run0000_0000 timepoint 0.52185
 finished parsing run0000_0000 timepoint 0.5219
 finished parsing run0000_0000 timepoint 0.52195
 finished parsing run0000_0000 timepoint 0.522
 finished parsing run0000_0000 timepoint 0.52205
 finished parsing run0000_0000 timepoint 0.5221
 finished parsing run0000_0000 timepoint 0.52215
 finished parsing run0000_0000 timepoint 0.5222
 finished parsing run0000_0000 timepoint 0.52225
 finished parsing run0000_0000 timepoint 0.5223
 finished parsing run0000_0000 timepoint 0.52235
 finished parsing run0000_0000 timepoint 0.5224
 finished parsing run0000_0000 timepoint 0.52245
 finished parsing run0000_0000 timepoint 0.5225
 finished parsing run0000_0000 timepoint 0.52255
 finished parsing run0000_0000 timepoint 0.5226
 finished parsing run0000_0000 timepoint 0.52265
 finished parsing run0000_0000 timepoint 0.5227
 finished parsing run0000_0000 timepoint 0.52275
 finished parsing run0000_0000 timepoint 0.5228
 finished parsing run0000_0000 timepoint 0.52285
 finished parsing run0000_0000 timepoint 0.5229
 finished parsing run0000_0000 timepoint 0.52295
 finished parsing run0000_0000 timepoint 0.523
 finished parsing run0000_0000 timepoint 0.52305
 finished parsing run0000_0000 timepoint 0.5231
 finished parsing run0000_0000 timepoint 0.52315
 finished parsing run0000_0000 timepoint 0.5232
 finished parsing run0000_0000 timepoint 0.52325
 finished parsing run0000_0000 timepoint 0.5233
 finished parsing run0000_0000 timepoint 0.52335
 finished parsing run0000_0000 timepoint 0.5234
 finished parsing run0000_0000 timepoint 0.52345
 finished parsing run0000_0000 timepoint 0.5235
 finished parsing run0000_0000 timepoint 0.52355
 finished parsing run0000_0000 timepoint 0.5236
 finished parsing run0000_0000 timepoint 0.52365
 finished parsing run0000_0000 timepoint 0.5237
 finished parsing run0000_0000 timepoint 0.52375
 finished parsing run0000_0000 timepoint 0.5238
 finished parsing run0000_0000 timepoint 0.52385
 finished parsing run0000_0000 timepoint 0.5239
 finished parsing run0000_0000 timepoint 0.52395
 finished parsing run0000_0000 timepoint 0.524
 finished parsing run0000_0000 timepoint 0.52405
 finished parsing run0000_0000 timepoint 0.5241
 finished parsing run0000_0000 timepoint 0.52415
 finished parsing run0000_0000 timepoint 0.5242
 finished parsing run0000_0000 timepoint 0.52425
 finished parsing run0000_0000 timepoint 0.5243
 finished parsing run0000_0000 timepoint 0.52435
 finished parsing run0000_0000 timepoint 0.5244
 finished parsing run0000_0000 timepoint 0.52445
 finished parsing run0000_0000 timepoint 0.5245
 finished parsing run0000_0000 timepoint 0.52455
 finished parsing run0000_0000 timepoint 0.5246
 finished parsing run0000_0000 timepoint 0.52465
 finished parsing run0000_0000 timepoint 0.5247
 finished parsing run0000_0000 timepoint 0.52475
 finished parsing run0000_0000 timepoint 0.5248
 finished parsing run0000_0000 timepoint 0.52485
 finished parsing run0000_0000 timepoint 0.5249
 finished parsing run0000_0000 timepoint 0.52495
 finished parsing run0000_0000 timepoint 0.525
 finished parsing run0000_0000 timepoint 0.52505
 finished parsing run0000_0000 timepoint 0.5251
 finished parsing run0000_0000 timepoint 0.52515
 finished parsing run0000_0000 timepoint 0.5252
 finished parsing run0000_0000 timepoint 0.52525
 finished parsing run0000_0000 timepoint 0.5253
 finished parsing run0000_0000 timepoint 0.52535
 finished parsing run0000_0000 timepoint 0.5254
 finished parsing run0000_0000 timepoint 0.52545
 finished parsing run0000_0000 timepoint 0.5255
 finished parsing run0000_0000 timepoint 0.52555
 finished parsing run0000_0000 timepoint 0.5256
 finished parsing run0000_0000 timepoint 0.52565
 finished parsing run0000_0000 timepoint 0.5257
 finished parsing run0000_0000 timepoint 0.52575
 finished parsing run0000_0000 timepoint 0.5258
 finished parsing run0000_0000 timepoint 0.52585
 finished parsing run0000_0000 timepoint 0.5259
 finished parsing run0000_0000 timepoint 0.52595
 finished parsing run0000_0000 timepoint 0.526
 finished parsing run0000_0000 timepoint 0.52605
 finished parsing run0000_0000 timepoint 0.5261
 finished parsing run0000_0000 timepoint 0.52615
 finished parsing run0000_0000 timepoint 0.5262
 finished parsing run0000_0000 timepoint 0.52625
 finished parsing run0000_0000 timepoint 0.5263
 finished parsing run0000_0000 timepoint 0.52635
 finished parsing run0000_0000 timepoint 0.5264
 finished parsing run0000_0000 timepoint 0.52645
 finished parsing run0000_0000 timepoint 0.5265
 finished parsing run0000_0000 timepoint 0.52655
 finished parsing run0000_0000 timepoint 0.5266
 finished parsing run0000_0000 timepoint 0.52665
 finished parsing run0000_0000 timepoint 0.5267
 finished parsing run0000_0000 timepoint 0.52675
 finished parsing run0000_0000 timepoint 0.5268
 finished parsing run0000_0000 timepoint 0.52685
 finished parsing run0000_0000 timepoint 0.5269
 finished parsing run0000_0000 timepoint 0.52695
 finished parsing run0000_0000 timepoint 0.527
 finished parsing run0000_0000 timepoint 0.52705
 finished parsing run0000_0000 timepoint 0.5271
 finished parsing run0000_0000 timepoint 0.52715
 finished parsing run0000_0000 timepoint 0.5272
 finished parsing run0000_0000 timepoint 0.52725
 finished parsing run0000_0000 timepoint 0.5273
 finished parsing run0000_0000 timepoint 0.52735
 finished parsing run0000_0000 timepoint 0.5274
 finished parsing run0000_0000 timepoint 0.52745
 finished parsing run0000_0000 timepoint 0.5275
 finished parsing run0000_0000 timepoint 0.52755
 finished parsing run0000_0000 timepoint 0.5276
 finished parsing run0000_0000 timepoint 0.52765
 finished parsing run0000_0000 timepoint 0.5277
 finished parsing run0000_0000 timepoint 0.52775
 finished parsing run0000_0000 timepoint 0.5278
 finished parsing run0000_0000 timepoint 0.52785
 finished parsing run0000_0000 timepoint 0.5279
 finished parsing run0000_0000 timepoint 0.52795
 finished parsing run0000_0000 timepoint 0.528
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.51505
 finished parsing run0000_0000 timepoint 0.5151
 finished parsing run0000_0000 timepoint 0.51515
 finished parsing run0000_0000 timepoint 0.5152
 finished parsing run0000_0000 timepoint 0.51525
 finished parsing run0000_0000 timepoint 0.5153
 finished parsing run0000_0000 timepoint 0.51535
 finished parsing run0000_0000 timepoint 0.5154
 finished parsing run0000_0000 timepoint 0.51545
 finished parsing run0000_0000 timepoint 0.5155
 finished parsing run0000_0000 timepoint 0.51555
 finished parsing run0000_0000 timepoint 0.5156
 finished parsing run0000_0000 timepoint 0.51565
 finished parsing run0000_0000 timepoint 0.5157
 finished parsing run0000_0000 timepoint 0.51575
 finished parsing run0000_0000 timepoint 0.5158
 finished parsing run0000_0000 timepoint 0.51585
 finished parsing run0000_0000 timepoint 0.5159
 finished parsing run0000_0000 timepoint 0.51595
 finished parsing run0000_0000 timepoint 0.516
 finished parsing run0000_0000 timepoint 0.51605
 finished parsing run0000_0000 timepoint 0.5161
 finished parsing run0000_0000 timepoint 0.51615
 finished parsing run0000_0000 timepoint 0.5162
 finished parsing run0000_0000 timepoint 0.51625
 finished parsing run0000_0000 timepoint 0.5163
 finished parsing run0000_0000 timepoint 0.51635
 finished parsing run0000_0000 timepoint 0.5164
 finished parsing run0000_0000 timepoint 0.51645
 finished parsing run0000_0000 timepoint 0.5165
 finished parsing run0000_0000 timepoint 0.51655
 finished parsing run0000_0000 timepoint 0.5166
 finished parsing run0000_0000 timepoint 0.51665
 finished parsing run0000_0000 timepoint 0.5167
 finished parsing run0000_0000 timepoint 0.51675
 finished parsing run0000_0000 timepoint 0.5168
 finished parsing run0000_0000 timepoint 0.51685
 finished parsing run0000_0000 timepoint 0.5169
 finished parsing run0000_0000 timepoint 0.51695
 finished parsing run0000_0000 timepoint 0.517
 finished parsing run0000_0000 timepoint 0.51705
 finished parsing run0000_0000 timepoint 0.5171
 finished parsing run0000_0000 timepoint 0.51715
 finished parsing run0000_0000 timepoint 0.5172
 finished parsing run0000_0000 timepoint 0.51725
 finished parsing run0000_0000 timepoint 0.5173
 finished parsing run0000_0000 timepoint 0.51735
 finished parsing run0000_0000 timepoint 0.5174
 finished parsing run0000_0000 timepoint 0.51745
 finished parsing run0000_0000 timepoint 0.5175
 finished parsing run0000_0000 timepoint 0.51755
 finished parsing run0000_0000 timepoint 0.5176
 finished parsing run0000_0000 timepoint 0.51765
 finished parsing run0000_0000 timepoint 0.5177
 finished parsing run0000_0000 timepoint 0.51775
 finished parsing run0000_0000 timepoint 0.5178
 finished parsing run0000_0000 timepoint 0.51785
 finished parsing run0000_0000 timepoint 0.5179
 finished parsing run0000_0000 timepoint 0.51795
 finished parsing run0000_0000 timepoint 0.518
 finished parsing run0000_0000 timepoint 0.51805
 finished parsing run0000_0000 timepoint 0.5181
 finished parsing run0000_0000 timepoint 0.51815
 finished parsing run0000_0000 timepoint 0.5182
 finished parsing run0000_0000 timepoint 0.51825
 finished parsing run0000_0000 timepoint 0.5183
 finished parsing run0000_0000 timepoint 0.51835
 finished parsing run0000_0000 timepoint 0.5184
 finished parsing run0000_0000 timepoint 0.51845
 finished parsing run0000_0000 timepoint 0.5185
 finished parsing run0000_0000 timepoint 0.51855
 finished parsing run0000_0000 timepoint 0.5186
 finished parsing run0000_0000 timepoint 0.51865
 finished parsing run0000_0000 timepoint 0.5187
 finished parsing run0000_0000 timepoint 0.51875
 finished parsing run0000_0000 timepoint 0.5188
 finished parsing run0000_0000 timepoint 0.51885
 finished parsing run0000_0000 timepoint 0.5189
 finished parsing run0000_0000 timepoint 0.51895
 finished parsing run0000_0000 timepoint 0.519
 finished parsing run0000_0000 timepoint 0.51905
 finished parsing run0000_0000 timepoint 0.5191
 finished parsing run0000_0000 timepoint 0.51915
 finished parsing run0000_0000 timepoint 0.5192
 finished parsing run0000_0000 timepoint 0.51925
 finished parsing run0000_0000 timepoint 0.5193
 finished parsing run0000_0000 timepoint 0.51935
 finished parsing run0000_0000 timepoint 0.5194
 finished parsing run0000_0000 timepoint 0.51945
 finished parsing run0000_0000 timepoint 0.5195
 finished parsing run0000_0000 timepoint 0.51955
 finished parsing run0000_0000 timepoint 0.5196
 finished parsing run0000_0000 timepoint 0.51965
 finished parsing run0000_0000 timepoint 0.5197
 finished parsing run0000_0000 timepoint 0.51975
 finished parsing run0000_0000 timepoint 0.5198
 finished parsing run0000_0000 timepoint 0.51985
 finished parsing run0000_0000 timepoint 0.5199
 finished parsing run0000_0000 timepoint 0.51995
 finished parsing run0000_0000 timepoint 0.52
 finished parsing run0000_0000 timepoint 0.52005
 finished parsing run0000_0000 timepoint 0.5201
 finished parsing run0000_0000 timepoint 0.52015
 finished parsing run0000_0000 timepoint 0.5202
 finished parsing run0000_0000 timepoint 0.52025
 finished parsing run0000_0000 timepoint 0.5203
 finished parsing run0000_0000 timepoint 0.52035
 finished parsing run0000_0000 timepoint 0.5204
 finished parsing run0000_0000 timepoint 0.52045
 finished parsing run0000_0000 timepoint 0.5205
 finished parsing run0000_0000 timepoint 0.52055
 finished parsing run0000_0000 timepoint 0.5206
 finished parsing run0000_0000 timepoint 0.52065
 finished parsing run0000_0000 timepoint 0.5207
 finished parsing run0000_0000 timepoint 0.52075
 finished parsing run0000_0000 timepoint 0.5208
 finished parsing run0000_0000 timepoint 0.52085
 finished parsing run0000_0000 timepoint 0.5209
 finished parsing run0000_0000 timepoint 0.52095
 finished parsing run0000_0000 timepoint 0.521
 finished parsing run0000_0000 timepoint 0.52105
 finished parsing run0000_0000 timepoint 0.5211
 finished parsing run0000_0000 timepoint 0.52115
 finished parsing run0000_0000 timepoint 0.5212
 finished parsing run0000_0000 timepoint 0.52125
 finished parsing run0000_0000 timepoint 0.5213
 finished parsing run0000_0000 timepoint 0.52135
 finished parsing run0000_0000 timepoint 0.5214
 finished parsing run0000_0000 timepoint 0.52145
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.5087
 finished parsing run0000_0000 timepoint 0.50875
 finished parsing run0000_0000 timepoint 0.5088
 finished parsing run0000_0000 timepoint 0.50885
 finished parsing run0000_0000 timepoint 0.5089
 finished parsing run0000_0000 timepoint 0.50895
 finished parsing run0000_0000 timepoint 0.509
 finished parsing run0000_0000 timepoint 0.50905
 finished parsing run0000_0000 timepoint 0.5091
 finished parsing run0000_0000 timepoint 0.50915
 finished parsing run0000_0000 timepoint 0.5092
 finished parsing run0000_0000 timepoint 0.50925
 finished parsing run0000_0000 timepoint 0.5093
 finished parsing run0000_0000 timepoint 0.50935
 finished parsing run0000_0000 timepoint 0.5094
 finished parsing run0000_0000 timepoint 0.50945
 finished parsing run0000_0000 timepoint 0.5095
 finished parsing run0000_0000 timepoint 0.50955
 finished parsing run0000_0000 timepoint 0.5096
 finished parsing run0000_0000 timepoint 0.50965
 finished parsing run0000_0000 timepoint 0.5097
 finished parsing run0000_0000 timepoint 0.50975
 finished parsing run0000_0000 timepoint 0.5098
 finished parsing run0000_0000 timepoint 0.50985
 finished parsing run0000_0000 timepoint 0.5099
 finished parsing run0000_0000 timepoint 0.50995
 finished parsing run0000_0000 timepoint 0.51
 finished parsing run0000_0000 timepoint 0.51005
 finished parsing run0000_0000 timepoint 0.5101
 finished parsing run0000_0000 timepoint 0.51015
 finished parsing run0000_0000 timepoint 0.5102
 finished parsing run0000_0000 timepoint 0.51025
 finished parsing run0000_0000 timepoint 0.5103
 finished parsing run0000_0000 timepoint 0.51035
 finished parsing run0000_0000 timepoint 0.5104
 finished parsing run0000_0000 timepoint 0.51045
 finished parsing run0000_0000 timepoint 0.5105
 finished parsing run0000_0000 timepoint 0.51055
 finished parsing run0000_0000 timepoint 0.5106
 finished parsing run0000_0000 timepoint 0.51065
 finished parsing run0000_0000 timepoint 0.5107
 finished parsing run0000_0000 timepoint 0.51075
 finished parsing run0000_0000 timepoint 0.5108
 finished parsing run0000_0000 timepoint 0.51085
 finished parsing run0000_0000 timepoint 0.5109
 finished parsing run0000_0000 timepoint 0.51095
 finished parsing run0000_0000 timepoint 0.511
 finished parsing run0000_0000 timepoint 0.51105
 finished parsing run0000_0000 timepoint 0.5111
 finished parsing run0000_0000 timepoint 0.51115
 finished parsing run0000_0000 timepoint 0.5112
 finished parsing run0000_0000 timepoint 0.51125
 finished parsing run0000_0000 timepoint 0.5113
 finished parsing run0000_0000 timepoint 0.51135
 finished parsing run0000_0000 timepoint 0.5114
 finished parsing run0000_0000 timepoint 0.51145
 finished parsing run0000_0000 timepoint 0.5115
 finished parsing run0000_0000 timepoint 0.51155
 finished parsing run0000_0000 timepoint 0.5116
 finished parsing run0000_0000 timepoint 0.51165
 finished parsing run0000_0000 timepoint 0.5117
 finished parsing run0000_0000 timepoint 0.51175
 finished parsing run0000_0000 timepoint 0.5118
 finished parsing run0000_0000 timepoint 0.51185
 finished parsing run0000_0000 timepoint 0.5119
 finished parsing run0000_0000 timepoint 0.51195
 finished parsing run0000_0000 timepoint 0.512
 finished parsing run0000_0000 timepoint 0.51205
 finished parsing run0000_0000 timepoint 0.5121
 finished parsing run0000_0000 timepoint 0.51215
 finished parsing run0000_0000 timepoint 0.5122
 finished parsing run0000_0000 timepoint 0.51225
 finished parsing run0000_0000 timepoint 0.5123
 finished parsing run0000_0000 timepoint 0.51235
 finished parsing run0000_0000 timepoint 0.5124
 finished parsing run0000_0000 timepoint 0.51245
 finished parsing run0000_0000 timepoint 0.5125
 finished parsing run0000_0000 timepoint 0.51255
 finished parsing run0000_0000 timepoint 0.5126
 finished parsing run0000_0000 timepoint 0.51265
 finished parsing run0000_0000 timepoint 0.5127
 finished parsing run0000_0000 timepoint 0.51275
 finished parsing run0000_0000 timepoint 0.5128
 finished parsing run0000_0000 timepoint 0.51285
 finished parsing run0000_0000 timepoint 0.5129
 finished parsing run0000_0000 timepoint 0.51295
 finished parsing run0000_0000 timepoint 0.513
 finished parsing run0000_0000 timepoint 0.51305
 finished parsing run0000_0000 timepoint 0.5131
 finished parsing run0000_0000 timepoint 0.51315
 finished parsing run0000_0000 timepoint 0.5132
 finished parsing run0000_0000 timepoint 0.51325
 finished parsing run0000_0000 timepoint 0.5133
 finished parsing run0000_0000 timepoint 0.51335
 finished parsing run0000_0000 timepoint 0.5134
 finished parsing run0000_0000 timepoint 0.51345
 finished parsing run0000_0000 timepoint 0.5135
 finished parsing run0000_0000 timepoint 0.51355
 finished parsing run0000_0000 timepoint 0.5136
 finished parsing run0000_0000 timepoint 0.51365
 finished parsing run0000_0000 timepoint 0.5137
 finished parsing run0000_0000 timepoint 0.51375
 finished parsing run0000_0000 timepoint 0.5138
 finished parsing run0000_0000 timepoint 0.51385
 finished parsing run0000_0000 timepoint 0.5139
 finished parsing run0000_0000 timepoint 0.51395
 finished parsing run0000_0000 timepoint 0.514
 finished parsing run0000_0000 timepoint 0.51405
 finished parsing run0000_0000 timepoint 0.5141
 finished parsing run0000_0000 timepoint 0.51415
 finished parsing run0000_0000 timepoint 0.5142
 finished parsing run0000_0000 timepoint 0.51425
 finished parsing run0000_0000 timepoint 0.5143
 finished parsing run0000_0000 timepoint 0.51435
 finished parsing run0000_0000 timepoint 0.5144
 finished parsing run0000_0000 timepoint 0.51445
 finished parsing run0000_0000 timepoint 0.5145
 finished parsing run0000_0000 timepoint 0.51455
 finished parsing run0000_0000 timepoint 0.5146
 finished parsing run0000_0000 timepoint 0.51465
 finished parsing run0000_0000 timepoint 0.5147
 finished parsing run0000_0000 timepoint 0.51475
 finished parsing run0000_0000 timepoint 0.5148
 finished parsing run0000_0000 timepoint 0.51485
 finished parsing run0000_0000 timepoint 0.5149
 finished parsing run0000_0000 timepoint 0.51495
 finished parsing run0000_0000 timepoint 0.515
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.50225
 finished parsing run0000_0000 timepoint 0.5023
 finished parsing run0000_0000 timepoint 0.50235
 finished parsing run0000_0000 timepoint 0.5024
 finished parsing run0000_0000 timepoint 0.50245
 finished parsing run0000_0000 timepoint 0.5025
 finished parsing run0000_0000 timepoint 0.50255
 finished parsing run0000_0000 timepoint 0.5026
 finished parsing run0000_0000 timepoint 0.50265
 finished parsing run0000_0000 timepoint 0.5027
 finished parsing run0000_0000 timepoint 0.50275
 finished parsing run0000_0000 timepoint 0.5028
 finished parsing run0000_0000 timepoint 0.50285
 finished parsing run0000_0000 timepoint 0.5029
 finished parsing run0000_0000 timepoint 0.50295
 finished parsing run0000_0000 timepoint 0.503
 finished parsing run0000_0000 timepoint 0.50305
 finished parsing run0000_0000 timepoint 0.5031
 finished parsing run0000_0000 timepoint 0.50315
 finished parsing run0000_0000 timepoint 0.5032
 finished parsing run0000_0000 timepoint 0.50325
 finished parsing run0000_0000 timepoint 0.5033
 finished parsing run0000_0000 timepoint 0.50335
 finished parsing run0000_0000 timepoint 0.5034
 finished parsing run0000_0000 timepoint 0.50345
 finished parsing run0000_0000 timepoint 0.5035
 finished parsing run0000_0000 timepoint 0.50355
 finished parsing run0000_0000 timepoint 0.5036
 finished parsing run0000_0000 timepoint 0.50365
 finished parsing run0000_0000 timepoint 0.5037
 finished parsing run0000_0000 timepoint 0.50375
 finished parsing run0000_0000 timepoint 0.5038
 finished parsing run0000_0000 timepoint 0.50385
 finished parsing run0000_0000 timepoint 0.5039
 finished parsing run0000_0000 timepoint 0.50395
 finished parsing run0000_0000 timepoint 0.504
 finished parsing run0000_0000 timepoint 0.50405
 finished parsing run0000_0000 timepoint 0.5041
 finished parsing run0000_0000 timepoint 0.50415
 finished parsing run0000_0000 timepoint 0.5042
 finished parsing run0000_0000 timepoint 0.50425
 finished parsing run0000_0000 timepoint 0.5043
 finished parsing run0000_0000 timepoint 0.50435
 finished parsing run0000_0000 timepoint 0.5044
 finished parsing run0000_0000 timepoint 0.50445
 finished parsing run0000_0000 timepoint 0.5045
 finished parsing run0000_0000 timepoint 0.50455
 finished parsing run0000_0000 timepoint 0.5046
 finished parsing run0000_0000 timepoint 0.50465
 finished parsing run0000_0000 timepoint 0.5047
 finished parsing run0000_0000 timepoint 0.50475
 finished parsing run0000_0000 timepoint 0.5048
 finished parsing run0000_0000 timepoint 0.50485
 finished parsing run0000_0000 timepoint 0.5049
 finished parsing run0000_0000 timepoint 0.50495
 finished parsing run0000_0000 timepoint 0.505
 finished parsing run0000_0000 timepoint 0.50505
 finished parsing run0000_0000 timepoint 0.5051
 finished parsing run0000_0000 timepoint 0.50515
 finished parsing run0000_0000 timepoint 0.5052
 finished parsing run0000_0000 timepoint 0.50525
 finished parsing run0000_0000 timepoint 0.5053
 finished parsing run0000_0000 timepoint 0.50535
 finished parsing run0000_0000 timepoint 0.5054
 finished parsing run0000_0000 timepoint 0.50545
 finished parsing run0000_0000 timepoint 0.5055
 finished parsing run0000_0000 timepoint 0.50555
 finished parsing run0000_0000 timepoint 0.5056
 finished parsing run0000_0000 timepoint 0.50565
 finished parsing run0000_0000 timepoint 0.5057
 finished parsing run0000_0000 timepoint 0.50575
 finished parsing run0000_0000 timepoint 0.5058
 finished parsing run0000_0000 timepoint 0.50585
 finished parsing run0000_0000 timepoint 0.5059
 finished parsing run0000_0000 timepoint 0.50595
 finished parsing run0000_0000 timepoint 0.506
 finished parsing run0000_0000 timepoint 0.50605
 finished parsing run0000_0000 timepoint 0.5061
 finished parsing run0000_0000 timepoint 0.50615
 finished parsing run0000_0000 timepoint 0.5062
 finished parsing run0000_0000 timepoint 0.50625
 finished parsing run0000_0000 timepoint 0.5063
 finished parsing run0000_0000 timepoint 0.50635
 finished parsing run0000_0000 timepoint 0.5064
 finished parsing run0000_0000 timepoint 0.50645
 finished parsing run0000_0000 timepoint 0.5065
 finished parsing run0000_0000 timepoint 0.50655
 finished parsing run0000_0000 timepoint 0.5066
 finished parsing run0000_0000 timepoint 0.50665
 finished parsing run0000_0000 timepoint 0.5067
 finished parsing run0000_0000 timepoint 0.50675
 finished parsing run0000_0000 timepoint 0.5068
 finished parsing run0000_0000 timepoint 0.50685
 finished parsing run0000_0000 timepoint 0.5069
 finished parsing run0000_0000 timepoint 0.50695
 finished parsing run0000_0000 timepoint 0.507
 finished parsing run0000_0000 timepoint 0.50705
 finished parsing run0000_0000 timepoint 0.5071
 finished parsing run0000_0000 timepoint 0.50715
 finished parsing run0000_0000 timepoint 0.5072
 finished parsing run0000_0000 timepoint 0.50725
 finished parsing run0000_0000 timepoint 0.5073
 finished parsing run0000_0000 timepoint 0.50735
 finished parsing run0000_0000 timepoint 0.5074
 finished parsing run0000_0000 timepoint 0.50745
 finished parsing run0000_0000 timepoint 0.5075
 finished parsing run0000_0000 timepoint 0.50755
 finished parsing run0000_0000 timepoint 0.5076
 finished parsing run0000_0000 timepoint 0.50765
 finished parsing run0000_0000 timepoint 0.5077
 finished parsing run0000_0000 timepoint 0.50775
 finished parsing run0000_0000 timepoint 0.5078
 finished parsing run0000_0000 timepoint 0.50785
 finished parsing run0000_0000 timepoint 0.5079
 finished parsing run0000_0000 timepoint 0.50795
 finished parsing run0000_0000 timepoint 0.508
 finished parsing run0000_0000 timepoint 0.50805
 finished parsing run0000_0000 timepoint 0.5081
 finished parsing run0000_0000 timepoint 0.50815
 finished parsing run0000_0000 timepoint 0.5082
 finished parsing run0000_0000 timepoint 0.50825
 finished parsing run0000_0000 timepoint 0.5083
 finished parsing run0000_0000 timepoint 0.50835
 finished parsing run0000_0000 timepoint 0.5084
 finished parsing run0000_0000 timepoint 0.50845
 finished parsing run0000_0000 timepoint 0.5085
 finished parsing run0000_0000 timepoint 0.50855
 finished parsing run0000_0000 timepoint 0.5086
 finished parsing run0000_0000 timepoint 0.50865
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.49575
 finished parsing run0000_0000 timepoint 0.4958
 finished parsing run0000_0000 timepoint 0.49585
 finished parsing run0000_0000 timepoint 0.4959
 finished parsing run0000_0000 timepoint 0.49595
 finished parsing run0000_0000 timepoint 0.496
 finished parsing run0000_0000 timepoint 0.49605
 finished parsing run0000_0000 timepoint 0.4961
 finished parsing run0000_0000 timepoint 0.49615
 finished parsing run0000_0000 timepoint 0.4962
 finished parsing run0000_0000 timepoint 0.49625
 finished parsing run0000_0000 timepoint 0.4963
 finished parsing run0000_0000 timepoint 0.49635
 finished parsing run0000_0000 timepoint 0.4964
 finished parsing run0000_0000 timepoint 0.49645
 finished parsing run0000_0000 timepoint 0.4965
 finished parsing run0000_0000 timepoint 0.49655
 finished parsing run0000_0000 timepoint 0.4966
 finished parsing run0000_0000 timepoint 0.49665
 finished parsing run0000_0000 timepoint 0.4967
 finished parsing run0000_0000 timepoint 0.49675
 finished parsing run0000_0000 timepoint 0.4968
 finished parsing run0000_0000 timepoint 0.49685
 finished parsing run0000_0000 timepoint 0.4969
 finished parsing run0000_0000 timepoint 0.49695
 finished parsing run0000_0000 timepoint 0.497
 finished parsing run0000_0000 timepoint 0.49705
 finished parsing run0000_0000 timepoint 0.4971
 finished parsing run0000_0000 timepoint 0.49715
 finished parsing run0000_0000 timepoint 0.4972
 finished parsing run0000_0000 timepoint 0.49725
 finished parsing run0000_0000 timepoint 0.4973
 finished parsing run0000_0000 timepoint 0.49735
 finished parsing run0000_0000 timepoint 0.4974
 finished parsing run0000_0000 timepoint 0.49745
 finished parsing run0000_0000 timepoint 0.4975
 finished parsing run0000_0000 timepoint 0.49755
 finished parsing run0000_0000 timepoint 0.4976
 finished parsing run0000_0000 timepoint 0.49765
 finished parsing run0000_0000 timepoint 0.4977
 finished parsing run0000_0000 timepoint 0.49775
 finished parsing run0000_0000 timepoint 0.4978
 finished parsing run0000_0000 timepoint 0.49785
 finished parsing run0000_0000 timepoint 0.4979
 finished parsing run0000_0000 timepoint 0.49795
 finished parsing run0000_0000 timepoint 0.498
 finished parsing run0000_0000 timepoint 0.49805
 finished parsing run0000_0000 timepoint 0.4981
 finished parsing run0000_0000 timepoint 0.49815
 finished parsing run0000_0000 timepoint 0.4982
 finished parsing run0000_0000 timepoint 0.49825
 finished parsing run0000_0000 timepoint 0.4983
 finished parsing run0000_0000 timepoint 0.49835
 finished parsing run0000_0000 timepoint 0.4984
 finished parsing run0000_0000 timepoint 0.49845
 finished parsing run0000_0000 timepoint 0.4985
 finished parsing run0000_0000 timepoint 0.49855
 finished parsing run0000_0000 timepoint 0.4986
 finished parsing run0000_0000 timepoint 0.49865
 finished parsing run0000_0000 timepoint 0.4987
 finished parsing run0000_0000 timepoint 0.49875
 finished parsing run0000_0000 timepoint 0.4988
 finished parsing run0000_0000 timepoint 0.49885
 finished parsing run0000_0000 timepoint 0.4989
 finished parsing run0000_0000 timepoint 0.49895
 finished parsing run0000_0000 timepoint 0.499
 finished parsing run0000_0000 timepoint 0.49905
 finished parsing run0000_0000 timepoint 0.4991
 finished parsing run0000_0000 timepoint 0.49915
 finished parsing run0000_0000 timepoint 0.4992
 finished parsing run0000_0000 timepoint 0.49925
 finished parsing run0000_0000 timepoint 0.4993
 finished parsing run0000_0000 timepoint 0.49935
 finished parsing run0000_0000 timepoint 0.4994
 finished parsing run0000_0000 timepoint 0.49945
 finished parsing run0000_0000 timepoint 0.4995
 finished parsing run0000_0000 timepoint 0.49955
 finished parsing run0000_0000 timepoint 0.4996
 finished parsing run0000_0000 timepoint 0.49965
 finished parsing run0000_0000 timepoint 0.4997
 finished parsing run0000_0000 timepoint 0.49975
 finished parsing run0000_0000 timepoint 0.4998
 finished parsing run0000_0000 timepoint 0.49985
 finished parsing run0000_0000 timepoint 0.4999
 finished parsing run0000_0000 timepoint 0.49995
 finished parsing run0000_0000 timepoint 0.5
 finished parsing run0000_0000 timepoint 0.50005
 finished parsing run0000_0000 timepoint 0.5001
 finished parsing run0000_0000 timepoint 0.50015
 finished parsing run0000_0000 timepoint 0.5002
 finished parsing run0000_0000 timepoint 0.50025
 finished parsing run0000_0000 timepoint 0.5003
 finished parsing run0000_0000 timepoint 0.50035
 finished parsing run0000_0000 timepoint 0.5004
 finished parsing run0000_0000 timepoint 0.50045
 finished parsing run0000_0000 timepoint 0.5005
 finished parsing run0000_0000 timepoint 0.50055
 finished parsing run0000_0000 timepoint 0.5006
 finished parsing run0000_0000 timepoint 0.50065
 finished parsing run0000_0000 timepoint 0.5007
 finished parsing run0000_0000 timepoint 0.50075
 finished parsing run0000_0000 timepoint 0.5008
 finished parsing run0000_0000 timepoint 0.50085
 finished parsing run0000_0000 timepoint 0.5009
 finished parsing run0000_0000 timepoint 0.50095
 finished parsing run0000_0000 timepoint 0.501
 finished parsing run0000_0000 timepoint 0.50105
 finished parsing run0000_0000 timepoint 0.5011
 finished parsing run0000_0000 timepoint 0.50115
 finished parsing run0000_0000 timepoint 0.5012
 finished parsing run0000_0000 timepoint 0.50125
 finished parsing run0000_0000 timepoint 0.5013
 finished parsing run0000_0000 timepoint 0.50135
 finished parsing run0000_0000 timepoint 0.5014
 finished parsing run0000_0000 timepoint 0.50145
 finished parsing run0000_0000 timepoint 0.5015
 finished parsing run0000_0000 timepoint 0.50155
 finished parsing run0000_0000 timepoint 0.5016
 finished parsing run0000_0000 timepoint 0.50165
 finished parsing run0000_0000 timepoint 0.5017
 finished parsing run0000_0000 timepoint 0.50175
 finished parsing run0000_0000 timepoint 0.5018
 finished parsing run0000_0000 timepoint 0.50185
 finished parsing run0000_0000 timepoint 0.5019
 finished parsing run0000_0000 timepoint 0.50195
 finished parsing run0000_0000 timepoint 0.502
 finished parsing run0000_0000 timepoint 0.50205
 finished parsing run0000_0000 timepoint 0.5021
 finished parsing run0000_0000 timepoint 0.50215
 finished parsing run0000_0000 timepoint 0.5022
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4891
 finished parsing run0000_0000 timepoint 0.48915
 finished parsing run0000_0000 timepoint 0.4892
 finished parsing run0000_0000 timepoint 0.48925
 finished parsing run0000_0000 timepoint 0.4893
 finished parsing run0000_0000 timepoint 0.48935
 finished parsing run0000_0000 timepoint 0.4894
 finished parsing run0000_0000 timepoint 0.48945
 finished parsing run0000_0000 timepoint 0.4895
 finished parsing run0000_0000 timepoint 0.48955
 finished parsing run0000_0000 timepoint 0.4896
 finished parsing run0000_0000 timepoint 0.48965
 finished parsing run0000_0000 timepoint 0.4897
 finished parsing run0000_0000 timepoint 0.48975
 finished parsing run0000_0000 timepoint 0.4898
 finished parsing run0000_0000 timepoint 0.48985
 finished parsing run0000_0000 timepoint 0.4899
 finished parsing run0000_0000 timepoint 0.48995
 finished parsing run0000_0000 timepoint 0.49
 finished parsing run0000_0000 timepoint 0.49005
 finished parsing run0000_0000 timepoint 0.4901
 finished parsing run0000_0000 timepoint 0.49015
 finished parsing run0000_0000 timepoint 0.4902
 finished parsing run0000_0000 timepoint 0.49025
 finished parsing run0000_0000 timepoint 0.4903
 finished parsing run0000_0000 timepoint 0.49035
 finished parsing run0000_0000 timepoint 0.4904
 finished parsing run0000_0000 timepoint 0.49045
 finished parsing run0000_0000 timepoint 0.4905
 finished parsing run0000_0000 timepoint 0.49055
 finished parsing run0000_0000 timepoint 0.4906
 finished parsing run0000_0000 timepoint 0.49065
 finished parsing run0000_0000 timepoint 0.4907
 finished parsing run0000_0000 timepoint 0.49075
 finished parsing run0000_0000 timepoint 0.4908
 finished parsing run0000_0000 timepoint 0.49085
 finished parsing run0000_0000 timepoint 0.4909
 finished parsing run0000_0000 timepoint 0.49095
 finished parsing run0000_0000 timepoint 0.491
 finished parsing run0000_0000 timepoint 0.49105
 finished parsing run0000_0000 timepoint 0.4911
 finished parsing run0000_0000 timepoint 0.49115
 finished parsing run0000_0000 timepoint 0.4912
 finished parsing run0000_0000 timepoint 0.49125
 finished parsing run0000_0000 timepoint 0.4913
 finished parsing run0000_0000 timepoint 0.49135
 finished parsing run0000_0000 timepoint 0.4914
 finished parsing run0000_0000 timepoint 0.49145
 finished parsing run0000_0000 timepoint 0.4915
 finished parsing run0000_0000 timepoint 0.49155
 finished parsing run0000_0000 timepoint 0.4916
 finished parsing run0000_0000 timepoint 0.49165
 finished parsing run0000_0000 timepoint 0.4917
 finished parsing run0000_0000 timepoint 0.49175
 finished parsing run0000_0000 timepoint 0.4918
 finished parsing run0000_0000 timepoint 0.49185
 finished parsing run0000_0000 timepoint 0.4919
 finished parsing run0000_0000 timepoint 0.49195
 finished parsing run0000_0000 timepoint 0.492
 finished parsing run0000_0000 timepoint 0.49205
 finished parsing run0000_0000 timepoint 0.4921
 finished parsing run0000_0000 timepoint 0.49215
 finished parsing run0000_0000 timepoint 0.4922
 finished parsing run0000_0000 timepoint 0.49225
 finished parsing run0000_0000 timepoint 0.4923
 finished parsing run0000_0000 timepoint 0.49235
 finished parsing run0000_0000 timepoint 0.4924
 finished parsing run0000_0000 timepoint 0.49245
 finished parsing run0000_0000 timepoint 0.4925
 finished parsing run0000_0000 timepoint 0.49255
 finished parsing run0000_0000 timepoint 0.4926
 finished parsing run0000_0000 timepoint 0.49265
 finished parsing run0000_0000 timepoint 0.4927
 finished parsing run0000_0000 timepoint 0.49275
 finished parsing run0000_0000 timepoint 0.4928
 finished parsing run0000_0000 timepoint 0.49285
 finished parsing run0000_0000 timepoint 0.4929
 finished parsing run0000_0000 timepoint 0.49295
 finished parsing run0000_0000 timepoint 0.493
 finished parsing run0000_0000 timepoint 0.49305
 finished parsing run0000_0000 timepoint 0.4931
 finished parsing run0000_0000 timepoint 0.49315
 finished parsing run0000_0000 timepoint 0.4932
 finished parsing run0000_0000 timepoint 0.49325
 finished parsing run0000_0000 timepoint 0.4933
 finished parsing run0000_0000 timepoint 0.49335
 finished parsing run0000_0000 timepoint 0.4934
 finished parsing run0000_0000 timepoint 0.49345
 finished parsing run0000_0000 timepoint 0.4935
 finished parsing run0000_0000 timepoint 0.49355
 finished parsing run0000_0000 timepoint 0.4936
 finished parsing run0000_0000 timepoint 0.49365
 finished parsing run0000_0000 timepoint 0.4937
 finished parsing run0000_0000 timepoint 0.49375
 finished parsing run0000_0000 timepoint 0.4938
 finished parsing run0000_0000 timepoint 0.49385
 finished parsing run0000_0000 timepoint 0.4939
 finished parsing run0000_0000 timepoint 0.49395
 finished parsing run0000_0000 timepoint 0.494
 finished parsing run0000_0000 timepoint 0.49405
 finished parsing run0000_0000 timepoint 0.4941
 finished parsing run0000_0000 timepoint 0.49415
 finished parsing run0000_0000 timepoint 0.4942
 finished parsing run0000_0000 timepoint 0.49425
 finished parsing run0000_0000 timepoint 0.4943
 finished parsing run0000_0000 timepoint 0.49435
 finished parsing run0000_0000 timepoint 0.4944
 finished parsing run0000_0000 timepoint 0.49445
 finished parsing run0000_0000 timepoint 0.4945
 finished parsing run0000_0000 timepoint 0.49455
 finished parsing run0000_0000 timepoint 0.4946
 finished parsing run0000_0000 timepoint 0.49465
 finished parsing run0000_0000 timepoint 0.4947
 finished parsing run0000_0000 timepoint 0.49475
 finished parsing run0000_0000 timepoint 0.4948
 finished parsing run0000_0000 timepoint 0.49485
 finished parsing run0000_0000 timepoint 0.4949
 finished parsing run0000_0000 timepoint 0.49495
 finished parsing run0000_0000 timepoint 0.495
 finished parsing run0000_0000 timepoint 0.49505
 finished parsing run0000_0000 timepoint 0.4951
 finished parsing run0000_0000 timepoint 0.49515
 finished parsing run0000_0000 timepoint 0.4952
 finished parsing run0000_0000 timepoint 0.49525
 finished parsing run0000_0000 timepoint 0.4953
 finished parsing run0000_0000 timepoint 0.49535
 finished parsing run0000_0000 timepoint 0.4954
 finished parsing run0000_0000 timepoint 0.49545
 finished parsing run0000_0000 timepoint 0.4955
 finished parsing run0000_0000 timepoint 0.49555
 finished parsing run0000_0000 timepoint 0.4956
 finished parsing run0000_0000 timepoint 0.49565
 finished parsing run0000_0000 timepoint 0.4957
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.48255
 finished parsing run0000_0000 timepoint 0.4826
 finished parsing run0000_0000 timepoint 0.48265
 finished parsing run0000_0000 timepoint 0.4827
 finished parsing run0000_0000 timepoint 0.48275
 finished parsing run0000_0000 timepoint 0.4828
 finished parsing run0000_0000 timepoint 0.48285
 finished parsing run0000_0000 timepoint 0.4829
 finished parsing run0000_0000 timepoint 0.48295
 finished parsing run0000_0000 timepoint 0.483
 finished parsing run0000_0000 timepoint 0.48305
 finished parsing run0000_0000 timepoint 0.4831
 finished parsing run0000_0000 timepoint 0.48315
 finished parsing run0000_0000 timepoint 0.4832
 finished parsing run0000_0000 timepoint 0.48325
 finished parsing run0000_0000 timepoint 0.4833
 finished parsing run0000_0000 timepoint 0.48335
 finished parsing run0000_0000 timepoint 0.4834
 finished parsing run0000_0000 timepoint 0.48345
 finished parsing run0000_0000 timepoint 0.4835
 finished parsing run0000_0000 timepoint 0.48355
 finished parsing run0000_0000 timepoint 0.4836
 finished parsing run0000_0000 timepoint 0.48365
 finished parsing run0000_0000 timepoint 0.4837
 finished parsing run0000_0000 timepoint 0.48375
 finished parsing run0000_0000 timepoint 0.4838
 finished parsing run0000_0000 timepoint 0.48385
 finished parsing run0000_0000 timepoint 0.4839
 finished parsing run0000_0000 timepoint 0.48395
 finished parsing run0000_0000 timepoint 0.484
 finished parsing run0000_0000 timepoint 0.48405
 finished parsing run0000_0000 timepoint 0.4841
 finished parsing run0000_0000 timepoint 0.48415
 finished parsing run0000_0000 timepoint 0.4842
 finished parsing run0000_0000 timepoint 0.48425
 finished parsing run0000_0000 timepoint 0.4843
 finished parsing run0000_0000 timepoint 0.48435
 finished parsing run0000_0000 timepoint 0.4844
 finished parsing run0000_0000 timepoint 0.48445
 finished parsing run0000_0000 timepoint 0.4845
 finished parsing run0000_0000 timepoint 0.48455
 finished parsing run0000_0000 timepoint 0.4846
 finished parsing run0000_0000 timepoint 0.48465
 finished parsing run0000_0000 timepoint 0.4847
 finished parsing run0000_0000 timepoint 0.48475
 finished parsing run0000_0000 timepoint 0.4848
 finished parsing run0000_0000 timepoint 0.48485
 finished parsing run0000_0000 timepoint 0.4849
 finished parsing run0000_0000 timepoint 0.48495
 finished parsing run0000_0000 timepoint 0.485
 finished parsing run0000_0000 timepoint 0.48505
 finished parsing run0000_0000 timepoint 0.4851
 finished parsing run0000_0000 timepoint 0.48515
 finished parsing run0000_0000 timepoint 0.4852
 finished parsing run0000_0000 timepoint 0.48525
 finished parsing run0000_0000 timepoint 0.4853
 finished parsing run0000_0000 timepoint 0.48535
 finished parsing run0000_0000 timepoint 0.4854
 finished parsing run0000_0000 timepoint 0.48545
 finished parsing run0000_0000 timepoint 0.4855
 finished parsing run0000_0000 timepoint 0.48555
 finished parsing run0000_0000 timepoint 0.4856
 finished parsing run0000_0000 timepoint 0.48565
 finished parsing run0000_0000 timepoint 0.4857
 finished parsing run0000_0000 timepoint 0.48575
 finished parsing run0000_0000 timepoint 0.4858
 finished parsing run0000_0000 timepoint 0.48585
 finished parsing run0000_0000 timepoint 0.4859
 finished parsing run0000_0000 timepoint 0.48595
 finished parsing run0000_0000 timepoint 0.486
 finished parsing run0000_0000 timepoint 0.48605
 finished parsing run0000_0000 timepoint 0.4861
 finished parsing run0000_0000 timepoint 0.48615
 finished parsing run0000_0000 timepoint 0.4862
 finished parsing run0000_0000 timepoint 0.48625
 finished parsing run0000_0000 timepoint 0.4863
 finished parsing run0000_0000 timepoint 0.48635
 finished parsing run0000_0000 timepoint 0.4864
 finished parsing run0000_0000 timepoint 0.48645
 finished parsing run0000_0000 timepoint 0.4865
 finished parsing run0000_0000 timepoint 0.48655
 finished parsing run0000_0000 timepoint 0.4866
 finished parsing run0000_0000 timepoint 0.48665
 finished parsing run0000_0000 timepoint 0.4867
 finished parsing run0000_0000 timepoint 0.48675
 finished parsing run0000_0000 timepoint 0.4868
 finished parsing run0000_0000 timepoint 0.48685
 finished parsing run0000_0000 timepoint 0.4869
 finished parsing run0000_0000 timepoint 0.48695
 finished parsing run0000_0000 timepoint 0.487
 finished parsing run0000_0000 timepoint 0.48705
 finished parsing run0000_0000 timepoint 0.4871
 finished parsing run0000_0000 timepoint 0.48715
 finished parsing run0000_0000 timepoint 0.4872
 finished parsing run0000_0000 timepoint 0.48725
 finished parsing run0000_0000 timepoint 0.4873
 finished parsing run0000_0000 timepoint 0.48735
 finished parsing run0000_0000 timepoint 0.4874
 finished parsing run0000_0000 timepoint 0.48745
 finished parsing run0000_0000 timepoint 0.4875
 finished parsing run0000_0000 timepoint 0.48755
 finished parsing run0000_0000 timepoint 0.4876
 finished parsing run0000_0000 timepoint 0.48765
 finished parsing run0000_0000 timepoint 0.4877
 finished parsing run0000_0000 timepoint 0.48775
 finished parsing run0000_0000 timepoint 0.4878
 finished parsing run0000_0000 timepoint 0.48785
 finished parsing run0000_0000 timepoint 0.4879
 finished parsing run0000_0000 timepoint 0.48795
 finished parsing run0000_0000 timepoint 0.488
 finished parsing run0000_0000 timepoint 0.48805
 finished parsing run0000_0000 timepoint 0.4881
 finished parsing run0000_0000 timepoint 0.48815
 finished parsing run0000_0000 timepoint 0.4882
 finished parsing run0000_0000 timepoint 0.48825
 finished parsing run0000_0000 timepoint 0.4883
 finished parsing run0000_0000 timepoint 0.48835
 finished parsing run0000_0000 timepoint 0.4884
 finished parsing run0000_0000 timepoint 0.48845
 finished parsing run0000_0000 timepoint 0.4885
 finished parsing run0000_0000 timepoint 0.48855
 finished parsing run0000_0000 timepoint 0.4886
 finished parsing run0000_0000 timepoint 0.48865
 finished parsing run0000_0000 timepoint 0.4887
 finished parsing run0000_0000 timepoint 0.48875
 finished parsing run0000_0000 timepoint 0.4888
 finished parsing run0000_0000 timepoint 0.48885
 finished parsing run0000_0000 timepoint 0.4889
 finished parsing run0000_0000 timepoint 0.48895
 finished parsing run0000_0000 timepoint 0.489
 finished parsing run0000_0000 timepoint 0.48905
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.47615
 finished parsing run0000_0000 timepoint 0.4762
 finished parsing run0000_0000 timepoint 0.47625
 finished parsing run0000_0000 timepoint 0.4763
 finished parsing run0000_0000 timepoint 0.47635
 finished parsing run0000_0000 timepoint 0.4764
 finished parsing run0000_0000 timepoint 0.47645
 finished parsing run0000_0000 timepoint 0.4765
 finished parsing run0000_0000 timepoint 0.47655
 finished parsing run0000_0000 timepoint 0.4766
 finished parsing run0000_0000 timepoint 0.47665
 finished parsing run0000_0000 timepoint 0.4767
 finished parsing run0000_0000 timepoint 0.47675
 finished parsing run0000_0000 timepoint 0.4768
 finished parsing run0000_0000 timepoint 0.47685
 finished parsing run0000_0000 timepoint 0.4769
 finished parsing run0000_0000 timepoint 0.47695
 finished parsing run0000_0000 timepoint 0.477
 finished parsing run0000_0000 timepoint 0.47705
 finished parsing run0000_0000 timepoint 0.4771
 finished parsing run0000_0000 timepoint 0.47715
 finished parsing run0000_0000 timepoint 0.4772
 finished parsing run0000_0000 timepoint 0.47725
 finished parsing run0000_0000 timepoint 0.4773
 finished parsing run0000_0000 timepoint 0.47735
 finished parsing run0000_0000 timepoint 0.4774
 finished parsing run0000_0000 timepoint 0.47745
 finished parsing run0000_0000 timepoint 0.4775
 finished parsing run0000_0000 timepoint 0.47755
 finished parsing run0000_0000 timepoint 0.4776
 finished parsing run0000_0000 timepoint 0.47765
 finished parsing run0000_0000 timepoint 0.4777
 finished parsing run0000_0000 timepoint 0.47775
 finished parsing run0000_0000 timepoint 0.4778
 finished parsing run0000_0000 timepoint 0.47785
 finished parsing run0000_0000 timepoint 0.4779
 finished parsing run0000_0000 timepoint 0.47795
 finished parsing run0000_0000 timepoint 0.478
 finished parsing run0000_0000 timepoint 0.47805
 finished parsing run0000_0000 timepoint 0.4781
 finished parsing run0000_0000 timepoint 0.47815
 finished parsing run0000_0000 timepoint 0.4782
 finished parsing run0000_0000 timepoint 0.47825
 finished parsing run0000_0000 timepoint 0.4783
 finished parsing run0000_0000 timepoint 0.47835
 finished parsing run0000_0000 timepoint 0.4784
 finished parsing run0000_0000 timepoint 0.47845
 finished parsing run0000_0000 timepoint 0.4785
 finished parsing run0000_0000 timepoint 0.47855
 finished parsing run0000_0000 timepoint 0.4786
 finished parsing run0000_0000 timepoint 0.47865
 finished parsing run0000_0000 timepoint 0.4787
 finished parsing run0000_0000 timepoint 0.47875
 finished parsing run0000_0000 timepoint 0.4788
 finished parsing run0000_0000 timepoint 0.47885
 finished parsing run0000_0000 timepoint 0.4789
 finished parsing run0000_0000 timepoint 0.47895
 finished parsing run0000_0000 timepoint 0.479
 finished parsing run0000_0000 timepoint 0.47905
 finished parsing run0000_0000 timepoint 0.4791
 finished parsing run0000_0000 timepoint 0.47915
 finished parsing run0000_0000 timepoint 0.4792
 finished parsing run0000_0000 timepoint 0.47925
 finished parsing run0000_0000 timepoint 0.4793
 finished parsing run0000_0000 timepoint 0.47935
 finished parsing run0000_0000 timepoint 0.4794
 finished parsing run0000_0000 timepoint 0.47945
 finished parsing run0000_0000 timepoint 0.4795
 finished parsing run0000_0000 timepoint 0.47955
 finished parsing run0000_0000 timepoint 0.4796
 finished parsing run0000_0000 timepoint 0.47965
 finished parsing run0000_0000 timepoint 0.4797
 finished parsing run0000_0000 timepoint 0.47975
 finished parsing run0000_0000 timepoint 0.4798
 finished parsing run0000_0000 timepoint 0.47985
 finished parsing run0000_0000 timepoint 0.4799
 finished parsing run0000_0000 timepoint 0.47995
 finished parsing run0000_0000 timepoint 0.48
 finished parsing run0000_0000 timepoint 0.48005
 finished parsing run0000_0000 timepoint 0.4801
 finished parsing run0000_0000 timepoint 0.48015
 finished parsing run0000_0000 timepoint 0.4802
 finished parsing run0000_0000 timepoint 0.48025
 finished parsing run0000_0000 timepoint 0.4803
 finished parsing run0000_0000 timepoint 0.48035
 finished parsing run0000_0000 timepoint 0.4804
 finished parsing run0000_0000 timepoint 0.48045
 finished parsing run0000_0000 timepoint 0.4805
 finished parsing run0000_0000 timepoint 0.48055
 finished parsing run0000_0000 timepoint 0.4806
 finished parsing run0000_0000 timepoint 0.48065
 finished parsing run0000_0000 timepoint 0.4807
 finished parsing run0000_0000 timepoint 0.48075
 finished parsing run0000_0000 timepoint 0.4808
 finished parsing run0000_0000 timepoint 0.48085
 finished parsing run0000_0000 timepoint 0.4809
 finished parsing run0000_0000 timepoint 0.48095
 finished parsing run0000_0000 timepoint 0.481
 finished parsing run0000_0000 timepoint 0.48105
 finished parsing run0000_0000 timepoint 0.4811
 finished parsing run0000_0000 timepoint 0.48115
 finished parsing run0000_0000 timepoint 0.4812
 finished parsing run0000_0000 timepoint 0.48125
 finished parsing run0000_0000 timepoint 0.4813
 finished parsing run0000_0000 timepoint 0.48135
 finished parsing run0000_0000 timepoint 0.4814
 finished parsing run0000_0000 timepoint 0.48145
 finished parsing run0000_0000 timepoint 0.4815
 finished parsing run0000_0000 timepoint 0.48155
 finished parsing run0000_0000 timepoint 0.4816
 finished parsing run0000_0000 timepoint 0.48165
 finished parsing run0000_0000 timepoint 0.4817
 finished parsing run0000_0000 timepoint 0.48175
 finished parsing run0000_0000 timepoint 0.4818
 finished parsing run0000_0000 timepoint 0.48185
 finished parsing run0000_0000 timepoint 0.4819
 finished parsing run0000_0000 timepoint 0.48195
 finished parsing run0000_0000 timepoint 0.482
 finished parsing run0000_0000 timepoint 0.48205
 finished parsing run0000_0000 timepoint 0.4821
 finished parsing run0000_0000 timepoint 0.48215
 finished parsing run0000_0000 timepoint 0.4822
 finished parsing run0000_0000 timepoint 0.48225
 finished parsing run0000_0000 timepoint 0.4823
 finished parsing run0000_0000 timepoint 0.48235
 finished parsing run0000_0000 timepoint 0.4824
 finished parsing run0000_0000 timepoint 0.48245
 finished parsing run0000_0000 timepoint 0.4825
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.46975
 finished parsing run0000_0000 timepoint 0.4698
 finished parsing run0000_0000 timepoint 0.46985
 finished parsing run0000_0000 timepoint 0.4699
 finished parsing run0000_0000 timepoint 0.46995
 finished parsing run0000_0000 timepoint 0.47
 finished parsing run0000_0000 timepoint 0.47005
 finished parsing run0000_0000 timepoint 0.4701
 finished parsing run0000_0000 timepoint 0.47015
 finished parsing run0000_0000 timepoint 0.4702
 finished parsing run0000_0000 timepoint 0.47025
 finished parsing run0000_0000 timepoint 0.4703
 finished parsing run0000_0000 timepoint 0.47035
 finished parsing run0000_0000 timepoint 0.4704
 finished parsing run0000_0000 timepoint 0.47045
 finished parsing run0000_0000 timepoint 0.4705
 finished parsing run0000_0000 timepoint 0.47055
 finished parsing run0000_0000 timepoint 0.4706
 finished parsing run0000_0000 timepoint 0.47065
 finished parsing run0000_0000 timepoint 0.4707
 finished parsing run0000_0000 timepoint 0.47075
 finished parsing run0000_0000 timepoint 0.4708
 finished parsing run0000_0000 timepoint 0.47085
 finished parsing run0000_0000 timepoint 0.4709
 finished parsing run0000_0000 timepoint 0.47095
 finished parsing run0000_0000 timepoint 0.471
 finished parsing run0000_0000 timepoint 0.47105
 finished parsing run0000_0000 timepoint 0.4711
 finished parsing run0000_0000 timepoint 0.47115
 finished parsing run0000_0000 timepoint 0.4712
 finished parsing run0000_0000 timepoint 0.47125
 finished parsing run0000_0000 timepoint 0.4713
 finished parsing run0000_0000 timepoint 0.47135
 finished parsing run0000_0000 timepoint 0.4714
 finished parsing run0000_0000 timepoint 0.47145
 finished parsing run0000_0000 timepoint 0.4715
 finished parsing run0000_0000 timepoint 0.47155
 finished parsing run0000_0000 timepoint 0.4716
 finished parsing run0000_0000 timepoint 0.47165
 finished parsing run0000_0000 timepoint 0.4717
 finished parsing run0000_0000 timepoint 0.47175
 finished parsing run0000_0000 timepoint 0.4718
 finished parsing run0000_0000 timepoint 0.47185
 finished parsing run0000_0000 timepoint 0.4719
 finished parsing run0000_0000 timepoint 0.47195
 finished parsing run0000_0000 timepoint 0.472
 finished parsing run0000_0000 timepoint 0.47205
 finished parsing run0000_0000 timepoint 0.4721
 finished parsing run0000_0000 timepoint 0.47215
 finished parsing run0000_0000 timepoint 0.4722
 finished parsing run0000_0000 timepoint 0.47225
 finished parsing run0000_0000 timepoint 0.4723
 finished parsing run0000_0000 timepoint 0.47235
 finished parsing run0000_0000 timepoint 0.4724
 finished parsing run0000_0000 timepoint 0.47245
 finished parsing run0000_0000 timepoint 0.4725
 finished parsing run0000_0000 timepoint 0.47255
 finished parsing run0000_0000 timepoint 0.4726
 finished parsing run0000_0000 timepoint 0.47265
 finished parsing run0000_0000 timepoint 0.4727
 finished parsing run0000_0000 timepoint 0.47275
 finished parsing run0000_0000 timepoint 0.4728
 finished parsing run0000_0000 timepoint 0.47285
 finished parsing run0000_0000 timepoint 0.4729
 finished parsing run0000_0000 timepoint 0.47295
 finished parsing run0000_0000 timepoint 0.473
 finished parsing run0000_0000 timepoint 0.47305
 finished parsing run0000_0000 timepoint 0.4731
 finished parsing run0000_0000 timepoint 0.47315
 finished parsing run0000_0000 timepoint 0.4732
 finished parsing run0000_0000 timepoint 0.47325
 finished parsing run0000_0000 timepoint 0.4733
 finished parsing run0000_0000 timepoint 0.47335
 finished parsing run0000_0000 timepoint 0.4734
 finished parsing run0000_0000 timepoint 0.47345
 finished parsing run0000_0000 timepoint 0.4735
 finished parsing run0000_0000 timepoint 0.47355
 finished parsing run0000_0000 timepoint 0.4736
 finished parsing run0000_0000 timepoint 0.47365
 finished parsing run0000_0000 timepoint 0.4737
 finished parsing run0000_0000 timepoint 0.47375
 finished parsing run0000_0000 timepoint 0.4738
 finished parsing run0000_0000 timepoint 0.47385
 finished parsing run0000_0000 timepoint 0.4739
 finished parsing run0000_0000 timepoint 0.47395
 finished parsing run0000_0000 timepoint 0.474
 finished parsing run0000_0000 timepoint 0.47405
 finished parsing run0000_0000 timepoint 0.4741
 finished parsing run0000_0000 timepoint 0.47415
 finished parsing run0000_0000 timepoint 0.4742
 finished parsing run0000_0000 timepoint 0.47425
 finished parsing run0000_0000 timepoint 0.4743
 finished parsing run0000_0000 timepoint 0.47435
 finished parsing run0000_0000 timepoint 0.4744
 finished parsing run0000_0000 timepoint 0.47445
 finished parsing run0000_0000 timepoint 0.4745
 finished parsing run0000_0000 timepoint 0.47455
 finished parsing run0000_0000 timepoint 0.4746
 finished parsing run0000_0000 timepoint 0.47465
 finished parsing run0000_0000 timepoint 0.4747
 finished parsing run0000_0000 timepoint 0.47475
 finished parsing run0000_0000 timepoint 0.4748
 finished parsing run0000_0000 timepoint 0.47485
 finished parsing run0000_0000 timepoint 0.4749
 finished parsing run0000_0000 timepoint 0.47495
 finished parsing run0000_0000 timepoint 0.475
 finished parsing run0000_0000 timepoint 0.47505
 finished parsing run0000_0000 timepoint 0.4751
 finished parsing run0000_0000 timepoint 0.47515
 finished parsing run0000_0000 timepoint 0.4752
 finished parsing run0000_0000 timepoint 0.47525
 finished parsing run0000_0000 timepoint 0.4753
 finished parsing run0000_0000 timepoint 0.47535
 finished parsing run0000_0000 timepoint 0.4754
 finished parsing run0000_0000 timepoint 0.47545
 finished parsing run0000_0000 timepoint 0.4755
 finished parsing run0000_0000 timepoint 0.47555
 finished parsing run0000_0000 timepoint 0.4756
 finished parsing run0000_0000 timepoint 0.47565
 finished parsing run0000_0000 timepoint 0.4757
 finished parsing run0000_0000 timepoint 0.47575
 finished parsing run0000_0000 timepoint 0.4758
 finished parsing run0000_0000 timepoint 0.47585
 finished parsing run0000_0000 timepoint 0.4759
 finished parsing run0000_0000 timepoint 0.47595
 finished parsing run0000_0000 timepoint 0.476
 finished parsing run0000_0000 timepoint 0.47605
 finished parsing run0000_0000 timepoint 0.4761
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4631
 finished parsing run0000_0000 timepoint 0.46315
 finished parsing run0000_0000 timepoint 0.4632
 finished parsing run0000_0000 timepoint 0.46325
 finished parsing run0000_0000 timepoint 0.4633
 finished parsing run0000_0000 timepoint 0.46335
 finished parsing run0000_0000 timepoint 0.4634
 finished parsing run0000_0000 timepoint 0.46345
 finished parsing run0000_0000 timepoint 0.4635
 finished parsing run0000_0000 timepoint 0.46355
 finished parsing run0000_0000 timepoint 0.4636
 finished parsing run0000_0000 timepoint 0.46365
 finished parsing run0000_0000 timepoint 0.4637
 finished parsing run0000_0000 timepoint 0.46375
 finished parsing run0000_0000 timepoint 0.4638
 finished parsing run0000_0000 timepoint 0.46385
 finished parsing run0000_0000 timepoint 0.4639
 finished parsing run0000_0000 timepoint 0.46395
 finished parsing run0000_0000 timepoint 0.464
 finished parsing run0000_0000 timepoint 0.46405
 finished parsing run0000_0000 timepoint 0.4641
 finished parsing run0000_0000 timepoint 0.46415
 finished parsing run0000_0000 timepoint 0.4642
 finished parsing run0000_0000 timepoint 0.46425
 finished parsing run0000_0000 timepoint 0.4643
 finished parsing run0000_0000 timepoint 0.46435
 finished parsing run0000_0000 timepoint 0.4644
 finished parsing run0000_0000 timepoint 0.46445
 finished parsing run0000_0000 timepoint 0.4645
 finished parsing run0000_0000 timepoint 0.46455
 finished parsing run0000_0000 timepoint 0.4646
 finished parsing run0000_0000 timepoint 0.46465
 finished parsing run0000_0000 timepoint 0.4647
 finished parsing run0000_0000 timepoint 0.46475
 finished parsing run0000_0000 timepoint 0.4648
 finished parsing run0000_0000 timepoint 0.46485
 finished parsing run0000_0000 timepoint 0.4649
 finished parsing run0000_0000 timepoint 0.46495
 finished parsing run0000_0000 timepoint 0.465
 finished parsing run0000_0000 timepoint 0.46505
 finished parsing run0000_0000 timepoint 0.4651
 finished parsing run0000_0000 timepoint 0.46515
 finished parsing run0000_0000 timepoint 0.4652
 finished parsing run0000_0000 timepoint 0.46525
 finished parsing run0000_0000 timepoint 0.4653
 finished parsing run0000_0000 timepoint 0.46535
 finished parsing run0000_0000 timepoint 0.4654
 finished parsing run0000_0000 timepoint 0.46545
 finished parsing run0000_0000 timepoint 0.4655
 finished parsing run0000_0000 timepoint 0.46555
 finished parsing run0000_0000 timepoint 0.4656
 finished parsing run0000_0000 timepoint 0.46565
 finished parsing run0000_0000 timepoint 0.4657
 finished parsing run0000_0000 timepoint 0.46575
 finished parsing run0000_0000 timepoint 0.4658
 finished parsing run0000_0000 timepoint 0.46585
 finished parsing run0000_0000 timepoint 0.4659
 finished parsing run0000_0000 timepoint 0.46595
 finished parsing run0000_0000 timepoint 0.466
 finished parsing run0000_0000 timepoint 0.46605
 finished parsing run0000_0000 timepoint 0.4661
 finished parsing run0000_0000 timepoint 0.46615
 finished parsing run0000_0000 timepoint 0.4662
 finished parsing run0000_0000 timepoint 0.46625
 finished parsing run0000_0000 timepoint 0.4663
 finished parsing run0000_0000 timepoint 0.46635
 finished parsing run0000_0000 timepoint 0.4664
 finished parsing run0000_0000 timepoint 0.46645
 finished parsing run0000_0000 timepoint 0.4665
 finished parsing run0000_0000 timepoint 0.46655
 finished parsing run0000_0000 timepoint 0.4666
 finished parsing run0000_0000 timepoint 0.46665
 finished parsing run0000_0000 timepoint 0.4667
 finished parsing run0000_0000 timepoint 0.46675
 finished parsing run0000_0000 timepoint 0.4668
 finished parsing run0000_0000 timepoint 0.46685
 finished parsing run0000_0000 timepoint 0.4669
 finished parsing run0000_0000 timepoint 0.46695
 finished parsing run0000_0000 timepoint 0.467
 finished parsing run0000_0000 timepoint 0.46705
 finished parsing run0000_0000 timepoint 0.4671
 finished parsing run0000_0000 timepoint 0.46715
 finished parsing run0000_0000 timepoint 0.4672
 finished parsing run0000_0000 timepoint 0.46725
 finished parsing run0000_0000 timepoint 0.4673
 finished parsing run0000_0000 timepoint 0.46735
 finished parsing run0000_0000 timepoint 0.4674
 finished parsing run0000_0000 timepoint 0.46745
 finished parsing run0000_0000 timepoint 0.4675
 finished parsing run0000_0000 timepoint 0.46755
 finished parsing run0000_0000 timepoint 0.4676
 finished parsing run0000_0000 timepoint 0.46765
 finished parsing run0000_0000 timepoint 0.4677
 finished parsing run0000_0000 timepoint 0.46775
 finished parsing run0000_0000 timepoint 0.4678
 finished parsing run0000_0000 timepoint 0.46785
 finished parsing run0000_0000 timepoint 0.4679
 finished parsing run0000_0000 timepoint 0.46795
 finished parsing run0000_0000 timepoint 0.468
 finished parsing run0000_0000 timepoint 0.46805
 finished parsing run0000_0000 timepoint 0.4681
 finished parsing run0000_0000 timepoint 0.46815
 finished parsing run0000_0000 timepoint 0.4682
 finished parsing run0000_0000 timepoint 0.46825
 finished parsing run0000_0000 timepoint 0.4683
 finished parsing run0000_0000 timepoint 0.46835
 finished parsing run0000_0000 timepoint 0.4684
 finished parsing run0000_0000 timepoint 0.46845
 finished parsing run0000_0000 timepoint 0.4685
 finished parsing run0000_0000 timepoint 0.46855
 finished parsing run0000_0000 timepoint 0.4686
 finished parsing run0000_0000 timepoint 0.46865
 finished parsing run0000_0000 timepoint 0.4687
 finished parsing run0000_0000 timepoint 0.46875
 finished parsing run0000_0000 timepoint 0.4688
 finished parsing run0000_0000 timepoint 0.46885
 finished parsing run0000_0000 timepoint 0.4689
 finished parsing run0000_0000 timepoint 0.46895
 finished parsing run0000_0000 timepoint 0.469
 finished parsing run0000_0000 timepoint 0.46905
 finished parsing run0000_0000 timepoint 0.4691
 finished parsing run0000_0000 timepoint 0.46915
 finished parsing run0000_0000 timepoint 0.4692
 finished parsing run0000_0000 timepoint 0.46925
 finished parsing run0000_0000 timepoint 0.4693
 finished parsing run0000_0000 timepoint 0.46935
 finished parsing run0000_0000 timepoint 0.4694
 finished parsing run0000_0000 timepoint 0.46945
 finished parsing run0000_0000 timepoint 0.4695
 finished parsing run0000_0000 timepoint 0.46955
 finished parsing run0000_0000 timepoint 0.4696
 finished parsing run0000_0000 timepoint 0.46965
 finished parsing run0000_0000 timepoint 0.4697
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4565
 finished parsing run0000_0000 timepoint 0.45655
 finished parsing run0000_0000 timepoint 0.4566
 finished parsing run0000_0000 timepoint 0.45665
 finished parsing run0000_0000 timepoint 0.4567
 finished parsing run0000_0000 timepoint 0.45675
 finished parsing run0000_0000 timepoint 0.4568
 finished parsing run0000_0000 timepoint 0.45685
 finished parsing run0000_0000 timepoint 0.4569
 finished parsing run0000_0000 timepoint 0.45695
 finished parsing run0000_0000 timepoint 0.457
 finished parsing run0000_0000 timepoint 0.45705
 finished parsing run0000_0000 timepoint 0.4571
 finished parsing run0000_0000 timepoint 0.45715
 finished parsing run0000_0000 timepoint 0.4572
 finished parsing run0000_0000 timepoint 0.45725
 finished parsing run0000_0000 timepoint 0.4573
 finished parsing run0000_0000 timepoint 0.45735
 finished parsing run0000_0000 timepoint 0.4574
 finished parsing run0000_0000 timepoint 0.45745
 finished parsing run0000_0000 timepoint 0.4575
 finished parsing run0000_0000 timepoint 0.45755
 finished parsing run0000_0000 timepoint 0.4576
 finished parsing run0000_0000 timepoint 0.45765
 finished parsing run0000_0000 timepoint 0.4577
 finished parsing run0000_0000 timepoint 0.45775
 finished parsing run0000_0000 timepoint 0.4578
 finished parsing run0000_0000 timepoint 0.45785
 finished parsing run0000_0000 timepoint 0.4579
 finished parsing run0000_0000 timepoint 0.45795
 finished parsing run0000_0000 timepoint 0.458
 finished parsing run0000_0000 timepoint 0.45805
 finished parsing run0000_0000 timepoint 0.4581
 finished parsing run0000_0000 timepoint 0.45815
 finished parsing run0000_0000 timepoint 0.4582
 finished parsing run0000_0000 timepoint 0.45825
 finished parsing run0000_0000 timepoint 0.4583
 finished parsing run0000_0000 timepoint 0.45835
 finished parsing run0000_0000 timepoint 0.4584
 finished parsing run0000_0000 timepoint 0.45845
 finished parsing run0000_0000 timepoint 0.4585
 finished parsing run0000_0000 timepoint 0.45855
 finished parsing run0000_0000 timepoint 0.4586
 finished parsing run0000_0000 timepoint 0.45865
 finished parsing run0000_0000 timepoint 0.4587
 finished parsing run0000_0000 timepoint 0.45875
 finished parsing run0000_0000 timepoint 0.4588
 finished parsing run0000_0000 timepoint 0.45885
 finished parsing run0000_0000 timepoint 0.4589
 finished parsing run0000_0000 timepoint 0.45895
 finished parsing run0000_0000 timepoint 0.459
 finished parsing run0000_0000 timepoint 0.45905
 finished parsing run0000_0000 timepoint 0.4591
 finished parsing run0000_0000 timepoint 0.45915
 finished parsing run0000_0000 timepoint 0.4592
 finished parsing run0000_0000 timepoint 0.45925
 finished parsing run0000_0000 timepoint 0.4593
 finished parsing run0000_0000 timepoint 0.45935
 finished parsing run0000_0000 timepoint 0.4594
 finished parsing run0000_0000 timepoint 0.45945
 finished parsing run0000_0000 timepoint 0.4595
 finished parsing run0000_0000 timepoint 0.45955
 finished parsing run0000_0000 timepoint 0.4596
 finished parsing run0000_0000 timepoint 0.45965
 finished parsing run0000_0000 timepoint 0.4597
 finished parsing run0000_0000 timepoint 0.45975
 finished parsing run0000_0000 timepoint 0.4598
 finished parsing run0000_0000 timepoint 0.45985
 finished parsing run0000_0000 timepoint 0.4599
 finished parsing run0000_0000 timepoint 0.45995
 finished parsing run0000_0000 timepoint 0.46
 finished parsing run0000_0000 timepoint 0.46005
 finished parsing run0000_0000 timepoint 0.4601
 finished parsing run0000_0000 timepoint 0.46015
 finished parsing run0000_0000 timepoint 0.4602
 finished parsing run0000_0000 timepoint 0.46025
 finished parsing run0000_0000 timepoint 0.4603
 finished parsing run0000_0000 timepoint 0.46035
 finished parsing run0000_0000 timepoint 0.4604
 finished parsing run0000_0000 timepoint 0.46045
 finished parsing run0000_0000 timepoint 0.4605
 finished parsing run0000_0000 timepoint 0.46055
 finished parsing run0000_0000 timepoint 0.4606
 finished parsing run0000_0000 timepoint 0.46065
 finished parsing run0000_0000 timepoint 0.4607
 finished parsing run0000_0000 timepoint 0.46075
 finished parsing run0000_0000 timepoint 0.4608
 finished parsing run0000_0000 timepoint 0.46085
 finished parsing run0000_0000 timepoint 0.4609
 finished parsing run0000_0000 timepoint 0.46095
 finished parsing run0000_0000 timepoint 0.461
 finished parsing run0000_0000 timepoint 0.46105
 finished parsing run0000_0000 timepoint 0.4611
 finished parsing run0000_0000 timepoint 0.46115
 finished parsing run0000_0000 timepoint 0.4612
 finished parsing run0000_0000 timepoint 0.46125
 finished parsing run0000_0000 timepoint 0.4613
 finished parsing run0000_0000 timepoint 0.46135
 finished parsing run0000_0000 timepoint 0.4614
 finished parsing run0000_0000 timepoint 0.46145
 finished parsing run0000_0000 timepoint 0.4615
 finished parsing run0000_0000 timepoint 0.46155
 finished parsing run0000_0000 timepoint 0.4616
 finished parsing run0000_0000 timepoint 0.46165
 finished parsing run0000_0000 timepoint 0.4617
 finished parsing run0000_0000 timepoint 0.46175
 finished parsing run0000_0000 timepoint 0.4618
 finished parsing run0000_0000 timepoint 0.46185
 finished parsing run0000_0000 timepoint 0.4619
 finished parsing run0000_0000 timepoint 0.46195
 finished parsing run0000_0000 timepoint 0.462
 finished parsing run0000_0000 timepoint 0.46205
 finished parsing run0000_0000 timepoint 0.4621
 finished parsing run0000_0000 timepoint 0.46215
 finished parsing run0000_0000 timepoint 0.4622
 finished parsing run0000_0000 timepoint 0.46225
 finished parsing run0000_0000 timepoint 0.4623
 finished parsing run0000_0000 timepoint 0.46235
 finished parsing run0000_0000 timepoint 0.4624
 finished parsing run0000_0000 timepoint 0.46245
 finished parsing run0000_0000 timepoint 0.4625
 finished parsing run0000_0000 timepoint 0.46255
 finished parsing run0000_0000 timepoint 0.4626
 finished parsing run0000_0000 timepoint 0.46265
 finished parsing run0000_0000 timepoint 0.4627
 finished parsing run0000_0000 timepoint 0.46275
 finished parsing run0000_0000 timepoint 0.4628
 finished parsing run0000_0000 timepoint 0.46285
 finished parsing run0000_0000 timepoint 0.4629
 finished parsing run0000_0000 timepoint 0.46295
 finished parsing run0000_0000 timepoint 0.463
 finished parsing run0000_0000 timepoint 0.46305
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.44995
 finished parsing run0000_0000 timepoint 0.45
 finished parsing run0000_0000 timepoint 0.45005
 finished parsing run0000_0000 timepoint 0.4501
 finished parsing run0000_0000 timepoint 0.45015
 finished parsing run0000_0000 timepoint 0.4502
 finished parsing run0000_0000 timepoint 0.45025
 finished parsing run0000_0000 timepoint 0.4503
 finished parsing run0000_0000 timepoint 0.45035
 finished parsing run0000_0000 timepoint 0.4504
 finished parsing run0000_0000 timepoint 0.45045
 finished parsing run0000_0000 timepoint 0.4505
 finished parsing run0000_0000 timepoint 0.45055
 finished parsing run0000_0000 timepoint 0.4506
 finished parsing run0000_0000 timepoint 0.45065
 finished parsing run0000_0000 timepoint 0.4507
 finished parsing run0000_0000 timepoint 0.45075
 finished parsing run0000_0000 timepoint 0.4508
 finished parsing run0000_0000 timepoint 0.45085
 finished parsing run0000_0000 timepoint 0.4509
 finished parsing run0000_0000 timepoint 0.45095
 finished parsing run0000_0000 timepoint 0.451
 finished parsing run0000_0000 timepoint 0.45105
 finished parsing run0000_0000 timepoint 0.4511
 finished parsing run0000_0000 timepoint 0.45115
 finished parsing run0000_0000 timepoint 0.4512
 finished parsing run0000_0000 timepoint 0.45125
 finished parsing run0000_0000 timepoint 0.4513
 finished parsing run0000_0000 timepoint 0.45135
 finished parsing run0000_0000 timepoint 0.4514
 finished parsing run0000_0000 timepoint 0.45145
 finished parsing run0000_0000 timepoint 0.4515
 finished parsing run0000_0000 timepoint 0.45155
 finished parsing run0000_0000 timepoint 0.4516
 finished parsing run0000_0000 timepoint 0.45165
 finished parsing run0000_0000 timepoint 0.4517
 finished parsing run0000_0000 timepoint 0.45175
 finished parsing run0000_0000 timepoint 0.4518
 finished parsing run0000_0000 timepoint 0.45185
 finished parsing run0000_0000 timepoint 0.4519
 finished parsing run0000_0000 timepoint 0.45195
 finished parsing run0000_0000 timepoint 0.452
 finished parsing run0000_0000 timepoint 0.45205
 finished parsing run0000_0000 timepoint 0.4521
 finished parsing run0000_0000 timepoint 0.45215
 finished parsing run0000_0000 timepoint 0.4522
 finished parsing run0000_0000 timepoint 0.45225
 finished parsing run0000_0000 timepoint 0.4523
 finished parsing run0000_0000 timepoint 0.45235
 finished parsing run0000_0000 timepoint 0.4524
 finished parsing run0000_0000 timepoint 0.45245
 finished parsing run0000_0000 timepoint 0.4525
 finished parsing run0000_0000 timepoint 0.45255
 finished parsing run0000_0000 timepoint 0.4526
 finished parsing run0000_0000 timepoint 0.45265
 finished parsing run0000_0000 timepoint 0.4527
 finished parsing run0000_0000 timepoint 0.45275
 finished parsing run0000_0000 timepoint 0.4528
 finished parsing run0000_0000 timepoint 0.45285
 finished parsing run0000_0000 timepoint 0.4529
 finished parsing run0000_0000 timepoint 0.45295
 finished parsing run0000_0000 timepoint 0.453
 finished parsing run0000_0000 timepoint 0.45305
 finished parsing run0000_0000 timepoint 0.4531
 finished parsing run0000_0000 timepoint 0.45315
 finished parsing run0000_0000 timepoint 0.4532
 finished parsing run0000_0000 timepoint 0.45325
 finished parsing run0000_0000 timepoint 0.4533
 finished parsing run0000_0000 timepoint 0.45335
 finished parsing run0000_0000 timepoint 0.4534
 finished parsing run0000_0000 timepoint 0.45345
 finished parsing run0000_0000 timepoint 0.4535
 finished parsing run0000_0000 timepoint 0.45355
 finished parsing run0000_0000 timepoint 0.4536
 finished parsing run0000_0000 timepoint 0.45365
 finished parsing run0000_0000 timepoint 0.4537
 finished parsing run0000_0000 timepoint 0.45375
 finished parsing run0000_0000 timepoint 0.4538
 finished parsing run0000_0000 timepoint 0.45385
 finished parsing run0000_0000 timepoint 0.4539
 finished parsing run0000_0000 timepoint 0.45395
 finished parsing run0000_0000 timepoint 0.454
 finished parsing run0000_0000 timepoint 0.45405
 finished parsing run0000_0000 timepoint 0.4541
 finished parsing run0000_0000 timepoint 0.45415
 finished parsing run0000_0000 timepoint 0.4542
 finished parsing run0000_0000 timepoint 0.45425
 finished parsing run0000_0000 timepoint 0.4543
 finished parsing run0000_0000 timepoint 0.45435
 finished parsing run0000_0000 timepoint 0.4544
 finished parsing run0000_0000 timepoint 0.45445
 finished parsing run0000_0000 timepoint 0.4545
 finished parsing run0000_0000 timepoint 0.45455
 finished parsing run0000_0000 timepoint 0.4546
 finished parsing run0000_0000 timepoint 0.45465
 finished parsing run0000_0000 timepoint 0.4547
 finished parsing run0000_0000 timepoint 0.45475
 finished parsing run0000_0000 timepoint 0.4548
 finished parsing run0000_0000 timepoint 0.45485
 finished parsing run0000_0000 timepoint 0.4549
 finished parsing run0000_0000 timepoint 0.45495
 finished parsing run0000_0000 timepoint 0.455
 finished parsing run0000_0000 timepoint 0.45505
 finished parsing run0000_0000 timepoint 0.4551
 finished parsing run0000_0000 timepoint 0.45515
 finished parsing run0000_0000 timepoint 0.4552
 finished parsing run0000_0000 timepoint 0.45525
 finished parsing run0000_0000 timepoint 0.4553
 finished parsing run0000_0000 timepoint 0.45535
 finished parsing run0000_0000 timepoint 0.4554
 finished parsing run0000_0000 timepoint 0.45545
 finished parsing run0000_0000 timepoint 0.4555
 finished parsing run0000_0000 timepoint 0.45555
 finished parsing run0000_0000 timepoint 0.4556
 finished parsing run0000_0000 timepoint 0.45565
 finished parsing run0000_0000 timepoint 0.4557
 finished parsing run0000_0000 timepoint 0.45575
 finished parsing run0000_0000 timepoint 0.4558
 finished parsing run0000_0000 timepoint 0.45585
 finished parsing run0000_0000 timepoint 0.4559
 finished parsing run0000_0000 timepoint 0.45595
 finished parsing run0000_0000 timepoint 0.456
 finished parsing run0000_0000 timepoint 0.45605
 finished parsing run0000_0000 timepoint 0.4561
 finished parsing run0000_0000 timepoint 0.45615
 finished parsing run0000_0000 timepoint 0.4562
 finished parsing run0000_0000 timepoint 0.45625
 finished parsing run0000_0000 timepoint 0.4563
 finished parsing run0000_0000 timepoint 0.45635
 finished parsing run0000_0000 timepoint 0.4564
 finished parsing run0000_0000 timepoint 0.45645
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.44335
 finished parsing run0000_0000 timepoint 0.4434
 finished parsing run0000_0000 timepoint 0.44345
 finished parsing run0000_0000 timepoint 0.4435
 finished parsing run0000_0000 timepoint 0.44355
 finished parsing run0000_0000 timepoint 0.4436
 finished parsing run0000_0000 timepoint 0.44365
 finished parsing run0000_0000 timepoint 0.4437
 finished parsing run0000_0000 timepoint 0.44375
 finished parsing run0000_0000 timepoint 0.4438
 finished parsing run0000_0000 timepoint 0.44385
 finished parsing run0000_0000 timepoint 0.4439
 finished parsing run0000_0000 timepoint 0.44395
 finished parsing run0000_0000 timepoint 0.444
 finished parsing run0000_0000 timepoint 0.44405
 finished parsing run0000_0000 timepoint 0.4441
 finished parsing run0000_0000 timepoint 0.44415
 finished parsing run0000_0000 timepoint 0.4442
 finished parsing run0000_0000 timepoint 0.44425
 finished parsing run0000_0000 timepoint 0.4443
 finished parsing run0000_0000 timepoint 0.44435
 finished parsing run0000_0000 timepoint 0.4444
 finished parsing run0000_0000 timepoint 0.44445
 finished parsing run0000_0000 timepoint 0.4445
 finished parsing run0000_0000 timepoint 0.44455
 finished parsing run0000_0000 timepoint 0.4446
 finished parsing run0000_0000 timepoint 0.44465
 finished parsing run0000_0000 timepoint 0.4447
 finished parsing run0000_0000 timepoint 0.44475
 finished parsing run0000_0000 timepoint 0.4448
 finished parsing run0000_0000 timepoint 0.44485
 finished parsing run0000_0000 timepoint 0.4449
 finished parsing run0000_0000 timepoint 0.44495
 finished parsing run0000_0000 timepoint 0.445
 finished parsing run0000_0000 timepoint 0.44505
 finished parsing run0000_0000 timepoint 0.4451
 finished parsing run0000_0000 timepoint 0.44515
 finished parsing run0000_0000 timepoint 0.4452
 finished parsing run0000_0000 timepoint 0.44525
 finished parsing run0000_0000 timepoint 0.4453
 finished parsing run0000_0000 timepoint 0.44535
 finished parsing run0000_0000 timepoint 0.4454
 finished parsing run0000_0000 timepoint 0.44545
 finished parsing run0000_0000 timepoint 0.4455
 finished parsing run0000_0000 timepoint 0.44555
 finished parsing run0000_0000 timepoint 0.4456
 finished parsing run0000_0000 timepoint 0.44565
 finished parsing run0000_0000 timepoint 0.4457
 finished parsing run0000_0000 timepoint 0.44575
 finished parsing run0000_0000 timepoint 0.4458
 finished parsing run0000_0000 timepoint 0.44585
 finished parsing run0000_0000 timepoint 0.4459
 finished parsing run0000_0000 timepoint 0.44595
 finished parsing run0000_0000 timepoint 0.446
 finished parsing run0000_0000 timepoint 0.44605
 finished parsing run0000_0000 timepoint 0.4461
 finished parsing run0000_0000 timepoint 0.44615
 finished parsing run0000_0000 timepoint 0.4462
 finished parsing run0000_0000 timepoint 0.44625
 finished parsing run0000_0000 timepoint 0.4463
 finished parsing run0000_0000 timepoint 0.44635
 finished parsing run0000_0000 timepoint 0.4464
 finished parsing run0000_0000 timepoint 0.44645
 finished parsing run0000_0000 timepoint 0.4465
 finished parsing run0000_0000 timepoint 0.44655
 finished parsing run0000_0000 timepoint 0.4466
 finished parsing run0000_0000 timepoint 0.44665
 finished parsing run0000_0000 timepoint 0.4467
 finished parsing run0000_0000 timepoint 0.44675
 finished parsing run0000_0000 timepoint 0.4468
 finished parsing run0000_0000 timepoint 0.44685
 finished parsing run0000_0000 timepoint 0.4469
 finished parsing run0000_0000 timepoint 0.44695
 finished parsing run0000_0000 timepoint 0.447
 finished parsing run0000_0000 timepoint 0.44705
 finished parsing run0000_0000 timepoint 0.4471
 finished parsing run0000_0000 timepoint 0.44715
 finished parsing run0000_0000 timepoint 0.4472
 finished parsing run0000_0000 timepoint 0.44725
 finished parsing run0000_0000 timepoint 0.4473
 finished parsing run0000_0000 timepoint 0.44735
 finished parsing run0000_0000 timepoint 0.4474
 finished parsing run0000_0000 timepoint 0.44745
 finished parsing run0000_0000 timepoint 0.4475
 finished parsing run0000_0000 timepoint 0.44755
 finished parsing run0000_0000 timepoint 0.4476
 finished parsing run0000_0000 timepoint 0.44765
 finished parsing run0000_0000 timepoint 0.4477
 finished parsing run0000_0000 timepoint 0.44775
 finished parsing run0000_0000 timepoint 0.4478
 finished parsing run0000_0000 timepoint 0.44785
 finished parsing run0000_0000 timepoint 0.4479
 finished parsing run0000_0000 timepoint 0.44795
 finished parsing run0000_0000 timepoint 0.448
 finished parsing run0000_0000 timepoint 0.44805
 finished parsing run0000_0000 timepoint 0.4481
 finished parsing run0000_0000 timepoint 0.44815
 finished parsing run0000_0000 timepoint 0.4482
 finished parsing run0000_0000 timepoint 0.44825
 finished parsing run0000_0000 timepoint 0.4483
 finished parsing run0000_0000 timepoint 0.44835
 finished parsing run0000_0000 timepoint 0.4484
 finished parsing run0000_0000 timepoint 0.44845
 finished parsing run0000_0000 timepoint 0.4485
 finished parsing run0000_0000 timepoint 0.44855
 finished parsing run0000_0000 timepoint 0.4486
 finished parsing run0000_0000 timepoint 0.44865
 finished parsing run0000_0000 timepoint 0.4487
 finished parsing run0000_0000 timepoint 0.44875
 finished parsing run0000_0000 timepoint 0.4488
 finished parsing run0000_0000 timepoint 0.44885
 finished parsing run0000_0000 timepoint 0.4489
 finished parsing run0000_0000 timepoint 0.44895
 finished parsing run0000_0000 timepoint 0.449
 finished parsing run0000_0000 timepoint 0.44905
 finished parsing run0000_0000 timepoint 0.4491
 finished parsing run0000_0000 timepoint 0.44915
 finished parsing run0000_0000 timepoint 0.4492
 finished parsing run0000_0000 timepoint 0.44925
 finished parsing run0000_0000 timepoint 0.4493
 finished parsing run0000_0000 timepoint 0.44935
 finished parsing run0000_0000 timepoint 0.4494
 finished parsing run0000_0000 timepoint 0.44945
 finished parsing run0000_0000 timepoint 0.4495
 finished parsing run0000_0000 timepoint 0.44955
 finished parsing run0000_0000 timepoint 0.4496
 finished parsing run0000_0000 timepoint 0.44965
 finished parsing run0000_0000 timepoint 0.4497
 finished parsing run0000_0000 timepoint 0.44975
 finished parsing run0000_0000 timepoint 0.4498
 finished parsing run0000_0000 timepoint 0.44985
 finished parsing run0000_0000 timepoint 0.4499
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4369
 finished parsing run0000_0000 timepoint 0.43695
 finished parsing run0000_0000 timepoint 0.437
 finished parsing run0000_0000 timepoint 0.43705
 finished parsing run0000_0000 timepoint 0.4371
 finished parsing run0000_0000 timepoint 0.43715
 finished parsing run0000_0000 timepoint 0.4372
 finished parsing run0000_0000 timepoint 0.43725
 finished parsing run0000_0000 timepoint 0.4373
 finished parsing run0000_0000 timepoint 0.43735
 finished parsing run0000_0000 timepoint 0.4374
 finished parsing run0000_0000 timepoint 0.43745
 finished parsing run0000_0000 timepoint 0.4375
 finished parsing run0000_0000 timepoint 0.43755
 finished parsing run0000_0000 timepoint 0.4376
 finished parsing run0000_0000 timepoint 0.43765
 finished parsing run0000_0000 timepoint 0.4377
 finished parsing run0000_0000 timepoint 0.43775
 finished parsing run0000_0000 timepoint 0.4378
 finished parsing run0000_0000 timepoint 0.43785
 finished parsing run0000_0000 timepoint 0.4379
 finished parsing run0000_0000 timepoint 0.43795
 finished parsing run0000_0000 timepoint 0.438
 finished parsing run0000_0000 timepoint 0.43805
 finished parsing run0000_0000 timepoint 0.4381
 finished parsing run0000_0000 timepoint 0.43815
 finished parsing run0000_0000 timepoint 0.4382
 finished parsing run0000_0000 timepoint 0.43825
 finished parsing run0000_0000 timepoint 0.4383
 finished parsing run0000_0000 timepoint 0.43835
 finished parsing run0000_0000 timepoint 0.4384
 finished parsing run0000_0000 timepoint 0.43845
 finished parsing run0000_0000 timepoint 0.4385
 finished parsing run0000_0000 timepoint 0.43855
 finished parsing run0000_0000 timepoint 0.4386
 finished parsing run0000_0000 timepoint 0.43865
 finished parsing run0000_0000 timepoint 0.4387
 finished parsing run0000_0000 timepoint 0.43875
 finished parsing run0000_0000 timepoint 0.4388
 finished parsing run0000_0000 timepoint 0.43885
 finished parsing run0000_0000 timepoint 0.4389
 finished parsing run0000_0000 timepoint 0.43895
 finished parsing run0000_0000 timepoint 0.439
 finished parsing run0000_0000 timepoint 0.43905
 finished parsing run0000_0000 timepoint 0.4391
 finished parsing run0000_0000 timepoint 0.43915
 finished parsing run0000_0000 timepoint 0.4392
 finished parsing run0000_0000 timepoint 0.43925
 finished parsing run0000_0000 timepoint 0.4393
 finished parsing run0000_0000 timepoint 0.43935
 finished parsing run0000_0000 timepoint 0.4394
 finished parsing run0000_0000 timepoint 0.43945
 finished parsing run0000_0000 timepoint 0.4395
 finished parsing run0000_0000 timepoint 0.43955
 finished parsing run0000_0000 timepoint 0.4396
 finished parsing run0000_0000 timepoint 0.43965
 finished parsing run0000_0000 timepoint 0.4397
 finished parsing run0000_0000 timepoint 0.43975
 finished parsing run0000_0000 timepoint 0.4398
 finished parsing run0000_0000 timepoint 0.43985
 finished parsing run0000_0000 timepoint 0.4399
 finished parsing run0000_0000 timepoint 0.43995
 finished parsing run0000_0000 timepoint 0.44
 finished parsing run0000_0000 timepoint 0.44005
 finished parsing run0000_0000 timepoint 0.4401
 finished parsing run0000_0000 timepoint 0.44015
 finished parsing run0000_0000 timepoint 0.4402
 finished parsing run0000_0000 timepoint 0.44025
 finished parsing run0000_0000 timepoint 0.4403
 finished parsing run0000_0000 timepoint 0.44035
 finished parsing run0000_0000 timepoint 0.4404
 finished parsing run0000_0000 timepoint 0.44045
 finished parsing run0000_0000 timepoint 0.4405
 finished parsing run0000_0000 timepoint 0.44055
 finished parsing run0000_0000 timepoint 0.4406
 finished parsing run0000_0000 timepoint 0.44065
 finished parsing run0000_0000 timepoint 0.4407
 finished parsing run0000_0000 timepoint 0.44075
 finished parsing run0000_0000 timepoint 0.4408
 finished parsing run0000_0000 timepoint 0.44085
 finished parsing run0000_0000 timepoint 0.4409
 finished parsing run0000_0000 timepoint 0.44095
 finished parsing run0000_0000 timepoint 0.441
 finished parsing run0000_0000 timepoint 0.44105
 finished parsing run0000_0000 timepoint 0.4411
 finished parsing run0000_0000 timepoint 0.44115
 finished parsing run0000_0000 timepoint 0.4412
 finished parsing run0000_0000 timepoint 0.44125
 finished parsing run0000_0000 timepoint 0.4413
 finished parsing run0000_0000 timepoint 0.44135
 finished parsing run0000_0000 timepoint 0.4414
 finished parsing run0000_0000 timepoint 0.44145
 finished parsing run0000_0000 timepoint 0.4415
 finished parsing run0000_0000 timepoint 0.44155
 finished parsing run0000_0000 timepoint 0.4416
 finished parsing run0000_0000 timepoint 0.44165
 finished parsing run0000_0000 timepoint 0.4417
 finished parsing run0000_0000 timepoint 0.44175
 finished parsing run0000_0000 timepoint 0.4418
 finished parsing run0000_0000 timepoint 0.44185
 finished parsing run0000_0000 timepoint 0.4419
 finished parsing run0000_0000 timepoint 0.44195
 finished parsing run0000_0000 timepoint 0.442
 finished parsing run0000_0000 timepoint 0.44205
 finished parsing run0000_0000 timepoint 0.4421
 finished parsing run0000_0000 timepoint 0.44215
 finished parsing run0000_0000 timepoint 0.4422
 finished parsing run0000_0000 timepoint 0.44225
 finished parsing run0000_0000 timepoint 0.4423
 finished parsing run0000_0000 timepoint 0.44235
 finished parsing run0000_0000 timepoint 0.4424
 finished parsing run0000_0000 timepoint 0.44245
 finished parsing run0000_0000 timepoint 0.4425
 finished parsing run0000_0000 timepoint 0.44255
 finished parsing run0000_0000 timepoint 0.4426
 finished parsing run0000_0000 timepoint 0.44265
 finished parsing run0000_0000 timepoint 0.4427
 finished parsing run0000_0000 timepoint 0.44275
 finished parsing run0000_0000 timepoint 0.4428
 finished parsing run0000_0000 timepoint 0.44285
 finished parsing run0000_0000 timepoint 0.4429
 finished parsing run0000_0000 timepoint 0.44295
 finished parsing run0000_0000 timepoint 0.443
 finished parsing run0000_0000 timepoint 0.44305
 finished parsing run0000_0000 timepoint 0.4431
 finished parsing run0000_0000 timepoint 0.44315
 finished parsing run0000_0000 timepoint 0.4432
 finished parsing run0000_0000 timepoint 0.44325
 finished parsing run0000_0000 timepoint 0.4433
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4303
 finished parsing run0000_0000 timepoint 0.43035
 finished parsing run0000_0000 timepoint 0.4304
 finished parsing run0000_0000 timepoint 0.43045
 finished parsing run0000_0000 timepoint 0.4305
 finished parsing run0000_0000 timepoint 0.43055
 finished parsing run0000_0000 timepoint 0.4306
 finished parsing run0000_0000 timepoint 0.43065
 finished parsing run0000_0000 timepoint 0.4307
 finished parsing run0000_0000 timepoint 0.43075
 finished parsing run0000_0000 timepoint 0.4308
 finished parsing run0000_0000 timepoint 0.43085
 finished parsing run0000_0000 timepoint 0.4309
 finished parsing run0000_0000 timepoint 0.43095
 finished parsing run0000_0000 timepoint 0.431
 finished parsing run0000_0000 timepoint 0.43105
 finished parsing run0000_0000 timepoint 0.4311
 finished parsing run0000_0000 timepoint 0.43115
 finished parsing run0000_0000 timepoint 0.4312
 finished parsing run0000_0000 timepoint 0.43125
 finished parsing run0000_0000 timepoint 0.4313
 finished parsing run0000_0000 timepoint 0.43135
 finished parsing run0000_0000 timepoint 0.4314
 finished parsing run0000_0000 timepoint 0.43145
 finished parsing run0000_0000 timepoint 0.4315
 finished parsing run0000_0000 timepoint 0.43155
 finished parsing run0000_0000 timepoint 0.4316
 finished parsing run0000_0000 timepoint 0.43165
 finished parsing run0000_0000 timepoint 0.4317
 finished parsing run0000_0000 timepoint 0.43175
 finished parsing run0000_0000 timepoint 0.4318
 finished parsing run0000_0000 timepoint 0.43185
 finished parsing run0000_0000 timepoint 0.4319
 finished parsing run0000_0000 timepoint 0.43195
 finished parsing run0000_0000 timepoint 0.432
 finished parsing run0000_0000 timepoint 0.43205
 finished parsing run0000_0000 timepoint 0.4321
 finished parsing run0000_0000 timepoint 0.43215
 finished parsing run0000_0000 timepoint 0.4322
 finished parsing run0000_0000 timepoint 0.43225
 finished parsing run0000_0000 timepoint 0.4323
 finished parsing run0000_0000 timepoint 0.43235
 finished parsing run0000_0000 timepoint 0.4324
 finished parsing run0000_0000 timepoint 0.43245
 finished parsing run0000_0000 timepoint 0.4325
 finished parsing run0000_0000 timepoint 0.43255
 finished parsing run0000_0000 timepoint 0.4326
 finished parsing run0000_0000 timepoint 0.43265
 finished parsing run0000_0000 timepoint 0.4327
 finished parsing run0000_0000 timepoint 0.43275
 finished parsing run0000_0000 timepoint 0.4328
 finished parsing run0000_0000 timepoint 0.43285
 finished parsing run0000_0000 timepoint 0.4329
 finished parsing run0000_0000 timepoint 0.43295
 finished parsing run0000_0000 timepoint 0.433
 finished parsing run0000_0000 timepoint 0.43305
 finished parsing run0000_0000 timepoint 0.4331
 finished parsing run0000_0000 timepoint 0.43315
 finished parsing run0000_0000 timepoint 0.4332
 finished parsing run0000_0000 timepoint 0.43325
 finished parsing run0000_0000 timepoint 0.4333
 finished parsing run0000_0000 timepoint 0.43335
 finished parsing run0000_0000 timepoint 0.4334
 finished parsing run0000_0000 timepoint 0.43345
 finished parsing run0000_0000 timepoint 0.4335
 finished parsing run0000_0000 timepoint 0.43355
 finished parsing run0000_0000 timepoint 0.4336
 finished parsing run0000_0000 timepoint 0.43365
 finished parsing run0000_0000 timepoint 0.4337
 finished parsing run0000_0000 timepoint 0.43375
 finished parsing run0000_0000 timepoint 0.4338
 finished parsing run0000_0000 timepoint 0.43385
 finished parsing run0000_0000 timepoint 0.4339
 finished parsing run0000_0000 timepoint 0.43395
 finished parsing run0000_0000 timepoint 0.434
 finished parsing run0000_0000 timepoint 0.43405
 finished parsing run0000_0000 timepoint 0.4341
 finished parsing run0000_0000 timepoint 0.43415
 finished parsing run0000_0000 timepoint 0.4342
 finished parsing run0000_0000 timepoint 0.43425
 finished parsing run0000_0000 timepoint 0.4343
 finished parsing run0000_0000 timepoint 0.43435
 finished parsing run0000_0000 timepoint 0.4344
 finished parsing run0000_0000 timepoint 0.43445
 finished parsing run0000_0000 timepoint 0.4345
 finished parsing run0000_0000 timepoint 0.43455
 finished parsing run0000_0000 timepoint 0.4346
 finished parsing run0000_0000 timepoint 0.43465
 finished parsing run0000_0000 timepoint 0.4347
 finished parsing run0000_0000 timepoint 0.43475
 finished parsing run0000_0000 timepoint 0.4348
 finished parsing run0000_0000 timepoint 0.43485
 finished parsing run0000_0000 timepoint 0.4349
 finished parsing run0000_0000 timepoint 0.43495
 finished parsing run0000_0000 timepoint 0.435
 finished parsing run0000_0000 timepoint 0.43505
 finished parsing run0000_0000 timepoint 0.4351
 finished parsing run0000_0000 timepoint 0.43515
 finished parsing run0000_0000 timepoint 0.4352
 finished parsing run0000_0000 timepoint 0.43525
 finished parsing run0000_0000 timepoint 0.4353
 finished parsing run0000_0000 timepoint 0.43535
 finished parsing run0000_0000 timepoint 0.4354
 finished parsing run0000_0000 timepoint 0.43545
 finished parsing run0000_0000 timepoint 0.4355
 finished parsing run0000_0000 timepoint 0.43555
 finished parsing run0000_0000 timepoint 0.4356
 finished parsing run0000_0000 timepoint 0.43565
 finished parsing run0000_0000 timepoint 0.4357
 finished parsing run0000_0000 timepoint 0.43575
 finished parsing run0000_0000 timepoint 0.4358
 finished parsing run0000_0000 timepoint 0.43585
 finished parsing run0000_0000 timepoint 0.4359
 finished parsing run0000_0000 timepoint 0.43595
 finished parsing run0000_0000 timepoint 0.436
 finished parsing run0000_0000 timepoint 0.43605
 finished parsing run0000_0000 timepoint 0.4361
 finished parsing run0000_0000 timepoint 0.43615
 finished parsing run0000_0000 timepoint 0.4362
 finished parsing run0000_0000 timepoint 0.43625
 finished parsing run0000_0000 timepoint 0.4363
 finished parsing run0000_0000 timepoint 0.43635
 finished parsing run0000_0000 timepoint 0.4364
 finished parsing run0000_0000 timepoint 0.43645
 finished parsing run0000_0000 timepoint 0.4365
 finished parsing run0000_0000 timepoint 0.43655
 finished parsing run0000_0000 timepoint 0.4366
 finished parsing run0000_0000 timepoint 0.43665
 finished parsing run0000_0000 timepoint 0.4367
 finished parsing run0000_0000 timepoint 0.43675
 finished parsing run0000_0000 timepoint 0.4368
 finished parsing run0000_0000 timepoint 0.43685
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.42365
 finished parsing run0000_0000 timepoint 0.4237
 finished parsing run0000_0000 timepoint 0.42375
 finished parsing run0000_0000 timepoint 0.4238
 finished parsing run0000_0000 timepoint 0.42385
 finished parsing run0000_0000 timepoint 0.4239
 finished parsing run0000_0000 timepoint 0.42395
 finished parsing run0000_0000 timepoint 0.424
 finished parsing run0000_0000 timepoint 0.42405
 finished parsing run0000_0000 timepoint 0.4241
 finished parsing run0000_0000 timepoint 0.42415
 finished parsing run0000_0000 timepoint 0.4242
 finished parsing run0000_0000 timepoint 0.42425
 finished parsing run0000_0000 timepoint 0.4243
 finished parsing run0000_0000 timepoint 0.42435
 finished parsing run0000_0000 timepoint 0.4244
 finished parsing run0000_0000 timepoint 0.42445
 finished parsing run0000_0000 timepoint 0.4245
 finished parsing run0000_0000 timepoint 0.42455
 finished parsing run0000_0000 timepoint 0.4246
 finished parsing run0000_0000 timepoint 0.42465
 finished parsing run0000_0000 timepoint 0.4247
 finished parsing run0000_0000 timepoint 0.42475
 finished parsing run0000_0000 timepoint 0.4248
 finished parsing run0000_0000 timepoint 0.42485
 finished parsing run0000_0000 timepoint 0.4249
 finished parsing run0000_0000 timepoint 0.42495
 finished parsing run0000_0000 timepoint 0.425
 finished parsing run0000_0000 timepoint 0.42505
 finished parsing run0000_0000 timepoint 0.4251
 finished parsing run0000_0000 timepoint 0.42515
 finished parsing run0000_0000 timepoint 0.4252
 finished parsing run0000_0000 timepoint 0.42525
 finished parsing run0000_0000 timepoint 0.4253
 finished parsing run0000_0000 timepoint 0.42535
 finished parsing run0000_0000 timepoint 0.4254
 finished parsing run0000_0000 timepoint 0.42545
 finished parsing run0000_0000 timepoint 0.4255
 finished parsing run0000_0000 timepoint 0.42555
 finished parsing run0000_0000 timepoint 0.4256
 finished parsing run0000_0000 timepoint 0.42565
 finished parsing run0000_0000 timepoint 0.4257
 finished parsing run0000_0000 timepoint 0.42575
 finished parsing run0000_0000 timepoint 0.4258
 finished parsing run0000_0000 timepoint 0.42585
 finished parsing run0000_0000 timepoint 0.4259
 finished parsing run0000_0000 timepoint 0.42595
 finished parsing run0000_0000 timepoint 0.426
 finished parsing run0000_0000 timepoint 0.42605
 finished parsing run0000_0000 timepoint 0.4261
 finished parsing run0000_0000 timepoint 0.42615
 finished parsing run0000_0000 timepoint 0.4262
 finished parsing run0000_0000 timepoint 0.42625
 finished parsing run0000_0000 timepoint 0.4263
 finished parsing run0000_0000 timepoint 0.42635
 finished parsing run0000_0000 timepoint 0.4264
 finished parsing run0000_0000 timepoint 0.42645
 finished parsing run0000_0000 timepoint 0.4265
 finished parsing run0000_0000 timepoint 0.42655
 finished parsing run0000_0000 timepoint 0.4266
 finished parsing run0000_0000 timepoint 0.42665
 finished parsing run0000_0000 timepoint 0.4267
 finished parsing run0000_0000 timepoint 0.42675
 finished parsing run0000_0000 timepoint 0.4268
 finished parsing run0000_0000 timepoint 0.42685
 finished parsing run0000_0000 timepoint 0.4269
 finished parsing run0000_0000 timepoint 0.42695
 finished parsing run0000_0000 timepoint 0.427
 finished parsing run0000_0000 timepoint 0.42705
 finished parsing run0000_0000 timepoint 0.4271
 finished parsing run0000_0000 timepoint 0.42715
 finished parsing run0000_0000 timepoint 0.4272
 finished parsing run0000_0000 timepoint 0.42725
 finished parsing run0000_0000 timepoint 0.4273
 finished parsing run0000_0000 timepoint 0.42735
 finished parsing run0000_0000 timepoint 0.4274
 finished parsing run0000_0000 timepoint 0.42745
 finished parsing run0000_0000 timepoint 0.4275
 finished parsing run0000_0000 timepoint 0.42755
 finished parsing run0000_0000 timepoint 0.4276
 finished parsing run0000_0000 timepoint 0.42765
 finished parsing run0000_0000 timepoint 0.4277
 finished parsing run0000_0000 timepoint 0.42775
 finished parsing run0000_0000 timepoint 0.4278
 finished parsing run0000_0000 timepoint 0.42785
 finished parsing run0000_0000 timepoint 0.4279
 finished parsing run0000_0000 timepoint 0.42795
 finished parsing run0000_0000 timepoint 0.428
 finished parsing run0000_0000 timepoint 0.42805
 finished parsing run0000_0000 timepoint 0.4281
 finished parsing run0000_0000 timepoint 0.42815
 finished parsing run0000_0000 timepoint 0.4282
 finished parsing run0000_0000 timepoint 0.42825
 finished parsing run0000_0000 timepoint 0.4283
 finished parsing run0000_0000 timepoint 0.42835
 finished parsing run0000_0000 timepoint 0.4284
 finished parsing run0000_0000 timepoint 0.42845
 finished parsing run0000_0000 timepoint 0.4285
 finished parsing run0000_0000 timepoint 0.42855
 finished parsing run0000_0000 timepoint 0.4286
 finished parsing run0000_0000 timepoint 0.42865
 finished parsing run0000_0000 timepoint 0.4287
 finished parsing run0000_0000 timepoint 0.42875
 finished parsing run0000_0000 timepoint 0.4288
 finished parsing run0000_0000 timepoint 0.42885
 finished parsing run0000_0000 timepoint 0.4289
 finished parsing run0000_0000 timepoint 0.42895
 finished parsing run0000_0000 timepoint 0.429
 finished parsing run0000_0000 timepoint 0.42905
 finished parsing run0000_0000 timepoint 0.4291
 finished parsing run0000_0000 timepoint 0.42915
 finished parsing run0000_0000 timepoint 0.4292
 finished parsing run0000_0000 timepoint 0.42925
 finished parsing run0000_0000 timepoint 0.4293
 finished parsing run0000_0000 timepoint 0.42935
 finished parsing run0000_0000 timepoint 0.4294
 finished parsing run0000_0000 timepoint 0.42945
 finished parsing run0000_0000 timepoint 0.4295
 finished parsing run0000_0000 timepoint 0.42955
 finished parsing run0000_0000 timepoint 0.4296
 finished parsing run0000_0000 timepoint 0.42965
 finished parsing run0000_0000 timepoint 0.4297
 finished parsing run0000_0000 timepoint 0.42975
 finished parsing run0000_0000 timepoint 0.4298
 finished parsing run0000_0000 timepoint 0.42985
 finished parsing run0000_0000 timepoint 0.4299
 finished parsing run0000_0000 timepoint 0.42995
 finished parsing run0000_0000 timepoint 0.43
 finished parsing run0000_0000 timepoint 0.43005
 finished parsing run0000_0000 timepoint 0.4301
 finished parsing run0000_0000 timepoint 0.43015
 finished parsing run0000_0000 timepoint 0.4302
 finished parsing run0000_0000 timepoint 0.43025
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4171
 finished parsing run0000_0000 timepoint 0.41715
 finished parsing run0000_0000 timepoint 0.4172
 finished parsing run0000_0000 timepoint 0.41725
 finished parsing run0000_0000 timepoint 0.4173
 finished parsing run0000_0000 timepoint 0.41735
 finished parsing run0000_0000 timepoint 0.4174
 finished parsing run0000_0000 timepoint 0.41745
 finished parsing run0000_0000 timepoint 0.4175
 finished parsing run0000_0000 timepoint 0.41755
 finished parsing run0000_0000 timepoint 0.4176
 finished parsing run0000_0000 timepoint 0.41765
 finished parsing run0000_0000 timepoint 0.4177
 finished parsing run0000_0000 timepoint 0.41775
 finished parsing run0000_0000 timepoint 0.4178
 finished parsing run0000_0000 timepoint 0.41785
 finished parsing run0000_0000 timepoint 0.4179
 finished parsing run0000_0000 timepoint 0.41795
 finished parsing run0000_0000 timepoint 0.418
 finished parsing run0000_0000 timepoint 0.41805
 finished parsing run0000_0000 timepoint 0.4181
 finished parsing run0000_0000 timepoint 0.41815
 finished parsing run0000_0000 timepoint 0.4182
 finished parsing run0000_0000 timepoint 0.41825
 finished parsing run0000_0000 timepoint 0.4183
 finished parsing run0000_0000 timepoint 0.41835
 finished parsing run0000_0000 timepoint 0.4184
 finished parsing run0000_0000 timepoint 0.41845
 finished parsing run0000_0000 timepoint 0.4185
 finished parsing run0000_0000 timepoint 0.41855
 finished parsing run0000_0000 timepoint 0.4186
 finished parsing run0000_0000 timepoint 0.41865
 finished parsing run0000_0000 timepoint 0.4187
 finished parsing run0000_0000 timepoint 0.41875
 finished parsing run0000_0000 timepoint 0.4188
 finished parsing run0000_0000 timepoint 0.41885
 finished parsing run0000_0000 timepoint 0.4189
 finished parsing run0000_0000 timepoint 0.41895
 finished parsing run0000_0000 timepoint 0.419
 finished parsing run0000_0000 timepoint 0.41905
 finished parsing run0000_0000 timepoint 0.4191
 finished parsing run0000_0000 timepoint 0.41915
 finished parsing run0000_0000 timepoint 0.4192
 finished parsing run0000_0000 timepoint 0.41925
 finished parsing run0000_0000 timepoint 0.4193
 finished parsing run0000_0000 timepoint 0.41935
 finished parsing run0000_0000 timepoint 0.4194
 finished parsing run0000_0000 timepoint 0.41945
 finished parsing run0000_0000 timepoint 0.4195
 finished parsing run0000_0000 timepoint 0.41955
 finished parsing run0000_0000 timepoint 0.4196
 finished parsing run0000_0000 timepoint 0.41965
 finished parsing run0000_0000 timepoint 0.4197
 finished parsing run0000_0000 timepoint 0.41975
 finished parsing run0000_0000 timepoint 0.4198
 finished parsing run0000_0000 timepoint 0.41985
 finished parsing run0000_0000 timepoint 0.4199
 finished parsing run0000_0000 timepoint 0.41995
 finished parsing run0000_0000 timepoint 0.42
 finished parsing run0000_0000 timepoint 0.42005
 finished parsing run0000_0000 timepoint 0.4201
 finished parsing run0000_0000 timepoint 0.42015
 finished parsing run0000_0000 timepoint 0.4202
 finished parsing run0000_0000 timepoint 0.42025
 finished parsing run0000_0000 timepoint 0.4203
 finished parsing run0000_0000 timepoint 0.42035
 finished parsing run0000_0000 timepoint 0.4204
 finished parsing run0000_0000 timepoint 0.42045
 finished parsing run0000_0000 timepoint 0.4205
 finished parsing run0000_0000 timepoint 0.42055
 finished parsing run0000_0000 timepoint 0.4206
 finished parsing run0000_0000 timepoint 0.42065
 finished parsing run0000_0000 timepoint 0.4207
 finished parsing run0000_0000 timepoint 0.42075
 finished parsing run0000_0000 timepoint 0.4208
 finished parsing run0000_0000 timepoint 0.42085
 finished parsing run0000_0000 timepoint 0.4209
 finished parsing run0000_0000 timepoint 0.42095
 finished parsing run0000_0000 timepoint 0.421
 finished parsing run0000_0000 timepoint 0.42105
 finished parsing run0000_0000 timepoint 0.4211
 finished parsing run0000_0000 timepoint 0.42115
 finished parsing run0000_0000 timepoint 0.4212
 finished parsing run0000_0000 timepoint 0.42125
 finished parsing run0000_0000 timepoint 0.4213
 finished parsing run0000_0000 timepoint 0.42135
 finished parsing run0000_0000 timepoint 0.4214
 finished parsing run0000_0000 timepoint 0.42145
 finished parsing run0000_0000 timepoint 0.4215
 finished parsing run0000_0000 timepoint 0.42155
 finished parsing run0000_0000 timepoint 0.4216
 finished parsing run0000_0000 timepoint 0.42165
 finished parsing run0000_0000 timepoint 0.4217
 finished parsing run0000_0000 timepoint 0.42175
 finished parsing run0000_0000 timepoint 0.4218
 finished parsing run0000_0000 timepoint 0.42185
 finished parsing run0000_0000 timepoint 0.4219
 finished parsing run0000_0000 timepoint 0.42195
 finished parsing run0000_0000 timepoint 0.422
 finished parsing run0000_0000 timepoint 0.42205
 finished parsing run0000_0000 timepoint 0.4221
 finished parsing run0000_0000 timepoint 0.42215
 finished parsing run0000_0000 timepoint 0.4222
 finished parsing run0000_0000 timepoint 0.42225
 finished parsing run0000_0000 timepoint 0.4223
 finished parsing run0000_0000 timepoint 0.42235
 finished parsing run0000_0000 timepoint 0.4224
 finished parsing run0000_0000 timepoint 0.42245
 finished parsing run0000_0000 timepoint 0.4225
 finished parsing run0000_0000 timepoint 0.42255
 finished parsing run0000_0000 timepoint 0.4226
 finished parsing run0000_0000 timepoint 0.42265
 finished parsing run0000_0000 timepoint 0.4227
 finished parsing run0000_0000 timepoint 0.42275
 finished parsing run0000_0000 timepoint 0.4228
 finished parsing run0000_0000 timepoint 0.42285
 finished parsing run0000_0000 timepoint 0.4229
 finished parsing run0000_0000 timepoint 0.42295
 finished parsing run0000_0000 timepoint 0.423
 finished parsing run0000_0000 timepoint 0.42305
 finished parsing run0000_0000 timepoint 0.4231
 finished parsing run0000_0000 timepoint 0.42315
 finished parsing run0000_0000 timepoint 0.4232
 finished parsing run0000_0000 timepoint 0.42325
 finished parsing run0000_0000 timepoint 0.4233
 finished parsing run0000_0000 timepoint 0.42335
 finished parsing run0000_0000 timepoint 0.4234
 finished parsing run0000_0000 timepoint 0.42345
 finished parsing run0000_0000 timepoint 0.4235
 finished parsing run0000_0000 timepoint 0.42355
 finished parsing run0000_0000 timepoint 0.4236
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.4102
 finished parsing run0000_0000 timepoint 0.41025
 finished parsing run0000_0000 timepoint 0.4103
 finished parsing run0000_0000 timepoint 0.41035
 finished parsing run0000_0000 timepoint 0.4104
 finished parsing run0000_0000 timepoint 0.41045
 finished parsing run0000_0000 timepoint 0.4105
 finished parsing run0000_0000 timepoint 0.41055
 finished parsing run0000_0000 timepoint 0.4106
 finished parsing run0000_0000 timepoint 0.41065
 finished parsing run0000_0000 timepoint 0.4107
 finished parsing run0000_0000 timepoint 0.41075
 finished parsing run0000_0000 timepoint 0.4108
 finished parsing run0000_0000 timepoint 0.41085
 finished parsing run0000_0000 timepoint 0.4109
 finished parsing run0000_0000 timepoint 0.41095
 finished parsing run0000_0000 timepoint 0.411
 finished parsing run0000_0000 timepoint 0.41105
 finished parsing run0000_0000 timepoint 0.4111
 finished parsing run0000_0000 timepoint 0.41115
 finished parsing run0000_0000 timepoint 0.4112
 finished parsing run0000_0000 timepoint 0.41125
 finished parsing run0000_0000 timepoint 0.4113
 finished parsing run0000_0000 timepoint 0.41135
 finished parsing run0000_0000 timepoint 0.4114
 finished parsing run0000_0000 timepoint 0.41145
 finished parsing run0000_0000 timepoint 0.4115
 finished parsing run0000_0000 timepoint 0.41155
 finished parsing run0000_0000 timepoint 0.4116
 finished parsing run0000_0000 timepoint 0.41165
 finished parsing run0000_0000 timepoint 0.4117
 finished parsing run0000_0000 timepoint 0.41175
 finished parsing run0000_0000 timepoint 0.4118
 finished parsing run0000_0000 timepoint 0.41185
 finished parsing run0000_0000 timepoint 0.4119
 finished parsing run0000_0000 timepoint 0.41195
 finished parsing run0000_0000 timepoint 0.412
 finished parsing run0000_0000 timepoint 0.41205
 finished parsing run0000_0000 timepoint 0.4121
 finished parsing run0000_0000 timepoint 0.41215
 finished parsing run0000_0000 timepoint 0.4122
 finished parsing run0000_0000 timepoint 0.41225
 finished parsing run0000_0000 timepoint 0.4123
 finished parsing run0000_0000 timepoint 0.41235
 finished parsing run0000_0000 timepoint 0.4124
 finished parsing run0000_0000 timepoint 0.41245
 finished parsing run0000_0000 timepoint 0.4125
 finished parsing run0000_0000 timepoint 0.41255
 finished parsing run0000_0000 timepoint 0.4126
 finished parsing run0000_0000 timepoint 0.41265
 finished parsing run0000_0000 timepoint 0.4127
 finished parsing run0000_0000 timepoint 0.41275
 finished parsing run0000_0000 timepoint 0.4128
 finished parsing run0000_0000 timepoint 0.41285
 finished parsing run0000_0000 timepoint 0.4129
 finished parsing run0000_0000 timepoint 0.41295
 finished parsing run0000_0000 timepoint 0.413
 finished parsing run0000_0000 timepoint 0.41305
 finished parsing run0000_0000 timepoint 0.4131
 finished parsing run0000_0000 timepoint 0.41315
 finished parsing run0000_0000 timepoint 0.4132
 finished parsing run0000_0000 timepoint 0.41325
 finished parsing run0000_0000 timepoint 0.4133
 finished parsing run0000_0000 timepoint 0.41335
 finished parsing run0000_0000 timepoint 0.4134
 finished parsing run0000_0000 timepoint 0.41345
 finished parsing run0000_0000 timepoint 0.4135
 finished parsing run0000_0000 timepoint 0.41355
 finished parsing run0000_0000 timepoint 0.4136
 finished parsing run0000_0000 timepoint 0.41365
 finished parsing run0000_0000 timepoint 0.4137
 finished parsing run0000_0000 timepoint 0.41375
 finished parsing run0000_0000 timepoint 0.4138
 finished parsing run0000_0000 timepoint 0.41385
 finished parsing run0000_0000 timepoint 0.4139
 finished parsing run0000_0000 timepoint 0.41395
 finished parsing run0000_0000 timepoint 0.414
 finished parsing run0000_0000 timepoint 0.41405
 finished parsing run0000_0000 timepoint 0.4141
 finished parsing run0000_0000 timepoint 0.41415
 finished parsing run0000_0000 timepoint 0.4142
 finished parsing run0000_0000 timepoint 0.41425
 finished parsing run0000_0000 timepoint 0.4143
 finished parsing run0000_0000 timepoint 0.41435
 finished parsing run0000_0000 timepoint 0.4144
 finished parsing run0000_0000 timepoint 0.41445
 finished parsing run0000_0000 timepoint 0.4145
 finished parsing run0000_0000 timepoint 0.41455
 finished parsing run0000_0000 timepoint 0.4146
 finished parsing run0000_0000 timepoint 0.41465
 finished parsing run0000_0000 timepoint 0.4147
 finished parsing run0000_0000 timepoint 0.41475
 finished parsing run0000_0000 timepoint 0.4148
 finished parsing run0000_0000 timepoint 0.41485
 finished parsing run0000_0000 timepoint 0.4149
 finished parsing run0000_0000 timepoint 0.41495
 finished parsing run0000_0000 timepoint 0.415
 finished parsing run0000_0000 timepoint 0.41505
 finished parsing run0000_0000 timepoint 0.4151
 finished parsing run0000_0000 timepoint 0.41515
 finished parsing run0000_0000 timepoint 0.4152
 finished parsing run0000_0000 timepoint 0.41525
 finished parsing run0000_0000 timepoint 0.4153
 finished parsing run0000_0000 timepoint 0.41535
 finished parsing run0000_0000 timepoint 0.4154
 finished parsing run0000_0000 timepoint 0.41545
 finished parsing run0000_0000 timepoint 0.4155
 finished parsing run0000_0000 timepoint 0.41555
 finished parsing run0000_0000 timepoint 0.4156
 finished parsing run0000_0000 timepoint 0.41565
 finished parsing run0000_0000 timepoint 0.4157
 finished parsing run0000_0000 timepoint 0.41575
 finished parsing run0000_0000 timepoint 0.4158
 finished parsing run0000_0000 timepoint 0.41585
 finished parsing run0000_0000 timepoint 0.4159
 finished parsing run0000_0000 timepoint 0.41595
 finished parsing run0000_0000 timepoint 0.416
 finished parsing run0000_0000 timepoint 0.41605
 finished parsing run0000_0000 timepoint 0.4161
 finished parsing run0000_0000 timepoint 0.41615
 finished parsing run0000_0000 timepoint 0.4162
 finished parsing run0000_0000 timepoint 0.41625
 finished parsing run0000_0000 timepoint 0.4163
 finished parsing run0000_0000 timepoint 0.41635
 finished parsing run0000_0000 timepoint 0.4164
 finished parsing run0000_0000 timepoint 0.41645
 finished parsing run0000_0000 timepoint 0.4165
 finished parsing run0000_0000 timepoint 0.41655
 finished parsing run0000_0000 timepoint 0.4166
 finished parsing run0000_0000 timepoint 0.41665
 finished parsing run0000_0000 timepoint 0.4167
 finished parsing run0000_0000 timepoint 0.41675
 finished parsing run0000_0000 timepoint 0.4168
 finished parsing run0000_0000 timepoint 0.41685
 finished parsing run0000_0000 timepoint 0.4169
 finished parsing run0000_0000 timepoint 0.41695
 finished parsing run0000_0000 timepoint 0.417
 finished parsing run0000_0000 timepoint 0.41705
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.40365
 finished parsing run0000_0000 timepoint 0.4037
 finished parsing run0000_0000 timepoint 0.40375
 finished parsing run0000_0000 timepoint 0.4038
 finished parsing run0000_0000 timepoint 0.40385
 finished parsing run0000_0000 timepoint 0.4039
 finished parsing run0000_0000 timepoint 0.40395
 finished parsing run0000_0000 timepoint 0.404
 finished parsing run0000_0000 timepoint 0.40405
 finished parsing run0000_0000 timepoint 0.4041
 finished parsing run0000_0000 timepoint 0.40415
 finished parsing run0000_0000 timepoint 0.4042
 finished parsing run0000_0000 timepoint 0.40425
 finished parsing run0000_0000 timepoint 0.4043
 finished parsing run0000_0000 timepoint 0.40435
 finished parsing run0000_0000 timepoint 0.4044
 finished parsing run0000_0000 timepoint 0.40445
 finished parsing run0000_0000 timepoint 0.4045
 finished parsing run0000_0000 timepoint 0.40455
 finished parsing run0000_0000 timepoint 0.4046
 finished parsing run0000_0000 timepoint 0.40465
 finished parsing run0000_0000 timepoint 0.4047
 finished parsing run0000_0000 timepoint 0.40475
 finished parsing run0000_0000 timepoint 0.4048
 finished parsing run0000_0000 timepoint 0.40485
 finished parsing run0000_0000 timepoint 0.4049
 finished parsing run0000_0000 timepoint 0.40495
 finished parsing run0000_0000 timepoint 0.405
 finished parsing run0000_0000 timepoint 0.40505
 finished parsing run0000_0000 timepoint 0.4051
 finished parsing run0000_0000 timepoint 0.40515
 finished parsing run0000_0000 timepoint 0.4052
 finished parsing run0000_0000 timepoint 0.40525
 finished parsing run0000_0000 timepoint 0.4053
 finished parsing run0000_0000 timepoint 0.40535
 finished parsing run0000_0000 timepoint 0.4054
 finished parsing run0000_0000 timepoint 0.40545
 finished parsing run0000_0000 timepoint 0.4055
 finished parsing run0000_0000 timepoint 0.40555
 finished parsing run0000_0000 timepoint 0.4056
 finished parsing run0000_0000 timepoint 0.40565
 finished parsing run0000_0000 timepoint 0.4057
 finished parsing run0000_0000 timepoint 0.40575
 finished parsing run0000_0000 timepoint 0.4058
 finished parsing run0000_0000 timepoint 0.40585
 finished parsing run0000_0000 timepoint 0.4059
 finished parsing run0000_0000 timepoint 0.40595
 finished parsing run0000_0000 timepoint 0.406
 finished parsing run0000_0000 timepoint 0.40605
 finished parsing run0000_0000 timepoint 0.4061
 finished parsing run0000_0000 timepoint 0.40615
 finished parsing run0000_0000 timepoint 0.4062
 finished parsing run0000_0000 timepoint 0.40625
 finished parsing run0000_0000 timepoint 0.4063
 finished parsing run0000_0000 timepoint 0.40635
 finished parsing run0000_0000 timepoint 0.4064
 finished parsing run0000_0000 timepoint 0.40645
 finished parsing run0000_0000 timepoint 0.4065
 finished parsing run0000_0000 timepoint 0.40655
 finished parsing run0000_0000 timepoint 0.4066
 finished parsing run0000_0000 timepoint 0.40665
 finished parsing run0000_0000 timepoint 0.4067
 finished parsing run0000_0000 timepoint 0.40675
 finished parsing run0000_0000 timepoint 0.4068
 finished parsing run0000_0000 timepoint 0.40685
 finished parsing run0000_0000 timepoint 0.4069
 finished parsing run0000_0000 timepoint 0.40695
 finished parsing run0000_0000 timepoint 0.407
 finished parsing run0000_0000 timepoint 0.40705
 finished parsing run0000_0000 timepoint 0.4071
 finished parsing run0000_0000 timepoint 0.40715
 finished parsing run0000_0000 timepoint 0.4072
 finished parsing run0000_0000 timepoint 0.40725
 finished parsing run0000_0000 timepoint 0.4073
 finished parsing run0000_0000 timepoint 0.40735
 finished parsing run0000_0000 timepoint 0.4074
 finished parsing run0000_0000 timepoint 0.40745
 finished parsing run0000_0000 timepoint 0.4075
 finished parsing run0000_0000 timepoint 0.40755
 finished parsing run0000_0000 timepoint 0.4076
 finished parsing run0000_0000 timepoint 0.40765
 finished parsing run0000_0000 timepoint 0.4077
 finished parsing run0000_0000 timepoint 0.40775
 finished parsing run0000_0000 timepoint 0.4078
 finished parsing run0000_0000 timepoint 0.40785
 finished parsing run0000_0000 timepoint 0.4079
 finished parsing run0000_0000 timepoint 0.40795
 finished parsing run0000_0000 timepoint 0.408
 finished parsing run0000_0000 timepoint 0.40805
 finished parsing run0000_0000 timepoint 0.4081
 finished parsing run0000_0000 timepoint 0.40815
 finished parsing run0000_0000 timepoint 0.4082
 finished parsing run0000_0000 timepoint 0.40825
 finished parsing run0000_0000 timepoint 0.4083
 finished parsing run0000_0000 timepoint 0.40835
 finished parsing run0000_0000 timepoint 0.4084
 finished parsing run0000_0000 timepoint 0.40845
 finished parsing run0000_0000 timepoint 0.4085
 finished parsing run0000_0000 timepoint 0.40855
 finished parsing run0000_0000 timepoint 0.4086
 finished parsing run0000_0000 timepoint 0.40865
 finished parsing run0000_0000 timepoint 0.4087
 finished parsing run0000_0000 timepoint 0.40875
 finished parsing run0000_0000 timepoint 0.4088
 finished parsing run0000_0000 timepoint 0.40885
 finished parsing run0000_0000 timepoint 0.4089
 finished parsing run0000_0000 timepoint 0.40895
 finished parsing run0000_0000 timepoint 0.409
 finished parsing run0000_0000 timepoint 0.40905
 finished parsing run0000_0000 timepoint 0.4091
 finished parsing run0000_0000 timepoint 0.40915
 finished parsing run0000_0000 timepoint 0.4092
 finished parsing run0000_0000 timepoint 0.40925
 finished parsing run0000_0000 timepoint 0.4093
 finished parsing run0000_0000 timepoint 0.40935
 finished parsing run0000_0000 timepoint 0.4094
 finished parsing run0000_0000 timepoint 0.40945
 finished parsing run0000_0000 timepoint 0.4095
 finished parsing run0000_0000 timepoint 0.40955
 finished parsing run0000_0000 timepoint 0.4096
 finished parsing run0000_0000 timepoint 0.40965
 finished parsing run0000_0000 timepoint 0.4097
 finished parsing run0000_0000 timepoint 0.40975
 finished parsing run0000_0000 timepoint 0.4098
 finished parsing run0000_0000 timepoint 0.40985
 finished parsing run0000_0000 timepoint 0.4099
 finished parsing run0000_0000 timepoint 0.40995
 finished parsing run0000_0000 timepoint 0.41
 finished parsing run0000_0000 timepoint 0.41005
 finished parsing run0000_0000 timepoint 0.4101
 finished parsing run0000_0000 timepoint 0.41015
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.39715
 finished parsing run0000_0000 timepoint 0.3972
 finished parsing run0000_0000 timepoint 0.39725
 finished parsing run0000_0000 timepoint 0.3973
 finished parsing run0000_0000 timepoint 0.39735
 finished parsing run0000_0000 timepoint 0.3974
 finished parsing run0000_0000 timepoint 0.39745
 finished parsing run0000_0000 timepoint 0.3975
 finished parsing run0000_0000 timepoint 0.39755
 finished parsing run0000_0000 timepoint 0.3976
 finished parsing run0000_0000 timepoint 0.39765
 finished parsing run0000_0000 timepoint 0.3977
 finished parsing run0000_0000 timepoint 0.39775
 finished parsing run0000_0000 timepoint 0.3978
 finished parsing run0000_0000 timepoint 0.39785
 finished parsing run0000_0000 timepoint 0.3979
 finished parsing run0000_0000 timepoint 0.39795
 finished parsing run0000_0000 timepoint 0.398
 finished parsing run0000_0000 timepoint 0.39805
 finished parsing run0000_0000 timepoint 0.3981
 finished parsing run0000_0000 timepoint 0.39815
 finished parsing run0000_0000 timepoint 0.3982
 finished parsing run0000_0000 timepoint 0.39825
 finished parsing run0000_0000 timepoint 0.3983
 finished parsing run0000_0000 timepoint 0.39835
 finished parsing run0000_0000 timepoint 0.3984
 finished parsing run0000_0000 timepoint 0.39845
 finished parsing run0000_0000 timepoint 0.3985
 finished parsing run0000_0000 timepoint 0.39855
 finished parsing run0000_0000 timepoint 0.3986
 finished parsing run0000_0000 timepoint 0.39865
 finished parsing run0000_0000 timepoint 0.3987
 finished parsing run0000_0000 timepoint 0.39875
 finished parsing run0000_0000 timepoint 0.3988
 finished parsing run0000_0000 timepoint 0.39885
 finished parsing run0000_0000 timepoint 0.3989
 finished parsing run0000_0000 timepoint 0.39895
 finished parsing run0000_0000 timepoint 0.399
 finished parsing run0000_0000 timepoint 0.39905
 finished parsing run0000_0000 timepoint 0.3991
 finished parsing run0000_0000 timepoint 0.39915
 finished parsing run0000_0000 timepoint 0.3992
 finished parsing run0000_0000 timepoint 0.39925
 finished parsing run0000_0000 timepoint 0.3993
 finished parsing run0000_0000 timepoint 0.39935
 finished parsing run0000_0000 timepoint 0.3994
 finished parsing run0000_0000 timepoint 0.39945
 finished parsing run0000_0000 timepoint 0.3995
 finished parsing run0000_0000 timepoint 0.39955
 finished parsing run0000_0000 timepoint 0.3996
 finished parsing run0000_0000 timepoint 0.39965
 finished parsing run0000_0000 timepoint 0.3997
 finished parsing run0000_0000 timepoint 0.39975
 finished parsing run0000_0000 timepoint 0.3998
 finished parsing run0000_0000 timepoint 0.39985
 finished parsing run0000_0000 timepoint 0.3999
 finished parsing run0000_0000 timepoint 0.39995
 finished parsing run0000_0000 timepoint 0.4
 finished parsing run0000_0000 timepoint 0.40005
 finished parsing run0000_0000 timepoint 0.4001
 finished parsing run0000_0000 timepoint 0.40015
 finished parsing run0000_0000 timepoint 0.4002
 finished parsing run0000_0000 timepoint 0.40025
 finished parsing run0000_0000 timepoint 0.4003
 finished parsing run0000_0000 timepoint 0.40035
 finished parsing run0000_0000 timepoint 0.4004
 finished parsing run0000_0000 timepoint 0.40045
 finished parsing run0000_0000 timepoint 0.4005
 finished parsing run0000_0000 timepoint 0.40055
 finished parsing run0000_0000 timepoint 0.4006
 finished parsing run0000_0000 timepoint 0.40065
 finished parsing run0000_0000 timepoint 0.4007
 finished parsing run0000_0000 timepoint 0.40075
 finished parsing run0000_0000 timepoint 0.4008
 finished parsing run0000_0000 timepoint 0.40085
 finished parsing run0000_0000 timepoint 0.4009
 finished parsing run0000_0000 timepoint 0.40095
 finished parsing run0000_0000 timepoint 0.401
 finished parsing run0000_0000 timepoint 0.40105
 finished parsing run0000_0000 timepoint 0.4011
 finished parsing run0000_0000 timepoint 0.40115
 finished parsing run0000_0000 timepoint 0.4012
 finished parsing run0000_0000 timepoint 0.40125
 finished parsing run0000_0000 timepoint 0.4013
 finished parsing run0000_0000 timepoint 0.40135
 finished parsing run0000_0000 timepoint 0.4014
 finished parsing run0000_0000 timepoint 0.40145
 finished parsing run0000_0000 timepoint 0.4015
 finished parsing run0000_0000 timepoint 0.40155
 finished parsing run0000_0000 timepoint 0.4016
 finished parsing run0000_0000 timepoint 0.40165
 finished parsing run0000_0000 timepoint 0.4017
 finished parsing run0000_0000 timepoint 0.40175
 finished parsing run0000_0000 timepoint 0.4018
 finished parsing run0000_0000 timepoint 0.40185
 finished parsing run0000_0000 timepoint 0.4019
 finished parsing run0000_0000 timepoint 0.40195
 finished parsing run0000_0000 timepoint 0.402
 finished parsing run0000_0000 timepoint 0.40205
 finished parsing run0000_0000 timepoint 0.4021
 finished parsing run0000_0000 timepoint 0.40215
 finished parsing run0000_0000 timepoint 0.4022
 finished parsing run0000_0000 timepoint 0.40225
 finished parsing run0000_0000 timepoint 0.4023
 finished parsing run0000_0000 timepoint 0.40235
 finished parsing run0000_0000 timepoint 0.4024
 finished parsing run0000_0000 timepoint 0.40245
 finished parsing run0000_0000 timepoint 0.4025
 finished parsing run0000_0000 timepoint 0.40255
 finished parsing run0000_0000 timepoint 0.4026
 finished parsing run0000_0000 timepoint 0.40265
 finished parsing run0000_0000 timepoint 0.4027
 finished parsing run0000_0000 timepoint 0.40275
 finished parsing run0000_0000 timepoint 0.4028
 finished parsing run0000_0000 timepoint 0.40285
 finished parsing run0000_0000 timepoint 0.4029
 finished parsing run0000_0000 timepoint 0.40295
 finished parsing run0000_0000 timepoint 0.403
 finished parsing run0000_0000 timepoint 0.40305
 finished parsing run0000_0000 timepoint 0.4031
 finished parsing run0000_0000 timepoint 0.40315
 finished parsing run0000_0000 timepoint 0.4032
 finished parsing run0000_0000 timepoint 0.40325
 finished parsing run0000_0000 timepoint 0.4033
 finished parsing run0000_0000 timepoint 0.40335
 finished parsing run0000_0000 timepoint 0.4034
 finished parsing run0000_0000 timepoint 0.40345
 finished parsing run0000_0000 timepoint 0.4035
 finished parsing run0000_0000 timepoint 0.40355
 finished parsing run0000_0000 timepoint 0.4036
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.3905
 finished parsing run0000_0000 timepoint 0.39055
 finished parsing run0000_0000 timepoint 0.3906
 finished parsing run0000_0000 timepoint 0.39065
 finished parsing run0000_0000 timepoint 0.3907
 finished parsing run0000_0000 timepoint 0.39075
 finished parsing run0000_0000 timepoint 0.3908
 finished parsing run0000_0000 timepoint 0.39085
 finished parsing run0000_0000 timepoint 0.3909
 finished parsing run0000_0000 timepoint 0.39095
 finished parsing run0000_0000 timepoint 0.391
 finished parsing run0000_0000 timepoint 0.39105
 finished parsing run0000_0000 timepoint 0.3911
 finished parsing run0000_0000 timepoint 0.39115
 finished parsing run0000_0000 timepoint 0.3912
 finished parsing run0000_0000 timepoint 0.39125
 finished parsing run0000_0000 timepoint 0.3913
 finished parsing run0000_0000 timepoint 0.39135
 finished parsing run0000_0000 timepoint 0.3914
 finished parsing run0000_0000 timepoint 0.39145
 finished parsing run0000_0000 timepoint 0.3915
 finished parsing run0000_0000 timepoint 0.39155
 finished parsing run0000_0000 timepoint 0.3916
 finished parsing run0000_0000 timepoint 0.39165
 finished parsing run0000_0000 timepoint 0.3917
 finished parsing run0000_0000 timepoint 0.39175
 finished parsing run0000_0000 timepoint 0.3918
 finished parsing run0000_0000 timepoint 0.39185
 finished parsing run0000_0000 timepoint 0.3919
 finished parsing run0000_0000 timepoint 0.39195
 finished parsing run0000_0000 timepoint 0.392
 finished parsing run0000_0000 timepoint 0.39205
 finished parsing run0000_0000 timepoint 0.3921
 finished parsing run0000_0000 timepoint 0.39215
 finished parsing run0000_0000 timepoint 0.3922
 finished parsing run0000_0000 timepoint 0.39225
 finished parsing run0000_0000 timepoint 0.3923
 finished parsing run0000_0000 timepoint 0.39235
 finished parsing run0000_0000 timepoint 0.3924
 finished parsing run0000_0000 timepoint 0.39245
 finished parsing run0000_0000 timepoint 0.3925
 finished parsing run0000_0000 timepoint 0.39255
 finished parsing run0000_0000 timepoint 0.3926
 finished parsing run0000_0000 timepoint 0.39265
 finished parsing run0000_0000 timepoint 0.3927
 finished parsing run0000_0000 timepoint 0.39275
 finished parsing run0000_0000 timepoint 0.3928
 finished parsing run0000_0000 timepoint 0.39285
 finished parsing run0000_0000 timepoint 0.3929
 finished parsing run0000_0000 timepoint 0.39295
 finished parsing run0000_0000 timepoint 0.393
 finished parsing run0000_0000 timepoint 0.39305
 finished parsing run0000_0000 timepoint 0.3931
 finished parsing run0000_0000 timepoint 0.39315
 finished parsing run0000_0000 timepoint 0.3932
 finished parsing run0000_0000 timepoint 0.39325
 finished parsing run0000_0000 timepoint 0.3933
 finished parsing run0000_0000 timepoint 0.39335
 finished parsing run0000_0000 timepoint 0.3934
 finished parsing run0000_0000 timepoint 0.39345
 finished parsing run0000_0000 timepoint 0.3935
 finished parsing run0000_0000 timepoint 0.39355
 finished parsing run0000_0000 timepoint 0.3936
 finished parsing run0000_0000 timepoint 0.39365
 finished parsing run0000_0000 timepoint 0.3937
 finished parsing run0000_0000 timepoint 0.39375
 finished parsing run0000_0000 timepoint 0.3938
 finished parsing run0000_0000 timepoint 0.39385
 finished parsing run0000_0000 timepoint 0.3939
 finished parsing run0000_0000 timepoint 0.39395
 finished parsing run0000_0000 timepoint 0.394
 finished parsing run0000_0000 timepoint 0.39405
 finished parsing run0000_0000 timepoint 0.3941
 finished parsing run0000_0000 timepoint 0.39415
 finished parsing run0000_0000 timepoint 0.3942
 finished parsing run0000_0000 timepoint 0.39425
 finished parsing run0000_0000 timepoint 0.3943
 finished parsing run0000_0000 timepoint 0.39435
 finished parsing run0000_0000 timepoint 0.3944
 finished parsing run0000_0000 timepoint 0.39445
 finished parsing run0000_0000 timepoint 0.3945
 finished parsing run0000_0000 timepoint 0.39455
 finished parsing run0000_0000 timepoint 0.3946
 finished parsing run0000_0000 timepoint 0.39465
 finished parsing run0000_0000 timepoint 0.3947
 finished parsing run0000_0000 timepoint 0.39475
 finished parsing run0000_0000 timepoint 0.3948
 finished parsing run0000_0000 timepoint 0.39485
 finished parsing run0000_0000 timepoint 0.3949
 finished parsing run0000_0000 timepoint 0.39495
 finished parsing run0000_0000 timepoint 0.395
 finished parsing run0000_0000 timepoint 0.39505
 finished parsing run0000_0000 timepoint 0.3951
 finished parsing run0000_0000 timepoint 0.39515
 finished parsing run0000_0000 timepoint 0.3952
 finished parsing run0000_0000 timepoint 0.39525
 finished parsing run0000_0000 timepoint 0.3953
 finished parsing run0000_0000 timepoint 0.39535
 finished parsing run0000_0000 timepoint 0.3954
 finished parsing run0000_0000 timepoint 0.39545
 finished parsing run0000_0000 timepoint 0.3955
 finished parsing run0000_0000 timepoint 0.39555
 finished parsing run0000_0000 timepoint 0.3956
 finished parsing run0000_0000 timepoint 0.39565
 finished parsing run0000_0000 timepoint 0.3957
 finished parsing run0000_0000 timepoint 0.39575
 finished parsing run0000_0000 timepoint 0.3958
 finished parsing run0000_0000 timepoint 0.39585
 finished parsing run0000_0000 timepoint 0.3959
 finished parsing run0000_0000 timepoint 0.39595
 finished parsing run0000_0000 timepoint 0.396
 finished parsing run0000_0000 timepoint 0.39605
 finished parsing run0000_0000 timepoint 0.3961
 finished parsing run0000_0000 timepoint 0.39615
 finished parsing run0000_0000 timepoint 0.3962
 finished parsing run0000_0000 timepoint 0.39625
 finished parsing run0000_0000 timepoint 0.3963
 finished parsing run0000_0000 timepoint 0.39635
 finished parsing run0000_0000 timepoint 0.3964
 finished parsing run0000_0000 timepoint 0.39645
 finished parsing run0000_0000 timepoint 0.3965
 finished parsing run0000_0000 timepoint 0.39655
 finished parsing run0000_0000 timepoint 0.3966
 finished parsing run0000_0000 timepoint 0.39665
 finished parsing run0000_0000 timepoint 0.3967
 finished parsing run0000_0000 timepoint 0.39675
 finished parsing run0000_0000 timepoint 0.3968
 finished parsing run0000_0000 timepoint 0.39685
 finished parsing run0000_0000 timepoint 0.3969
 finished parsing run0000_0000 timepoint 0.39695
 finished parsing run0000_0000 timepoint 0.397
 finished parsing run0000_0000 timepoint 0.39705
 finished parsing run0000_0000 timepoint 0.3971
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.384
 finished parsing run0000_0000 timepoint 0.38405
 finished parsing run0000_0000 timepoint 0.3841
 finished parsing run0000_0000 timepoint 0.38415
 finished parsing run0000_0000 timepoint 0.3842
 finished parsing run0000_0000 timepoint 0.38425
 finished parsing run0000_0000 timepoint 0.3843
 finished parsing run0000_0000 timepoint 0.38435
 finished parsing run0000_0000 timepoint 0.3844
 finished parsing run0000_0000 timepoint 0.38445
 finished parsing run0000_0000 timepoint 0.3845
 finished parsing run0000_0000 timepoint 0.38455
 finished parsing run0000_0000 timepoint 0.3846
 finished parsing run0000_0000 timepoint 0.38465
 finished parsing run0000_0000 timepoint 0.3847
 finished parsing run0000_0000 timepoint 0.38475
 finished parsing run0000_0000 timepoint 0.3848
 finished parsing run0000_0000 timepoint 0.38485
 finished parsing run0000_0000 timepoint 0.3849
 finished parsing run0000_0000 timepoint 0.38495
 finished parsing run0000_0000 timepoint 0.385
 finished parsing run0000_0000 timepoint 0.38505
 finished parsing run0000_0000 timepoint 0.3851
 finished parsing run0000_0000 timepoint 0.38515
 finished parsing run0000_0000 timepoint 0.3852
 finished parsing run0000_0000 timepoint 0.38525
 finished parsing run0000_0000 timepoint 0.3853
 finished parsing run0000_0000 timepoint 0.38535
 finished parsing run0000_0000 timepoint 0.3854
 finished parsing run0000_0000 timepoint 0.38545
 finished parsing run0000_0000 timepoint 0.3855
 finished parsing run0000_0000 timepoint 0.38555
 finished parsing run0000_0000 timepoint 0.3856
 finished parsing run0000_0000 timepoint 0.38565
 finished parsing run0000_0000 timepoint 0.3857
 finished parsing run0000_0000 timepoint 0.38575
 finished parsing run0000_0000 timepoint 0.3858
 finished parsing run0000_0000 timepoint 0.38585
 finished parsing run0000_0000 timepoint 0.3859
 finished parsing run0000_0000 timepoint 0.38595
 finished parsing run0000_0000 timepoint 0.386
 finished parsing run0000_0000 timepoint 0.38605
 finished parsing run0000_0000 timepoint 0.3861
 finished parsing run0000_0000 timepoint 0.38615
 finished parsing run0000_0000 timepoint 0.3862
 finished parsing run0000_0000 timepoint 0.38625
 finished parsing run0000_0000 timepoint 0.3863
 finished parsing run0000_0000 timepoint 0.38635
 finished parsing run0000_0000 timepoint 0.3864
 finished parsing run0000_0000 timepoint 0.38645
 finished parsing run0000_0000 timepoint 0.3865
 finished parsing run0000_0000 timepoint 0.38655
 finished parsing run0000_0000 timepoint 0.3866
 finished parsing run0000_0000 timepoint 0.38665
 finished parsing run0000_0000 timepoint 0.3867
 finished parsing run0000_0000 timepoint 0.38675
 finished parsing run0000_0000 timepoint 0.3868
 finished parsing run0000_0000 timepoint 0.38685
 finished parsing run0000_0000 timepoint 0.3869
 finished parsing run0000_0000 timepoint 0.38695
 finished parsing run0000_0000 timepoint 0.387
 finished parsing run0000_0000 timepoint 0.38705
 finished parsing run0000_0000 timepoint 0.3871
 finished parsing run0000_0000 timepoint 0.38715
 finished parsing run0000_0000 timepoint 0.3872
 finished parsing run0000_0000 timepoint 0.38725
 finished parsing run0000_0000 timepoint 0.3873
 finished parsing run0000_0000 timepoint 0.38735
 finished parsing run0000_0000 timepoint 0.3874
 finished parsing run0000_0000 timepoint 0.38745
 finished parsing run0000_0000 timepoint 0.3875
 finished parsing run0000_0000 timepoint 0.38755
 finished parsing run0000_0000 timepoint 0.3876
 finished parsing run0000_0000 timepoint 0.38765
 finished parsing run0000_0000 timepoint 0.3877
 finished parsing run0000_0000 timepoint 0.38775
 finished parsing run0000_0000 timepoint 0.3878
 finished parsing run0000_0000 timepoint 0.38785
 finished parsing run0000_0000 timepoint 0.3879
 finished parsing run0000_0000 timepoint 0.38795
 finished parsing run0000_0000 timepoint 0.388
 finished parsing run0000_0000 timepoint 0.38805
 finished parsing run0000_0000 timepoint 0.3881
 finished parsing run0000_0000 timepoint 0.38815
 finished parsing run0000_0000 timepoint 0.3882
 finished parsing run0000_0000 timepoint 0.38825
 finished parsing run0000_0000 timepoint 0.3883
 finished parsing run0000_0000 timepoint 0.38835
 finished parsing run0000_0000 timepoint 0.3884
 finished parsing run0000_0000 timepoint 0.38845
 finished parsing run0000_0000 timepoint 0.3885
 finished parsing run0000_0000 timepoint 0.38855
 finished parsing run0000_0000 timepoint 0.3886
 finished parsing run0000_0000 timepoint 0.38865
 finished parsing run0000_0000 timepoint 0.3887
 finished parsing run0000_0000 timepoint 0.38875
 finished parsing run0000_0000 timepoint 0.3888
 finished parsing run0000_0000 timepoint 0.38885
 finished parsing run0000_0000 timepoint 0.3889
 finished parsing run0000_0000 timepoint 0.38895
 finished parsing run0000_0000 timepoint 0.389
 finished parsing run0000_0000 timepoint 0.38905
 finished parsing run0000_0000 timepoint 0.3891
 finished parsing run0000_0000 timepoint 0.38915
 finished parsing run0000_0000 timepoint 0.3892
 finished parsing run0000_0000 timepoint 0.38925
 finished parsing run0000_0000 timepoint 0.3893
 finished parsing run0000_0000 timepoint 0.38935
 finished parsing run0000_0000 timepoint 0.3894
 finished parsing run0000_0000 timepoint 0.38945
 finished parsing run0000_0000 timepoint 0.3895
 finished parsing run0000_0000 timepoint 0.38955
 finished parsing run0000_0000 timepoint 0.3896
 finished parsing run0000_0000 timepoint 0.38965
 finished parsing run0000_0000 timepoint 0.3897
 finished parsing run0000_0000 timepoint 0.38975
 finished parsing run0000_0000 timepoint 0.3898
 finished parsing run0000_0000 timepoint 0.38985
 finished parsing run0000_0000 timepoint 0.3899
 finished parsing run0000_0000 timepoint 0.38995
 finished parsing run0000_0000 timepoint 0.39
 finished parsing run0000_0000 timepoint 0.39005
 finished parsing run0000_0000 timepoint 0.3901
 finished parsing run0000_0000 timepoint 0.39015
 finished parsing run0000_0000 timepoint 0.3902
 finished parsing run0000_0000 timepoint 0.39025
 finished parsing run0000_0000 timepoint 0.3903
 finished parsing run0000_0000 timepoint 0.39035
 finished parsing run0000_0000 timepoint 0.3904
 finished parsing run0000_0000 timepoint 0.39045
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.37755
 finished parsing run0000_0000 timepoint 0.3776
 finished parsing run0000_0000 timepoint 0.37765
 finished parsing run0000_0000 timepoint 0.3777
 finished parsing run0000_0000 timepoint 0.37775
 finished parsing run0000_0000 timepoint 0.3778
 finished parsing run0000_0000 timepoint 0.37785
 finished parsing run0000_0000 timepoint 0.3779
 finished parsing run0000_0000 timepoint 0.37795
 finished parsing run0000_0000 timepoint 0.378
 finished parsing run0000_0000 timepoint 0.37805
 finished parsing run0000_0000 timepoint 0.3781
 finished parsing run0000_0000 timepoint 0.37815
 finished parsing run0000_0000 timepoint 0.3782
 finished parsing run0000_0000 timepoint 0.37825
 finished parsing run0000_0000 timepoint 0.3783
 finished parsing run0000_0000 timepoint 0.37835
 finished parsing run0000_0000 timepoint 0.3784
 finished parsing run0000_0000 timepoint 0.37845
 finished parsing run0000_0000 timepoint 0.3785
 finished parsing run0000_0000 timepoint 0.37855
 finished parsing run0000_0000 timepoint 0.3786
 finished parsing run0000_0000 timepoint 0.37865
 finished parsing run0000_0000 timepoint 0.3787
 finished parsing run0000_0000 timepoint 0.37875
 finished parsing run0000_0000 timepoint 0.3788
 finished parsing run0000_0000 timepoint 0.37885
 finished parsing run0000_0000 timepoint 0.3789
 finished parsing run0000_0000 timepoint 0.37895
 finished parsing run0000_0000 timepoint 0.379
 finished parsing run0000_0000 timepoint 0.37905
 finished parsing run0000_0000 timepoint 0.3791
 finished parsing run0000_0000 timepoint 0.37915
 finished parsing run0000_0000 timepoint 0.3792
 finished parsing run0000_0000 timepoint 0.37925
 finished parsing run0000_0000 timepoint 0.3793
 finished parsing run0000_0000 timepoint 0.37935
 finished parsing run0000_0000 timepoint 0.3794
 finished parsing run0000_0000 timepoint 0.37945
 finished parsing run0000_0000 timepoint 0.3795
 finished parsing run0000_0000 timepoint 0.37955
 finished parsing run0000_0000 timepoint 0.3796
 finished parsing run0000_0000 timepoint 0.37965
 finished parsing run0000_0000 timepoint 0.3797
 finished parsing run0000_0000 timepoint 0.37975
 finished parsing run0000_0000 timepoint 0.3798
 finished parsing run0000_0000 timepoint 0.37985
 finished parsing run0000_0000 timepoint 0.3799
 finished parsing run0000_0000 timepoint 0.37995
 finished parsing run0000_0000 timepoint 0.38
 finished parsing run0000_0000 timepoint 0.38005
 finished parsing run0000_0000 timepoint 0.3801
 finished parsing run0000_0000 timepoint 0.38015
 finished parsing run0000_0000 timepoint 0.3802
 finished parsing run0000_0000 timepoint 0.38025
 finished parsing run0000_0000 timepoint 0.3803
 finished parsing run0000_0000 timepoint 0.38035
 finished parsing run0000_0000 timepoint 0.3804
 finished parsing run0000_0000 timepoint 0.38045
 finished parsing run0000_0000 timepoint 0.3805
 finished parsing run0000_0000 timepoint 0.38055
 finished parsing run0000_0000 timepoint 0.3806
 finished parsing run0000_0000 timepoint 0.38065
 finished parsing run0000_0000 timepoint 0.3807
 finished parsing run0000_0000 timepoint 0.38075
 finished parsing run0000_0000 timepoint 0.3808
 finished parsing run0000_0000 timepoint 0.38085
 finished parsing run0000_0000 timepoint 0.3809
 finished parsing run0000_0000 timepoint 0.38095
 finished parsing run0000_0000 timepoint 0.381
 finished parsing run0000_0000 timepoint 0.38105
 finished parsing run0000_0000 timepoint 0.3811
 finished parsing run0000_0000 timepoint 0.38115
 finished parsing run0000_0000 timepoint 0.3812
 finished parsing run0000_0000 timepoint 0.38125
 finished parsing run0000_0000 timepoint 0.3813
 finished parsing run0000_0000 timepoint 0.38135
 finished parsing run0000_0000 timepoint 0.3814
 finished parsing run0000_0000 timepoint 0.38145
 finished parsing run0000_0000 timepoint 0.3815
 finished parsing run0000_0000 timepoint 0.38155
 finished parsing run0000_0000 timepoint 0.3816
 finished parsing run0000_0000 timepoint 0.38165
 finished parsing run0000_0000 timepoint 0.3817
 finished parsing run0000_0000 timepoint 0.38175
 finished parsing run0000_0000 timepoint 0.3818
 finished parsing run0000_0000 timepoint 0.38185
 finished parsing run0000_0000 timepoint 0.3819
 finished parsing run0000_0000 timepoint 0.38195
 finished parsing run0000_0000 timepoint 0.382
 finished parsing run0000_0000 timepoint 0.38205
 finished parsing run0000_0000 timepoint 0.3821
 finished parsing run0000_0000 timepoint 0.38215
 finished parsing run0000_0000 timepoint 0.3822
 finished parsing run0000_0000 timepoint 0.38225
 finished parsing run0000_0000 timepoint 0.3823
 finished parsing run0000_0000 timepoint 0.38235
 finished parsing run0000_0000 timepoint 0.3824
 finished parsing run0000_0000 timepoint 0.38245
 finished parsing run0000_0000 timepoint 0.3825
 finished parsing run0000_0000 timepoint 0.38255
 finished parsing run0000_0000 timepoint 0.3826
 finished parsing run0000_0000 timepoint 0.38265
 finished parsing run0000_0000 timepoint 0.3827
 finished parsing run0000_0000 timepoint 0.38275
 finished parsing run0000_0000 timepoint 0.3828
 finished parsing run0000_0000 timepoint 0.38285
 finished parsing run0000_0000 timepoint 0.3829
 finished parsing run0000_0000 timepoint 0.38295
 finished parsing run0000_0000 timepoint 0.383
 finished parsing run0000_0000 timepoint 0.38305
 finished parsing run0000_0000 timepoint 0.3831
 finished parsing run0000_0000 timepoint 0.38315
 finished parsing run0000_0000 timepoint 0.3832
 finished parsing run0000_0000 timepoint 0.38325
 finished parsing run0000_0000 timepoint 0.3833
 finished parsing run0000_0000 timepoint 0.38335
 finished parsing run0000_0000 timepoint 0.3834
 finished parsing run0000_0000 timepoint 0.38345
 finished parsing run0000_0000 timepoint 0.3835
 finished parsing run0000_0000 timepoint 0.38355
 finished parsing run0000_0000 timepoint 0.3836
 finished parsing run0000_0000 timepoint 0.38365
 finished parsing run0000_0000 timepoint 0.3837
 finished parsing run0000_0000 timepoint 0.38375
 finished parsing run0000_0000 timepoint 0.3838
 finished parsing run0000_0000 timepoint 0.38385
 finished parsing run0000_0000 timepoint 0.3839
 finished parsing run0000_0000 timepoint 0.38395
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.3711
 finished parsing run0000_0000 timepoint 0.37115
 finished parsing run0000_0000 timepoint 0.3712
 finished parsing run0000_0000 timepoint 0.37125
 finished parsing run0000_0000 timepoint 0.3713
 finished parsing run0000_0000 timepoint 0.37135
 finished parsing run0000_0000 timepoint 0.3714
 finished parsing run0000_0000 timepoint 0.37145
 finished parsing run0000_0000 timepoint 0.3715
 finished parsing run0000_0000 timepoint 0.37155
 finished parsing run0000_0000 timepoint 0.3716
 finished parsing run0000_0000 timepoint 0.37165
 finished parsing run0000_0000 timepoint 0.3717
 finished parsing run0000_0000 timepoint 0.37175
 finished parsing run0000_0000 timepoint 0.3718
 finished parsing run0000_0000 timepoint 0.37185
 finished parsing run0000_0000 timepoint 0.3719
 finished parsing run0000_0000 timepoint 0.37195
 finished parsing run0000_0000 timepoint 0.372
 finished parsing run0000_0000 timepoint 0.37205
 finished parsing run0000_0000 timepoint 0.3721
 finished parsing run0000_0000 timepoint 0.37215
 finished parsing run0000_0000 timepoint 0.3722
 finished parsing run0000_0000 timepoint 0.37225
 finished parsing run0000_0000 timepoint 0.3723
 finished parsing run0000_0000 timepoint 0.37235
 finished parsing run0000_0000 timepoint 0.3724
 finished parsing run0000_0000 timepoint 0.37245
 finished parsing run0000_0000 timepoint 0.3725
 finished parsing run0000_0000 timepoint 0.37255
 finished parsing run0000_0000 timepoint 0.3726
 finished parsing run0000_0000 timepoint 0.37265
 finished parsing run0000_0000 timepoint 0.3727
 finished parsing run0000_0000 timepoint 0.37275
 finished parsing run0000_0000 timepoint 0.3728
 finished parsing run0000_0000 timepoint 0.37285
 finished parsing run0000_0000 timepoint 0.3729
 finished parsing run0000_0000 timepoint 0.37295
 finished parsing run0000_0000 timepoint 0.373
 finished parsing run0000_0000 timepoint 0.37305
 finished parsing run0000_0000 timepoint 0.3731
 finished parsing run0000_0000 timepoint 0.37315
 finished parsing run0000_0000 timepoint 0.3732
 finished parsing run0000_0000 timepoint 0.37325
 finished parsing run0000_0000 timepoint 0.3733
 finished parsing run0000_0000 timepoint 0.37335
 finished parsing run0000_0000 timepoint 0.3734
 finished parsing run0000_0000 timepoint 0.37345
 finished parsing run0000_0000 timepoint 0.3735
 finished parsing run0000_0000 timepoint 0.37355
 finished parsing run0000_0000 timepoint 0.3736
 finished parsing run0000_0000 timepoint 0.37365
 finished parsing run0000_0000 timepoint 0.3737
 finished parsing run0000_0000 timepoint 0.37375
 finished parsing run0000_0000 timepoint 0.3738
 finished parsing run0000_0000 timepoint 0.37385
 finished parsing run0000_0000 timepoint 0.3739
 finished parsing run0000_0000 timepoint 0.37395
 finished parsing run0000_0000 timepoint 0.374
 finished parsing run0000_0000 timepoint 0.37405
 finished parsing run0000_0000 timepoint 0.3741
 finished parsing run0000_0000 timepoint 0.37415
 finished parsing run0000_0000 timepoint 0.3742
 finished parsing run0000_0000 timepoint 0.37425
 finished parsing run0000_0000 timepoint 0.3743
 finished parsing run0000_0000 timepoint 0.37435
 finished parsing run0000_0000 timepoint 0.3744
 finished parsing run0000_0000 timepoint 0.37445
 finished parsing run0000_0000 timepoint 0.3745
 finished parsing run0000_0000 timepoint 0.37455
 finished parsing run0000_0000 timepoint 0.3746
 finished parsing run0000_0000 timepoint 0.37465
 finished parsing run0000_0000 timepoint 0.3747
 finished parsing run0000_0000 timepoint 0.37475
 finished parsing run0000_0000 timepoint 0.3748
 finished parsing run0000_0000 timepoint 0.37485
 finished parsing run0000_0000 timepoint 0.3749
 finished parsing run0000_0000 timepoint 0.37495
 finished parsing run0000_0000 timepoint 0.375
 finished parsing run0000_0000 timepoint 0.37505
 finished parsing run0000_0000 timepoint 0.3751
 finished parsing run0000_0000 timepoint 0.37515
 finished parsing run0000_0000 timepoint 0.3752
 finished parsing run0000_0000 timepoint 0.37525
 finished parsing run0000_0000 timepoint 0.3753
 finished parsing run0000_0000 timepoint 0.37535
 finished parsing run0000_0000 timepoint 0.3754
 finished parsing run0000_0000 timepoint 0.37545
 finished parsing run0000_0000 timepoint 0.3755
 finished parsing run0000_0000 timepoint 0.37555
 finished parsing run0000_0000 timepoint 0.3756
 finished parsing run0000_0000 timepoint 0.37565
 finished parsing run0000_0000 timepoint 0.3757
 finished parsing run0000_0000 timepoint 0.37575
 finished parsing run0000_0000 timepoint 0.3758
 finished parsing run0000_0000 timepoint 0.37585
 finished parsing run0000_0000 timepoint 0.3759
 finished parsing run0000_0000 timepoint 0.37595
 finished parsing run0000_0000 timepoint 0.376
 finished parsing run0000_0000 timepoint 0.37605
 finished parsing run0000_0000 timepoint 0.3761
 finished parsing run0000_0000 timepoint 0.37615
 finished parsing run0000_0000 timepoint 0.3762
 finished parsing run0000_0000 timepoint 0.37625
 finished parsing run0000_0000 timepoint 0.3763
 finished parsing run0000_0000 timepoint 0.37635
 finished parsing run0000_0000 timepoint 0.3764
 finished parsing run0000_0000 timepoint 0.37645
 finished parsing run0000_0000 timepoint 0.3765
 finished parsing run0000_0000 timepoint 0.37655
 finished parsing run0000_0000 timepoint 0.3766
 finished parsing run0000_0000 timepoint 0.37665
 finished parsing run0000_0000 timepoint 0.3767
 finished parsing run0000_0000 timepoint 0.37675
 finished parsing run0000_0000 timepoint 0.3768
 finished parsing run0000_0000 timepoint 0.37685
 finished parsing run0000_0000 timepoint 0.3769
 finished parsing run0000_0000 timepoint 0.37695
 finished parsing run0000_0000 timepoint 0.377
 finished parsing run0000_0000 timepoint 0.37705
 finished parsing run0000_0000 timepoint 0.3771
 finished parsing run0000_0000 timepoint 0.37715
 finished parsing run0000_0000 timepoint 0.3772
 finished parsing run0000_0000 timepoint 0.37725
 finished parsing run0000_0000 timepoint 0.3773
 finished parsing run0000_0000 timepoint 0.37735
 finished parsing run0000_0000 timepoint 0.3774
 finished parsing run0000_0000 timepoint 0.37745
 finished parsing run0000_0000 timepoint 0.3775
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.36455
 finished parsing run0000_0000 timepoint 0.3646
 finished parsing run0000_0000 timepoint 0.36465
 finished parsing run0000_0000 timepoint 0.3647
 finished parsing run0000_0000 timepoint 0.36475
 finished parsing run0000_0000 timepoint 0.3648
 finished parsing run0000_0000 timepoint 0.36485
 finished parsing run0000_0000 timepoint 0.3649
 finished parsing run0000_0000 timepoint 0.36495
 finished parsing run0000_0000 timepoint 0.365
 finished parsing run0000_0000 timepoint 0.36505
 finished parsing run0000_0000 timepoint 0.3651
 finished parsing run0000_0000 timepoint 0.36515
 finished parsing run0000_0000 timepoint 0.3652
 finished parsing run0000_0000 timepoint 0.36525
 finished parsing run0000_0000 timepoint 0.3653
 finished parsing run0000_0000 timepoint 0.36535
 finished parsing run0000_0000 timepoint 0.3654
 finished parsing run0000_0000 timepoint 0.36545
 finished parsing run0000_0000 timepoint 0.3655
 finished parsing run0000_0000 timepoint 0.36555
 finished parsing run0000_0000 timepoint 0.3656
 finished parsing run0000_0000 timepoint 0.36565
 finished parsing run0000_0000 timepoint 0.3657
 finished parsing run0000_0000 timepoint 0.36575
 finished parsing run0000_0000 timepoint 0.3658
 finished parsing run0000_0000 timepoint 0.36585
 finished parsing run0000_0000 timepoint 0.3659
 finished parsing run0000_0000 timepoint 0.36595
 finished parsing run0000_0000 timepoint 0.366
 finished parsing run0000_0000 timepoint 0.36605
 finished parsing run0000_0000 timepoint 0.3661
 finished parsing run0000_0000 timepoint 0.36615
 finished parsing run0000_0000 timepoint 0.3662
 finished parsing run0000_0000 timepoint 0.36625
 finished parsing run0000_0000 timepoint 0.3663
 finished parsing run0000_0000 timepoint 0.36635
 finished parsing run0000_0000 timepoint 0.3664
 finished parsing run0000_0000 timepoint 0.36645
 finished parsing run0000_0000 timepoint 0.3665
 finished parsing run0000_0000 timepoint 0.36655
 finished parsing run0000_0000 timepoint 0.3666
 finished parsing run0000_0000 timepoint 0.36665
 finished parsing run0000_0000 timepoint 0.3667
 finished parsing run0000_0000 timepoint 0.36675
 finished parsing run0000_0000 timepoint 0.3668
 finished parsing run0000_0000 timepoint 0.36685
 finished parsing run0000_0000 timepoint 0.3669
 finished parsing run0000_0000 timepoint 0.36695
 finished parsing run0000_0000 timepoint 0.367
 finished parsing run0000_0000 timepoint 0.36705
 finished parsing run0000_0000 timepoint 0.3671
 finished parsing run0000_0000 timepoint 0.36715
 finished parsing run0000_0000 timepoint 0.3672
 finished parsing run0000_0000 timepoint 0.36725
 finished parsing run0000_0000 timepoint 0.3673
 finished parsing run0000_0000 timepoint 0.36735
 finished parsing run0000_0000 timepoint 0.3674
 finished parsing run0000_0000 timepoint 0.36745
 finished parsing run0000_0000 timepoint 0.3675
 finished parsing run0000_0000 timepoint 0.36755
 finished parsing run0000_0000 timepoint 0.3676
 finished parsing run0000_0000 timepoint 0.36765
 finished parsing run0000_0000 timepoint 0.3677
 finished parsing run0000_0000 timepoint 0.36775
 finished parsing run0000_0000 timepoint 0.3678
 finished parsing run0000_0000 timepoint 0.36785
 finished parsing run0000_0000 timepoint 0.3679
 finished parsing run0000_0000 timepoint 0.36795
 finished parsing run0000_0000 timepoint 0.368
 finished parsing run0000_0000 timepoint 0.36805
 finished parsing run0000_0000 timepoint 0.3681
 finished parsing run0000_0000 timepoint 0.36815
 finished parsing run0000_0000 timepoint 0.3682
 finished parsing run0000_0000 timepoint 0.36825
 finished parsing run0000_0000 timepoint 0.3683
 finished parsing run0000_0000 timepoint 0.36835
 finished parsing run0000_0000 timepoint 0.3684
 finished parsing run0000_0000 timepoint 0.36845
 finished parsing run0000_0000 timepoint 0.3685
 finished parsing run0000_0000 timepoint 0.36855
 finished parsing run0000_0000 timepoint 0.3686
 finished parsing run0000_0000 timepoint 0.36865
 finished parsing run0000_0000 timepoint 0.3687
 finished parsing run0000_0000 timepoint 0.36875
 finished parsing run0000_0000 timepoint 0.3688
 finished parsing run0000_0000 timepoint 0.36885
 finished parsing run0000_0000 timepoint 0.3689
 finished parsing run0000_0000 timepoint 0.36895
 finished parsing run0000_0000 timepoint 0.369
 finished parsing run0000_0000 timepoint 0.36905
 finished parsing run0000_0000 timepoint 0.3691
 finished parsing run0000_0000 timepoint 0.36915
 finished parsing run0000_0000 timepoint 0.3692
 finished parsing run0000_0000 timepoint 0.36925
 finished parsing run0000_0000 timepoint 0.3693
 finished parsing run0000_0000 timepoint 0.36935
 finished parsing run0000_0000 timepoint 0.3694
 finished parsing run0000_0000 timepoint 0.36945
 finished parsing run0000_0000 timepoint 0.3695
 finished parsing run0000_0000 timepoint 0.36955
 finished parsing run0000_0000 timepoint 0.3696
 finished parsing run0000_0000 timepoint 0.36965
 finished parsing run0000_0000 timepoint 0.3697
 finished parsing run0000_0000 timepoint 0.36975
 finished parsing run0000_0000 timepoint 0.3698
 finished parsing run0000_0000 timepoint 0.36985
 finished parsing run0000_0000 timepoint 0.3699
 finished parsing run0000_0000 timepoint 0.36995
 finished parsing run0000_0000 timepoint 0.37
 finished parsing run0000_0000 timepoint 0.37005
 finished parsing run0000_0000 timepoint 0.3701
 finished parsing run0000_0000 timepoint 0.37015
 finished parsing run0000_0000 timepoint 0.3702
 finished parsing run0000_0000 timepoint 0.37025
 finished parsing run0000_0000 timepoint 0.3703
 finished parsing run0000_0000 timepoint 0.37035
 finished parsing run0000_0000 timepoint 0.3704
 finished parsing run0000_0000 timepoint 0.37045
 finished parsing run0000_0000 timepoint 0.3705
 finished parsing run0000_0000 timepoint 0.37055
 finished parsing run0000_0000 timepoint 0.3706
 finished parsing run0000_0000 timepoint 0.37065
 finished parsing run0000_0000 timepoint 0.3707
 finished parsing run0000_0000 timepoint 0.37075
 finished parsing run0000_0000 timepoint 0.3708
 finished parsing run0000_0000 timepoint 0.37085
 finished parsing run0000_0000 timepoint 0.3709
 finished parsing run0000_0000 timepoint 0.37095
 finished parsing run0000_0000 timepoint 0.371
 finished parsing run0000_0000 timepoint 0.37105
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.35795
 finished parsing run0000_0000 timepoint 0.358
 finished parsing run0000_0000 timepoint 0.35805
 finished parsing run0000_0000 timepoint 0.3581
 finished parsing run0000_0000 timepoint 0.35815
 finished parsing run0000_0000 timepoint 0.3582
 finished parsing run0000_0000 timepoint 0.35825
 finished parsing run0000_0000 timepoint 0.3583
 finished parsing run0000_0000 timepoint 0.35835
 finished parsing run0000_0000 timepoint 0.3584
 finished parsing run0000_0000 timepoint 0.35845
 finished parsing run0000_0000 timepoint 0.3585
 finished parsing run0000_0000 timepoint 0.35855
 finished parsing run0000_0000 timepoint 0.3586
 finished parsing run0000_0000 timepoint 0.35865
 finished parsing run0000_0000 timepoint 0.3587
 finished parsing run0000_0000 timepoint 0.35875
 finished parsing run0000_0000 timepoint 0.3588
 finished parsing run0000_0000 timepoint 0.35885
 finished parsing run0000_0000 timepoint 0.3589
 finished parsing run0000_0000 timepoint 0.35895
 finished parsing run0000_0000 timepoint 0.359
 finished parsing run0000_0000 timepoint 0.35905
 finished parsing run0000_0000 timepoint 0.3591
 finished parsing run0000_0000 timepoint 0.35915
 finished parsing run0000_0000 timepoint 0.3592
 finished parsing run0000_0000 timepoint 0.35925
 finished parsing run0000_0000 timepoint 0.3593
 finished parsing run0000_0000 timepoint 0.35935
 finished parsing run0000_0000 timepoint 0.3594
 finished parsing run0000_0000 timepoint 0.35945
 finished parsing run0000_0000 timepoint 0.3595
 finished parsing run0000_0000 timepoint 0.35955
 finished parsing run0000_0000 timepoint 0.3596
 finished parsing run0000_0000 timepoint 0.35965
 finished parsing run0000_0000 timepoint 0.3597
 finished parsing run0000_0000 timepoint 0.35975
 finished parsing run0000_0000 timepoint 0.3598
 finished parsing run0000_0000 timepoint 0.35985
 finished parsing run0000_0000 timepoint 0.3599
 finished parsing run0000_0000 timepoint 0.35995
 finished parsing run0000_0000 timepoint 0.36
 finished parsing run0000_0000 timepoint 0.36005
 finished parsing run0000_0000 timepoint 0.3601
 finished parsing run0000_0000 timepoint 0.36015
 finished parsing run0000_0000 timepoint 0.3602
 finished parsing run0000_0000 timepoint 0.36025
 finished parsing run0000_0000 timepoint 0.3603
 finished parsing run0000_0000 timepoint 0.36035
 finished parsing run0000_0000 timepoint 0.3604
 finished parsing run0000_0000 timepoint 0.36045
 finished parsing run0000_0000 timepoint 0.3605
 finished parsing run0000_0000 timepoint 0.36055
 finished parsing run0000_0000 timepoint 0.3606
 finished parsing run0000_0000 timepoint 0.36065
 finished parsing run0000_0000 timepoint 0.3607
 finished parsing run0000_0000 timepoint 0.36075
 finished parsing run0000_0000 timepoint 0.3608
 finished parsing run0000_0000 timepoint 0.36085
 finished parsing run0000_0000 timepoint 0.3609
 finished parsing run0000_0000 timepoint 0.36095
 finished parsing run0000_0000 timepoint 0.361
 finished parsing run0000_0000 timepoint 0.36105
 finished parsing run0000_0000 timepoint 0.3611
 finished parsing run0000_0000 timepoint 0.36115
 finished parsing run0000_0000 timepoint 0.3612
 finished parsing run0000_0000 timepoint 0.36125
 finished parsing run0000_0000 timepoint 0.3613
 finished parsing run0000_0000 timepoint 0.36135
 finished parsing run0000_0000 timepoint 0.3614
 finished parsing run0000_0000 timepoint 0.36145
 finished parsing run0000_0000 timepoint 0.3615
 finished parsing run0000_0000 timepoint 0.36155
 finished parsing run0000_0000 timepoint 0.3616
 finished parsing run0000_0000 timepoint 0.36165
 finished parsing run0000_0000 timepoint 0.3617
 finished parsing run0000_0000 timepoint 0.36175
 finished parsing run0000_0000 timepoint 0.3618
 finished parsing run0000_0000 timepoint 0.36185
 finished parsing run0000_0000 timepoint 0.3619
 finished parsing run0000_0000 timepoint 0.36195
 finished parsing run0000_0000 timepoint 0.362
 finished parsing run0000_0000 timepoint 0.36205
 finished parsing run0000_0000 timepoint 0.3621
 finished parsing run0000_0000 timepoint 0.36215
 finished parsing run0000_0000 timepoint 0.3622
 finished parsing run0000_0000 timepoint 0.36225
 finished parsing run0000_0000 timepoint 0.3623
 finished parsing run0000_0000 timepoint 0.36235
 finished parsing run0000_0000 timepoint 0.3624
 finished parsing run0000_0000 timepoint 0.36245
 finished parsing run0000_0000 timepoint 0.3625
 finished parsing run0000_0000 timepoint 0.36255
 finished parsing run0000_0000 timepoint 0.3626
 finished parsing run0000_0000 timepoint 0.36265
 finished parsing run0000_0000 timepoint 0.3627
 finished parsing run0000_0000 timepoint 0.36275
 finished parsing run0000_0000 timepoint 0.3628
 finished parsing run0000_0000 timepoint 0.36285
 finished parsing run0000_0000 timepoint 0.3629
 finished parsing run0000_0000 timepoint 0.36295
 finished parsing run0000_0000 timepoint 0.363
 finished parsing run0000_0000 timepoint 0.36305
 finished parsing run0000_0000 timepoint 0.3631
 finished parsing run0000_0000 timepoint 0.36315
 finished parsing run0000_0000 timepoint 0.3632
 finished parsing run0000_0000 timepoint 0.36325
 finished parsing run0000_0000 timepoint 0.3633
 finished parsing run0000_0000 timepoint 0.36335
 finished parsing run0000_0000 timepoint 0.3634
 finished parsing run0000_0000 timepoint 0.36345
 finished parsing run0000_0000 timepoint 0.3635
 finished parsing run0000_0000 timepoint 0.36355
 finished parsing run0000_0000 timepoint 0.3636
 finished parsing run0000_0000 timepoint 0.36365
 finished parsing run0000_0000 timepoint 0.3637
 finished parsing run0000_0000 timepoint 0.36375
 finished parsing run0000_0000 timepoint 0.3638
 finished parsing run0000_0000 timepoint 0.36385
 finished parsing run0000_0000 timepoint 0.3639
 finished parsing run0000_0000 timepoint 0.36395
 finished parsing run0000_0000 timepoint 0.364
 finished parsing run0000_0000 timepoint 0.36405
 finished parsing run0000_0000 timepoint 0.3641
 finished parsing run0000_0000 timepoint 0.36415
 finished parsing run0000_0000 timepoint 0.3642
 finished parsing run0000_0000 timepoint 0.36425
 finished parsing run0000_0000 timepoint 0.3643
 finished parsing run0000_0000 timepoint 0.36435
 finished parsing run0000_0000 timepoint 0.3644
 finished parsing run0000_0000 timepoint 0.36445
 finished parsing run0000_0000 timepoint 0.3645
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.35145
 finished parsing run0000_0000 timepoint 0.3515
 finished parsing run0000_0000 timepoint 0.35155
 finished parsing run0000_0000 timepoint 0.3516
 finished parsing run0000_0000 timepoint 0.35165
 finished parsing run0000_0000 timepoint 0.3517
 finished parsing run0000_0000 timepoint 0.35175
 finished parsing run0000_0000 timepoint 0.3518
 finished parsing run0000_0000 timepoint 0.35185
 finished parsing run0000_0000 timepoint 0.3519
 finished parsing run0000_0000 timepoint 0.35195
 finished parsing run0000_0000 timepoint 0.352
 finished parsing run0000_0000 timepoint 0.35205
 finished parsing run0000_0000 timepoint 0.3521
 finished parsing run0000_0000 timepoint 0.35215
 finished parsing run0000_0000 timepoint 0.3522
 finished parsing run0000_0000 timepoint 0.35225
 finished parsing run0000_0000 timepoint 0.3523
 finished parsing run0000_0000 timepoint 0.35235
 finished parsing run0000_0000 timepoint 0.3524
 finished parsing run0000_0000 timepoint 0.35245
 finished parsing run0000_0000 timepoint 0.3525
 finished parsing run0000_0000 timepoint 0.35255
 finished parsing run0000_0000 timepoint 0.3526
 finished parsing run0000_0000 timepoint 0.35265
 finished parsing run0000_0000 timepoint 0.3527
 finished parsing run0000_0000 timepoint 0.35275
 finished parsing run0000_0000 timepoint 0.3528
 finished parsing run0000_0000 timepoint 0.35285
 finished parsing run0000_0000 timepoint 0.3529
 finished parsing run0000_0000 timepoint 0.35295
 finished parsing run0000_0000 timepoint 0.353
 finished parsing run0000_0000 timepoint 0.35305
 finished parsing run0000_0000 timepoint 0.3531
 finished parsing run0000_0000 timepoint 0.35315
 finished parsing run0000_0000 timepoint 0.3532
 finished parsing run0000_0000 timepoint 0.35325
 finished parsing run0000_0000 timepoint 0.3533
 finished parsing run0000_0000 timepoint 0.35335
 finished parsing run0000_0000 timepoint 0.3534
 finished parsing run0000_0000 timepoint 0.35345
 finished parsing run0000_0000 timepoint 0.3535
 finished parsing run0000_0000 timepoint 0.35355
 finished parsing run0000_0000 timepoint 0.3536
 finished parsing run0000_0000 timepoint 0.35365
 finished parsing run0000_0000 timepoint 0.3537
 finished parsing run0000_0000 timepoint 0.35375
 finished parsing run0000_0000 timepoint 0.3538
 finished parsing run0000_0000 timepoint 0.35385
 finished parsing run0000_0000 timepoint 0.3539
 finished parsing run0000_0000 timepoint 0.35395
 finished parsing run0000_0000 timepoint 0.354
 finished parsing run0000_0000 timepoint 0.35405
 finished parsing run0000_0000 timepoint 0.3541
 finished parsing run0000_0000 timepoint 0.35415
 finished parsing run0000_0000 timepoint 0.3542
 finished parsing run0000_0000 timepoint 0.35425
 finished parsing run0000_0000 timepoint 0.3543
 finished parsing run0000_0000 timepoint 0.35435
 finished parsing run0000_0000 timepoint 0.3544
 finished parsing run0000_0000 timepoint 0.35445
 finished parsing run0000_0000 timepoint 0.3545
 finished parsing run0000_0000 timepoint 0.35455
 finished parsing run0000_0000 timepoint 0.3546
 finished parsing run0000_0000 timepoint 0.35465
 finished parsing run0000_0000 timepoint 0.3547
 finished parsing run0000_0000 timepoint 0.35475
 finished parsing run0000_0000 timepoint 0.3548
 finished parsing run0000_0000 timepoint 0.35485
 finished parsing run0000_0000 timepoint 0.3549
 finished parsing run0000_0000 timepoint 0.35495
 finished parsing run0000_0000 timepoint 0.355
 finished parsing run0000_0000 timepoint 0.35505
 finished parsing run0000_0000 timepoint 0.3551
 finished parsing run0000_0000 timepoint 0.35515
 finished parsing run0000_0000 timepoint 0.3552
 finished parsing run0000_0000 timepoint 0.35525
 finished parsing run0000_0000 timepoint 0.3553
 finished parsing run0000_0000 timepoint 0.35535
 finished parsing run0000_0000 timepoint 0.3554
 finished parsing run0000_0000 timepoint 0.35545
 finished parsing run0000_0000 timepoint 0.3555
 finished parsing run0000_0000 timepoint 0.35555
 finished parsing run0000_0000 timepoint 0.3556
 finished parsing run0000_0000 timepoint 0.35565
 finished parsing run0000_0000 timepoint 0.3557
 finished parsing run0000_0000 timepoint 0.35575
 finished parsing run0000_0000 timepoint 0.3558
 finished parsing run0000_0000 timepoint 0.35585
 finished parsing run0000_0000 timepoint 0.3559
 finished parsing run0000_0000 timepoint 0.35595
 finished parsing run0000_0000 timepoint 0.356
 finished parsing run0000_0000 timepoint 0.35605
 finished parsing run0000_0000 timepoint 0.3561
 finished parsing run0000_0000 timepoint 0.35615
 finished parsing run0000_0000 timepoint 0.3562
 finished parsing run0000_0000 timepoint 0.35625
 finished parsing run0000_0000 timepoint 0.3563
 finished parsing run0000_0000 timepoint 0.35635
 finished parsing run0000_0000 timepoint 0.3564
 finished parsing run0000_0000 timepoint 0.35645
 finished parsing run0000_0000 timepoint 0.3565
 finished parsing run0000_0000 timepoint 0.35655
 finished parsing run0000_0000 timepoint 0.3566
 finished parsing run0000_0000 timepoint 0.35665
 finished parsing run0000_0000 timepoint 0.3567
 finished parsing run0000_0000 timepoint 0.35675
 finished parsing run0000_0000 timepoint 0.3568
 finished parsing run0000_0000 timepoint 0.35685
 finished parsing run0000_0000 timepoint 0.3569
 finished parsing run0000_0000 timepoint 0.35695
 finished parsing run0000_0000 timepoint 0.357
 finished parsing run0000_0000 timepoint 0.35705
 finished parsing run0000_0000 timepoint 0.3571
 finished parsing run0000_0000 timepoint 0.35715
 finished parsing run0000_0000 timepoint 0.3572
 finished parsing run0000_0000 timepoint 0.35725
 finished parsing run0000_0000 timepoint 0.3573
 finished parsing run0000_0000 timepoint 0.35735
 finished parsing run0000_0000 timepoint 0.3574
 finished parsing run0000_0000 timepoint 0.35745
 finished parsing run0000_0000 timepoint 0.3575
 finished parsing run0000_0000 timepoint 0.35755
 finished parsing run0000_0000 timepoint 0.3576
 finished parsing run0000_0000 timepoint 0.35765
 finished parsing run0000_0000 timepoint 0.3577
 finished parsing run0000_0000 timepoint 0.35775
 finished parsing run0000_0000 timepoint 0.3578
 finished parsing run0000_0000 timepoint 0.35785
 finished parsing run0000_0000 timepoint 0.3579
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.345
 finished parsing run0000_0000 timepoint 0.34505
 finished parsing run0000_0000 timepoint 0.3451
 finished parsing run0000_0000 timepoint 0.34515
 finished parsing run0000_0000 timepoint 0.3452
 finished parsing run0000_0000 timepoint 0.34525
 finished parsing run0000_0000 timepoint 0.3453
 finished parsing run0000_0000 timepoint 0.34535
 finished parsing run0000_0000 timepoint 0.3454
 finished parsing run0000_0000 timepoint 0.34545
 finished parsing run0000_0000 timepoint 0.3455
 finished parsing run0000_0000 timepoint 0.34555
 finished parsing run0000_0000 timepoint 0.3456
 finished parsing run0000_0000 timepoint 0.34565
 finished parsing run0000_0000 timepoint 0.3457
 finished parsing run0000_0000 timepoint 0.34575
 finished parsing run0000_0000 timepoint 0.3458
 finished parsing run0000_0000 timepoint 0.34585
 finished parsing run0000_0000 timepoint 0.3459
 finished parsing run0000_0000 timepoint 0.34595
 finished parsing run0000_0000 timepoint 0.346
 finished parsing run0000_0000 timepoint 0.34605
 finished parsing run0000_0000 timepoint 0.3461
 finished parsing run0000_0000 timepoint 0.34615
 finished parsing run0000_0000 timepoint 0.3462
 finished parsing run0000_0000 timepoint 0.34625
 finished parsing run0000_0000 timepoint 0.3463
 finished parsing run0000_0000 timepoint 0.34635
 finished parsing run0000_0000 timepoint 0.3464
 finished parsing run0000_0000 timepoint 0.34645
 finished parsing run0000_0000 timepoint 0.3465
 finished parsing run0000_0000 timepoint 0.34655
 finished parsing run0000_0000 timepoint 0.3466
 finished parsing run0000_0000 timepoint 0.34665
 finished parsing run0000_0000 timepoint 0.3467
 finished parsing run0000_0000 timepoint 0.34675
 finished parsing run0000_0000 timepoint 0.3468
 finished parsing run0000_0000 timepoint 0.34685
 finished parsing run0000_0000 timepoint 0.3469
 finished parsing run0000_0000 timepoint 0.34695
 finished parsing run0000_0000 timepoint 0.347
 finished parsing run0000_0000 timepoint 0.34705
 finished parsing run0000_0000 timepoint 0.3471
 finished parsing run0000_0000 timepoint 0.34715
 finished parsing run0000_0000 timepoint 0.3472
 finished parsing run0000_0000 timepoint 0.34725
 finished parsing run0000_0000 timepoint 0.3473
 finished parsing run0000_0000 timepoint 0.34735
 finished parsing run0000_0000 timepoint 0.3474
 finished parsing run0000_0000 timepoint 0.34745
 finished parsing run0000_0000 timepoint 0.3475
 finished parsing run0000_0000 timepoint 0.34755
 finished parsing run0000_0000 timepoint 0.3476
 finished parsing run0000_0000 timepoint 0.34765
 finished parsing run0000_0000 timepoint 0.3477
 finished parsing run0000_0000 timepoint 0.34775
 finished parsing run0000_0000 timepoint 0.3478
 finished parsing run0000_0000 timepoint 0.34785
 finished parsing run0000_0000 timepoint 0.3479
 finished parsing run0000_0000 timepoint 0.34795
 finished parsing run0000_0000 timepoint 0.348
 finished parsing run0000_0000 timepoint 0.34805
 finished parsing run0000_0000 timepoint 0.3481
 finished parsing run0000_0000 timepoint 0.34815
 finished parsing run0000_0000 timepoint 0.3482
 finished parsing run0000_0000 timepoint 0.34825
 finished parsing run0000_0000 timepoint 0.3483
 finished parsing run0000_0000 timepoint 0.34835
 finished parsing run0000_0000 timepoint 0.3484
 finished parsing run0000_0000 timepoint 0.34845
 finished parsing run0000_0000 timepoint 0.3485
 finished parsing run0000_0000 timepoint 0.34855
 finished parsing run0000_0000 timepoint 0.3486
 finished parsing run0000_0000 timepoint 0.34865
 finished parsing run0000_0000 timepoint 0.3487
 finished parsing run0000_0000 timepoint 0.34875
 finished parsing run0000_0000 timepoint 0.3488
 finished parsing run0000_0000 timepoint 0.34885
 finished parsing run0000_0000 timepoint 0.3489
 finished parsing run0000_0000 timepoint 0.34895
 finished parsing run0000_0000 timepoint 0.349
 finished parsing run0000_0000 timepoint 0.34905
 finished parsing run0000_0000 timepoint 0.3491
 finished parsing run0000_0000 timepoint 0.34915
 finished parsing run0000_0000 timepoint 0.3492
 finished parsing run0000_0000 timepoint 0.34925
 finished parsing run0000_0000 timepoint 0.3493
 finished parsing run0000_0000 timepoint 0.34935
 finished parsing run0000_0000 timepoint 0.3494
 finished parsing run0000_0000 timepoint 0.34945
 finished parsing run0000_0000 timepoint 0.3495
 finished parsing run0000_0000 timepoint 0.34955
 finished parsing run0000_0000 timepoint 0.3496
 finished parsing run0000_0000 timepoint 0.34965
 finished parsing run0000_0000 timepoint 0.3497
 finished parsing run0000_0000 timepoint 0.34975
 finished parsing run0000_0000 timepoint 0.3498
 finished parsing run0000_0000 timepoint 0.34985
 finished parsing run0000_0000 timepoint 0.3499
 finished parsing run0000_0000 timepoint 0.34995
 finished parsing run0000_0000 timepoint 0.35
 finished parsing run0000_0000 timepoint 0.35005
 finished parsing run0000_0000 timepoint 0.3501
 finished parsing run0000_0000 timepoint 0.35015
 finished parsing run0000_0000 timepoint 0.3502
 finished parsing run0000_0000 timepoint 0.35025
 finished parsing run0000_0000 timepoint 0.3503
 finished parsing run0000_0000 timepoint 0.35035
 finished parsing run0000_0000 timepoint 0.3504
 finished parsing run0000_0000 timepoint 0.35045
 finished parsing run0000_0000 timepoint 0.3505
 finished parsing run0000_0000 timepoint 0.35055
 finished parsing run0000_0000 timepoint 0.3506
 finished parsing run0000_0000 timepoint 0.35065
 finished parsing run0000_0000 timepoint 0.3507
 finished parsing run0000_0000 timepoint 0.35075
 finished parsing run0000_0000 timepoint 0.3508
 finished parsing run0000_0000 timepoint 0.35085
 finished parsing run0000_0000 timepoint 0.3509
 finished parsing run0000_0000 timepoint 0.35095
 finished parsing run0000_0000 timepoint 0.351
 finished parsing run0000_0000 timepoint 0.35105
 finished parsing run0000_0000 timepoint 0.3511
 finished parsing run0000_0000 timepoint 0.35115
 finished parsing run0000_0000 timepoint 0.3512
 finished parsing run0000_0000 timepoint 0.35125
 finished parsing run0000_0000 timepoint 0.3513
 finished parsing run0000_0000 timepoint 0.35135
 finished parsing run0000_0000 timepoint 0.3514
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.3383
 finished parsing run0000_0000 timepoint 0.33835
 finished parsing run0000_0000 timepoint 0.3384
 finished parsing run0000_0000 timepoint 0.33845
 finished parsing run0000_0000 timepoint 0.3385
 finished parsing run0000_0000 timepoint 0.33855
 finished parsing run0000_0000 timepoint 0.3386
 finished parsing run0000_0000 timepoint 0.33865
 finished parsing run0000_0000 timepoint 0.3387
 finished parsing run0000_0000 timepoint 0.33875
 finished parsing run0000_0000 timepoint 0.3388
 finished parsing run0000_0000 timepoint 0.33885
 finished parsing run0000_0000 timepoint 0.3389
 finished parsing run0000_0000 timepoint 0.33895
 finished parsing run0000_0000 timepoint 0.339
 finished parsing run0000_0000 timepoint 0.33905
 finished parsing run0000_0000 timepoint 0.3391
 finished parsing run0000_0000 timepoint 0.33915
 finished parsing run0000_0000 timepoint 0.3392
 finished parsing run0000_0000 timepoint 0.33925
 finished parsing run0000_0000 timepoint 0.3393
 finished parsing run0000_0000 timepoint 0.33935
 finished parsing run0000_0000 timepoint 0.3394
 finished parsing run0000_0000 timepoint 0.33945
 finished parsing run0000_0000 timepoint 0.3395
 finished parsing run0000_0000 timepoint 0.33955
 finished parsing run0000_0000 timepoint 0.3396
 finished parsing run0000_0000 timepoint 0.33965
 finished parsing run0000_0000 timepoint 0.3397
 finished parsing run0000_0000 timepoint 0.33975
 finished parsing run0000_0000 timepoint 0.3398
 finished parsing run0000_0000 timepoint 0.33985
 finished parsing run0000_0000 timepoint 0.3399
 finished parsing run0000_0000 timepoint 0.33995
 finished parsing run0000_0000 timepoint 0.34
 finished parsing run0000_0000 timepoint 0.34005
 finished parsing run0000_0000 timepoint 0.3401
 finished parsing run0000_0000 timepoint 0.34015
 finished parsing run0000_0000 timepoint 0.3402
 finished parsing run0000_0000 timepoint 0.34025
 finished parsing run0000_0000 timepoint 0.3403
 finished parsing run0000_0000 timepoint 0.34035
 finished parsing run0000_0000 timepoint 0.3404
 finished parsing run0000_0000 timepoint 0.34045
 finished parsing run0000_0000 timepoint 0.3405
 finished parsing run0000_0000 timepoint 0.34055
 finished parsing run0000_0000 timepoint 0.3406
 finished parsing run0000_0000 timepoint 0.34065
 finished parsing run0000_0000 timepoint 0.3407
 finished parsing run0000_0000 timepoint 0.34075
 finished parsing run0000_0000 timepoint 0.3408
 finished parsing run0000_0000 timepoint 0.34085
 finished parsing run0000_0000 timepoint 0.3409
 finished parsing run0000_0000 timepoint 0.34095
 finished parsing run0000_0000 timepoint 0.341
 finished parsing run0000_0000 timepoint 0.34105
 finished parsing run0000_0000 timepoint 0.3411
 finished parsing run0000_0000 timepoint 0.34115
 finished parsing run0000_0000 timepoint 0.3412
 finished parsing run0000_0000 timepoint 0.34125
 finished parsing run0000_0000 timepoint 0.3413
 finished parsing run0000_0000 timepoint 0.34135
 finished parsing run0000_0000 timepoint 0.3414
 finished parsing run0000_0000 timepoint 0.34145
 finished parsing run0000_0000 timepoint 0.3415
 finished parsing run0000_0000 timepoint 0.34155
 finished parsing run0000_0000 timepoint 0.3416
 finished parsing run0000_0000 timepoint 0.34165
 finished parsing run0000_0000 timepoint 0.3417
 finished parsing run0000_0000 timepoint 0.34175
 finished parsing run0000_0000 timepoint 0.3418
 finished parsing run0000_0000 timepoint 0.34185
 finished parsing run0000_0000 timepoint 0.3419
 finished parsing run0000_0000 timepoint 0.34195
 finished parsing run0000_0000 timepoint 0.342
 finished parsing run0000_0000 timepoint 0.34205
 finished parsing run0000_0000 timepoint 0.3421
 finished parsing run0000_0000 timepoint 0.34215
 finished parsing run0000_0000 timepoint 0.3422
 finished parsing run0000_0000 timepoint 0.34225
 finished parsing run0000_0000 timepoint 0.3423
 finished parsing run0000_0000 timepoint 0.34235
 finished parsing run0000_0000 timepoint 0.3424
 finished parsing run0000_0000 timepoint 0.34245
 finished parsing run0000_0000 timepoint 0.3425
 finished parsing run0000_0000 timepoint 0.34255
 finished parsing run0000_0000 timepoint 0.3426
 finished parsing run0000_0000 timepoint 0.34265
 finished parsing run0000_0000 timepoint 0.3427
 finished parsing run0000_0000 timepoint 0.34275
 finished parsing run0000_0000 timepoint 0.3428
 finished parsing run0000_0000 timepoint 0.34285
 finished parsing run0000_0000 timepoint 0.3429
 finished parsing run0000_0000 timepoint 0.34295
 finished parsing run0000_0000 timepoint 0.343
 finished parsing run0000_0000 timepoint 0.34305
 finished parsing run0000_0000 timepoint 0.3431
 finished parsing run0000_0000 timepoint 0.34315
 finished parsing run0000_0000 timepoint 0.3432
 finished parsing run0000_0000 timepoint 0.34325
 finished parsing run0000_0000 timepoint 0.3433
 finished parsing run0000_0000 timepoint 0.34335
 finished parsing run0000_0000 timepoint 0.3434
 finished parsing run0000_0000 timepoint 0.34345
 finished parsing run0000_0000 timepoint 0.3435
 finished parsing run0000_0000 timepoint 0.34355
 finished parsing run0000_0000 timepoint 0.3436
 finished parsing run0000_0000 timepoint 0.34365
 finished parsing run0000_0000 timepoint 0.3437
 finished parsing run0000_0000 timepoint 0.34375
 finished parsing run0000_0000 timepoint 0.3438
 finished parsing run0000_0000 timepoint 0.34385
 finished parsing run0000_0000 timepoint 0.3439
 finished parsing run0000_0000 timepoint 0.34395
 finished parsing run0000_0000 timepoint 0.344
 finished parsing run0000_0000 timepoint 0.34405
 finished parsing run0000_0000 timepoint 0.3441
 finished parsing run0000_0000 timepoint 0.34415
 finished parsing run0000_0000 timepoint 0.3442
 finished parsing run0000_0000 timepoint 0.34425
 finished parsing run0000_0000 timepoint 0.3443
 finished parsing run0000_0000 timepoint 0.34435
 finished parsing run0000_0000 timepoint 0.3444
 finished parsing run0000_0000 timepoint 0.34445
 finished parsing run0000_0000 timepoint 0.3445
 finished parsing run0000_0000 timepoint 0.34455
 finished parsing run0000_0000 timepoint 0.3446
 finished parsing run0000_0000 timepoint 0.34465
 finished parsing run0000_0000 timepoint 0.3447
 finished parsing run0000_0000 timepoint 0.34475
 finished parsing run0000_0000 timepoint 0.3448
 finished parsing run0000_0000 timepoint 0.34485
 finished parsing run0000_0000 timepoint 0.3449
 finished parsing run0000_0000 timepoint 0.34495
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.33175
 finished parsing run0000_0000 timepoint 0.3318
 finished parsing run0000_0000 timepoint 0.33185
 finished parsing run0000_0000 timepoint 0.3319
 finished parsing run0000_0000 timepoint 0.33195
 finished parsing run0000_0000 timepoint 0.332
 finished parsing run0000_0000 timepoint 0.33205
 finished parsing run0000_0000 timepoint 0.3321
 finished parsing run0000_0000 timepoint 0.33215
 finished parsing run0000_0000 timepoint 0.3322
 finished parsing run0000_0000 timepoint 0.33225
 finished parsing run0000_0000 timepoint 0.3323
 finished parsing run0000_0000 timepoint 0.33235
 finished parsing run0000_0000 timepoint 0.3324
 finished parsing run0000_0000 timepoint 0.33245
 finished parsing run0000_0000 timepoint 0.3325
 finished parsing run0000_0000 timepoint 0.33255
 finished parsing run0000_0000 timepoint 0.3326
 finished parsing run0000_0000 timepoint 0.33265
 finished parsing run0000_0000 timepoint 0.3327
 finished parsing run0000_0000 timepoint 0.33275
 finished parsing run0000_0000 timepoint 0.3328
 finished parsing run0000_0000 timepoint 0.33285
 finished parsing run0000_0000 timepoint 0.3329
 finished parsing run0000_0000 timepoint 0.33295
 finished parsing run0000_0000 timepoint 0.333
 finished parsing run0000_0000 timepoint 0.33305
 finished parsing run0000_0000 timepoint 0.3331
 finished parsing run0000_0000 timepoint 0.33315
 finished parsing run0000_0000 timepoint 0.3332
 finished parsing run0000_0000 timepoint 0.33325
 finished parsing run0000_0000 timepoint 0.3333
 finished parsing run0000_0000 timepoint 0.33335
 finished parsing run0000_0000 timepoint 0.3334
 finished parsing run0000_0000 timepoint 0.33345
 finished parsing run0000_0000 timepoint 0.3335
 finished parsing run0000_0000 timepoint 0.33355
 finished parsing run0000_0000 timepoint 0.3336
 finished parsing run0000_0000 timepoint 0.33365
 finished parsing run0000_0000 timepoint 0.3337
 finished parsing run0000_0000 timepoint 0.33375
 finished parsing run0000_0000 timepoint 0.3338
 finished parsing run0000_0000 timepoint 0.33385
 finished parsing run0000_0000 timepoint 0.3339
 finished parsing run0000_0000 timepoint 0.33395
 finished parsing run0000_0000 timepoint 0.334
 finished parsing run0000_0000 timepoint 0.33405
 finished parsing run0000_0000 timepoint 0.3341
 finished parsing run0000_0000 timepoint 0.33415
 finished parsing run0000_0000 timepoint 0.3342
 finished parsing run0000_0000 timepoint 0.33425
 finished parsing run0000_0000 timepoint 0.3343
 finished parsing run0000_0000 timepoint 0.33435
 finished parsing run0000_0000 timepoint 0.3344
 finished parsing run0000_0000 timepoint 0.33445
 finished parsing run0000_0000 timepoint 0.3345
 finished parsing run0000_0000 timepoint 0.33455
 finished parsing run0000_0000 timepoint 0.3346
 finished parsing run0000_0000 timepoint 0.33465
 finished parsing run0000_0000 timepoint 0.3347
 finished parsing run0000_0000 timepoint 0.33475
 finished parsing run0000_0000 timepoint 0.3348
 finished parsing run0000_0000 timepoint 0.33485
 finished parsing run0000_0000 timepoint 0.3349
 finished parsing run0000_0000 timepoint 0.33495
 finished parsing run0000_0000 timepoint 0.335
 finished parsing run0000_0000 timepoint 0.33505
 finished parsing run0000_0000 timepoint 0.3351
 finished parsing run0000_0000 timepoint 0.33515
 finished parsing run0000_0000 timepoint 0.3352
 finished parsing run0000_0000 timepoint 0.33525
 finished parsing run0000_0000 timepoint 0.3353
 finished parsing run0000_0000 timepoint 0.33535
 finished parsing run0000_0000 timepoint 0.3354
 finished parsing run0000_0000 timepoint 0.33545
 finished parsing run0000_0000 timepoint 0.3355
 finished parsing run0000_0000 timepoint 0.33555
 finished parsing run0000_0000 timepoint 0.3356
 finished parsing run0000_0000 timepoint 0.33565
 finished parsing run0000_0000 timepoint 0.3357
 finished parsing run0000_0000 timepoint 0.33575
 finished parsing run0000_0000 timepoint 0.3358
 finished parsing run0000_0000 timepoint 0.33585
 finished parsing run0000_0000 timepoint 0.3359
 finished parsing run0000_0000 timepoint 0.33595
 finished parsing run0000_0000 timepoint 0.336
 finished parsing run0000_0000 timepoint 0.33605
 finished parsing run0000_0000 timepoint 0.3361
 finished parsing run0000_0000 timepoint 0.33615
 finished parsing run0000_0000 timepoint 0.3362
 finished parsing run0000_0000 timepoint 0.33625
 finished parsing run0000_0000 timepoint 0.3363
 finished parsing run0000_0000 timepoint 0.33635
 finished parsing run0000_0000 timepoint 0.3364
 finished parsing run0000_0000 timepoint 0.33645
 finished parsing run0000_0000 timepoint 0.3365
 finished parsing run0000_0000 timepoint 0.33655
 finished parsing run0000_0000 timepoint 0.3366
 finished parsing run0000_0000 timepoint 0.33665
 finished parsing run0000_0000 timepoint 0.3367
 finished parsing run0000_0000 timepoint 0.33675
 finished parsing run0000_0000 timepoint 0.3368
 finished parsing run0000_0000 timepoint 0.33685
 finished parsing run0000_0000 timepoint 0.3369
 finished parsing run0000_0000 timepoint 0.33695
 finished parsing run0000_0000 timepoint 0.337
 finished parsing run0000_0000 timepoint 0.33705
 finished parsing run0000_0000 timepoint 0.3371
 finished parsing run0000_0000 timepoint 0.33715
 finished parsing run0000_0000 timepoint 0.3372
 finished parsing run0000_0000 timepoint 0.33725
 finished parsing run0000_0000 timepoint 0.3373
 finished parsing run0000_0000 timepoint 0.33735
 finished parsing run0000_0000 timepoint 0.3374
 finished parsing run0000_0000 timepoint 0.33745
 finished parsing run0000_0000 timepoint 0.3375
 finished parsing run0000_0000 timepoint 0.33755
 finished parsing run0000_0000 timepoint 0.3376
 finished parsing run0000_0000 timepoint 0.33765
 finished parsing run0000_0000 timepoint 0.3377
 finished parsing run0000_0000 timepoint 0.33775
 finished parsing run0000_0000 timepoint 0.3378
 finished parsing run0000_0000 timepoint 0.33785
 finished parsing run0000_0000 timepoint 0.3379
 finished parsing run0000_0000 timepoint 0.33795
 finished parsing run0000_0000 timepoint 0.338
 finished parsing run0000_0000 timepoint 0.33805
 finished parsing run0000_0000 timepoint 0.3381
 finished parsing run0000_0000 timepoint 0.33815
 finished parsing run0000_0000 timepoint 0.3382
 finished parsing run0000_0000 timepoint 0.33825
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.32505
 finished parsing run0000_0000 timepoint 0.3251
 finished parsing run0000_0000 timepoint 0.32515
 finished parsing run0000_0000 timepoint 0.3252
 finished parsing run0000_0000 timepoint 0.32525
 finished parsing run0000_0000 timepoint 0.3253
 finished parsing run0000_0000 timepoint 0.32535
 finished parsing run0000_0000 timepoint 0.3254
 finished parsing run0000_0000 timepoint 0.32545
 finished parsing run0000_0000 timepoint 0.3255
 finished parsing run0000_0000 timepoint 0.32555
 finished parsing run0000_0000 timepoint 0.3256
 finished parsing run0000_0000 timepoint 0.32565
 finished parsing run0000_0000 timepoint 0.3257
 finished parsing run0000_0000 timepoint 0.32575
 finished parsing run0000_0000 timepoint 0.3258
 finished parsing run0000_0000 timepoint 0.32585
 finished parsing run0000_0000 timepoint 0.3259
 finished parsing run0000_0000 timepoint 0.32595
 finished parsing run0000_0000 timepoint 0.326
 finished parsing run0000_0000 timepoint 0.32605
 finished parsing run0000_0000 timepoint 0.3261
 finished parsing run0000_0000 timepoint 0.32615
 finished parsing run0000_0000 timepoint 0.3262
 finished parsing run0000_0000 timepoint 0.32625
 finished parsing run0000_0000 timepoint 0.3263
 finished parsing run0000_0000 timepoint 0.32635
 finished parsing run0000_0000 timepoint 0.3264
 finished parsing run0000_0000 timepoint 0.32645
 finished parsing run0000_0000 timepoint 0.3265
 finished parsing run0000_0000 timepoint 0.32655
 finished parsing run0000_0000 timepoint 0.3266
 finished parsing run0000_0000 timepoint 0.32665
 finished parsing run0000_0000 timepoint 0.3267
 finished parsing run0000_0000 timepoint 0.32675
 finished parsing run0000_0000 timepoint 0.3268
 finished parsing run0000_0000 timepoint 0.32685
 finished parsing run0000_0000 timepoint 0.3269
 finished parsing run0000_0000 timepoint 0.32695
 finished parsing run0000_0000 timepoint 0.327
 finished parsing run0000_0000 timepoint 0.32705
 finished parsing run0000_0000 timepoint 0.3271
 finished parsing run0000_0000 timepoint 0.32715
 finished parsing run0000_0000 timepoint 0.3272
 finished parsing run0000_0000 timepoint 0.32725
 finished parsing run0000_0000 timepoint 0.3273
 finished parsing run0000_0000 timepoint 0.32735
 finished parsing run0000_0000 timepoint 0.3274
 finished parsing run0000_0000 timepoint 0.32745
 finished parsing run0000_0000 timepoint 0.3275
 finished parsing run0000_0000 timepoint 0.32755
 finished parsing run0000_0000 timepoint 0.3276
 finished parsing run0000_0000 timepoint 0.32765
 finished parsing run0000_0000 timepoint 0.3277
 finished parsing run0000_0000 timepoint 0.32775
 finished parsing run0000_0000 timepoint 0.3278
 finished parsing run0000_0000 timepoint 0.32785
 finished parsing run0000_0000 timepoint 0.3279
 finished parsing run0000_0000 timepoint 0.32795
 finished parsing run0000_0000 timepoint 0.328
 finished parsing run0000_0000 timepoint 0.32805
 finished parsing run0000_0000 timepoint 0.3281
 finished parsing run0000_0000 timepoint 0.32815
 finished parsing run0000_0000 timepoint 0.3282
 finished parsing run0000_0000 timepoint 0.32825
 finished parsing run0000_0000 timepoint 0.3283
 finished parsing run0000_0000 timepoint 0.32835
 finished parsing run0000_0000 timepoint 0.3284
 finished parsing run0000_0000 timepoint 0.32845
 finished parsing run0000_0000 timepoint 0.3285
 finished parsing run0000_0000 timepoint 0.32855
 finished parsing run0000_0000 timepoint 0.3286
 finished parsing run0000_0000 timepoint 0.32865
 finished parsing run0000_0000 timepoint 0.3287
 finished parsing run0000_0000 timepoint 0.32875
 finished parsing run0000_0000 timepoint 0.3288
 finished parsing run0000_0000 timepoint 0.32885
 finished parsing run0000_0000 timepoint 0.3289
 finished parsing run0000_0000 timepoint 0.32895
 finished parsing run0000_0000 timepoint 0.329
 finished parsing run0000_0000 timepoint 0.32905
 finished parsing run0000_0000 timepoint 0.3291
 finished parsing run0000_0000 timepoint 0.32915
 finished parsing run0000_0000 timepoint 0.3292
 finished parsing run0000_0000 timepoint 0.32925
 finished parsing run0000_0000 timepoint 0.3293
 finished parsing run0000_0000 timepoint 0.32935
 finished parsing run0000_0000 timepoint 0.3294
 finished parsing run0000_0000 timepoint 0.32945
 finished parsing run0000_0000 timepoint 0.3295
 finished parsing run0000_0000 timepoint 0.32955
 finished parsing run0000_0000 timepoint 0.3296
 finished parsing run0000_0000 timepoint 0.32965
 finished parsing run0000_0000 timepoint 0.3297
 finished parsing run0000_0000 timepoint 0.32975
 finished parsing run0000_0000 timepoint 0.3298
 finished parsing run0000_0000 timepoint 0.32985
 finished parsing run0000_0000 timepoint 0.3299
 finished parsing run0000_0000 timepoint 0.32995
 finished parsing run0000_0000 timepoint 0.33
 finished parsing run0000_0000 timepoint 0.33005
 finished parsing run0000_0000 timepoint 0.3301
 finished parsing run0000_0000 timepoint 0.33015
 finished parsing run0000_0000 timepoint 0.3302
 finished parsing run0000_0000 timepoint 0.33025
 finished parsing run0000_0000 timepoint 0.3303
 finished parsing run0000_0000 timepoint 0.33035
 finished parsing run0000_0000 timepoint 0.3304
 finished parsing run0000_0000 timepoint 0.33045
 finished parsing run0000_0000 timepoint 0.3305
 finished parsing run0000_0000 timepoint 0.33055
 finished parsing run0000_0000 timepoint 0.3306
 finished parsing run0000_0000 timepoint 0.33065
 finished parsing run0000_0000 timepoint 0.3307
 finished parsing run0000_0000 timepoint 0.33075
 finished parsing run0000_0000 timepoint 0.3308
 finished parsing run0000_0000 timepoint 0.33085
 finished parsing run0000_0000 timepoint 0.3309
 finished parsing run0000_0000 timepoint 0.33095
 finished parsing run0000_0000 timepoint 0.331
 finished parsing run0000_0000 timepoint 0.33105
 finished parsing run0000_0000 timepoint 0.3311
 finished parsing run0000_0000 timepoint 0.33115
 finished parsing run0000_0000 timepoint 0.3312
 finished parsing run0000_0000 timepoint 0.33125
 finished parsing run0000_0000 timepoint 0.3313
 finished parsing run0000_0000 timepoint 0.33135
 finished parsing run0000_0000 timepoint 0.3314
 finished parsing run0000_0000 timepoint 0.33145
 finished parsing run0000_0000 timepoint 0.3315
 finished parsing run0000_0000 timepoint 0.33155
 finished parsing run0000_0000 timepoint 0.3316
 finished parsing run0000_0000 timepoint 0.33165
 finished parsing run0000_0000 timepoint 0.3317
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.3181
 finished parsing run0000_0000 timepoint 0.31815
 finished parsing run0000_0000 timepoint 0.3182
 finished parsing run0000_0000 timepoint 0.31825
 finished parsing run0000_0000 timepoint 0.3183
 finished parsing run0000_0000 timepoint 0.31835
 finished parsing run0000_0000 timepoint 0.3184
 finished parsing run0000_0000 timepoint 0.31845
 finished parsing run0000_0000 timepoint 0.3185
 finished parsing run0000_0000 timepoint 0.31855
 finished parsing run0000_0000 timepoint 0.3186
 finished parsing run0000_0000 timepoint 0.31865
 finished parsing run0000_0000 timepoint 0.3187
 finished parsing run0000_0000 timepoint 0.31875
 finished parsing run0000_0000 timepoint 0.3188
 finished parsing run0000_0000 timepoint 0.31885
 finished parsing run0000_0000 timepoint 0.3189
 finished parsing run0000_0000 timepoint 0.31895
 finished parsing run0000_0000 timepoint 0.319
 finished parsing run0000_0000 timepoint 0.31905
 finished parsing run0000_0000 timepoint 0.3191
 finished parsing run0000_0000 timepoint 0.31915
 finished parsing run0000_0000 timepoint 0.3192
 finished parsing run0000_0000 timepoint 0.31925
 finished parsing run0000_0000 timepoint 0.3193
 finished parsing run0000_0000 timepoint 0.31935
 finished parsing run0000_0000 timepoint 0.3194
 finished parsing run0000_0000 timepoint 0.31945
 finished parsing run0000_0000 timepoint 0.3195
 finished parsing run0000_0000 timepoint 0.31955
 finished parsing run0000_0000 timepoint 0.3196
 finished parsing run0000_0000 timepoint 0.31965
 finished parsing run0000_0000 timepoint 0.3197
 finished parsing run0000_0000 timepoint 0.31975
 finished parsing run0000_0000 timepoint 0.3198
 finished parsing run0000_0000 timepoint 0.31985
 finished parsing run0000_0000 timepoint 0.3199
 finished parsing run0000_0000 timepoint 0.31995
 finished parsing run0000_0000 timepoint 0.32
 finished parsing run0000_0000 timepoint 0.32005
 finished parsing run0000_0000 timepoint 0.3201
 finished parsing run0000_0000 timepoint 0.32015
 finished parsing run0000_0000 timepoint 0.3202
 finished parsing run0000_0000 timepoint 0.32025
 finished parsing run0000_0000 timepoint 0.3203
 finished parsing run0000_0000 timepoint 0.32035
 finished parsing run0000_0000 timepoint 0.3204
 finished parsing run0000_0000 timepoint 0.32045
 finished parsing run0000_0000 timepoint 0.3205
 finished parsing run0000_0000 timepoint 0.32055
 finished parsing run0000_0000 timepoint 0.3206
 finished parsing run0000_0000 timepoint 0.32065
 finished parsing run0000_0000 timepoint 0.3207
 finished parsing run0000_0000 timepoint 0.32075
 finished parsing run0000_0000 timepoint 0.3208
 finished parsing run0000_0000 timepoint 0.32085
 finished parsing run0000_0000 timepoint 0.3209
 finished parsing run0000_0000 timepoint 0.32095
 finished parsing run0000_0000 timepoint 0.321
 finished parsing run0000_0000 timepoint 0.32105
 finished parsing run0000_0000 timepoint 0.3211
 finished parsing run0000_0000 timepoint 0.32115
 finished parsing run0000_0000 timepoint 0.3212
 finished parsing run0000_0000 timepoint 0.32125
 finished parsing run0000_0000 timepoint 0.3213
 finished parsing run0000_0000 timepoint 0.32135
 finished parsing run0000_0000 timepoint 0.3214
 finished parsing run0000_0000 timepoint 0.32145
 finished parsing run0000_0000 timepoint 0.3215
 finished parsing run0000_0000 timepoint 0.32155
 finished parsing run0000_0000 timepoint 0.3216
 finished parsing run0000_0000 timepoint 0.32165
 finished parsing run0000_0000 timepoint 0.3217
 finished parsing run0000_0000 timepoint 0.32175
 finished parsing run0000_0000 timepoint 0.3218
 finished parsing run0000_0000 timepoint 0.32185
 finished parsing run0000_0000 timepoint 0.3219
 finished parsing run0000_0000 timepoint 0.32195
 finished parsing run0000_0000 timepoint 0.322
 finished parsing run0000_0000 timepoint 0.32205
 finished parsing run0000_0000 timepoint 0.3221
 finished parsing run0000_0000 timepoint 0.32215
 finished parsing run0000_0000 timepoint 0.3222
 finished parsing run0000_0000 timepoint 0.32225
 finished parsing run0000_0000 timepoint 0.3223
 finished parsing run0000_0000 timepoint 0.32235
 finished parsing run0000_0000 timepoint 0.3224
 finished parsing run0000_0000 timepoint 0.32245
 finished parsing run0000_0000 timepoint 0.3225
 finished parsing run0000_0000 timepoint 0.32255
 finished parsing run0000_0000 timepoint 0.3226
 finished parsing run0000_0000 timepoint 0.32265
 finished parsing run0000_0000 timepoint 0.3227
 finished parsing run0000_0000 timepoint 0.32275
 finished parsing run0000_0000 timepoint 0.3228
 finished parsing run0000_0000 timepoint 0.32285
 finished parsing run0000_0000 timepoint 0.3229
 finished parsing run0000_0000 timepoint 0.32295
 finished parsing run0000_0000 timepoint 0.323
 finished parsing run0000_0000 timepoint 0.32305
 finished parsing run0000_0000 timepoint 0.3231
 finished parsing run0000_0000 timepoint 0.32315
 finished parsing run0000_0000 timepoint 0.3232
 finished parsing run0000_0000 timepoint 0.32325
 finished parsing run0000_0000 timepoint 0.3233
 finished parsing run0000_0000 timepoint 0.32335
 finished parsing run0000_0000 timepoint 0.3234
 finished parsing run0000_0000 timepoint 0.32345
 finished parsing run0000_0000 timepoint 0.3235
 finished parsing run0000_0000 timepoint 0.32355
 finished parsing run0000_0000 timepoint 0.3236
 finished parsing run0000_0000 timepoint 0.32365
 finished parsing run0000_0000 timepoint 0.3237
 finished parsing run0000_0000 timepoint 0.32375
 finished parsing run0000_0000 timepoint 0.3238
 finished parsing run0000_0000 timepoint 0.32385
 finished parsing run0000_0000 timepoint 0.3239
 finished parsing run0000_0000 timepoint 0.32395
 finished parsing run0000_0000 timepoint 0.324
 finished parsing run0000_0000 timepoint 0.32405
 finished parsing run0000_0000 timepoint 0.3241
 finished parsing run0000_0000 timepoint 0.32415
 finished parsing run0000_0000 timepoint 0.3242
 finished parsing run0000_0000 timepoint 0.32425
 finished parsing run0000_0000 timepoint 0.3243
 finished parsing run0000_0000 timepoint 0.32435
 finished parsing run0000_0000 timepoint 0.3244
 finished parsing run0000_0000 timepoint 0.32445
 finished parsing run0000_0000 timepoint 0.3245
 finished parsing run0000_0000 timepoint 0.32455
 finished parsing run0000_0000 timepoint 0.3246
 finished parsing run0000_0000 timepoint 0.32465
 finished parsing run0000_0000 timepoint 0.3247
 finished parsing run0000_0000 timepoint 0.32475
 finished parsing run0000_0000 timepoint 0.3248
 finished parsing run0000_0000 timepoint 0.32485
 finished parsing run0000_0000 timepoint 0.3249
 finished parsing run0000_0000 timepoint 0.32495
 finished parsing run0000_0000 timepoint 0.325
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.31135
 finished parsing run0000_0000 timepoint 0.3114
 finished parsing run0000_0000 timepoint 0.31145
 finished parsing run0000_0000 timepoint 0.3115
 finished parsing run0000_0000 timepoint 0.31155
 finished parsing run0000_0000 timepoint 0.3116
 finished parsing run0000_0000 timepoint 0.31165
 finished parsing run0000_0000 timepoint 0.3117
 finished parsing run0000_0000 timepoint 0.31175
 finished parsing run0000_0000 timepoint 0.3118
 finished parsing run0000_0000 timepoint 0.31185
 finished parsing run0000_0000 timepoint 0.3119
 finished parsing run0000_0000 timepoint 0.31195
 finished parsing run0000_0000 timepoint 0.312
 finished parsing run0000_0000 timepoint 0.31205
 finished parsing run0000_0000 timepoint 0.3121
 finished parsing run0000_0000 timepoint 0.31215
 finished parsing run0000_0000 timepoint 0.3122
 finished parsing run0000_0000 timepoint 0.31225
 finished parsing run0000_0000 timepoint 0.3123
 finished parsing run0000_0000 timepoint 0.31235
 finished parsing run0000_0000 timepoint 0.3124
 finished parsing run0000_0000 timepoint 0.31245
 finished parsing run0000_0000 timepoint 0.3125
 finished parsing run0000_0000 timepoint 0.31255
 finished parsing run0000_0000 timepoint 0.3126
 finished parsing run0000_0000 timepoint 0.31265
 finished parsing run0000_0000 timepoint 0.3127
 finished parsing run0000_0000 timepoint 0.31275
 finished parsing run0000_0000 timepoint 0.3128
 finished parsing run0000_0000 timepoint 0.31285
 finished parsing run0000_0000 timepoint 0.3129
 finished parsing run0000_0000 timepoint 0.31295
 finished parsing run0000_0000 timepoint 0.313
 finished parsing run0000_0000 timepoint 0.31305
 finished parsing run0000_0000 timepoint 0.3131
 finished parsing run0000_0000 timepoint 0.31315
 finished parsing run0000_0000 timepoint 0.3132
 finished parsing run0000_0000 timepoint 0.31325
 finished parsing run0000_0000 timepoint 0.3133
 finished parsing run0000_0000 timepoint 0.31335
 finished parsing run0000_0000 timepoint 0.3134
 finished parsing run0000_0000 timepoint 0.31345
 finished parsing run0000_0000 timepoint 0.3135
 finished parsing run0000_0000 timepoint 0.31355
 finished parsing run0000_0000 timepoint 0.3136
 finished parsing run0000_0000 timepoint 0.31365
 finished parsing run0000_0000 timepoint 0.3137
 finished parsing run0000_0000 timepoint 0.31375
 finished parsing run0000_0000 timepoint 0.3138
 finished parsing run0000_0000 timepoint 0.31385
 finished parsing run0000_0000 timepoint 0.3139
 finished parsing run0000_0000 timepoint 0.31395
 finished parsing run0000_0000 timepoint 0.314
 finished parsing run0000_0000 timepoint 0.31405
 finished parsing run0000_0000 timepoint 0.3141
 finished parsing run0000_0000 timepoint 0.31415
 finished parsing run0000_0000 timepoint 0.3142
 finished parsing run0000_0000 timepoint 0.31425
 finished parsing run0000_0000 timepoint 0.3143
 finished parsing run0000_0000 timepoint 0.31435
 finished parsing run0000_0000 timepoint 0.3144
 finished parsing run0000_0000 timepoint 0.31445
 finished parsing run0000_0000 timepoint 0.3145
 finished parsing run0000_0000 timepoint 0.31455
 finished parsing run0000_0000 timepoint 0.3146
 finished parsing run0000_0000 timepoint 0.31465
 finished parsing run0000_0000 timepoint 0.3147
 finished parsing run0000_0000 timepoint 0.31475
 finished parsing run0000_0000 timepoint 0.3148
 finished parsing run0000_0000 timepoint 0.31485
 finished parsing run0000_0000 timepoint 0.3149
 finished parsing run0000_0000 timepoint 0.31495
 finished parsing run0000_0000 timepoint 0.315
 finished parsing run0000_0000 timepoint 0.31505
 finished parsing run0000_0000 timepoint 0.3151
 finished parsing run0000_0000 timepoint 0.31515
 finished parsing run0000_0000 timepoint 0.3152
 finished parsing run0000_0000 timepoint 0.31525
 finished parsing run0000_0000 timepoint 0.3153
 finished parsing run0000_0000 timepoint 0.31535
 finished parsing run0000_0000 timepoint 0.3154
 finished parsing run0000_0000 timepoint 0.31545
 finished parsing run0000_0000 timepoint 0.3155
 finished parsing run0000_0000 timepoint 0.31555
 finished parsing run0000_0000 timepoint 0.3156
 finished parsing run0000_0000 timepoint 0.31565
 finished parsing run0000_0000 timepoint 0.3157
 finished parsing run0000_0000 timepoint 0.31575
 finished parsing run0000_0000 timepoint 0.3158
 finished parsing run0000_0000 timepoint 0.31585
 finished parsing run0000_0000 timepoint 0.3159
 finished parsing run0000_0000 timepoint 0.31595
 finished parsing run0000_0000 timepoint 0.316
 finished parsing run0000_0000 timepoint 0.31605
 finished parsing run0000_0000 timepoint 0.3161
 finished parsing run0000_0000 timepoint 0.31615
 finished parsing run0000_0000 timepoint 0.3162
 finished parsing run0000_0000 timepoint 0.31625
 finished parsing run0000_0000 timepoint 0.3163
 finished parsing run0000_0000 timepoint 0.31635
 finished parsing run0000_0000 timepoint 0.3164
 finished parsing run0000_0000 timepoint 0.31645
 finished parsing run0000_0000 timepoint 0.3165
 finished parsing run0000_0000 timepoint 0.31655
 finished parsing run0000_0000 timepoint 0.3166
 finished parsing run0000_0000 timepoint 0.31665
 finished parsing run0000_0000 timepoint 0.3167
 finished parsing run0000_0000 timepoint 0.31675
 finished parsing run0000_0000 timepoint 0.3168
 finished parsing run0000_0000 timepoint 0.31685
 finished parsing run0000_0000 timepoint 0.3169
 finished parsing run0000_0000 timepoint 0.31695
 finished parsing run0000_0000 timepoint 0.317
 finished parsing run0000_0000 timepoint 0.31705
 finished parsing run0000_0000 timepoint 0.3171
 finished parsing run0000_0000 timepoint 0.31715
 finished parsing run0000_0000 timepoint 0.3172
 finished parsing run0000_0000 timepoint 0.31725
 finished parsing run0000_0000 timepoint 0.3173
 finished parsing run0000_0000 timepoint 0.31735
 finished parsing run0000_0000 timepoint 0.3174
 finished parsing run0000_0000 timepoint 0.31745
 finished parsing run0000_0000 timepoint 0.3175
 finished parsing run0000_0000 timepoint 0.31755
 finished parsing run0000_0000 timepoint 0.3176
 finished parsing run0000_0000 timepoint 0.31765
 finished parsing run0000_0000 timepoint 0.3177
 finished parsing run0000_0000 timepoint 0.31775
 finished parsing run0000_0000 timepoint 0.3178
 finished parsing run0000_0000 timepoint 0.31785
 finished parsing run0000_0000 timepoint 0.3179
 finished parsing run0000_0000 timepoint 0.31795
 finished parsing run0000_0000 timepoint 0.318
 finished parsing run0000_0000 timepoint 0.31805
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.30475
 finished parsing run0000_0000 timepoint 0.3048
 finished parsing run0000_0000 timepoint 0.30485
 finished parsing run0000_0000 timepoint 0.3049
 finished parsing run0000_0000 timepoint 0.30495
 finished parsing run0000_0000 timepoint 0.305
 finished parsing run0000_0000 timepoint 0.30505
 finished parsing run0000_0000 timepoint 0.3051
 finished parsing run0000_0000 timepoint 0.30515
 finished parsing run0000_0000 timepoint 0.3052
 finished parsing run0000_0000 timepoint 0.30525
 finished parsing run0000_0000 timepoint 0.3053
 finished parsing run0000_0000 timepoint 0.30535
 finished parsing run0000_0000 timepoint 0.3054
 finished parsing run0000_0000 timepoint 0.30545
 finished parsing run0000_0000 timepoint 0.3055
 finished parsing run0000_0000 timepoint 0.30555
 finished parsing run0000_0000 timepoint 0.3056
 finished parsing run0000_0000 timepoint 0.30565
 finished parsing run0000_0000 timepoint 0.3057
 finished parsing run0000_0000 timepoint 0.30575
 finished parsing run0000_0000 timepoint 0.3058
 finished parsing run0000_0000 timepoint 0.30585
 finished parsing run0000_0000 timepoint 0.3059
 finished parsing run0000_0000 timepoint 0.30595
 finished parsing run0000_0000 timepoint 0.306
 finished parsing run0000_0000 timepoint 0.30605
 finished parsing run0000_0000 timepoint 0.3061
 finished parsing run0000_0000 timepoint 0.30615
 finished parsing run0000_0000 timepoint 0.3062
 finished parsing run0000_0000 timepoint 0.30625
 finished parsing run0000_0000 timepoint 0.3063
 finished parsing run0000_0000 timepoint 0.30635
 finished parsing run0000_0000 timepoint 0.3064
 finished parsing run0000_0000 timepoint 0.30645
 finished parsing run0000_0000 timepoint 0.3065
 finished parsing run0000_0000 timepoint 0.30655
 finished parsing run0000_0000 timepoint 0.3066
 finished parsing run0000_0000 timepoint 0.30665
 finished parsing run0000_0000 timepoint 0.3067
 finished parsing run0000_0000 timepoint 0.30675
 finished parsing run0000_0000 timepoint 0.3068
 finished parsing run0000_0000 timepoint 0.30685
 finished parsing run0000_0000 timepoint 0.3069
 finished parsing run0000_0000 timepoint 0.30695
 finished parsing run0000_0000 timepoint 0.307
 finished parsing run0000_0000 timepoint 0.30705
 finished parsing run0000_0000 timepoint 0.3071
 finished parsing run0000_0000 timepoint 0.30715
 finished parsing run0000_0000 timepoint 0.3072
 finished parsing run0000_0000 timepoint 0.30725
 finished parsing run0000_0000 timepoint 0.3073
 finished parsing run0000_0000 timepoint 0.30735
 finished parsing run0000_0000 timepoint 0.3074
 finished parsing run0000_0000 timepoint 0.30745
 finished parsing run0000_0000 timepoint 0.3075
 finished parsing run0000_0000 timepoint 0.30755
 finished parsing run0000_0000 timepoint 0.3076
 finished parsing run0000_0000 timepoint 0.30765
 finished parsing run0000_0000 timepoint 0.3077
 finished parsing run0000_0000 timepoint 0.30775
 finished parsing run0000_0000 timepoint 0.3078
 finished parsing run0000_0000 timepoint 0.30785
 finished parsing run0000_0000 timepoint 0.3079
 finished parsing run0000_0000 timepoint 0.30795
 finished parsing run0000_0000 timepoint 0.308
 finished parsing run0000_0000 timepoint 0.30805
 finished parsing run0000_0000 timepoint 0.3081
 finished parsing run0000_0000 timepoint 0.30815
 finished parsing run0000_0000 timepoint 0.3082
 finished parsing run0000_0000 timepoint 0.30825
 finished parsing run0000_0000 timepoint 0.3083
 finished parsing run0000_0000 timepoint 0.30835
 finished parsing run0000_0000 timepoint 0.3084
 finished parsing run0000_0000 timepoint 0.30845
 finished parsing run0000_0000 timepoint 0.3085
 finished parsing run0000_0000 timepoint 0.30855
 finished parsing run0000_0000 timepoint 0.3086
 finished parsing run0000_0000 timepoint 0.30865
 finished parsing run0000_0000 timepoint 0.3087
 finished parsing run0000_0000 timepoint 0.30875
 finished parsing run0000_0000 timepoint 0.3088
 finished parsing run0000_0000 timepoint 0.30885
 finished parsing run0000_0000 timepoint 0.3089
 finished parsing run0000_0000 timepoint 0.30895
 finished parsing run0000_0000 timepoint 0.309
 finished parsing run0000_0000 timepoint 0.30905
 finished parsing run0000_0000 timepoint 0.3091
 finished parsing run0000_0000 timepoint 0.30915
 finished parsing run0000_0000 timepoint 0.3092
 finished parsing run0000_0000 timepoint 0.30925
 finished parsing run0000_0000 timepoint 0.3093
 finished parsing run0000_0000 timepoint 0.30935
 finished parsing run0000_0000 timepoint 0.3094
 finished parsing run0000_0000 timepoint 0.30945
 finished parsing run0000_0000 timepoint 0.3095
 finished parsing run0000_0000 timepoint 0.30955
 finished parsing run0000_0000 timepoint 0.3096
 finished parsing run0000_0000 timepoint 0.30965
 finished parsing run0000_0000 timepoint 0.3097
 finished parsing run0000_0000 timepoint 0.30975
 finished parsing run0000_0000 timepoint 0.3098
 finished parsing run0000_0000 timepoint 0.30985
 finished parsing run0000_0000 timepoint 0.3099
 finished parsing run0000_0000 timepoint 0.30995
 finished parsing run0000_0000 timepoint 0.31
 finished parsing run0000_0000 timepoint 0.31005
 finished parsing run0000_0000 timepoint 0.3101
 finished parsing run0000_0000 timepoint 0.31015
 finished parsing run0000_0000 timepoint 0.3102
 finished parsing run0000_0000 timepoint 0.31025
 finished parsing run0000_0000 timepoint 0.3103
 finished parsing run0000_0000 timepoint 0.31035
 finished parsing run0000_0000 timepoint 0.3104
 finished parsing run0000_0000 timepoint 0.31045
 finished parsing run0000_0000 timepoint 0.3105
 finished parsing run0000_0000 timepoint 0.31055
 finished parsing run0000_0000 timepoint 0.3106
 finished parsing run0000_0000 timepoint 0.31065
 finished parsing run0000_0000 timepoint 0.3107
 finished parsing run0000_0000 timepoint 0.31075
 finished parsing run0000_0000 timepoint 0.3108
 finished parsing run0000_0000 timepoint 0.31085
 finished parsing run0000_0000 timepoint 0.3109
 finished parsing run0000_0000 timepoint 0.31095
 finished parsing run0000_0000 timepoint 0.311
 finished parsing run0000_0000 timepoint 0.31105
 finished parsing run0000_0000 timepoint 0.3111
 finished parsing run0000_0000 timepoint 0.31115
 finished parsing run0000_0000 timepoint 0.3112
 finished parsing run0000_0000 timepoint 0.31125
 finished parsing run0000_0000 timepoint 0.3113
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2982
 finished parsing run0000_0000 timepoint 0.29825
 finished parsing run0000_0000 timepoint 0.2983
 finished parsing run0000_0000 timepoint 0.29835
 finished parsing run0000_0000 timepoint 0.2984
 finished parsing run0000_0000 timepoint 0.29845
 finished parsing run0000_0000 timepoint 0.2985
 finished parsing run0000_0000 timepoint 0.29855
 finished parsing run0000_0000 timepoint 0.2986
 finished parsing run0000_0000 timepoint 0.29865
 finished parsing run0000_0000 timepoint 0.2987
 finished parsing run0000_0000 timepoint 0.29875
 finished parsing run0000_0000 timepoint 0.2988
 finished parsing run0000_0000 timepoint 0.29885
 finished parsing run0000_0000 timepoint 0.2989
 finished parsing run0000_0000 timepoint 0.29895
 finished parsing run0000_0000 timepoint 0.299
 finished parsing run0000_0000 timepoint 0.29905
 finished parsing run0000_0000 timepoint 0.2991
 finished parsing run0000_0000 timepoint 0.29915
 finished parsing run0000_0000 timepoint 0.2992
 finished parsing run0000_0000 timepoint 0.29925
 finished parsing run0000_0000 timepoint 0.2993
 finished parsing run0000_0000 timepoint 0.29935
 finished parsing run0000_0000 timepoint 0.2994
 finished parsing run0000_0000 timepoint 0.29945
 finished parsing run0000_0000 timepoint 0.2995
 finished parsing run0000_0000 timepoint 0.29955
 finished parsing run0000_0000 timepoint 0.2996
 finished parsing run0000_0000 timepoint 0.29965
 finished parsing run0000_0000 timepoint 0.2997
 finished parsing run0000_0000 timepoint 0.29975
 finished parsing run0000_0000 timepoint 0.2998
 finished parsing run0000_0000 timepoint 0.29985
 finished parsing run0000_0000 timepoint 0.2999
 finished parsing run0000_0000 timepoint 0.29995
 finished parsing run0000_0000 timepoint 0.3
 finished parsing run0000_0000 timepoint 0.30005
 finished parsing run0000_0000 timepoint 0.3001
 finished parsing run0000_0000 timepoint 0.30015
 finished parsing run0000_0000 timepoint 0.3002
 finished parsing run0000_0000 timepoint 0.30025
 finished parsing run0000_0000 timepoint 0.3003
 finished parsing run0000_0000 timepoint 0.30035
 finished parsing run0000_0000 timepoint 0.3004
 finished parsing run0000_0000 timepoint 0.30045
 finished parsing run0000_0000 timepoint 0.3005
 finished parsing run0000_0000 timepoint 0.30055
 finished parsing run0000_0000 timepoint 0.3006
 finished parsing run0000_0000 timepoint 0.30065
 finished parsing run0000_0000 timepoint 0.3007
 finished parsing run0000_0000 timepoint 0.30075
 finished parsing run0000_0000 timepoint 0.3008
 finished parsing run0000_0000 timepoint 0.30085
 finished parsing run0000_0000 timepoint 0.3009
 finished parsing run0000_0000 timepoint 0.30095
 finished parsing run0000_0000 timepoint 0.301
 finished parsing run0000_0000 timepoint 0.30105
 finished parsing run0000_0000 timepoint 0.3011
 finished parsing run0000_0000 timepoint 0.30115
 finished parsing run0000_0000 timepoint 0.3012
 finished parsing run0000_0000 timepoint 0.30125
 finished parsing run0000_0000 timepoint 0.3013
 finished parsing run0000_0000 timepoint 0.30135
 finished parsing run0000_0000 timepoint 0.3014
 finished parsing run0000_0000 timepoint 0.30145
 finished parsing run0000_0000 timepoint 0.3015
 finished parsing run0000_0000 timepoint 0.30155
 finished parsing run0000_0000 timepoint 0.3016
 finished parsing run0000_0000 timepoint 0.30165
 finished parsing run0000_0000 timepoint 0.3017
 finished parsing run0000_0000 timepoint 0.30175
 finished parsing run0000_0000 timepoint 0.3018
 finished parsing run0000_0000 timepoint 0.30185
 finished parsing run0000_0000 timepoint 0.3019
 finished parsing run0000_0000 timepoint 0.30195
 finished parsing run0000_0000 timepoint 0.302
 finished parsing run0000_0000 timepoint 0.30205
 finished parsing run0000_0000 timepoint 0.3021
 finished parsing run0000_0000 timepoint 0.30215
 finished parsing run0000_0000 timepoint 0.3022
 finished parsing run0000_0000 timepoint 0.30225
 finished parsing run0000_0000 timepoint 0.3023
 finished parsing run0000_0000 timepoint 0.30235
 finished parsing run0000_0000 timepoint 0.3024
 finished parsing run0000_0000 timepoint 0.30245
 finished parsing run0000_0000 timepoint 0.3025
 finished parsing run0000_0000 timepoint 0.30255
 finished parsing run0000_0000 timepoint 0.3026
 finished parsing run0000_0000 timepoint 0.30265
 finished parsing run0000_0000 timepoint 0.3027
 finished parsing run0000_0000 timepoint 0.30275
 finished parsing run0000_0000 timepoint 0.3028
 finished parsing run0000_0000 timepoint 0.30285
 finished parsing run0000_0000 timepoint 0.3029
 finished parsing run0000_0000 timepoint 0.30295
 finished parsing run0000_0000 timepoint 0.303
 finished parsing run0000_0000 timepoint 0.30305
 finished parsing run0000_0000 timepoint 0.3031
 finished parsing run0000_0000 timepoint 0.30315
 finished parsing run0000_0000 timepoint 0.3032
 finished parsing run0000_0000 timepoint 0.30325
 finished parsing run0000_0000 timepoint 0.3033
 finished parsing run0000_0000 timepoint 0.30335
 finished parsing run0000_0000 timepoint 0.3034
 finished parsing run0000_0000 timepoint 0.30345
 finished parsing run0000_0000 timepoint 0.3035
 finished parsing run0000_0000 timepoint 0.30355
 finished parsing run0000_0000 timepoint 0.3036
 finished parsing run0000_0000 timepoint 0.30365
 finished parsing run0000_0000 timepoint 0.3037
 finished parsing run0000_0000 timepoint 0.30375
 finished parsing run0000_0000 timepoint 0.3038
 finished parsing run0000_0000 timepoint 0.30385
 finished parsing run0000_0000 timepoint 0.3039
 finished parsing run0000_0000 timepoint 0.30395
 finished parsing run0000_0000 timepoint 0.304
 finished parsing run0000_0000 timepoint 0.30405
 finished parsing run0000_0000 timepoint 0.3041
 finished parsing run0000_0000 timepoint 0.30415
 finished parsing run0000_0000 timepoint 0.3042
 finished parsing run0000_0000 timepoint 0.30425
 finished parsing run0000_0000 timepoint 0.3043
 finished parsing run0000_0000 timepoint 0.30435
 finished parsing run0000_0000 timepoint 0.3044
 finished parsing run0000_0000 timepoint 0.30445
 finished parsing run0000_0000 timepoint 0.3045
 finished parsing run0000_0000 timepoint 0.30455
 finished parsing run0000_0000 timepoint 0.3046
 finished parsing run0000_0000 timepoint 0.30465
 finished parsing run0000_0000 timepoint 0.3047
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.29165
 finished parsing run0000_0000 timepoint 0.2917
 finished parsing run0000_0000 timepoint 0.29175
 finished parsing run0000_0000 timepoint 0.2918
 finished parsing run0000_0000 timepoint 0.29185
 finished parsing run0000_0000 timepoint 0.2919
 finished parsing run0000_0000 timepoint 0.29195
 finished parsing run0000_0000 timepoint 0.292
 finished parsing run0000_0000 timepoint 0.29205
 finished parsing run0000_0000 timepoint 0.2921
 finished parsing run0000_0000 timepoint 0.29215
 finished parsing run0000_0000 timepoint 0.2922
 finished parsing run0000_0000 timepoint 0.29225
 finished parsing run0000_0000 timepoint 0.2923
 finished parsing run0000_0000 timepoint 0.29235
 finished parsing run0000_0000 timepoint 0.2924
 finished parsing run0000_0000 timepoint 0.29245
 finished parsing run0000_0000 timepoint 0.2925
 finished parsing run0000_0000 timepoint 0.29255
 finished parsing run0000_0000 timepoint 0.2926
 finished parsing run0000_0000 timepoint 0.29265
 finished parsing run0000_0000 timepoint 0.2927
 finished parsing run0000_0000 timepoint 0.29275
 finished parsing run0000_0000 timepoint 0.2928
 finished parsing run0000_0000 timepoint 0.29285
 finished parsing run0000_0000 timepoint 0.2929
 finished parsing run0000_0000 timepoint 0.29295
 finished parsing run0000_0000 timepoint 0.293
 finished parsing run0000_0000 timepoint 0.29305
 finished parsing run0000_0000 timepoint 0.2931
 finished parsing run0000_0000 timepoint 0.29315
 finished parsing run0000_0000 timepoint 0.2932
 finished parsing run0000_0000 timepoint 0.29325
 finished parsing run0000_0000 timepoint 0.2933
 finished parsing run0000_0000 timepoint 0.29335
 finished parsing run0000_0000 timepoint 0.2934
 finished parsing run0000_0000 timepoint 0.29345
 finished parsing run0000_0000 timepoint 0.2935
 finished parsing run0000_0000 timepoint 0.29355
 finished parsing run0000_0000 timepoint 0.2936
 finished parsing run0000_0000 timepoint 0.29365
 finished parsing run0000_0000 timepoint 0.2937
 finished parsing run0000_0000 timepoint 0.29375
 finished parsing run0000_0000 timepoint 0.2938
 finished parsing run0000_0000 timepoint 0.29385
 finished parsing run0000_0000 timepoint 0.2939
 finished parsing run0000_0000 timepoint 0.29395
 finished parsing run0000_0000 timepoint 0.294
 finished parsing run0000_0000 timepoint 0.29405
 finished parsing run0000_0000 timepoint 0.2941
 finished parsing run0000_0000 timepoint 0.29415
 finished parsing run0000_0000 timepoint 0.2942
 finished parsing run0000_0000 timepoint 0.29425
 finished parsing run0000_0000 timepoint 0.2943
 finished parsing run0000_0000 timepoint 0.29435
 finished parsing run0000_0000 timepoint 0.2944
 finished parsing run0000_0000 timepoint 0.29445
 finished parsing run0000_0000 timepoint 0.2945
 finished parsing run0000_0000 timepoint 0.29455
 finished parsing run0000_0000 timepoint 0.2946
 finished parsing run0000_0000 timepoint 0.29465
 finished parsing run0000_0000 timepoint 0.2947
 finished parsing run0000_0000 timepoint 0.29475
 finished parsing run0000_0000 timepoint 0.2948
 finished parsing run0000_0000 timepoint 0.29485
 finished parsing run0000_0000 timepoint 0.2949
 finished parsing run0000_0000 timepoint 0.29495
 finished parsing run0000_0000 timepoint 0.295
 finished parsing run0000_0000 timepoint 0.29505
 finished parsing run0000_0000 timepoint 0.2951
 finished parsing run0000_0000 timepoint 0.29515
 finished parsing run0000_0000 timepoint 0.2952
 finished parsing run0000_0000 timepoint 0.29525
 finished parsing run0000_0000 timepoint 0.2953
 finished parsing run0000_0000 timepoint 0.29535
 finished parsing run0000_0000 timepoint 0.2954
 finished parsing run0000_0000 timepoint 0.29545
 finished parsing run0000_0000 timepoint 0.2955
 finished parsing run0000_0000 timepoint 0.29555
 finished parsing run0000_0000 timepoint 0.2956
 finished parsing run0000_0000 timepoint 0.29565
 finished parsing run0000_0000 timepoint 0.2957
 finished parsing run0000_0000 timepoint 0.29575
 finished parsing run0000_0000 timepoint 0.2958
 finished parsing run0000_0000 timepoint 0.29585
 finished parsing run0000_0000 timepoint 0.2959
 finished parsing run0000_0000 timepoint 0.29595
 finished parsing run0000_0000 timepoint 0.296
 finished parsing run0000_0000 timepoint 0.29605
 finished parsing run0000_0000 timepoint 0.2961
 finished parsing run0000_0000 timepoint 0.29615
 finished parsing run0000_0000 timepoint 0.2962
 finished parsing run0000_0000 timepoint 0.29625
 finished parsing run0000_0000 timepoint 0.2963
 finished parsing run0000_0000 timepoint 0.29635
 finished parsing run0000_0000 timepoint 0.2964
 finished parsing run0000_0000 timepoint 0.29645
 finished parsing run0000_0000 timepoint 0.2965
 finished parsing run0000_0000 timepoint 0.29655
 finished parsing run0000_0000 timepoint 0.2966
 finished parsing run0000_0000 timepoint 0.29665
 finished parsing run0000_0000 timepoint 0.2967
 finished parsing run0000_0000 timepoint 0.29675
 finished parsing run0000_0000 timepoint 0.2968
 finished parsing run0000_0000 timepoint 0.29685
 finished parsing run0000_0000 timepoint 0.2969
 finished parsing run0000_0000 timepoint 0.29695
 finished parsing run0000_0000 timepoint 0.297
 finished parsing run0000_0000 timepoint 0.29705
 finished parsing run0000_0000 timepoint 0.2971
 finished parsing run0000_0000 timepoint 0.29715
 finished parsing run0000_0000 timepoint 0.2972
 finished parsing run0000_0000 timepoint 0.29725
 finished parsing run0000_0000 timepoint 0.2973
 finished parsing run0000_0000 timepoint 0.29735
 finished parsing run0000_0000 timepoint 0.2974
 finished parsing run0000_0000 timepoint 0.29745
 finished parsing run0000_0000 timepoint 0.2975
 finished parsing run0000_0000 timepoint 0.29755
 finished parsing run0000_0000 timepoint 0.2976
 finished parsing run0000_0000 timepoint 0.29765
 finished parsing run0000_0000 timepoint 0.2977
 finished parsing run0000_0000 timepoint 0.29775
 finished parsing run0000_0000 timepoint 0.2978
 finished parsing run0000_0000 timepoint 0.29785
 finished parsing run0000_0000 timepoint 0.2979
 finished parsing run0000_0000 timepoint 0.29795
 finished parsing run0000_0000 timepoint 0.298
 finished parsing run0000_0000 timepoint 0.29805
 finished parsing run0000_0000 timepoint 0.2981
 finished parsing run0000_0000 timepoint 0.29815
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.28505
 finished parsing run0000_0000 timepoint 0.2851
 finished parsing run0000_0000 timepoint 0.28515
 finished parsing run0000_0000 timepoint 0.2852
 finished parsing run0000_0000 timepoint 0.28525
 finished parsing run0000_0000 timepoint 0.2853
 finished parsing run0000_0000 timepoint 0.28535
 finished parsing run0000_0000 timepoint 0.2854
 finished parsing run0000_0000 timepoint 0.28545
 finished parsing run0000_0000 timepoint 0.2855
 finished parsing run0000_0000 timepoint 0.28555
 finished parsing run0000_0000 timepoint 0.2856
 finished parsing run0000_0000 timepoint 0.28565
 finished parsing run0000_0000 timepoint 0.2857
 finished parsing run0000_0000 timepoint 0.28575
 finished parsing run0000_0000 timepoint 0.2858
 finished parsing run0000_0000 timepoint 0.28585
 finished parsing run0000_0000 timepoint 0.2859
 finished parsing run0000_0000 timepoint 0.28595
 finished parsing run0000_0000 timepoint 0.286
 finished parsing run0000_0000 timepoint 0.28605
 finished parsing run0000_0000 timepoint 0.2861
 finished parsing run0000_0000 timepoint 0.28615
 finished parsing run0000_0000 timepoint 0.2862
 finished parsing run0000_0000 timepoint 0.28625
 finished parsing run0000_0000 timepoint 0.2863
 finished parsing run0000_0000 timepoint 0.28635
 finished parsing run0000_0000 timepoint 0.2864
 finished parsing run0000_0000 timepoint 0.28645
 finished parsing run0000_0000 timepoint 0.2865
 finished parsing run0000_0000 timepoint 0.28655
 finished parsing run0000_0000 timepoint 0.2866
 finished parsing run0000_0000 timepoint 0.28665
 finished parsing run0000_0000 timepoint 0.2867
 finished parsing run0000_0000 timepoint 0.28675
 finished parsing run0000_0000 timepoint 0.2868
 finished parsing run0000_0000 timepoint 0.28685
 finished parsing run0000_0000 timepoint 0.2869
 finished parsing run0000_0000 timepoint 0.28695
 finished parsing run0000_0000 timepoint 0.287
 finished parsing run0000_0000 timepoint 0.28705
 finished parsing run0000_0000 timepoint 0.2871
 finished parsing run0000_0000 timepoint 0.28715
 finished parsing run0000_0000 timepoint 0.2872
 finished parsing run0000_0000 timepoint 0.28725
 finished parsing run0000_0000 timepoint 0.2873
 finished parsing run0000_0000 timepoint 0.28735
 finished parsing run0000_0000 timepoint 0.2874
 finished parsing run0000_0000 timepoint 0.28745
 finished parsing run0000_0000 timepoint 0.2875
 finished parsing run0000_0000 timepoint 0.28755
 finished parsing run0000_0000 timepoint 0.2876
 finished parsing run0000_0000 timepoint 0.28765
 finished parsing run0000_0000 timepoint 0.2877
 finished parsing run0000_0000 timepoint 0.28775
 finished parsing run0000_0000 timepoint 0.2878
 finished parsing run0000_0000 timepoint 0.28785
 finished parsing run0000_0000 timepoint 0.2879
 finished parsing run0000_0000 timepoint 0.28795
 finished parsing run0000_0000 timepoint 0.288
 finished parsing run0000_0000 timepoint 0.28805
 finished parsing run0000_0000 timepoint 0.2881
 finished parsing run0000_0000 timepoint 0.28815
 finished parsing run0000_0000 timepoint 0.2882
 finished parsing run0000_0000 timepoint 0.28825
 finished parsing run0000_0000 timepoint 0.2883
 finished parsing run0000_0000 timepoint 0.28835
 finished parsing run0000_0000 timepoint 0.2884
 finished parsing run0000_0000 timepoint 0.28845
 finished parsing run0000_0000 timepoint 0.2885
 finished parsing run0000_0000 timepoint 0.28855
 finished parsing run0000_0000 timepoint 0.2886
 finished parsing run0000_0000 timepoint 0.28865
 finished parsing run0000_0000 timepoint 0.2887
 finished parsing run0000_0000 timepoint 0.28875
 finished parsing run0000_0000 timepoint 0.2888
 finished parsing run0000_0000 timepoint 0.28885
 finished parsing run0000_0000 timepoint 0.2889
 finished parsing run0000_0000 timepoint 0.28895
 finished parsing run0000_0000 timepoint 0.289
 finished parsing run0000_0000 timepoint 0.28905
 finished parsing run0000_0000 timepoint 0.2891
 finished parsing run0000_0000 timepoint 0.28915
 finished parsing run0000_0000 timepoint 0.2892
 finished parsing run0000_0000 timepoint 0.28925
 finished parsing run0000_0000 timepoint 0.2893
 finished parsing run0000_0000 timepoint 0.28935
 finished parsing run0000_0000 timepoint 0.2894
 finished parsing run0000_0000 timepoint 0.28945
 finished parsing run0000_0000 timepoint 0.2895
 finished parsing run0000_0000 timepoint 0.28955
 finished parsing run0000_0000 timepoint 0.2896
 finished parsing run0000_0000 timepoint 0.28965
 finished parsing run0000_0000 timepoint 0.2897
 finished parsing run0000_0000 timepoint 0.28975
 finished parsing run0000_0000 timepoint 0.2898
 finished parsing run0000_0000 timepoint 0.28985
 finished parsing run0000_0000 timepoint 0.2899
 finished parsing run0000_0000 timepoint 0.28995
 finished parsing run0000_0000 timepoint 0.29
 finished parsing run0000_0000 timepoint 0.29005
 finished parsing run0000_0000 timepoint 0.2901
 finished parsing run0000_0000 timepoint 0.29015
 finished parsing run0000_0000 timepoint 0.2902
 finished parsing run0000_0000 timepoint 0.29025
 finished parsing run0000_0000 timepoint 0.2903
 finished parsing run0000_0000 timepoint 0.29035
 finished parsing run0000_0000 timepoint 0.2904
 finished parsing run0000_0000 timepoint 0.29045
 finished parsing run0000_0000 timepoint 0.2905
 finished parsing run0000_0000 timepoint 0.29055
 finished parsing run0000_0000 timepoint 0.2906
 finished parsing run0000_0000 timepoint 0.29065
 finished parsing run0000_0000 timepoint 0.2907
 finished parsing run0000_0000 timepoint 0.29075
 finished parsing run0000_0000 timepoint 0.2908
 finished parsing run0000_0000 timepoint 0.29085
 finished parsing run0000_0000 timepoint 0.2909
 finished parsing run0000_0000 timepoint 0.29095
 finished parsing run0000_0000 timepoint 0.291
 finished parsing run0000_0000 timepoint 0.29105
 finished parsing run0000_0000 timepoint 0.2911
 finished parsing run0000_0000 timepoint 0.29115
 finished parsing run0000_0000 timepoint 0.2912
 finished parsing run0000_0000 timepoint 0.29125
 finished parsing run0000_0000 timepoint 0.2913
 finished parsing run0000_0000 timepoint 0.29135
 finished parsing run0000_0000 timepoint 0.2914
 finished parsing run0000_0000 timepoint 0.29145
 finished parsing run0000_0000 timepoint 0.2915
 finished parsing run0000_0000 timepoint 0.29155
 finished parsing run0000_0000 timepoint 0.2916
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.27815
 finished parsing run0000_0000 timepoint 0.2782
 finished parsing run0000_0000 timepoint 0.27825
 finished parsing run0000_0000 timepoint 0.2783
 finished parsing run0000_0000 timepoint 0.27835
 finished parsing run0000_0000 timepoint 0.2784
 finished parsing run0000_0000 timepoint 0.27845
 finished parsing run0000_0000 timepoint 0.2785
 finished parsing run0000_0000 timepoint 0.27855
 finished parsing run0000_0000 timepoint 0.2786
 finished parsing run0000_0000 timepoint 0.27865
 finished parsing run0000_0000 timepoint 0.2787
 finished parsing run0000_0000 timepoint 0.27875
 finished parsing run0000_0000 timepoint 0.2788
 finished parsing run0000_0000 timepoint 0.27885
 finished parsing run0000_0000 timepoint 0.2789
 finished parsing run0000_0000 timepoint 0.27895
 finished parsing run0000_0000 timepoint 0.279
 finished parsing run0000_0000 timepoint 0.27905
 finished parsing run0000_0000 timepoint 0.2791
 finished parsing run0000_0000 timepoint 0.27915
 finished parsing run0000_0000 timepoint 0.2792
 finished parsing run0000_0000 timepoint 0.27925
 finished parsing run0000_0000 timepoint 0.2793
 finished parsing run0000_0000 timepoint 0.27935
 finished parsing run0000_0000 timepoint 0.2794
 finished parsing run0000_0000 timepoint 0.27945
 finished parsing run0000_0000 timepoint 0.2795
 finished parsing run0000_0000 timepoint 0.27955
 finished parsing run0000_0000 timepoint 0.2796
 finished parsing run0000_0000 timepoint 0.27965
 finished parsing run0000_0000 timepoint 0.2797
 finished parsing run0000_0000 timepoint 0.27975
 finished parsing run0000_0000 timepoint 0.2798
 finished parsing run0000_0000 timepoint 0.27985
 finished parsing run0000_0000 timepoint 0.2799
 finished parsing run0000_0000 timepoint 0.27995
 finished parsing run0000_0000 timepoint 0.28
 finished parsing run0000_0000 timepoint 0.28005
 finished parsing run0000_0000 timepoint 0.2801
 finished parsing run0000_0000 timepoint 0.28015
 finished parsing run0000_0000 timepoint 0.2802
 finished parsing run0000_0000 timepoint 0.28025
 finished parsing run0000_0000 timepoint 0.2803
 finished parsing run0000_0000 timepoint 0.28035
 finished parsing run0000_0000 timepoint 0.2804
 finished parsing run0000_0000 timepoint 0.28045
 finished parsing run0000_0000 timepoint 0.2805
 finished parsing run0000_0000 timepoint 0.28055
 finished parsing run0000_0000 timepoint 0.2806
 finished parsing run0000_0000 timepoint 0.28065
 finished parsing run0000_0000 timepoint 0.2807
 finished parsing run0000_0000 timepoint 0.28075
 finished parsing run0000_0000 timepoint 0.2808
 finished parsing run0000_0000 timepoint 0.28085
 finished parsing run0000_0000 timepoint 0.2809
 finished parsing run0000_0000 timepoint 0.28095
 finished parsing run0000_0000 timepoint 0.281
 finished parsing run0000_0000 timepoint 0.28105
 finished parsing run0000_0000 timepoint 0.2811
 finished parsing run0000_0000 timepoint 0.28115
 finished parsing run0000_0000 timepoint 0.2812
 finished parsing run0000_0000 timepoint 0.28125
 finished parsing run0000_0000 timepoint 0.2813
 finished parsing run0000_0000 timepoint 0.28135
 finished parsing run0000_0000 timepoint 0.2814
 finished parsing run0000_0000 timepoint 0.28145
 finished parsing run0000_0000 timepoint 0.2815
 finished parsing run0000_0000 timepoint 0.28155
 finished parsing run0000_0000 timepoint 0.2816
 finished parsing run0000_0000 timepoint 0.28165
 finished parsing run0000_0000 timepoint 0.2817
 finished parsing run0000_0000 timepoint 0.28175
 finished parsing run0000_0000 timepoint 0.2818
 finished parsing run0000_0000 timepoint 0.28185
 finished parsing run0000_0000 timepoint 0.2819
 finished parsing run0000_0000 timepoint 0.28195
 finished parsing run0000_0000 timepoint 0.282
 finished parsing run0000_0000 timepoint 0.28205
 finished parsing run0000_0000 timepoint 0.2821
 finished parsing run0000_0000 timepoint 0.28215
 finished parsing run0000_0000 timepoint 0.2822
 finished parsing run0000_0000 timepoint 0.28225
 finished parsing run0000_0000 timepoint 0.2823
 finished parsing run0000_0000 timepoint 0.28235
 finished parsing run0000_0000 timepoint 0.2824
 finished parsing run0000_0000 timepoint 0.28245
 finished parsing run0000_0000 timepoint 0.2825
 finished parsing run0000_0000 timepoint 0.28255
 finished parsing run0000_0000 timepoint 0.2826
 finished parsing run0000_0000 timepoint 0.28265
 finished parsing run0000_0000 timepoint 0.2827
 finished parsing run0000_0000 timepoint 0.28275
 finished parsing run0000_0000 timepoint 0.2828
 finished parsing run0000_0000 timepoint 0.28285
 finished parsing run0000_0000 timepoint 0.2829
 finished parsing run0000_0000 timepoint 0.28295
 finished parsing run0000_0000 timepoint 0.283
 finished parsing run0000_0000 timepoint 0.28305
 finished parsing run0000_0000 timepoint 0.2831
 finished parsing run0000_0000 timepoint 0.28315
 finished parsing run0000_0000 timepoint 0.2832
 finished parsing run0000_0000 timepoint 0.28325
 finished parsing run0000_0000 timepoint 0.2833
 finished parsing run0000_0000 timepoint 0.28335
 finished parsing run0000_0000 timepoint 0.2834
 finished parsing run0000_0000 timepoint 0.28345
 finished parsing run0000_0000 timepoint 0.2835
 finished parsing run0000_0000 timepoint 0.28355
 finished parsing run0000_0000 timepoint 0.2836
 finished parsing run0000_0000 timepoint 0.28365
 finished parsing run0000_0000 timepoint 0.2837
 finished parsing run0000_0000 timepoint 0.28375
 finished parsing run0000_0000 timepoint 0.2838
 finished parsing run0000_0000 timepoint 0.28385
 finished parsing run0000_0000 timepoint 0.2839
 finished parsing run0000_0000 timepoint 0.28395
 finished parsing run0000_0000 timepoint 0.284
 finished parsing run0000_0000 timepoint 0.28405
 finished parsing run0000_0000 timepoint 0.2841
 finished parsing run0000_0000 timepoint 0.28415
 finished parsing run0000_0000 timepoint 0.2842
 finished parsing run0000_0000 timepoint 0.28425
 finished parsing run0000_0000 timepoint 0.2843
 finished parsing run0000_0000 timepoint 0.28435
 finished parsing run0000_0000 timepoint 0.2844
 finished parsing run0000_0000 timepoint 0.28445
 finished parsing run0000_0000 timepoint 0.2845
 finished parsing run0000_0000 timepoint 0.28455
 finished parsing run0000_0000 timepoint 0.2846
 finished parsing run0000_0000 timepoint 0.28465
 finished parsing run0000_0000 timepoint 0.2847
 finished parsing run0000_0000 timepoint 0.28475
 finished parsing run0000_0000 timepoint 0.2848
 finished parsing run0000_0000 timepoint 0.28485
 finished parsing run0000_0000 timepoint 0.2849
 finished parsing run0000_0000 timepoint 0.28495
 finished parsing run0000_0000 timepoint 0.285
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2717
 finished parsing run0000_0000 timepoint 0.27175
 finished parsing run0000_0000 timepoint 0.2718
 finished parsing run0000_0000 timepoint 0.27185
 finished parsing run0000_0000 timepoint 0.2719
 finished parsing run0000_0000 timepoint 0.27195
 finished parsing run0000_0000 timepoint 0.272
 finished parsing run0000_0000 timepoint 0.27205
 finished parsing run0000_0000 timepoint 0.2721
 finished parsing run0000_0000 timepoint 0.27215
 finished parsing run0000_0000 timepoint 0.2722
 finished parsing run0000_0000 timepoint 0.27225
 finished parsing run0000_0000 timepoint 0.2723
 finished parsing run0000_0000 timepoint 0.27235
 finished parsing run0000_0000 timepoint 0.2724
 finished parsing run0000_0000 timepoint 0.27245
 finished parsing run0000_0000 timepoint 0.2725
 finished parsing run0000_0000 timepoint 0.27255
 finished parsing run0000_0000 timepoint 0.2726
 finished parsing run0000_0000 timepoint 0.27265
 finished parsing run0000_0000 timepoint 0.2727
 finished parsing run0000_0000 timepoint 0.27275
 finished parsing run0000_0000 timepoint 0.2728
 finished parsing run0000_0000 timepoint 0.27285
 finished parsing run0000_0000 timepoint 0.2729
 finished parsing run0000_0000 timepoint 0.27295
 finished parsing run0000_0000 timepoint 0.273
 finished parsing run0000_0000 timepoint 0.27305
 finished parsing run0000_0000 timepoint 0.2731
 finished parsing run0000_0000 timepoint 0.27315
 finished parsing run0000_0000 timepoint 0.2732
 finished parsing run0000_0000 timepoint 0.27325
 finished parsing run0000_0000 timepoint 0.2733
 finished parsing run0000_0000 timepoint 0.27335
 finished parsing run0000_0000 timepoint 0.2734
 finished parsing run0000_0000 timepoint 0.27345
 finished parsing run0000_0000 timepoint 0.2735
 finished parsing run0000_0000 timepoint 0.27355
 finished parsing run0000_0000 timepoint 0.2736
 finished parsing run0000_0000 timepoint 0.27365
 finished parsing run0000_0000 timepoint 0.2737
 finished parsing run0000_0000 timepoint 0.27375
 finished parsing run0000_0000 timepoint 0.2738
 finished parsing run0000_0000 timepoint 0.27385
 finished parsing run0000_0000 timepoint 0.2739
 finished parsing run0000_0000 timepoint 0.27395
 finished parsing run0000_0000 timepoint 0.274
 finished parsing run0000_0000 timepoint 0.27405
 finished parsing run0000_0000 timepoint 0.2741
 finished parsing run0000_0000 timepoint 0.27415
 finished parsing run0000_0000 timepoint 0.2742
 finished parsing run0000_0000 timepoint 0.27425
 finished parsing run0000_0000 timepoint 0.2743
 finished parsing run0000_0000 timepoint 0.27435
 finished parsing run0000_0000 timepoint 0.2744
 finished parsing run0000_0000 timepoint 0.27445
 finished parsing run0000_0000 timepoint 0.2745
 finished parsing run0000_0000 timepoint 0.27455
 finished parsing run0000_0000 timepoint 0.2746
 finished parsing run0000_0000 timepoint 0.27465
 finished parsing run0000_0000 timepoint 0.2747
 finished parsing run0000_0000 timepoint 0.27475
 finished parsing run0000_0000 timepoint 0.2748
 finished parsing run0000_0000 timepoint 0.27485
 finished parsing run0000_0000 timepoint 0.2749
 finished parsing run0000_0000 timepoint 0.27495
 finished parsing run0000_0000 timepoint 0.275
 finished parsing run0000_0000 timepoint 0.27505
 finished parsing run0000_0000 timepoint 0.2751
 finished parsing run0000_0000 timepoint 0.27515
 finished parsing run0000_0000 timepoint 0.2752
 finished parsing run0000_0000 timepoint 0.27525
 finished parsing run0000_0000 timepoint 0.2753
 finished parsing run0000_0000 timepoint 0.27535
 finished parsing run0000_0000 timepoint 0.2754
 finished parsing run0000_0000 timepoint 0.27545
 finished parsing run0000_0000 timepoint 0.2755
 finished parsing run0000_0000 timepoint 0.27555
 finished parsing run0000_0000 timepoint 0.2756
 finished parsing run0000_0000 timepoint 0.27565
 finished parsing run0000_0000 timepoint 0.2757
 finished parsing run0000_0000 timepoint 0.27575
 finished parsing run0000_0000 timepoint 0.2758
 finished parsing run0000_0000 timepoint 0.27585
 finished parsing run0000_0000 timepoint 0.2759
 finished parsing run0000_0000 timepoint 0.27595
 finished parsing run0000_0000 timepoint 0.276
 finished parsing run0000_0000 timepoint 0.27605
 finished parsing run0000_0000 timepoint 0.2761
 finished parsing run0000_0000 timepoint 0.27615
 finished parsing run0000_0000 timepoint 0.2762
 finished parsing run0000_0000 timepoint 0.27625
 finished parsing run0000_0000 timepoint 0.2763
 finished parsing run0000_0000 timepoint 0.27635
 finished parsing run0000_0000 timepoint 0.2764
 finished parsing run0000_0000 timepoint 0.27645
 finished parsing run0000_0000 timepoint 0.2765
 finished parsing run0000_0000 timepoint 0.27655
 finished parsing run0000_0000 timepoint 0.2766
 finished parsing run0000_0000 timepoint 0.27665
 finished parsing run0000_0000 timepoint 0.2767
 finished parsing run0000_0000 timepoint 0.27675
 finished parsing run0000_0000 timepoint 0.2768
 finished parsing run0000_0000 timepoint 0.27685
 finished parsing run0000_0000 timepoint 0.2769
 finished parsing run0000_0000 timepoint 0.27695
 finished parsing run0000_0000 timepoint 0.277
 finished parsing run0000_0000 timepoint 0.27705
 finished parsing run0000_0000 timepoint 0.2771
 finished parsing run0000_0000 timepoint 0.27715
 finished parsing run0000_0000 timepoint 0.2772
 finished parsing run0000_0000 timepoint 0.27725
 finished parsing run0000_0000 timepoint 0.2773
 finished parsing run0000_0000 timepoint 0.27735
 finished parsing run0000_0000 timepoint 0.2774
 finished parsing run0000_0000 timepoint 0.27745
 finished parsing run0000_0000 timepoint 0.2775
 finished parsing run0000_0000 timepoint 0.27755
 finished parsing run0000_0000 timepoint 0.2776
 finished parsing run0000_0000 timepoint 0.27765
 finished parsing run0000_0000 timepoint 0.2777
 finished parsing run0000_0000 timepoint 0.27775
 finished parsing run0000_0000 timepoint 0.2778
 finished parsing run0000_0000 timepoint 0.27785
 finished parsing run0000_0000 timepoint 0.2779
 finished parsing run0000_0000 timepoint 0.27795
 finished parsing run0000_0000 timepoint 0.278
 finished parsing run0000_0000 timepoint 0.27805
 finished parsing run0000_0000 timepoint 0.2781
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2648
 finished parsing run0000_0000 timepoint 0.26485
 finished parsing run0000_0000 timepoint 0.2649
 finished parsing run0000_0000 timepoint 0.26495
 finished parsing run0000_0000 timepoint 0.265
 finished parsing run0000_0000 timepoint 0.26505
 finished parsing run0000_0000 timepoint 0.2651
 finished parsing run0000_0000 timepoint 0.26515
 finished parsing run0000_0000 timepoint 0.2652
 finished parsing run0000_0000 timepoint 0.26525
 finished parsing run0000_0000 timepoint 0.2653
 finished parsing run0000_0000 timepoint 0.26535
 finished parsing run0000_0000 timepoint 0.2654
 finished parsing run0000_0000 timepoint 0.26545
 finished parsing run0000_0000 timepoint 0.2655
 finished parsing run0000_0000 timepoint 0.26555
 finished parsing run0000_0000 timepoint 0.2656
 finished parsing run0000_0000 timepoint 0.26565
 finished parsing run0000_0000 timepoint 0.2657
 finished parsing run0000_0000 timepoint 0.26575
 finished parsing run0000_0000 timepoint 0.2658
 finished parsing run0000_0000 timepoint 0.26585
 finished parsing run0000_0000 timepoint 0.2659
 finished parsing run0000_0000 timepoint 0.26595
 finished parsing run0000_0000 timepoint 0.266
 finished parsing run0000_0000 timepoint 0.26605
 finished parsing run0000_0000 timepoint 0.2661
 finished parsing run0000_0000 timepoint 0.26615
 finished parsing run0000_0000 timepoint 0.2662
 finished parsing run0000_0000 timepoint 0.26625
 finished parsing run0000_0000 timepoint 0.2663
 finished parsing run0000_0000 timepoint 0.26635
 finished parsing run0000_0000 timepoint 0.2664
 finished parsing run0000_0000 timepoint 0.26645
 finished parsing run0000_0000 timepoint 0.2665
 finished parsing run0000_0000 timepoint 0.26655
 finished parsing run0000_0000 timepoint 0.2666
 finished parsing run0000_0000 timepoint 0.26665
 finished parsing run0000_0000 timepoint 0.2667
 finished parsing run0000_0000 timepoint 0.26675
 finished parsing run0000_0000 timepoint 0.2668
 finished parsing run0000_0000 timepoint 0.26685
 finished parsing run0000_0000 timepoint 0.2669
 finished parsing run0000_0000 timepoint 0.26695
 finished parsing run0000_0000 timepoint 0.267
 finished parsing run0000_0000 timepoint 0.26705
 finished parsing run0000_0000 timepoint 0.2671
 finished parsing run0000_0000 timepoint 0.26715
 finished parsing run0000_0000 timepoint 0.2672
 finished parsing run0000_0000 timepoint 0.26725
 finished parsing run0000_0000 timepoint 0.2673
 finished parsing run0000_0000 timepoint 0.26735
 finished parsing run0000_0000 timepoint 0.2674
 finished parsing run0000_0000 timepoint 0.26745
 finished parsing run0000_0000 timepoint 0.2675
 finished parsing run0000_0000 timepoint 0.26755
 finished parsing run0000_0000 timepoint 0.2676
 finished parsing run0000_0000 timepoint 0.26765
 finished parsing run0000_0000 timepoint 0.2677
 finished parsing run0000_0000 timepoint 0.26775
 finished parsing run0000_0000 timepoint 0.2678
 finished parsing run0000_0000 timepoint 0.26785
 finished parsing run0000_0000 timepoint 0.2679
 finished parsing run0000_0000 timepoint 0.26795
 finished parsing run0000_0000 timepoint 0.268
 finished parsing run0000_0000 timepoint 0.26805
 finished parsing run0000_0000 timepoint 0.2681
 finished parsing run0000_0000 timepoint 0.26815
 finished parsing run0000_0000 timepoint 0.2682
 finished parsing run0000_0000 timepoint 0.26825
 finished parsing run0000_0000 timepoint 0.2683
 finished parsing run0000_0000 timepoint 0.26835
 finished parsing run0000_0000 timepoint 0.2684
 finished parsing run0000_0000 timepoint 0.26845
 finished parsing run0000_0000 timepoint 0.2685
 finished parsing run0000_0000 timepoint 0.26855
 finished parsing run0000_0000 timepoint 0.2686
 finished parsing run0000_0000 timepoint 0.26865
 finished parsing run0000_0000 timepoint 0.2687
 finished parsing run0000_0000 timepoint 0.26875
 finished parsing run0000_0000 timepoint 0.2688
 finished parsing run0000_0000 timepoint 0.26885
 finished parsing run0000_0000 timepoint 0.2689
 finished parsing run0000_0000 timepoint 0.26895
 finished parsing run0000_0000 timepoint 0.269
 finished parsing run0000_0000 timepoint 0.26905
 finished parsing run0000_0000 timepoint 0.2691
 finished parsing run0000_0000 timepoint 0.26915
 finished parsing run0000_0000 timepoint 0.2692
 finished parsing run0000_0000 timepoint 0.26925
 finished parsing run0000_0000 timepoint 0.2693
 finished parsing run0000_0000 timepoint 0.26935
 finished parsing run0000_0000 timepoint 0.2694
 finished parsing run0000_0000 timepoint 0.26945
 finished parsing run0000_0000 timepoint 0.2695
 finished parsing run0000_0000 timepoint 0.26955
 finished parsing run0000_0000 timepoint 0.2696
 finished parsing run0000_0000 timepoint 0.26965
 finished parsing run0000_0000 timepoint 0.2697
 finished parsing run0000_0000 timepoint 0.26975
 finished parsing run0000_0000 timepoint 0.2698
 finished parsing run0000_0000 timepoint 0.26985
 finished parsing run0000_0000 timepoint 0.2699
 finished parsing run0000_0000 timepoint 0.26995
 finished parsing run0000_0000 timepoint 0.27
 finished parsing run0000_0000 timepoint 0.27005
 finished parsing run0000_0000 timepoint 0.2701
 finished parsing run0000_0000 timepoint 0.27015
 finished parsing run0000_0000 timepoint 0.2702
 finished parsing run0000_0000 timepoint 0.27025
 finished parsing run0000_0000 timepoint 0.2703
 finished parsing run0000_0000 timepoint 0.27035
 finished parsing run0000_0000 timepoint 0.2704
 finished parsing run0000_0000 timepoint 0.27045
 finished parsing run0000_0000 timepoint 0.2705
 finished parsing run0000_0000 timepoint 0.27055
 finished parsing run0000_0000 timepoint 0.2706
 finished parsing run0000_0000 timepoint 0.27065
 finished parsing run0000_0000 timepoint 0.2707
 finished parsing run0000_0000 timepoint 0.27075
 finished parsing run0000_0000 timepoint 0.2708
 finished parsing run0000_0000 timepoint 0.27085
 finished parsing run0000_0000 timepoint 0.2709
 finished parsing run0000_0000 timepoint 0.27095
 finished parsing run0000_0000 timepoint 0.271
 finished parsing run0000_0000 timepoint 0.27105
 finished parsing run0000_0000 timepoint 0.2711
 finished parsing run0000_0000 timepoint 0.27115
 finished parsing run0000_0000 timepoint 0.2712
 finished parsing run0000_0000 timepoint 0.27125
 finished parsing run0000_0000 timepoint 0.2713
 finished parsing run0000_0000 timepoint 0.27135
 finished parsing run0000_0000 timepoint 0.2714
 finished parsing run0000_0000 timepoint 0.27145
 finished parsing run0000_0000 timepoint 0.2715
 finished parsing run0000_0000 timepoint 0.27155
 finished parsing run0000_0000 timepoint 0.2716
 finished parsing run0000_0000 timepoint 0.27165
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2583
 finished parsing run0000_0000 timepoint 0.25835
 finished parsing run0000_0000 timepoint 0.2584
 finished parsing run0000_0000 timepoint 0.25845
 finished parsing run0000_0000 timepoint 0.2585
 finished parsing run0000_0000 timepoint 0.25855
 finished parsing run0000_0000 timepoint 0.2586
 finished parsing run0000_0000 timepoint 0.25865
 finished parsing run0000_0000 timepoint 0.2587
 finished parsing run0000_0000 timepoint 0.25875
 finished parsing run0000_0000 timepoint 0.2588
 finished parsing run0000_0000 timepoint 0.25885
 finished parsing run0000_0000 timepoint 0.2589
 finished parsing run0000_0000 timepoint 0.25895
 finished parsing run0000_0000 timepoint 0.259
 finished parsing run0000_0000 timepoint 0.25905
 finished parsing run0000_0000 timepoint 0.2591
 finished parsing run0000_0000 timepoint 0.25915
 finished parsing run0000_0000 timepoint 0.2592
 finished parsing run0000_0000 timepoint 0.25925
 finished parsing run0000_0000 timepoint 0.2593
 finished parsing run0000_0000 timepoint 0.25935
 finished parsing run0000_0000 timepoint 0.2594
 finished parsing run0000_0000 timepoint 0.25945
 finished parsing run0000_0000 timepoint 0.2595
 finished parsing run0000_0000 timepoint 0.25955
 finished parsing run0000_0000 timepoint 0.2596
 finished parsing run0000_0000 timepoint 0.25965
 finished parsing run0000_0000 timepoint 0.2597
 finished parsing run0000_0000 timepoint 0.25975
 finished parsing run0000_0000 timepoint 0.2598
 finished parsing run0000_0000 timepoint 0.25985
 finished parsing run0000_0000 timepoint 0.2599
 finished parsing run0000_0000 timepoint 0.25995
 finished parsing run0000_0000 timepoint 0.26
 finished parsing run0000_0000 timepoint 0.26005
 finished parsing run0000_0000 timepoint 0.2601
 finished parsing run0000_0000 timepoint 0.26015
 finished parsing run0000_0000 timepoint 0.2602
 finished parsing run0000_0000 timepoint 0.26025
 finished parsing run0000_0000 timepoint 0.2603
 finished parsing run0000_0000 timepoint 0.26035
 finished parsing run0000_0000 timepoint 0.2604
 finished parsing run0000_0000 timepoint 0.26045
 finished parsing run0000_0000 timepoint 0.2605
 finished parsing run0000_0000 timepoint 0.26055
 finished parsing run0000_0000 timepoint 0.2606
 finished parsing run0000_0000 timepoint 0.26065
 finished parsing run0000_0000 timepoint 0.2607
 finished parsing run0000_0000 timepoint 0.26075
 finished parsing run0000_0000 timepoint 0.2608
 finished parsing run0000_0000 timepoint 0.26085
 finished parsing run0000_0000 timepoint 0.2609
 finished parsing run0000_0000 timepoint 0.26095
 finished parsing run0000_0000 timepoint 0.261
 finished parsing run0000_0000 timepoint 0.26105
 finished parsing run0000_0000 timepoint 0.2611
 finished parsing run0000_0000 timepoint 0.26115
 finished parsing run0000_0000 timepoint 0.2612
 finished parsing run0000_0000 timepoint 0.26125
 finished parsing run0000_0000 timepoint 0.2613
 finished parsing run0000_0000 timepoint 0.26135
 finished parsing run0000_0000 timepoint 0.2614
 finished parsing run0000_0000 timepoint 0.26145
 finished parsing run0000_0000 timepoint 0.2615
 finished parsing run0000_0000 timepoint 0.26155
 finished parsing run0000_0000 timepoint 0.2616
 finished parsing run0000_0000 timepoint 0.26165
 finished parsing run0000_0000 timepoint 0.2617
 finished parsing run0000_0000 timepoint 0.26175
 finished parsing run0000_0000 timepoint 0.2618
 finished parsing run0000_0000 timepoint 0.26185
 finished parsing run0000_0000 timepoint 0.2619
 finished parsing run0000_0000 timepoint 0.26195
 finished parsing run0000_0000 timepoint 0.262
 finished parsing run0000_0000 timepoint 0.26205
 finished parsing run0000_0000 timepoint 0.2621
 finished parsing run0000_0000 timepoint 0.26215
 finished parsing run0000_0000 timepoint 0.2622
 finished parsing run0000_0000 timepoint 0.26225
 finished parsing run0000_0000 timepoint 0.2623
 finished parsing run0000_0000 timepoint 0.26235
 finished parsing run0000_0000 timepoint 0.2624
 finished parsing run0000_0000 timepoint 0.26245
 finished parsing run0000_0000 timepoint 0.2625
 finished parsing run0000_0000 timepoint 0.26255
 finished parsing run0000_0000 timepoint 0.2626
 finished parsing run0000_0000 timepoint 0.26265
 finished parsing run0000_0000 timepoint 0.2627
 finished parsing run0000_0000 timepoint 0.26275
 finished parsing run0000_0000 timepoint 0.2628
 finished parsing run0000_0000 timepoint 0.26285
 finished parsing run0000_0000 timepoint 0.2629
 finished parsing run0000_0000 timepoint 0.26295
 finished parsing run0000_0000 timepoint 0.263
 finished parsing run0000_0000 timepoint 0.26305
 finished parsing run0000_0000 timepoint 0.2631
 finished parsing run0000_0000 timepoint 0.26315
 finished parsing run0000_0000 timepoint 0.2632
 finished parsing run0000_0000 timepoint 0.26325
 finished parsing run0000_0000 timepoint 0.2633
 finished parsing run0000_0000 timepoint 0.26335
 finished parsing run0000_0000 timepoint 0.2634
 finished parsing run0000_0000 timepoint 0.26345
 finished parsing run0000_0000 timepoint 0.2635
 finished parsing run0000_0000 timepoint 0.26355
 finished parsing run0000_0000 timepoint 0.2636
 finished parsing run0000_0000 timepoint 0.26365
 finished parsing run0000_0000 timepoint 0.2637
 finished parsing run0000_0000 timepoint 0.26375
 finished parsing run0000_0000 timepoint 0.2638
 finished parsing run0000_0000 timepoint 0.26385
 finished parsing run0000_0000 timepoint 0.2639
 finished parsing run0000_0000 timepoint 0.26395
 finished parsing run0000_0000 timepoint 0.264
 finished parsing run0000_0000 timepoint 0.26405
 finished parsing run0000_0000 timepoint 0.2641
 finished parsing run0000_0000 timepoint 0.26415
 finished parsing run0000_0000 timepoint 0.2642
 finished parsing run0000_0000 timepoint 0.26425
 finished parsing run0000_0000 timepoint 0.2643
 finished parsing run0000_0000 timepoint 0.26435
 finished parsing run0000_0000 timepoint 0.2644
 finished parsing run0000_0000 timepoint 0.26445
 finished parsing run0000_0000 timepoint 0.2645
 finished parsing run0000_0000 timepoint 0.26455
 finished parsing run0000_0000 timepoint 0.2646
 finished parsing run0000_0000 timepoint 0.26465
 finished parsing run0000_0000 timepoint 0.2647
 finished parsing run0000_0000 timepoint 0.26475
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.25165
 finished parsing run0000_0000 timepoint 0.2517
 finished parsing run0000_0000 timepoint 0.25175
 finished parsing run0000_0000 timepoint 0.2518
 finished parsing run0000_0000 timepoint 0.25185
 finished parsing run0000_0000 timepoint 0.2519
 finished parsing run0000_0000 timepoint 0.25195
 finished parsing run0000_0000 timepoint 0.252
 finished parsing run0000_0000 timepoint 0.25205
 finished parsing run0000_0000 timepoint 0.2521
 finished parsing run0000_0000 timepoint 0.25215
 finished parsing run0000_0000 timepoint 0.2522
 finished parsing run0000_0000 timepoint 0.25225
 finished parsing run0000_0000 timepoint 0.2523
 finished parsing run0000_0000 timepoint 0.25235
 finished parsing run0000_0000 timepoint 0.2524
 finished parsing run0000_0000 timepoint 0.25245
 finished parsing run0000_0000 timepoint 0.2525
 finished parsing run0000_0000 timepoint 0.25255
 finished parsing run0000_0000 timepoint 0.2526
 finished parsing run0000_0000 timepoint 0.25265
 finished parsing run0000_0000 timepoint 0.2527
 finished parsing run0000_0000 timepoint 0.25275
 finished parsing run0000_0000 timepoint 0.2528
 finished parsing run0000_0000 timepoint 0.25285
 finished parsing run0000_0000 timepoint 0.2529
 finished parsing run0000_0000 timepoint 0.25295
 finished parsing run0000_0000 timepoint 0.253
 finished parsing run0000_0000 timepoint 0.25305
 finished parsing run0000_0000 timepoint 0.2531
 finished parsing run0000_0000 timepoint 0.25315
 finished parsing run0000_0000 timepoint 0.2532
 finished parsing run0000_0000 timepoint 0.25325
 finished parsing run0000_0000 timepoint 0.2533
 finished parsing run0000_0000 timepoint 0.25335
 finished parsing run0000_0000 timepoint 0.2534
 finished parsing run0000_0000 timepoint 0.25345
 finished parsing run0000_0000 timepoint 0.2535
 finished parsing run0000_0000 timepoint 0.25355
 finished parsing run0000_0000 timepoint 0.2536
 finished parsing run0000_0000 timepoint 0.25365
 finished parsing run0000_0000 timepoint 0.2537
 finished parsing run0000_0000 timepoint 0.25375
 finished parsing run0000_0000 timepoint 0.2538
 finished parsing run0000_0000 timepoint 0.25385
 finished parsing run0000_0000 timepoint 0.2539
 finished parsing run0000_0000 timepoint 0.25395
 finished parsing run0000_0000 timepoint 0.254
 finished parsing run0000_0000 timepoint 0.25405
 finished parsing run0000_0000 timepoint 0.2541
 finished parsing run0000_0000 timepoint 0.25415
 finished parsing run0000_0000 timepoint 0.2542
 finished parsing run0000_0000 timepoint 0.25425
 finished parsing run0000_0000 timepoint 0.2543
 finished parsing run0000_0000 timepoint 0.25435
 finished parsing run0000_0000 timepoint 0.2544
 finished parsing run0000_0000 timepoint 0.25445
 finished parsing run0000_0000 timepoint 0.2545
 finished parsing run0000_0000 timepoint 0.25455
 finished parsing run0000_0000 timepoint 0.2546
 finished parsing run0000_0000 timepoint 0.25465
 finished parsing run0000_0000 timepoint 0.2547
 finished parsing run0000_0000 timepoint 0.25475
 finished parsing run0000_0000 timepoint 0.2548
 finished parsing run0000_0000 timepoint 0.25485
 finished parsing run0000_0000 timepoint 0.2549
 finished parsing run0000_0000 timepoint 0.25495
 finished parsing run0000_0000 timepoint 0.255
 finished parsing run0000_0000 timepoint 0.25505
 finished parsing run0000_0000 timepoint 0.2551
 finished parsing run0000_0000 timepoint 0.25515
 finished parsing run0000_0000 timepoint 0.2552
 finished parsing run0000_0000 timepoint 0.25525
 finished parsing run0000_0000 timepoint 0.2553
 finished parsing run0000_0000 timepoint 0.25535
 finished parsing run0000_0000 timepoint 0.2554
 finished parsing run0000_0000 timepoint 0.25545
 finished parsing run0000_0000 timepoint 0.2555
 finished parsing run0000_0000 timepoint 0.25555
 finished parsing run0000_0000 timepoint 0.2556
 finished parsing run0000_0000 timepoint 0.25565
 finished parsing run0000_0000 timepoint 0.2557
 finished parsing run0000_0000 timepoint 0.25575
 finished parsing run0000_0000 timepoint 0.2558
 finished parsing run0000_0000 timepoint 0.25585
 finished parsing run0000_0000 timepoint 0.2559
 finished parsing run0000_0000 timepoint 0.25595
 finished parsing run0000_0000 timepoint 0.256
 finished parsing run0000_0000 timepoint 0.25605
 finished parsing run0000_0000 timepoint 0.2561
 finished parsing run0000_0000 timepoint 0.25615
 finished parsing run0000_0000 timepoint 0.2562
 finished parsing run0000_0000 timepoint 0.25625
 finished parsing run0000_0000 timepoint 0.2563
 finished parsing run0000_0000 timepoint 0.25635
 finished parsing run0000_0000 timepoint 0.2564
 finished parsing run0000_0000 timepoint 0.25645
 finished parsing run0000_0000 timepoint 0.2565
 finished parsing run0000_0000 timepoint 0.25655
 finished parsing run0000_0000 timepoint 0.2566
 finished parsing run0000_0000 timepoint 0.25665
 finished parsing run0000_0000 timepoint 0.2567
 finished parsing run0000_0000 timepoint 0.25675
 finished parsing run0000_0000 timepoint 0.2568
 finished parsing run0000_0000 timepoint 0.25685
 finished parsing run0000_0000 timepoint 0.2569
 finished parsing run0000_0000 timepoint 0.25695
 finished parsing run0000_0000 timepoint 0.257
 finished parsing run0000_0000 timepoint 0.25705
 finished parsing run0000_0000 timepoint 0.2571
 finished parsing run0000_0000 timepoint 0.25715
 finished parsing run0000_0000 timepoint 0.2572
 finished parsing run0000_0000 timepoint 0.25725
 finished parsing run0000_0000 timepoint 0.2573
 finished parsing run0000_0000 timepoint 0.25735
 finished parsing run0000_0000 timepoint 0.2574
 finished parsing run0000_0000 timepoint 0.25745
 finished parsing run0000_0000 timepoint 0.2575
 finished parsing run0000_0000 timepoint 0.25755
 finished parsing run0000_0000 timepoint 0.2576
 finished parsing run0000_0000 timepoint 0.25765
 finished parsing run0000_0000 timepoint 0.2577
 finished parsing run0000_0000 timepoint 0.25775
 finished parsing run0000_0000 timepoint 0.2578
 finished parsing run0000_0000 timepoint 0.25785
 finished parsing run0000_0000 timepoint 0.2579
 finished parsing run0000_0000 timepoint 0.25795
 finished parsing run0000_0000 timepoint 0.258
 finished parsing run0000_0000 timepoint 0.25805
 finished parsing run0000_0000 timepoint 0.2581
 finished parsing run0000_0000 timepoint 0.25815
 finished parsing run0000_0000 timepoint 0.2582
 finished parsing run0000_0000 timepoint 0.25825
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2445
 finished parsing run0000_0000 timepoint 0.24455
 finished parsing run0000_0000 timepoint 0.2446
 finished parsing run0000_0000 timepoint 0.24465
 finished parsing run0000_0000 timepoint 0.2447
 finished parsing run0000_0000 timepoint 0.24475
 finished parsing run0000_0000 timepoint 0.2448
 finished parsing run0000_0000 timepoint 0.24485
 finished parsing run0000_0000 timepoint 0.2449
 finished parsing run0000_0000 timepoint 0.24495
 finished parsing run0000_0000 timepoint 0.245
 finished parsing run0000_0000 timepoint 0.24505
 finished parsing run0000_0000 timepoint 0.2451
 finished parsing run0000_0000 timepoint 0.24515
 finished parsing run0000_0000 timepoint 0.2452
 finished parsing run0000_0000 timepoint 0.24525
 finished parsing run0000_0000 timepoint 0.2453
 finished parsing run0000_0000 timepoint 0.24535
 finished parsing run0000_0000 timepoint 0.2454
 finished parsing run0000_0000 timepoint 0.24545
 finished parsing run0000_0000 timepoint 0.2455
 finished parsing run0000_0000 timepoint 0.24555
 finished parsing run0000_0000 timepoint 0.2456
 finished parsing run0000_0000 timepoint 0.24565
 finished parsing run0000_0000 timepoint 0.2457
 finished parsing run0000_0000 timepoint 0.24575
 finished parsing run0000_0000 timepoint 0.2458
 finished parsing run0000_0000 timepoint 0.24585
 finished parsing run0000_0000 timepoint 0.2459
 finished parsing run0000_0000 timepoint 0.24595
 finished parsing run0000_0000 timepoint 0.246
 finished parsing run0000_0000 timepoint 0.24605
 finished parsing run0000_0000 timepoint 0.2461
 finished parsing run0000_0000 timepoint 0.24615
 finished parsing run0000_0000 timepoint 0.2462
 finished parsing run0000_0000 timepoint 0.24625
 finished parsing run0000_0000 timepoint 0.2463
 finished parsing run0000_0000 timepoint 0.24635
 finished parsing run0000_0000 timepoint 0.2464
 finished parsing run0000_0000 timepoint 0.24645
 finished parsing run0000_0000 timepoint 0.2465
 finished parsing run0000_0000 timepoint 0.24655
 finished parsing run0000_0000 timepoint 0.2466
 finished parsing run0000_0000 timepoint 0.24665
 finished parsing run0000_0000 timepoint 0.2467
 finished parsing run0000_0000 timepoint 0.24675
 finished parsing run0000_0000 timepoint 0.2468
 finished parsing run0000_0000 timepoint 0.24685
 finished parsing run0000_0000 timepoint 0.2469
 finished parsing run0000_0000 timepoint 0.24695
 finished parsing run0000_0000 timepoint 0.247
 finished parsing run0000_0000 timepoint 0.24705
 finished parsing run0000_0000 timepoint 0.2471
 finished parsing run0000_0000 timepoint 0.24715
 finished parsing run0000_0000 timepoint 0.2472
 finished parsing run0000_0000 timepoint 0.24725
 finished parsing run0000_0000 timepoint 0.2473
 finished parsing run0000_0000 timepoint 0.24735
 finished parsing run0000_0000 timepoint 0.2474
 finished parsing run0000_0000 timepoint 0.24745
 finished parsing run0000_0000 timepoint 0.2475
 finished parsing run0000_0000 timepoint 0.24755
 finished parsing run0000_0000 timepoint 0.2476
 finished parsing run0000_0000 timepoint 0.24765
 finished parsing run0000_0000 timepoint 0.2477
 finished parsing run0000_0000 timepoint 0.24775
 finished parsing run0000_0000 timepoint 0.2478
 finished parsing run0000_0000 timepoint 0.24785
 finished parsing run0000_0000 timepoint 0.2479
 finished parsing run0000_0000 timepoint 0.24795
 finished parsing run0000_0000 timepoint 0.248
 finished parsing run0000_0000 timepoint 0.24805
 finished parsing run0000_0000 timepoint 0.2481
 finished parsing run0000_0000 timepoint 0.24815
 finished parsing run0000_0000 timepoint 0.2482
 finished parsing run0000_0000 timepoint 0.24825
 finished parsing run0000_0000 timepoint 0.2483
 finished parsing run0000_0000 timepoint 0.24835
 finished parsing run0000_0000 timepoint 0.2484
 finished parsing run0000_0000 timepoint 0.24845
 finished parsing run0000_0000 timepoint 0.2485
 finished parsing run0000_0000 timepoint 0.24855
 finished parsing run0000_0000 timepoint 0.2486
 finished parsing run0000_0000 timepoint 0.24865
 finished parsing run0000_0000 timepoint 0.2487
 finished parsing run0000_0000 timepoint 0.24875
 finished parsing run0000_0000 timepoint 0.2488
 finished parsing run0000_0000 timepoint 0.24885
 finished parsing run0000_0000 timepoint 0.2489
 finished parsing run0000_0000 timepoint 0.24895
 finished parsing run0000_0000 timepoint 0.249
 finished parsing run0000_0000 timepoint 0.24905
 finished parsing run0000_0000 timepoint 0.2491
 finished parsing run0000_0000 timepoint 0.24915
 finished parsing run0000_0000 timepoint 0.2492
 finished parsing run0000_0000 timepoint 0.24925
 finished parsing run0000_0000 timepoint 0.2493
 finished parsing run0000_0000 timepoint 0.24935
 finished parsing run0000_0000 timepoint 0.2494
 finished parsing run0000_0000 timepoint 0.24945
 finished parsing run0000_0000 timepoint 0.2495
 finished parsing run0000_0000 timepoint 0.24955
 finished parsing run0000_0000 timepoint 0.2496
 finished parsing run0000_0000 timepoint 0.24965
 finished parsing run0000_0000 timepoint 0.2497
 finished parsing run0000_0000 timepoint 0.24975
 finished parsing run0000_0000 timepoint 0.2498
 finished parsing run0000_0000 timepoint 0.24985
 finished parsing run0000_0000 timepoint 0.2499
 finished parsing run0000_0000 timepoint 0.24995
 finished parsing run0000_0000 timepoint 0.25
 finished parsing run0000_0000 timepoint 0.25005
 finished parsing run0000_0000 timepoint 0.2501
 finished parsing run0000_0000 timepoint 0.25015
 finished parsing run0000_0000 timepoint 0.2502
 finished parsing run0000_0000 timepoint 0.25025
 finished parsing run0000_0000 timepoint 0.2503
 finished parsing run0000_0000 timepoint 0.25035
 finished parsing run0000_0000 timepoint 0.2504
 finished parsing run0000_0000 timepoint 0.25045
 finished parsing run0000_0000 timepoint 0.2505
 finished parsing run0000_0000 timepoint 0.25055
 finished parsing run0000_0000 timepoint 0.2506
 finished parsing run0000_0000 timepoint 0.25065
 finished parsing run0000_0000 timepoint 0.2507
 finished parsing run0000_0000 timepoint 0.25075
 finished parsing run0000_0000 timepoint 0.2508
 finished parsing run0000_0000 timepoint 0.25085
 finished parsing run0000_0000 timepoint 0.2509
 finished parsing run0000_0000 timepoint 0.25095
 finished parsing run0000_0000 timepoint 0.251
 finished parsing run0000_0000 timepoint 0.25105
 finished parsing run0000_0000 timepoint 0.2511
 finished parsing run0000_0000 timepoint 0.25115
 finished parsing run0000_0000 timepoint 0.2512
 finished parsing run0000_0000 timepoint 0.25125
 finished parsing run0000_0000 timepoint 0.2513
 finished parsing run0000_0000 timepoint 0.25135
 finished parsing run0000_0000 timepoint 0.2514
 finished parsing run0000_0000 timepoint 0.25145
 finished parsing run0000_0000 timepoint 0.2515
 finished parsing run0000_0000 timepoint 0.25155
 finished parsing run0000_0000 timepoint 0.2516
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.2377
 finished parsing run0000_0000 timepoint 0.23775
 finished parsing run0000_0000 timepoint 0.2378
 finished parsing run0000_0000 timepoint 0.23785
 finished parsing run0000_0000 timepoint 0.2379
 finished parsing run0000_0000 timepoint 0.23795
 finished parsing run0000_0000 timepoint 0.238
 finished parsing run0000_0000 timepoint 0.23805
 finished parsing run0000_0000 timepoint 0.2381
 finished parsing run0000_0000 timepoint 0.23815
 finished parsing run0000_0000 timepoint 0.2382
 finished parsing run0000_0000 timepoint 0.23825
 finished parsing run0000_0000 timepoint 0.2383
 finished parsing run0000_0000 timepoint 0.23835
 finished parsing run0000_0000 timepoint 0.2384
 finished parsing run0000_0000 timepoint 0.23845
 finished parsing run0000_0000 timepoint 0.2385
 finished parsing run0000_0000 timepoint 0.23855
 finished parsing run0000_0000 timepoint 0.2386
 finished parsing run0000_0000 timepoint 0.23865
 finished parsing run0000_0000 timepoint 0.2387
 finished parsing run0000_0000 timepoint 0.23875
 finished parsing run0000_0000 timepoint 0.2388
 finished parsing run0000_0000 timepoint 0.23885
 finished parsing run0000_0000 timepoint 0.2389
 finished parsing run0000_0000 timepoint 0.23895
 finished parsing run0000_0000 timepoint 0.239
 finished parsing run0000_0000 timepoint 0.23905
 finished parsing run0000_0000 timepoint 0.2391
 finished parsing run0000_0000 timepoint 0.23915
 finished parsing run0000_0000 timepoint 0.2392
 finished parsing run0000_0000 timepoint 0.23925
 finished parsing run0000_0000 timepoint 0.2393
 finished parsing run0000_0000 timepoint 0.23935
 finished parsing run0000_0000 timepoint 0.2394
 finished parsing run0000_0000 timepoint 0.23945
 finished parsing run0000_0000 timepoint 0.2395
 finished parsing run0000_0000 timepoint 0.23955
 finished parsing run0000_0000 timepoint 0.2396
 finished parsing run0000_0000 timepoint 0.23965
 finished parsing run0000_0000 timepoint 0.2397
 finished parsing run0000_0000 timepoint 0.23975
 finished parsing run0000_0000 timepoint 0.2398
 finished parsing run0000_0000 timepoint 0.23985
 finished parsing run0000_0000 timepoint 0.2399
 finished parsing run0000_0000 timepoint 0.23995
 finished parsing run0000_0000 timepoint 0.24
 finished parsing run0000_0000 timepoint 0.24005
 finished parsing run0000_0000 timepoint 0.2401
 finished parsing run0000_0000 timepoint 0.24015
 finished parsing run0000_0000 timepoint 0.2402
 finished parsing run0000_0000 timepoint 0.24025
 finished parsing run0000_0000 timepoint 0.2403
 finished parsing run0000_0000 timepoint 0.24035
 finished parsing run0000_0000 timepoint 0.2404
 finished parsing run0000_0000 timepoint 0.24045
 finished parsing run0000_0000 timepoint 0.2405
 finished parsing run0000_0000 timepoint 0.24055
 finished parsing run0000_0000 timepoint 0.2406
 finished parsing run0000_0000 timepoint 0.24065
 finished parsing run0000_0000 timepoint 0.2407
 finished parsing run0000_0000 timepoint 0.24075
 finished parsing run0000_0000 timepoint 0.2408
 finished parsing run0000_0000 timepoint 0.24085
 finished parsing run0000_0000 timepoint 0.2409
 finished parsing run0000_0000 timepoint 0.24095
 finished parsing run0000_0000 timepoint 0.241
 finished parsing run0000_0000 timepoint 0.24105
 finished parsing run0000_0000 timepoint 0.2411
 finished parsing run0000_0000 timepoint 0.24115
 finished parsing run0000_0000 timepoint 0.2412
 finished parsing run0000_0000 timepoint 0.24125
 finished parsing run0000_0000 timepoint 0.2413
 finished parsing run0000_0000 timepoint 0.24135
 finished parsing run0000_0000 timepoint 0.2414
 finished parsing run0000_0000 timepoint 0.24145
 finished parsing run0000_0000 timepoint 0.2415
 finished parsing run0000_0000 timepoint 0.24155
 finished parsing run0000_0000 timepoint 0.2416
 finished parsing run0000_0000 timepoint 0.24165
 finished parsing run0000_0000 timepoint 0.2417
 finished parsing run0000_0000 timepoint 0.24175
 finished parsing run0000_0000 timepoint 0.2418
 finished parsing run0000_0000 timepoint 0.24185
 finished parsing run0000_0000 timepoint 0.2419
 finished parsing run0000_0000 timepoint 0.24195
 finished parsing run0000_0000 timepoint 0.242
 finished parsing run0000_0000 timepoint 0.24205
 finished parsing run0000_0000 timepoint 0.2421
 finished parsing run0000_0000 timepoint 0.24215
 finished parsing run0000_0000 timepoint 0.2422
 finished parsing run0000_0000 timepoint 0.24225
 finished parsing run0000_0000 timepoint 0.2423
 finished parsing run0000_0000 timepoint 0.24235
 finished parsing run0000_0000 timepoint 0.2424
 finished parsing run0000_0000 timepoint 0.24245
 finished parsing run0000_0000 timepoint 0.2425
 finished parsing run0000_0000 timepoint 0.24255
 finished parsing run0000_0000 timepoint 0.2426
 finished parsing run0000_0000 timepoint 0.24265
 finished parsing run0000_0000 timepoint 0.2427
 finished parsing run0000_0000 timepoint 0.24275
 finished parsing run0000_0000 timepoint 0.2428
 finished parsing run0000_0000 timepoint 0.24285
 finished parsing run0000_0000 timepoint 0.2429
 finished parsing run0000_0000 timepoint 0.24295
 finished parsing run0000_0000 timepoint 0.243
 finished parsing run0000_0000 timepoint 0.24305
 finished parsing run0000_0000 timepoint 0.2431
 finished parsing run0000_0000 timepoint 0.24315
 finished parsing run0000_0000 timepoint 0.2432
 finished parsing run0000_0000 timepoint 0.24325
 finished parsing run0000_0000 timepoint 0.2433
 finished parsing run0000_0000 timepoint 0.24335
 finished parsing run0000_0000 timepoint 0.2434
 finished parsing run0000_0000 timepoint 0.24345
 finished parsing run0000_0000 timepoint 0.2435
 finished parsing run0000_0000 timepoint 0.24355
 finished parsing run0000_0000 timepoint 0.2436
 finished parsing run0000_0000 timepoint 0.24365
 finished parsing run0000_0000 timepoint 0.2437
 finished parsing run0000_0000 timepoint 0.24375
 finished parsing run0000_0000 timepoint 0.2438
 finished parsing run0000_0000 timepoint 0.24385
 finished parsing run0000_0000 timepoint 0.2439
 finished parsing run0000_0000 timepoint 0.24395
 finished parsing run0000_0000 timepoint 0.244
 finished parsing run0000_0000 timepoint 0.24405
 finished parsing run0000_0000 timepoint 0.2441
 finished parsing run0000_0000 timepoint 0.24415
 finished parsing run0000_0000 timepoint 0.2442
 finished parsing run0000_0000 timepoint 0.24425
 finished parsing run0000_0000 timepoint 0.2443
 finished parsing run0000_0000 timepoint 0.24435
 finished parsing run0000_0000 timepoint 0.2444
 finished parsing run0000_0000 timepoint 0.24445
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.23115
 finished parsing run0000_0000 timepoint 0.2312
 finished parsing run0000_0000 timepoint 0.23125
 finished parsing run0000_0000 timepoint 0.2313
 finished parsing run0000_0000 timepoint 0.23135
 finished parsing run0000_0000 timepoint 0.2314
 finished parsing run0000_0000 timepoint 0.23145
 finished parsing run0000_0000 timepoint 0.2315
 finished parsing run0000_0000 timepoint 0.23155
 finished parsing run0000_0000 timepoint 0.2316
 finished parsing run0000_0000 timepoint 0.23165
 finished parsing run0000_0000 timepoint 0.2317
 finished parsing run0000_0000 timepoint 0.23175
 finished parsing run0000_0000 timepoint 0.2318
 finished parsing run0000_0000 timepoint 0.23185
 finished parsing run0000_0000 timepoint 0.2319
 finished parsing run0000_0000 timepoint 0.23195
 finished parsing run0000_0000 timepoint 0.232
 finished parsing run0000_0000 timepoint 0.23205
 finished parsing run0000_0000 timepoint 0.2321
 finished parsing run0000_0000 timepoint 0.23215
 finished parsing run0000_0000 timepoint 0.2322
 finished parsing run0000_0000 timepoint 0.23225
 finished parsing run0000_0000 timepoint 0.2323
 finished parsing run0000_0000 timepoint 0.23235
 finished parsing run0000_0000 timepoint 0.2324
 finished parsing run0000_0000 timepoint 0.23245
 finished parsing run0000_0000 timepoint 0.2325
 finished parsing run0000_0000 timepoint 0.23255
 finished parsing run0000_0000 timepoint 0.2326
 finished parsing run0000_0000 timepoint 0.23265
 finished parsing run0000_0000 timepoint 0.2327
 finished parsing run0000_0000 timepoint 0.23275
 finished parsing run0000_0000 timepoint 0.2328
 finished parsing run0000_0000 timepoint 0.23285
 finished parsing run0000_0000 timepoint 0.2329
 finished parsing run0000_0000 timepoint 0.23295
 finished parsing run0000_0000 timepoint 0.233
 finished parsing run0000_0000 timepoint 0.23305
 finished parsing run0000_0000 timepoint 0.2331
 finished parsing run0000_0000 timepoint 0.23315
 finished parsing run0000_0000 timepoint 0.2332
 finished parsing run0000_0000 timepoint 0.23325
 finished parsing run0000_0000 timepoint 0.2333
 finished parsing run0000_0000 timepoint 0.23335
 finished parsing run0000_0000 timepoint 0.2334
 finished parsing run0000_0000 timepoint 0.23345
 finished parsing run0000_0000 timepoint 0.2335
 finished parsing run0000_0000 timepoint 0.23355
 finished parsing run0000_0000 timepoint 0.2336
 finished parsing run0000_0000 timepoint 0.23365
 finished parsing run0000_0000 timepoint 0.2337
 finished parsing run0000_0000 timepoint 0.23375
 finished parsing run0000_0000 timepoint 0.2338
 finished parsing run0000_0000 timepoint 0.23385
 finished parsing run0000_0000 timepoint 0.2339
 finished parsing run0000_0000 timepoint 0.23395
 finished parsing run0000_0000 timepoint 0.234
 finished parsing run0000_0000 timepoint 0.23405
 finished parsing run0000_0000 timepoint 0.2341
 finished parsing run0000_0000 timepoint 0.23415
 finished parsing run0000_0000 timepoint 0.2342
 finished parsing run0000_0000 timepoint 0.23425
 finished parsing run0000_0000 timepoint 0.2343
 finished parsing run0000_0000 timepoint 0.23435
 finished parsing run0000_0000 timepoint 0.2344
 finished parsing run0000_0000 timepoint 0.23445
 finished parsing run0000_0000 timepoint 0.2345
 finished parsing run0000_0000 timepoint 0.23455
 finished parsing run0000_0000 timepoint 0.2346
 finished parsing run0000_0000 timepoint 0.23465
 finished parsing run0000_0000 timepoint 0.2347
 finished parsing run0000_0000 timepoint 0.23475
 finished parsing run0000_0000 timepoint 0.2348
 finished parsing run0000_0000 timepoint 0.23485
 finished parsing run0000_0000 timepoint 0.2349
 finished parsing run0000_0000 timepoint 0.23495
 finished parsing run0000_0000 timepoint 0.235
 finished parsing run0000_0000 timepoint 0.23505
 finished parsing run0000_0000 timepoint 0.2351
 finished parsing run0000_0000 timepoint 0.23515
 finished parsing run0000_0000 timepoint 0.2352
 finished parsing run0000_0000 timepoint 0.23525
 finished parsing run0000_0000 timepoint 0.2353
 finished parsing run0000_0000 timepoint 0.23535
 finished parsing run0000_0000 timepoint 0.2354
 finished parsing run0000_0000 timepoint 0.23545
 finished parsing run0000_0000 timepoint 0.2355
 finished parsing run0000_0000 timepoint 0.23555
 finished parsing run0000_0000 timepoint 0.2356
 finished parsing run0000_0000 timepoint 0.23565
 finished parsing run0000_0000 timepoint 0.2357
 finished parsing run0000_0000 timepoint 0.23575
 finished parsing run0000_0000 timepoint 0.2358
 finished parsing run0000_0000 timepoint 0.23585
 finished parsing run0000_0000 timepoint 0.2359
 finished parsing run0000_0000 timepoint 0.23595
 finished parsing run0000_0000 timepoint 0.236
 finished parsing run0000_0000 timepoint 0.23605
 finished parsing run0000_0000 timepoint 0.2361
 finished parsing run0000_0000 timepoint 0.23615
 finished parsing run0000_0000 timepoint 0.2362
 finished parsing run0000_0000 timepoint 0.23625
 finished parsing run0000_0000 timepoint 0.2363
 finished parsing run0000_0000 timepoint 0.23635
 finished parsing run0000_0000 timepoint 0.2364
 finished parsing run0000_0000 timepoint 0.23645
 finished parsing run0000_0000 timepoint 0.2365
 finished parsing run0000_0000 timepoint 0.23655
 finished parsing run0000_0000 timepoint 0.2366
 finished parsing run0000_0000 timepoint 0.23665
 finished parsing run0000_0000 timepoint 0.2367
 finished parsing run0000_0000 timepoint 0.23675
 finished parsing run0000_0000 timepoint 0.2368
 finished parsing run0000_0000 timepoint 0.23685
 finished parsing run0000_0000 timepoint 0.2369
 finished parsing run0000_0000 timepoint 0.23695
 finished parsing run0000_0000 timepoint 0.237
 finished parsing run0000_0000 timepoint 0.23705
 finished parsing run0000_0000 timepoint 0.2371
 finished parsing run0000_0000 timepoint 0.23715
 finished parsing run0000_0000 timepoint 0.2372
 finished parsing run0000_0000 timepoint 0.23725
 finished parsing run0000_0000 timepoint 0.2373
 finished parsing run0000_0000 timepoint 0.23735
 finished parsing run0000_0000 timepoint 0.2374
 finished parsing run0000_0000 timepoint 0.23745
 finished parsing run0000_0000 timepoint 0.2375
 finished parsing run0000_0000 timepoint 0.23755
 finished parsing run0000_0000 timepoint 0.2376
 finished parsing run0000_0000 timepoint 0.23765
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.22455
 finished parsing run0000_0000 timepoint 0.2246
 finished parsing run0000_0000 timepoint 0.22465
 finished parsing run0000_0000 timepoint 0.2247
 finished parsing run0000_0000 timepoint 0.22475
 finished parsing run0000_0000 timepoint 0.2248
 finished parsing run0000_0000 timepoint 0.22485
 finished parsing run0000_0000 timepoint 0.2249
 finished parsing run0000_0000 timepoint 0.22495
 finished parsing run0000_0000 timepoint 0.225
 finished parsing run0000_0000 timepoint 0.22505
 finished parsing run0000_0000 timepoint 0.2251
 finished parsing run0000_0000 timepoint 0.22515
 finished parsing run0000_0000 timepoint 0.2252
 finished parsing run0000_0000 timepoint 0.22525
 finished parsing run0000_0000 timepoint 0.2253
 finished parsing run0000_0000 timepoint 0.22535
 finished parsing run0000_0000 timepoint 0.2254
 finished parsing run0000_0000 timepoint 0.22545
 finished parsing run0000_0000 timepoint 0.2255
 finished parsing run0000_0000 timepoint 0.22555
 finished parsing run0000_0000 timepoint 0.2256
 finished parsing run0000_0000 timepoint 0.22565
 finished parsing run0000_0000 timepoint 0.2257
 finished parsing run0000_0000 timepoint 0.22575
 finished parsing run0000_0000 timepoint 0.2258
 finished parsing run0000_0000 timepoint 0.22585
 finished parsing run0000_0000 timepoint 0.2259
 finished parsing run0000_0000 timepoint 0.22595
 finished parsing run0000_0000 timepoint 0.226
 finished parsing run0000_0000 timepoint 0.22605
 finished parsing run0000_0000 timepoint 0.2261
 finished parsing run0000_0000 timepoint 0.22615
 finished parsing run0000_0000 timepoint 0.2262
 finished parsing run0000_0000 timepoint 0.22625
 finished parsing run0000_0000 timepoint 0.2263
 finished parsing run0000_0000 timepoint 0.22635
 finished parsing run0000_0000 timepoint 0.2264
 finished parsing run0000_0000 timepoint 0.22645
 finished parsing run0000_0000 timepoint 0.2265
 finished parsing run0000_0000 timepoint 0.22655
 finished parsing run0000_0000 timepoint 0.2266
 finished parsing run0000_0000 timepoint 0.22665
 finished parsing run0000_0000 timepoint 0.2267
 finished parsing run0000_0000 timepoint 0.22675
 finished parsing run0000_0000 timepoint 0.2268
 finished parsing run0000_0000 timepoint 0.22685
 finished parsing run0000_0000 timepoint 0.2269
 finished parsing run0000_0000 timepoint 0.22695
 finished parsing run0000_0000 timepoint 0.227
 finished parsing run0000_0000 timepoint 0.22705
 finished parsing run0000_0000 timepoint 0.2271
 finished parsing run0000_0000 timepoint 0.22715
 finished parsing run0000_0000 timepoint 0.2272
 finished parsing run0000_0000 timepoint 0.22725
 finished parsing run0000_0000 timepoint 0.2273
 finished parsing run0000_0000 timepoint 0.22735
 finished parsing run0000_0000 timepoint 0.2274
 finished parsing run0000_0000 timepoint 0.22745
 finished parsing run0000_0000 timepoint 0.2275
 finished parsing run0000_0000 timepoint 0.22755
 finished parsing run0000_0000 timepoint 0.2276
 finished parsing run0000_0000 timepoint 0.22765
 finished parsing run0000_0000 timepoint 0.2277
 finished parsing run0000_0000 timepoint 0.22775
 finished parsing run0000_0000 timepoint 0.2278
 finished parsing run0000_0000 timepoint 0.22785
 finished parsing run0000_0000 timepoint 0.2279
 finished parsing run0000_0000 timepoint 0.22795
 finished parsing run0000_0000 timepoint 0.228
 finished parsing run0000_0000 timepoint 0.22805
 finished parsing run0000_0000 timepoint 0.2281
 finished parsing run0000_0000 timepoint 0.22815
 finished parsing run0000_0000 timepoint 0.2282
 finished parsing run0000_0000 timepoint 0.22825
 finished parsing run0000_0000 timepoint 0.2283
 finished parsing run0000_0000 timepoint 0.22835
 finished parsing run0000_0000 timepoint 0.2284
 finished parsing run0000_0000 timepoint 0.22845
 finished parsing run0000_0000 timepoint 0.2285
 finished parsing run0000_0000 timepoint 0.22855
 finished parsing run0000_0000 timepoint 0.2286
 finished parsing run0000_0000 timepoint 0.22865
 finished parsing run0000_0000 timepoint 0.2287
 finished parsing run0000_0000 timepoint 0.22875
 finished parsing run0000_0000 timepoint 0.2288
 finished parsing run0000_0000 timepoint 0.22885
 finished parsing run0000_0000 timepoint 0.2289
 finished parsing run0000_0000 timepoint 0.22895
 finished parsing run0000_0000 timepoint 0.229
 finished parsing run0000_0000 timepoint 0.22905
 finished parsing run0000_0000 timepoint 0.2291
 finished parsing run0000_0000 timepoint 0.22915
 finished parsing run0000_0000 timepoint 0.2292
 finished parsing run0000_0000 timepoint 0.22925
 finished parsing run0000_0000 timepoint 0.2293
 finished parsing run0000_0000 timepoint 0.22935
 finished parsing run0000_0000 timepoint 0.2294
 finished parsing run0000_0000 timepoint 0.22945
 finished parsing run0000_0000 timepoint 0.2295
 finished parsing run0000_0000 timepoint 0.22955
 finished parsing run0000_0000 timepoint 0.2296
 finished parsing run0000_0000 timepoint 0.22965
 finished parsing run0000_0000 timepoint 0.2297
 finished parsing run0000_0000 timepoint 0.22975
 finished parsing run0000_0000 timepoint 0.2298
 finished parsing run0000_0000 timepoint 0.22985
 finished parsing run0000_0000 timepoint 0.2299
 finished parsing run0000_0000 timepoint 0.22995
 finished parsing run0000_0000 timepoint 0.23
 finished parsing run0000_0000 timepoint 0.23005
 finished parsing run0000_0000 timepoint 0.2301
 finished parsing run0000_0000 timepoint 0.23015
 finished parsing run0000_0000 timepoint 0.2302
 finished parsing run0000_0000 timepoint 0.23025
 finished parsing run0000_0000 timepoint 0.2303
 finished parsing run0000_0000 timepoint 0.23035
 finished parsing run0000_0000 timepoint 0.2304
 finished parsing run0000_0000 timepoint 0.23045
 finished parsing run0000_0000 timepoint 0.2305
 finished parsing run0000_0000 timepoint 0.23055
 finished parsing run0000_0000 timepoint 0.2306
 finished parsing run0000_0000 timepoint 0.23065
 finished parsing run0000_0000 timepoint 0.2307
 finished parsing run0000_0000 timepoint 0.23075
 finished parsing run0000_0000 timepoint 0.2308
 finished parsing run0000_0000 timepoint 0.23085
 finished parsing run0000_0000 timepoint 0.2309
 finished parsing run0000_0000 timepoint 0.23095
 finished parsing run0000_0000 timepoint 0.231
 finished parsing run0000_0000 timepoint 0.23105
 finished parsing run0000_0000 timepoint 0.2311
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.218
 finished parsing run0000_0000 timepoint 0.21805
 finished parsing run0000_0000 timepoint 0.2181
 finished parsing run0000_0000 timepoint 0.21815
 finished parsing run0000_0000 timepoint 0.2182
 finished parsing run0000_0000 timepoint 0.21825
 finished parsing run0000_0000 timepoint 0.2183
 finished parsing run0000_0000 timepoint 0.21835
 finished parsing run0000_0000 timepoint 0.2184
 finished parsing run0000_0000 timepoint 0.21845
 finished parsing run0000_0000 timepoint 0.2185
 finished parsing run0000_0000 timepoint 0.21855
 finished parsing run0000_0000 timepoint 0.2186
 finished parsing run0000_0000 timepoint 0.21865
 finished parsing run0000_0000 timepoint 0.2187
 finished parsing run0000_0000 timepoint 0.21875
 finished parsing run0000_0000 timepoint 0.2188
 finished parsing run0000_0000 timepoint 0.21885
 finished parsing run0000_0000 timepoint 0.2189
 finished parsing run0000_0000 timepoint 0.21895
 finished parsing run0000_0000 timepoint 0.219
 finished parsing run0000_0000 timepoint 0.21905
 finished parsing run0000_0000 timepoint 0.2191
 finished parsing run0000_0000 timepoint 0.21915
 finished parsing run0000_0000 timepoint 0.2192
 finished parsing run0000_0000 timepoint 0.21925
 finished parsing run0000_0000 timepoint 0.2193
 finished parsing run0000_0000 timepoint 0.21935
 finished parsing run0000_0000 timepoint 0.2194
 finished parsing run0000_0000 timepoint 0.21945
 finished parsing run0000_0000 timepoint 0.2195
 finished parsing run0000_0000 timepoint 0.21955
 finished parsing run0000_0000 timepoint 0.2196
 finished parsing run0000_0000 timepoint 0.21965
 finished parsing run0000_0000 timepoint 0.2197
 finished parsing run0000_0000 timepoint 0.21975
 finished parsing run0000_0000 timepoint 0.2198
 finished parsing run0000_0000 timepoint 0.21985
 finished parsing run0000_0000 timepoint 0.2199
 finished parsing run0000_0000 timepoint 0.21995
 finished parsing run0000_0000 timepoint 0.22
 finished parsing run0000_0000 timepoint 0.22005
 finished parsing run0000_0000 timepoint 0.2201
 finished parsing run0000_0000 timepoint 0.22015
 finished parsing run0000_0000 timepoint 0.2202
 finished parsing run0000_0000 timepoint 0.22025
 finished parsing run0000_0000 timepoint 0.2203
 finished parsing run0000_0000 timepoint 0.22035
 finished parsing run0000_0000 timepoint 0.2204
 finished parsing run0000_0000 timepoint 0.22045
 finished parsing run0000_0000 timepoint 0.2205
 finished parsing run0000_0000 timepoint 0.22055
 finished parsing run0000_0000 timepoint 0.2206
 finished parsing run0000_0000 timepoint 0.22065
 finished parsing run0000_0000 timepoint 0.2207
 finished parsing run0000_0000 timepoint 0.22075
 finished parsing run0000_0000 timepoint 0.2208
 finished parsing run0000_0000 timepoint 0.22085
 finished parsing run0000_0000 timepoint 0.2209
 finished parsing run0000_0000 timepoint 0.22095
 finished parsing run0000_0000 timepoint 0.221
 finished parsing run0000_0000 timepoint 0.22105
 finished parsing run0000_0000 timepoint 0.2211
 finished parsing run0000_0000 timepoint 0.22115
 finished parsing run0000_0000 timepoint 0.2212
 finished parsing run0000_0000 timepoint 0.22125
 finished parsing run0000_0000 timepoint 0.2213
 finished parsing run0000_0000 timepoint 0.22135
 finished parsing run0000_0000 timepoint 0.2214
 finished parsing run0000_0000 timepoint 0.22145
 finished parsing run0000_0000 timepoint 0.2215
 finished parsing run0000_0000 timepoint 0.22155
 finished parsing run0000_0000 timepoint 0.2216
 finished parsing run0000_0000 timepoint 0.22165
 finished parsing run0000_0000 timepoint 0.2217
 finished parsing run0000_0000 timepoint 0.22175
 finished parsing run0000_0000 timepoint 0.2218
 finished parsing run0000_0000 timepoint 0.22185
 finished parsing run0000_0000 timepoint 0.2219
 finished parsing run0000_0000 timepoint 0.22195
 finished parsing run0000_0000 timepoint 0.222
 finished parsing run0000_0000 timepoint 0.22205
 finished parsing run0000_0000 timepoint 0.2221
 finished parsing run0000_0000 timepoint 0.22215
 finished parsing run0000_0000 timepoint 0.2222
 finished parsing run0000_0000 timepoint 0.22225
 finished parsing run0000_0000 timepoint 0.2223
 finished parsing run0000_0000 timepoint 0.22235
 finished parsing run0000_0000 timepoint 0.2224
 finished parsing run0000_0000 timepoint 0.22245
 finished parsing run0000_0000 timepoint 0.2225
 finished parsing run0000_0000 timepoint 0.22255
 finished parsing run0000_0000 timepoint 0.2226
 finished parsing run0000_0000 timepoint 0.22265
 finished parsing run0000_0000 timepoint 0.2227
 finished parsing run0000_0000 timepoint 0.22275
 finished parsing run0000_0000 timepoint 0.2228
 finished parsing run0000_0000 timepoint 0.22285
 finished parsing run0000_0000 timepoint 0.2229
 finished parsing run0000_0000 timepoint 0.22295
 finished parsing run0000_0000 timepoint 0.223
 finished parsing run0000_0000 timepoint 0.22305
 finished parsing run0000_0000 timepoint 0.2231
 finished parsing run0000_0000 timepoint 0.22315
 finished parsing run0000_0000 timepoint 0.2232
 finished parsing run0000_0000 timepoint 0.22325
 finished parsing run0000_0000 timepoint 0.2233
 finished parsing run0000_0000 timepoint 0.22335
 finished parsing run0000_0000 timepoint 0.2234
 finished parsing run0000_0000 timepoint 0.22345
 finished parsing run0000_0000 timepoint 0.2235
 finished parsing run0000_0000 timepoint 0.22355
 finished parsing run0000_0000 timepoint 0.2236
 finished parsing run0000_0000 timepoint 0.22365
 finished parsing run0000_0000 timepoint 0.2237
 finished parsing run0000_0000 timepoint 0.22375
 finished parsing run0000_0000 timepoint 0.2238
 finished parsing run0000_0000 timepoint 0.22385
 finished parsing run0000_0000 timepoint 0.2239
 finished parsing run0000_0000 timepoint 0.22395
 finished parsing run0000_0000 timepoint 0.224
 finished parsing run0000_0000 timepoint 0.22405
 finished parsing run0000_0000 timepoint 0.2241
 finished parsing run0000_0000 timepoint 0.22415
 finished parsing run0000_0000 timepoint 0.2242
 finished parsing run0000_0000 timepoint 0.22425
 finished parsing run0000_0000 timepoint 0.2243
 finished parsing run0000_0000 timepoint 0.22435
 finished parsing run0000_0000 timepoint 0.2244
 finished parsing run0000_0000 timepoint 0.22445
 finished parsing run0000_0000 timepoint 0.2245
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.21125
 finished parsing run0000_0000 timepoint 0.2113
 finished parsing run0000_0000 timepoint 0.21135
 finished parsing run0000_0000 timepoint 0.2114
 finished parsing run0000_0000 timepoint 0.21145
 finished parsing run0000_0000 timepoint 0.2115
 finished parsing run0000_0000 timepoint 0.21155
 finished parsing run0000_0000 timepoint 0.2116
 finished parsing run0000_0000 timepoint 0.21165
 finished parsing run0000_0000 timepoint 0.2117
 finished parsing run0000_0000 timepoint 0.21175
 finished parsing run0000_0000 timepoint 0.2118
 finished parsing run0000_0000 timepoint 0.21185
 finished parsing run0000_0000 timepoint 0.2119
 finished parsing run0000_0000 timepoint 0.21195
 finished parsing run0000_0000 timepoint 0.212
 finished parsing run0000_0000 timepoint 0.21205
 finished parsing run0000_0000 timepoint 0.2121
 finished parsing run0000_0000 timepoint 0.21215
 finished parsing run0000_0000 timepoint 0.2122
 finished parsing run0000_0000 timepoint 0.21225
 finished parsing run0000_0000 timepoint 0.2123
 finished parsing run0000_0000 timepoint 0.21235
 finished parsing run0000_0000 timepoint 0.2124
 finished parsing run0000_0000 timepoint 0.21245
 finished parsing run0000_0000 timepoint 0.2125
 finished parsing run0000_0000 timepoint 0.21255
 finished parsing run0000_0000 timepoint 0.2126
 finished parsing run0000_0000 timepoint 0.21265
 finished parsing run0000_0000 timepoint 0.2127
 finished parsing run0000_0000 timepoint 0.21275
 finished parsing run0000_0000 timepoint 0.2128
 finished parsing run0000_0000 timepoint 0.21285
 finished parsing run0000_0000 timepoint 0.2129
 finished parsing run0000_0000 timepoint 0.21295
 finished parsing run0000_0000 timepoint 0.213
 finished parsing run0000_0000 timepoint 0.21305
 finished parsing run0000_0000 timepoint 0.2131
 finished parsing run0000_0000 timepoint 0.21315
 finished parsing run0000_0000 timepoint 0.2132
 finished parsing run0000_0000 timepoint 0.21325
 finished parsing run0000_0000 timepoint 0.2133
 finished parsing run0000_0000 timepoint 0.21335
 finished parsing run0000_0000 timepoint 0.2134
 finished parsing run0000_0000 timepoint 0.21345
 finished parsing run0000_0000 timepoint 0.2135
 finished parsing run0000_0000 timepoint 0.21355
 finished parsing run0000_0000 timepoint 0.2136
 finished parsing run0000_0000 timepoint 0.21365
 finished parsing run0000_0000 timepoint 0.2137
 finished parsing run0000_0000 timepoint 0.21375
 finished parsing run0000_0000 timepoint 0.2138
 finished parsing run0000_0000 timepoint 0.21385
 finished parsing run0000_0000 timepoint 0.2139
 finished parsing run0000_0000 timepoint 0.21395
 finished parsing run0000_0000 timepoint 0.214
 finished parsing run0000_0000 timepoint 0.21405
 finished parsing run0000_0000 timepoint 0.2141
 finished parsing run0000_0000 timepoint 0.21415
 finished parsing run0000_0000 timepoint 0.2142
 finished parsing run0000_0000 timepoint 0.21425
 finished parsing run0000_0000 timepoint 0.2143
 finished parsing run0000_0000 timepoint 0.21435
 finished parsing run0000_0000 timepoint 0.2144
 finished parsing run0000_0000 timepoint 0.21445
 finished parsing run0000_0000 timepoint 0.2145
 finished parsing run0000_0000 timepoint 0.21455
 finished parsing run0000_0000 timepoint 0.2146
 finished parsing run0000_0000 timepoint 0.21465
 finished parsing run0000_0000 timepoint 0.2147
 finished parsing run0000_0000 timepoint 0.21475
 finished parsing run0000_0000 timepoint 0.2148
 finished parsing run0000_0000 timepoint 0.21485
 finished parsing run0000_0000 timepoint 0.2149
 finished parsing run0000_0000 timepoint 0.21495
 finished parsing run0000_0000 timepoint 0.215
 finished parsing run0000_0000 timepoint 0.21505
 finished parsing run0000_0000 timepoint 0.2151
 finished parsing run0000_0000 timepoint 0.21515
 finished parsing run0000_0000 timepoint 0.2152
 finished parsing run0000_0000 timepoint 0.21525
 finished parsing run0000_0000 timepoint 0.2153
 finished parsing run0000_0000 timepoint 0.21535
 finished parsing run0000_0000 timepoint 0.2154
 finished parsing run0000_0000 timepoint 0.21545
 finished parsing run0000_0000 timepoint 0.2155
 finished parsing run0000_0000 timepoint 0.21555
 finished parsing run0000_0000 timepoint 0.2156
 finished parsing run0000_0000 timepoint 0.21565
 finished parsing run0000_0000 timepoint 0.2157
 finished parsing run0000_0000 timepoint 0.21575
 finished parsing run0000_0000 timepoint 0.2158
 finished parsing run0000_0000 timepoint 0.21585
 finished parsing run0000_0000 timepoint 0.2159
 finished parsing run0000_0000 timepoint 0.21595
 finished parsing run0000_0000 timepoint 0.216
 finished parsing run0000_0000 timepoint 0.21605
 finished parsing run0000_0000 timepoint 0.2161
 finished parsing run0000_0000 timepoint 0.21615
 finished parsing run0000_0000 timepoint 0.2162
 finished parsing run0000_0000 timepoint 0.21625
 finished parsing run0000_0000 timepoint 0.2163
 finished parsing run0000_0000 timepoint 0.21635
 finished parsing run0000_0000 timepoint 0.2164
 finished parsing run0000_0000 timepoint 0.21645
 finished parsing run0000_0000 timepoint 0.2165
 finished parsing run0000_0000 timepoint 0.21655
 finished parsing run0000_0000 timepoint 0.2166
 finished parsing run0000_0000 timepoint 0.21665
 finished parsing run0000_0000 timepoint 0.2167
 finished parsing run0000_0000 timepoint 0.21675
 finished parsing run0000_0000 timepoint 0.2168
 finished parsing run0000_0000 timepoint 0.21685
 finished parsing run0000_0000 timepoint 0.2169
 finished parsing run0000_0000 timepoint 0.21695
 finished parsing run0000_0000 timepoint 0.217
 finished parsing run0000_0000 timepoint 0.21705
 finished parsing run0000_0000 timepoint 0.2171
 finished parsing run0000_0000 timepoint 0.21715
 finished parsing run0000_0000 timepoint 0.2172
 finished parsing run0000_0000 timepoint 0.21725
 finished parsing run0000_0000 timepoint 0.2173
 finished parsing run0000_0000 timepoint 0.21735
 finished parsing run0000_0000 timepoint 0.2174
 finished parsing run0000_0000 timepoint 0.21745
 finished parsing run0000_0000 timepoint 0.2175
 finished parsing run0000_0000 timepoint 0.21755
 finished parsing run0000_0000 timepoint 0.2176
 finished parsing run0000_0000 timepoint 0.21765
 finished parsing run0000_0000 timepoint 0.2177
 finished parsing run0000_0000 timepoint 0.21775
 finished parsing run0000_0000 timepoint 0.2178
 finished parsing run0000_0000 timepoint 0.21785
 finished parsing run0000_0000 timepoint 0.2179
 finished parsing run0000_0000 timepoint 0.21795
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.20475
 finished parsing run0000_0000 timepoint 0.2048
 finished parsing run0000_0000 timepoint 0.20485
 finished parsing run0000_0000 timepoint 0.2049
 finished parsing run0000_0000 timepoint 0.20495
 finished parsing run0000_0000 timepoint 0.205
 finished parsing run0000_0000 timepoint 0.20505
 finished parsing run0000_0000 timepoint 0.2051
 finished parsing run0000_0000 timepoint 0.20515
 finished parsing run0000_0000 timepoint 0.2052
 finished parsing run0000_0000 timepoint 0.20525
 finished parsing run0000_0000 timepoint 0.2053
 finished parsing run0000_0000 timepoint 0.20535
 finished parsing run0000_0000 timepoint 0.2054
 finished parsing run0000_0000 timepoint 0.20545
 finished parsing run0000_0000 timepoint 0.2055
 finished parsing run0000_0000 timepoint 0.20555
 finished parsing run0000_0000 timepoint 0.2056
 finished parsing run0000_0000 timepoint 0.20565
 finished parsing run0000_0000 timepoint 0.2057
 finished parsing run0000_0000 timepoint 0.20575
 finished parsing run0000_0000 timepoint 0.2058
 finished parsing run0000_0000 timepoint 0.20585
 finished parsing run0000_0000 timepoint 0.2059
 finished parsing run0000_0000 timepoint 0.20595
 finished parsing run0000_0000 timepoint 0.206
 finished parsing run0000_0000 timepoint 0.20605
 finished parsing run0000_0000 timepoint 0.2061
 finished parsing run0000_0000 timepoint 0.20615
 finished parsing run0000_0000 timepoint 0.2062
 finished parsing run0000_0000 timepoint 0.20625
 finished parsing run0000_0000 timepoint 0.2063
 finished parsing run0000_0000 timepoint 0.20635
 finished parsing run0000_0000 timepoint 0.2064
 finished parsing run0000_0000 timepoint 0.20645
 finished parsing run0000_0000 timepoint 0.2065
 finished parsing run0000_0000 timepoint 0.20655
 finished parsing run0000_0000 timepoint 0.2066
 finished parsing run0000_0000 timepoint 0.20665
 finished parsing run0000_0000 timepoint 0.2067
 finished parsing run0000_0000 timepoint 0.20675
 finished parsing run0000_0000 timepoint 0.2068
 finished parsing run0000_0000 timepoint 0.20685
 finished parsing run0000_0000 timepoint 0.2069
 finished parsing run0000_0000 timepoint 0.20695
 finished parsing run0000_0000 timepoint 0.207
 finished parsing run0000_0000 timepoint 0.20705
 finished parsing run0000_0000 timepoint 0.2071
 finished parsing run0000_0000 timepoint 0.20715
 finished parsing run0000_0000 timepoint 0.2072
 finished parsing run0000_0000 timepoint 0.20725
 finished parsing run0000_0000 timepoint 0.2073
 finished parsing run0000_0000 timepoint 0.20735
 finished parsing run0000_0000 timepoint 0.2074
 finished parsing run0000_0000 timepoint 0.20745
 finished parsing run0000_0000 timepoint 0.2075
 finished parsing run0000_0000 timepoint 0.20755
 finished parsing run0000_0000 timepoint 0.2076
 finished parsing run0000_0000 timepoint 0.20765
 finished parsing run0000_0000 timepoint 0.2077
 finished parsing run0000_0000 timepoint 0.20775
 finished parsing run0000_0000 timepoint 0.2078
 finished parsing run0000_0000 timepoint 0.20785
 finished parsing run0000_0000 timepoint 0.2079
 finished parsing run0000_0000 timepoint 0.20795
 finished parsing run0000_0000 timepoint 0.208
 finished parsing run0000_0000 timepoint 0.20805
 finished parsing run0000_0000 timepoint 0.2081
 finished parsing run0000_0000 timepoint 0.20815
 finished parsing run0000_0000 timepoint 0.2082
 finished parsing run0000_0000 timepoint 0.20825
 finished parsing run0000_0000 timepoint 0.2083
 finished parsing run0000_0000 timepoint 0.20835
 finished parsing run0000_0000 timepoint 0.2084
 finished parsing run0000_0000 timepoint 0.20845
 finished parsing run0000_0000 timepoint 0.2085
 finished parsing run0000_0000 timepoint 0.20855
 finished parsing run0000_0000 timepoint 0.2086
 finished parsing run0000_0000 timepoint 0.20865
 finished parsing run0000_0000 timepoint 0.2087
 finished parsing run0000_0000 timepoint 0.20875
 finished parsing run0000_0000 timepoint 0.2088
 finished parsing run0000_0000 timepoint 0.20885
 finished parsing run0000_0000 timepoint 0.2089
 finished parsing run0000_0000 timepoint 0.20895
 finished parsing run0000_0000 timepoint 0.209
 finished parsing run0000_0000 timepoint 0.20905
 finished parsing run0000_0000 timepoint 0.2091
 finished parsing run0000_0000 timepoint 0.20915
 finished parsing run0000_0000 timepoint 0.2092
 finished parsing run0000_0000 timepoint 0.20925
 finished parsing run0000_0000 timepoint 0.2093
 finished parsing run0000_0000 timepoint 0.20935
 finished parsing run0000_0000 timepoint 0.2094
 finished parsing run0000_0000 timepoint 0.20945
 finished parsing run0000_0000 timepoint 0.2095
 finished parsing run0000_0000 timepoint 0.20955
 finished parsing run0000_0000 timepoint 0.2096
 finished parsing run0000_0000 timepoint 0.20965
 finished parsing run0000_0000 timepoint 0.2097
 finished parsing run0000_0000 timepoint 0.20975
 finished parsing run0000_0000 timepoint 0.2098
 finished parsing run0000_0000 timepoint 0.20985
 finished parsing run0000_0000 timepoint 0.2099
 finished parsing run0000_0000 timepoint 0.20995
 finished parsing run0000_0000 timepoint 0.21
 finished parsing run0000_0000 timepoint 0.21005
 finished parsing run0000_0000 timepoint 0.2101
 finished parsing run0000_0000 timepoint 0.21015
 finished parsing run0000_0000 timepoint 0.2102
 finished parsing run0000_0000 timepoint 0.21025
 finished parsing run0000_0000 timepoint 0.2103
 finished parsing run0000_0000 timepoint 0.21035
 finished parsing run0000_0000 timepoint 0.2104
 finished parsing run0000_0000 timepoint 0.21045
 finished parsing run0000_0000 timepoint 0.2105
 finished parsing run0000_0000 timepoint 0.21055
 finished parsing run0000_0000 timepoint 0.2106
 finished parsing run0000_0000 timepoint 0.21065
 finished parsing run0000_0000 timepoint 0.2107
 finished parsing run0000_0000 timepoint 0.21075
 finished parsing run0000_0000 timepoint 0.2108
 finished parsing run0000_0000 timepoint 0.21085
 finished parsing run0000_0000 timepoint 0.2109
 finished parsing run0000_0000 timepoint 0.21095
 finished parsing run0000_0000 timepoint 0.211
 finished parsing run0000_0000 timepoint 0.21105
 finished parsing run0000_0000 timepoint 0.2111
 finished parsing run0000_0000 timepoint 0.21115
 finished parsing run0000_0000 timepoint 0.2112
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.1981
 finished parsing run0000_0000 timepoint 0.19815
 finished parsing run0000_0000 timepoint 0.1982
 finished parsing run0000_0000 timepoint 0.19825
 finished parsing run0000_0000 timepoint 0.1983
 finished parsing run0000_0000 timepoint 0.19835
 finished parsing run0000_0000 timepoint 0.1984
 finished parsing run0000_0000 timepoint 0.19845
 finished parsing run0000_0000 timepoint 0.1985
 finished parsing run0000_0000 timepoint 0.19855
 finished parsing run0000_0000 timepoint 0.1986
 finished parsing run0000_0000 timepoint 0.19865
 finished parsing run0000_0000 timepoint 0.1987
 finished parsing run0000_0000 timepoint 0.19875
 finished parsing run0000_0000 timepoint 0.1988
 finished parsing run0000_0000 timepoint 0.19885
 finished parsing run0000_0000 timepoint 0.1989
 finished parsing run0000_0000 timepoint 0.19895
 finished parsing run0000_0000 timepoint 0.199
 finished parsing run0000_0000 timepoint 0.19905
 finished parsing run0000_0000 timepoint 0.1991
 finished parsing run0000_0000 timepoint 0.19915
 finished parsing run0000_0000 timepoint 0.1992
 finished parsing run0000_0000 timepoint 0.19925
 finished parsing run0000_0000 timepoint 0.1993
 finished parsing run0000_0000 timepoint 0.19935
 finished parsing run0000_0000 timepoint 0.1994
 finished parsing run0000_0000 timepoint 0.19945
 finished parsing run0000_0000 timepoint 0.1995
 finished parsing run0000_0000 timepoint 0.19955
 finished parsing run0000_0000 timepoint 0.1996
 finished parsing run0000_0000 timepoint 0.19965
 finished parsing run0000_0000 timepoint 0.1997
 finished parsing run0000_0000 timepoint 0.19975
 finished parsing run0000_0000 timepoint 0.1998
 finished parsing run0000_0000 timepoint 0.19985
 finished parsing run0000_0000 timepoint 0.1999
 finished parsing run0000_0000 timepoint 0.19995
 finished parsing run0000_0000 timepoint 0.2
 finished parsing run0000_0000 timepoint 0.20005
 finished parsing run0000_0000 timepoint 0.2001
 finished parsing run0000_0000 timepoint 0.20015
 finished parsing run0000_0000 timepoint 0.2002
 finished parsing run0000_0000 timepoint 0.20025
 finished parsing run0000_0000 timepoint 0.2003
 finished parsing run0000_0000 timepoint 0.20035
 finished parsing run0000_0000 timepoint 0.2004
 finished parsing run0000_0000 timepoint 0.20045
 finished parsing run0000_0000 timepoint 0.2005
 finished parsing run0000_0000 timepoint 0.20055
 finished parsing run0000_0000 timepoint 0.2006
 finished parsing run0000_0000 timepoint 0.20065
 finished parsing run0000_0000 timepoint 0.2007
 finished parsing run0000_0000 timepoint 0.20075
 finished parsing run0000_0000 timepoint 0.2008
 finished parsing run0000_0000 timepoint 0.20085
 finished parsing run0000_0000 timepoint 0.2009
 finished parsing run0000_0000 timepoint 0.20095
 finished parsing run0000_0000 timepoint 0.201
 finished parsing run0000_0000 timepoint 0.20105
 finished parsing run0000_0000 timepoint 0.2011
 finished parsing run0000_0000 timepoint 0.20115
 finished parsing run0000_0000 timepoint 0.2012
 finished parsing run0000_0000 timepoint 0.20125
 finished parsing run0000_0000 timepoint 0.2013
 finished parsing run0000_0000 timepoint 0.20135
 finished parsing run0000_0000 timepoint 0.2014
 finished parsing run0000_0000 timepoint 0.20145
 finished parsing run0000_0000 timepoint 0.2015
 finished parsing run0000_0000 timepoint 0.20155
 finished parsing run0000_0000 timepoint 0.2016
 finished parsing run0000_0000 timepoint 0.20165
 finished parsing run0000_0000 timepoint 0.2017
 finished parsing run0000_0000 timepoint 0.20175
 finished parsing run0000_0000 timepoint 0.2018
 finished parsing run0000_0000 timepoint 0.20185
 finished parsing run0000_0000 timepoint 0.2019
 finished parsing run0000_0000 timepoint 0.20195
 finished parsing run0000_0000 timepoint 0.202
 finished parsing run0000_0000 timepoint 0.20205
 finished parsing run0000_0000 timepoint 0.2021
 finished parsing run0000_0000 timepoint 0.20215
 finished parsing run0000_0000 timepoint 0.2022
 finished parsing run0000_0000 timepoint 0.20225
 finished parsing run0000_0000 timepoint 0.2023
 finished parsing run0000_0000 timepoint 0.20235
 finished parsing run0000_0000 timepoint 0.2024
 finished parsing run0000_0000 timepoint 0.20245
 finished parsing run0000_0000 timepoint 0.2025
 finished parsing run0000_0000 timepoint 0.20255
 finished parsing run0000_0000 timepoint 0.2026
 finished parsing run0000_0000 timepoint 0.20265
 finished parsing run0000_0000 timepoint 0.2027
 finished parsing run0000_0000 timepoint 0.20275
 finished parsing run0000_0000 timepoint 0.2028
 finished parsing run0000_0000 timepoint 0.20285
 finished parsing run0000_0000 timepoint 0.2029
 finished parsing run0000_0000 timepoint 0.20295
 finished parsing run0000_0000 timepoint 0.203
 finished parsing run0000_0000 timepoint 0.20305
 finished parsing run0000_0000 timepoint 0.2031
 finished parsing run0000_0000 timepoint 0.20315
 finished parsing run0000_0000 timepoint 0.2032
 finished parsing run0000_0000 timepoint 0.20325
 finished parsing run0000_0000 timepoint 0.2033
 finished parsing run0000_0000 timepoint 0.20335
 finished parsing run0000_0000 timepoint 0.2034
 finished parsing run0000_0000 timepoint 0.20345
 finished parsing run0000_0000 timepoint 0.2035
 finished parsing run0000_0000 timepoint 0.20355
 finished parsing run0000_0000 timepoint 0.2036
 finished parsing run0000_0000 timepoint 0.20365
 finished parsing run0000_0000 timepoint 0.2037
 finished parsing run0000_0000 timepoint 0.20375
 finished parsing run0000_0000 timepoint 0.2038
 finished parsing run0000_0000 timepoint 0.20385
 finished parsing run0000_0000 timepoint 0.2039
 finished parsing run0000_0000 timepoint 0.20395
 finished parsing run0000_0000 timepoint 0.204
 finished parsing run0000_0000 timepoint 0.20405
 finished parsing run0000_0000 timepoint 0.2041
 finished parsing run0000_0000 timepoint 0.20415
 finished parsing run0000_0000 timepoint 0.2042
 finished parsing run0000_0000 timepoint 0.20425
 finished parsing run0000_0000 timepoint 0.2043
 finished parsing run0000_0000 timepoint 0.20435
 finished parsing run0000_0000 timepoint 0.2044
 finished parsing run0000_0000 timepoint 0.20445
 finished parsing run0000_0000 timepoint 0.2045
 finished parsing run0000_0000 timepoint 0.20455
 finished parsing run0000_0000 timepoint 0.2046
 finished parsing run0000_0000 timepoint 0.20465
 finished parsing run0000_0000 timepoint 0.2047
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.1915
 finished parsing run0000_0000 timepoint 0.19155
 finished parsing run0000_0000 timepoint 0.1916
 finished parsing run0000_0000 timepoint 0.19165
 finished parsing run0000_0000 timepoint 0.1917
 finished parsing run0000_0000 timepoint 0.19175
 finished parsing run0000_0000 timepoint 0.1918
 finished parsing run0000_0000 timepoint 0.19185
 finished parsing run0000_0000 timepoint 0.1919
 finished parsing run0000_0000 timepoint 0.19195
 finished parsing run0000_0000 timepoint 0.192
 finished parsing run0000_0000 timepoint 0.19205
 finished parsing run0000_0000 timepoint 0.1921
 finished parsing run0000_0000 timepoint 0.19215
 finished parsing run0000_0000 timepoint 0.1922
 finished parsing run0000_0000 timepoint 0.19225
 finished parsing run0000_0000 timepoint 0.1923
 finished parsing run0000_0000 timepoint 0.19235
 finished parsing run0000_0000 timepoint 0.1924
 finished parsing run0000_0000 timepoint 0.19245
 finished parsing run0000_0000 timepoint 0.1925
 finished parsing run0000_0000 timepoint 0.19255
 finished parsing run0000_0000 timepoint 0.1926
 finished parsing run0000_0000 timepoint 0.19265
 finished parsing run0000_0000 timepoint 0.1927
 finished parsing run0000_0000 timepoint 0.19275
 finished parsing run0000_0000 timepoint 0.1928
 finished parsing run0000_0000 timepoint 0.19285
 finished parsing run0000_0000 timepoint 0.1929
 finished parsing run0000_0000 timepoint 0.19295
 finished parsing run0000_0000 timepoint 0.193
 finished parsing run0000_0000 timepoint 0.19305
 finished parsing run0000_0000 timepoint 0.1931
 finished parsing run0000_0000 timepoint 0.19315
 finished parsing run0000_0000 timepoint 0.1932
 finished parsing run0000_0000 timepoint 0.19325
 finished parsing run0000_0000 timepoint 0.1933
 finished parsing run0000_0000 timepoint 0.19335
 finished parsing run0000_0000 timepoint 0.1934
 finished parsing run0000_0000 timepoint 0.19345
 finished parsing run0000_0000 timepoint 0.1935
 finished parsing run0000_0000 timepoint 0.19355
 finished parsing run0000_0000 timepoint 0.1936
 finished parsing run0000_0000 timepoint 0.19365
 finished parsing run0000_0000 timepoint 0.1937
 finished parsing run0000_0000 timepoint 0.19375
 finished parsing run0000_0000 timepoint 0.1938
 finished parsing run0000_0000 timepoint 0.19385
 finished parsing run0000_0000 timepoint 0.1939
 finished parsing run0000_0000 timepoint 0.19395
 finished parsing run0000_0000 timepoint 0.194
 finished parsing run0000_0000 timepoint 0.19405
 finished parsing run0000_0000 timepoint 0.1941
 finished parsing run0000_0000 timepoint 0.19415
 finished parsing run0000_0000 timepoint 0.1942
 finished parsing run0000_0000 timepoint 0.19425
 finished parsing run0000_0000 timepoint 0.1943
 finished parsing run0000_0000 timepoint 0.19435
 finished parsing run0000_0000 timepoint 0.1944
 finished parsing run0000_0000 timepoint 0.19445
 finished parsing run0000_0000 timepoint 0.1945
 finished parsing run0000_0000 timepoint 0.19455
 finished parsing run0000_0000 timepoint 0.1946
 finished parsing run0000_0000 timepoint 0.19465
 finished parsing run0000_0000 timepoint 0.1947
 finished parsing run0000_0000 timepoint 0.19475
 finished parsing run0000_0000 timepoint 0.1948
 finished parsing run0000_0000 timepoint 0.19485
 finished parsing run0000_0000 timepoint 0.1949
 finished parsing run0000_0000 timepoint 0.19495
 finished parsing run0000_0000 timepoint 0.195
 finished parsing run0000_0000 timepoint 0.19505
 finished parsing run0000_0000 timepoint 0.1951
 finished parsing run0000_0000 timepoint 0.19515
 finished parsing run0000_0000 timepoint 0.1952
 finished parsing run0000_0000 timepoint 0.19525
 finished parsing run0000_0000 timepoint 0.1953
 finished parsing run0000_0000 timepoint 0.19535
 finished parsing run0000_0000 timepoint 0.1954
 finished parsing run0000_0000 timepoint 0.19545
 finished parsing run0000_0000 timepoint 0.1955
 finished parsing run0000_0000 timepoint 0.19555
 finished parsing run0000_0000 timepoint 0.1956
 finished parsing run0000_0000 timepoint 0.19565
 finished parsing run0000_0000 timepoint 0.1957
 finished parsing run0000_0000 timepoint 0.19575
 finished parsing run0000_0000 timepoint 0.1958
 finished parsing run0000_0000 timepoint 0.19585
 finished parsing run0000_0000 timepoint 0.1959
 finished parsing run0000_0000 timepoint 0.19595
 finished parsing run0000_0000 timepoint 0.196
 finished parsing run0000_0000 timepoint 0.19605
 finished parsing run0000_0000 timepoint 0.1961
 finished parsing run0000_0000 timepoint 0.19615
 finished parsing run0000_0000 timepoint 0.1962
 finished parsing run0000_0000 timepoint 0.19625
 finished parsing run0000_0000 timepoint 0.1963
 finished parsing run0000_0000 timepoint 0.19635
 finished parsing run0000_0000 timepoint 0.1964
 finished parsing run0000_0000 timepoint 0.19645
 finished parsing run0000_0000 timepoint 0.1965
 finished parsing run0000_0000 timepoint 0.19655
 finished parsing run0000_0000 timepoint 0.1966
 finished parsing run0000_0000 timepoint 0.19665
 finished parsing run0000_0000 timepoint 0.1967
 finished parsing run0000_0000 timepoint 0.19675
 finished parsing run0000_0000 timepoint 0.1968
 finished parsing run0000_0000 timepoint 0.19685
 finished parsing run0000_0000 timepoint 0.1969
 finished parsing run0000_0000 timepoint 0.19695
 finished parsing run0000_0000 timepoint 0.197
 finished parsing run0000_0000 timepoint 0.19705
 finished parsing run0000_0000 timepoint 0.1971
 finished parsing run0000_0000 timepoint 0.19715
 finished parsing run0000_0000 timepoint 0.1972
 finished parsing run0000_0000 timepoint 0.19725
 finished parsing run0000_0000 timepoint 0.1973
 finished parsing run0000_0000 timepoint 0.19735
 finished parsing run0000_0000 timepoint 0.1974
 finished parsing run0000_0000 timepoint 0.19745
 finished parsing run0000_0000 timepoint 0.1975
 finished parsing run0000_0000 timepoint 0.19755
 finished parsing run0000_0000 timepoint 0.1976
 finished parsing run0000_0000 timepoint 0.19765
 finished parsing run0000_0000 timepoint 0.1977
 finished parsing run0000_0000 timepoint 0.19775
 finished parsing run0000_0000 timepoint 0.1978
 finished parsing run0000_0000 timepoint 0.19785
 finished parsing run0000_0000 timepoint 0.1979
 finished parsing run0000_0000 timepoint 0.19795
 finished parsing run0000_0000 timepoint 0.198
 finished parsing run0000_0000 timepoint 0.19805
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.18475
 finished parsing run0000_0000 timepoint 0.1848
 finished parsing run0000_0000 timepoint 0.18485
 finished parsing run0000_0000 timepoint 0.1849
 finished parsing run0000_0000 timepoint 0.18495
 finished parsing run0000_0000 timepoint 0.185
 finished parsing run0000_0000 timepoint 0.18505
 finished parsing run0000_0000 timepoint 0.1851
 finished parsing run0000_0000 timepoint 0.18515
 finished parsing run0000_0000 timepoint 0.1852
 finished parsing run0000_0000 timepoint 0.18525
 finished parsing run0000_0000 timepoint 0.1853
 finished parsing run0000_0000 timepoint 0.18535
 finished parsing run0000_0000 timepoint 0.1854
 finished parsing run0000_0000 timepoint 0.18545
 finished parsing run0000_0000 timepoint 0.1855
 finished parsing run0000_0000 timepoint 0.18555
 finished parsing run0000_0000 timepoint 0.1856
 finished parsing run0000_0000 timepoint 0.18565
 finished parsing run0000_0000 timepoint 0.1857
 finished parsing run0000_0000 timepoint 0.18575
 finished parsing run0000_0000 timepoint 0.1858
 finished parsing run0000_0000 timepoint 0.18585
 finished parsing run0000_0000 timepoint 0.1859
 finished parsing run0000_0000 timepoint 0.18595
 finished parsing run0000_0000 timepoint 0.186
 finished parsing run0000_0000 timepoint 0.18605
 finished parsing run0000_0000 timepoint 0.1861
 finished parsing run0000_0000 timepoint 0.18615
 finished parsing run0000_0000 timepoint 0.1862
 finished parsing run0000_0000 timepoint 0.18625
 finished parsing run0000_0000 timepoint 0.1863
 finished parsing run0000_0000 timepoint 0.18635
 finished parsing run0000_0000 timepoint 0.1864
 finished parsing run0000_0000 timepoint 0.18645
 finished parsing run0000_0000 timepoint 0.1865
 finished parsing run0000_0000 timepoint 0.18655
 finished parsing run0000_0000 timepoint 0.1866
 finished parsing run0000_0000 timepoint 0.18665
 finished parsing run0000_0000 timepoint 0.1867
 finished parsing run0000_0000 timepoint 0.18675
 finished parsing run0000_0000 timepoint 0.1868
 finished parsing run0000_0000 timepoint 0.18685
 finished parsing run0000_0000 timepoint 0.1869
 finished parsing run0000_0000 timepoint 0.18695
 finished parsing run0000_0000 timepoint 0.187
 finished parsing run0000_0000 timepoint 0.18705
 finished parsing run0000_0000 timepoint 0.1871
 finished parsing run0000_0000 timepoint 0.18715
 finished parsing run0000_0000 timepoint 0.1872
 finished parsing run0000_0000 timepoint 0.18725
 finished parsing run0000_0000 timepoint 0.1873
 finished parsing run0000_0000 timepoint 0.18735
 finished parsing run0000_0000 timepoint 0.1874
 finished parsing run0000_0000 timepoint 0.18745
 finished parsing run0000_0000 timepoint 0.1875
 finished parsing run0000_0000 timepoint 0.18755
 finished parsing run0000_0000 timepoint 0.1876
 finished parsing run0000_0000 timepoint 0.18765
 finished parsing run0000_0000 timepoint 0.1877
 finished parsing run0000_0000 timepoint 0.18775
 finished parsing run0000_0000 timepoint 0.1878
 finished parsing run0000_0000 timepoint 0.18785
 finished parsing run0000_0000 timepoint 0.1879
 finished parsing run0000_0000 timepoint 0.18795
 finished parsing run0000_0000 timepoint 0.188
 finished parsing run0000_0000 timepoint 0.18805
 finished parsing run0000_0000 timepoint 0.1881
 finished parsing run0000_0000 timepoint 0.18815
 finished parsing run0000_0000 timepoint 0.1882
 finished parsing run0000_0000 timepoint 0.18825
 finished parsing run0000_0000 timepoint 0.1883
 finished parsing run0000_0000 timepoint 0.18835
 finished parsing run0000_0000 timepoint 0.1884
 finished parsing run0000_0000 timepoint 0.18845
 finished parsing run0000_0000 timepoint 0.1885
 finished parsing run0000_0000 timepoint 0.18855
 finished parsing run0000_0000 timepoint 0.1886
 finished parsing run0000_0000 timepoint 0.18865
 finished parsing run0000_0000 timepoint 0.1887
 finished parsing run0000_0000 timepoint 0.18875
 finished parsing run0000_0000 timepoint 0.1888
 finished parsing run0000_0000 timepoint 0.18885
 finished parsing run0000_0000 timepoint 0.1889
 finished parsing run0000_0000 timepoint 0.18895
 finished parsing run0000_0000 timepoint 0.189
 finished parsing run0000_0000 timepoint 0.18905
 finished parsing run0000_0000 timepoint 0.1891
 finished parsing run0000_0000 timepoint 0.18915
 finished parsing run0000_0000 timepoint 0.1892
 finished parsing run0000_0000 timepoint 0.18925
 finished parsing run0000_0000 timepoint 0.1893
 finished parsing run0000_0000 timepoint 0.18935
 finished parsing run0000_0000 timepoint 0.1894
 finished parsing run0000_0000 timepoint 0.18945
 finished parsing run0000_0000 timepoint 0.1895
 finished parsing run0000_0000 timepoint 0.18955
 finished parsing run0000_0000 timepoint 0.1896
 finished parsing run0000_0000 timepoint 0.18965
 finished parsing run0000_0000 timepoint 0.1897
 finished parsing run0000_0000 timepoint 0.18975
 finished parsing run0000_0000 timepoint 0.1898
 finished parsing run0000_0000 timepoint 0.18985
 finished parsing run0000_0000 timepoint 0.1899
 finished parsing run0000_0000 timepoint 0.18995
 finished parsing run0000_0000 timepoint 0.19
 finished parsing run0000_0000 timepoint 0.19005
 finished parsing run0000_0000 timepoint 0.1901
 finished parsing run0000_0000 timepoint 0.19015
 finished parsing run0000_0000 timepoint 0.1902
 finished parsing run0000_0000 timepoint 0.19025
 finished parsing run0000_0000 timepoint 0.1903
 finished parsing run0000_0000 timepoint 0.19035
 finished parsing run0000_0000 timepoint 0.1904
 finished parsing run0000_0000 timepoint 0.19045
 finished parsing run0000_0000 timepoint 0.1905
 finished parsing run0000_0000 timepoint 0.19055
 finished parsing run0000_0000 timepoint 0.1906
 finished parsing run0000_0000 timepoint 0.19065
 finished parsing run0000_0000 timepoint 0.1907
 finished parsing run0000_0000 timepoint 0.19075
 finished parsing run0000_0000 timepoint 0.1908
 finished parsing run0000_0000 timepoint 0.19085
 finished parsing run0000_0000 timepoint 0.1909
 finished parsing run0000_0000 timepoint 0.19095
 finished parsing run0000_0000 timepoint 0.191
 finished parsing run0000_0000 timepoint 0.19105
 finished parsing run0000_0000 timepoint 0.1911
 finished parsing run0000_0000 timepoint 0.19115
 finished parsing run0000_0000 timepoint 0.1912
 finished parsing run0000_0000 timepoint 0.19125
 finished parsing run0000_0000 timepoint 0.1913
 finished parsing run0000_0000 timepoint 0.19135
 finished parsing run0000_0000 timepoint 0.1914
 finished parsing run0000_0000 timepoint 0.19145
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.1783
 finished parsing run0000_0000 timepoint 0.17835
 finished parsing run0000_0000 timepoint 0.1784
 finished parsing run0000_0000 timepoint 0.17845
 finished parsing run0000_0000 timepoint 0.1785
 finished parsing run0000_0000 timepoint 0.17855
 finished parsing run0000_0000 timepoint 0.1786
 finished parsing run0000_0000 timepoint 0.17865
 finished parsing run0000_0000 timepoint 0.1787
 finished parsing run0000_0000 timepoint 0.17875
 finished parsing run0000_0000 timepoint 0.1788
 finished parsing run0000_0000 timepoint 0.17885
 finished parsing run0000_0000 timepoint 0.1789
 finished parsing run0000_0000 timepoint 0.17895
 finished parsing run0000_0000 timepoint 0.179
 finished parsing run0000_0000 timepoint 0.17905
 finished parsing run0000_0000 timepoint 0.1791
 finished parsing run0000_0000 timepoint 0.17915
 finished parsing run0000_0000 timepoint 0.1792
 finished parsing run0000_0000 timepoint 0.17925
 finished parsing run0000_0000 timepoint 0.1793
 finished parsing run0000_0000 timepoint 0.17935
 finished parsing run0000_0000 timepoint 0.1794
 finished parsing run0000_0000 timepoint 0.17945
 finished parsing run0000_0000 timepoint 0.1795
 finished parsing run0000_0000 timepoint 0.17955
 finished parsing run0000_0000 timepoint 0.1796
 finished parsing run0000_0000 timepoint 0.17965
 finished parsing run0000_0000 timepoint 0.1797
 finished parsing run0000_0000 timepoint 0.17975
 finished parsing run0000_0000 timepoint 0.1798
 finished parsing run0000_0000 timepoint 0.17985
 finished parsing run0000_0000 timepoint 0.1799
 finished parsing run0000_0000 timepoint 0.17995
 finished parsing run0000_0000 timepoint 0.18
 finished parsing run0000_0000 timepoint 0.18005
 finished parsing run0000_0000 timepoint 0.1801
 finished parsing run0000_0000 timepoint 0.18015
 finished parsing run0000_0000 timepoint 0.1802
 finished parsing run0000_0000 timepoint 0.18025
 finished parsing run0000_0000 timepoint 0.1803
 finished parsing run0000_0000 timepoint 0.18035
 finished parsing run0000_0000 timepoint 0.1804
 finished parsing run0000_0000 timepoint 0.18045
 finished parsing run0000_0000 timepoint 0.1805
 finished parsing run0000_0000 timepoint 0.18055
 finished parsing run0000_0000 timepoint 0.1806
 finished parsing run0000_0000 timepoint 0.18065
 finished parsing run0000_0000 timepoint 0.1807
 finished parsing run0000_0000 timepoint 0.18075
 finished parsing run0000_0000 timepoint 0.1808
 finished parsing run0000_0000 timepoint 0.18085
 finished parsing run0000_0000 timepoint 0.1809
 finished parsing run0000_0000 timepoint 0.18095
 finished parsing run0000_0000 timepoint 0.181
 finished parsing run0000_0000 timepoint 0.18105
 finished parsing run0000_0000 timepoint 0.1811
 finished parsing run0000_0000 timepoint 0.18115
 finished parsing run0000_0000 timepoint 0.1812
 finished parsing run0000_0000 timepoint 0.18125
 finished parsing run0000_0000 timepoint 0.1813
 finished parsing run0000_0000 timepoint 0.18135
 finished parsing run0000_0000 timepoint 0.1814
 finished parsing run0000_0000 timepoint 0.18145
 finished parsing run0000_0000 timepoint 0.1815
 finished parsing run0000_0000 timepoint 0.18155
 finished parsing run0000_0000 timepoint 0.1816
 finished parsing run0000_0000 timepoint 0.18165
 finished parsing run0000_0000 timepoint 0.1817
 finished parsing run0000_0000 timepoint 0.18175
 finished parsing run0000_0000 timepoint 0.1818
 finished parsing run0000_0000 timepoint 0.18185
 finished parsing run0000_0000 timepoint 0.1819
 finished parsing run0000_0000 timepoint 0.18195
 finished parsing run0000_0000 timepoint 0.182
 finished parsing run0000_0000 timepoint 0.18205
 finished parsing run0000_0000 timepoint 0.1821
 finished parsing run0000_0000 timepoint 0.18215
 finished parsing run0000_0000 timepoint 0.1822
 finished parsing run0000_0000 timepoint 0.18225
 finished parsing run0000_0000 timepoint 0.1823
 finished parsing run0000_0000 timepoint 0.18235
 finished parsing run0000_0000 timepoint 0.1824
 finished parsing run0000_0000 timepoint 0.18245
 finished parsing run0000_0000 timepoint 0.1825
 finished parsing run0000_0000 timepoint 0.18255
 finished parsing run0000_0000 timepoint 0.1826
 finished parsing run0000_0000 timepoint 0.18265
 finished parsing run0000_0000 timepoint 0.1827
 finished parsing run0000_0000 timepoint 0.18275
 finished parsing run0000_0000 timepoint 0.1828
 finished parsing run0000_0000 timepoint 0.18285
 finished parsing run0000_0000 timepoint 0.1829
 finished parsing run0000_0000 timepoint 0.18295
 finished parsing run0000_0000 timepoint 0.183
 finished parsing run0000_0000 timepoint 0.18305
 finished parsing run0000_0000 timepoint 0.1831
 finished parsing run0000_0000 timepoint 0.18315
 finished parsing run0000_0000 timepoint 0.1832
 finished parsing run0000_0000 timepoint 0.18325
 finished parsing run0000_0000 timepoint 0.1833
 finished parsing run0000_0000 timepoint 0.18335
 finished parsing run0000_0000 timepoint 0.1834
 finished parsing run0000_0000 timepoint 0.18345
 finished parsing run0000_0000 timepoint 0.1835
 finished parsing run0000_0000 timepoint 0.18355
 finished parsing run0000_0000 timepoint 0.1836
 finished parsing run0000_0000 timepoint 0.18365
 finished parsing run0000_0000 timepoint 0.1837
 finished parsing run0000_0000 timepoint 0.18375
 finished parsing run0000_0000 timepoint 0.1838
 finished parsing run0000_0000 timepoint 0.18385
 finished parsing run0000_0000 timepoint 0.1839
 finished parsing run0000_0000 timepoint 0.18395
 finished parsing run0000_0000 timepoint 0.184
 finished parsing run0000_0000 timepoint 0.18405
 finished parsing run0000_0000 timepoint 0.1841
 finished parsing run0000_0000 timepoint 0.18415
 finished parsing run0000_0000 timepoint 0.1842
 finished parsing run0000_0000 timepoint 0.18425
 finished parsing run0000_0000 timepoint 0.1843
 finished parsing run0000_0000 timepoint 0.18435
 finished parsing run0000_0000 timepoint 0.1844
 finished parsing run0000_0000 timepoint 0.18445
 finished parsing run0000_0000 timepoint 0.1845
 finished parsing run0000_0000 timepoint 0.18455
 finished parsing run0000_0000 timepoint 0.1846
 finished parsing run0000_0000 timepoint 0.18465
 finished parsing run0000_0000 timepoint 0.1847
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.17165
 finished parsing run0000_0000 timepoint 0.1717
 finished parsing run0000_0000 timepoint 0.17175
 finished parsing run0000_0000 timepoint 0.1718
 finished parsing run0000_0000 timepoint 0.17185
 finished parsing run0000_0000 timepoint 0.1719
 finished parsing run0000_0000 timepoint 0.17195
 finished parsing run0000_0000 timepoint 0.172
 finished parsing run0000_0000 timepoint 0.17205
 finished parsing run0000_0000 timepoint 0.1721
 finished parsing run0000_0000 timepoint 0.17215
 finished parsing run0000_0000 timepoint 0.1722
 finished parsing run0000_0000 timepoint 0.17225
 finished parsing run0000_0000 timepoint 0.1723
 finished parsing run0000_0000 timepoint 0.17235
 finished parsing run0000_0000 timepoint 0.1724
 finished parsing run0000_0000 timepoint 0.17245
 finished parsing run0000_0000 timepoint 0.1725
 finished parsing run0000_0000 timepoint 0.17255
 finished parsing run0000_0000 timepoint 0.1726
 finished parsing run0000_0000 timepoint 0.17265
 finished parsing run0000_0000 timepoint 0.1727
 finished parsing run0000_0000 timepoint 0.17275
 finished parsing run0000_0000 timepoint 0.1728
 finished parsing run0000_0000 timepoint 0.17285
 finished parsing run0000_0000 timepoint 0.1729
 finished parsing run0000_0000 timepoint 0.17295
 finished parsing run0000_0000 timepoint 0.173
 finished parsing run0000_0000 timepoint 0.17305
 finished parsing run0000_0000 timepoint 0.1731
 finished parsing run0000_0000 timepoint 0.17315
 finished parsing run0000_0000 timepoint 0.1732
 finished parsing run0000_0000 timepoint 0.17325
 finished parsing run0000_0000 timepoint 0.1733
 finished parsing run0000_0000 timepoint 0.17335
 finished parsing run0000_0000 timepoint 0.1734
 finished parsing run0000_0000 timepoint 0.17345
 finished parsing run0000_0000 timepoint 0.1735
 finished parsing run0000_0000 timepoint 0.17355
 finished parsing run0000_0000 timepoint 0.1736
 finished parsing run0000_0000 timepoint 0.17365
 finished parsing run0000_0000 timepoint 0.1737
 finished parsing run0000_0000 timepoint 0.17375
 finished parsing run0000_0000 timepoint 0.1738
 finished parsing run0000_0000 timepoint 0.17385
 finished parsing run0000_0000 timepoint 0.1739
 finished parsing run0000_0000 timepoint 0.17395
 finished parsing run0000_0000 timepoint 0.174
 finished parsing run0000_0000 timepoint 0.17405
 finished parsing run0000_0000 timepoint 0.1741
 finished parsing run0000_0000 timepoint 0.17415
 finished parsing run0000_0000 timepoint 0.1742
 finished parsing run0000_0000 timepoint 0.17425
 finished parsing run0000_0000 timepoint 0.1743
 finished parsing run0000_0000 timepoint 0.17435
 finished parsing run0000_0000 timepoint 0.1744
 finished parsing run0000_0000 timepoint 0.17445
 finished parsing run0000_0000 timepoint 0.1745
 finished parsing run0000_0000 timepoint 0.17455
 finished parsing run0000_0000 timepoint 0.1746
 finished parsing run0000_0000 timepoint 0.17465
 finished parsing run0000_0000 timepoint 0.1747
 finished parsing run0000_0000 timepoint 0.17475
 finished parsing run0000_0000 timepoint 0.1748
 finished parsing run0000_0000 timepoint 0.17485
 finished parsing run0000_0000 timepoint 0.1749
 finished parsing run0000_0000 timepoint 0.17495
 finished parsing run0000_0000 timepoint 0.175
 finished parsing run0000_0000 timepoint 0.17505
 finished parsing run0000_0000 timepoint 0.1751
 finished parsing run0000_0000 timepoint 0.17515
 finished parsing run0000_0000 timepoint 0.1752
 finished parsing run0000_0000 timepoint 0.17525
 finished parsing run0000_0000 timepoint 0.1753
 finished parsing run0000_0000 timepoint 0.17535
 finished parsing run0000_0000 timepoint 0.1754
 finished parsing run0000_0000 timepoint 0.17545
 finished parsing run0000_0000 timepoint 0.1755
 finished parsing run0000_0000 timepoint 0.17555
 finished parsing run0000_0000 timepoint 0.1756
 finished parsing run0000_0000 timepoint 0.17565
 finished parsing run0000_0000 timepoint 0.1757
 finished parsing run0000_0000 timepoint 0.17575
 finished parsing run0000_0000 timepoint 0.1758
 finished parsing run0000_0000 timepoint 0.17585
 finished parsing run0000_0000 timepoint 0.1759
 finished parsing run0000_0000 timepoint 0.17595
 finished parsing run0000_0000 timepoint 0.176
 finished parsing run0000_0000 timepoint 0.17605
 finished parsing run0000_0000 timepoint 0.1761
 finished parsing run0000_0000 timepoint 0.17615
 finished parsing run0000_0000 timepoint 0.1762
 finished parsing run0000_0000 timepoint 0.17625
 finished parsing run0000_0000 timepoint 0.1763
 finished parsing run0000_0000 timepoint 0.17635
 finished parsing run0000_0000 timepoint 0.1764
 finished parsing run0000_0000 timepoint 0.17645
 finished parsing run0000_0000 timepoint 0.1765
 finished parsing run0000_0000 timepoint 0.17655
 finished parsing run0000_0000 timepoint 0.1766
 finished parsing run0000_0000 timepoint 0.17665
 finished parsing run0000_0000 timepoint 0.1767
 finished parsing run0000_0000 timepoint 0.17675
 finished parsing run0000_0000 timepoint 0.1768
 finished parsing run0000_0000 timepoint 0.17685
 finished parsing run0000_0000 timepoint 0.1769
 finished parsing run0000_0000 timepoint 0.17695
 finished parsing run0000_0000 timepoint 0.177
 finished parsing run0000_0000 timepoint 0.17705
 finished parsing run0000_0000 timepoint 0.1771
 finished parsing run0000_0000 timepoint 0.17715
 finished parsing run0000_0000 timepoint 0.1772
 finished parsing run0000_0000 timepoint 0.17725
 finished parsing run0000_0000 timepoint 0.1773
 finished parsing run0000_0000 timepoint 0.17735
 finished parsing run0000_0000 timepoint 0.1774
 finished parsing run0000_0000 timepoint 0.17745
 finished parsing run0000_0000 timepoint 0.1775
 finished parsing run0000_0000 timepoint 0.17755
 finished parsing run0000_0000 timepoint 0.1776
 finished parsing run0000_0000 timepoint 0.17765
 finished parsing run0000_0000 timepoint 0.1777
 finished parsing run0000_0000 timepoint 0.17775
 finished parsing run0000_0000 timepoint 0.1778
 finished parsing run0000_0000 timepoint 0.17785
 finished parsing run0000_0000 timepoint 0.1779
 finished parsing run0000_0000 timepoint 0.17795
 finished parsing run0000_0000 timepoint 0.178
 finished parsing run0000_0000 timepoint 0.17805
 finished parsing run0000_0000 timepoint 0.1781
 finished parsing run0000_0000 timepoint 0.17815
 finished parsing run0000_0000 timepoint 0.1782
 finished parsing run0000_0000 timepoint 0.17825
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.16505
 finished parsing run0000_0000 timepoint 0.1651
 finished parsing run0000_0000 timepoint 0.16515
 finished parsing run0000_0000 timepoint 0.1652
 finished parsing run0000_0000 timepoint 0.16525
 finished parsing run0000_0000 timepoint 0.1653
 finished parsing run0000_0000 timepoint 0.16535
 finished parsing run0000_0000 timepoint 0.1654
 finished parsing run0000_0000 timepoint 0.16545
 finished parsing run0000_0000 timepoint 0.1655
 finished parsing run0000_0000 timepoint 0.16555
 finished parsing run0000_0000 timepoint 0.1656
 finished parsing run0000_0000 timepoint 0.16565
 finished parsing run0000_0000 timepoint 0.1657
 finished parsing run0000_0000 timepoint 0.16575
 finished parsing run0000_0000 timepoint 0.1658
 finished parsing run0000_0000 timepoint 0.16585
 finished parsing run0000_0000 timepoint 0.1659
 finished parsing run0000_0000 timepoint 0.16595
 finished parsing run0000_0000 timepoint 0.166
 finished parsing run0000_0000 timepoint 0.16605
 finished parsing run0000_0000 timepoint 0.1661
 finished parsing run0000_0000 timepoint 0.16615
 finished parsing run0000_0000 timepoint 0.1662
 finished parsing run0000_0000 timepoint 0.16625
 finished parsing run0000_0000 timepoint 0.1663
 finished parsing run0000_0000 timepoint 0.16635
 finished parsing run0000_0000 timepoint 0.1664
 finished parsing run0000_0000 timepoint 0.16645
 finished parsing run0000_0000 timepoint 0.1665
 finished parsing run0000_0000 timepoint 0.16655
 finished parsing run0000_0000 timepoint 0.1666
 finished parsing run0000_0000 timepoint 0.16665
 finished parsing run0000_0000 timepoint 0.1667
 finished parsing run0000_0000 timepoint 0.16675
 finished parsing run0000_0000 timepoint 0.1668
 finished parsing run0000_0000 timepoint 0.16685
 finished parsing run0000_0000 timepoint 0.1669
 finished parsing run0000_0000 timepoint 0.16695
 finished parsing run0000_0000 timepoint 0.167
 finished parsing run0000_0000 timepoint 0.16705
 finished parsing run0000_0000 timepoint 0.1671
 finished parsing run0000_0000 timepoint 0.16715
 finished parsing run0000_0000 timepoint 0.1672
 finished parsing run0000_0000 timepoint 0.16725
 finished parsing run0000_0000 timepoint 0.1673
 finished parsing run0000_0000 timepoint 0.16735
 finished parsing run0000_0000 timepoint 0.1674
 finished parsing run0000_0000 timepoint 0.16745
 finished parsing run0000_0000 timepoint 0.1675
 finished parsing run0000_0000 timepoint 0.16755
 finished parsing run0000_0000 timepoint 0.1676
 finished parsing run0000_0000 timepoint 0.16765
 finished parsing run0000_0000 timepoint 0.1677
 finished parsing run0000_0000 timepoint 0.16775
 finished parsing run0000_0000 timepoint 0.1678
 finished parsing run0000_0000 timepoint 0.16785
 finished parsing run0000_0000 timepoint 0.1679
 finished parsing run0000_0000 timepoint 0.16795
 finished parsing run0000_0000 timepoint 0.168
 finished parsing run0000_0000 timepoint 0.16805
 finished parsing run0000_0000 timepoint 0.1681
 finished parsing run0000_0000 timepoint 0.16815
 finished parsing run0000_0000 timepoint 0.1682
 finished parsing run0000_0000 timepoint 0.16825
 finished parsing run0000_0000 timepoint 0.1683
 finished parsing run0000_0000 timepoint 0.16835
 finished parsing run0000_0000 timepoint 0.1684
 finished parsing run0000_0000 timepoint 0.16845
 finished parsing run0000_0000 timepoint 0.1685
 finished parsing run0000_0000 timepoint 0.16855
 finished parsing run0000_0000 timepoint 0.1686
 finished parsing run0000_0000 timepoint 0.16865
 finished parsing run0000_0000 timepoint 0.1687
 finished parsing run0000_0000 timepoint 0.16875
 finished parsing run0000_0000 timepoint 0.1688
 finished parsing run0000_0000 timepoint 0.16885
 finished parsing run0000_0000 timepoint 0.1689
 finished parsing run0000_0000 timepoint 0.16895
 finished parsing run0000_0000 timepoint 0.169
 finished parsing run0000_0000 timepoint 0.16905
 finished parsing run0000_0000 timepoint 0.1691
 finished parsing run0000_0000 timepoint 0.16915
 finished parsing run0000_0000 timepoint 0.1692
 finished parsing run0000_0000 timepoint 0.16925
 finished parsing run0000_0000 timepoint 0.1693
 finished parsing run0000_0000 timepoint 0.16935
 finished parsing run0000_0000 timepoint 0.1694
 finished parsing run0000_0000 timepoint 0.16945
 finished parsing run0000_0000 timepoint 0.1695
 finished parsing run0000_0000 timepoint 0.16955
 finished parsing run0000_0000 timepoint 0.1696
 finished parsing run0000_0000 timepoint 0.16965
 finished parsing run0000_0000 timepoint 0.1697
 finished parsing run0000_0000 timepoint 0.16975
 finished parsing run0000_0000 timepoint 0.1698
 finished parsing run0000_0000 timepoint 0.16985
 finished parsing run0000_0000 timepoint 0.1699
 finished parsing run0000_0000 timepoint 0.16995
 finished parsing run0000_0000 timepoint 0.17
 finished parsing run0000_0000 timepoint 0.17005
 finished parsing run0000_0000 timepoint 0.1701
 finished parsing run0000_0000 timepoint 0.17015
 finished parsing run0000_0000 timepoint 0.1702
 finished parsing run0000_0000 timepoint 0.17025
 finished parsing run0000_0000 timepoint 0.1703
 finished parsing run0000_0000 timepoint 0.17035
 finished parsing run0000_0000 timepoint 0.1704
 finished parsing run0000_0000 timepoint 0.17045
 finished parsing run0000_0000 timepoint 0.1705
 finished parsing run0000_0000 timepoint 0.17055
 finished parsing run0000_0000 timepoint 0.1706
 finished parsing run0000_0000 timepoint 0.17065
 finished parsing run0000_0000 timepoint 0.1707
 finished parsing run0000_0000 timepoint 0.17075
 finished parsing run0000_0000 timepoint 0.1708
 finished parsing run0000_0000 timepoint 0.17085
 finished parsing run0000_0000 timepoint 0.1709
 finished parsing run0000_0000 timepoint 0.17095
 finished parsing run0000_0000 timepoint 0.171
 finished parsing run0000_0000 timepoint 0.17105
 finished parsing run0000_0000 timepoint 0.1711
 finished parsing run0000_0000 timepoint 0.17115
 finished parsing run0000_0000 timepoint 0.1712
 finished parsing run0000_0000 timepoint 0.17125
 finished parsing run0000_0000 timepoint 0.1713
 finished parsing run0000_0000 timepoint 0.17135
 finished parsing run0000_0000 timepoint 0.1714
 finished parsing run0000_0000 timepoint 0.17145
 finished parsing run0000_0000 timepoint 0.1715
 finished parsing run0000_0000 timepoint 0.17155
 finished parsing run0000_0000 timepoint 0.1716
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.15845
 finished parsing run0000_0000 timepoint 0.1585
 finished parsing run0000_0000 timepoint 0.15855
 finished parsing run0000_0000 timepoint 0.1586
 finished parsing run0000_0000 timepoint 0.15865
 finished parsing run0000_0000 timepoint 0.1587
 finished parsing run0000_0000 timepoint 0.15875
 finished parsing run0000_0000 timepoint 0.1588
 finished parsing run0000_0000 timepoint 0.15885
 finished parsing run0000_0000 timepoint 0.1589
 finished parsing run0000_0000 timepoint 0.15895
 finished parsing run0000_0000 timepoint 0.159
 finished parsing run0000_0000 timepoint 0.15905
 finished parsing run0000_0000 timepoint 0.1591
 finished parsing run0000_0000 timepoint 0.15915
 finished parsing run0000_0000 timepoint 0.1592
 finished parsing run0000_0000 timepoint 0.15925
 finished parsing run0000_0000 timepoint 0.1593
 finished parsing run0000_0000 timepoint 0.15935
 finished parsing run0000_0000 timepoint 0.1594
 finished parsing run0000_0000 timepoint 0.15945
 finished parsing run0000_0000 timepoint 0.1595
 finished parsing run0000_0000 timepoint 0.15955
 finished parsing run0000_0000 timepoint 0.1596
 finished parsing run0000_0000 timepoint 0.15965
 finished parsing run0000_0000 timepoint 0.1597
 finished parsing run0000_0000 timepoint 0.15975
 finished parsing run0000_0000 timepoint 0.1598
 finished parsing run0000_0000 timepoint 0.15985
 finished parsing run0000_0000 timepoint 0.1599
 finished parsing run0000_0000 timepoint 0.15995
 finished parsing run0000_0000 timepoint 0.16
 finished parsing run0000_0000 timepoint 0.16005
 finished parsing run0000_0000 timepoint 0.1601
 finished parsing run0000_0000 timepoint 0.16015
 finished parsing run0000_0000 timepoint 0.1602
 finished parsing run0000_0000 timepoint 0.16025
 finished parsing run0000_0000 timepoint 0.1603
 finished parsing run0000_0000 timepoint 0.16035
 finished parsing run0000_0000 timepoint 0.1604
 finished parsing run0000_0000 timepoint 0.16045
 finished parsing run0000_0000 timepoint 0.1605
 finished parsing run0000_0000 timepoint 0.16055
 finished parsing run0000_0000 timepoint 0.1606
 finished parsing run0000_0000 timepoint 0.16065
 finished parsing run0000_0000 timepoint 0.1607
 finished parsing run0000_0000 timepoint 0.16075
 finished parsing run0000_0000 timepoint 0.1608
 finished parsing run0000_0000 timepoint 0.16085
 finished parsing run0000_0000 timepoint 0.1609
 finished parsing run0000_0000 timepoint 0.16095
 finished parsing run0000_0000 timepoint 0.161
 finished parsing run0000_0000 timepoint 0.16105
 finished parsing run0000_0000 timepoint 0.1611
 finished parsing run0000_0000 timepoint 0.16115
 finished parsing run0000_0000 timepoint 0.1612
 finished parsing run0000_0000 timepoint 0.16125
 finished parsing run0000_0000 timepoint 0.1613
 finished parsing run0000_0000 timepoint 0.16135
 finished parsing run0000_0000 timepoint 0.1614
 finished parsing run0000_0000 timepoint 0.16145
 finished parsing run0000_0000 timepoint 0.1615
 finished parsing run0000_0000 timepoint 0.16155
 finished parsing run0000_0000 timepoint 0.1616
 finished parsing run0000_0000 timepoint 0.16165
 finished parsing run0000_0000 timepoint 0.1617
 finished parsing run0000_0000 timepoint 0.16175
 finished parsing run0000_0000 timepoint 0.1618
 finished parsing run0000_0000 timepoint 0.16185
 finished parsing run0000_0000 timepoint 0.1619
 finished parsing run0000_0000 timepoint 0.16195
 finished parsing run0000_0000 timepoint 0.162
 finished parsing run0000_0000 timepoint 0.16205
 finished parsing run0000_0000 timepoint 0.1621
 finished parsing run0000_0000 timepoint 0.16215
 finished parsing run0000_0000 timepoint 0.1622
 finished parsing run0000_0000 timepoint 0.16225
 finished parsing run0000_0000 timepoint 0.1623
 finished parsing run0000_0000 timepoint 0.16235
 finished parsing run0000_0000 timepoint 0.1624
 finished parsing run0000_0000 timepoint 0.16245
 finished parsing run0000_0000 timepoint 0.1625
 finished parsing run0000_0000 timepoint 0.16255
 finished parsing run0000_0000 timepoint 0.1626
 finished parsing run0000_0000 timepoint 0.16265
 finished parsing run0000_0000 timepoint 0.1627
 finished parsing run0000_0000 timepoint 0.16275
 finished parsing run0000_0000 timepoint 0.1628
 finished parsing run0000_0000 timepoint 0.16285
 finished parsing run0000_0000 timepoint 0.1629
 finished parsing run0000_0000 timepoint 0.16295
 finished parsing run0000_0000 timepoint 0.163
 finished parsing run0000_0000 timepoint 0.16305
 finished parsing run0000_0000 timepoint 0.1631
 finished parsing run0000_0000 timepoint 0.16315
 finished parsing run0000_0000 timepoint 0.1632
 finished parsing run0000_0000 timepoint 0.16325
 finished parsing run0000_0000 timepoint 0.1633
 finished parsing run0000_0000 timepoint 0.16335
 finished parsing run0000_0000 timepoint 0.1634
 finished parsing run0000_0000 timepoint 0.16345
 finished parsing run0000_0000 timepoint 0.1635
 finished parsing run0000_0000 timepoint 0.16355
 finished parsing run0000_0000 timepoint 0.1636
 finished parsing run0000_0000 timepoint 0.16365
 finished parsing run0000_0000 timepoint 0.1637
 finished parsing run0000_0000 timepoint 0.16375
 finished parsing run0000_0000 timepoint 0.1638
 finished parsing run0000_0000 timepoint 0.16385
 finished parsing run0000_0000 timepoint 0.1639
 finished parsing run0000_0000 timepoint 0.16395
 finished parsing run0000_0000 timepoint 0.164
 finished parsing run0000_0000 timepoint 0.16405
 finished parsing run0000_0000 timepoint 0.1641
 finished parsing run0000_0000 timepoint 0.16415
 finished parsing run0000_0000 timepoint 0.1642
 finished parsing run0000_0000 timepoint 0.16425
 finished parsing run0000_0000 timepoint 0.1643
 finished parsing run0000_0000 timepoint 0.16435
 finished parsing run0000_0000 timepoint 0.1644
 finished parsing run0000_0000 timepoint 0.16445
 finished parsing run0000_0000 timepoint 0.1645
 finished parsing run0000_0000 timepoint 0.16455
 finished parsing run0000_0000 timepoint 0.1646
 finished parsing run0000_0000 timepoint 0.16465
 finished parsing run0000_0000 timepoint 0.1647
 finished parsing run0000_0000 timepoint 0.16475
 finished parsing run0000_0000 timepoint 0.1648
 finished parsing run0000_0000 timepoint 0.16485
 finished parsing run0000_0000 timepoint 0.1649
 finished parsing run0000_0000 timepoint 0.16495
 finished parsing run0000_0000 timepoint 0.165
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.15185
 finished parsing run0000_0000 timepoint 0.1519
 finished parsing run0000_0000 timepoint 0.15195
 finished parsing run0000_0000 timepoint 0.152
 finished parsing run0000_0000 timepoint 0.15205
 finished parsing run0000_0000 timepoint 0.1521
 finished parsing run0000_0000 timepoint 0.15215
 finished parsing run0000_0000 timepoint 0.1522
 finished parsing run0000_0000 timepoint 0.15225
 finished parsing run0000_0000 timepoint 0.1523
 finished parsing run0000_0000 timepoint 0.15235
 finished parsing run0000_0000 timepoint 0.1524
 finished parsing run0000_0000 timepoint 0.15245
 finished parsing run0000_0000 timepoint 0.1525
 finished parsing run0000_0000 timepoint 0.15255
 finished parsing run0000_0000 timepoint 0.1526
 finished parsing run0000_0000 timepoint 0.15265
 finished parsing run0000_0000 timepoint 0.1527
 finished parsing run0000_0000 timepoint 0.15275
 finished parsing run0000_0000 timepoint 0.1528
 finished parsing run0000_0000 timepoint 0.15285
 finished parsing run0000_0000 timepoint 0.1529
 finished parsing run0000_0000 timepoint 0.15295
 finished parsing run0000_0000 timepoint 0.153
 finished parsing run0000_0000 timepoint 0.15305
 finished parsing run0000_0000 timepoint 0.1531
 finished parsing run0000_0000 timepoint 0.15315
 finished parsing run0000_0000 timepoint 0.1532
 finished parsing run0000_0000 timepoint 0.15325
 finished parsing run0000_0000 timepoint 0.1533
 finished parsing run0000_0000 timepoint 0.15335
 finished parsing run0000_0000 timepoint 0.1534
 finished parsing run0000_0000 timepoint 0.15345
 finished parsing run0000_0000 timepoint 0.1535
 finished parsing run0000_0000 timepoint 0.15355
 finished parsing run0000_0000 timepoint 0.1536
 finished parsing run0000_0000 timepoint 0.15365
 finished parsing run0000_0000 timepoint 0.1537
 finished parsing run0000_0000 timepoint 0.15375
 finished parsing run0000_0000 timepoint 0.1538
 finished parsing run0000_0000 timepoint 0.15385
 finished parsing run0000_0000 timepoint 0.1539
 finished parsing run0000_0000 timepoint 0.15395
 finished parsing run0000_0000 timepoint 0.154
 finished parsing run0000_0000 timepoint 0.15405
 finished parsing run0000_0000 timepoint 0.1541
 finished parsing run0000_0000 timepoint 0.15415
 finished parsing run0000_0000 timepoint 0.1542
 finished parsing run0000_0000 timepoint 0.15425
 finished parsing run0000_0000 timepoint 0.1543
 finished parsing run0000_0000 timepoint 0.15435
 finished parsing run0000_0000 timepoint 0.1544
 finished parsing run0000_0000 timepoint 0.15445
 finished parsing run0000_0000 timepoint 0.1545
 finished parsing run0000_0000 timepoint 0.15455
 finished parsing run0000_0000 timepoint 0.1546
 finished parsing run0000_0000 timepoint 0.15465
 finished parsing run0000_0000 timepoint 0.1547
 finished parsing run0000_0000 timepoint 0.15475
 finished parsing run0000_0000 timepoint 0.1548
 finished parsing run0000_0000 timepoint 0.15485
 finished parsing run0000_0000 timepoint 0.1549
 finished parsing run0000_0000 timepoint 0.15495
 finished parsing run0000_0000 timepoint 0.155
 finished parsing run0000_0000 timepoint 0.15505
 finished parsing run0000_0000 timepoint 0.1551
 finished parsing run0000_0000 timepoint 0.15515
 finished parsing run0000_0000 timepoint 0.1552
 finished parsing run0000_0000 timepoint 0.15525
 finished parsing run0000_0000 timepoint 0.1553
 finished parsing run0000_0000 timepoint 0.15535
 finished parsing run0000_0000 timepoint 0.1554
 finished parsing run0000_0000 timepoint 0.15545
 finished parsing run0000_0000 timepoint 0.1555
 finished parsing run0000_0000 timepoint 0.15555
 finished parsing run0000_0000 timepoint 0.1556
 finished parsing run0000_0000 timepoint 0.15565
 finished parsing run0000_0000 timepoint 0.1557
 finished parsing run0000_0000 timepoint 0.15575
 finished parsing run0000_0000 timepoint 0.1558
 finished parsing run0000_0000 timepoint 0.15585
 finished parsing run0000_0000 timepoint 0.1559
 finished parsing run0000_0000 timepoint 0.15595
 finished parsing run0000_0000 timepoint 0.156
 finished parsing run0000_0000 timepoint 0.15605
 finished parsing run0000_0000 timepoint 0.1561
 finished parsing run0000_0000 timepoint 0.15615
 finished parsing run0000_0000 timepoint 0.1562
 finished parsing run0000_0000 timepoint 0.15625
 finished parsing run0000_0000 timepoint 0.1563
 finished parsing run0000_0000 timepoint 0.15635
 finished parsing run0000_0000 timepoint 0.1564
 finished parsing run0000_0000 timepoint 0.15645
 finished parsing run0000_0000 timepoint 0.1565
 finished parsing run0000_0000 timepoint 0.15655
 finished parsing run0000_0000 timepoint 0.1566
 finished parsing run0000_0000 timepoint 0.15665
 finished parsing run0000_0000 timepoint 0.1567
 finished parsing run0000_0000 timepoint 0.15675
 finished parsing run0000_0000 timepoint 0.1568
 finished parsing run0000_0000 timepoint 0.15685
 finished parsing run0000_0000 timepoint 0.1569
 finished parsing run0000_0000 timepoint 0.15695
 finished parsing run0000_0000 timepoint 0.157
 finished parsing run0000_0000 timepoint 0.15705
 finished parsing run0000_0000 timepoint 0.1571
 finished parsing run0000_0000 timepoint 0.15715
 finished parsing run0000_0000 timepoint 0.1572
 finished parsing run0000_0000 timepoint 0.15725
 finished parsing run0000_0000 timepoint 0.1573
 finished parsing run0000_0000 timepoint 0.15735
 finished parsing run0000_0000 timepoint 0.1574
 finished parsing run0000_0000 timepoint 0.15745
 finished parsing run0000_0000 timepoint 0.1575
 finished parsing run0000_0000 timepoint 0.15755
 finished parsing run0000_0000 timepoint 0.1576
 finished parsing run0000_0000 timepoint 0.15765
 finished parsing run0000_0000 timepoint 0.1577
 finished parsing run0000_0000 timepoint 0.15775
 finished parsing run0000_0000 timepoint 0.1578
 finished parsing run0000_0000 timepoint 0.15785
 finished parsing run0000_0000 timepoint 0.1579
 finished parsing run0000_0000 timepoint 0.15795
 finished parsing run0000_0000 timepoint 0.158
 finished parsing run0000_0000 timepoint 0.15805
 finished parsing run0000_0000 timepoint 0.1581
 finished parsing run0000_0000 timepoint 0.15815
 finished parsing run0000_0000 timepoint 0.1582
 finished parsing run0000_0000 timepoint 0.15825
 finished parsing run0000_0000 timepoint 0.1583
 finished parsing run0000_0000 timepoint 0.15835
 finished parsing run0000_0000 timepoint 0.1584
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.1453
 finished parsing run0000_0000 timepoint 0.14535
 finished parsing run0000_0000 timepoint 0.1454
 finished parsing run0000_0000 timepoint 0.14545
 finished parsing run0000_0000 timepoint 0.1455
 finished parsing run0000_0000 timepoint 0.14555
 finished parsing run0000_0000 timepoint 0.1456
 finished parsing run0000_0000 timepoint 0.14565
 finished parsing run0000_0000 timepoint 0.1457
 finished parsing run0000_0000 timepoint 0.14575
 finished parsing run0000_0000 timepoint 0.1458
 finished parsing run0000_0000 timepoint 0.14585
 finished parsing run0000_0000 timepoint 0.1459
 finished parsing run0000_0000 timepoint 0.14595
 finished parsing run0000_0000 timepoint 0.146
 finished parsing run0000_0000 timepoint 0.14605
 finished parsing run0000_0000 timepoint 0.1461
 finished parsing run0000_0000 timepoint 0.14615
 finished parsing run0000_0000 timepoint 0.1462
 finished parsing run0000_0000 timepoint 0.14625
 finished parsing run0000_0000 timepoint 0.1463
 finished parsing run0000_0000 timepoint 0.14635
 finished parsing run0000_0000 timepoint 0.1464
 finished parsing run0000_0000 timepoint 0.14645
 finished parsing run0000_0000 timepoint 0.1465
 finished parsing run0000_0000 timepoint 0.14655
 finished parsing run0000_0000 timepoint 0.1466
 finished parsing run0000_0000 timepoint 0.14665
 finished parsing run0000_0000 timepoint 0.1467
 finished parsing run0000_0000 timepoint 0.14675
 finished parsing run0000_0000 timepoint 0.1468
 finished parsing run0000_0000 timepoint 0.14685
 finished parsing run0000_0000 timepoint 0.1469
 finished parsing run0000_0000 timepoint 0.14695
 finished parsing run0000_0000 timepoint 0.147
 finished parsing run0000_0000 timepoint 0.14705
 finished parsing run0000_0000 timepoint 0.1471
 finished parsing run0000_0000 timepoint 0.14715
 finished parsing run0000_0000 timepoint 0.1472
 finished parsing run0000_0000 timepoint 0.14725
 finished parsing run0000_0000 timepoint 0.1473
 finished parsing run0000_0000 timepoint 0.14735
 finished parsing run0000_0000 timepoint 0.1474
 finished parsing run0000_0000 timepoint 0.14745
 finished parsing run0000_0000 timepoint 0.1475
 finished parsing run0000_0000 timepoint 0.14755
 finished parsing run0000_0000 timepoint 0.1476
 finished parsing run0000_0000 timepoint 0.14765
 finished parsing run0000_0000 timepoint 0.1477
 finished parsing run0000_0000 timepoint 0.14775
 finished parsing run0000_0000 timepoint 0.1478
 finished parsing run0000_0000 timepoint 0.14785
 finished parsing run0000_0000 timepoint 0.1479
 finished parsing run0000_0000 timepoint 0.14795
 finished parsing run0000_0000 timepoint 0.148
 finished parsing run0000_0000 timepoint 0.14805
 finished parsing run0000_0000 timepoint 0.1481
 finished parsing run0000_0000 timepoint 0.14815
 finished parsing run0000_0000 timepoint 0.1482
 finished parsing run0000_0000 timepoint 0.14825
 finished parsing run0000_0000 timepoint 0.1483
 finished parsing run0000_0000 timepoint 0.14835
 finished parsing run0000_0000 timepoint 0.1484
 finished parsing run0000_0000 timepoint 0.14845
 finished parsing run0000_0000 timepoint 0.1485
 finished parsing run0000_0000 timepoint 0.14855
 finished parsing run0000_0000 timepoint 0.1486
 finished parsing run0000_0000 timepoint 0.14865
 finished parsing run0000_0000 timepoint 0.1487
 finished parsing run0000_0000 timepoint 0.14875
 finished parsing run0000_0000 timepoint 0.1488
 finished parsing run0000_0000 timepoint 0.14885
 finished parsing run0000_0000 timepoint 0.1489
 finished parsing run0000_0000 timepoint 0.14895
 finished parsing run0000_0000 timepoint 0.149
 finished parsing run0000_0000 timepoint 0.14905
 finished parsing run0000_0000 timepoint 0.1491
 finished parsing run0000_0000 timepoint 0.14915
 finished parsing run0000_0000 timepoint 0.1492
 finished parsing run0000_0000 timepoint 0.14925
 finished parsing run0000_0000 timepoint 0.1493
 finished parsing run0000_0000 timepoint 0.14935
 finished parsing run0000_0000 timepoint 0.1494
 finished parsing run0000_0000 timepoint 0.14945
 finished parsing run0000_0000 timepoint 0.1495
 finished parsing run0000_0000 timepoint 0.14955
 finished parsing run0000_0000 timepoint 0.1496
 finished parsing run0000_0000 timepoint 0.14965
 finished parsing run0000_0000 timepoint 0.1497
 finished parsing run0000_0000 timepoint 0.14975
 finished parsing run0000_0000 timepoint 0.1498
 finished parsing run0000_0000 timepoint 0.14985
 finished parsing run0000_0000 timepoint 0.1499
 finished parsing run0000_0000 timepoint 0.14995
 finished parsing run0000_0000 timepoint 0.15
 finished parsing run0000_0000 timepoint 0.15005
 finished parsing run0000_0000 timepoint 0.1501
 finished parsing run0000_0000 timepoint 0.15015
 finished parsing run0000_0000 timepoint 0.1502
 finished parsing run0000_0000 timepoint 0.15025
 finished parsing run0000_0000 timepoint 0.1503
 finished parsing run0000_0000 timepoint 0.15035
 finished parsing run0000_0000 timepoint 0.1504
 finished parsing run0000_0000 timepoint 0.15045
 finished parsing run0000_0000 timepoint 0.1505
 finished parsing run0000_0000 timepoint 0.15055
 finished parsing run0000_0000 timepoint 0.1506
 finished parsing run0000_0000 timepoint 0.15065
 finished parsing run0000_0000 timepoint 0.1507
 finished parsing run0000_0000 timepoint 0.15075
 finished parsing run0000_0000 timepoint 0.1508
 finished parsing run0000_0000 timepoint 0.15085
 finished parsing run0000_0000 timepoint 0.1509
 finished parsing run0000_0000 timepoint 0.15095
 finished parsing run0000_0000 timepoint 0.151
 finished parsing run0000_0000 timepoint 0.15105
 finished parsing run0000_0000 timepoint 0.1511
 finished parsing run0000_0000 timepoint 0.15115
 finished parsing run0000_0000 timepoint 0.1512
 finished parsing run0000_0000 timepoint 0.15125
 finished parsing run0000_0000 timepoint 0.1513
 finished parsing run0000_0000 timepoint 0.15135
 finished parsing run0000_0000 timepoint 0.1514
 finished parsing run0000_0000 timepoint 0.15145
 finished parsing run0000_0000 timepoint 0.1515
 finished parsing run0000_0000 timepoint 0.15155
 finished parsing run0000_0000 timepoint 0.1516
 finished parsing run0000_0000 timepoint 0.15165
 finished parsing run0000_0000 timepoint 0.1517
 finished parsing run0000_0000 timepoint 0.15175
 finished parsing run0000_0000 timepoint 0.1518
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.13835
 finished parsing run0000_0000 timepoint 0.1384
 finished parsing run0000_0000 timepoint 0.13845
 finished parsing run0000_0000 timepoint 0.1385
 finished parsing run0000_0000 timepoint 0.13855
 finished parsing run0000_0000 timepoint 0.1386
 finished parsing run0000_0000 timepoint 0.13865
 finished parsing run0000_0000 timepoint 0.1387
 finished parsing run0000_0000 timepoint 0.13875
 finished parsing run0000_0000 timepoint 0.1388
 finished parsing run0000_0000 timepoint 0.13885
 finished parsing run0000_0000 timepoint 0.1389
 finished parsing run0000_0000 timepoint 0.13895
 finished parsing run0000_0000 timepoint 0.139
 finished parsing run0000_0000 timepoint 0.13905
 finished parsing run0000_0000 timepoint 0.1391
 finished parsing run0000_0000 timepoint 0.13915
 finished parsing run0000_0000 timepoint 0.1392
 finished parsing run0000_0000 timepoint 0.13925
 finished parsing run0000_0000 timepoint 0.1393
 finished parsing run0000_0000 timepoint 0.13935
 finished parsing run0000_0000 timepoint 0.1394
 finished parsing run0000_0000 timepoint 0.13945
 finished parsing run0000_0000 timepoint 0.1395
 finished parsing run0000_0000 timepoint 0.13955
 finished parsing run0000_0000 timepoint 0.1396
 finished parsing run0000_0000 timepoint 0.13965
 finished parsing run0000_0000 timepoint 0.1397
 finished parsing run0000_0000 timepoint 0.13975
 finished parsing run0000_0000 timepoint 0.1398
 finished parsing run0000_0000 timepoint 0.13985
 finished parsing run0000_0000 timepoint 0.1399
 finished parsing run0000_0000 timepoint 0.13995
 finished parsing run0000_0000 timepoint 0.14
 finished parsing run0000_0000 timepoint 0.14005
 finished parsing run0000_0000 timepoint 0.1401
 finished parsing run0000_0000 timepoint 0.14015
 finished parsing run0000_0000 timepoint 0.1402
 finished parsing run0000_0000 timepoint 0.14025
 finished parsing run0000_0000 timepoint 0.1403
 finished parsing run0000_0000 timepoint 0.14035
 finished parsing run0000_0000 timepoint 0.1404
 finished parsing run0000_0000 timepoint 0.14045
 finished parsing run0000_0000 timepoint 0.1405
 finished parsing run0000_0000 timepoint 0.14055
 finished parsing run0000_0000 timepoint 0.1406
 finished parsing run0000_0000 timepoint 0.14065
 finished parsing run0000_0000 timepoint 0.1407
 finished parsing run0000_0000 timepoint 0.14075
 finished parsing run0000_0000 timepoint 0.1408
 finished parsing run0000_0000 timepoint 0.14085
 finished parsing run0000_0000 timepoint 0.1409
 finished parsing run0000_0000 timepoint 0.14095
 finished parsing run0000_0000 timepoint 0.141
 finished parsing run0000_0000 timepoint 0.14105
 finished parsing run0000_0000 timepoint 0.1411
 finished parsing run0000_0000 timepoint 0.14115
 finished parsing run0000_0000 timepoint 0.1412
 finished parsing run0000_0000 timepoint 0.14125
 finished parsing run0000_0000 timepoint 0.1413
 finished parsing run0000_0000 timepoint 0.14135
 finished parsing run0000_0000 timepoint 0.1414
 finished parsing run0000_0000 timepoint 0.14145
 finished parsing run0000_0000 timepoint 0.1415
 finished parsing run0000_0000 timepoint 0.14155
 finished parsing run0000_0000 timepoint 0.1416
 finished parsing run0000_0000 timepoint 0.14165
 finished parsing run0000_0000 timepoint 0.1417
 finished parsing run0000_0000 timepoint 0.14175
 finished parsing run0000_0000 timepoint 0.1418
 finished parsing run0000_0000 timepoint 0.14185
 finished parsing run0000_0000 timepoint 0.1419
 finished parsing run0000_0000 timepoint 0.14195
 finished parsing run0000_0000 timepoint 0.142
 finished parsing run0000_0000 timepoint 0.14205
 finished parsing run0000_0000 timepoint 0.1421
 finished parsing run0000_0000 timepoint 0.14215
 finished parsing run0000_0000 timepoint 0.1422
 finished parsing run0000_0000 timepoint 0.14225
 finished parsing run0000_0000 timepoint 0.1423
 finished parsing run0000_0000 timepoint 0.14235
 finished parsing run0000_0000 timepoint 0.1424
 finished parsing run0000_0000 timepoint 0.14245
 finished parsing run0000_0000 timepoint 0.1425
 finished parsing run0000_0000 timepoint 0.14255
 finished parsing run0000_0000 timepoint 0.1426
 finished parsing run0000_0000 timepoint 0.14265
 finished parsing run0000_0000 timepoint 0.1427
 finished parsing run0000_0000 timepoint 0.14275
 finished parsing run0000_0000 timepoint 0.1428
 finished parsing run0000_0000 timepoint 0.14285
 finished parsing run0000_0000 timepoint 0.1429
 finished parsing run0000_0000 timepoint 0.14295
 finished parsing run0000_0000 timepoint 0.143
 finished parsing run0000_0000 timepoint 0.14305
 finished parsing run0000_0000 timepoint 0.1431
 finished parsing run0000_0000 timepoint 0.14315
 finished parsing run0000_0000 timepoint 0.1432
 finished parsing run0000_0000 timepoint 0.14325
 finished parsing run0000_0000 timepoint 0.1433
 finished parsing run0000_0000 timepoint 0.14335
 finished parsing run0000_0000 timepoint 0.1434
 finished parsing run0000_0000 timepoint 0.14345
 finished parsing run0000_0000 timepoint 0.1435
 finished parsing run0000_0000 timepoint 0.14355
 finished parsing run0000_0000 timepoint 0.1436
 finished parsing run0000_0000 timepoint 0.14365
 finished parsing run0000_0000 timepoint 0.1437
 finished parsing run0000_0000 timepoint 0.14375
 finished parsing run0000_0000 timepoint 0.1438
 finished parsing run0000_0000 timepoint 0.14385
 finished parsing run0000_0000 timepoint 0.1439
 finished parsing run0000_0000 timepoint 0.14395
 finished parsing run0000_0000 timepoint 0.144
 finished parsing run0000_0000 timepoint 0.14405
 finished parsing run0000_0000 timepoint 0.1441
 finished parsing run0000_0000 timepoint 0.14415
 finished parsing run0000_0000 timepoint 0.1442
 finished parsing run0000_0000 timepoint 0.14425
 finished parsing run0000_0000 timepoint 0.1443
 finished parsing run0000_0000 timepoint 0.14435
 finished parsing run0000_0000 timepoint 0.1444
 finished parsing run0000_0000 timepoint 0.14445
 finished parsing run0000_0000 timepoint 0.1445
 finished parsing run0000_0000 timepoint 0.14455
 finished parsing run0000_0000 timepoint 0.1446
 finished parsing run0000_0000 timepoint 0.14465
 finished parsing run0000_0000 timepoint 0.1447
 finished parsing run0000_0000 timepoint 0.14475
 finished parsing run0000_0000 timepoint 0.1448
 finished parsing run0000_0000 timepoint 0.14485
 finished parsing run0000_0000 timepoint 0.1449
 finished parsing run0000_0000 timepoint 0.14495
 finished parsing run0000_0000 timepoint 0.145
 finished parsing run0000_0000 timepoint 0.14505
 finished parsing run0000_0000 timepoint 0.1451
 finished parsing run0000_0000 timepoint 0.14515
 finished parsing run0000_0000 timepoint 0.1452
 finished parsing run0000_0000 timepoint 0.14525
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.13165
 finished parsing run0000_0000 timepoint 0.1317
 finished parsing run0000_0000 timepoint 0.13175
 finished parsing run0000_0000 timepoint 0.1318
 finished parsing run0000_0000 timepoint 0.13185
 finished parsing run0000_0000 timepoint 0.1319
 finished parsing run0000_0000 timepoint 0.13195
 finished parsing run0000_0000 timepoint 0.132
 finished parsing run0000_0000 timepoint 0.13205
 finished parsing run0000_0000 timepoint 0.1321
 finished parsing run0000_0000 timepoint 0.13215
 finished parsing run0000_0000 timepoint 0.1322
 finished parsing run0000_0000 timepoint 0.13225
 finished parsing run0000_0000 timepoint 0.1323
 finished parsing run0000_0000 timepoint 0.13235
 finished parsing run0000_0000 timepoint 0.1324
 finished parsing run0000_0000 timepoint 0.13245
 finished parsing run0000_0000 timepoint 0.1325
 finished parsing run0000_0000 timepoint 0.13255
 finished parsing run0000_0000 timepoint 0.1326
 finished parsing run0000_0000 timepoint 0.13265
 finished parsing run0000_0000 timepoint 0.1327
 finished parsing run0000_0000 timepoint 0.13275
 finished parsing run0000_0000 timepoint 0.1328
 finished parsing run0000_0000 timepoint 0.13285
 finished parsing run0000_0000 timepoint 0.1329
 finished parsing run0000_0000 timepoint 0.13295
 finished parsing run0000_0000 timepoint 0.133
 finished parsing run0000_0000 timepoint 0.13305
 finished parsing run0000_0000 timepoint 0.1331
 finished parsing run0000_0000 timepoint 0.13315
 finished parsing run0000_0000 timepoint 0.1332
 finished parsing run0000_0000 timepoint 0.13325
 finished parsing run0000_0000 timepoint 0.1333
 finished parsing run0000_0000 timepoint 0.13335
 finished parsing run0000_0000 timepoint 0.1334
 finished parsing run0000_0000 timepoint 0.13345
 finished parsing run0000_0000 timepoint 0.1335
 finished parsing run0000_0000 timepoint 0.13355
 finished parsing run0000_0000 timepoint 0.1336
 finished parsing run0000_0000 timepoint 0.13365
 finished parsing run0000_0000 timepoint 0.1337
 finished parsing run0000_0000 timepoint 0.13375
 finished parsing run0000_0000 timepoint 0.1338
 finished parsing run0000_0000 timepoint 0.13385
 finished parsing run0000_0000 timepoint 0.1339
 finished parsing run0000_0000 timepoint 0.13395
 finished parsing run0000_0000 timepoint 0.134
 finished parsing run0000_0000 timepoint 0.13405
 finished parsing run0000_0000 timepoint 0.1341
 finished parsing run0000_0000 timepoint 0.13415
 finished parsing run0000_0000 timepoint 0.1342
 finished parsing run0000_0000 timepoint 0.13425
 finished parsing run0000_0000 timepoint 0.1343
 finished parsing run0000_0000 timepoint 0.13435
 finished parsing run0000_0000 timepoint 0.1344
 finished parsing run0000_0000 timepoint 0.13445
 finished parsing run0000_0000 timepoint 0.1345
 finished parsing run0000_0000 timepoint 0.13455
 finished parsing run0000_0000 timepoint 0.1346
 finished parsing run0000_0000 timepoint 0.13465
 finished parsing run0000_0000 timepoint 0.1347
 finished parsing run0000_0000 timepoint 0.13475
 finished parsing run0000_0000 timepoint 0.1348
 finished parsing run0000_0000 timepoint 0.13485
 finished parsing run0000_0000 timepoint 0.1349
 finished parsing run0000_0000 timepoint 0.13495
 finished parsing run0000_0000 timepoint 0.135
 finished parsing run0000_0000 timepoint 0.13505
 finished parsing run0000_0000 timepoint 0.1351
 finished parsing run0000_0000 timepoint 0.13515
 finished parsing run0000_0000 timepoint 0.1352
 finished parsing run0000_0000 timepoint 0.13525
 finished parsing run0000_0000 timepoint 0.1353
 finished parsing run0000_0000 timepoint 0.13535
 finished parsing run0000_0000 timepoint 0.1354
 finished parsing run0000_0000 timepoint 0.13545
 finished parsing run0000_0000 timepoint 0.1355
 finished parsing run0000_0000 timepoint 0.13555
 finished parsing run0000_0000 timepoint 0.1356
 finished parsing run0000_0000 timepoint 0.13565
 finished parsing run0000_0000 timepoint 0.1357
 finished parsing run0000_0000 timepoint 0.13575
 finished parsing run0000_0000 timepoint 0.1358
 finished parsing run0000_0000 timepoint 0.13585
 finished parsing run0000_0000 timepoint 0.1359
 finished parsing run0000_0000 timepoint 0.13595
 finished parsing run0000_0000 timepoint 0.136
 finished parsing run0000_0000 timepoint 0.13605
 finished parsing run0000_0000 timepoint 0.1361
 finished parsing run0000_0000 timepoint 0.13615
 finished parsing run0000_0000 timepoint 0.1362
 finished parsing run0000_0000 timepoint 0.13625
 finished parsing run0000_0000 timepoint 0.1363
 finished parsing run0000_0000 timepoint 0.13635
 finished parsing run0000_0000 timepoint 0.1364
 finished parsing run0000_0000 timepoint 0.13645
 finished parsing run0000_0000 timepoint 0.1365
 finished parsing run0000_0000 timepoint 0.13655
 finished parsing run0000_0000 timepoint 0.1366
 finished parsing run0000_0000 timepoint 0.13665
 finished parsing run0000_0000 timepoint 0.1367
 finished parsing run0000_0000 timepoint 0.13675
 finished parsing run0000_0000 timepoint 0.1368
 finished parsing run0000_0000 timepoint 0.13685
 finished parsing run0000_0000 timepoint 0.1369
 finished parsing run0000_0000 timepoint 0.13695
 finished parsing run0000_0000 timepoint 0.137
 finished parsing run0000_0000 timepoint 0.13705
 finished parsing run0000_0000 timepoint 0.1371
 finished parsing run0000_0000 timepoint 0.13715
 finished parsing run0000_0000 timepoint 0.1372
 finished parsing run0000_0000 timepoint 0.13725
 finished parsing run0000_0000 timepoint 0.1373
 finished parsing run0000_0000 timepoint 0.13735
 finished parsing run0000_0000 timepoint 0.1374
 finished parsing run0000_0000 timepoint 0.13745
 finished parsing run0000_0000 timepoint 0.1375
 finished parsing run0000_0000 timepoint 0.13755
 finished parsing run0000_0000 timepoint 0.1376
 finished parsing run0000_0000 timepoint 0.13765
 finished parsing run0000_0000 timepoint 0.1377
 finished parsing run0000_0000 timepoint 0.13775
 finished parsing run0000_0000 timepoint 0.1378
 finished parsing run0000_0000 timepoint 0.13785
 finished parsing run0000_0000 timepoint 0.1379
 finished parsing run0000_0000 timepoint 0.13795
 finished parsing run0000_0000 timepoint 0.138
 finished parsing run0000_0000 timepoint 0.13805
 finished parsing run0000_0000 timepoint 0.1381
 finished parsing run0000_0000 timepoint 0.13815
 finished parsing run0000_0000 timepoint 0.1382
 finished parsing run0000_0000 timepoint 0.13825
 finished parsing run0000_0000 timepoint 0.1383
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.1251
 finished parsing run0000_0000 timepoint 0.12515
 finished parsing run0000_0000 timepoint 0.1252
 finished parsing run0000_0000 timepoint 0.12525
 finished parsing run0000_0000 timepoint 0.1253
 finished parsing run0000_0000 timepoint 0.12535
 finished parsing run0000_0000 timepoint 0.1254
 finished parsing run0000_0000 timepoint 0.12545
 finished parsing run0000_0000 timepoint 0.1255
 finished parsing run0000_0000 timepoint 0.12555
 finished parsing run0000_0000 timepoint 0.1256
 finished parsing run0000_0000 timepoint 0.12565
 finished parsing run0000_0000 timepoint 0.1257
 finished parsing run0000_0000 timepoint 0.12575
 finished parsing run0000_0000 timepoint 0.1258
 finished parsing run0000_0000 timepoint 0.12585
 finished parsing run0000_0000 timepoint 0.1259
 finished parsing run0000_0000 timepoint 0.12595
 finished parsing run0000_0000 timepoint 0.126
 finished parsing run0000_0000 timepoint 0.12605
 finished parsing run0000_0000 timepoint 0.1261
 finished parsing run0000_0000 timepoint 0.12615
 finished parsing run0000_0000 timepoint 0.1262
 finished parsing run0000_0000 timepoint 0.12625
 finished parsing run0000_0000 timepoint 0.1263
 finished parsing run0000_0000 timepoint 0.12635
 finished parsing run0000_0000 timepoint 0.1264
 finished parsing run0000_0000 timepoint 0.12645
 finished parsing run0000_0000 timepoint 0.1265
 finished parsing run0000_0000 timepoint 0.12655
 finished parsing run0000_0000 timepoint 0.1266
 finished parsing run0000_0000 timepoint 0.12665
 finished parsing run0000_0000 timepoint 0.1267
 finished parsing run0000_0000 timepoint 0.12675
 finished parsing run0000_0000 timepoint 0.1268
 finished parsing run0000_0000 timepoint 0.12685
 finished parsing run0000_0000 timepoint 0.1269
 finished parsing run0000_0000 timepoint 0.12695
 finished parsing run0000_0000 timepoint 0.127
 finished parsing run0000_0000 timepoint 0.12705
 finished parsing run0000_0000 timepoint 0.1271
 finished parsing run0000_0000 timepoint 0.12715
 finished parsing run0000_0000 timepoint 0.1272
 finished parsing run0000_0000 timepoint 0.12725
 finished parsing run0000_0000 timepoint 0.1273
 finished parsing run0000_0000 timepoint 0.12735
 finished parsing run0000_0000 timepoint 0.1274
 finished parsing run0000_0000 timepoint 0.12745
 finished parsing run0000_0000 timepoint 0.1275
 finished parsing run0000_0000 timepoint 0.12755
 finished parsing run0000_0000 timepoint 0.1276
 finished parsing run0000_0000 timepoint 0.12765
 finished parsing run0000_0000 timepoint 0.1277
 finished parsing run0000_0000 timepoint 0.12775
 finished parsing run0000_0000 timepoint 0.1278
 finished parsing run0000_0000 timepoint 0.12785
 finished parsing run0000_0000 timepoint 0.1279
 finished parsing run0000_0000 timepoint 0.12795
 finished parsing run0000_0000 timepoint 0.128
 finished parsing run0000_0000 timepoint 0.12805
 finished parsing run0000_0000 timepoint 0.1281
 finished parsing run0000_0000 timepoint 0.12815
 finished parsing run0000_0000 timepoint 0.1282
 finished parsing run0000_0000 timepoint 0.12825
 finished parsing run0000_0000 timepoint 0.1283
 finished parsing run0000_0000 timepoint 0.12835
 finished parsing run0000_0000 timepoint 0.1284
 finished parsing run0000_0000 timepoint 0.12845
 finished parsing run0000_0000 timepoint 0.1285
 finished parsing run0000_0000 timepoint 0.12855
 finished parsing run0000_0000 timepoint 0.1286
 finished parsing run0000_0000 timepoint 0.12865
 finished parsing run0000_0000 timepoint 0.1287
 finished parsing run0000_0000 timepoint 0.12875
 finished parsing run0000_0000 timepoint 0.1288
 finished parsing run0000_0000 timepoint 0.12885
 finished parsing run0000_0000 timepoint 0.1289
 finished parsing run0000_0000 timepoint 0.12895
 finished parsing run0000_0000 timepoint 0.129
 finished parsing run0000_0000 timepoint 0.12905
 finished parsing run0000_0000 timepoint 0.1291
 finished parsing run0000_0000 timepoint 0.12915
 finished parsing run0000_0000 timepoint 0.1292
 finished parsing run0000_0000 timepoint 0.12925
 finished parsing run0000_0000 timepoint 0.1293
 finished parsing run0000_0000 timepoint 0.12935
 finished parsing run0000_0000 timepoint 0.1294
 finished parsing run0000_0000 timepoint 0.12945
 finished parsing run0000_0000 timepoint 0.1295
 finished parsing run0000_0000 timepoint 0.12955
 finished parsing run0000_0000 timepoint 0.1296
 finished parsing run0000_0000 timepoint 0.12965
 finished parsing run0000_0000 timepoint 0.1297
 finished parsing run0000_0000 timepoint 0.12975
 finished parsing run0000_0000 timepoint 0.1298
 finished parsing run0000_0000 timepoint 0.12985
 finished parsing run0000_0000 timepoint 0.1299
 finished parsing run0000_0000 timepoint 0.12995
 finished parsing run0000_0000 timepoint 0.13
 finished parsing run0000_0000 timepoint 0.13005
 finished parsing run0000_0000 timepoint 0.1301
 finished parsing run0000_0000 timepoint 0.13015
 finished parsing run0000_0000 timepoint 0.1302
 finished parsing run0000_0000 timepoint 0.13025
 finished parsing run0000_0000 timepoint 0.1303
 finished parsing run0000_0000 timepoint 0.13035
 finished parsing run0000_0000 timepoint 0.1304
 finished parsing run0000_0000 timepoint 0.13045
 finished parsing run0000_0000 timepoint 0.1305
 finished parsing run0000_0000 timepoint 0.13055
 finished parsing run0000_0000 timepoint 0.1306
 finished parsing run0000_0000 timepoint 0.13065
 finished parsing run0000_0000 timepoint 0.1307
 finished parsing run0000_0000 timepoint 0.13075
 finished parsing run0000_0000 timepoint 0.1308
 finished parsing run0000_0000 timepoint 0.13085
 finished parsing run0000_0000 timepoint 0.1309
 finished parsing run0000_0000 timepoint 0.13095
 finished parsing run0000_0000 timepoint 0.131
 finished parsing run0000_0000 timepoint 0.13105
 finished parsing run0000_0000 timepoint 0.1311
 finished parsing run0000_0000 timepoint 0.13115
 finished parsing run0000_0000 timepoint 0.1312
 finished parsing run0000_0000 timepoint 0.13125
 finished parsing run0000_0000 timepoint 0.1313
 finished parsing run0000_0000 timepoint 0.13135
 finished parsing run0000_0000 timepoint 0.1314
 finished parsing run0000_0000 timepoint 0.13145
 finished parsing run0000_0000 timepoint 0.1315
 finished parsing run0000_0000 timepoint 0.13155
 finished parsing run0000_0000 timepoint 0.1316
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.11825
 finished parsing run0000_0000 timepoint 0.1183
 finished parsing run0000_0000 timepoint 0.11835
 finished parsing run0000_0000 timepoint 0.1184
 finished parsing run0000_0000 timepoint 0.11845
 finished parsing run0000_0000 timepoint 0.1185
 finished parsing run0000_0000 timepoint 0.11855
 finished parsing run0000_0000 timepoint 0.1186
 finished parsing run0000_0000 timepoint 0.11865
 finished parsing run0000_0000 timepoint 0.1187
 finished parsing run0000_0000 timepoint 0.11875
 finished parsing run0000_0000 timepoint 0.1188
 finished parsing run0000_0000 timepoint 0.11885
 finished parsing run0000_0000 timepoint 0.1189
 finished parsing run0000_0000 timepoint 0.11895
 finished parsing run0000_0000 timepoint 0.119
 finished parsing run0000_0000 timepoint 0.11905
 finished parsing run0000_0000 timepoint 0.1191
 finished parsing run0000_0000 timepoint 0.11915
 finished parsing run0000_0000 timepoint 0.1192
 finished parsing run0000_0000 timepoint 0.11925
 finished parsing run0000_0000 timepoint 0.1193
 finished parsing run0000_0000 timepoint 0.11935
 finished parsing run0000_0000 timepoint 0.1194
 finished parsing run0000_0000 timepoint 0.11945
 finished parsing run0000_0000 timepoint 0.1195
 finished parsing run0000_0000 timepoint 0.11955
 finished parsing run0000_0000 timepoint 0.1196
 finished parsing run0000_0000 timepoint 0.11965
 finished parsing run0000_0000 timepoint 0.1197
 finished parsing run0000_0000 timepoint 0.11975
 finished parsing run0000_0000 timepoint 0.1198
 finished parsing run0000_0000 timepoint 0.11985
 finished parsing run0000_0000 timepoint 0.1199
 finished parsing run0000_0000 timepoint 0.11995
 finished parsing run0000_0000 timepoint 0.12
 finished parsing run0000_0000 timepoint 0.12005
 finished parsing run0000_0000 timepoint 0.1201
 finished parsing run0000_0000 timepoint 0.12015
 finished parsing run0000_0000 timepoint 0.1202
 finished parsing run0000_0000 timepoint 0.12025
 finished parsing run0000_0000 timepoint 0.1203
 finished parsing run0000_0000 timepoint 0.12035
 finished parsing run0000_0000 timepoint 0.1204
 finished parsing run0000_0000 timepoint 0.12045
 finished parsing run0000_0000 timepoint 0.1205
 finished parsing run0000_0000 timepoint 0.12055
 finished parsing run0000_0000 timepoint 0.1206
 finished parsing run0000_0000 timepoint 0.12065
 finished parsing run0000_0000 timepoint 0.1207
 finished parsing run0000_0000 timepoint 0.12075
 finished parsing run0000_0000 timepoint 0.1208
 finished parsing run0000_0000 timepoint 0.12085
 finished parsing run0000_0000 timepoint 0.1209
 finished parsing run0000_0000 timepoint 0.12095
 finished parsing run0000_0000 timepoint 0.121
 finished parsing run0000_0000 timepoint 0.12105
 finished parsing run0000_0000 timepoint 0.1211
 finished parsing run0000_0000 timepoint 0.12115
 finished parsing run0000_0000 timepoint 0.1212
 finished parsing run0000_0000 timepoint 0.12125
 finished parsing run0000_0000 timepoint 0.1213
 finished parsing run0000_0000 timepoint 0.12135
 finished parsing run0000_0000 timepoint 0.1214
 finished parsing run0000_0000 timepoint 0.12145
 finished parsing run0000_0000 timepoint 0.1215
 finished parsing run0000_0000 timepoint 0.12155
 finished parsing run0000_0000 timepoint 0.1216
 finished parsing run0000_0000 timepoint 0.12165
 finished parsing run0000_0000 timepoint 0.1217
 finished parsing run0000_0000 timepoint 0.12175
 finished parsing run0000_0000 timepoint 0.1218
 finished parsing run0000_0000 timepoint 0.12185
 finished parsing run0000_0000 timepoint 0.1219
 finished parsing run0000_0000 timepoint 0.12195
 finished parsing run0000_0000 timepoint 0.122
 finished parsing run0000_0000 timepoint 0.12205
 finished parsing run0000_0000 timepoint 0.1221
 finished parsing run0000_0000 timepoint 0.12215
 finished parsing run0000_0000 timepoint 0.1222
 finished parsing run0000_0000 timepoint 0.12225
 finished parsing run0000_0000 timepoint 0.1223
 finished parsing run0000_0000 timepoint 0.12235
 finished parsing run0000_0000 timepoint 0.1224
 finished parsing run0000_0000 timepoint 0.12245
 finished parsing run0000_0000 timepoint 0.1225
 finished parsing run0000_0000 timepoint 0.12255
 finished parsing run0000_0000 timepoint 0.1226
 finished parsing run0000_0000 timepoint 0.12265
 finished parsing run0000_0000 timepoint 0.1227
 finished parsing run0000_0000 timepoint 0.12275
 finished parsing run0000_0000 timepoint 0.1228
 finished parsing run0000_0000 timepoint 0.12285
 finished parsing run0000_0000 timepoint 0.1229
 finished parsing run0000_0000 timepoint 0.12295
 finished parsing run0000_0000 timepoint 0.123
 finished parsing run0000_0000 timepoint 0.12305
 finished parsing run0000_0000 timepoint 0.1231
 finished parsing run0000_0000 timepoint 0.12315
 finished parsing run0000_0000 timepoint 0.1232
 finished parsing run0000_0000 timepoint 0.12325
 finished parsing run0000_0000 timepoint 0.1233
 finished parsing run0000_0000 timepoint 0.12335
 finished parsing run0000_0000 timepoint 0.1234
 finished parsing run0000_0000 timepoint 0.12345
 finished parsing run0000_0000 timepoint 0.1235
 finished parsing run0000_0000 timepoint 0.12355
 finished parsing run0000_0000 timepoint 0.1236
 finished parsing run0000_0000 timepoint 0.12365
 finished parsing run0000_0000 timepoint 0.1237
 finished parsing run0000_0000 timepoint 0.12375
 finished parsing run0000_0000 timepoint 0.1238
 finished parsing run0000_0000 timepoint 0.12385
 finished parsing run0000_0000 timepoint 0.1239
 finished parsing run0000_0000 timepoint 0.12395
 finished parsing run0000_0000 timepoint 0.124
 finished parsing run0000_0000 timepoint 0.12405
 finished parsing run0000_0000 timepoint 0.1241
 finished parsing run0000_0000 timepoint 0.12415
 finished parsing run0000_0000 timepoint 0.1242
 finished parsing run0000_0000 timepoint 0.12425
 finished parsing run0000_0000 timepoint 0.1243
 finished parsing run0000_0000 timepoint 0.12435
 finished parsing run0000_0000 timepoint 0.1244
 finished parsing run0000_0000 timepoint 0.12445
 finished parsing run0000_0000 timepoint 0.1245
 finished parsing run0000_0000 timepoint 0.12455
 finished parsing run0000_0000 timepoint 0.1246
 finished parsing run0000_0000 timepoint 0.12465
 finished parsing run0000_0000 timepoint 0.1247
 finished parsing run0000_0000 timepoint 0.12475
 finished parsing run0000_0000 timepoint 0.1248
 finished parsing run0000_0000 timepoint 0.12485
 finished parsing run0000_0000 timepoint 0.1249
 finished parsing run0000_0000 timepoint 0.12495
 finished parsing run0000_0000 timepoint 0.125
 finished parsing run0000_0000 timepoint 0.12505
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.11145
 finished parsing run0000_0000 timepoint 0.1115
 finished parsing run0000_0000 timepoint 0.11155
 finished parsing run0000_0000 timepoint 0.1116
 finished parsing run0000_0000 timepoint 0.11165
 finished parsing run0000_0000 timepoint 0.1117
 finished parsing run0000_0000 timepoint 0.11175
 finished parsing run0000_0000 timepoint 0.1118
 finished parsing run0000_0000 timepoint 0.11185
 finished parsing run0000_0000 timepoint 0.1119
 finished parsing run0000_0000 timepoint 0.11195
 finished parsing run0000_0000 timepoint 0.112
 finished parsing run0000_0000 timepoint 0.11205
 finished parsing run0000_0000 timepoint 0.1121
 finished parsing run0000_0000 timepoint 0.11215
 finished parsing run0000_0000 timepoint 0.1122
 finished parsing run0000_0000 timepoint 0.11225
 finished parsing run0000_0000 timepoint 0.1123
 finished parsing run0000_0000 timepoint 0.11235
 finished parsing run0000_0000 timepoint 0.1124
 finished parsing run0000_0000 timepoint 0.11245
 finished parsing run0000_0000 timepoint 0.1125
 finished parsing run0000_0000 timepoint 0.11255
 finished parsing run0000_0000 timepoint 0.1126
 finished parsing run0000_0000 timepoint 0.11265
 finished parsing run0000_0000 timepoint 0.1127
 finished parsing run0000_0000 timepoint 0.11275
 finished parsing run0000_0000 timepoint 0.1128
 finished parsing run0000_0000 timepoint 0.11285
 finished parsing run0000_0000 timepoint 0.1129
 finished parsing run0000_0000 timepoint 0.11295
 finished parsing run0000_0000 timepoint 0.113
 finished parsing run0000_0000 timepoint 0.11305
 finished parsing run0000_0000 timepoint 0.1131
 finished parsing run0000_0000 timepoint 0.11315
 finished parsing run0000_0000 timepoint 0.1132
 finished parsing run0000_0000 timepoint 0.11325
 finished parsing run0000_0000 timepoint 0.1133
 finished parsing run0000_0000 timepoint 0.11335
 finished parsing run0000_0000 timepoint 0.1134
 finished parsing run0000_0000 timepoint 0.11345
 finished parsing run0000_0000 timepoint 0.1135
 finished parsing run0000_0000 timepoint 0.11355
 finished parsing run0000_0000 timepoint 0.1136
 finished parsing run0000_0000 timepoint 0.11365
 finished parsing run0000_0000 timepoint 0.1137
 finished parsing run0000_0000 timepoint 0.11375
 finished parsing run0000_0000 timepoint 0.1138
 finished parsing run0000_0000 timepoint 0.11385
 finished parsing run0000_0000 timepoint 0.1139
 finished parsing run0000_0000 timepoint 0.11395
 finished parsing run0000_0000 timepoint 0.114
 finished parsing run0000_0000 timepoint 0.11405
 finished parsing run0000_0000 timepoint 0.1141
 finished parsing run0000_0000 timepoint 0.11415
 finished parsing run0000_0000 timepoint 0.1142
 finished parsing run0000_0000 timepoint 0.11425
 finished parsing run0000_0000 timepoint 0.1143
 finished parsing run0000_0000 timepoint 0.11435
 finished parsing run0000_0000 timepoint 0.1144
 finished parsing run0000_0000 timepoint 0.11445
 finished parsing run0000_0000 timepoint 0.1145
 finished parsing run0000_0000 timepoint 0.11455
 finished parsing run0000_0000 timepoint 0.1146
 finished parsing run0000_0000 timepoint 0.11465
 finished parsing run0000_0000 timepoint 0.1147
 finished parsing run0000_0000 timepoint 0.11475
 finished parsing run0000_0000 timepoint 0.1148
 finished parsing run0000_0000 timepoint 0.11485
 finished parsing run0000_0000 timepoint 0.1149
 finished parsing run0000_0000 timepoint 0.11495
 finished parsing run0000_0000 timepoint 0.115
 finished parsing run0000_0000 timepoint 0.11505
 finished parsing run0000_0000 timepoint 0.1151
 finished parsing run0000_0000 timepoint 0.11515
 finished parsing run0000_0000 timepoint 0.1152
 finished parsing run0000_0000 timepoint 0.11525
 finished parsing run0000_0000 timepoint 0.1153
 finished parsing run0000_0000 timepoint 0.11535
 finished parsing run0000_0000 timepoint 0.1154
 finished parsing run0000_0000 timepoint 0.11545
 finished parsing run0000_0000 timepoint 0.1155
 finished parsing run0000_0000 timepoint 0.11555
 finished parsing run0000_0000 timepoint 0.1156
 finished parsing run0000_0000 timepoint 0.11565
 finished parsing run0000_0000 timepoint 0.1157
 finished parsing run0000_0000 timepoint 0.11575
 finished parsing run0000_0000 timepoint 0.1158
 finished parsing run0000_0000 timepoint 0.11585
 finished parsing run0000_0000 timepoint 0.1159
 finished parsing run0000_0000 timepoint 0.11595
 finished parsing run0000_0000 timepoint 0.116
 finished parsing run0000_0000 timepoint 0.11605
 finished parsing run0000_0000 timepoint 0.1161
 finished parsing run0000_0000 timepoint 0.11615
 finished parsing run0000_0000 timepoint 0.1162
 finished parsing run0000_0000 timepoint 0.11625
 finished parsing run0000_0000 timepoint 0.1163
 finished parsing run0000_0000 timepoint 0.11635
 finished parsing run0000_0000 timepoint 0.1164
 finished parsing run0000_0000 timepoint 0.11645
 finished parsing run0000_0000 timepoint 0.1165
 finished parsing run0000_0000 timepoint 0.11655
 finished parsing run0000_0000 timepoint 0.1166
 finished parsing run0000_0000 timepoint 0.11665
 finished parsing run0000_0000 timepoint 0.1167
 finished parsing run0000_0000 timepoint 0.11675
 finished parsing run0000_0000 timepoint 0.1168
 finished parsing run0000_0000 timepoint 0.11685
 finished parsing run0000_0000 timepoint 0.1169
 finished parsing run0000_0000 timepoint 0.11695
 finished parsing run0000_0000 timepoint 0.117
 finished parsing run0000_0000 timepoint 0.11705
 finished parsing run0000_0000 timepoint 0.1171
 finished parsing run0000_0000 timepoint 0.11715
 finished parsing run0000_0000 timepoint 0.1172
 finished parsing run0000_0000 timepoint 0.11725
 finished parsing run0000_0000 timepoint 0.1173
 finished parsing run0000_0000 timepoint 0.11735
 finished parsing run0000_0000 timepoint 0.1174
 finished parsing run0000_0000 timepoint 0.11745
 finished parsing run0000_0000 timepoint 0.1175
 finished parsing run0000_0000 timepoint 0.11755
 finished parsing run0000_0000 timepoint 0.1176
 finished parsing run0000_0000 timepoint 0.11765
 finished parsing run0000_0000 timepoint 0.1177
 finished parsing run0000_0000 timepoint 0.11775
 finished parsing run0000_0000 timepoint 0.1178
 finished parsing run0000_0000 timepoint 0.11785
 finished parsing run0000_0000 timepoint 0.1179
 finished parsing run0000_0000 timepoint 0.11795
 finished parsing run0000_0000 timepoint 0.118
 finished parsing run0000_0000 timepoint 0.11805
 finished parsing run0000_0000 timepoint 0.1181
 finished parsing run0000_0000 timepoint 0.11815
 finished parsing run0000_0000 timepoint 0.1182
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.10465
 finished parsing run0000_0000 timepoint 0.1047
 finished parsing run0000_0000 timepoint 0.10475
 finished parsing run0000_0000 timepoint 0.1048
 finished parsing run0000_0000 timepoint 0.10485
 finished parsing run0000_0000 timepoint 0.1049
 finished parsing run0000_0000 timepoint 0.10495
 finished parsing run0000_0000 timepoint 0.105
 finished parsing run0000_0000 timepoint 0.10505
 finished parsing run0000_0000 timepoint 0.1051
 finished parsing run0000_0000 timepoint 0.10515
 finished parsing run0000_0000 timepoint 0.1052
 finished parsing run0000_0000 timepoint 0.10525
 finished parsing run0000_0000 timepoint 0.1053
 finished parsing run0000_0000 timepoint 0.10535
 finished parsing run0000_0000 timepoint 0.1054
 finished parsing run0000_0000 timepoint 0.10545
 finished parsing run0000_0000 timepoint 0.1055
 finished parsing run0000_0000 timepoint 0.10555
 finished parsing run0000_0000 timepoint 0.1056
 finished parsing run0000_0000 timepoint 0.10565
 finished parsing run0000_0000 timepoint 0.1057
 finished parsing run0000_0000 timepoint 0.10575
 finished parsing run0000_0000 timepoint 0.1058
 finished parsing run0000_0000 timepoint 0.10585
 finished parsing run0000_0000 timepoint 0.1059
 finished parsing run0000_0000 timepoint 0.10595
 finished parsing run0000_0000 timepoint 0.106
 finished parsing run0000_0000 timepoint 0.10605
 finished parsing run0000_0000 timepoint 0.1061
 finished parsing run0000_0000 timepoint 0.10615
 finished parsing run0000_0000 timepoint 0.1062
 finished parsing run0000_0000 timepoint 0.10625
 finished parsing run0000_0000 timepoint 0.1063
 finished parsing run0000_0000 timepoint 0.10635
 finished parsing run0000_0000 timepoint 0.1064
 finished parsing run0000_0000 timepoint 0.10645
 finished parsing run0000_0000 timepoint 0.1065
 finished parsing run0000_0000 timepoint 0.10655
 finished parsing run0000_0000 timepoint 0.1066
 finished parsing run0000_0000 timepoint 0.10665
 finished parsing run0000_0000 timepoint 0.1067
 finished parsing run0000_0000 timepoint 0.10675
 finished parsing run0000_0000 timepoint 0.1068
 finished parsing run0000_0000 timepoint 0.10685
 finished parsing run0000_0000 timepoint 0.1069
 finished parsing run0000_0000 timepoint 0.10695
 finished parsing run0000_0000 timepoint 0.107
 finished parsing run0000_0000 timepoint 0.10705
 finished parsing run0000_0000 timepoint 0.1071
 finished parsing run0000_0000 timepoint 0.10715
 finished parsing run0000_0000 timepoint 0.1072
 finished parsing run0000_0000 timepoint 0.10725
 finished parsing run0000_0000 timepoint 0.1073
 finished parsing run0000_0000 timepoint 0.10735
 finished parsing run0000_0000 timepoint 0.1074
 finished parsing run0000_0000 timepoint 0.10745
 finished parsing run0000_0000 timepoint 0.1075
 finished parsing run0000_0000 timepoint 0.10755
 finished parsing run0000_0000 timepoint 0.1076
 finished parsing run0000_0000 timepoint 0.10765
 finished parsing run0000_0000 timepoint 0.1077
 finished parsing run0000_0000 timepoint 0.10775
 finished parsing run0000_0000 timepoint 0.1078
 finished parsing run0000_0000 timepoint 0.10785
 finished parsing run0000_0000 timepoint 0.1079
 finished parsing run0000_0000 timepoint 0.10795
 finished parsing run0000_0000 timepoint 0.108
 finished parsing run0000_0000 timepoint 0.10805
 finished parsing run0000_0000 timepoint 0.1081
 finished parsing run0000_0000 timepoint 0.10815
 finished parsing run0000_0000 timepoint 0.1082
 finished parsing run0000_0000 timepoint 0.10825
 finished parsing run0000_0000 timepoint 0.1083
 finished parsing run0000_0000 timepoint 0.10835
 finished parsing run0000_0000 timepoint 0.1084
 finished parsing run0000_0000 timepoint 0.10845
 finished parsing run0000_0000 timepoint 0.1085
 finished parsing run0000_0000 timepoint 0.10855
 finished parsing run0000_0000 timepoint 0.1086
 finished parsing run0000_0000 timepoint 0.10865
 finished parsing run0000_0000 timepoint 0.1087
 finished parsing run0000_0000 timepoint 0.10875
 finished parsing run0000_0000 timepoint 0.1088
 finished parsing run0000_0000 timepoint 0.10885
 finished parsing run0000_0000 timepoint 0.1089
 finished parsing run0000_0000 timepoint 0.10895
 finished parsing run0000_0000 timepoint 0.109
 finished parsing run0000_0000 timepoint 0.10905
 finished parsing run0000_0000 timepoint 0.1091
 finished parsing run0000_0000 timepoint 0.10915
 finished parsing run0000_0000 timepoint 0.1092
 finished parsing run0000_0000 timepoint 0.10925
 finished parsing run0000_0000 timepoint 0.1093
 finished parsing run0000_0000 timepoint 0.10935
 finished parsing run0000_0000 timepoint 0.1094
 finished parsing run0000_0000 timepoint 0.10945
 finished parsing run0000_0000 timepoint 0.1095
 finished parsing run0000_0000 timepoint 0.10955
 finished parsing run0000_0000 timepoint 0.1096
 finished parsing run0000_0000 timepoint 0.10965
 finished parsing run0000_0000 timepoint 0.1097
 finished parsing run0000_0000 timepoint 0.10975
 finished parsing run0000_0000 timepoint 0.1098
 finished parsing run0000_0000 timepoint 0.10985
 finished parsing run0000_0000 timepoint 0.1099
 finished parsing run0000_0000 timepoint 0.10995
 finished parsing run0000_0000 timepoint 0.11
 finished parsing run0000_0000 timepoint 0.11005
 finished parsing run0000_0000 timepoint 0.1101
 finished parsing run0000_0000 timepoint 0.11015
 finished parsing run0000_0000 timepoint 0.1102
 finished parsing run0000_0000 timepoint 0.11025
 finished parsing run0000_0000 timepoint 0.1103
 finished parsing run0000_0000 timepoint 0.11035
 finished parsing run0000_0000 timepoint 0.1104
 finished parsing run0000_0000 timepoint 0.11045
 finished parsing run0000_0000 timepoint 0.1105
 finished parsing run0000_0000 timepoint 0.11055
 finished parsing run0000_0000 timepoint 0.1106
 finished parsing run0000_0000 timepoint 0.11065
 finished parsing run0000_0000 timepoint 0.1107
 finished parsing run0000_0000 timepoint 0.11075
 finished parsing run0000_0000 timepoint 0.1108
 finished parsing run0000_0000 timepoint 0.11085
 finished parsing run0000_0000 timepoint 0.1109
 finished parsing run0000_0000 timepoint 0.11095
 finished parsing run0000_0000 timepoint 0.111
 finished parsing run0000_0000 timepoint 0.11105
 finished parsing run0000_0000 timepoint 0.1111
 finished parsing run0000_0000 timepoint 0.11115
 finished parsing run0000_0000 timepoint 0.1112
 finished parsing run0000_0000 timepoint 0.11125
 finished parsing run0000_0000 timepoint 0.1113
 finished parsing run0000_0000 timepoint 0.11135
 finished parsing run0000_0000 timepoint 0.1114
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.0982
 finished parsing run0000_0000 timepoint 0.09825
 finished parsing run0000_0000 timepoint 0.0983
 finished parsing run0000_0000 timepoint 0.09835
 finished parsing run0000_0000 timepoint 0.0984
 finished parsing run0000_0000 timepoint 0.09845
 finished parsing run0000_0000 timepoint 0.0985
 finished parsing run0000_0000 timepoint 0.09855
 finished parsing run0000_0000 timepoint 0.0986
 finished parsing run0000_0000 timepoint 0.09865
 finished parsing run0000_0000 timepoint 0.0987
 finished parsing run0000_0000 timepoint 0.09875
 finished parsing run0000_0000 timepoint 0.0988
 finished parsing run0000_0000 timepoint 0.09885
 finished parsing run0000_0000 timepoint 0.0989
 finished parsing run0000_0000 timepoint 0.09895
 finished parsing run0000_0000 timepoint 0.099
 finished parsing run0000_0000 timepoint 0.09905
 finished parsing run0000_0000 timepoint 0.0991
 finished parsing run0000_0000 timepoint 0.09915
 finished parsing run0000_0000 timepoint 0.0992
 finished parsing run0000_0000 timepoint 0.09925
 finished parsing run0000_0000 timepoint 0.0993
 finished parsing run0000_0000 timepoint 0.09935
 finished parsing run0000_0000 timepoint 0.0994
 finished parsing run0000_0000 timepoint 0.09945
 finished parsing run0000_0000 timepoint 0.0995
 finished parsing run0000_0000 timepoint 0.09955
 finished parsing run0000_0000 timepoint 0.0996
 finished parsing run0000_0000 timepoint 0.09965
 finished parsing run0000_0000 timepoint 0.0997
 finished parsing run0000_0000 timepoint 0.09975
 finished parsing run0000_0000 timepoint 0.0998
 finished parsing run0000_0000 timepoint 0.09985
 finished parsing run0000_0000 timepoint 0.0999
 finished parsing run0000_0000 timepoint 0.09995
 finished parsing run0000_0000 timepoint 0.1
 finished parsing run0000_0000 timepoint 0.10005
 finished parsing run0000_0000 timepoint 0.1001
 finished parsing run0000_0000 timepoint 0.10015
 finished parsing run0000_0000 timepoint 0.1002
 finished parsing run0000_0000 timepoint 0.10025
 finished parsing run0000_0000 timepoint 0.1003
 finished parsing run0000_0000 timepoint 0.10035
 finished parsing run0000_0000 timepoint 0.1004
 finished parsing run0000_0000 timepoint 0.10045
 finished parsing run0000_0000 timepoint 0.1005
 finished parsing run0000_0000 timepoint 0.10055
 finished parsing run0000_0000 timepoint 0.1006
 finished parsing run0000_0000 timepoint 0.10065
 finished parsing run0000_0000 timepoint 0.1007
 finished parsing run0000_0000 timepoint 0.10075
 finished parsing run0000_0000 timepoint 0.1008
 finished parsing run0000_0000 timepoint 0.10085
 finished parsing run0000_0000 timepoint 0.1009
 finished parsing run0000_0000 timepoint 0.10095
 finished parsing run0000_0000 timepoint 0.101
 finished parsing run0000_0000 timepoint 0.10105
 finished parsing run0000_0000 timepoint 0.1011
 finished parsing run0000_0000 timepoint 0.10115
 finished parsing run0000_0000 timepoint 0.1012
 finished parsing run0000_0000 timepoint 0.10125
 finished parsing run0000_0000 timepoint 0.1013
 finished parsing run0000_0000 timepoint 0.10135
 finished parsing run0000_0000 timepoint 0.1014
 finished parsing run0000_0000 timepoint 0.10145
 finished parsing run0000_0000 timepoint 0.1015
 finished parsing run0000_0000 timepoint 0.10155
 finished parsing run0000_0000 timepoint 0.1016
 finished parsing run0000_0000 timepoint 0.10165
 finished parsing run0000_0000 timepoint 0.1017
 finished parsing run0000_0000 timepoint 0.10175
 finished parsing run0000_0000 timepoint 0.1018
 finished parsing run0000_0000 timepoint 0.10185
 finished parsing run0000_0000 timepoint 0.1019
 finished parsing run0000_0000 timepoint 0.10195
 finished parsing run0000_0000 timepoint 0.102
 finished parsing run0000_0000 timepoint 0.10205
 finished parsing run0000_0000 timepoint 0.1021
 finished parsing run0000_0000 timepoint 0.10215
 finished parsing run0000_0000 timepoint 0.1022
 finished parsing run0000_0000 timepoint 0.10225
 finished parsing run0000_0000 timepoint 0.1023
 finished parsing run0000_0000 timepoint 0.10235
 finished parsing run0000_0000 timepoint 0.1024
 finished parsing run0000_0000 timepoint 0.10245
 finished parsing run0000_0000 timepoint 0.1025
 finished parsing run0000_0000 timepoint 0.10255
 finished parsing run0000_0000 timepoint 0.1026
 finished parsing run0000_0000 timepoint 0.10265
 finished parsing run0000_0000 timepoint 0.1027
 finished parsing run0000_0000 timepoint 0.10275
 finished parsing run0000_0000 timepoint 0.1028
 finished parsing run0000_0000 timepoint 0.10285
 finished parsing run0000_0000 timepoint 0.1029
 finished parsing run0000_0000 timepoint 0.10295
 finished parsing run0000_0000 timepoint 0.103
 finished parsing run0000_0000 timepoint 0.10305
 finished parsing run0000_0000 timepoint 0.1031
 finished parsing run0000_0000 timepoint 0.10315
 finished parsing run0000_0000 timepoint 0.1032
 finished parsing run0000_0000 timepoint 0.10325
 finished parsing run0000_0000 timepoint 0.1033
 finished parsing run0000_0000 timepoint 0.10335
 finished parsing run0000_0000 timepoint 0.1034
 finished parsing run0000_0000 timepoint 0.10345
 finished parsing run0000_0000 timepoint 0.1035
 finished parsing run0000_0000 timepoint 0.10355
 finished parsing run0000_0000 timepoint 0.1036
 finished parsing run0000_0000 timepoint 0.10365
 finished parsing run0000_0000 timepoint 0.1037
 finished parsing run0000_0000 timepoint 0.10375
 finished parsing run0000_0000 timepoint 0.1038
 finished parsing run0000_0000 timepoint 0.10385
 finished parsing run0000_0000 timepoint 0.1039
 finished parsing run0000_0000 timepoint 0.10395
 finished parsing run0000_0000 timepoint 0.104
 finished parsing run0000_0000 timepoint 0.10405
 finished parsing run0000_0000 timepoint 0.1041
 finished parsing run0000_0000 timepoint 0.10415
 finished parsing run0000_0000 timepoint 0.1042
 finished parsing run0000_0000 timepoint 0.10425
 finished parsing run0000_0000 timepoint 0.1043
 finished parsing run0000_0000 timepoint 0.10435
 finished parsing run0000_0000 timepoint 0.1044
 finished parsing run0000_0000 timepoint 0.10445
 finished parsing run0000_0000 timepoint 0.1045
 finished parsing run0000_0000 timepoint 0.10455
 finished parsing run0000_0000 timepoint 0.1046
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.09165
 finished parsing run0000_0000 timepoint 0.0917
 finished parsing run0000_0000 timepoint 0.09175
 finished parsing run0000_0000 timepoint 0.0918
 finished parsing run0000_0000 timepoint 0.09185
 finished parsing run0000_0000 timepoint 0.0919
 finished parsing run0000_0000 timepoint 0.09195
 finished parsing run0000_0000 timepoint 0.092
 finished parsing run0000_0000 timepoint 0.09205
 finished parsing run0000_0000 timepoint 0.0921
 finished parsing run0000_0000 timepoint 0.09215
 finished parsing run0000_0000 timepoint 0.0922
 finished parsing run0000_0000 timepoint 0.09225
 finished parsing run0000_0000 timepoint 0.0923
 finished parsing run0000_0000 timepoint 0.09235
 finished parsing run0000_0000 timepoint 0.0924
 finished parsing run0000_0000 timepoint 0.09245
 finished parsing run0000_0000 timepoint 0.0925
 finished parsing run0000_0000 timepoint 0.09255
 finished parsing run0000_0000 timepoint 0.0926
 finished parsing run0000_0000 timepoint 0.09265
 finished parsing run0000_0000 timepoint 0.0927
 finished parsing run0000_0000 timepoint 0.09275
 finished parsing run0000_0000 timepoint 0.0928
 finished parsing run0000_0000 timepoint 0.09285
 finished parsing run0000_0000 timepoint 0.0929
 finished parsing run0000_0000 timepoint 0.09295
 finished parsing run0000_0000 timepoint 0.093
 finished parsing run0000_0000 timepoint 0.09305
 finished parsing run0000_0000 timepoint 0.0931
 finished parsing run0000_0000 timepoint 0.09315
 finished parsing run0000_0000 timepoint 0.0932
 finished parsing run0000_0000 timepoint 0.09325
 finished parsing run0000_0000 timepoint 0.0933
 finished parsing run0000_0000 timepoint 0.09335
 finished parsing run0000_0000 timepoint 0.0934
 finished parsing run0000_0000 timepoint 0.09345
 finished parsing run0000_0000 timepoint 0.0935
 finished parsing run0000_0000 timepoint 0.09355
 finished parsing run0000_0000 timepoint 0.0936
 finished parsing run0000_0000 timepoint 0.09365
 finished parsing run0000_0000 timepoint 0.0937
 finished parsing run0000_0000 timepoint 0.09375
 finished parsing run0000_0000 timepoint 0.0938
 finished parsing run0000_0000 timepoint 0.09385
 finished parsing run0000_0000 timepoint 0.0939
 finished parsing run0000_0000 timepoint 0.09395
 finished parsing run0000_0000 timepoint 0.094
 finished parsing run0000_0000 timepoint 0.09405
 finished parsing run0000_0000 timepoint 0.0941
 finished parsing run0000_0000 timepoint 0.09415
 finished parsing run0000_0000 timepoint 0.0942
 finished parsing run0000_0000 timepoint 0.09425
 finished parsing run0000_0000 timepoint 0.0943
 finished parsing run0000_0000 timepoint 0.09435
 finished parsing run0000_0000 timepoint 0.0944
 finished parsing run0000_0000 timepoint 0.09445
 finished parsing run0000_0000 timepoint 0.0945
 finished parsing run0000_0000 timepoint 0.09455
 finished parsing run0000_0000 timepoint 0.0946
 finished parsing run0000_0000 timepoint 0.09465
 finished parsing run0000_0000 timepoint 0.0947
 finished parsing run0000_0000 timepoint 0.09475
 finished parsing run0000_0000 timepoint 0.0948
 finished parsing run0000_0000 timepoint 0.09485
 finished parsing run0000_0000 timepoint 0.0949
 finished parsing run0000_0000 timepoint 0.09495
 finished parsing run0000_0000 timepoint 0.095
 finished parsing run0000_0000 timepoint 0.09505
 finished parsing run0000_0000 timepoint 0.0951
 finished parsing run0000_0000 timepoint 0.09515
 finished parsing run0000_0000 timepoint 0.0952
 finished parsing run0000_0000 timepoint 0.09525
 finished parsing run0000_0000 timepoint 0.0953
 finished parsing run0000_0000 timepoint 0.09535
 finished parsing run0000_0000 timepoint 0.0954
 finished parsing run0000_0000 timepoint 0.09545
 finished parsing run0000_0000 timepoint 0.0955
 finished parsing run0000_0000 timepoint 0.09555
 finished parsing run0000_0000 timepoint 0.0956
 finished parsing run0000_0000 timepoint 0.09565
 finished parsing run0000_0000 timepoint 0.0957
 finished parsing run0000_0000 timepoint 0.09575
 finished parsing run0000_0000 timepoint 0.0958
 finished parsing run0000_0000 timepoint 0.09585
 finished parsing run0000_0000 timepoint 0.0959
 finished parsing run0000_0000 timepoint 0.09595
 finished parsing run0000_0000 timepoint 0.096
 finished parsing run0000_0000 timepoint 0.09605
 finished parsing run0000_0000 timepoint 0.0961
 finished parsing run0000_0000 timepoint 0.09615
 finished parsing run0000_0000 timepoint 0.0962
 finished parsing run0000_0000 timepoint 0.09625
 finished parsing run0000_0000 timepoint 0.0963
 finished parsing run0000_0000 timepoint 0.09635
 finished parsing run0000_0000 timepoint 0.0964
 finished parsing run0000_0000 timepoint 0.09645
 finished parsing run0000_0000 timepoint 0.0965
 finished parsing run0000_0000 timepoint 0.09655
 finished parsing run0000_0000 timepoint 0.0966
 finished parsing run0000_0000 timepoint 0.09665
 finished parsing run0000_0000 timepoint 0.0967
 finished parsing run0000_0000 timepoint 0.09675
 finished parsing run0000_0000 timepoint 0.0968
 finished parsing run0000_0000 timepoint 0.09685
 finished parsing run0000_0000 timepoint 0.0969
 finished parsing run0000_0000 timepoint 0.09695
 finished parsing run0000_0000 timepoint 0.097
 finished parsing run0000_0000 timepoint 0.09705
 finished parsing run0000_0000 timepoint 0.0971
 finished parsing run0000_0000 timepoint 0.09715
 finished parsing run0000_0000 timepoint 0.0972
 finished parsing run0000_0000 timepoint 0.09725
 finished parsing run0000_0000 timepoint 0.0973
 finished parsing run0000_0000 timepoint 0.09735
 finished parsing run0000_0000 timepoint 0.0974
 finished parsing run0000_0000 timepoint 0.09745
 finished parsing run0000_0000 timepoint 0.0975
 finished parsing run0000_0000 timepoint 0.09755
 finished parsing run0000_0000 timepoint 0.0976
 finished parsing run0000_0000 timepoint 0.09765
 finished parsing run0000_0000 timepoint 0.0977
 finished parsing run0000_0000 timepoint 0.09775
 finished parsing run0000_0000 timepoint 0.0978
 finished parsing run0000_0000 timepoint 0.09785
 finished parsing run0000_0000 timepoint 0.0979
 finished parsing run0000_0000 timepoint 0.09795
 finished parsing run0000_0000 timepoint 0.098
 finished parsing run0000_0000 timepoint 0.09805
 finished parsing run0000_0000 timepoint 0.0981
 finished parsing run0000_0000 timepoint 0.09815
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.08515
 finished parsing run0000_0000 timepoint 0.0852
 finished parsing run0000_0000 timepoint 0.08525
 finished parsing run0000_0000 timepoint 0.0853
 finished parsing run0000_0000 timepoint 0.08535
 finished parsing run0000_0000 timepoint 0.0854
 finished parsing run0000_0000 timepoint 0.08545
 finished parsing run0000_0000 timepoint 0.0855
 finished parsing run0000_0000 timepoint 0.08555
 finished parsing run0000_0000 timepoint 0.0856
 finished parsing run0000_0000 timepoint 0.08565
 finished parsing run0000_0000 timepoint 0.0857
 finished parsing run0000_0000 timepoint 0.08575
 finished parsing run0000_0000 timepoint 0.0858
 finished parsing run0000_0000 timepoint 0.08585
 finished parsing run0000_0000 timepoint 0.0859
 finished parsing run0000_0000 timepoint 0.08595
 finished parsing run0000_0000 timepoint 0.086
 finished parsing run0000_0000 timepoint 0.08605
 finished parsing run0000_0000 timepoint 0.0861
 finished parsing run0000_0000 timepoint 0.08615
 finished parsing run0000_0000 timepoint 0.0862
 finished parsing run0000_0000 timepoint 0.08625
 finished parsing run0000_0000 timepoint 0.0863
 finished parsing run0000_0000 timepoint 0.08635
 finished parsing run0000_0000 timepoint 0.0864
 finished parsing run0000_0000 timepoint 0.08645
 finished parsing run0000_0000 timepoint 0.0865
 finished parsing run0000_0000 timepoint 0.08655
 finished parsing run0000_0000 timepoint 0.0866
 finished parsing run0000_0000 timepoint 0.08665
 finished parsing run0000_0000 timepoint 0.0867
 finished parsing run0000_0000 timepoint 0.08675
 finished parsing run0000_0000 timepoint 0.0868
 finished parsing run0000_0000 timepoint 0.08685
 finished parsing run0000_0000 timepoint 0.0869
 finished parsing run0000_0000 timepoint 0.08695
 finished parsing run0000_0000 timepoint 0.087
 finished parsing run0000_0000 timepoint 0.08705
 finished parsing run0000_0000 timepoint 0.0871
 finished parsing run0000_0000 timepoint 0.08715
 finished parsing run0000_0000 timepoint 0.0872
 finished parsing run0000_0000 timepoint 0.08725
 finished parsing run0000_0000 timepoint 0.0873
 finished parsing run0000_0000 timepoint 0.08735
 finished parsing run0000_0000 timepoint 0.0874
 finished parsing run0000_0000 timepoint 0.08745
 finished parsing run0000_0000 timepoint 0.0875
 finished parsing run0000_0000 timepoint 0.08755
 finished parsing run0000_0000 timepoint 0.0876
 finished parsing run0000_0000 timepoint 0.08765
 finished parsing run0000_0000 timepoint 0.0877
 finished parsing run0000_0000 timepoint 0.08775
 finished parsing run0000_0000 timepoint 0.0878
 finished parsing run0000_0000 timepoint 0.08785
 finished parsing run0000_0000 timepoint 0.0879
 finished parsing run0000_0000 timepoint 0.08795
 finished parsing run0000_0000 timepoint 0.088
 finished parsing run0000_0000 timepoint 0.08805
 finished parsing run0000_0000 timepoint 0.0881
 finished parsing run0000_0000 timepoint 0.08815
 finished parsing run0000_0000 timepoint 0.0882
 finished parsing run0000_0000 timepoint 0.08825
 finished parsing run0000_0000 timepoint 0.0883
 finished parsing run0000_0000 timepoint 0.08835
 finished parsing run0000_0000 timepoint 0.0884
 finished parsing run0000_0000 timepoint 0.08845
 finished parsing run0000_0000 timepoint 0.0885
 finished parsing run0000_0000 timepoint 0.08855
 finished parsing run0000_0000 timepoint 0.0886
 finished parsing run0000_0000 timepoint 0.08865
 finished parsing run0000_0000 timepoint 0.0887
 finished parsing run0000_0000 timepoint 0.08875
 finished parsing run0000_0000 timepoint 0.0888
 finished parsing run0000_0000 timepoint 0.08885
 finished parsing run0000_0000 timepoint 0.0889
 finished parsing run0000_0000 timepoint 0.08895
 finished parsing run0000_0000 timepoint 0.089
 finished parsing run0000_0000 timepoint 0.08905
 finished parsing run0000_0000 timepoint 0.0891
 finished parsing run0000_0000 timepoint 0.08915
 finished parsing run0000_0000 timepoint 0.0892
 finished parsing run0000_0000 timepoint 0.08925
 finished parsing run0000_0000 timepoint 0.0893
 finished parsing run0000_0000 timepoint 0.08935
 finished parsing run0000_0000 timepoint 0.0894
 finished parsing run0000_0000 timepoint 0.08945
 finished parsing run0000_0000 timepoint 0.0895
 finished parsing run0000_0000 timepoint 0.08955
 finished parsing run0000_0000 timepoint 0.0896
 finished parsing run0000_0000 timepoint 0.08965
 finished parsing run0000_0000 timepoint 0.0897
 finished parsing run0000_0000 timepoint 0.08975
 finished parsing run0000_0000 timepoint 0.0898
 finished parsing run0000_0000 timepoint 0.08985
 finished parsing run0000_0000 timepoint 0.0899
 finished parsing run0000_0000 timepoint 0.08995
 finished parsing run0000_0000 timepoint 0.09
 finished parsing run0000_0000 timepoint 0.09005
 finished parsing run0000_0000 timepoint 0.0901
 finished parsing run0000_0000 timepoint 0.09015
 finished parsing run0000_0000 timepoint 0.0902
 finished parsing run0000_0000 timepoint 0.09025
 finished parsing run0000_0000 timepoint 0.0903
 finished parsing run0000_0000 timepoint 0.09035
 finished parsing run0000_0000 timepoint 0.0904
 finished parsing run0000_0000 timepoint 0.09045
 finished parsing run0000_0000 timepoint 0.0905
 finished parsing run0000_0000 timepoint 0.09055
 finished parsing run0000_0000 timepoint 0.0906
 finished parsing run0000_0000 timepoint 0.09065
 finished parsing run0000_0000 timepoint 0.0907
 finished parsing run0000_0000 timepoint 0.09075
 finished parsing run0000_0000 timepoint 0.0908
 finished parsing run0000_0000 timepoint 0.09085
 finished parsing run0000_0000 timepoint 0.0909
 finished parsing run0000_0000 timepoint 0.09095
 finished parsing run0000_0000 timepoint 0.091
 finished parsing run0000_0000 timepoint 0.09105
 finished parsing run0000_0000 timepoint 0.0911
 finished parsing run0000_0000 timepoint 0.09115
 finished parsing run0000_0000 timepoint 0.0912
 finished parsing run0000_0000 timepoint 0.09125
 finished parsing run0000_0000 timepoint 0.0913
 finished parsing run0000_0000 timepoint 0.09135
 finished parsing run0000_0000 timepoint 0.0914
 finished parsing run0000_0000 timepoint 0.09145
 finished parsing run0000_0000 timepoint 0.0915
 finished parsing run0000_0000 timepoint 0.09155
 finished parsing run0000_0000 timepoint 0.0916
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.07855
 finished parsing run0000_0000 timepoint 0.0786
 finished parsing run0000_0000 timepoint 0.07865
 finished parsing run0000_0000 timepoint 0.0787
 finished parsing run0000_0000 timepoint 0.07875
 finished parsing run0000_0000 timepoint 0.0788
 finished parsing run0000_0000 timepoint 0.07885
 finished parsing run0000_0000 timepoint 0.0789
 finished parsing run0000_0000 timepoint 0.07895
 finished parsing run0000_0000 timepoint 0.079
 finished parsing run0000_0000 timepoint 0.07905
 finished parsing run0000_0000 timepoint 0.0791
 finished parsing run0000_0000 timepoint 0.07915
 finished parsing run0000_0000 timepoint 0.0792
 finished parsing run0000_0000 timepoint 0.07925
 finished parsing run0000_0000 timepoint 0.0793
 finished parsing run0000_0000 timepoint 0.07935
 finished parsing run0000_0000 timepoint 0.0794
 finished parsing run0000_0000 timepoint 0.07945
 finished parsing run0000_0000 timepoint 0.0795
 finished parsing run0000_0000 timepoint 0.07955
 finished parsing run0000_0000 timepoint 0.0796
 finished parsing run0000_0000 timepoint 0.07965
 finished parsing run0000_0000 timepoint 0.0797
 finished parsing run0000_0000 timepoint 0.07975
 finished parsing run0000_0000 timepoint 0.0798
 finished parsing run0000_0000 timepoint 0.07985
 finished parsing run0000_0000 timepoint 0.0799
 finished parsing run0000_0000 timepoint 0.07995
 finished parsing run0000_0000 timepoint 0.08
 finished parsing run0000_0000 timepoint 0.08005
 finished parsing run0000_0000 timepoint 0.0801
 finished parsing run0000_0000 timepoint 0.08015
 finished parsing run0000_0000 timepoint 0.0802
 finished parsing run0000_0000 timepoint 0.08025
 finished parsing run0000_0000 timepoint 0.0803
 finished parsing run0000_0000 timepoint 0.08035
 finished parsing run0000_0000 timepoint 0.0804
 finished parsing run0000_0000 timepoint 0.08045
 finished parsing run0000_0000 timepoint 0.0805
 finished parsing run0000_0000 timepoint 0.08055
 finished parsing run0000_0000 timepoint 0.0806
 finished parsing run0000_0000 timepoint 0.08065
 finished parsing run0000_0000 timepoint 0.0807
 finished parsing run0000_0000 timepoint 0.08075
 finished parsing run0000_0000 timepoint 0.0808
 finished parsing run0000_0000 timepoint 0.08085
 finished parsing run0000_0000 timepoint 0.0809
 finished parsing run0000_0000 timepoint 0.08095
 finished parsing run0000_0000 timepoint 0.081
 finished parsing run0000_0000 timepoint 0.08105
 finished parsing run0000_0000 timepoint 0.0811
 finished parsing run0000_0000 timepoint 0.08115
 finished parsing run0000_0000 timepoint 0.0812
 finished parsing run0000_0000 timepoint 0.08125
 finished parsing run0000_0000 timepoint 0.0813
 finished parsing run0000_0000 timepoint 0.08135
 finished parsing run0000_0000 timepoint 0.0814
 finished parsing run0000_0000 timepoint 0.08145
 finished parsing run0000_0000 timepoint 0.0815
 finished parsing run0000_0000 timepoint 0.08155
 finished parsing run0000_0000 timepoint 0.0816
 finished parsing run0000_0000 timepoint 0.08165
 finished parsing run0000_0000 timepoint 0.0817
 finished parsing run0000_0000 timepoint 0.08175
 finished parsing run0000_0000 timepoint 0.0818
 finished parsing run0000_0000 timepoint 0.08185
 finished parsing run0000_0000 timepoint 0.0819
 finished parsing run0000_0000 timepoint 0.08195
 finished parsing run0000_0000 timepoint 0.082
 finished parsing run0000_0000 timepoint 0.08205
 finished parsing run0000_0000 timepoint 0.0821
 finished parsing run0000_0000 timepoint 0.08215
 finished parsing run0000_0000 timepoint 0.0822
 finished parsing run0000_0000 timepoint 0.08225
 finished parsing run0000_0000 timepoint 0.0823
 finished parsing run0000_0000 timepoint 0.08235
 finished parsing run0000_0000 timepoint 0.0824
 finished parsing run0000_0000 timepoint 0.08245
 finished parsing run0000_0000 timepoint 0.0825
 finished parsing run0000_0000 timepoint 0.08255
 finished parsing run0000_0000 timepoint 0.0826
 finished parsing run0000_0000 timepoint 0.08265
 finished parsing run0000_0000 timepoint 0.0827
 finished parsing run0000_0000 timepoint 0.08275
 finished parsing run0000_0000 timepoint 0.0828
 finished parsing run0000_0000 timepoint 0.08285
 finished parsing run0000_0000 timepoint 0.0829
 finished parsing run0000_0000 timepoint 0.08295
 finished parsing run0000_0000 timepoint 0.083
 finished parsing run0000_0000 timepoint 0.08305
 finished parsing run0000_0000 timepoint 0.0831
 finished parsing run0000_0000 timepoint 0.08315
 finished parsing run0000_0000 timepoint 0.0832
 finished parsing run0000_0000 timepoint 0.08325
 finished parsing run0000_0000 timepoint 0.0833
 finished parsing run0000_0000 timepoint 0.08335
 finished parsing run0000_0000 timepoint 0.0834
 finished parsing run0000_0000 timepoint 0.08345
 finished parsing run0000_0000 timepoint 0.0835
 finished parsing run0000_0000 timepoint 0.08355
 finished parsing run0000_0000 timepoint 0.0836
 finished parsing run0000_0000 timepoint 0.08365
 finished parsing run0000_0000 timepoint 0.0837
 finished parsing run0000_0000 timepoint 0.08375
 finished parsing run0000_0000 timepoint 0.0838
 finished parsing run0000_0000 timepoint 0.08385
 finished parsing run0000_0000 timepoint 0.0839
 finished parsing run0000_0000 timepoint 0.08395
 finished parsing run0000_0000 timepoint 0.084
 finished parsing run0000_0000 timepoint 0.08405
 finished parsing run0000_0000 timepoint 0.0841
 finished parsing run0000_0000 timepoint 0.08415
 finished parsing run0000_0000 timepoint 0.0842
 finished parsing run0000_0000 timepoint 0.08425
 finished parsing run0000_0000 timepoint 0.0843
 finished parsing run0000_0000 timepoint 0.08435
 finished parsing run0000_0000 timepoint 0.0844
 finished parsing run0000_0000 timepoint 0.08445
 finished parsing run0000_0000 timepoint 0.0845
 finished parsing run0000_0000 timepoint 0.08455
 finished parsing run0000_0000 timepoint 0.0846
 finished parsing run0000_0000 timepoint 0.08465
 finished parsing run0000_0000 timepoint 0.0847
 finished parsing run0000_0000 timepoint 0.08475
 finished parsing run0000_0000 timepoint 0.0848
 finished parsing run0000_0000 timepoint 0.08485
 finished parsing run0000_0000 timepoint 0.0849
 finished parsing run0000_0000 timepoint 0.08495
 finished parsing run0000_0000 timepoint 0.085
 finished parsing run0000_0000 timepoint 0.08505
 finished parsing run0000_0000 timepoint 0.0851
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.07195
 finished parsing run0000_0000 timepoint 0.072
 finished parsing run0000_0000 timepoint 0.07205
 finished parsing run0000_0000 timepoint 0.0721
 finished parsing run0000_0000 timepoint 0.07215
 finished parsing run0000_0000 timepoint 0.0722
 finished parsing run0000_0000 timepoint 0.07225
 finished parsing run0000_0000 timepoint 0.0723
 finished parsing run0000_0000 timepoint 0.07235
 finished parsing run0000_0000 timepoint 0.0724
 finished parsing run0000_0000 timepoint 0.07245
 finished parsing run0000_0000 timepoint 0.0725
 finished parsing run0000_0000 timepoint 0.07255
 finished parsing run0000_0000 timepoint 0.0726
 finished parsing run0000_0000 timepoint 0.07265
 finished parsing run0000_0000 timepoint 0.0727
 finished parsing run0000_0000 timepoint 0.07275
 finished parsing run0000_0000 timepoint 0.0728
 finished parsing run0000_0000 timepoint 0.07285
 finished parsing run0000_0000 timepoint 0.0729
 finished parsing run0000_0000 timepoint 0.07295
 finished parsing run0000_0000 timepoint 0.073
 finished parsing run0000_0000 timepoint 0.07305
 finished parsing run0000_0000 timepoint 0.0731
 finished parsing run0000_0000 timepoint 0.07315
 finished parsing run0000_0000 timepoint 0.0732
 finished parsing run0000_0000 timepoint 0.07325
 finished parsing run0000_0000 timepoint 0.0733
 finished parsing run0000_0000 timepoint 0.07335
 finished parsing run0000_0000 timepoint 0.0734
 finished parsing run0000_0000 timepoint 0.07345
 finished parsing run0000_0000 timepoint 0.0735
 finished parsing run0000_0000 timepoint 0.07355
 finished parsing run0000_0000 timepoint 0.0736
 finished parsing run0000_0000 timepoint 0.07365
 finished parsing run0000_0000 timepoint 0.0737
 finished parsing run0000_0000 timepoint 0.07375
 finished parsing run0000_0000 timepoint 0.0738
 finished parsing run0000_0000 timepoint 0.07385
 finished parsing run0000_0000 timepoint 0.0739
 finished parsing run0000_0000 timepoint 0.07395
 finished parsing run0000_0000 timepoint 0.074
 finished parsing run0000_0000 timepoint 0.07405
 finished parsing run0000_0000 timepoint 0.0741
 finished parsing run0000_0000 timepoint 0.07415
 finished parsing run0000_0000 timepoint 0.0742
 finished parsing run0000_0000 timepoint 0.07425
 finished parsing run0000_0000 timepoint 0.0743
 finished parsing run0000_0000 timepoint 0.07435
 finished parsing run0000_0000 timepoint 0.0744
 finished parsing run0000_0000 timepoint 0.07445
 finished parsing run0000_0000 timepoint 0.0745
 finished parsing run0000_0000 timepoint 0.07455
 finished parsing run0000_0000 timepoint 0.0746
 finished parsing run0000_0000 timepoint 0.07465
 finished parsing run0000_0000 timepoint 0.0747
 finished parsing run0000_0000 timepoint 0.07475
 finished parsing run0000_0000 timepoint 0.0748
 finished parsing run0000_0000 timepoint 0.07485
 finished parsing run0000_0000 timepoint 0.0749
 finished parsing run0000_0000 timepoint 0.07495
 finished parsing run0000_0000 timepoint 0.075
 finished parsing run0000_0000 timepoint 0.07505
 finished parsing run0000_0000 timepoint 0.0751
 finished parsing run0000_0000 timepoint 0.07515
 finished parsing run0000_0000 timepoint 0.0752
 finished parsing run0000_0000 timepoint 0.07525
 finished parsing run0000_0000 timepoint 0.0753
 finished parsing run0000_0000 timepoint 0.07535
 finished parsing run0000_0000 timepoint 0.0754
 finished parsing run0000_0000 timepoint 0.07545
 finished parsing run0000_0000 timepoint 0.0755
 finished parsing run0000_0000 timepoint 0.07555
 finished parsing run0000_0000 timepoint 0.0756
 finished parsing run0000_0000 timepoint 0.07565
 finished parsing run0000_0000 timepoint 0.0757
 finished parsing run0000_0000 timepoint 0.07575
 finished parsing run0000_0000 timepoint 0.0758
 finished parsing run0000_0000 timepoint 0.07585
 finished parsing run0000_0000 timepoint 0.0759
 finished parsing run0000_0000 timepoint 0.07595
 finished parsing run0000_0000 timepoint 0.076
 finished parsing run0000_0000 timepoint 0.07605
 finished parsing run0000_0000 timepoint 0.0761
 finished parsing run0000_0000 timepoint 0.07615
 finished parsing run0000_0000 timepoint 0.0762
 finished parsing run0000_0000 timepoint 0.07625
 finished parsing run0000_0000 timepoint 0.0763
 finished parsing run0000_0000 timepoint 0.07635
 finished parsing run0000_0000 timepoint 0.0764
 finished parsing run0000_0000 timepoint 0.07645
 finished parsing run0000_0000 timepoint 0.0765
 finished parsing run0000_0000 timepoint 0.07655
 finished parsing run0000_0000 timepoint 0.0766
 finished parsing run0000_0000 timepoint 0.07665
 finished parsing run0000_0000 timepoint 0.0767
 finished parsing run0000_0000 timepoint 0.07675
 finished parsing run0000_0000 timepoint 0.0768
 finished parsing run0000_0000 timepoint 0.07685
 finished parsing run0000_0000 timepoint 0.0769
 finished parsing run0000_0000 timepoint 0.07695
 finished parsing run0000_0000 timepoint 0.077
 finished parsing run0000_0000 timepoint 0.07705
 finished parsing run0000_0000 timepoint 0.0771
 finished parsing run0000_0000 timepoint 0.07715
 finished parsing run0000_0000 timepoint 0.0772
 finished parsing run0000_0000 timepoint 0.07725
 finished parsing run0000_0000 timepoint 0.0773
 finished parsing run0000_0000 timepoint 0.07735
 finished parsing run0000_0000 timepoint 0.0774
 finished parsing run0000_0000 timepoint 0.07745
 finished parsing run0000_0000 timepoint 0.0775
 finished parsing run0000_0000 timepoint 0.07755
 finished parsing run0000_0000 timepoint 0.0776
 finished parsing run0000_0000 timepoint 0.07765
 finished parsing run0000_0000 timepoint 0.0777
 finished parsing run0000_0000 timepoint 0.07775
 finished parsing run0000_0000 timepoint 0.0778
 finished parsing run0000_0000 timepoint 0.07785
 finished parsing run0000_0000 timepoint 0.0779
 finished parsing run0000_0000 timepoint 0.07795
 finished parsing run0000_0000 timepoint 0.078
 finished parsing run0000_0000 timepoint 0.07805
 finished parsing run0000_0000 timepoint 0.0781
 finished parsing run0000_0000 timepoint 0.07815
 finished parsing run0000_0000 timepoint 0.0782
 finished parsing run0000_0000 timepoint 0.07825
 finished parsing run0000_0000 timepoint 0.0783
 finished parsing run0000_0000 timepoint 0.07835
 finished parsing run0000_0000 timepoint 0.0784
 finished parsing run0000_0000 timepoint 0.07845
 finished parsing run0000_0000 timepoint 0.0785
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.06515
 finished parsing run0000_0000 timepoint 0.0652
 finished parsing run0000_0000 timepoint 0.06525
 finished parsing run0000_0000 timepoint 0.0653
 finished parsing run0000_0000 timepoint 0.06535
 finished parsing run0000_0000 timepoint 0.0654
 finished parsing run0000_0000 timepoint 0.06545
 finished parsing run0000_0000 timepoint 0.0655
 finished parsing run0000_0000 timepoint 0.06555
 finished parsing run0000_0000 timepoint 0.0656
 finished parsing run0000_0000 timepoint 0.06565
 finished parsing run0000_0000 timepoint 0.0657
 finished parsing run0000_0000 timepoint 0.06575
 finished parsing run0000_0000 timepoint 0.0658
 finished parsing run0000_0000 timepoint 0.06585
 finished parsing run0000_0000 timepoint 0.0659
 finished parsing run0000_0000 timepoint 0.06595
 finished parsing run0000_0000 timepoint 0.066
 finished parsing run0000_0000 timepoint 0.06605
 finished parsing run0000_0000 timepoint 0.0661
 finished parsing run0000_0000 timepoint 0.06615
 finished parsing run0000_0000 timepoint 0.0662
 finished parsing run0000_0000 timepoint 0.06625
 finished parsing run0000_0000 timepoint 0.0663
 finished parsing run0000_0000 timepoint 0.06635
 finished parsing run0000_0000 timepoint 0.0664
 finished parsing run0000_0000 timepoint 0.06645
 finished parsing run0000_0000 timepoint 0.0665
 finished parsing run0000_0000 timepoint 0.06655
 finished parsing run0000_0000 timepoint 0.0666
 finished parsing run0000_0000 timepoint 0.06665
 finished parsing run0000_0000 timepoint 0.0667
 finished parsing run0000_0000 timepoint 0.06675
 finished parsing run0000_0000 timepoint 0.0668
 finished parsing run0000_0000 timepoint 0.06685
 finished parsing run0000_0000 timepoint 0.0669
 finished parsing run0000_0000 timepoint 0.06695
 finished parsing run0000_0000 timepoint 0.067
 finished parsing run0000_0000 timepoint 0.06705
 finished parsing run0000_0000 timepoint 0.0671
 finished parsing run0000_0000 timepoint 0.06715
 finished parsing run0000_0000 timepoint 0.0672
 finished parsing run0000_0000 timepoint 0.06725
 finished parsing run0000_0000 timepoint 0.0673
 finished parsing run0000_0000 timepoint 0.06735
 finished parsing run0000_0000 timepoint 0.0674
 finished parsing run0000_0000 timepoint 0.06745
 finished parsing run0000_0000 timepoint 0.0675
 finished parsing run0000_0000 timepoint 0.06755
 finished parsing run0000_0000 timepoint 0.0676
 finished parsing run0000_0000 timepoint 0.06765
 finished parsing run0000_0000 timepoint 0.0677
 finished parsing run0000_0000 timepoint 0.06775
 finished parsing run0000_0000 timepoint 0.0678
 finished parsing run0000_0000 timepoint 0.06785
 finished parsing run0000_0000 timepoint 0.0679
 finished parsing run0000_0000 timepoint 0.06795
 finished parsing run0000_0000 timepoint 0.068
 finished parsing run0000_0000 timepoint 0.06805
 finished parsing run0000_0000 timepoint 0.0681
 finished parsing run0000_0000 timepoint 0.06815
 finished parsing run0000_0000 timepoint 0.0682
 finished parsing run0000_0000 timepoint 0.06825
 finished parsing run0000_0000 timepoint 0.0683
 finished parsing run0000_0000 timepoint 0.06835
 finished parsing run0000_0000 timepoint 0.0684
 finished parsing run0000_0000 timepoint 0.06845
 finished parsing run0000_0000 timepoint 0.0685
 finished parsing run0000_0000 timepoint 0.06855
 finished parsing run0000_0000 timepoint 0.0686
 finished parsing run0000_0000 timepoint 0.06865
 finished parsing run0000_0000 timepoint 0.0687
 finished parsing run0000_0000 timepoint 0.06875
 finished parsing run0000_0000 timepoint 0.0688
 finished parsing run0000_0000 timepoint 0.06885
 finished parsing run0000_0000 timepoint 0.0689
 finished parsing run0000_0000 timepoint 0.06895
 finished parsing run0000_0000 timepoint 0.069
 finished parsing run0000_0000 timepoint 0.06905
 finished parsing run0000_0000 timepoint 0.0691
 finished parsing run0000_0000 timepoint 0.06915
 finished parsing run0000_0000 timepoint 0.0692
 finished parsing run0000_0000 timepoint 0.06925
 finished parsing run0000_0000 timepoint 0.0693
 finished parsing run0000_0000 timepoint 0.06935
 finished parsing run0000_0000 timepoint 0.0694
 finished parsing run0000_0000 timepoint 0.06945
 finished parsing run0000_0000 timepoint 0.0695
 finished parsing run0000_0000 timepoint 0.06955
 finished parsing run0000_0000 timepoint 0.0696
 finished parsing run0000_0000 timepoint 0.06965
 finished parsing run0000_0000 timepoint 0.0697
 finished parsing run0000_0000 timepoint 0.06975
 finished parsing run0000_0000 timepoint 0.0698
 finished parsing run0000_0000 timepoint 0.06985
 finished parsing run0000_0000 timepoint 0.0699
 finished parsing run0000_0000 timepoint 0.06995
 finished parsing run0000_0000 timepoint 0.07
 finished parsing run0000_0000 timepoint 0.07005
 finished parsing run0000_0000 timepoint 0.0701
 finished parsing run0000_0000 timepoint 0.07015
 finished parsing run0000_0000 timepoint 0.0702
 finished parsing run0000_0000 timepoint 0.07025
 finished parsing run0000_0000 timepoint 0.0703
 finished parsing run0000_0000 timepoint 0.07035
 finished parsing run0000_0000 timepoint 0.0704
 finished parsing run0000_0000 timepoint 0.07045
 finished parsing run0000_0000 timepoint 0.0705
 finished parsing run0000_0000 timepoint 0.07055
 finished parsing run0000_0000 timepoint 0.0706
 finished parsing run0000_0000 timepoint 0.07065
 finished parsing run0000_0000 timepoint 0.0707
 finished parsing run0000_0000 timepoint 0.07075
 finished parsing run0000_0000 timepoint 0.0708
 finished parsing run0000_0000 timepoint 0.07085
 finished parsing run0000_0000 timepoint 0.0709
 finished parsing run0000_0000 timepoint 0.07095
 finished parsing run0000_0000 timepoint 0.071
 finished parsing run0000_0000 timepoint 0.07105
 finished parsing run0000_0000 timepoint 0.0711
 finished parsing run0000_0000 timepoint 0.07115
 finished parsing run0000_0000 timepoint 0.0712
 finished parsing run0000_0000 timepoint 0.07125
 finished parsing run0000_0000 timepoint 0.0713
 finished parsing run0000_0000 timepoint 0.07135
 finished parsing run0000_0000 timepoint 0.0714
 finished parsing run0000_0000 timepoint 0.07145
 finished parsing run0000_0000 timepoint 0.0715
 finished parsing run0000_0000 timepoint 0.07155
 finished parsing run0000_0000 timepoint 0.0716
 finished parsing run0000_0000 timepoint 0.07165
 finished parsing run0000_0000 timepoint 0.0717
 finished parsing run0000_0000 timepoint 0.07175
 finished parsing run0000_0000 timepoint 0.0718
 finished parsing run0000_0000 timepoint 0.07185
 finished parsing run0000_0000 timepoint 0.0719
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.05855
 finished parsing run0000_0000 timepoint 0.0586
 finished parsing run0000_0000 timepoint 0.05865
 finished parsing run0000_0000 timepoint 0.0587
 finished parsing run0000_0000 timepoint 0.05875
 finished parsing run0000_0000 timepoint 0.0588
 finished parsing run0000_0000 timepoint 0.05885
 finished parsing run0000_0000 timepoint 0.0589
 finished parsing run0000_0000 timepoint 0.05895
 finished parsing run0000_0000 timepoint 0.059
 finished parsing run0000_0000 timepoint 0.05905
 finished parsing run0000_0000 timepoint 0.0591
 finished parsing run0000_0000 timepoint 0.05915
 finished parsing run0000_0000 timepoint 0.0592
 finished parsing run0000_0000 timepoint 0.05925
 finished parsing run0000_0000 timepoint 0.0593
 finished parsing run0000_0000 timepoint 0.05935
 finished parsing run0000_0000 timepoint 0.0594
 finished parsing run0000_0000 timepoint 0.05945
 finished parsing run0000_0000 timepoint 0.0595
 finished parsing run0000_0000 timepoint 0.05955
 finished parsing run0000_0000 timepoint 0.0596
 finished parsing run0000_0000 timepoint 0.05965
 finished parsing run0000_0000 timepoint 0.0597
 finished parsing run0000_0000 timepoint 0.05975
 finished parsing run0000_0000 timepoint 0.0598
 finished parsing run0000_0000 timepoint 0.05985
 finished parsing run0000_0000 timepoint 0.0599
 finished parsing run0000_0000 timepoint 0.05995
 finished parsing run0000_0000 timepoint 0.06
 finished parsing run0000_0000 timepoint 0.06005
 finished parsing run0000_0000 timepoint 0.0601
 finished parsing run0000_0000 timepoint 0.06015
 finished parsing run0000_0000 timepoint 0.0602
 finished parsing run0000_0000 timepoint 0.06025
 finished parsing run0000_0000 timepoint 0.0603
 finished parsing run0000_0000 timepoint 0.06035
 finished parsing run0000_0000 timepoint 0.0604
 finished parsing run0000_0000 timepoint 0.06045
 finished parsing run0000_0000 timepoint 0.0605
 finished parsing run0000_0000 timepoint 0.06055
 finished parsing run0000_0000 timepoint 0.0606
 finished parsing run0000_0000 timepoint 0.06065
 finished parsing run0000_0000 timepoint 0.0607
 finished parsing run0000_0000 timepoint 0.06075
 finished parsing run0000_0000 timepoint 0.0608
 finished parsing run0000_0000 timepoint 0.06085
 finished parsing run0000_0000 timepoint 0.0609
 finished parsing run0000_0000 timepoint 0.06095
 finished parsing run0000_0000 timepoint 0.061
 finished parsing run0000_0000 timepoint 0.06105
 finished parsing run0000_0000 timepoint 0.0611
 finished parsing run0000_0000 timepoint 0.06115
 finished parsing run0000_0000 timepoint 0.0612
 finished parsing run0000_0000 timepoint 0.06125
 finished parsing run0000_0000 timepoint 0.0613
 finished parsing run0000_0000 timepoint 0.06135
 finished parsing run0000_0000 timepoint 0.0614
 finished parsing run0000_0000 timepoint 0.06145
 finished parsing run0000_0000 timepoint 0.0615
 finished parsing run0000_0000 timepoint 0.06155
 finished parsing run0000_0000 timepoint 0.0616
 finished parsing run0000_0000 timepoint 0.06165
 finished parsing run0000_0000 timepoint 0.0617
 finished parsing run0000_0000 timepoint 0.06175
 finished parsing run0000_0000 timepoint 0.0618
 finished parsing run0000_0000 timepoint 0.06185
 finished parsing run0000_0000 timepoint 0.0619
 finished parsing run0000_0000 timepoint 0.06195
 finished parsing run0000_0000 timepoint 0.062
 finished parsing run0000_0000 timepoint 0.06205
 finished parsing run0000_0000 timepoint 0.0621
 finished parsing run0000_0000 timepoint 0.06215
 finished parsing run0000_0000 timepoint 0.0622
 finished parsing run0000_0000 timepoint 0.06225
 finished parsing run0000_0000 timepoint 0.0623
 finished parsing run0000_0000 timepoint 0.06235
 finished parsing run0000_0000 timepoint 0.0624
 finished parsing run0000_0000 timepoint 0.06245
 finished parsing run0000_0000 timepoint 0.0625
 finished parsing run0000_0000 timepoint 0.06255
 finished parsing run0000_0000 timepoint 0.0626
 finished parsing run0000_0000 timepoint 0.06265
 finished parsing run0000_0000 timepoint 0.0627
 finished parsing run0000_0000 timepoint 0.06275
 finished parsing run0000_0000 timepoint 0.0628
 finished parsing run0000_0000 timepoint 0.06285
 finished parsing run0000_0000 timepoint 0.0629
 finished parsing run0000_0000 timepoint 0.06295
 finished parsing run0000_0000 timepoint 0.063
 finished parsing run0000_0000 timepoint 0.06305
 finished parsing run0000_0000 timepoint 0.0631
 finished parsing run0000_0000 timepoint 0.06315
 finished parsing run0000_0000 timepoint 0.0632
 finished parsing run0000_0000 timepoint 0.06325
 finished parsing run0000_0000 timepoint 0.0633
 finished parsing run0000_0000 timepoint 0.06335
 finished parsing run0000_0000 timepoint 0.0634
 finished parsing run0000_0000 timepoint 0.06345
 finished parsing run0000_0000 timepoint 0.0635
 finished parsing run0000_0000 timepoint 0.06355
 finished parsing run0000_0000 timepoint 0.0636
 finished parsing run0000_0000 timepoint 0.06365
 finished parsing run0000_0000 timepoint 0.0637
 finished parsing run0000_0000 timepoint 0.06375
 finished parsing run0000_0000 timepoint 0.0638
 finished parsing run0000_0000 timepoint 0.06385
 finished parsing run0000_0000 timepoint 0.0639
 finished parsing run0000_0000 timepoint 0.06395
 finished parsing run0000_0000 timepoint 0.064
 finished parsing run0000_0000 timepoint 0.06405
 finished parsing run0000_0000 timepoint 0.0641
 finished parsing run0000_0000 timepoint 0.06415
 finished parsing run0000_0000 timepoint 0.0642
 finished parsing run0000_0000 timepoint 0.06425
 finished parsing run0000_0000 timepoint 0.0643
 finished parsing run0000_0000 timepoint 0.06435
 finished parsing run0000_0000 timepoint 0.0644
 finished parsing run0000_0000 timepoint 0.06445
 finished parsing run0000_0000 timepoint 0.0645
 finished parsing run0000_0000 timepoint 0.06455
 finished parsing run0000_0000 timepoint 0.0646
 finished parsing run0000_0000 timepoint 0.06465
 finished parsing run0000_0000 timepoint 0.0647
 finished parsing run0000_0000 timepoint 0.06475
 finished parsing run0000_0000 timepoint 0.0648
 finished parsing run0000_0000 timepoint 0.06485
 finished parsing run0000_0000 timepoint 0.0649
 finished parsing run0000_0000 timepoint 0.06495
 finished parsing run0000_0000 timepoint 0.065
 finished parsing run0000_0000 timepoint 0.06505
 finished parsing run0000_0000 timepoint 0.0651
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.0517
 finished parsing run0000_0000 timepoint 0.05175
 finished parsing run0000_0000 timepoint 0.0518
 finished parsing run0000_0000 timepoint 0.05185
 finished parsing run0000_0000 timepoint 0.0519
 finished parsing run0000_0000 timepoint 0.05195
 finished parsing run0000_0000 timepoint 0.052
 finished parsing run0000_0000 timepoint 0.05205
 finished parsing run0000_0000 timepoint 0.0521
 finished parsing run0000_0000 timepoint 0.05215
 finished parsing run0000_0000 timepoint 0.0522
 finished parsing run0000_0000 timepoint 0.05225
 finished parsing run0000_0000 timepoint 0.0523
 finished parsing run0000_0000 timepoint 0.05235
 finished parsing run0000_0000 timepoint 0.0524
 finished parsing run0000_0000 timepoint 0.05245
 finished parsing run0000_0000 timepoint 0.0525
 finished parsing run0000_0000 timepoint 0.05255
 finished parsing run0000_0000 timepoint 0.0526
 finished parsing run0000_0000 timepoint 0.05265
 finished parsing run0000_0000 timepoint 0.0527
 finished parsing run0000_0000 timepoint 0.05275
 finished parsing run0000_0000 timepoint 0.0528
 finished parsing run0000_0000 timepoint 0.05285
 finished parsing run0000_0000 timepoint 0.0529
 finished parsing run0000_0000 timepoint 0.05295
 finished parsing run0000_0000 timepoint 0.053
 finished parsing run0000_0000 timepoint 0.05305
 finished parsing run0000_0000 timepoint 0.0531
 finished parsing run0000_0000 timepoint 0.05315
 finished parsing run0000_0000 timepoint 0.0532
 finished parsing run0000_0000 timepoint 0.05325
 finished parsing run0000_0000 timepoint 0.0533
 finished parsing run0000_0000 timepoint 0.05335
 finished parsing run0000_0000 timepoint 0.0534
 finished parsing run0000_0000 timepoint 0.05345
 finished parsing run0000_0000 timepoint 0.0535
 finished parsing run0000_0000 timepoint 0.05355
 finished parsing run0000_0000 timepoint 0.0536
 finished parsing run0000_0000 timepoint 0.05365
 finished parsing run0000_0000 timepoint 0.0537
 finished parsing run0000_0000 timepoint 0.05375
 finished parsing run0000_0000 timepoint 0.0538
 finished parsing run0000_0000 timepoint 0.05385
 finished parsing run0000_0000 timepoint 0.0539
 finished parsing run0000_0000 timepoint 0.05395
 finished parsing run0000_0000 timepoint 0.054
 finished parsing run0000_0000 timepoint 0.05405
 finished parsing run0000_0000 timepoint 0.0541
 finished parsing run0000_0000 timepoint 0.05415
 finished parsing run0000_0000 timepoint 0.0542
 finished parsing run0000_0000 timepoint 0.05425
 finished parsing run0000_0000 timepoint 0.0543
 finished parsing run0000_0000 timepoint 0.05435
 finished parsing run0000_0000 timepoint 0.0544
 finished parsing run0000_0000 timepoint 0.05445
 finished parsing run0000_0000 timepoint 0.0545
 finished parsing run0000_0000 timepoint 0.05455
 finished parsing run0000_0000 timepoint 0.0546
 finished parsing run0000_0000 timepoint 0.05465
 finished parsing run0000_0000 timepoint 0.0547
 finished parsing run0000_0000 timepoint 0.05475
 finished parsing run0000_0000 timepoint 0.0548
 finished parsing run0000_0000 timepoint 0.05485
 finished parsing run0000_0000 timepoint 0.0549
 finished parsing run0000_0000 timepoint 0.05495
 finished parsing run0000_0000 timepoint 0.055
 finished parsing run0000_0000 timepoint 0.05505
 finished parsing run0000_0000 timepoint 0.0551
 finished parsing run0000_0000 timepoint 0.05515
 finished parsing run0000_0000 timepoint 0.0552
 finished parsing run0000_0000 timepoint 0.05525
 finished parsing run0000_0000 timepoint 0.0553
 finished parsing run0000_0000 timepoint 0.05535
 finished parsing run0000_0000 timepoint 0.0554
 finished parsing run0000_0000 timepoint 0.05545
 finished parsing run0000_0000 timepoint 0.0555
 finished parsing run0000_0000 timepoint 0.05555
 finished parsing run0000_0000 timepoint 0.0556
 finished parsing run0000_0000 timepoint 0.05565
 finished parsing run0000_0000 timepoint 0.0557
 finished parsing run0000_0000 timepoint 0.05575
 finished parsing run0000_0000 timepoint 0.0558
 finished parsing run0000_0000 timepoint 0.05585
 finished parsing run0000_0000 timepoint 0.0559
 finished parsing run0000_0000 timepoint 0.05595
 finished parsing run0000_0000 timepoint 0.056
 finished parsing run0000_0000 timepoint 0.05605
 finished parsing run0000_0000 timepoint 0.0561
 finished parsing run0000_0000 timepoint 0.05615
 finished parsing run0000_0000 timepoint 0.0562
 finished parsing run0000_0000 timepoint 0.05625
 finished parsing run0000_0000 timepoint 0.0563
 finished parsing run0000_0000 timepoint 0.05635
 finished parsing run0000_0000 timepoint 0.0564
 finished parsing run0000_0000 timepoint 0.05645
 finished parsing run0000_0000 timepoint 0.0565
 finished parsing run0000_0000 timepoint 0.05655
 finished parsing run0000_0000 timepoint 0.0566
 finished parsing run0000_0000 timepoint 0.05665
 finished parsing run0000_0000 timepoint 0.0567
 finished parsing run0000_0000 timepoint 0.05675
 finished parsing run0000_0000 timepoint 0.0568
 finished parsing run0000_0000 timepoint 0.05685
 finished parsing run0000_0000 timepoint 0.0569
 finished parsing run0000_0000 timepoint 0.05695
 finished parsing run0000_0000 timepoint 0.057
 finished parsing run0000_0000 timepoint 0.05705
 finished parsing run0000_0000 timepoint 0.0571
 finished parsing run0000_0000 timepoint 0.05715
 finished parsing run0000_0000 timepoint 0.0572
 finished parsing run0000_0000 timepoint 0.05725
 finished parsing run0000_0000 timepoint 0.0573
 finished parsing run0000_0000 timepoint 0.05735
 finished parsing run0000_0000 timepoint 0.0574
 finished parsing run0000_0000 timepoint 0.05745
 finished parsing run0000_0000 timepoint 0.0575
 finished parsing run0000_0000 timepoint 0.05755
 finished parsing run0000_0000 timepoint 0.0576
 finished parsing run0000_0000 timepoint 0.05765
 finished parsing run0000_0000 timepoint 0.0577
 finished parsing run0000_0000 timepoint 0.05775
 finished parsing run0000_0000 timepoint 0.0578
 finished parsing run0000_0000 timepoint 0.05785
 finished parsing run0000_0000 timepoint 0.0579
 finished parsing run0000_0000 timepoint 0.05795
 finished parsing run0000_0000 timepoint 0.058
 finished parsing run0000_0000 timepoint 0.05805
 finished parsing run0000_0000 timepoint 0.0581
 finished parsing run0000_0000 timepoint 0.05815
 finished parsing run0000_0000 timepoint 0.0582
 finished parsing run0000_0000 timepoint 0.05825
 finished parsing run0000_0000 timepoint 0.0583
 finished parsing run0000_0000 timepoint 0.05835
 finished parsing run0000_0000 timepoint 0.0584
 finished parsing run0000_0000 timepoint 0.05845
 finished parsing run0000_0000 timepoint 0.0585
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.04525
 finished parsing run0000_0000 timepoint 0.0453
 finished parsing run0000_0000 timepoint 0.04535
 finished parsing run0000_0000 timepoint 0.0454
 finished parsing run0000_0000 timepoint 0.04545
 finished parsing run0000_0000 timepoint 0.0455
 finished parsing run0000_0000 timepoint 0.04555
 finished parsing run0000_0000 timepoint 0.0456
 finished parsing run0000_0000 timepoint 0.04565
 finished parsing run0000_0000 timepoint 0.0457
 finished parsing run0000_0000 timepoint 0.04575
 finished parsing run0000_0000 timepoint 0.0458
 finished parsing run0000_0000 timepoint 0.04585
 finished parsing run0000_0000 timepoint 0.0459
 finished parsing run0000_0000 timepoint 0.04595
 finished parsing run0000_0000 timepoint 0.046
 finished parsing run0000_0000 timepoint 0.04605
 finished parsing run0000_0000 timepoint 0.0461
 finished parsing run0000_0000 timepoint 0.04615
 finished parsing run0000_0000 timepoint 0.0462
 finished parsing run0000_0000 timepoint 0.04625
 finished parsing run0000_0000 timepoint 0.0463
 finished parsing run0000_0000 timepoint 0.04635
 finished parsing run0000_0000 timepoint 0.0464
 finished parsing run0000_0000 timepoint 0.04645
 finished parsing run0000_0000 timepoint 0.0465
 finished parsing run0000_0000 timepoint 0.04655
 finished parsing run0000_0000 timepoint 0.0466
 finished parsing run0000_0000 timepoint 0.04665
 finished parsing run0000_0000 timepoint 0.0467
 finished parsing run0000_0000 timepoint 0.04675
 finished parsing run0000_0000 timepoint 0.0468
 finished parsing run0000_0000 timepoint 0.04685
 finished parsing run0000_0000 timepoint 0.0469
 finished parsing run0000_0000 timepoint 0.04695
 finished parsing run0000_0000 timepoint 0.047
 finished parsing run0000_0000 timepoint 0.04705
 finished parsing run0000_0000 timepoint 0.0471
 finished parsing run0000_0000 timepoint 0.04715
 finished parsing run0000_0000 timepoint 0.0472
 finished parsing run0000_0000 timepoint 0.04725
 finished parsing run0000_0000 timepoint 0.0473
 finished parsing run0000_0000 timepoint 0.04735
 finished parsing run0000_0000 timepoint 0.0474
 finished parsing run0000_0000 timepoint 0.04745
 finished parsing run0000_0000 timepoint 0.0475
 finished parsing run0000_0000 timepoint 0.04755
 finished parsing run0000_0000 timepoint 0.0476
 finished parsing run0000_0000 timepoint 0.04765
 finished parsing run0000_0000 timepoint 0.0477
 finished parsing run0000_0000 timepoint 0.04775
 finished parsing run0000_0000 timepoint 0.0478
 finished parsing run0000_0000 timepoint 0.04785
 finished parsing run0000_0000 timepoint 0.0479
 finished parsing run0000_0000 timepoint 0.04795
 finished parsing run0000_0000 timepoint 0.048
 finished parsing run0000_0000 timepoint 0.04805
 finished parsing run0000_0000 timepoint 0.0481
 finished parsing run0000_0000 timepoint 0.04815
 finished parsing run0000_0000 timepoint 0.0482
 finished parsing run0000_0000 timepoint 0.04825
 finished parsing run0000_0000 timepoint 0.0483
 finished parsing run0000_0000 timepoint 0.04835
 finished parsing run0000_0000 timepoint 0.0484
 finished parsing run0000_0000 timepoint 0.04845
 finished parsing run0000_0000 timepoint 0.0485
 finished parsing run0000_0000 timepoint 0.04855
 finished parsing run0000_0000 timepoint 0.0486
 finished parsing run0000_0000 timepoint 0.04865
 finished parsing run0000_0000 timepoint 0.0487
 finished parsing run0000_0000 timepoint 0.04875
 finished parsing run0000_0000 timepoint 0.0488
 finished parsing run0000_0000 timepoint 0.04885
 finished parsing run0000_0000 timepoint 0.0489
 finished parsing run0000_0000 timepoint 0.04895
 finished parsing run0000_0000 timepoint 0.049
 finished parsing run0000_0000 timepoint 0.04905
 finished parsing run0000_0000 timepoint 0.0491
 finished parsing run0000_0000 timepoint 0.04915
 finished parsing run0000_0000 timepoint 0.0492
 finished parsing run0000_0000 timepoint 0.04925
 finished parsing run0000_0000 timepoint 0.0493
 finished parsing run0000_0000 timepoint 0.04935
 finished parsing run0000_0000 timepoint 0.0494
 finished parsing run0000_0000 timepoint 0.04945
 finished parsing run0000_0000 timepoint 0.0495
 finished parsing run0000_0000 timepoint 0.04955
 finished parsing run0000_0000 timepoint 0.0496
 finished parsing run0000_0000 timepoint 0.04965
 finished parsing run0000_0000 timepoint 0.0497
 finished parsing run0000_0000 timepoint 0.04975
 finished parsing run0000_0000 timepoint 0.0498
 finished parsing run0000_0000 timepoint 0.04985
 finished parsing run0000_0000 timepoint 0.0499
 finished parsing run0000_0000 timepoint 0.04995
 finished parsing run0000_0000 timepoint 0.05
 finished parsing run0000_0000 timepoint 0.05005
 finished parsing run0000_0000 timepoint 0.0501
 finished parsing run0000_0000 timepoint 0.05015
 finished parsing run0000_0000 timepoint 0.0502
 finished parsing run0000_0000 timepoint 0.05025
 finished parsing run0000_0000 timepoint 0.0503
 finished parsing run0000_0000 timepoint 0.05035
 finished parsing run0000_0000 timepoint 0.0504
 finished parsing run0000_0000 timepoint 0.05045
 finished parsing run0000_0000 timepoint 0.0505
 finished parsing run0000_0000 timepoint 0.05055
 finished parsing run0000_0000 timepoint 0.0506
 finished parsing run0000_0000 timepoint 0.05065
 finished parsing run0000_0000 timepoint 0.0507
 finished parsing run0000_0000 timepoint 0.05075
 finished parsing run0000_0000 timepoint 0.0508
 finished parsing run0000_0000 timepoint 0.05085
 finished parsing run0000_0000 timepoint 0.0509
 finished parsing run0000_0000 timepoint 0.05095
 finished parsing run0000_0000 timepoint 0.051
 finished parsing run0000_0000 timepoint 0.05105
 finished parsing run0000_0000 timepoint 0.0511
 finished parsing run0000_0000 timepoint 0.05115
 finished parsing run0000_0000 timepoint 0.0512
 finished parsing run0000_0000 timepoint 0.05125
 finished parsing run0000_0000 timepoint 0.0513
 finished parsing run0000_0000 timepoint 0.05135
 finished parsing run0000_0000 timepoint 0.0514
 finished parsing run0000_0000 timepoint 0.05145
 finished parsing run0000_0000 timepoint 0.0515
 finished parsing run0000_0000 timepoint 0.05155
 finished parsing run0000_0000 timepoint 0.0516
 finished parsing run0000_0000 timepoint 0.05165
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.03875
 finished parsing run0000_0000 timepoint 0.0388
 finished parsing run0000_0000 timepoint 0.03885
 finished parsing run0000_0000 timepoint 0.0389
 finished parsing run0000_0000 timepoint 0.03895
 finished parsing run0000_0000 timepoint 0.039
 finished parsing run0000_0000 timepoint 0.03905
 finished parsing run0000_0000 timepoint 0.0391
 finished parsing run0000_0000 timepoint 0.03915
 finished parsing run0000_0000 timepoint 0.0392
 finished parsing run0000_0000 timepoint 0.03925
 finished parsing run0000_0000 timepoint 0.0393
 finished parsing run0000_0000 timepoint 0.03935
 finished parsing run0000_0000 timepoint 0.0394
 finished parsing run0000_0000 timepoint 0.03945
 finished parsing run0000_0000 timepoint 0.0395
 finished parsing run0000_0000 timepoint 0.03955
 finished parsing run0000_0000 timepoint 0.0396
 finished parsing run0000_0000 timepoint 0.03965
 finished parsing run0000_0000 timepoint 0.0397
 finished parsing run0000_0000 timepoint 0.03975
 finished parsing run0000_0000 timepoint 0.0398
 finished parsing run0000_0000 timepoint 0.03985
 finished parsing run0000_0000 timepoint 0.0399
 finished parsing run0000_0000 timepoint 0.03995
 finished parsing run0000_0000 timepoint 0.04
 finished parsing run0000_0000 timepoint 0.04005
 finished parsing run0000_0000 timepoint 0.0401
 finished parsing run0000_0000 timepoint 0.04015
 finished parsing run0000_0000 timepoint 0.0402
 finished parsing run0000_0000 timepoint 0.04025
 finished parsing run0000_0000 timepoint 0.0403
 finished parsing run0000_0000 timepoint 0.04035
 finished parsing run0000_0000 timepoint 0.0404
 finished parsing run0000_0000 timepoint 0.04045
 finished parsing run0000_0000 timepoint 0.0405
 finished parsing run0000_0000 timepoint 0.04055
 finished parsing run0000_0000 timepoint 0.0406
 finished parsing run0000_0000 timepoint 0.04065
 finished parsing run0000_0000 timepoint 0.0407
 finished parsing run0000_0000 timepoint 0.04075
 finished parsing run0000_0000 timepoint 0.0408
 finished parsing run0000_0000 timepoint 0.04085
 finished parsing run0000_0000 timepoint 0.0409
 finished parsing run0000_0000 timepoint 0.04095
 finished parsing run0000_0000 timepoint 0.041
 finished parsing run0000_0000 timepoint 0.04105
 finished parsing run0000_0000 timepoint 0.0411
 finished parsing run0000_0000 timepoint 0.04115
 finished parsing run0000_0000 timepoint 0.0412
 finished parsing run0000_0000 timepoint 0.04125
 finished parsing run0000_0000 timepoint 0.0413
 finished parsing run0000_0000 timepoint 0.04135
 finished parsing run0000_0000 timepoint 0.0414
 finished parsing run0000_0000 timepoint 0.04145
 finished parsing run0000_0000 timepoint 0.0415
 finished parsing run0000_0000 timepoint 0.04155
 finished parsing run0000_0000 timepoint 0.0416
 finished parsing run0000_0000 timepoint 0.04165
 finished parsing run0000_0000 timepoint 0.0417
 finished parsing run0000_0000 timepoint 0.04175
 finished parsing run0000_0000 timepoint 0.0418
 finished parsing run0000_0000 timepoint 0.04185
 finished parsing run0000_0000 timepoint 0.0419
 finished parsing run0000_0000 timepoint 0.04195
 finished parsing run0000_0000 timepoint 0.042
 finished parsing run0000_0000 timepoint 0.04205
 finished parsing run0000_0000 timepoint 0.0421
 finished parsing run0000_0000 timepoint 0.04215
 finished parsing run0000_0000 timepoint 0.0422
 finished parsing run0000_0000 timepoint 0.04225
 finished parsing run0000_0000 timepoint 0.0423
 finished parsing run0000_0000 timepoint 0.04235
 finished parsing run0000_0000 timepoint 0.0424
 finished parsing run0000_0000 timepoint 0.04245
 finished parsing run0000_0000 timepoint 0.0425
 finished parsing run0000_0000 timepoint 0.04255
 finished parsing run0000_0000 timepoint 0.0426
 finished parsing run0000_0000 timepoint 0.04265
 finished parsing run0000_0000 timepoint 0.0427
 finished parsing run0000_0000 timepoint 0.04275
 finished parsing run0000_0000 timepoint 0.0428
 finished parsing run0000_0000 timepoint 0.04285
 finished parsing run0000_0000 timepoint 0.0429
 finished parsing run0000_0000 timepoint 0.04295
 finished parsing run0000_0000 timepoint 0.043
 finished parsing run0000_0000 timepoint 0.04305
 finished parsing run0000_0000 timepoint 0.0431
 finished parsing run0000_0000 timepoint 0.04315
 finished parsing run0000_0000 timepoint 0.0432
 finished parsing run0000_0000 timepoint 0.04325
 finished parsing run0000_0000 timepoint 0.0433
 finished parsing run0000_0000 timepoint 0.04335
 finished parsing run0000_0000 timepoint 0.0434
 finished parsing run0000_0000 timepoint 0.04345
 finished parsing run0000_0000 timepoint 0.0435
 finished parsing run0000_0000 timepoint 0.04355
 finished parsing run0000_0000 timepoint 0.0436
 finished parsing run0000_0000 timepoint 0.04365
 finished parsing run0000_0000 timepoint 0.0437
 finished parsing run0000_0000 timepoint 0.04375
 finished parsing run0000_0000 timepoint 0.0438
 finished parsing run0000_0000 timepoint 0.04385
 finished parsing run0000_0000 timepoint 0.0439
 finished parsing run0000_0000 timepoint 0.04395
 finished parsing run0000_0000 timepoint 0.044
 finished parsing run0000_0000 timepoint 0.04405
 finished parsing run0000_0000 timepoint 0.0441
 finished parsing run0000_0000 timepoint 0.04415
 finished parsing run0000_0000 timepoint 0.0442
 finished parsing run0000_0000 timepoint 0.04425
 finished parsing run0000_0000 timepoint 0.0443
 finished parsing run0000_0000 timepoint 0.04435
 finished parsing run0000_0000 timepoint 0.0444
 finished parsing run0000_0000 timepoint 0.04445
 finished parsing run0000_0000 timepoint 0.0445
 finished parsing run0000_0000 timepoint 0.04455
 finished parsing run0000_0000 timepoint 0.0446
 finished parsing run0000_0000 timepoint 0.04465
 finished parsing run0000_0000 timepoint 0.0447
 finished parsing run0000_0000 timepoint 0.04475
 finished parsing run0000_0000 timepoint 0.0448
 finished parsing run0000_0000 timepoint 0.04485
 finished parsing run0000_0000 timepoint 0.0449
 finished parsing run0000_0000 timepoint 0.04495
 finished parsing run0000_0000 timepoint 0.045
 finished parsing run0000_0000 timepoint 0.04505
 finished parsing run0000_0000 timepoint 0.0451
 finished parsing run0000_0000 timepoint 0.04515
 finished parsing run0000_0000 timepoint 0.0452
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.032
 finished parsing run0000_0000 timepoint 0.03205
 finished parsing run0000_0000 timepoint 0.0321
 finished parsing run0000_0000 timepoint 0.03215
 finished parsing run0000_0000 timepoint 0.0322
 finished parsing run0000_0000 timepoint 0.03225
 finished parsing run0000_0000 timepoint 0.0323
 finished parsing run0000_0000 timepoint 0.03235
 finished parsing run0000_0000 timepoint 0.0324
 finished parsing run0000_0000 timepoint 0.03245
 finished parsing run0000_0000 timepoint 0.0325
 finished parsing run0000_0000 timepoint 0.03255
 finished parsing run0000_0000 timepoint 0.0326
 finished parsing run0000_0000 timepoint 0.03265
 finished parsing run0000_0000 timepoint 0.0327
 finished parsing run0000_0000 timepoint 0.03275
 finished parsing run0000_0000 timepoint 0.0328
 finished parsing run0000_0000 timepoint 0.03285
 finished parsing run0000_0000 timepoint 0.0329
 finished parsing run0000_0000 timepoint 0.03295
 finished parsing run0000_0000 timepoint 0.033
 finished parsing run0000_0000 timepoint 0.03305
 finished parsing run0000_0000 timepoint 0.0331
 finished parsing run0000_0000 timepoint 0.03315
 finished parsing run0000_0000 timepoint 0.0332
 finished parsing run0000_0000 timepoint 0.03325
 finished parsing run0000_0000 timepoint 0.0333
 finished parsing run0000_0000 timepoint 0.03335
 finished parsing run0000_0000 timepoint 0.0334
 finished parsing run0000_0000 timepoint 0.03345
 finished parsing run0000_0000 timepoint 0.0335
 finished parsing run0000_0000 timepoint 0.03355
 finished parsing run0000_0000 timepoint 0.0336
 finished parsing run0000_0000 timepoint 0.03365
 finished parsing run0000_0000 timepoint 0.0337
 finished parsing run0000_0000 timepoint 0.03375
 finished parsing run0000_0000 timepoint 0.0338
 finished parsing run0000_0000 timepoint 0.03385
 finished parsing run0000_0000 timepoint 0.0339
 finished parsing run0000_0000 timepoint 0.03395
 finished parsing run0000_0000 timepoint 0.034
 finished parsing run0000_0000 timepoint 0.03405
 finished parsing run0000_0000 timepoint 0.0341
 finished parsing run0000_0000 timepoint 0.03415
 finished parsing run0000_0000 timepoint 0.0342
 finished parsing run0000_0000 timepoint 0.03425
 finished parsing run0000_0000 timepoint 0.0343
 finished parsing run0000_0000 timepoint 0.03435
 finished parsing run0000_0000 timepoint 0.0344
 finished parsing run0000_0000 timepoint 0.03445
 finished parsing run0000_0000 timepoint 0.0345
 finished parsing run0000_0000 timepoint 0.03455
 finished parsing run0000_0000 timepoint 0.0346
 finished parsing run0000_0000 timepoint 0.03465
 finished parsing run0000_0000 timepoint 0.0347
 finished parsing run0000_0000 timepoint 0.03475
 finished parsing run0000_0000 timepoint 0.0348
 finished parsing run0000_0000 timepoint 0.03485
 finished parsing run0000_0000 timepoint 0.0349
 finished parsing run0000_0000 timepoint 0.03495
 finished parsing run0000_0000 timepoint 0.035
 finished parsing run0000_0000 timepoint 0.03505
 finished parsing run0000_0000 timepoint 0.0351
 finished parsing run0000_0000 timepoint 0.03515
 finished parsing run0000_0000 timepoint 0.0352
 finished parsing run0000_0000 timepoint 0.03525
 finished parsing run0000_0000 timepoint 0.0353
 finished parsing run0000_0000 timepoint 0.03535
 finished parsing run0000_0000 timepoint 0.0354
 finished parsing run0000_0000 timepoint 0.03545
 finished parsing run0000_0000 timepoint 0.0355
 finished parsing run0000_0000 timepoint 0.03555
 finished parsing run0000_0000 timepoint 0.0356
 finished parsing run0000_0000 timepoint 0.03565
 finished parsing run0000_0000 timepoint 0.0357
 finished parsing run0000_0000 timepoint 0.03575
 finished parsing run0000_0000 timepoint 0.0358
 finished parsing run0000_0000 timepoint 0.03585
 finished parsing run0000_0000 timepoint 0.0359
 finished parsing run0000_0000 timepoint 0.03595
 finished parsing run0000_0000 timepoint 0.036
 finished parsing run0000_0000 timepoint 0.03605
 finished parsing run0000_0000 timepoint 0.0361
 finished parsing run0000_0000 timepoint 0.03615
 finished parsing run0000_0000 timepoint 0.0362
 finished parsing run0000_0000 timepoint 0.03625
 finished parsing run0000_0000 timepoint 0.0363
 finished parsing run0000_0000 timepoint 0.03635
 finished parsing run0000_0000 timepoint 0.0364
 finished parsing run0000_0000 timepoint 0.03645
 finished parsing run0000_0000 timepoint 0.0365
 finished parsing run0000_0000 timepoint 0.03655
 finished parsing run0000_0000 timepoint 0.0366
 finished parsing run0000_0000 timepoint 0.03665
 finished parsing run0000_0000 timepoint 0.0367
 finished parsing run0000_0000 timepoint 0.03675
 finished parsing run0000_0000 timepoint 0.0368
 finished parsing run0000_0000 timepoint 0.03685
 finished parsing run0000_0000 timepoint 0.0369
 finished parsing run0000_0000 timepoint 0.03695
 finished parsing run0000_0000 timepoint 0.037
 finished parsing run0000_0000 timepoint 0.03705
 finished parsing run0000_0000 timepoint 0.0371
 finished parsing run0000_0000 timepoint 0.03715
 finished parsing run0000_0000 timepoint 0.0372
 finished parsing run0000_0000 timepoint 0.03725
 finished parsing run0000_0000 timepoint 0.0373
 finished parsing run0000_0000 timepoint 0.03735
 finished parsing run0000_0000 timepoint 0.0374
 finished parsing run0000_0000 timepoint 0.03745
 finished parsing run0000_0000 timepoint 0.0375
 finished parsing run0000_0000 timepoint 0.03755
 finished parsing run0000_0000 timepoint 0.0376
 finished parsing run0000_0000 timepoint 0.03765
 finished parsing run0000_0000 timepoint 0.0377
 finished parsing run0000_0000 timepoint 0.03775
 finished parsing run0000_0000 timepoint 0.0378
 finished parsing run0000_0000 timepoint 0.03785
 finished parsing run0000_0000 timepoint 0.0379
 finished parsing run0000_0000 timepoint 0.03795
 finished parsing run0000_0000 timepoint 0.038
 finished parsing run0000_0000 timepoint 0.03805
 finished parsing run0000_0000 timepoint 0.0381
 finished parsing run0000_0000 timepoint 0.03815
 finished parsing run0000_0000 timepoint 0.0382
 finished parsing run0000_0000 timepoint 0.03825
 finished parsing run0000_0000 timepoint 0.0383
 finished parsing run0000_0000 timepoint 0.03835
 finished parsing run0000_0000 timepoint 0.0384
 finished parsing run0000_0000 timepoint 0.03845
 finished parsing run0000_0000 timepoint 0.0385
 finished parsing run0000_0000 timepoint 0.03855
 finished parsing run0000_0000 timepoint 0.0386
 finished parsing run0000_0000 timepoint 0.03865
 finished parsing run0000_0000 timepoint 0.0387
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.02545
 finished parsing run0000_0000 timepoint 0.0255
 finished parsing run0000_0000 timepoint 0.02555
 finished parsing run0000_0000 timepoint 0.0256
 finished parsing run0000_0000 timepoint 0.02565
 finished parsing run0000_0000 timepoint 0.0257
 finished parsing run0000_0000 timepoint 0.02575
 finished parsing run0000_0000 timepoint 0.0258
 finished parsing run0000_0000 timepoint 0.02585
 finished parsing run0000_0000 timepoint 0.0259
 finished parsing run0000_0000 timepoint 0.02595
 finished parsing run0000_0000 timepoint 0.026
 finished parsing run0000_0000 timepoint 0.02605
 finished parsing run0000_0000 timepoint 0.0261
 finished parsing run0000_0000 timepoint 0.02615
 finished parsing run0000_0000 timepoint 0.0262
 finished parsing run0000_0000 timepoint 0.02625
 finished parsing run0000_0000 timepoint 0.0263
 finished parsing run0000_0000 timepoint 0.02635
 finished parsing run0000_0000 timepoint 0.0264
 finished parsing run0000_0000 timepoint 0.02645
 finished parsing run0000_0000 timepoint 0.0265
 finished parsing run0000_0000 timepoint 0.02655
 finished parsing run0000_0000 timepoint 0.0266
 finished parsing run0000_0000 timepoint 0.02665
 finished parsing run0000_0000 timepoint 0.0267
 finished parsing run0000_0000 timepoint 0.02675
 finished parsing run0000_0000 timepoint 0.0268
 finished parsing run0000_0000 timepoint 0.02685
 finished parsing run0000_0000 timepoint 0.0269
 finished parsing run0000_0000 timepoint 0.02695
 finished parsing run0000_0000 timepoint 0.027
 finished parsing run0000_0000 timepoint 0.02705
 finished parsing run0000_0000 timepoint 0.0271
 finished parsing run0000_0000 timepoint 0.02715
 finished parsing run0000_0000 timepoint 0.0272
 finished parsing run0000_0000 timepoint 0.02725
 finished parsing run0000_0000 timepoint 0.0273
 finished parsing run0000_0000 timepoint 0.02735
 finished parsing run0000_0000 timepoint 0.0274
 finished parsing run0000_0000 timepoint 0.02745
 finished parsing run0000_0000 timepoint 0.0275
 finished parsing run0000_0000 timepoint 0.02755
 finished parsing run0000_0000 timepoint 0.0276
 finished parsing run0000_0000 timepoint 0.02765
 finished parsing run0000_0000 timepoint 0.0277
 finished parsing run0000_0000 timepoint 0.02775
 finished parsing run0000_0000 timepoint 0.0278
 finished parsing run0000_0000 timepoint 0.02785
 finished parsing run0000_0000 timepoint 0.0279
 finished parsing run0000_0000 timepoint 0.02795
 finished parsing run0000_0000 timepoint 0.028
 finished parsing run0000_0000 timepoint 0.02805
 finished parsing run0000_0000 timepoint 0.0281
 finished parsing run0000_0000 timepoint 0.02815
 finished parsing run0000_0000 timepoint 0.0282
 finished parsing run0000_0000 timepoint 0.02825
 finished parsing run0000_0000 timepoint 0.0283
 finished parsing run0000_0000 timepoint 0.02835
 finished parsing run0000_0000 timepoint 0.0284
 finished parsing run0000_0000 timepoint 0.02845
 finished parsing run0000_0000 timepoint 0.0285
 finished parsing run0000_0000 timepoint 0.02855
 finished parsing run0000_0000 timepoint 0.0286
 finished parsing run0000_0000 timepoint 0.02865
 finished parsing run0000_0000 timepoint 0.0287
 finished parsing run0000_0000 timepoint 0.02875
 finished parsing run0000_0000 timepoint 0.0288
 finished parsing run0000_0000 timepoint 0.02885
 finished parsing run0000_0000 timepoint 0.0289
 finished parsing run0000_0000 timepoint 0.02895
 finished parsing run0000_0000 timepoint 0.029
 finished parsing run0000_0000 timepoint 0.02905
 finished parsing run0000_0000 timepoint 0.0291
 finished parsing run0000_0000 timepoint 0.02915
 finished parsing run0000_0000 timepoint 0.0292
 finished parsing run0000_0000 timepoint 0.02925
 finished parsing run0000_0000 timepoint 0.0293
 finished parsing run0000_0000 timepoint 0.02935
 finished parsing run0000_0000 timepoint 0.0294
 finished parsing run0000_0000 timepoint 0.02945
 finished parsing run0000_0000 timepoint 0.0295
 finished parsing run0000_0000 timepoint 0.02955
 finished parsing run0000_0000 timepoint 0.0296
 finished parsing run0000_0000 timepoint 0.02965
 finished parsing run0000_0000 timepoint 0.0297
 finished parsing run0000_0000 timepoint 0.02975
 finished parsing run0000_0000 timepoint 0.0298
 finished parsing run0000_0000 timepoint 0.02985
 finished parsing run0000_0000 timepoint 0.0299
 finished parsing run0000_0000 timepoint 0.02995
 finished parsing run0000_0000 timepoint 0.03
 finished parsing run0000_0000 timepoint 0.03005
 finished parsing run0000_0000 timepoint 0.0301
 finished parsing run0000_0000 timepoint 0.03015
 finished parsing run0000_0000 timepoint 0.0302
 finished parsing run0000_0000 timepoint 0.03025
 finished parsing run0000_0000 timepoint 0.0303
 finished parsing run0000_0000 timepoint 0.03035
 finished parsing run0000_0000 timepoint 0.0304
 finished parsing run0000_0000 timepoint 0.03045
 finished parsing run0000_0000 timepoint 0.0305
 finished parsing run0000_0000 timepoint 0.03055
 finished parsing run0000_0000 timepoint 0.0306
 finished parsing run0000_0000 timepoint 0.03065
 finished parsing run0000_0000 timepoint 0.0307
 finished parsing run0000_0000 timepoint 0.03075
 finished parsing run0000_0000 timepoint 0.0308
 finished parsing run0000_0000 timepoint 0.03085
 finished parsing run0000_0000 timepoint 0.0309
 finished parsing run0000_0000 timepoint 0.03095
 finished parsing run0000_0000 timepoint 0.031
 finished parsing run0000_0000 timepoint 0.03105
 finished parsing run0000_0000 timepoint 0.0311
 finished parsing run0000_0000 timepoint 0.03115
 finished parsing run0000_0000 timepoint 0.0312
 finished parsing run0000_0000 timepoint 0.03125
 finished parsing run0000_0000 timepoint 0.0313
 finished parsing run0000_0000 timepoint 0.03135
 finished parsing run0000_0000 timepoint 0.0314
 finished parsing run0000_0000 timepoint 0.03145
 finished parsing run0000_0000 timepoint 0.0315
 finished parsing run0000_0000 timepoint 0.03155
 finished parsing run0000_0000 timepoint 0.0316
 finished parsing run0000_0000 timepoint 0.03165
 finished parsing run0000_0000 timepoint 0.0317
 finished parsing run0000_0000 timepoint 0.03175
 finished parsing run0000_0000 timepoint 0.0318
 finished parsing run0000_0000 timepoint 0.03185
 finished parsing run0000_0000 timepoint 0.0319
 finished parsing run0000_0000 timepoint 0.03195
 #+end_example
 #+begin_example
 finished parsing run0000_0000 timepoint 0.01815
 finished parsing run0000_0000 timepoint 0.0182
 finished parsing run0000_0000 timepoint 0.01825
 finished parsing run0000_0000 timepoint 0.0183
 finished parsing run0000_0000 timepoint 0.01835
 finished parsing run0000_0000 timepoint 0.0184
 finished parsing run0000_0000 timepoint 0.01845
 finished parsing run0000_0000 timepoint 0.0185
 finished parsing run0000_0000 timepoint 0.01855
 finished parsing run0000_0000 timepoint 0.0186
 finished parsing run0000_0000 timepoint 0.01865
 finished parsing run0000_0000 timepoint 0.0187
 finished parsing run0000_0000 timepoint 0.01875
 finished parsing run0000_0000 timepoint 0.0188
 finished parsing run0000_0000 timepoint 0.01885
 finished parsing run0000_0000 timepoint 0.0189
 finished parsing run0000_0000 timepoint 0.01895
 finished parsing run0000_0000 timepoint 0.019
 finished parsing run0000_0000 timepoint 0.01905
 finished parsing run0000_0000 timepoint 0.0191
 finished parsing run0000_0000 timepoint 0.01915
 finished parsing run0000_0000 timepoint 0.0192
 finished parsing run0000_0000 timepoint 0.01925
 finished parsing run0000_0000 timepoint 0.0193
 finished parsing run0000_0000 timepoint 0.01935
 finished parsing run0000_0000 timepoint 0.0194
 finished parsing run0000_0000 timepoint 0.01945
 finished parsing run0000_0000 timepoint 0.0195
 finished parsing run0000_0000 timepoint 0.01955
 finished parsing run0000_0000 timepoint 0.0196
 finished parsing run0000_0000 timepoint 0.01965
 finished parsing run0000_0000 timepoint 0.0197
 finished parsing run0000_0000 timepoint 0.01975
 finished parsing run0000_0000 timepoint 0.0198
 finished parsing run0000_0000 timepoint 0.01985
 finished parsing run0000_0000 timepoint 0.0199
 finished parsing run0000_0000 timepoint 0.01995
 finished parsing run0000_0000 timepoint 0.02
 finished parsing run0000_0000 timepoint 0.02005
 finished parsing run0000_0000 timepoint 0.0201
 finished parsing run0000_0000 timepoint 0.02015
 finished parsing run0000_0000 timepoint 0.0202
 finished parsing run0000_0000 timepoint 0.02025
 finished parsing run0000_0000 timepoint 0.0203
 finished parsing run0000_0000 timepoint 0.02035
 finished parsing run0000_0000 timepoint 0.0204
 finished parsing run0000_0000 timepoint 0.02045
 finished parsing run0000_0000 timepoint 0.0205
 finished parsing run0000_0000 timepoint 0.02055
 finished parsing run0000_0000 timepoint 0.0206
 finished parsing run0000_0000 timepoint 0.02065
 finished parsing run0000_0000 timepoint 0.0207
 finished parsing run0000_0000 timepoint 0.02075
 finished parsing run0000_0000 timepoint 0.0208
 finished parsing run0000_0000 timepoint 0.02085
 finished parsing run0000_0000 timepoint 0.0209
 finished parsing run0000_0000 timepoint 0.02095
 finished parsing run0000_0000 timepoint 0.021
 finished parsing run0000_0000 timepoint 0.02105
 finished parsing run0000_0000 timepoint 0.0211
 finished parsing run0000_0000 timepoint 0.02115
 finished parsing run0000_0000 timepoint 0.0212
 finished parsing run0000_0000 timepoint 0.02125
 finished parsing run0000_0000 timepoint 0.0213
 finished parsing run0000_0000 timepoint 0.02135
 finished parsing run0000_0000 timepoint 0.0214
 finished parsing run0000_0000 timepoint 0.02145
 finished parsing run0000_0000 timepoint 0.0215
 finished parsing run0000_0000 timepoint 0.02155
 finished parsing run0000_0000 timepoint 0.0216
 finished parsing run0000_0000 timepoint 0.02165
 finished parsing run0000_0000 timepoint 0.0217
 finished parsing run0000_0000 timepoint 0.02175
 finished parsing run0000_0000 timepoint 0.0218
 finished parsing run0000_0000 timepoint 0.02185
 finished parsing run0000_0000 timepoint 0.0219
 finished parsing run0000_0000 timepoint 0.02195
 finished parsing run0000_0000 timepoint 0.022
 finished parsing run0000_0000 timepoint 0.02205
 finished parsing run0000_0000 timepoint 0.0221
 finished parsing run0000_0000 timepoint 0.02215
 finished parsing run0000_0000 timepoint 0.0222
 finished parsing run0000_0000 timepoint 0.02225
 finished parsing run0000_0000 timepoint 0.0223
 finished parsing run0000_0000 timepoint 0.02235
 finished parsing run0000_0000 timepoint 0.0224
 finished parsing run0000_0000 timepoint 0.02245
 finished parsing run0000_0000 timepoint 0.0225
 finished parsing run0000_0000 timepoint 0.02255
 finished parsing run0000_0000 timepoint 0.0226
 finished parsing run0000_0000 timepoint 0.02265
 finished parsing run0000_0000 timepoint 0.0227
 finished parsing run0000_0000 timepoint 0.02275
 finished parsing run0000_0000 timepoint 0.0228
 finished parsing run0000_0000 timepoint 0.02285
 finished parsing run0000_0000 timepoint 0.0229
 finished parsing run0000_0000 timepoint 0.02295
 finished parsing run0000_0000 timepoint 0.023
 finished parsing run0000_0000 timepoint 0.02305
 finished parsing run0000_0000 timepoint 0.0231
 finished parsing run0000_0000 timepoint 0.02315
 finished parsing run0000_0000 timepoint 0.0232
 finished parsing run0000_0000 timepoint 0.02325
 finished parsing run0000_0000 timepoint 0.0233
 finished parsing run0000_0000 timepoint 0.02335
 finished parsing run0000_0000 timepoint 0.0234
 finished parsing run0000_0000 timepoint 0.02345
 finished parsing run0000_0000 timepoint 0.0235
 finished parsing run0000_0000 timepoint 0.02355
 finished parsing run0000_0000 timepoint 0.0236
 finished parsing run0000_0000 timepoint 0.02365
 finished parsing run0000_0000 timepoint 0.0237
 finished parsing run0000_0000 timepoint 0.02375
 finished parsing run0000_0000 timepoint 0.0238
 finished parsing run0000_0000 timepoint 0.02385
 finished parsing run0000_0000 timepoint 0.0239
 finished parsing run0000_0000 timepoint 0.02395
 finished parsing run0000_0000 timepoint 0.024
 finished parsing run0000_0000 timepoint 0.02405
 finished parsing run0000_0000 timepoint 0.0241
 finished parsing run0000_0000 timepoint 0.02415
 finished parsing run0000_0000 timepoint 0.0242
 finished parsing run0000_0000 timepoint 0.02425
 finished parsing run0000_0000 timepoint 0.0243
 finished parsing run0000_0000 timepoint 0.02435
 finished parsing run0000_0000 timepoint 0.0244
 finished parsing run0000_0000 timepoint 0.02445
 finished parsing run0000_0000 timepoint 0.0245
 finished parsing run0000_0000 timepoint 0.02455
 finished parsing run0000_0000 timepoint 0.0246
 finished parsing run0000_0000 timepoint 0.02465
 finished parsing run0000_0000 timepoint 0.0247
 finished parsing run0000_0000 timepoint 0.02475
 finished parsing run0000_0000 timepoint 0.0248
 finished parsing run0000_0000 timepoint 0.02485
 finished parsing run0000_0000 timepoint 0.0249
 finished parsing run0000_0000 timepoint 0.02495
 finished parsing run0000_0000 timepoint 0.025
 finished parsing run0000_0000 timepoint 0.02505
 finished parsing run0000_0000 timepoint 0.0251
 finished parsing run0000_0000 timepoint 0.02515
 finished parsing run0000_0000 timepoint 0.0252
 finished parsing run0000_0000 timepoint 0.02525
 finished parsing run0000_0000 timepoint 0.0253
 finished parsing run0000_0000 timepoint 0.02535
 finished parsing run0000_0000 timepoint 0.0254
 #+end_example
 :results:
 #+begin_example
 finished parsing run0000_0000 timepoint 0.01015
 finished parsing run0000_0000 timepoint 0.0102
 finished parsing run0000_0000 timepoint 0.01025
 finished parsing run0000_0000 timepoint 0.0103
 finished parsing run0000_0000 timepoint 0.01035
 finished parsing run0000_0000 timepoint 0.0104
 finished parsing run0000_0000 timepoint 0.01045
 finished parsing run0000_0000 timepoint 0.0105
 finished parsing run0000_0000 timepoint 0.01055
 finished parsing run0000_0000 timepoint 0.0106
 finished parsing run0000_0000 timepoint 0.01065
 finished parsing run0000_0000 timepoint 0.0107
 finished parsing run0000_0000 timepoint 0.01075
 finished parsing run0000_0000 timepoint 0.0108
 finished parsing run0000_0000 timepoint 0.01085
 finished parsing run0000_0000 timepoint 0.0109
 finished parsing run0000_0000 timepoint 0.01095
 finished parsing run0000_0000 timepoint 0.011
 finished parsing run0000_0000 timepoint 0.01105
 finished parsing run0000_0000 timepoint 0.0111
 finished parsing run0000_0000 timepoint 0.01115
 finished parsing run0000_0000 timepoint 0.0112
 finished parsing run0000_0000 timepoint 0.01125
 finished parsing run0000_0000 timepoint 0.0113
 finished parsing run0000_0000 timepoint 0.01135
 finished parsing run0000_0000 timepoint 0.0114
 finished parsing run0000_0000 timepoint 0.01145
 finished parsing run0000_0000 timepoint 0.0115
 finished parsing run0000_0000 timepoint 0.01155
 finished parsing run0000_0000 timepoint 0.0116
 finished parsing run0000_0000 timepoint 0.01165
 finished parsing run0000_0000 timepoint 0.0117
 finished parsing run0000_0000 timepoint 0.01175
 finished parsing run0000_0000 timepoint 0.0118
 finished parsing run0000_0000 timepoint 0.01185
 finished parsing run0000_0000 timepoint 0.0119
 finished parsing run0000_0000 timepoint 0.01195
 finished parsing run0000_0000 timepoint 0.012
 finished parsing run0000_0000 timepoint 0.01205
 finished parsing run0000_0000 timepoint 0.0121
 finished parsing run0000_0000 timepoint 0.01215
 finished parsing run0000_0000 timepoint 0.0122
 finished parsing run0000_0000 timepoint 0.01225
 finished parsing run0000_0000 timepoint 0.0123
 finished parsing run0000_0000 timepoint 0.01235
 finished parsing run0000_0000 timepoint 0.0124
 finished parsing run0000_0000 timepoint 0.01245
 finished parsing run0000_0000 timepoint 0.0125
 finished parsing run0000_0000 timepoint 0.01255
 finished parsing run0000_0000 timepoint 0.0126
 finished parsing run0000_0000 timepoint 0.01265
 finished parsing run0000_0000 timepoint 0.0127
 finished parsing run0000_0000 timepoint 0.01275
 finished parsing run0000_0000 timepoint 0.0128
 finished parsing run0000_0000 timepoint 0.01285
 finished parsing run0000_0000 timepoint 0.0129
 finished parsing run0000_0000 timepoint 0.01295
 finished parsing run0000_0000 timepoint 0.013
 finished parsing run0000_0000 timepoint 0.01305
 finished parsing run0000_0000 timepoint 0.0131
 finished parsing run0000_0000 timepoint 0.01315
 finished parsing run0000_0000 timepoint 0.0132
 finished parsing run0000_0000 timepoint 0.01325
 finished parsing run0000_0000 timepoint 0.0133
 finished parsing run0000_0000 timepoint 0.01335
 finished parsing run0000_0000 timepoint 0.0134
 finished parsing run0000_0000 timepoint 0.01345
 finished parsing run0000_0000 timepoint 0.0135
 finished parsing run0000_0000 timepoint 0.01355
 finished parsing run0000_0000 timepoint 0.0136
 finished parsing run0000_0000 timepoint 0.01365
 finished parsing run0000_0000 timepoint 0.0137
 finished parsing run0000_0000 timepoint 0.01375
 finished parsing run0000_0000 timepoint 0.0138
 finished parsing run0000_0000 timepoint 0.01385
 finished parsing run0000_0000 timepoint 0.0139
 finished parsing run0000_0000 timepoint 0.01395
 finished parsing run0000_0000 timepoint 0.014
 finished parsing run0000_0000 timepoint 0.01405
 finished parsing run0000_0000 timepoint 0.0141
 finished parsing run0000_0000 timepoint 0.01415
 finished parsing run0000_0000 timepoint 0.0142
 finished parsing run0000_0000 timepoint 0.01425
 finished parsing run0000_0000 timepoint 0.0143
 finished parsing run0000_0000 timepoint 0.01435
 finished parsing run0000_0000 timepoint 0.0144
 finished parsing run0000_0000 timepoint 0.01445
 finished parsing run0000_0000 timepoint 0.0145
 finished parsing run0000_0000 timepoint 0.01455
 finished parsing run0000_0000 timepoint 0.0146
 finished parsing run0000_0000 timepoint 0.01465
 finished parsing run0000_0000 timepoint 0.0147
 finished parsing run0000_0000 timepoint 0.01475
 finished parsing run0000_0000 timepoint 0.0148
 finished parsing run0000_0000 timepoint 0.01485
 finished parsing run0000_0000 timepoint 0.0149
 finished parsing run0000_0000 timepoint 0.01495
 finished parsing run0000_0000 timepoint 0.015
 finished parsing run0000_0000 timepoint 0.01505
 finished parsing run0000_0000 timepoint 0.0151
 finished parsing run0000_0000 timepoint 0.01515
 finished parsing run0000_0000 timepoint 0.0152
 finished parsing run0000_0000 timepoint 0.01525
 finished parsing run0000_0000 timepoint 0.0153
 finished parsing run0000_0000 timepoint 0.01535
 finished parsing run0000_0000 timepoint 0.0154
 finished parsing run0000_0000 timepoint 0.01545
 finished parsing run0000_0000 timepoint 0.0155
 finished parsing run0000_0000 timepoint 0.01555
 finished parsing run0000_0000 timepoint 0.0156
 finished parsing run0000_0000 timepoint 0.01565
 finished parsing run0000_0000 timepoint 0.0157
 finished parsing run0000_0000 timepoint 0.01575
 finished parsing run0000_0000 timepoint 0.0158
 finished parsing run0000_0000 timepoint 0.01585
 finished parsing run0000_0000 timepoint 0.0159
 finished parsing run0000_0000 timepoint 0.01595
 finished parsing run0000_0000 timepoint 0.016
 finished parsing run0000_0000 timepoint 0.01605
 finished parsing run0000_0000 timepoint 0.0161
 finished parsing run0000_0000 timepoint 0.01615
 finished parsing run0000_0000 timepoint 0.0162
 finished parsing run0000_0000 timepoint 0.01625
 finished parsing run0000_0000 timepoint 0.0163
 finished parsing run0000_0000 timepoint 0.01635
 finished parsing run0000_0000 timepoint 0.0164
 finished parsing run0000_0000 timepoint 0.01645
 finished parsing run0000_0000 timepoint 0.0165
 finished parsing run0000_0000 timepoint 0.01655
 finished parsing run0000_0000 timepoint 0.0166
 finished parsing run0000_0000 timepoint 0.01665
 finished parsing run0000_0000 timepoint 0.0167
 finished parsing run0000_0000 timepoint 0.01675
 finished parsing run0000_0000 timepoint 0.0168
 finished parsing run0000_0000 timepoint 0.01685
 finished parsing run0000_0000 timepoint 0.0169
 finished parsing run0000_0000 timepoint 0.01695
 finished parsing run0000_0000 timepoint 0.017
 finished parsing run0000_0000 timepoint 0.01705
 finished parsing run0000_0000 timepoint 0.0171
 finished parsing run0000_0000 timepoint 0.01715
 finished parsing run0000_0000 timepoint 0.0172
 finished parsing run0000_0000 timepoint 0.01725
 finished parsing run0000_0000 timepoint 0.0173
 finished parsing run0000_0000 timepoint 0.01735
 finished parsing run0000_0000 timepoint 0.0174
 finished parsing run0000_0000 timepoint 0.01745
 finished parsing run0000_0000 timepoint 0.0175
 finished parsing run0000_0000 timepoint 0.01755
 finished parsing run0000_0000 timepoint 0.0176
 finished parsing run0000_0000 timepoint 0.01765
 finished parsing run0000_0000 timepoint 0.0177
 finished parsing run0000_0000 timepoint 0.01775
 finished parsing run0000_0000 timepoint 0.0178
 finished parsing run0000_0000 timepoint 0.01785
 finished parsing run0000_0000 timepoint 0.0179
 finished parsing run0000_0000 timepoint 0.01795
 finished parsing run0000_0000 timepoint 0.018
 finished parsing run0000_0000 timepoint 0.01805
 finished parsing run0000_0000 timepoint 0.0181
 #+end_example
 :end:

## summaries

In [ ]:

 associated_fibers_allruns = []
 associated_ends_allruns = []
 outputs_allruns = []
 summaries_allruns = []
 summaries_growing_allruns = []
 summaries_associated_growing_allruns = []

 for rundir in rundirs:
     fiber_outputs = []
     end_outputs = []
     lengths = dict()
     growth = dict()
     outputs = []
     summary = dict()
     summary_growing = dict()
     summary_associated_growing = dict()
     for i, timepoint in enumerate(timepoints):
         #bound_fibers = bound_hip1r.loc[rundir].loc[timepoint]['fiber_id']
         clusters = all_fiber_clusters.loc[rundir].loc[timepoint]['fiber_id_list']
         fiber_forces = all_fiber_forces.loc[rundir].loc[timepoint]
         fiber_ends = all_fiber_ends.loc[rundir].loc[timepoint]
         bound_fiber_list = []

         for fiber in bound_fibers:
             if fiber not in bound_fiber_list:
                 bound_fiber_list.append(fiber)

         associated_fiber_list = []

         for fiber in bound_fiber_list:
             for cluster in clusters:
                 if str(fiber) in cluster:
                     for cluster_fiber in cluster:
                         if cluster_fiber not in associated_fiber_list:
                             associated_fiber_list.append(cluster_fiber)

         fibers = fiber_ends['fiber_id']
         lengths[i] = fiber_ends['length']
         if i > 0:
             fiber_ends['growth'] = lengths[i] - lengths[i-1]
         else:
             fiber_ends['growth'] = lengths[i] - lengths[i]
         outputs.append(fiber_ends)

         associated_fibers = fiber_forces[fiber_forces['fiber_id'].isin(associated_fiber_list)]
         fiber_outputs.append(associated_fibers)

         associated_fiber_ends = fiber_ends[fiber_ends['fiber_id'].isin(associated_fiber_list)]
         end_outputs.append(associated_fiber_ends)

         mean = np.mean(fiber_ends['growth'])
         std = np.std(fiber_ends['growth'])
         summary[timepoint] = {'mean_growth' : mean, 'std_growth' : std}

         growing_fiber_ends = fiber_ends.loc[fiber_ends['growth'] > 0.0]
         mean_growing = np.mean(growing_fiber_ends['growth'])
         std_growing = np.std(growing_fiber_ends['growth'])
         summary_growing[timepoint] = {'mean_growth' : mean_growing,
                                       'std_growth' : std_growing}

         growing_associated_ends = associated_fiber_ends.loc[associated_fiber_ends['growth'] > 0.0]
         mean_growing_associated = np.mean(growing_associated_ends['growth'])
         std_growing_associated = np.std(growing_associated_ends['growth'])
         summary_associated_growing[timepoint] = {'mean_growth' : mean_growing_associated,
                                       'std_growth' : std_growing_associated}
     associated_fibers_alltimes = pd.concat(fiber_outputs, keys = timepoints,
                                            names = ['time', 'id'])
     associated_fibers_allruns.append(associated_fibers_alltimes)

     associated_ends_alltimes = pd.concat(end_outputs, keys = timepoints,
                                          names = ['time', 'id'])
     associated_ends_allruns.append(associated_ends_alltimes)

     outputs_alltimes = pd.concat(outputs, keys = timepoints,
                                  names = ['time', 'id'])

     outputs_allruns.append(outputs_alltimes)
     summary_df = pd.DataFrame.from_dict(summary, orient = 'index')
     summaries_allruns.append(summary_df)
     summary_growing_df = pd.DataFrame.from_dict(summary_growing, orient = 'index')
     summaries_growing_allruns.append(summary_growing_df)
     summary_associated_growing_df = pd.DataFrame.from_dict(summary_associated_growing, orient = 'index')
     summaries_associated_growing_allruns.append(summary_associated_growing_df)

     print("finished parsing " + rundir)


 hip1r_clusters = pd.concat(associated_fibers_allruns, keys = rundirs,
                             names = ['run', 'time', 'id'])

 hip1r_clusters_ends = pd.concat(associated_ends_allruns, keys = rundirs,
                                 names = ['run', 'time', 'id'])

 all_fiber_ends = pd.concat(outputs_allruns, keys = rundirs,
                         names = ['run', 'time', 'id'])

 all_growing_fiber_ends = all_fiber_ends.loc[all_fiber_ends['growth'] > 0.0]

 fiber_ends_summary = pd.concat(summaries_allruns, keys = rundirs,
                                names = ['run', 'time'])
 growing_ends_summary = pd.concat(summaries_growing_allruns, keys = rundirs,
                                  names = ['run', 'time'])
 hip1r_growing_ends_summary = pd.concat(summaries_associated_growing_allruns, keys = rundirs,
                                  names = ['run', 'time'])

 summaries = [fiber_ends_summary, growing_ends_summary, hip1r_growing_ends_summary]

 for i, df in enumerate(summaries):
     df = df.swaplevel('run', 'time', axis=0).sort_index()
     summary = dict()
     for timepoint in timepoints:
         timepoint_summary = df.loc[timepoint]
         timepoint_mean = np.mean(timepoint_summary['mean_growth'])
         timepoint_se = np.std(timepoint_summary['mean_growth'])
         summary[timepoint] = {'mean_growth' : timepoint_mean, 'se_growth' : timepoint_se}
     if i == 0:
         fiber_ends_allruns_summary = pd.DataFrame.from_dict(summary, orient = 'index')
     if i == 1:
         growing_ends_allruns_summary = pd.DataFrame.from_dict(summary, orient = 'index')
     if i == 2:
         hip1r_growing_ends_allruns_summary = pd.DataFrame.from_dict(summary, orient = 'index')

#+RESULTS[1307be88153fe2e15a704129683de80f1cb61e61]:
 :results:
 # [goto error]
 #+begin_example
 [0;31m---------------------------------------------------------------------------[0m
 [0;31mNameError[0m                                 Traceback (most recent call last)
 Cell [0;32mIn[894], line 19[0m
 [1;32m     16[0m summary_associated_growing [38;5;241m=[39m [38;5;28mdict[39m()
 [1;32m     17[0m [38;5;28;01mfor[39;00m i, timepoint [38;5;129;01min[39;00m [38;5;28menumerate[39m(timepoints):
 [1;32m     18[0m     [38;5;66;03m#bound_fibers = bound_hip1r.loc[rundir].loc[timepoint]['fiber_id'][39;00m
 [0;32m---> 19[0m     clusters [38;5;241m=[39m [43mall_fiber_clusters[49m[38;5;241m.[39mloc[rundir][38;5;241m.[39mloc[timepoint][[38;5;124m'[39m[38;5;124mfiber_id_list[39m[38;5;124m'[39m]
 [1;32m     20[0m     fiber_forces [38;5;241m=[39m all_fiber_forces[38;5;241m.[39mloc[rundir][38;5;241m.[39mloc[timepoint]
 [1;32m     21[0m     fiber_ends [38;5;241m=[39m all_fiber_ends[38;5;241m.[39mloc[rundir][38;5;241m.[39mloc[timepoint]

 [0;31mNameError[0m: name 'all_fiber_clusters' is not defined
 #+end_example
 :end:

## merge positions with run properties/configs

### myosin

In [ ]:
 membrane_myosin_config = pd.merge(config_unique.reset_index(), membrane_myosin_allparams.reset_index(),
                         on=['param_sweep', 'run'], how='outer').set_index(['param_sweep', 'run'])
 membrane_myosin_grouped = membrane_myosin_config.groupby(groups)
 membrane_myosin_zpos_means = membrane_myosin_grouped['zpos'].mean()
 membrane_myosin_zpos_stds = membrane_myosin_grouped['zpos'].std()
 membrane_myosin_zpos_means.head()

#+RESULTS[e98ae34cf31680f5bcf5453c66fa2b16b3fe3be3]:
 :results:
 # [goto error]
 :end:

## write dataframes to file

#+begin_src python
if save_dataframes == 'yes':
    #solid_allparams.to_pickle('dataframes/solid_allparams.pkl')
    membrane_myosin_allparams.to_pickle('dataframes/membrane_myosin_allparams.pkl')
    # means.to_pickle('dataframes/means.pkl')
    # stds.to_pickle('dataframes/stds.pkl')
    # sweep_means.to_pickle('dataframes/sweep_means.pkl')
    # sweep_stds.to_pickle('dataframes/sweep_stds.pkl')
    pd.DataFrame.from_dict(rundirs_allparams, orient = 'index').to_pickle('dataframes/rundirs_allparams.pkl')
    properties_allparams.to_pickle('dataframes/properties_allparams.pkl')
    properties_unique.to_pickle('dataframes/properties_unique.pkl')
#+end_src

# parse results into dataframe

## all myosin

In [ ]:
membrane_myosin_allparams = caf.all_myosin(output_dirs, rundirs_allparams_df, single_membrane_myosin_allruns_allparams)

if save_dataframes == 'yes':
    membrane_myosin_allparams.to_pickle(dataframes_dir+'membrane_myosin_allparams.pkl')

membrane_myosin_allparams.head()

finished parsing 20230504_6.6.5_myominimal_output


# plot results

## forces on myosins

### histogram

#+begin_src python
membrane_myosin_allparams['scalar_force'].hist(bins=25)
plt.xlabel('force on myosin (pN)')
if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_forces_all.png')
#+end_src

### by time

#+begin_src python
membrane_myosin_allparams['scalar_force'].reset_index().plot('time','scalar_force', kind='scatter', alpha=0.01,s=1)
if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_forces_time.png')
#+end_src

### by id

#+begin_src python
membrane_myosin_allparams['scalar_force'].reset_index().plot('id','scalar_force', kind='scatter', alpha=0.1,s=1)
if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_forces_id.png')
#+end_src

## myosin step size

#+begin_src python
#timestep = timepoints[0]
timestep = membrane_myosin_allparams.index[0][-2]
#total_time = len(timepoints) * timestep
total_time = membrane_myosin_allparams.index[-1][-2]

swapped =  membrane_myosin_allparams.loc['20230504_6.6.5_myominimal_output'].loc['run0000_0000'].swaplevel(0,1,axis=0).sort_index()

swapped['consecutive_frames'] = swapped.state.groupby((
    swapped.state != swapped.state.shift()).cumsum()).transform('size') * swapped.state
swapped['consecutive_time'] = swapped['consecutive_frames']*timestep

all_bound = swapped.loc[swapped['state'] == 1]
all_bound.head()
#+end_src


#+begin_src python
event_counter = 0
events = {}
last_consec = 0

for timepoint in all_bound.iterrows():
    consecutive_frames = timepoint[1]['consecutive_frames']
    if consecutive_frames != last_consec:
        last_consec = consecutive_frames
        single = timepoint[0][0]
        start_time = timepoint[0][1]
        consecutive_time = timepoint[1]['consecutive_time']
        end_time = round(start_time + consecutive_time - timestep, 5)
        if end_time < total_time:
            fiber_id = np.mean(all_bound.loc[single]['fiber_id'][start_time:end_time])
            single_id = np.mean(all_bound.loc[single]['single_id'][start_time:end_time])
            median_force = np.median(all_bound.loc[single]['scalar_force'][start_time:end_time])
            mean_force = np.mean(all_bound.loc[single]['scalar_force'][start_time:end_time])
            abscissa_displacement = (all_bound.loc[single]['abscissa'][end_time] -
                                    all_bound.loc[single]['abscissa'][start_time])
            events[event_counter] = {'median_force':median_force, 'mean_force':mean_force,
                                    'start_time':start_time, 'attachment_time':consecutive_time,
                                     'abscissa_displacement':abscissa_displacement,
                                     'fiber_id':fiber_id, 'single_id':single_id}
            event_counter += 1

attachment_events = pd.DataFrame.from_dict(events, orient = 'index')
#+end_src


#+begin_src python
attachment_events.head(25)
#+end_src


force when fiber 9 first binds myosin 1:
#+begin_src python
forces_allparams.loc[forces_allparams['fiber_id']==9].loc['20230504_6.6.5_myominimal_output'].loc['run0000_0000'].loc[0.20245:0.20255]
#+end_src


#+begin_src python
attachment_events['abscissa_displacement'].hist(bins=30)
#+end_src


#+begin_src python
abscissa_displacement = attachment_events['abscissa_displacement']*1000
time = attachment_events['attachment_time']

plt.figure(figsize=(6,5))
plt.subplot(1,1,1)
plt.scatter(time, abscissa_displacement, s=50)
plt.xlabel('Attachment lifetime ($s$)')
#plt.ylabel('abscissa displacement (nm)')
plt.ylabel('Actin countour\ndisplacement ($nm$)')
#plt.ylim(bottom = -0.001, top = 0.001)

plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/publish/myosin_displacement_lifetime.svg')
#+end_src


#+begin_src python
event_counter
#+end_src


#+begin_src python
attachment_events.plot('mean_force', 'abscissa_displacement', kind='scatter', s=2)
plt.xlabel('mean force (pN)')
plt.ylabel('abscissa displacement (μm)')
#plt.ylim(bottom = -0.001, top = 0.001)

plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_force_displacement.png')
#+end_src

## myosin attachment duration vs. force

#+begin_src python
x = all_bound['scalar_force']
y = all_bound['consecutive_time']

plt.figure(figsize=(10,10))
plt.scatter(x, y, s=1, alpha = 0.3)
plt.xlabel('force (pN)')
plt.ylabel('lifetime (s)')
plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_lifetime_force.png')
#+end_src


#+begin_src python
attachment_events.plot('mean_force', 'attachment_time', kind='scatter', s=50)
plt.xlabel('Mean force on myosin ($pN$)')
plt.ylabel('Attachment lifetime ($s$)')
#plt.ylim(bottom = -0.001, top = 0.001)

plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/publish/myosin_force_lifetime.svg')
#+end_src


#+begin_src python
all_bound.reset_index().plot('time','id',kind='scatter',s=1,alpha=0.1)
if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_id_lifetime.png')
#+end_src

## number of myosin attachments vs. force

#+begin_src python
bound_fibers_group = all_bound.groupby(['time','fiber_id'])
x = bound_fibers_group.count()['single_id']
y = bound_fibers_group.mean()['scalar_force']

plt.scatter(x,y,alpha=0.1)
plt.xlabel('simultaneous myosin attachments per filament')
plt.ylabel('mean myosin-actin force per filament (pN)')
plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_attachments_force.png')
#+end_src


#+begin_src python
b1 = bound_fibers_group.mean().loc[bound_fibers_group.count()['single_id']==1]['scalar_force']
b2 = bound_fibers_group.mean().loc[bound_fibers_group.count()['single_id']==2]['scalar_force']
b3 = bound_fibers_group.mean().loc[bound_fibers_group.count()['single_id']==3]['scalar_force']

# plt.hist(b1, label='1', range=(0,80), alpha=0.5)
# plt.hist(b2, label='2', range=(0,80), alpha=0.5)
# plt.hist(b3, label='3', range=(0,80), alpha=0.5)
plt.hist([b1,b2,b3], label=['1','2','3'], bins=20, range=(0,60))

plt.legend(title='simultaneous bound myosins')
plt.xlabel('mean myo-actin force (pN)')
plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_attachments_force_hist.png')
#+end_src



#+begin_src python
plt.scatter(b1.reset_index()['time'], b1.reset_index()['scalar_force'], s=1, alpha=0.2, label='1')
plt.scatter(b2.reset_index()['time'], b2.reset_index()['scalar_force'], s=1, alpha=0.2, label='2')
plt.scatter(b3.reset_index()['time'], b3.reset_index()['scalar_force'], s=1, alpha=0.2, label='3')

plt.legend(title='simultaneous bound myosins')
plt.xlabel('time (s)')
plt.ylabel('mean myo-actin force (pN)')
plt.tight_layout()

if save_figures == 'yes':
    plt.savefig(working_dir+'figures/'+pref+'_myosin_attachments_force_time.png')
#+end_src